# Filtering Synthetic Data with LLMs

In [1]:
from claudette import *
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from fastcore.utils import *


In [2]:
models

['claude-3-opus-20240229',
 'claude-3-5-sonnet-20240620',
 'claude-3-haiku-20240307']

In [3]:
model = models[1]

In [4]:
cli = Client(model)

In [5]:
FP = Path.cwd() ; print(FP)

/home/zardar/Desktop/synthetic-medical-datatsets


In [6]:
load_dotenv(FP/'.env')

True

In [7]:
class TranslationCritique(BasicRepr):
    "A critique of the translation."
    def __init__(
        self,
        critique: str, # A brief 1-line critique of the translation.
        score: int # A score of the translation from 1 to 5. 
    ): store_attr()
    
    __repr__ = basic_repr('critique, score')


In [8]:
sp = "As an expert radiologist, evaluate synthetic radiology reports for accuracy and quality, focusing on findings related to calcifications, including microcalcifications. Consider the description, size, distribution, and potential clinical significance of any calcifications mentioned."

In [9]:
"""df1 = df.iloc[:6]
df2 = df.iloc[6:]"""

'df1 = df.iloc[:6]\ndf2 = df.iloc[6:]'

In [10]:
df_test = pd.read_csv(FP/'claude-3-opus-synth-test-n126.csv')
df_train = pd.read_csv(FP/'claude-3-opus-synth-train-n1132.csv')
df = pd.concat([df_train, df_test], ignore_index=True)

In [11]:
df.head()

,Unnamed: 0,report_text,label
0,404,"REPORT:\n""BILATERAL SCREENING MAMMOGRAM\nCLINI...",Positive
1,1148,SCREENING MAMMOGRAM.\nCompared to Previous: Ye...,Not Stated
2,561,MAMMOGRAPHY - SCREENING - BILATERAL. Clinical ...,Negative
3,1027,BILATERAL SCREENING MAMMOGRAM\nINDICATION: Ann...,Positive
4,1066,SCREENING MAMMOGRAM\nINDICATION: Screening. No...,Not Stated


In [12]:
df.label.value_counts()

label
Negative      542
Positive      446
Not Stated    270
Name: count, dtype: int64

In [13]:
# df = df.sample(n=15) # can set random_seed=42 IF you want to be consistent with the sampling
# df.label.value_counts()

In [14]:
class Report():
    "Radiology Report + Label"
    def __init__(self, report: str, label: str): store_attr()
    def __repr__(self): return f"{self.report} ➡ *{self.label}*"

In [15]:
example_reports = [Report(report, label) for report, label in zip(df.report_text, df.label)]

In [16]:
example_reports[0]

REPORT:
"BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Annual screening mammogram. No known breast issues. 
COMPARISON: Compared to  prior studies from 07/2021.
FINDINGS: 
Right breast: 
There are scattered areas of fibroglandular density. No suspicious masses, calcifications, or areas of architectural distortion. 
Left breast:
There are scattered areas of fibroglandular density. In the upper outer quadrant, there is a new group of fine pleomorphic microcalcifications spanning approximately 6 mm. No associated mass, architectural distortion, or skin changes are seen. 
IMPRESSION:
RIGHT BREAST: BI-RADS 2 - Benign findings. Recommend continued routine screening in 1 year.
LEFT BREAST: BI-RADS 4 - Suspicious abnormality. Biopsy should be considered to rule out malignancy. 
ADDENDUM: Stereotactic core biopsy of the left breast microcalcifications at 2 o'clock was performed. Pathology results showed atypical ductal hyperplasia with calcifications. Surgical consultation is recommended for 

In [17]:
len(example_reports)

1258

In [18]:
from IPython.display import Markdown

In [19]:
clps_fmt = '- {s}\n\n<details>\n<summary> Click to show the rest </summary>\n{ls}\n</details>'
def to_md(ss, collapsible=False):
    ls = '\n'.join(f'- {s}' for s in ss) 
    return clps_fmt.format(s=str(ss[0]), ls=ls.replace(f'- {ss[0]}', '')) if collapsible else ls
def show(ss, collapsible=False): return Markdown(to_md(ss, collapsible=collapsible))

In [20]:
show(example_reports, collapsible=True)

- REPORT:
"BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Annual screening mammogram. No known breast issues. 
COMPARISON: Compared to  prior studies from 07/2021.
FINDINGS: 
Right breast: 
There are scattered areas of fibroglandular density. No suspicious masses, calcifications, or areas of architectural distortion. 
Left breast:
There are scattered areas of fibroglandular density. In the upper outer quadrant, there is a new group of fine pleomorphic microcalcifications spanning approximately 6 mm. No associated mass, architectural distortion, or skin changes are seen. 
IMPRESSION:
RIGHT BREAST: BI-RADS 2 - Benign findings. Recommend continued routine screening in 1 year.
LEFT BREAST: BI-RADS 4 - Suspicious abnormality. Biopsy should be considered to rule out malignancy. 
ADDENDUM: Stereotactic core biopsy of the left breast microcalcifications at 2 o'clock was performed. Pathology results showed atypical ductal hyperplasia with calcifications. Surgical consultation is recommended for further management. Post-biopsy BI-RADS category is 4A." ➡ *Positive*

<details>
<summary> Click to show the rest </summary>

- SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Date of Previous: 09/12/2021. No significant change from prior examination.
Clinical Indication: Routine screening.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
FINDINGS:
There are no dominant masses, suspicious microcalcifications, or focal asymmetries.
IMPRESSION:
NEGATIVE SCREENING MAMMOGRAM. BI-RADS Category 1.
RECOMMENDATION: Routine screening as per provincial guidelines. ➡ *Not Stated*
- MAMMOGRAPHY - SCREENING - BILATERAL. Clinical History: 51 year old female for routine screening mammogram. Patient reports that she is currently on hormone replacement therapy. Patient reports a family history of breast cancer in her aunt (maternal side). Comparison is made with the patient's previous mammogram performed on 10/10/2023. Findings: The breasts are heterogeneously dense. There are no masses, architectural distortion, or suspicious calcifications. Impression: No evidence of malignancy. BI-RADS 1. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram
COMPARISON: None
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.
At the 6 o'clock position of the right breast, approximately 7 cm from the nipple, there is a group of amorphous calcifications spanning an area of about 8 mm. This is a new finding compared to the patient's prior mammograms.
The left breast is unremarkable.
IMPRESSION:
BI-RADS Category 0: Incomplete, Need Additional Imaging Evaluation
Amorphous group of calcifications in the right breast at 6 o'clock position, new since prior exam. Additional magnification views are recommended for further evaluation.
ADDENDUM:
RIGHT BREAST SPOT COMPRESSION MAGNIFICATION VIEWS
FINDINGS:
Spot compression magnification views of the right breast in the region of the amorphous calcifications demonstrate a group of calcifications with varying sizes and shapes, spanning an area of approximately 8 mm. The distribution is clustered. The morphology is amorphous to coarse heterogeneous.
IMPRESSION:
BI-RADS Category 4: Suspicious Abnormality
Group of amorphous to coarse heterogeneous calcifications in the right breast at 6 o'clock position, spanning 8 mm. Stereotactic biopsy is recommended for definitive characterization. ➡ *Positive*
- SCREENING MAMMOGRAM
INDICATION: Screening. No current breast symptoms or complaints. Annual screening exam.

COMPARISON: None available for comparison. 

TECHNIQUE: Standard views of each breast including CC and MLO projections. 

FINDINGS:
Right Breast: Predominantly fatty replaced breast tissue. No significant masses, calcifications, or architectural distortions.  
Left Breast: Scattered fibroglandular breast tissue. No dominant masses, suspicious calcifications, or areas of architectural distortion. 

IMPRESSION:
1. Negative bilateral screening mammogram. No mammographic evidence of malignancy.
2. Fatty replaced breast tissue bilaterally. 
3. Recommend routine annual screening mammography in 1 year. 

BIRADS Category 1: Negative. ➡ *Not Stated*
- SCREENING BILATERAL MAMMOGRAM.
Indication: Patient is 54 years old with family history of breast cancer.
FINDINGS: 
The breast tissue consists of scattered areas of fibroglandular density. Scattered benign appearing masses and cysts appreciated on left breast. No dominant mass or architectural distortion identified in the right breast.
There is a cluster of fine polymorphic calcifications in an anterior distribution in the left upper outer quadrant that appear somewhat round and scattered. This measures 14 x 7mm and is approximately 7 cm from the nipple. This appears suspicious for malignancy.  Otherwise there are no suspicious calcifications.
IMPRESSION:
Suspicious calcifications in the left breast in the upper outer quadrant  7 cm from the nipple in an anterior depth. BIRADS 4. Stereotactic biopsy recommended.
Right breast benign. BIRADS 2. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM. HISTORY: Screening mammogram. Patient reports no family history of breast cancer. Patient denies personal history of breast cancer. COMPARISON: Comparison is made with previous examination from 11-21-2023. FINDINGS: The breasts are heterogeneously dense which may limit the sensitivity of mammography. There are scattered benign-appearing calcifications. No suspicious masses or other findings are identified. IMPRESSION: 1. No evidence of malignancy. 2. BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAM - BILATERAL DIAGNOSTIC

CLINICAL HISTORY: Palpable finding, right breast.

COMPARISON: Mammogram from 06/10/2024.

FINDINGS: The breasts are heterogeneously dense. There is a new, 5 mm cluster of fine pleomorphic calcifications in the right breast at the 3 o'clock position, 2 cm from the nipple. This was biopsied under stereotactic guidance. No architectural distortion or suspicious masses.

IMPRESSION:

1. New cluster of fine pleomorphic calcifications in the right breast at the 3 o'clock position, 2 cm from the nipple, status post stereotactic biopsy. Please see biopsy results.

BI-RADS: 0 (Incomplete). ➡ *Positive*
- MAMMOGRAPHY, RIGHT BREAST
CLINICAL INDICATION: Annual screening mammogram. Patient has maternal history of breast cancer.
COMPARISON: Prior study from 8/14/2021. 
TECHNIQUE: Standard CC and MLO views of the right breast were obtained. 
FINDINGS:
The breast is heterogeneously dense (type C). This may lower the sensitivity of mammography. 
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion.
Stable benign calcifications are noted in the upper outer quadrant, likely representing a small fibroadenoma. This has remained unchanged for several years. 
There are no significant interval changes compared to prior exam.
IMPRESSION:
BI-RADS Category 2: Benign finding. 
Recommend continued annual screening mammography in 12 months. 
ADDENDUM: 
Comparison was made to additional outside mammograms dating back to 2018, which were obtained for review. The benign appearing calcifications in the right upper outer breast have remained stable for at least 4 years, further supporting benignity. No other significant findings or changes over this interval. ➡ *Negative*
- FINDINGS:
Bilateral digital mammogram and left breast ultrasound performed. 

The breasts are heterogeneously dense, which may obscure detection of small masses.

Right Breast: There are no mammographic findings suspicious for malignancy. No significant changes compared to prior exam.

Left Breast: 
In the upper outer left breast at approximately 2:00, 8 cm from the nipple, there is a new 6 mm cluster of fine pleomorphic microcalcifications. At the 9:00 position of the left breast, 4 cm from the nipple, there is a 12 mm irregular mass with indistinct margins, highly suggestive of malignancy. The mass corresponds to a hypoechoic, non-parallel lesion with angular margins and posterior shadowing on targeted ultrasound. Additionally, skin thickening and retraction is noted in the central left breast.

Left axilla: Targeted ultrasound demonstrates an abnormal appearing lymph node with eccentric cortical thickening, measuring up to 15 mm in cortical thickness. 

IMPRESSION:
Suspicious pleomorphic microcalcifications upper outer left breast, highly suggestive of ductal carcinoma in situ (DCIS). Suspicious mass at 9:00 left breast with sonographic correlate, highly suggestive of invasive carcinoma. Abnormal left axillary lymph node, likely metastatic. Recommend ultrasound guided core biopsy of the left 9:00 mass and left axillary lymph node, and stereotactic biopsy of the left microcalcifications. BI-RADS 5. 

No suspicious findings in the right breast. BI-RADS 2. ➡ *Positive*
- Screening Mammography, Bilateral. History: 55 year old female presents for her annual screening mammogram. No new breast symptoms. Comparison is made to previous outside mammogram from 11/15/2023. Findings: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications in both breasts. No new or enlarging masses or areas of architectural distortion. Impression: 1. No mammographic evidence of malignancy. 2. BI-RADS 1. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS
History: Post menopausal screening.
FINDINGS: The breasts are fatty replaced.
There are no suspicious right mammographic findings.
In the left breast there are several clusters of heterogeneous microcalcifications in the central and upper outer left breast. The largest cluster measures approximately 12 mm and is located at the 10:00 position, 5 cm from the nipple. Additional scattered clusters of microcalcifications are present extending anteriorly. 
No dominant mass, architectural distortion or abnormal asymmetry. 
Skin is normal thickness. No adenopathy.
IMPRESSION:
BI-RADS 4: Suspicious microcalcifications in the left breast as described above. Stereotactic core biopsy is recommended for further characterization.
BI-RADS 1: Normal right breast.
ADDENDUM: Stereotactic core biopsy of the largest cluster of microcalcifications in the left breast at the 10:00 position, 5 cm from the nipple was performed. Specimen radiograph confirmed calcifications. Pathology pending. ➡ *Positive*
- MAMMOGRAPHY REPORT

CLINICAL HISTORY: 58-year-old female presents for screening mammogram. No current breast complaints. Personal history of right breast cancer, status post lumpectomy and radiation therapy 10 years ago. Strong family history of breast cancer in multiple maternal relatives.

COMPARISON: Screening mammogram from 1 year prior at outside facility.

FINDINGS:

Right Breast:
Stable post-surgical changes and radiation fibrosis in the upper outer quadrant. No suspicious masses, architectural distortion, or calcifications.

Left Breast: 
Scattered fibroglandular densities. No focal masses or architectural distortion. No suspicious calcifications.

IMPRESSION:

1. Stable post-treatment changes in the right breast. No evidence of recurrent malignancy.
2. No mammographic evidence of malignancy in the left breast.

BIRADS Category 1: Negative. Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- FINDINGS:
The breasts are heterogeneously dense. 
There are scattered areas of fibroglandular density bilaterally. No dominant mass, architectural distortion, or suspicious calcifications are seen in the right breast.
In the upper outer quadrant of the left breast, there is a new group of pleomorphic calcifications spanning approximately 8mm. The calcifications are irregular in size and shape, and some appear to be branching. No associated mass or architectural distortion is seen. 
The remainder of the left breast is unremarkable.

IMPRESSION:
New group of suspicious calcifications in the left upper outer breast, BIRADS 4B. Stereotactic biopsy is recommended for further evaluation. The remainder of the exam is benign, BIRADS 2.

ADDENDUM: 
The patient returned for stereotactic biopsy of the group of calcifications in the left upper outer breast. Specimen radiograph confirmed calcifications within the core samples.
Final pathology results: Ductal carcinoma in situ, intermediate grade, cribriform type. ➡ *Positive*
- ULTRASOUND LEFT BREAST

CLINICAL HISTORY: 62 year old female with palpable lump left breast lower inner quadrant. History of right breast cancer, status post right mastectomy. Mother and maternal aunt with history of breast cancer.

FINDINGS:
Targeted ultrasound of the palpable abnormality in the left lower inner breast at 7:00 demonstrates a 6 mm oval circumscribed hypoechoic mass, parallel to the skin. This likely represents a benign finding such as fibroadenoma. No suspicious solid or cystic mass is identified. The remainder of the left breast is heterogeneously echogenic, consistent with scattered fibroglandular tissue. No skin thickening or architectural distortion.

IMPRESSION:
6 mm likely benign mass left lower inner breast at site of palpable abnormality, probably fibroadenoma. Recommend follow up diagnostic mammogram for further evaluation and correlation.
BI-RADS 2 - Benign finding. Recommend routine screening. ➡ *Negative*
- MAMMOGRAM AND ULTRASOUND OF THE RIGHT BREAST.

CLINICAL HISTORY: Focal thickening in the upper outer quadrant of the right breast noted on clinical exam. Patient denies any palpable lump, nipple discharge, or skin changes. No personal or family history of breast cancer.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications.

Targeted ultrasound of the right upper outer quadrant demonstrates normal fibroglandular tissue with no focal solid or cystic lesions.

IMPRESSION:
No mammographic or sonographic evidence of malignancy in the right breast. The clinically noted thickening in the upper outer quadrant corresponds to normal fibroglandular tissue on targeted ultrasound.

BIRADS 1 - Negative. Recommend annual screening mammogram in 1 year. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Patient presents for routine screening mammogram. No palpable lumps, nipple discharge, or skin changes. No family history of breast cancer.

FINDINGS:
Both breasts are heterogeneously dense which could obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

In the left breast at the 10 o'clock position there is a small group of amorphous calcifications. These are more conspicuous compared to the prior study and have slightly increased in number. Additional magnification views are recommended for further evaluation.

The right breast is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete, need additional imaging evaluation. Recommend magnification views of the left breast for further assessment of the group of amorphous calcifications at 10 o'clock position. These calcifications are new compared to prior studies.

ADDENDUM:
Magnification views of the left breast at 10 o'clock demonstrate a group of amorphous calcifications within a focal asymmetry. The calcifications are indistinct and have slightly increased in number compared to the screening views. Biopsy is recommended for definitive characterization.

FINAL ASSESSMENT:
BI-RADS 4: Suspicious abnormality, biopsy should be considered. Small group of amorphous calcifications within a focal asymmetry at 10 o'clock position of the left breast, indistinct and increasing in number. Recommend stereotactic biopsy for further evaluation. ➡ *Positive*
- SCREENING BILATERAL DIGITAL MAMMOGRAPHY.

INDICATION: Routine screening.

TECHNIQUE: CC and MLO views of both breasts were obtained.

COMPARISON: Previous outside mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense (50-75% fibroglandular tissue). This may lower the sensitivity of mammography.

There are no dominant masses, architectural distortion, or suspicious calcifications.

Stable asymmetric density is noted in the upper outer quadrant of the right breast on the MLO view. This corresponds to superimposed fibroglandular tissue on the CC view. No sonographic correlate is seen.

IMPRESSION:
No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.

BIRADS Category 2: Benign. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:

Right breast:
The right breast consists of scattered areas of fibroglandular density. No focal dominant mass, suspicious calcifications or architectural distortion.

Left breast:
The left breast consists of scattered areas of fibroglandular density. No focal dominant mass or architectural distortion. 
There is a new group of pleomorphic microcalcifications in the left upper outer quadrant at 2 o'clock, 8 cm from the nipple. This group measures approximately 8 mm. No associated mass is seen.

IMPRESSION:

Right breast: Negative study. BI-RADS 1 - Recommend routine annual screening in 12 months.

Left breast: BI-RADS 4 - Suspicious abnormality. Recommend stereotactic biopsy of the new group of microcalcifications in the left upper outer quadrant. 

ADDENDUM: The patient returned for stereotactic biopsy of the group of microcalcifications in the left breast at 2:00, 8 cm from the nipple. Specimen radiograph confirmed that microcalcifications were sampled. The specimen was sent to pathology for analysis. ➡ *Positive*
- SCREENING MAMMOGRAM.
COMPARISON: 03/25/2021. 
TECHNIQUE: Standard views of both breasts were obtained. 
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Stable benign appearing calcifications are noted in the right breast.
IMPRESSION: 
NEGATIVE BILATERAL SCREENING MAMMOGRAM. BIRADS 2.
1-YEAR ROUTINE SCREENING RECOMMENDED. ➡ *Not Stated*
- MAMMOGRAPHY - BILATERAL - SCREENING. CLINICAL HISTORY: Screening mammogram. TECHNIQUE: Craniocaudal and mediolateral oblique views of the breasts were performed using digital mammography. COMPARISON: Comparison is made to the mammogram from 04/04/2023. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- ULTRASOUND LEFT BREAST

CLINICAL HISTORY: Left breast lump palpated by patient, first noticed 1 week ago. No prior imaging available for comparison.

TECHNIQUE: Targeted ultrasound was performed over the area of palpable concern using a 12 MHz linear transducer. 

FINDINGS:
The left breast is heterogeneously echogenic, consistent with scattered fibroglandular tissue. 

Over the area of palpable concern at the 3 o'clock position, 6 cm from the nipple, there is an ovoid, circumscribed, hypoechoic solid mass measuring 11 x 8 x 8 mm. This mass demonstrates slight posterior acoustic enhancement. No internal vascularity is seen on color Doppler interrogation. The surrounding tissue appears normal. No axillary lymphadenopathy is seen.

The remainder of the left breast is unremarkable. No additional masses, areas of architectural distortion, or suspicious calcifications are identified.

IMPRESSION:
Oval, circumscribed, hypoechoic solid mass at the 3 o'clock position of the left breast, measuring 11 x 8 x 8 mm. This likely represents a benign finding such as a fibroadenoma. However, given the palpable nature, a short term follow up ultrasound in 6 months is recommended to confirm stability. 

BI-RADS Category 3: Probably Benign Finding. Short interval follow-up suggested. ➡ *Negative*
- FINDINGS:
Right breast: Heterogeneously dense parenchyma. There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Left breast: Scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. No mammographic evidence of malignancy. Recommend continued annual screening in 12 months with mammography.

ASSESSMENT: BI-RADS 1 - Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: 63 year-old female referred for screening mammogram. No prior history of breast cancer. No family history of breast cancer. Patient denies breast lumps, pain, nipple discharge or skin changes.

FINDINGS:
Both breasts are composed of scattered fibroglandular densities (BI-RADS density category c).

Right breast:
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Left breast:
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Stable, benign-appearing lymph nodes are noted in both axillae.

IMPRESSION:
Bilateral mammogram, negative for malignancy. No mammographic evidence of microcalcifications.

RECOMMENDATION:
Resume routine annual screening mammography in 1 year.

BI-RADS category 1: Negative. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED MAGNIFICATION LEFT BREAST.
History: Follow-up to assess response to neoadjuvant chemotherapy for known left breast cancer.
FINDINGS: There is scattered fibroglandular density in both breasts. 
The right breast mammogram is within normal limits. BI-RADS 1.
In the left breast, residual asymmetry is noted at 6:00, extending to the nipple. At the site of the known mass, architectural distortion persists however no discrete mass is identified.
Magnification views of the left breast demonstrate two small clusters of microcalcifications. At 12:00, 6 cm from the nipple there is a cluster of fine linear microcalcifications in a segmental distribution spanning 5 mm. At 3:00, 4 cm from the nipple there is a cluster of fine pleomorphic microcalcifications spanning 8 mm. These were not present on the prior study.
The left axilla is clear.
IMPRESSION:
Residual asymmetry and architectural distortion at the site of known left breast cancer, consistent with partial response to neoadjuvant chemotherapy. Recommend further mammographic and MR imaging follow-up to determine stability.
New clusters of suspicious microcalcifications at 12:00 and 3:00 left breast, highly suspicious for multifocal disease. Stereotactic core biopsy recommended. BI-RADS 4C.
Ultrasound correlation is advised to further assess extent of residual disease.
ADDENDUM: Stereotactic core biopsy of the 12:00 and 3:00 left microcalcifications performed. Pathology shows ductal carcinoma in situ, high nuclear grade with comedonecrosis. Ductal carcinoma in situ also demonstrated at the site of the 3:00 biopsy. BI-RADS 6. ➡ *Positive*
- MAMMOGRAPHY REPORT

EXAMINATION: Bilateral Screening Mammogram

COMPARISON: 05/15/2022

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
Right Breast:
The breast is almost entirely fatty. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Left Breast:
The breast is heterogeneously dense, which may obscure detection of small masses. There are no dominant masses, suspicious calcifications, or areas of architectural distortion. There is no significant change compared to prior exam.

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Breast density: Heterogeneously dense (C).

RECOMMENDATIONS:
1. Recommend continued annual screening mammography in 12 months. 
2. Please inform the patient that her breast tissue is dense. Dense breast tissue is very common and is not abnormal. However, dense breast tissue can make it harder to find cancer on a mammogram and may also be associated with an increased risk of breast cancer. This information is given to raise awareness and to encourage discussion between the patient and her physician regarding the presence of other risk factors, in addition to dense breast tissue. A report of the results was sent to the patient's physician. The patient may also wish to discuss this information with her physician. For additional information, the patient can visit http://www.fda.gov/breasthealth.

BIRADS: 1 

Radiologist: Dr. Jane Smith ➡ *Negative*
- ULTRASOUND RIGHT BREAST

CLINICAL HISTORY: 47-year-old female with palpable right breast mass, upper outer quadrant.

COMPARISON: None.

FINDINGS: 
Right breast ultrasound was performed. 
The breast tissue is heterogeneously echogenic. 
A 1.2 x 0.8 x 0.9 cm oval circumscribed hypoechoic solid mass is identified at the 10 o'clock position, 6 cm from the nipple, corresponding to the palpable abnormality. 
No suspicious calcifications are identified.
No axillary lymphadenopathy.

IMPRESSION:
Right breast 10 o'clock 1.2 cm oval circumscribed solid mass, likely fibroadenoma. 
No microcalcifications identified.
BIRADS 2 - Benign finding. Recommend routine screening. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Patient presents for routine screening mammogram. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior exam dated 12/15/2021.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable benign calcifications are present in the right breast.
No significant interval change from prior exam.
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Recommend continued annual screening in 12 months.
BIRADS 1 - Negative.
ADDENDUM: Tomosynthesis was performed. No additional suspicious findings are identified on tomosynthesis images. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM

CLINICAL HISTORY: Right breast microcalcifications.

TECHNIQUE: Standard CC and MLO views of the bilateral breasts were obtained.

FINDINGS:

There are scattered 0.5 to 1.0 mm round calcifications in the right breast, unchanged from prior studies. These are benign in appearance. There is a 7 mm cluster of pleomorphic calcifications in the right breast at the 1 o'clock position, 3 cm from the nipple. This was biopsied under stereotactic guidance.

IMPRESSION:

1. Benign-appearing scattered right breast calcifications, stable.
2. Cluster of pleomorphic calcifications in the right breast at the 1 o'clock position, 3 cm from the nipple, status post stereotactic biopsy. Please see biopsy results.

BI-RADS: 0 (Incomplete). ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS:
INDICATION: Annual screening mammogram.
COMPARISON: None.
FINDINGS:
Both breasts show scattered areas of fibroglandular density. There are no dominant masses, focal asymmetries, or areas of architectural distortion.
In the left breast at the 12:00 position, 6 cm from the nipple, there is a cluster of amorphous calcifications within an area measuring approximately 8mm. There are no associated masses or architectural distortion.
In the right breast, there are no suspicious calcifications.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for further evaluation of the cluster of amorphous calcifications at 12:00 position, left breast, 6 cm from the nipple.
BI-RADS Category 1: Normal right breast.
ADDENDUM:
The patient returned for stereotactic-guided biopsy of the cluster of amorphous calcifications at the 12:00 position of the left breast on [DATE]. Pathology results showed benign fibrocystic changes with calcifications. No evidence of malignancy.
REVISED IMPRESSION:
BI-RADS Category 2: Benign finding in the left breast. Recommend routine screening in 1 year.
BI-RADS Category 1: Normal right breast. ➡ *Positive*
- MAMMOGRAM, BILATERAL.
CLINICAL INDICATION: Screening study.  
TECHNIQUE: Standard CC and MLO views of each breast were obtained using Digital Breast Tomosynthesis.  
COMPARISON: Prior mammogram from [**Hospital 73562**] dated [**2024-6-22**].
FINDINGS:
The breasts are predominantly fatty.
There are no suspicious masses, asymmetries, or architectural distortions.
There are no suspicious grouped calcifications.
Bilateral benign calcifications are noted, unchanged.
IMPRESSION: 
1. No mammographic evidence of malignancy. 
2. Bilateral breasts are predominantly fatty.
3. Benign calcifications noted bilaterally, unchanged from prior.
RECOMMENDATION: Routine annual screening mammography is recommended. 
BIRADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Right breast tissue predominantly fatty replaced. There are scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications.  
Left breast tissue heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, suspicious calcifications or architectural distortion. No significant change compared to prior mammograms.

IMPRESSION:
1. Negative bilateral screening mammogram.  
2. Heterogeneously dense breasts, which lowers the sensitivity of mammography. 

BIRADS 1 - Negative. Recommend annual screening mammography. ➡ *Negative*
- INDICATION: Left breast nodule. History of right breast cancer.

COMPARISON: Outside screening mammogram from April 2022.

TECHNIQUE: CC and MLO views of the left breast.

FINDINGS:
The left breast is predominantly fibroglandular tissue. No dominant mass, architectural distortion, or suspicious calcifications. The visualized left axilla is unremarkable.

The left breast skin is normal in thickness. No significant changes compared to outside screening mammogram.

IMPRESSION:
Left breast: No mammographic evidence of malignancy. No suspicious microcalcifications identified.
BI-RADS category 1: Negative.

12-month routine screening mammogram is recommended. ➡ *Negative*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
INDICATION: Screening mammogram and targeted ultrasound for left breast tenderness at 10 o'clock position.
TECHNIQUE: Digital mammographic views were obtained including craniocaudal and mediolateral oblique views.
Targeted ultrasound of the left breast 10 o'clock position was performed with a high frequency linear transducer.
COMPARISON: Mammogram from 1 year prior.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Targeted ultrasound of the left breast 10 o'clock position demonstrates normal fibroglandular tissue. No suspicious solid or cystic lesions are identified.
Stable benign-appearing calcifications are noted in the left breast, likely vascular in etiology.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. No sonographic abnormality in the area of concern. Correlate clinically for tenderness.
BIRADS 1 - Negative.
ADDENDUM: 
No suspicious clustered microcalcifications, pleomorphic calcifications, or segmental/linear calcifications are identified in either breast. Previously noted benign vascular calcifications are stable. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: This is a 54-year-old female patient presenting for routine annual screening mammography.
COMPARISON: Prior bilateral mammograms of 11/22/2020 were available for comparison.
FINDINGS: Breast parenchyma is heterogeneously dense, with scattered areas of fibroglandular density. There is no evidence of a mass, architectural distortion, or suspicious calcifications in either breast. No interval change since the prior examination. 
The skin, nipple and areola are unremarkable bilaterally. No axillary, supraclavicular, or internal mammary lymphadenopathy is evident.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography. Clinical breast examination is recommended.

BI-RADS Category 1: Negative.

ULTRASOUND LEFT BREAST:
INDICATION:  Area of palpable abnormality 7 o'clock position left breast.
TECHNIQUE: Targeted ultrasound of the palpable abnormality in the left breast 7 o'clock position was performed using a 12 MHz linear transducer.
FINDINGS: The area of palpable concern corresponds to normal fibroglandular tissue. No discrete solid mass, complex cystic mass, or dilated duct is identified. Normal Cooper's ligaments and fat lobules are noted. No skin thickening or retraction. 
IMPRESSION: Negative targeted ultrasound of the left breast 7 o'clock position. No sonographic evidence of malignancy. Recommend clinical follow-up and correlation with physical exam findings. 
BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM AND ULTRASOUND
HISTORY: Annual screening. No breast complaints. Prior benign right breast biopsy at 10 o'clock. Strong family history of breast cancer in sister and mother, both diagnosed premenopausally.
Comparison made to prior imaging from June 2021.
FINDINGS: Bilateral scattered fibroglandular densities. No significant change from previous mammograms. 
Right breast: Stable circumscribed mass at 10 o'clock, consistent with known fibroadenoma. No new masses, calcifications, or architectural distortion.
Left breast: No focal masses, suspicious calcifications, or architectural distortion. 
Bilateral targeted ultrasound was performed. Right breast 10 o'clock mass demonstrates benign features, measuring 1.2 cm, unchanged. No other suspicious lesions in either breast on ultrasound.
IMPRESSION:
1. Stable benign right 10 o'clock breast mass, consistent with fibroadenoma. No mammographic or sonographic evidence of malignancy in either breast.
2. Recommend annual screening mammography and ultrasound in 12 months due to elevated lifetime risk based on strong family history.
BIRADS 2.
ADDENDUM: No microcalcifications are identified in either breast. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or skin thickening in either breast.
At the 12:00 position of the right breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. No other suspicious calcifications are seen in either breast.
IMPRESSION:
BI-RADS 0: Incomplete. Additional imaging evaluation with magnification views is needed to further assess the group of amorphous calcifications in the right breast at 12:00, 6 cm from the nipple. 
RECOMMENDATIONS:
Magnification views of the right breast are recommended for further evaluation of the microcalcifications. Please contact the patient to schedule this additional imaging. A final assessment will be made following completion of the diagnostic evaluation. ➡ *Positive*
- MAMMOGRAM LEFT BREAST.
HISTORY: Screening.
COMPARISONS: 07/06/2023.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram. No palpable masses or focal areas of pain.

COMPARISON: Screening mammogram from 10/15/2020.

FINDINGS:
The breasts are heterogeneous dense, which can obscure small masses.

There are no dominant masses, architectural distortions, or suspicious areas of skin thickening in either breast.

In the right upper outer quadrant, there is a new cluster of indeterminate pleomorphic microcalcifications at 10:00, 6 cm from the nipple, spanning 8 mm. No other suspicious calcifications are noted in the right breast.

The left breast is stable in appearance. No suspicious masses, architectural distortion, or calcifications are identified.

IMPRESSION:
BI-RADS 0: Incomplete. New cluster of indeterminate microcalcifications in the right breast at 10:00, 6 cm from the nipple. Additional magnification views are recommended for further evaluation.

BI-RADS 2: Benign. The left breast is stable compared to prior mammograms.

ADDENDUM:
Right breast magnification views demonstrate a cluster of pleomorphic microcalcifications spanning 9 mm at 10:00, 6 cm from the nipple. Stereotactic biopsy is recommended for tissue diagnosis.

FINAL ASSESSMENT:
RIGHT BREAST, BI-RADS 4: Suspicious. Stereotactic biopsy recommended.
LEFT BREAST, BI-RADS 2: Benign. ➡ *Positive*
- MAMMOGRAPHIC FINDINGS:
LEFT BREAST:
There is heterogeneously dense fibroglandular tissue in the left breast. There are no dominant masses, architectural distortion, or suspicious calcifications.

RIGHT BREAST:
There is predominantly fatty tissue in the right breast. There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. The breasts are heterogeneously dense, which could obscure detection of small masses.
2. No suspicious microcalcifications are identified in either breast.

RECOMMENDATION:
Routine screening mammography in 1 year, unless clinically indicated earlier.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM

CLINICAL HISTORY: Screening mammogram.

COMPARISON: 05/15/2023.

FINDINGS: The breasts are heterogeneously dense which may obscure a small mass. There are scattered benign-appearing calcifications bilaterally, stable. There is a 0.8 cm cluster of new fine pleomorphic calcifications in the left breast at 9 o'clock, 5 cm from the nipple.

IMPRESSION:

1. New fine pleomorphic calcifications in the left breast - recommend spot magnification views for further evaluation. 

BI-RADS: 0 (Needs additional imaging evaluation). ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:

Right Breast:
The right breast is heterogeneously dense (BI-RADS density C). There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left Breast:
The left breast is heterogeneously dense (BI-RADS density C). There are scattered fibroglandular elements in the left breast. In the upper outer quadrant of the left breast, there is a small group of amorphous microcalcifications measuring approximately 5 mm in diameter. No associated mass or architectural distortion is seen. No other suspicious calcifications are noted.

Axillae:
There are no abnormal axillary lymph nodes bilaterally.

Skin:
The skin appears normal bilaterally with no thickening or retraction.

Nipples:
The nipples are in profile and appear normal bilaterally.

IMPRESSION:

Left Breast:
- Small group of amorphous microcalcifications in the upper outer quadrant, suspicious for early ductal carcinoma in situ (DCIS). Additional magnification views are recommended for further evaluation. BI-RADS 4B.

Right Breast:
- No mammographic evidence of malignancy. Recommend routine screening in 1 year. BI-RADS 1.

ADDENDUM:
Magnification views of the left breast upper outer quadrant were performed. The amorphous microcalcifications are better visualized and appear more conspicuous. Stereotactic biopsy is recommended for definitive diagnosis. ➡ *Positive*
- RADIOLOGY REPORT
PROCEDURE: Bilateral Digital Screening Mammogram

HISTORY: 55-year-old female, no prior mammograms available for comparison. 

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 

There are no dominant masses, architectural distortion or suspicious calcifications in the right breast. 

In the left breast, there is a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant posteriorly at the 2:00 position, 6 cm from the nipple. This cluster measures approximately 6 mm in greatest dimension. No associated mass, architectural distortion or skin thickening is identified. 

There are also scattered fibroglandular densities bilaterally, more prominent on the left. These are stable in appearance from the prior exam and likely represent normal breast parenchyma.

IMPRESSION:
1. New cluster of suspicious microcalcifications in the left breast at 2:00, 6 cm from the nipple. Additional evaluation with spot compression and magnification views is recommended for further characterization. 
2. Stable scattered fibroglandular densities bilaterally, likely representing normal breast tissue.

BI-RADS CATEGORY: 4 - Suspicious abnormality, biopsy should be considered.

RECOMMENDATION: Left diagnostic mammogram with spot compression and magnification views. Consider biopsy if indicated after additional imaging. ➡ *Positive*
- REPORT:"BILATERAL SCREENING MAMMOGRAM.
CLINICAL INDICATION: Screening mammogram.
COMPARISON: Comparison with prior studies from June 2020.
TECHNIQUE: Bilateral digital 2D mammogram with tomosynthesis views in CC and MLO projections. 
FINDINGS: 
RIGHT BREAST: The breast is heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, areas of architectural distortion or suspicious calcifications. No significant interval change since the prior exam.
LEFT BREAST: The breast is heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, areas of architectural distortion or suspicious calcifications. No significant interval change since the prior exam.
IMPRESSION:
1. No evidence of malignancy in either breast. 
2. Recommend continued annual screening mammography.
BIRADS 1 - Negative.
ADDENDUM: Reviewed with Dr. Jane Smith, Radiologist. No additional imaging needed at this time based on the screening mammogram findings alone." ➡ *Negative*
- SCREENING MAMMOGRAM BIRADS 1
Mammographic views for evaluation: standard FFDM CC and MLO views of the left and right breasts.
FINDINGS:
Bilateral predominantly fatty replaced breasts. No suspicious masses, architectural distortions or calcifications. No significant interval changes compared to prior studies.
IMPRESSION:
No mammographic findings that are concerning for malignancy. Recommend annual screening mammography.
Breast composition: The breasts are almost entirely fatty.
BIRADS Category 1: Negative. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Screening. 
TECHNIQUE: Screening mammogram performed of the bilateral breasts utilizing full field digital mammography.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense.
No suspicious masses or architectural distortions. 
No suspicious calcifications. Stable benign appearing calcifications in the left breast.
Skin is unremarkable. No significant axillary lymphadenopathy.
IMPRESSION: 
NEGATIVE SCREENING MAMMOGRAM. 
BIRADS Category 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- BREAST ULTRASOUND
CLINICAL HISTORY: 45 year old female with palpable lump left breast upper outer quadrant. No prior mammograms available for comparison.
TECHNIQUE: Bilateral breast ultrasound was performed using a 12 MHz linear array transducer.
FINDINGS:
RIGHT BREAST: The right breast demonstrates heterogeneously dense fibroglandular tissue. No focal masses, areas of architectural distortion, or suspicious calcifications are seen. 
LEFT BREAST: The left breast demonstrates heterogeneously dense fibroglandular tissue. A 1.2 cm oval circumscribed hypoechoic mass is identified at the 2 o'clock position, 5 cm from the nipple, corresponding to the palpable abnormality. No suspicious calcifications are seen.
IMPRESSION:
1. 1.2 cm circumscribed oval mass left breast 2 o'clock position, 5 cm from the nipple, corresponding to the palpable abnormality. This may represent a fibroadenoma. Recommend biopsy for definitive diagnosis.
2. Heterogeneously dense breast tissue bilaterally. No suspicious calcifications, architectural distortion or additional masses are seen in either breast.
BIRADS 4A: Suspicious abnormality, biopsy should be considered. ➡ *Negative*
- MAMMOGRAM REPORT
MEDICAL HISTORY: Annual screening mammogram. Previous benign biopsy in the right breast 2 years ago.
TECHNIQUE: Bilateral digital screening mammogram with 2D and tomosynthesis views.
COMPARISON: Previous mammogram from January 2022.  
FINDINGS:  
Right breast: The breast is heterogeneously dense which could obscure detection of small masses. There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Left breast: The breast is scattered fibroglandular density. There are no dominant masses, areas of architectural distortion or suspicious calcifications.  
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Recommend continued annual screening mammography in 1 year.
BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS: Bilateral breast tissue contains scattered fibroglandular densities. No dominant mass, suspicious calcifications, or architectural distortion.

Targeted ultrasound of the right breast 10:00 position demonstrates normal fibroglandular tissue with no sonographic correlate for the palpable abnormality.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy. 
2. Palpable abnormality right breast 10:00, no sonographic correlate. Likely represents normal fibroglandular tissue.

BIRADS 1 - Negative. Recommend annual screening mammography. ➡ *Negative*
- REPORT: "ABNORMAL FFDM ARM OBSP MAMMOGRAM.
This was a full field digital mammogram. No previous images were available for comparison. 
Right breast: Within dense breast tissue, a 1.5 cm irregular mass is seen in the right upper outer quadrant. Architectural distortion and associated skin thickening is also noted. No microcalcifications.
Left breast: Scattered areas of fibroglandular density are present. No dominant masses, areas of architectural distortion, or suspicious calcifications. 
Breast Density >= 75%: Yes.
OBSP to recall client for assessment: Yes.
BIRADS-0. Additional views are required for full assessment.

ADDENDUM:
Diagnostic mammographic views and targeted right breast ultrasound were performed. The irregular mass in the right breast demonstrates spiculated margins and posterior acoustic shadowing on ultrasound, highly suspicious for malignancy. Ultrasound-guided core biopsy is recommended for tissue diagnosis. BIRADS-5.

Recommend recall from screening for further assessment.
" ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
CLINICAL HISTORY: 49-year-old for routine screening. 
TECHNIQUE: Standard CC and MLO views of each breast.
COMPARISON: None.
FINDINGS:
The breasts are predominantly fatty. 
There is no significant asymmetry, architectural distortion or suspicious mass seen in either breast.
In the left breast upper outer quadrant, there is a cluster of fine pleomorphic microcalcifications spanning an area of approximately 12 mm. No other suspicious microcalcifications are seen elsewhere.
No radiographic evidence of skin thickening or nipple retraction.
IMPRESSION:
Left breast: Cluster of suspicious microcalcifications in the upper outer quadrant, BI-RADS 4B. Stereotactic biopsy is recommended for further evaluation.  
Right breast: No suspicious mammographic findings, BI-RADS 1. Annual screening is recommended.
ADDENDUM:
The patient proceeded with stereotactic biopsy of the left breast microcalcifications. Pathology results showed ductal carcinoma in situ, intermediate grade. Patient will be referred to surgery for further management. ➡ *Positive*
- SCREENING MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 

COMPARISON: None available.

FINDINGS:  
The breast tissue is predominantly fatty.

There are no masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
BIRADS category 1: Negative.

1-year routine screening mammogram is recommended. ➡ *Not Stated*
- TMIST PARTICIPANT: FULL FIELD DIGITAL MAMMOGRAPHY ARM.

Indication: Screening mammogram.

This patient is participating in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomly assigned to the full field digital mammography arm. Informed consent has been obtained.

Comparison: Previous screening mammogram from 1 year ago.

Technique: Standard CC and MLO views of both breasts were obtained.

Findings:
The breast tissue is heterogeneously dense, which may obscure small masses.
There are no masses, architectural distortions, or suspicious calcifications.
No significant changes compared to prior study.

Impression:
1. Negative bilateral screening mammogram.
2. Breast Imaging Reporting and Data System (BI-RADS) Category 1: Negative. Routine screening recommended in 1 year.

ADDENDUM:
This study was performed as part of the TMIST trial. The patient was randomly assigned to the FFDM arm of the study. ➡ *Not Stated*
- ULTRASOUND AND MAMMOGRAM OF THE RIGHT BREAST

CLINICAL HISTORY: 57-year-old female with right breast lump at 10 o'clock position, painful. No prior mammograms available for comparison.

FINDINGS:  
Mammogram: The breast is heterogeneously dense. No dominant mass, architectural distortion, or suspicious calcifications are identified. 

Ultrasound: Targeted ultrasound of the right breast at 10 o'clock position 3 cm from the nipple demonstrates a 0.8 x 0.6 x 0.7 cm oval, circumscribed, hypoechoic solid mass. This corresponds to the palpable abnormality. No suspicious axillary lymph nodes are seen.

IMPRESSION:
1. Oval circumscribed hypoechoic solid mass at 10 o'clock position of the right breast corresponding to the palpable lump. This is most likely a fibroadenoma. 6-month follow-up ultrasound is recommended to document stability.

2. No suspicious mammographic findings in the right breast. Recommend annual screening mammography.

BIRADS 2 - Benign findings. Recommend 6-month follow-up ultrasound of the right breast mass.

ADDENDUM: Pathology from ultrasound-guided core biopsy of the right breast mass confirms a fibroadenoma. Recommend clinical follow-up as needed. Annual screening mammography is recommended. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.
INDICATION: Annual screening. No palpable masses or other breast concerns.
COMPARISON: Prior imaging from 1 year ago.
FINDINGS:
Right breast: Scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications.  
Left breast: Scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications.
IMPRESSION:
1. Bilateral mammogram shows no mammographic evidence of malignancy. 
2. Annual screening mammography is recommended. 
BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening mammogram.
Comparison available dating from 02/2022.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- OBSP FFDM MAMMOGRAM RIGHT AND LEFT.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
There are no suspicious mammographic findings. Specifically, no dominant masses, architectural distortion or suspicious calcifications are identified.
IMPRESSION: BIRADS-2 Benign finding. Routine screening recommended.
ADDENDUM: Spoke with client regarding benign finding of bilateral fibroglandular density. No further imaging is required at this time. Routine screening mammogram is recommended in one year through the Ontario Breast Screening Program. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast. 

At the 10:00 position of the left breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. Additional magnification views confirm these calcifications. No associated mass, distortion or skin changes are identified.

The right breast is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation. 
Suspicious grouped amorphous microcalcifications in the left breast. Additional diagnostic workup with spot compression magnification views is recommended for further evaluation.

ADDENDUM - DIAGNOSTIC MAMMOGRAM LEFT BREAST: 
Magnification views of the left breast in the area of concern demonstrate a group of suspicious pleomorphic microcalcifications at the 10:00 position, 6 cm from the nipple, spanning approximately 12 mm. No associated mass or distortion is identified. 

FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- TMIST FFDM ONLY NORMAL MAMMOGRAM
This 58 year old patient participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.  

FINDINGS:
There are no suspicious masses, architectural distortions or concerning asymmetries. 
The breast parenchyma is heterogeneously dense, which could obscure detection of small masses. 
There are no dominant masses or suspicious calcifications. 

IMPRESSION:
1. No mammographic evidence of malignancy.
2. Predominantly fatty replaced breasts.
3. Recommend routine screening mammography in 1 year.

BIRADS: 2 
Compared to previous exam: Yes
Breast density <75%: Yes
Recommend OBSP recall in 1 year: Yes ➡ *Not Stated*
- FINDINGS:
Bilateral digital screening mammogram was performed.
The breasts are almost entirely fatty.
There are no suspicious masses, architectural distortion or dominant masses in either breast. 
In the right breast at the 12:00 position 4 cm from the nipple there is a small cluster of indeterminate microcalcifications measuring about 5 mm in size. No previous mammograms are available for comparison. 
The left breast is unremarkable. 
SUMMARY:
Small cluster of indeterminate microcalcifications right breast at 12:00 position 4 cm from the nipple. These are new since no previous mammograms are available. Recommend magnification views of this area and possibly biopsy depending on the appearance on the magnification views. 
BIRADS 4 right breast.
BIRADS 1 left breast. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening. No current breast complaints.

COMPARISON: Screening exam dated 9/15/2019 at this facility.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

There is an oval equal density circumscribed mass in the right breast at 2 o'clock posterior depth measuring approximately 9 mm, most consistent with an isolated fibroadenoma.

A few scattered punctate and amorphous microcalcifications are noted in the upper outer quadrant of the left breast which are new since the prior study, distributed over an area spanning approximately 2 cm. These are indeterminate but likely benign. Follow-up magnification views are recommended for further evaluation.

The visualized axillary regions appear unremarkable bilaterally.

Unchanged benign-appearing vascular calcifications are noted bilaterally.

IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation needed. Magnification views are recommended to further characterize the scattered microcalcifications in the left breast upper outer quadrant.

ADDENDUM:
Magnification views of the left breast confirm a cluster of indeterminate microcalcifications in the upper outer quadrant. Stereotactic biopsy is recommended for tissue diagnosis. BI-RADS Category 4. ➡ *Positive*
- FINDINGS:
Bilateral diagnostic mammogram with coned compression magnification views of the left breast. 

Mammography shows the breasts are heterogeneously dense, which may obscure detection of small masses. 

No suspicious findings are seen in the right breast. There are no dominant masses, areas of architectural distortion or suspicious calcifications.

In the left breast at the 11:00 position, 7 cm from the nipple, there is an 11 mm irregular mass with indistinct margins. No other discrete masses or areas of architectural distortion are identified in the left breast. 

Magnification views of the left breast demonstrate a grouped distribution of pleomorphic microcalcifications extending over an area of approximately 2 cm, centered at the 10:00 position, 5 cm from the nipple. A few additional scattered calcifications are seen in the vicinity.

IMPRESSION:
1. Right breast: No mammographic evidence of malignancy. BI-RADS 1. Recommend routine screening in 1 year.

2. Left breast:
- 11 mm irregular mass at 11:00, 7 cm from the nipple, suspicious for malignancy. 
- Pleomorphic microcalcifications in a grouped distribution over 2 cm at 10:00, 5 cm from the nipple, suspicious for malignancy. BI-RADS 4C. 

RECOMMENDATIONS:
Ultrasound-guided core needle biopsy is recommended for the irregular mass and stereotactic core needle biopsy for the pleomorphic calcifications in the left breast, to exclude malignancy. 

This is a BI-RADS 4C assessment of the left breast, with a high suspicion of malignancy based on the imaging findings described above. ➡ *Positive*
- ULTRASOUND AND MAMMOGRAM OF LEFT BREAST:

CLINICAL HISTORY: 45 year old female with palpable lump in upper outer left breast. Previous mammograms from 2022 at St Michael's Hospital were compared.

FINDINGS:
The breast tissue is heterogeneously dense which may lower the sensitivity of mammography. 
There is a 1 cm hypoechoic irregular shaped mass at 2:00, 6 cm from the nipple on ultrasound. It demonstrates posterior acoustic shadowing. No increased vascularity is seen.
At the site of palpable abnormality in the upper outer left breast, there is a group of pleomorphic calcifications spanning 8mm. The calcifications are new compared to previous mammograms. 
No other suspicious masses, calcifications, asymmetries or architectural distortion in either breast. 
No skin thickening or retraction. The nipple and areola are unremarkable. 
There are no abnormal axillary lymph nodes.

IMPRESSION:
New group of suspicious pleomorphic calcifications at site of palpable abnormality in the upper outer quadrant of the left breast, spanning 8 mm, BI-RADS 4C. Ultrasound guided core biopsy is recommended for definitive diagnosis. 

1 cm irregular hypoechoic mass at 2:00 in the left breast, BI-RADS 4B. Ultrasound guided core biopsy is recommended.

No other suspicious findings.

BI-RADS 4C left, BI-RADS 1 right

ADDENDUM: Left breast core biopsy showed invasive ductal carcinoma at both biopsy sites, grade 2. ➡ *Positive*
- TMIST FFDM ARM OBSP MAMMOGRAM - NORMAL FINDINGS. 
The patient was enrolled in the TMIST study and randomized to the full field digital mammography arm after providing informed consent as per Research Ethics Board guidelines.
Comparison to prior studies: Available.
Breast composition: Scattered fibroglandular densities (< 75% density).  
Routine OBSP screening recommended in 12 months.
No mammographic abnormalities suspicious for malignancy are identified. There is no mention of microcalcifications.
Final BIRADS Assessment Category: 1 - Negative. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available.

FINDINGS:
The breasts are composed of scattered fibroglandular densities. There are no dominant masses, suspicious calcifications or architectural distortion in either breast.

In the right breast at the 9 o'clock position, approximately 6 cm from the nipple, there is an 8 mm cluster of indeterminate microcalcifications. Additional magnification views confirm the presence of the microcalcifications. No associated mass or architectural distortion is identified.

The left breast is unremarkable.

IMPRESSION:

Right breast: 8 mm cluster of indeterminate microcalcifications at 9 o'clock position, 6 cm from the nipple. Additional imaging evaluation with magnification views confirms the presence of microcalcifications. BI-RADS 4B - Suspicious abnormality, biopsy should be considered.

Left breast: Negative. BI-RADS 1 - Negative.

RECOMMENDATIONS:
1. Stereotactic core biopsy of the right breast microcalcifications is recommended for further evaluation.
2. Routine annual screening mammogram of the left breast in 12 months.

ADDENDUM:
The patient proceeded with stereotactic core biopsy of the right breast microcalcifications. Pathology results showed ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. The patient was referred to a breast surgeon for further management. ➡ *Positive*
- Breast Ultrasound Report
Patient Name: [Patient Name]
Date of Exam: 2024-07-10

Clinical History:
Screening breast ultrasound. Recent mammogram showed a cluster of microcalcifications in the right breast.

Findings:
Right Breast:
- A small cluster of microcalcifications is identified in the right breast at the 3 o'clock position, correlating with findings on mammography. The microcalcifications are located within a slightly ill-defined area of hyperechogenicity measuring approximately 7 mm in diameter. No distinct mass is visualized.
Left Breast:
- No suspicious masses or calcifications are identified.

Impression:
- Right breast: Ultrasound findings correlate with mammographic microcalcifications. The ill-defined hyperechogenicity surrounding the microcalcifications is slightly suspicious. Biopsy is recommended for further evaluation.
- Left breast: No suspicious sonographic abnormalities.

Recommendations:
- Ultrasound-guided biopsy of the microcalcifications in the right breast.
- Correlation with mammographic findings.
- Further management as clinically indicated.

BI-RADS Category: 4 (Suspicious) ➡ *Positive*
- TMIST FFDM ONLY MAMMOGRAM 
BI-RADS 1 Negative study

Left breast tissue was designated as scattered areas of fibroglandular density. 
No abnormal findings were identified in either breast. Specifically, no dominant masses, suspicious calcifications, or other findings that would be suspicious for malignancy are seen in either breast.
The client will be recalled for routine screening through TMIST in 1 year. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAMS.
INDICATION: Family history of breast cancer.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Additional views included spot compression views of the left breast upper outer quadrant.
COMPARISON: None.
FINDINGS:
Both breasts are almost entirely fatty.
In the right breast no suspicious masses, calcifications or other abnormalities are seen. BIRADS 1. 
In the left breast, no suspicious masses or significant architectural distortion is identified. However, there is a small group of faint, amorphous microcalcifications at the 2:00 position of the left breast, about 7 cm from the nipple on the CC view. These measure about 4 mm in extent. They are not clearly seen on the MLO view. Additional spot compression magnification views confirm these calcifications. 
No other significant mammographic abnormalities are seen in the left breast.
SUMMARY:  
Right breast: Negative. BIRADS 1.
Left breast: Small group of amorphous microcalcifications at 2:00, 7 cm from the nipple, measuring about 4 mm in extent, seen on CC and spot compression views. These are new and of uncertain significance. Biopsy should be considered. BI-RADS 4b. Recommend ultrasound correlation and possible stereotactic biopsy. ➡ *Positive*
- NORMAL TMIST FFDM OBSP MAMMOGRAM.  
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography arm.
Compared to Previous: No prior mammograms available for comparison.  
Breast Density >= 75%: No.
Breast Density < 75%: Yes. The breast tissue is heterogeneously dense, which may obscure small masses.
OBSP to recall client in 2 years: Yes. 
The mammogram shows no dominant masses, architectural distortion, or suspicious calcifications. 
BIRADS-2: Benign findings. Recommend routine screening in 2 years. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: This is a routine screening study.   
TECHNIQUE: Standard CC and MLO views obtained.
COMPARISON: None available.
FINDINGS: 
The breast tissue is heterogeneously dense. No dominant mass, distortion or asymmetry.   
IMPRESSION:
Negative screening mammogram.  
BIRADS Category 1 - Normal mammogram. Recommend routine screening in 1 year. ➡ *Not Stated*
- BILATERAL DIAGNOSTIC MAMMOGRAMS WITH ADDITIONAL LEFT MAGNIFICATION VIEWS
INDICATION: Right lateral fullness, last mammogram over a year ago. No known problems. Annual screening.
PROCEDURE: Craniocaudal and MLO views both breasts. Spot compression magnification view upper outer left.
COMPARISON: Hospital studies from 06/19.
FINDINGS:
The breasts are heterogeneously dense which may lower the sensitivity of mammography.
There is stable benign appearing calcification in the left subareolar area.
There are no dominant masses or areas of architectural distortion.
Magnification views of the left breast demonstrate a small group of pleomorphic calcifications in the upper outer quadrant measuring 6 mm in diameter. These are new compared to the previous studies.
There are no suspicious findings in the right breast.
IMPRESSIONS:
1. New group of suspicious pleomorphic microcalcifications left upper outer quadrant, BI-RADS 4B. Stereotactic core biopsy is recommended. The patient is aware and will call to book.
2. No suspicious findings right breast, BI-RADS 2. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS AND MAGNIFICATION VIEWS.

HISTORY: Routine screening. No breast complaints.

FINDINGS: The breasts have scattered fibroglandular tissue.

There are no suspicious right mammographic findings.

In the upper inner left breast 7.5 cm from the nipple there is a 4.5 mm cluster of amorphous calcifications. This is an indeterminate finding.

In the left subareolar breast there is a 1 cm low density mass. This is probably benign. The remainder of the left breast is unremarkable.

SUMMARY:
Small cluster of amorphous calcifications at 11:00 in the upper inner left breast. Since this patient has had no prior mammograms for comparison, these are considered an indeterminate finding (probably benign - BI-RADS 0). A short term 6 month follow-up mammogram is recommended to evaluate stability.

Probably benign mass left subareolar breast. BI-RADS 3. Follow-up mammogram 1 year. 

No suspicious findings right breast. BI-RADS 1.

ADDENDUM:
Discussed results with patient. She will return in 6 months for diagnostic spot compression magnification views of the left upper inner quadrant calcifications. She understands that a stereotactic biopsy may be considered at that time if there is any interval change or increase in the number or suspiciousness of the calcifications. ➡ *Positive*
- MAMMOGRAM REPORT
INDICATION: Annual screening mammogram.
COMPARISON: Comparison is made to the patient's prior mammogram from 04/09/2021 at this facility.
FINDINGS:
Right breast: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. There are no dominant masses, suspicious calcifications, or areas of focal asymmetry.
Left breast: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. There are no dominant masses, suspicious calcifications, or areas of focal asymmetry.
IMPRESSION:
Bilateral mammogram, BI-RADS Category 1: Negative. 
The breasts are heterogeneously dense, which may obscure detection of small masses.
RECOMMENDATION:
Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- MAMMOGRAPHIC EXAMINATION
INDICATION: Screening.
COMPARISON: Screening mammogram from 1 year prior.
TECHNIQUE: Bilateral digital mammography including CC and MLO views.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses. 
No spiculated masses, architectural distortion, or suspicious calcifications are identified.
There are stable bilateral scattered fibroglandular densities.
Skin is unremarkable. 
IMPRESSION:
There are no mammographic findings suspicious for malignancy. No mention of microcalcifications.
BIRADS Category 2: Benign findings. Recommend continued routine screening mammography.
ADDENDUM:
Automated volumetric breast density calculation estimates the breast tissue density to be approximately 56%, which is considered heterogeneously dense (category C). ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
The breast tissue is heterogeneously dense, which may obscure small masses.
There are no suspicious masses, architectural distortions, or suspicious calcifications.
BIRADS-2. Benign finding.
Continue routine screening mammography. ➡ *Not Stated*
- MAMMOGRAPHIC EXAMINATION - BILATERAL

HISTORY: Annual screening exam. No palpable findings. No family history of breast cancer.

FINDINGS:
RIGHT BREAST: The breast is predominantly fatty replaced with minimal scattered fibroglandular densities. No focal mass, significant calcifications or architectural distortion.

LEFT BREAST: The breast parenchyma is almost completely fatty replaced. No focal mass, significant calcifications or architectural distortion.

IMPRESSION:
Bilateral mammogram, negative. No significant changes from prior study.

BI-RADS Category 1 - Normal mammogram. Recommend annual screening mammography. ➡ *Negative*
- MAMMOGRAM: BILATERAL
INDICATION: Annual screening. No current breast complaints.
COMPARISON: Prior study from 12/20/2022 at Bayside Imaging Center
FINDINGS:
RIGHT BREAST: 
The breast tissue is heterogeneously dense. No significant masses, focal asymmetries, or areas of architectural distortion. No suspicious calcifications are identified. The skin appears normal in thickness. No abnormal axillary lymph nodes are seen.
LEFT BREAST:
The breast tissue is heterogeneously dense. No significant masses, focal asymmetries, or areas of architectural distortion. No suspicious calcifications are identified. The skin appears normal in thickness. No abnormal axillary lymph nodes are seen. 
IMPRESSION: 
Bilateral mammogram, BI-RADS category 1: Negative. 
No evidence of malignancy. No suspicious calcifications.
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- REPORT:
"RIGHT MAMMOGRAM AND ULTRASOUND
HISTORY: Patient reports feeling a lump in the right breast at 2 o'clock position. Last mammogram was performed in June 2022 at City Hospital and was reported as normal. No family history of breast cancer.
FINDINGS: The right breast tissue is predominantly fatty with scattered fibroglandular densities. No focal mass, architectural distortion, or suspicious calcifications are identified. The skin and nipple are unremarkable. No significant change from the prior study.
Targeted right breast ultrasound was performed at the 2 o'clock position. No focal solid or cystic lesion is identified in this area. The palpable abnormality corresponds to normal fibroglandular tissue.
IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. Routine annual screening mammography is recommended.
2. No sonographic abnormality identified at the site of palpable concern in the right breast at 2 o'clock. The palpable finding corresponds to normal fibroglandular tissue. Clinical correlation is recommended.
BIRADS 1." ➡ *Negative*
- BILATERAL MAMMOGRAM
CLINICAL HISTORY: 45-year-old female, high risk screening.
COMPARISON: None.
FINDINGS:
Right breast: The breast is heterogeneously dense (BI-RADS density C). There are no suspicious masses, architectural distortion, or suspicious calcifications.
Left breast: The breast is heterogeneously dense (BI-RADS density C). There are scattered areas of fibroglandular density. No discrete mass or area of architectural distortion is identified. However, there is a new group of fine pleomorphic calcifications in the upper outer quadrant, 10 cm from the nipple, spanning an area of approximately 8 mm. Additional magnification views confirm these are suspicious calcifications.
IMPRESSION:
Right breast: No mammographic evidence of malignancy. BI-RADS 1 - Negative.
Left breast: New group of suspicious calcifications in the upper outer quadrant. Biopsy is recommended. BI-RADS 4 - Suspicious.
RECOMMENDATION: Stereotactic core biopsy of the left breast calcifications is recommended. Please send a requisition to the department to arrange the biopsy. ➡ *Positive*
- TMIST FFDM ARM SCREENING MAMMOGRAM.
The patient is a participant in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomly assigned to the full field digital mammography only arm after providing informed consent as per the Research Ethics Board approval.
Comparison to previous mammograms: Yes.
Breast density: Heterogeneously dense (50-75% fibroglandular tissue).
No suspicious mammographic findings. Specifically, no masses, architectural distortion, or significant asymmetries are identified. 
BIRADS Category 1: Negative.
Routine OBSP screening recommended in 2 years. ➡ *Not Stated*
- ADDENDUM TO BREAST MRI
Clinical History: Patient returns today for an MRI-guided biopsy of the left breast.
Comparison: Prior left breast MRI performed on 04/25/2025
Technique: The patient was placed in the prone position. The left breast was imaged with a dedicated breast coil. Intravenous gadolinium contrast was administered
Findings:
The previously described area of enhancement in the left breast corresponding to the area of non-mass enhancement is again identified in the left breast, Using real-time imaging, the lesion was localized with an 18-gauge needle. Three cores of tissue were obtained from this region and submitted for pathologic analysis.
IMPRESSION:
Successful left breast MRI-guided biopsy performed without complication. ➡ *Not Stated*
- LEFT SCREENING MAMMOGRAM.
CLINICAL HISTORY: 48 year old female with remote history of left benign breast biopsy. Maternal aunt with breast cancer.
TECHNIQUE: Digital mammography.
COMPARISON: None.
FINDINGS:
There are scattered fibroglandular densities. No spiculated masses, architectural distortions, or suspicious calcifications.
No significant axillary lymphadenopathy.
Skin is unremarkable. No breast edema.
IMPRESSION:
Negative digital mammogram.
BIRADS 1: Annual screening mammogram is recommended. Clinical breast exam is recommended to correlate with imaging findings. ➡ *Negative*
- MAMMOGRAM LEFT BREAST WITH SPOT MAGNIFICATION
HISTORY: Left breast pain
COMPARISONS: Prior from [redacted], 10/12/2023
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Spot magnification views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, or suspicious calcifications are seen. No skin thickening or nipple retraction is seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- MAMMOGRAM AND ULTRASOUND REPORT.
HISTORY: Annual screening exams.
Comparison made to prior imaging from two years ago.
FINDINGS: The patient is status post bilateral breast reduction.
There are scattered areas of benign-appearing fibroglandular density in both breasts. No dominant masses, architectural distortion, or suspicious calcifications are seen in either breast. The breasts remain symmetric.
Bilateral breast ultrasound was performed. No suspicious solid or cystic lesions are identified in either breast. Several small, simple-appearing cysts are noted bilaterally, each measuring less than 5mm. No evidence of inflammatory changes, skin thickening, or enlarged axillary lymph nodes.
IMPRESSION: Benign mammographic and sonographic bilateral breast exam. No mammographic or sonographic evidence of malignancy. Annual routine screening is recommended.
BIRADS 1. ➡ *Negative*
- DIAGNOSTIC MAMMOGRAPHY, BILATERAL WITH TOMOSYNTHESIS

INDICATION: Annual screening. No current symptoms. History of right breast cancer status post lumpectomy in 2000.

COMPARISON: Mammogram from 05/10/2021.

TECHNIQUE: Standard CC and MLO views were obtained bilaterally using tomosynthesis.

FINDINGS:
The breasts are predominantly fatty with scattered areas of fibroglandular density. 

RIGHT BREAST:
There are postsurgical changes in the upper outer right breast consistent with prior lumpectomy. No concerning masses, architectural distortion, or suspicious calcifications.

LEFT BREAST: 
Stable scattered vascular calcifications are present. In the upper inner left breast, there is a new 5 mm cluster of heterogeneous microcalcifications at 2:00, 6 cm from the nipple. No suspicious masses or areas of architectural distortion. 

IMPRESSION: 
RIGHT BREAST: Negative. BI-RADS 2 - Benign.
LEFT BREAST:  BI-RADS 4 - Suspicious abnormality. 5 mm cluster of microcalcifications in the upper inner left breast, new since prior exam. Biopsy should be considered in the absence of a benign explanation.

RECOMMENDATION: Stereotactic biopsy of the left breast microcalcifications is recommended. Please send a requisition to schedule the biopsy. ➡ *Positive*
- MAMMOGRAM FINDINGS:
Bilateral full field digital mammogram views were obtained, compression was adequate and the study is of good diagnostic quality.
 
The breast tissue is of scattered fibroglandular density.

There are no mammographic findings suspicious for malignancy in either breast. Specifically, no masses, no architectural distortion, no asymmetries, and no suspicious calcifications are identified.

IMPRESSION:
BIRADS 2 - Benign finding. Routine screening is recommended in 1 year. ➡ *Not Stated*
- MAMMOGRAPHIC EXAMINATION OF THE BREASTS
HISTORY: 36 year old female for screening mammogram. No significant family history of breast cancer. No breast complaints.
TECHNIQUE: Standard screening mammogram views were obtained.
COMPARISON: Comparison is made with prior studies from June 2022.
FINDINGS:
Both breasts are predominantly fatty replaced.
There is a small amount of scattered fibroglandular density.
There are no suspicious masses, architectural distortion, or malignant type calcifications noted.
Skin is unremarkable.
No lymphadenopathy in the visualized axillae.
Stable benign calcifications are seen bilaterally.
No significant change from prior study.
IMPRESSION:
Benign/negative bilateral screening mammogram.
No mammographic evidence of malignancy.
Annual screening mammography is recommended.
BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening mammogram
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available
FINDINGS:
The breasts are almost entirely fatty.
There are no dominant masses, areas of architectural distortion or asymmetries in either breast.
In the upper outer quadrant of the left breast, there is a new cluster of fine pleomorphic microcalcifications, measuring approximately 8 mm. This cluster was not present on any prior mammograms. No other groups of suspicious microcalcifications are seen in either breast.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic core biopsy is recommended for the cluster of pleomorphic microcalcifications in the upper outer left breast.
BI-RADS Category 1: Normal right breast.
ADDENDUM:
The patient proceeded to stereotactic core biopsy of the suspicious calcifications in the left breast. Pathology results showed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- MAMMOGRAPHY - BILATERAL. HISTORY: Screening mammogram. COMPARISON: Comparison is made to previous mammogram from 09/19/2023. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.

INDICATION: Annual screening mammogram. 

COMPARISON: 10/15/2021

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
Both breasts demonstrate heterogeneously dense parenchyma. The amount of dense tissue may lower the sensitivity of mammography.

There are no suspicious masses, architectural distortions, or suspicious calcifications in either breast. 

Stable benign appearing calcifications are noted in the right breast.

No significant interval change compared to prior mammogram.

IMPRESSION: 
BI-RADS Category 1: Negative bilaterally. Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of each breast were obtained.

FINDINGS:

RIGHT BREAST:
The right breast is composed of scattered fibroglandular densities. Skin and nipple are unremarkable. No masses, calcifications, architectural distortion, or other suspicious findings are identified.

LEFT BREAST:
The left breast is also composed of scattered fibroglandular densities. In the upper outer quadrant, there is a small cluster of pleomorphic microcalcifications spanning an area of approximately 8 mm. No associated mass, architectural distortion, or skin thickening is identified. The remainder of the left breast is unremarkable.

IMPRESSION:
1. Small cluster of pleomorphic microcalcifications in the left upper outer quadrant, suspicious for DCIS. Recommend stereotactic biopsy for definitive diagnosis. BI-RADS 4B.

2. Scattered fibroglandular densities bilaterally. No suspicious findings in the right breast. BI-RADS 2. Recommend continued annual screening mammography in the right breast.

ADDENDUM:
The patient returned for stereotactic biopsy of the microcalcifications in the left breast. Pathology revealed ductal carcinoma in situ (DCIS), intermediate grade, cribriform and solid types. The patient will be referred to surgical oncology for further management. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM. History: Patient is referred for evaluation of a palpable nodule in her left breast at the 10 o'clock position, palpable for the past 2 months. The patient reports she was doing a self breast exam when she felt the nodule. She denies any nipple discharge, skin changes, or axillary lymphadenopathy. Past medical history is significant for hypertension and hyperlipidemia. No history of breast biopsies or breast cancer. No family history of breast cancer. Medications include lisinopril and atorvastatin. Physical exam: Examination of the left breast reveals a palpable, firm, mobile, non-tender nodule measuring approximately 1 cm in diameter located in the left breast at the 10 o'clock position, approximately 2 cm from the nipple. No skin changes or nipple discharge. No axillary lymphadenopathy. Technique: Bilateral CC and MLO views were obtained, along with spot compression and magnification views of the left breast. Findings: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications in both breasts. In the left breast at the 10 o'clock position, there is a 1.0 cm, well-circumscribed, oval-shaped density corresponding to the palpable finding. No architectural distortion or associated calcifications are seen. Impression: 1. Left breast, 10 o'clock position, 2 cm from the nipple: 1.0 cm well-circumscribed, oval-shaped density likely representing a benign fibroadenoma. Ultrasound correlation is recommended for further evaluation. BI-RADS Category 3: Probably benign finding - short-interval follow-up suggested. 2. No other suspicious findings. BI-RADS Category 1. ➡ *Negative*
- SCREENING MAMMOGRAM.
INDICATION: Screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available.
FINDINGS:
Both breasts are heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications are identified.
IMPRESSION:
Negative screening mammogram. No evidence of malignancy.
BIRADS Category 1 - Negative. Routine screening is recommended in 1 year.
ADDENDUM: Breast density is estimated as heterogeneously dense, which may obscure small masses. The breasts are not extremely dense.  ➡ *Not Stated*
- FINDINGS:
Digital bilateral mammograms demonstrate heterogeneously dense breasts. There are multiple areas of architectural distortion and asymmetry in the upper outer left breast. A cluster of amorphous and pleomorphic microcalcifications is seen at 2:00, 5 cm from the nipple, spanning approximately 1 cm.

Targeted ultrasound of the left breast at 2:00 shows an irregular hypoechoic mass with indistinct margins and posterior acoustic shadowing, corresponding to the area of microcalcifications on mammography. The mass measures 15 x 12 x 10 mm. No suspicious axillary lymphadenopathy is identified.

The right breast shows no suspicious masses, calcifications, or architectural distortion. 

IMPRESSION:
1. Suspicious cluster of pleomorphic microcalcifications in the left breast at 2:00, with associated irregular hypoechoic mass on targeted ultrasound, highly suspicious for malignancy. Biopsy is recommended. BI-RADS 5.

2. No suspicious findings in the right breast. BI-RADS 1. ➡ *Positive*
- FINDINGS:
Both breasts are heterogeneously dense.  There are scattered areas of fibroglandular density.  No dominant mass, suspicious calcifications or architectural distortion.

Magnification views in the craniocaudal and mediolateral projections demonstrate a cluster of pleomorphic microcalcifications in the left upper outer quadrant anteriorly (11:00 position).  This cluster measures approximately 6 mm in greatest dimension.  The calcifications are variable in size and density.

There are no significant changes compared with the prior exams.

IMPRESSION:
Suspicious cluster of pleomorphic microcalcifications in the left upper outer quadrant as described above.  This is considered indeterminate but worrisome for possible ductal carcinoma in situ (DCIS).  Recommend stereotactic core biopsy.  

ASSESSMENT:
BI-RADS 4 for the suspicious microcalcifications
BI-RADS 2 for the remainder of both breasts

ADDENDUM:
The patient returned for stereotactic core biopsy of the microcalcifications in the left upper outer quadrant using an 8 gauge core biopsy device.  Specimen radiograph confirmed that the calcifications were sampled.

ADDENDUM 2:
Pathology results from the stereotactic core biopsy of the left breast microcalcifications revealed ductal carcinoma in situ (DCIS), nuclear grade 2.  The patient will be referred to breast surgery for further management. ➡ *Positive*
- BILATERAL MAMMOGRAM. CLINICAL HISTORY: 47 year old female for routine screening mammogram. Findings: There are scattered areas of benign-appearing calcifications in both breasts. There are no suspicious masses, architectural distortion, or suspicious calcifications. Impressions: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM

CLINICAL HISTORY: Patient is s/p chemo and radiation for right breast cancer. Presents today for routine follow up.

COMPARISON: Prior mammogram 02/12/2024.

FINDINGS: The breasts are heterogeneously dense, which could obscure a small mass. There are scattered benign appearing calcifications in both breasts, unchanged. There is a new 5 mm cluster of fine pleomorphic calcifications in the left breast at the 10 o'clock position, 3 cm from the nipple.

IMPRESSION:

1. New cluster of fine pleomorphic calcifications in the left breast. Recommend spot magnification views for further evaluation. 

BI-RADS: 0 (Needs additional imaging). ➡ *Positive*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
The patient is participating in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomized to the full field digital mammography arm after providing informed consent.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
The breast tissue is heterogeneously dense. There are no dominant masses, architectural distortions, or suspicious calcifications. The skin and nipples are unremarkable.
BIRADS-2. Routine screening recommended.

ADDENDUM: Comparison was made to outside screening mammograms from July 15, 2020, which showed stable breast tissue with no suspicious findings.  ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAMS
INDICATION: Family history of breast cancer, maternal aunt in her 60s.
TECHNIQUE: Hologic Digital Breast Tomosynthesis, CC and MLO views.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
No dominant masses, architectural distortion, or suspicious asymmetries.
There are scattered benign-appearing punctate and short segmental calcifications in both breasts.
In the medial right breast, there is a small cluster of indeterminate amorphous calcifications at 3:00, 4 cm from the nipple.
IMPRESSION:
BIRADS 0 - Incomplete. Additional imaging evaluation is needed.
Small cluster of indeterminate amorphous calcifications medial right breast. Recommend diagnostic mammographic views with magnification for further characterization.
The patient is advised to have these additional images today if possible, or otherwise scheduled as soon as possible. A final BIRADS assessment will be made after additional imaging. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram

COMPARISON: None available

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:

Right Breast:
The right breast is heterogeneously dense. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Left Breast:
The left breast is also heterogeneously dense. In the upper outer quadrant of the left breast, there is a new group of fine pleomorphic calcifications spanning an area of approximately 1 cm at the 2:00 position, 6 cm from the nipple. No associated mass or architectural distortion is seen. The remainder of the left breast is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete Assessment - Need additional imaging evaluation of left breast.

RECOMMENDATION:
Magnification views of the left breast are recommended for further evaluation of the new group of pleomorphic calcifications in the upper outer quadrant. The patient should return for additional imaging as soon as possible.

ADDENDUM:
Magnification views of the left breast demonstrate a cluster of fine pleomorphic calcifications in the upper outer quadrant, spanning approximately 8 mm. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS). Biopsy is recommended. 

FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious abnormality - Biopsy should be considered. ➡ *Positive*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior exam dated 1/15/2020.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Stable bilateral scattered fibroglandular densities are noted.
IMPRESSION:
NEGATIVE MAMMOGRAM. BIRADS 2 - Benign findings. Routine screening mammography is recommended in 1 year.
ADDENDUM: No microcalcifications are identified in either breast. ➡ *Not Stated*
- ULTRASOUND RIGHT BREAST AND MAMMOGRAPHY RIGHT BREAST
CLINICAL HISTORY: 65 year old female with history of left mastectomy presents for screening of the right breast.
FINDINGS: The breast is heterogeneously dense (category C).
Multiple magnification spot compression views of the right breast demonstrate a cluster of microcalcifications at the 9 o'clock position, 7 cm from the nipple. The calcifications are relatively coarse but pleomorphic in shape and grouped in distribution.
Right breast ultrasound was also performed. Targeted ultrasound over the area of mammographic calcifications at 9:00 demonstrates a vague area of hypoechogenicity measuring approximately 10 mm. No discrete mass, architectural distortion or posterior acoustic shadowing is seen.
The remainder of the right breast was scanned and there are no other sonographic abnormalities. No axillary lymphadenopathy.
IMPRESSION:
New cluster of microcalcifications at 9:00, 7 cm from the nipple in the right breast. Targeted ultrasound is suspicious. Stereotactic core biopsy is recommended for definitive diagnosis. BI-RADS 4B.
ADDENDUM: An ultrasound-guided core biopsy of the suspicious calcifications at 9:00 in the right breast was performed using a 14G spring-loaded biopsy device. Specimen radiograph confirms calcifications within the cores. Pathology is pending. ➡ *Positive*
- BREAST MRI
Clinical History: Right breast palpable abnormality.
Comparison: Outside mammogram
Technique: Magnetic resonance imaging of both breasts was performed before and after the intravenous administration of gadolinium contrast
Findings:
There is heterogeneous enhancement of the breast parenchyma bilaterally, in keeping with the patient's physiologic status.
There is no abnormal enhancement to suggest malignancy within either breast. No suspicious masses or non-mass enhancement is identified to explain the palpable abnormality. There is no axillary, internal mammary or supraclavicular lymphadenopathy
IMPRESSION:
Negative MRI of the breasts ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:

Right Breast:
The right breast is heterogeneously dense (BI-RADS density category C), which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left Breast:
The left breast is also heterogeneously dense (BI-RADS density category C), which may obscure detection of small masses.
A new cluster of fine pleomorphic microcalcifications is seen in the upper outer quadrant of the left breast, approximately 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension.
No associated mass, architectural distortion, or skin thickening is identified.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation
A cluster of suspicious microcalcifications is identified in the left breast, not seen on prior exams. Additional magnification views are needed for further characterization. Stereotactic biopsy may be necessary depending on the morphology and distribution of the calcifications on magnification views.
The patient will be recalled for diagnostic mammographic workup of the left breast.

BIRADS ASSESSMENT CATEGORIES:
Right breast: BI-RADS 2 - Benign
Left breast: BI-RADS 0 - Incomplete

ADDENDUM:
Magnification views of the left breast demonstrate a 10 mm cluster of fine pleomorphic microcalcifications in the upper outer quadrant, with segmental distribution. Stereotactic biopsy is recommended for definitive characterization. Updated BIRADS assessment of the left breast: BI-RADS 4 - Suspicious abnormality. ➡ *Positive*
- MAMMOGRAM
INDICATION: Screening.
TECHNIQUE: Bilateral digital screening mammography was performed. Comparison is made with prior exams.
FINDINGS:
The breast tissue is heterogeneously dense which could obscure small masses.
There are no dominant masses, architectural distortion or suspicious calcifications.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening in 12 months with comparison to prior exams.
BIRADS 1 - NEGATIVE. ➡ *Negative*
- BREAST IMAGING REPORT

CLINICAL INFORMATION: 65-year-old female with family history of breast cancer presents with right breast lump.

COMPARISON: Prior mammogram and ultrasound from 08/15/2022 at ABC Imaging Center.

FINDINGS:
Right CC and MLO views demonstrate heterogeneously dense breast tissue. There is a 1.2 cm irregular mass at the 10 o'clock position of the right breast, corresponding to the palpable abnormality. No suspicious calcifications are identified. The left breast is unremarkable.

Targeted right breast ultrasound demonstrates an irregular, hypoechoic mass with angular margins and posterior shadowing at the 10 o'clock position, 4 cm from the nipple, measuring 1.2 x 1.0 x 0.9 cm. This corresponds to the mammographic finding and the palpable mass. No abnormal axillary lymph nodes are seen.

IMPRESSION:
Highly suspicious irregular mass right breast 10 o'clock position, BI-RADS 5. Ultrasound-guided core needle biopsy is recommended. No suspicious microcalcifications are present in either breast.

ADDENDUM:
Ultrasound-guided core needle biopsy of the right 10 o'clock mass was performed. Pathology results: Invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. The patient will be referred to oncology for further management. ➡ *Negative*
- SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:

Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.

No dominant masses, architectural distortion or suspicious calcifications are identified in either breast.

Scattered areas of fibroglandular density are present bilaterally.

IMPRESSION:

BIRADS Category 1: Negative. Routine screening mammogram is recommended in 1 year. ➡ *Not Stated*
- NORMAL TMIST DBT ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the digital breast tomosynthesis arm.  
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breasts are composed of scattered areas of fibroglandular density. No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2. 
Routine screening in 1 year is recommended. ➡ *Not Stated*
- REPORT:
"LEFT MAMMOGRAM AND ULTRASOUND.
HISTORY: Intermittent left breast pain, nipple inversion and milky discharge. Patient also reports a palpable lump at 4 o'clock position. Last mammogram 1 year prior was normal. No family history of breast cancer.  
FINDINGS: The breast tissue is heterogeneously dense. No significant masses, architectural distortion or suspicious calcifications are identified. The palpable area of concern in the 4 o'clock position demonstrates normal appearing fibroglandular tissue. No sonographic correlate for the palpable abnormality. 
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year. 
2. Normal left breast ultrasound. No suspicious lesions to correlate with the area of palpable concern. 
3. Recommend clinical follow up for nipple changes and discharge as needed.
BIRADS 1 - Negative." ➡ *Negative*
- MAMMOGRAM - BILATERAL WITH CAD

CLINICAL HISTORY: 45-year-old female for screening mammogram.

COMPARISON: 01/02/2024.

FINDINGS: The breasts are heterogeneously dense. Comparison is made to prior examination. Again seen are scattered benign-appearing calcifications in both breasts. There is a new focus of highly suspicious, grouped, fine, pleomorphic calcifications in the left breast at approximately the 10 o'clock position. This measures approximately 1.2 cm in greatest diameter and is located approximately 5 cm from the nipple.

IMPRESSION:

1. New focus of highly suspicious, grouped, fine, pleomorphic calcifications in the left breast. Given the patient's age, this is highly suspicious for malignancy. Correlation with spot magnification views and possible biopsy is recommended.

BI-RADS: 4C (Highly suspicious for malignancy). ➡ *Positive*
- REPORT:
"RIGHT DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS AND RIGHT BREAST ULTRASOUND

History: Focal right breast pain. 
Compared to previous outside mammograms from 2019.

FINDINGS: The breasts are heterogeneously dense. 
There is a round 7 mm noncalcified circumscribed mass at 6:00 3 cm from the nipple in the right breast which appears stable compared to the previous mammogram and corresponds to a simple cyst sonographically. 
There is a cluster of finely pleomorphic calcifications across a 6 mm span at 2:00 6 cm from the nipple in the right breast which appear slightly larger in number than on the previous mammogram. Magnification views confirm these calcifications.
Sonographically this corresponds to an irregular 6 x 4 x 6 mm hypoechoic mass with mild peripheral vascularity.
No other suspicious mammographic or sonographic findings in the right breast. 
The left breast was not imaged.

IMPRESSION: 
New cluster of pleomorphic calcifications at 2:00 right breast with a correlating sonographic finding, suspicious for malignancy. BIRADS 4C. Recommend stereotactic core biopsy. 

ADDENDUM: 
Stereotactic core biopsy of the right breast calcifications was performed. Pathology showed ductal carcinoma in situ, intermediate nuclear grade with comedonecrosis."

LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- REPORT:
"SCREENING BILATERAL MAMMOGRAM
HISTORY: 46-year-old female presents for routine screening mammography. No prior imaging for comparison.

FINDINGS: 
RIGHT BREAST: The breast is composed of heterogeneously dense fibroglandular tissue. There are no masses, architectural distortion, or suspicious calcifications. 

LEFT BREAST: The breast is composed of scattered fibroglandular densities. There are no masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Heterogeneously dense breast tissue in the right breast, which could obscure detection of small masses. No dominant mass, suspicious calcifications, or architectural distortion.
2. Scattered fibroglandular densities in the left breast. No dominant mass, suspicious calcifications, or architectural distortion.
3. Recommend routine screening in 1 year.

ASSESSMENT: 
RIGHT BREAST: BI-RADS 2 - Benign findings 
LEFT BREAST: BI-RADS 2 - Benign findings" ➡ *Negative*
- TMIST MAMMOGRAPHY - SCREENING STUDY.
The patient is enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), a Research Ethics Board approved study. She provided informed consent and was randomly allocated to undergo standard digital mammography only.  
Comparison to prior exams: Yes.
Breast tissue density: Heterogeneously dense. The breasts are not extremely dense.
Routine annual screening is recommended through the Ontario Breast Screening Program (OBSP).
There are no mammographic abnormalities suspicious for malignancy. Microcalcifications are not mentioned.
BIRADS Category 1 or 2, depending on local protocol for a negative screening mammogram. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: October 2, 2019

FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

RIGHT BREAST:
There are no masses, architectural distortion, or suspicious calcifications.

LEFT BREAST:
There are scattered areas of fibroglandular density. No suspicious masses or architectural distortion.
There is a new group of amorphous calcifications in the upper outer quadrant posteriorly. These are new since the prior exam and are suspicious for early malignancy.

IMPRESSION:
Left breast: BI-RADS 4B: Suspicious calcifications in the left upper outer breast, biopsy should be considered.
Right breast: BI-RADS 2: Benign.

RECOMMENDATION:
Stereotactic biopsy of the left breast calcifications is recommended. The patient should follow up with the ordering physician to discuss the results and management options. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM - FOR SCREENING. CLINICAL HISTORY: Patient is a 50-year-old female presenting for her screening mammogram. Patient is on hormone replacement therapy. No history of breast cancer, no family history, no breast symptoms. TECHNIQUE: Standard CC and MLO views were obtained. COMPARISON: Comparison is made to the patient's prior mammogram dated 12/19/2023. FINDINGS: The breasts are heterogeneously dense. There is no mammographic evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Bilateral digital screening mammogram demonstrates the breasts are heterogeneously dense. In the right breast, there are no suspicious masses, calcifications or architectural distortion.

In the left breast, at the 2:00 position 6 cm from the nipple, there is a new cluster of fine pleomorphic calcifications spanning approximately 8 mm. At the 10:00 position 4 cm from the nipple, there is a new cluster of fine pleomorphic calcifications spanning 6 mm. The remaining left breast is unremarkable.

Bilateral axillae are unremarkable. No skin thickening or nipple retraction.

IMPRESSION:
BI-RADS 4: Suspicious for malignancy in the left breast. Stereotactic core biopsy is recommended of the new clusters of pleomorphic calcifications at 2:00 and 10:00 in the left breast. 

BI-RADS 1: Normal right breast.

ADDENDUM:
Stereotactic core biopsy of the left breast calcifications at 2:00 and 10:00 was performed. Pathology results demonstrate ductal carcinoma in situ, intermediate grade, in both biopsies. Surgical consultation is recommended for further management.

Post-biopsy mammogram demonstrates biopsy markers in good position at the sites of the biopsied calcifications. No pneumothorax. ➡ *Positive*
- BREAST MRI
Clinical History: Left breast palpable abnormality
Comparison: Outside mammogram.
Technique: Magnetic resonance imaging of both breasts was performed before and after the intravenous administration of gadolinium contrast
Findings:
There is heterogeneous enhancement of the breast parenchyma bilaterally, in keeping with the patient's physiologic status
There is no abnormal enhancement to suggest malignancy within either breast. No suspicious masses or non-mass enhancement is identified to explain the palpable abnormality. There is no axillary, internal mammary or supraclavicular lymphadenopathy
IMPRESSION:
Negative MRI of the breasts ➡ *Negative*
- RADIOLOGY REPORT
HISTORY: Left breast lump for 2 weeks.
CLINICAL INFORMATION: No previous mammograms. Breast cancer in maternal grandmother. Patient complains of a tender, moving mass in the left breast upper outer quadrant, noticed in the shower.
EXAMINATION: Digital 2D left diagnostic mammogram views and left ultrasound performed.
FINDINGS:
2D Left Mammogram:
Left breast composed of scattered fibroglandular densities. The breast is heterogeneously dense, which may obscure detection of small masses. No dominant mass, architectural distortion or suspicious microcalcifications.
Left Ultrasound:
Targeted ultrasound to the left upper outer breast reveals no discrete mass. Normal fibroglandular tissue and fat lobules present. No abnormal vascularity or fluid collection. The area of palpable concern corresponds to normal glandular tissue.
IMPRESSION:
1. Left breast heterogeneously dense. No suspicious mammographic findings.
2. Targeted left ultrasound is negative. Clinically palpated area corresponds to normal glandular tissue.
BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- SCREENING MAMMOGRAM AND SPOT COMPRESSION VIEWS LEFT BREAST
HISTORY: Annual screening mammogram
TECHNIQUE:  Bilateral screening mammogram performed with additional spot compression views of left breast.
FINDINGS: The overall density of the breasts is heterogeneously dense which can obscure small masses.
There are no suspicious findings in the right breast.
In the left upper outer breast at 2:00 11 cm from the nipple there are clustered amorphous calcifications. On spot compression mag views this measures 4 mm. There is associated ill-defined density. An ultrasound was performed and did not show an underlying mass. A marker clip was placed at this site for localization.
IMPRESSION:
BI-RADS 4: Suspicious grouped amorphous microcalcifications in the left upper breast at 2:00 with no ultrasound correlate. Stereotactic biopsy advised. Marker placed at site of concern.
BI-RADS 2. Benign. Right breast.
ADDENDUM: Left stereotactic core biopsy showed ductal carcinoma in situ, intermediate grade, with comedonecrosis. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammography was performed. The breast tissue is heterogeneously dense which may obscure small masses. 
There are scattered areas of fibroglandular density. No dominant mass, suspicious calcifications or architectural distortion.
Additional magnification views were performed of a group of calcifications in the upper outer left breast at the 2:00 position 8 cm from the nipple. The calcifications are punctate and amorphous and extend over an area measuring approximately 10 mm. They are new compared to the prior study and are suspicious for malignancy. An ultrasound was recommended for further evaluation.
Targeted left breast ultrasound demonstrates:
At 2:00 8 cm from the nipple, corresponding to the mammographic finding, there is an 11 x 5 x 6 mm irregular hypoechoic mass with indistinct margins and posterior shadowing. Increased vascularity is present.  
IMPRESSION:
New group of suspicious calcifications upper outer left breast at 2:00, 8 cm from the nipple with sonographic correlate demonstrating an irregular hypoechoic mass with posterior shadowing, highly suggestive of malignancy. Ultrasound guided biopsy is recommended. BIRADS 5.
Remainder of both breasts are BIRADS 2, benign. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense. In the left upper outer breast, there is a 5 mm cluster of pleomorphic microcalcifications at 2:30, 7 cm from the nipple. Additional magnification views confirm these calcifications have varying size and density and appear new compared to prior exams. Given their new appearance and pleomorphic morphology, these are suspicious for malignancy. No other suspicious calcifications or masses are seen in either breast. Specifically, no mass, asymmetry or architectural distortion is seen in the region of palpable concern at 8:00, 4 cm from the nipple of the left breast. The right breast demonstrates no suspicious masses, calcifications or areas of architectural distortion. The skin is normal bilaterally with no skin thickening or retraction. No suspicious axillary lymphadenopathy.

IMPRESSION:
New cluster of pleomorphic microcalcifications in the left upper outer breast at 2:30, 7 cm from the nipple, suspicious for malignancy. Stereotactic biopsy is recommended. BI-RADS 4C.

The remainder of the left breast is BI-RADS 2, benign finding. The palpable area of concern at 8:00, 4 cm from the nipple demonstrates no sonographic correlate and is likely benign, but clinical followup is recommended.

The right breast is BI-RADS 1, negative. ➡ *Positive*
- MAMMOGRAPHIC REPORT
INDICATION: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Digital screening mammography including craniocaudal and mediolateral oblique views of both breasts was performed.
COMPARISON: 08/24/2020
FINDINGS:
Both breasts show scattered fibroglandular tissue which is heterogeneously dense. This may obscure detection of small masses.
There are no suspicious masses, architectural distortion, or suspicious calcifications.
There are no significant changes compared to prior mammogram.
IMPRESSION:
BI-RADS 1: Negative bilateral screening mammogram. Recommend continued annual screening in 12 months.
ADDENDUM:
3D tomosynthesis was subsequently performed for further evaluation, confirming no suspicious findings. No microcalcifications were identified on the tomosynthesis images. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening.
COMPARISON: None.
FINDINGS:
Both breasts demonstrate heterogeneously dense fibroglandular tissue.
At the 12:00 position of the right breast, 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of 1 cm. No associated mass, architectural distortion or skin changes are identified.
The left breast is unremarkable, negative for masses, architectural distortion, asymmetries or suspicious calcifications.
No axillary, supraclavicular or internal mammary lymphadenopathy.
Benign vascular calcifications are present bilaterally.
IMPRESSION:
BI-RADS 4: Suspicious grouped amorphous microcalcifications in the right breast at 12:00, 6 cm from the nipple. Stereotactic core needle biopsy is recommended for tissue diagnosis. The left breast is BI-RADS 1, negative.
ADDENDUM: 
The patient returned for stereotactic core biopsy of the right breast microcalcifications. Specimen radiograph confirmed retrieval of microcalcifications. Final pathology revealed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. Wide local excision is recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening mammogram.

FINDING: The breasts are predominantly fatty with fibroglandular density. 
There is no interval change in the scattered vague asymmetries bilaterally which are likely summation artifacts.

In the left upper outer quadrant, there is a new cluster of suspicious round pleomorphic microcalcifications spanning approximately 0.6 cm without apparent associated mass, architectural distortion or skin thickening. These are located at approximately 10-11:00 in posterior third, 7 cm from nipple.

No other suspicious mammographic findings bilaterally. No skin thickening, architectural distortion, or lymphadenopathy.

IMPRESSION: 

Suspicious cluster of microcalcifications in the left upper outer quadrant. BI-RADS 4B. Stereotactic biopsy recommended. 

The remainder of the mammogram is benign bilaterally. BI-RADS 2.

ADDENDUM:
The patient proceeded to stereotactic core biopsy of the suspicious microcalcifications in the left upper outer quadrant under ultrasound guidance using an 11G vacuum assisted biopsy device.

Post-biopsy mammogram confirmed adequate sampling of the microcalcifications with clip placement.

The biopsy results showed pleomorphic DCIS with comedo necrosis (DCIS Grade 3).

Final Assessment: BI-RADS 6: Known biopsy-proven malignancy (DCIS). ➡ *Positive*
- BILATERAL MAMMOGRAMS AND ULTRASOUND LEFT BREAST.   
INDICATION: Left bloody nipple discharge. 
FINDINGS: Scattered fibroglandular densities.
In the right breast there is a spiculated oval mass in the superomedial quadrant 6 cm from the nipple measuring 12 x 10 mm. Focally associated pleomorphic microcalcifications are noted. The mass appears to abut the pectoral muscle. Skin thickening is noted overlying the lesion. On ultrasound the mass is solid but with some cystic spaces, measures 13 x 11 x 14 mm and demonstrates increased peripheral vascularity.
There is an additional 8 mm well circumscribed oval mass at the 11:00 position right breast seen on ultrasound only.
In the left breast there are multiple round to oval smooth masses at 2:00 5cm from the nipple, the largest 10 x 8 mm. These likely represent fibroadenomas. No suspicious masses or calcifications are seen in the left breast. No significant axillary lymphadenopathy.
IMPRESSION:
Right breast: Highly suspicious spiculated mass with associated pleomorphic microcalcifications and skin thickening at 12:00 right breast. Additional probably benign mass at 11:00. BI-RADS 5. Right breast ultrasound guided biopsies recommended. 
Left breast: Negative. Likely fibroadenomas at 2:00 left breast. BI-RADS 2. 
BIOPSY ADDENDUM: Right breast 12:00 mass, core biopsy: Invasive ductal carcinoma, grade 2. ER+, PR-, HER2 negative. Right 11:00 mass FNA: Benign cyst contents. ➡ *Positive*
- MAMMOGRAM AND ULTRASOUND OF LEFT BREAST

HISTORY: 45-year-old female with new palpable lump in the upper outer quadrant of the left breast, noticed on self-exam. No prior mammograms available for comparison. Family history of breast cancer in maternal grandmother.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There is a 1.2 cm irregular mass in the upper outer quadrant of the left breast at the 10 o'clock position, corresponding to the palpable abnormality. The mass has indistinct margins and is hypoechoic on ultrasound. No suspicious calcifications are associated with the mass.

The remainder of the left breast is unremarkable. No suspicious calcifications, architectural distortion, or additional masses are seen.

Ultrasound of the left axilla demonstrates normal appearing lymph nodes.

IMPRESSION:
1. Suspicious 1.2 cm irregular mass in the upper outer quadrant of the left breast with indistinct margins, highly suggestive of malignancy. Biopsy is recommended. BIRADS 4.

2. No suspicious microcalcifications are identified in the left breast.

ADDENDUM:
Ultrasound-guided core biopsy of the left breast mass was performed. Pathology results are pending. Surgical consultation is recommended following biopsy results for further management. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED VIEWS AND ULTRASOUND LEFT BREAST
History: Palpable lump left breast.
Compared to previous exams dated November 2018.
FINDINGS:
The breasts are heterogeneously dense.
There is slight asymmetry between the breasts but otherwise no significant asymmetry or concerning areas of architectural distortion.
In the left lateral breast at 3:00 9 cm from the nipple is a 1 cm circumscribed oval hypoechoic mass with slight posterior shadowing. There is no internal vascularity. This was palpable on ultrasound. Ultrasound of the left axilla is unremarkable.
In the left upper breast at 11:00 6 cm from the nipple is a 7 mm group of clustered pleomorphic calcifications. These were not present on the November 2018 exam.
In the left breast laterally near the chest wall there are a few probably vascular calcifications.
The right breast is unremarkable.
BI-RADS 4B left for new cluster microcalcifications. Recommend stereotactic core biopsy. Probable fibroadenoma at 3:00 left breast - biopsy could be considered for a definitive diagnosis.
BI-RADS 1 right.
ADDENDUM: Patient called back for additional views. The group of microcalcifications in the left upper breast at 11:00 6 cm from the nipple is again seen and appears more prominent on these views. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views were obtained.

COMPARISON: None available.

FINDINGS:
The breasts are predominantly fatty.

There are no significant masses, architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant at approximately 10 o'clock, 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No associated mass, architectural distortion, or skin thickening is identified.

The remainder of the left breast is unremarkable. No other suspicious masses, calcifications, or areas of architectural distortion are identified.

IMPRESSION:
RIGHT BREAST: Negative. BI-RADS 1.

LEFT BREAST: New cluster of pleomorphic microcalcifications in the left upper outer quadrant, suspicious for malignancy. BI-RADS 4C. Recommend stereotactic biopsy for further evaluation.

ADDENDUM (post-biopsy):
Stereotactic biopsy of the left breast microcalcifications at 10:00, 6 cm from the nipple was performed. Pathology results demonstrate ductal carcinoma in situ (DCIS), intermediate nuclear grade. Surgical consultation is recommended for further management. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Follow up for known right breast carcinoma, status post lumpectomy and radiation therapy. 
TECHNIQUE: Bilateral digital screening mammogram. 
COMPARISON: Mammogram from 11/15/2021.
FINDINGS:  
RIGHT BREAST: 
There are postsurgical changes from prior lumpectomy in the upper outer quadrant. No suspicious mass, architectural distortion or suspicious calcifications.
LEFT BREAST:  
The breast is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications. 
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Postsurgical changes right breast, stable.
3. No evidence of malignancy.
RECOMMENDATION: Annual screening mammogram in 1 year. 
BIRADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM.
HISTORY: 54 year old female presents for annual screening mammogram. No current breast complaints.  
FINDINGS: Bilateral CC and MLO views demonstrate scattered fibroglandular densities. No suspicious masses, calcifications or areas of architectural distortion. Stable post-surgical changes present in the upper outer left breast from prior benign biopsy. No interval change compared to prior exam.
IMPRESSION: 
1. No mammographic evidence of malignancy. Recommend continued annual screening with mammography. 
2. Benign post-surgical changes left breast, stable.
BIRADS 2. ➡ *Negative*
- FINDINGS:
The breast tissue is heterogeneously dense, which could obscure detection of small masses. 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
There are no new dominant masses, suspicious calcifications, or areas of architectural distortion compared to prior examination.

IMPRESSION:
Heterogeneously dense breast tissue with no dominant masses, suspicious calcifications, or areas of architectural distortion. No significant change compared to prior exam.
BI-RADS Category 1: Negative.

ADDENDUM:
Reviewed additional clinical history indicating a palpable lump in the left breast at 7 o'clock position. Targeted ultrasound performed at the site of the palpable finding demonstrates normal fibroglandular tissue with no discrete solid mass or suspicious finding. Recommend correlation with clinical breast exam. If there is continued clinical concern, short-term follow up could be considered to document stability. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
COMPARISON: 2019-10-05 Humber River Hospital.
FINDINGS:
Both breasts are almost entirely fatty.
There are multiple round and calcified fibroadenomas in the right breast, the largest measuring 16 mm at 1:00, 6 cm from the nipple. No new dominant masses, suspicious calcifications or architectural distortion.
In the left breast, there is a new 8 mm cluster of pleomorphic calcifications at 3:00, 5 cm from the nipple. Magnification views confirm these are suspicious. No associated mass is seen on the mammogram or targeted ultrasound.
Also present in the left breast are a few intact intramammary lymph nodes and multiple simple cysts. No other dominant masses, suspicious calcifications or architectural distortion.
IMPRESSION:
BI-RADS 1 - Right breast.
BI-RADS 4B - Left breast. Suspicious cluster of pleomorphic calcifications 3:00, 5 cm from the nipple. Recommend stereotactic core biopsy.
ADDENDUM:
LEFT BREAST, 3:00, 5 cm from the nipple - Stereotactic guided core biopsy performed using 9G vacuum assisted needle. Specimen radiograph confirms calcifications. Marker clip placed. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH TOMOSYNTHESIS

INDICATION: Annual screening mammogram.

COMPARISON: 10/1/2020

FINDINGS:
Both breasts are heterogeneously dense (BI-RADS C).

RIGHT BREAST:
There are no suspicious masses, architectural distortions, or calcifications.

LEFT BREAST:
There are scattered fibroglandular densities. In the upper outer quadrant there is a new group of fine pleomorphic microcalcifications spanning approximately 10 mm. No associated mass, architectural distortion, or skin thickening.

IMPRESSION:
BI-RADS 4: Suspicious abnormality in the left breast, biopsy recommended. New group of pleomorphic microcalcifications in the upper outer quadrant of the left breast, spanning 10 mm.

BI-RADS 1: Normal right breast.

MANAGEMENT:
Stereotactic core needle biopsy is recommended for the new group of microcalcifications in the left breast. The patient should return to clinic to discuss biopsy results. ➡ *Positive*
- BILATERAL MAMMOGRAM

INDICATION: Screening mammogram

COMPARISON: None.

FINDINGS:

Both breasts show scattered areas of fibroglandular density. Breast parenchymal density is heterogeneously dense, which may obscure detection of small masses.

Right breast: Negative.

Left breast: 
There is a 6 mm cluster of amorphous microcalcifications in the upper outer quadrant, 8 cm from the nipple at the 1:00 position. No associated mass, distortion or asymmetry.

There are no dominant masses, areas of architectural distortion or other suspicious calcifications in either breast. 

Skin is normal bilaterally with no evidence of thickening or retraction. 

IMPRESSION:

BI-RADS Category 0: Incomplete. Need additional imaging evaluation. 
Stereotactic biopsy is recommended for the cluster of amorphous microcalcifications in the left breast upper outer quadrant as described above.

BI-RADS Category 1 for the right breast.

ADDENDUM: Patient was informed of the results and the need for additional evaluation of the left breast. She is agreeable to stereotactic biopsy. This will be scheduled for the earliest available appointment. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening.
Comparison is available dating from 04/2022.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- LEFT BREAST ULTRASOUND
Clinical History: Left breast palpable abnormality.
Comparison: None
Technique: The patient was positioned for optimal exposure of the left breast. Ultrasound images were obtained
FINDINGS:
Within the palpable area, there is a 0.7 cm circumscribed, oval, wider-than-tall, hypoechoic mass. This is likely a fibroadenoma. No suspicious axillary lymph nodes are identified
IMPRESSION:
Left breast palpable abnormality - Ultrasound demonstrates a likely benign appearing mass, see above. This correlates with the palpable abnormality. BI-RADS Category 2 ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM. Clinical History: 58 year old female presenting for a screening mammogram. Technique: Bilateral CC and MLO images were obtained. Comparison is made to prior mammographic examination from 09/21/2023. Findings: The breasts are heterogeneously dense. There are no masses, architectural distortion, or suspicious calcifications. Impression: Negative mammogram. BI-RADS 1. ➡ *Negative*
- BILATERAL MAMMOGRAM:
COMPARISON: None.
INDICATION: Annual screening. No concerns.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
Both breasts are heterogeneously dense which may lower the sensitivity of mammography.
There are scattered areas of fibroglandular density bilaterally.
No dominant masses, architectural distortion, or suspicious calcifications are identified in either breast.
At the 9 o'clock position of the right breast, approximately 6 cm from the nipple, there is a small group of amorphous calcifications spanning an area of about 6 mm. These are new since any prior mammograms. Additional magnification views in the CC and lateral projections confirm these are distinct calcifications and not vascular calcifications or artifact.
Skin is unremarkable. No evidence of significant axillary, intramammary, or internal mammary adenopathy. 
IMPRESSION:
BI-RADS Category 0: Incomplete. Small group of amorphous calcifications in the right breast as described above. Additional magnification views confirm these to be true calcifications. Recommend biopsy for definitive characterization as the presence of a grouped distribution of amorphous calcifications raises the possibility of early ductal carcinoma in situ (DCIS). 
Remainder of bilateral mammogram is negative.
ADDENDUM: Stereotactic guided vacuum assisted biopsy of the group of right breast calcifications was performed. Specimen radiograph confirmed calcifications were sampled. Final pathology showed benign changes with microcalcifications and no evidence of malignancy. 
With biopsy proven benign calcifications, the final BI-RADS category is 2: Benign findings. Recommend annual screening mammography. ➡ *Positive*
- REPORT:
CLINICAL INDICATION: Annual screening mammogram. No current breast complaints.

COMPARISON: 10/15/2021.

FINDINGS:
Both breasts are composed of scattered fibroglandular density (BI-RADS density category b). 

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

There are stable bilateral breast implants. No evidence of implant rupture.

Visualized portions of the axillae appear unremarkable.

IMPRESSION:
NEGATIVE bilateral screening mammogram.
No suspicious mammographic findings.

RECOMMENDATIONS:
Recommend continued annual screening mammography per guidelines.
BIRADS 1. ➡ *Negative*
- FINDINGS: Bilateral digital screening mammogram was performed. The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications in the right breast. 

In the left breast, there is a new cluster of fine pleomorphic calcifications in the upper outer quadrant at approximately 2 o'clock, 6 cm from the nipple. This cluster measures approximately 6 mm in greatest dimension. No associated mass, architectural distortion or skin thickening is identified. 

The visualized left axillary lymph nodes appear unremarkable with no suspicious morphology or cortical thickening. 

Remainder of the left breast is unremarkable. No other suspicious calcifications, masses or areas of architectural distortion are identified.

IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation is needed. 
New cluster of suspicious microcalcifications in the left upper outer breast at 2:00, 6 cm from the nipple, measuring 6 mm. Recommend spot compression magnification views for further characterization. 

ADDENDUM: Spot compression magnification views of the left breast were performed. The cluster of calcifications in question persists and appears more concerning, demonstrating fine linear and branching morphology, suspicious for ductal carcinoma in situ (DCIS). Biopsy is recommended for definitive characterization.

REVISED IMPRESSION: 
BI-RADS Category 4: Suspicious abnormality. Biopsy should be considered.
Suspicious cluster of pleomorphic microcalcifications left upper outer breast, spanning 6 mm. Stereotactic core biopsy recommended. ➡ *Positive*
- ULTRASOUND LEFT BREAST

INDICATION: Palpable abnormality left breast upper outer quadrant.

FINDINGS:
The breasts are heterogeneously dense. Targeted ultrasound left breast 2:00 region 6 cm from the nipple demonstrates an irregular hypoechoic mass with angular margins and posterior shadowing measuring 11 x 8 x 7 mm. This corresponds to the palpable abnormality. 

At 2:00 3 cm from the nipple there is a 4 mm cluster of punctate echogenic foci consistent with microcalcifications.

The remainder of the left breast was scanned. There are several simple appearing cysts bilaterally, largest measuring 8 mm. No other suspicious masses, architectural distortion or calcifications are seen in either breast. 

IMPRESSION:
1. Irregular hypoechoic mass left breast 2:00, suspicious for malignancy. Ultrasound guided core biopsy is recommended. 
2. 4 mm cluster of microcalcifications left breast 2:00, suspicious for malignancy in this clinical setting. Ultrasound guided core biopsy is recommended.
3. Benign appearing cysts bilaterally.

BIRADS 4C: Suspicious finding, biopsy recommended. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM. HISTORY: Patient is referred for a diagnostic mammogram due to dense breast tissue on her screening mammogram performed at an outside facility on 02/07/2024. She reports that she is not currently on hormone replacement therapy. She has a family history of breast cancer in her mother. TECHNIQUE: Standard views were obtained. COMPARISON: Comparison is made with the outside mammograms. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- MAMMOGRAM REPORT - BILATERAL

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to prior mammograms from 11/15/2021.

FINDINGS:

RIGHT BREAST:
The breast is heterogeneously dense. There are no dominant masses, architectural distortion, or suspicious calcifications. The skin and nipple are unremarkable.

LEFT BREAST:
The breast is heterogeneously dense. There are no dominant masses, architectural distortion, or suspicious calcifications. The skin and nipple are unremarkable.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. No suspicious microcalcifications are identified.
2. Heterogeneously dense breasts, which may limit sensitivity of mammography.

RECOMMENDATIONS:
1. Continue annual screening mammography and clinical breast exams.
2. Consider supplemental screening with breast ultrasound or MRI due to dense breast tissue.

BI-RADS CATEGORY 1 - NEGATIVE. ➡ *Negative*
- SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
There are no suspicious masses, architectural distortions or microcalcifications identified in either breast.
Benign-appearing lymph nodes are noted in the left axilla.
The pectoralis muscles are within normal limits bilaterally.
IMPRESSION:
Negative screening mammogram. 
BIRADS-2. Recommend routine screening in 1 year. ➡ *Not Stated*
- FINDINGS: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. There are no dominant masses, architectural distortion, or suspicious calcifications.

At the 10:00 position of the right breast there is a 7 mm well-circumscribed isodense nodule, likely benign. This appears unchanged compared to prior mammograms.

The left breast demonstrates benign scattered fibroglandular elements. Surgical clips are seen in the upper outer quadrant from prior biopsy, stable in appearance. No other focal abnormalities.

IMPRESSION:
1. Heterogeneously dense breasts, which may lower the sensitivity of mammography. 
2. Likely benign 7 mm right breast nodule at 10:00, unchanged. Short-term follow up could be considered to establish stability.
3. No evidence of malignancy. Recommend annual screening mammography.

BIRADS 2 - Benign findings. ➡ *Negative*
- CLINICAL HISTORY: Annual screening mammogram. No current breast complaints. Patient has a strong family history of breast cancer, with mother and maternal aunt diagnosed with breast cancer in their 50s.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.

LEFT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications. 

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Recommend continued annual screening mammography and clinical breast exams due to strong family history of breast cancer.

BIRADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY.
History: Screening.
Comparison available dating from 12/2021.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- FINDINGS: The right breast contains scattered areas of fibroglandular density. There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or areas of architectural distortion.

Targeted right breast ultrasound was performed at the site of palpable abnormality at the 10:00 position 3 cm from the nipple. Ultrasound demonstrates a 6 mm simple cyst at this location. No solid mass is identified. 

IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. 
2. 6 mm simple cyst corresponding to the area of palpable abnormality in the right breast at 10:00, 3 cm from the nipple. No suspicious solid mass is identified on ultrasound. 

BI-RADS Category 2: Benign finding. Continued routine screening mammography is recommended.

ADDENDUM:
Comparison is made to prior mammogram from November 2021 at St. Joseph's Hospital. The 6 mm cyst in the right breast at 10:00 is new. There are no other significant changes compared to prior exam. ➡ *Negative*
- FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No concerning mass, architectural distortion or suspicious calcifications are identified.

There is a 0.5 cm well-circumscribed mass in the left breast at 9 o'clock middle depth. This corresponds to the palpable abnormality described by the patient. Targeted ultrasound was performed at this site which demonstrates a simple cyst.

A 5 mm well-circumscribed mass is noted in the right breast at 3 o'clock, posterior depth. This is unchanged compared to prior mammograms and has been previously characterized as a fibroadenoma. No other mass, suspicious calcifications or architectural distortion is identified. The right axillary lymph nodes appear normal.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. 5 mm stable right breast mass, likely fibroadenoma.
3. 0.5 cm left breast simple cyst corresponding to the patient's palpable finding.

BIRADS 2: Benign findings. Routine screening in 1 year is recommended. ➡ *Negative*
- SCREENING MAMMOGRAM OBSP BI-RADS CATEGORY 2

Compared to previous screening mammogram from 2 years ago at this facility, no concerning changes.

Breasts are heterogeneously dense (51-75% fibroglandular), which may obscure detection of small masses.

There are no suspicious masses, architectural distortions, or suspicious calcifications.

IMPRESSION:
BI-RADS Category 2 - Benign findings. Routine screening recommended. ➡ *Not Stated*
- SCREENING BILATERAL MAMMOGRAM
INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of each breast. 
COMPARISON: Screening mammogram from 4/2021.

FINDINGS:  
The breasts are predominantly fatty.
There are no masses, architectural distortion or suspicious calcifications in either breast.
There are scattered areas of fibroglandular density bilaterally.
In the left breast upper outer quadrant there is a new cluster of fine pleomorphic microcalcifications spanning approximately 12 mm. This is a new finding compared to prior examination and is suspicious for malignancy.
No suspicious right breast findings.
Skin is normal in thickness.  No adenopathy.

IMPRESSION:  
New cluster of suspicious microcalcifications left breast at 2:00, 6 cm from the nipple, spanning 12 mm. BIRADS 4B. Stereotactic biopsy recommended. 

BIRADS 2 right breast, benign. Recommend continued routine screening in 1 year. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No breast complaints.  
COMPARISON: Prior studies from 11/2021.
FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses.
There are no focal masses, significant architectural distortions or suspicious calcifications.  
There are stable scattered fibroglandular densities and bilateral intrammamary lymph nodes.   
IMPRESSION:
1. Negative bilateral screening mammogram.  
2. Recommend continued annual screening with mammography in 12 months.  
BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAM, BILATERAL DIAGNOSTIC
CLINICAL INDICATION: Annual screening mammogram. No clinical concerns.
COMPARISON: 2019-03-26 
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
There are no suspicious masses, architectural distortion or suspicious calcifications in either breast. 
In the left breast upper outer quadrant, there is a new 6 mm cluster of indeterminate amorphous microcalcifications at 1 o'clock, 8 cm from the nipple. This was not present on the prior study.
Stable postsurgical changes are noted in the right breast consistent with prior history of lumpectomy.  
There are stable benign-appearing calcifications bilaterally.
IMPRESSION: 
BI-RADS 0: Incomplete. Need additional imaging evaluation. 
New cluster of indeterminate microcalcifications in the left breast at 1 o'clock as described above. Recommend spot compression magnification views for further characterization.
ADDENDUM:
Spot compression magnification views of the left breast confirm a 6 mm cluster of indeterminate amorphous microcalcifications at 1 o'clock position, 8 cm from the nipple. Biopsy is recommended for definitive diagnosis.
FINAL ASSESSMENT: BI-RADS 4: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- IMPRESSION:
Bilateral diagnostic mammography demonstrates a new cluster of suspicious pleomorphic microcalcifications in the upper outer left breast at the 2:00 position, 6 cm from the nipple, spanning approximately 6 mm. This cluster is highly concerning for ductal carcinoma in situ (DCIS). Additionally, there is a 1.2 cm irregular mass with spiculated margins at the 10:00 position of the left breast, 3 cm from the nipple, highly suspicious for invasive ductal carcinoma.

Ultrasound of the left breast confirms an irregular hypoechoic mass with angular margins at 10:00, corresponding to the mammographic finding and measuring 11 x 9 x 8 mm. Ultrasound-guided core needle biopsy is recommended for definitive diagnosis. 

No suspicious masses, architectural distortion, or microcalcifications are identified in the right breast.

BIRADS ASSESSMENT:
Right breast: BI-RADS 1, Negative.
Left breast: BI-RADS 4C, Highly suspicious for malignancy. Biopsy recommended. ➡ *Positive*
- FINDINGS: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. No suspicious masses, architectural distortion or suspicious calcifications are seen. There are no significant changes compared to prior exam.

Targeted ultrasound was performed at the site of the palpable abnormality in the left lower inner quadrant at 7 o'clock, 3 cm from the nipple. No sonographic correlate was found. The palpable finding likely represents normal fibroglandular tissue.

IMPRESSION:
1. Negative digital breast tomosynthesis examination. No evidence of malignancy.
2. No sonographic correlate for the palpable finding in the left breast at 7 o'clock. This likely represents normal fibroglandular tissue.

BI-RADS Category 1: Negative. Recommend annual screening mammography in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening exam.
TECHNIQUE: Standard CC and MLO views of the breasts were obtained. 
COMPARISON: Comparison is made to prior studies from 09/15/2020.
FINDINGS:
Right breast:
The right breast shows scattered areas of fibroglandular density. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.  
Left breast: 
The left breast shows scattered areas of fibroglandular density. There is an area of grouped amorphous calcifications in the upper outer left breast, in a segmental distribution, spanning approximately 2 cm. This is a new finding compared to prior exams. No dominant masses or areas of architectural distortion.
IMPRESSION:
BI-RADS Category 0: Incomplete: Need additional imaging evaluation. Magnification views of the left breast are recommended for further characterization of the new grouped amorphous calcifications in the upper outer left breast. 
ADDENDUM:
Left breast magnification views demonstrate a segmental group of amorphous calcifications in the upper outer left breast, spanning approximately 2 cm. The distribution is suspicious for ductal carcinoma in situ (DCIS). Biopsy is recommended. 
BI-RADS Category 4B: Suspicious. Biopsy should be considered in the absence of clinical contraindication. ➡ *Positive*
- FINDINGS: Bilateral digital screening mammography was performed.

The breast tissue is heterogeneously dense, which may obscure small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a new cluster of fine pleomorphic calcifications in the upper outer quadrant at the 2:00 position, approximately 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No associated mass or architectural distortion is identified.

The remainder of the left breast is unremarkable, with no dominant masses, areas of architectural distortion, or additional suspicious calcifications.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation
Indeterminate cluster of microcalcifications in the left breast at 2:00, 6 cm from the nipple, as described above. Additional magnification views are recommended for further evaluation.

ADDENDUM:
Magnification views of the left breast were performed, confirming the presence of a cluster of fine pleomorphic calcifications at the 2:00 position of the left breast, approximately 6 cm from the nipple. The calcifications are more conspicuous on the magnification views and are concerning for ductal carcinoma in situ (DCIS).

REVISED IMPRESSION:
BI-RADS Category 4: Suspicious
Suspicious cluster of microcalcifications in the left breast at 2:00, 6 cm from the nipple, as described above. Stereotactic core biopsy is recommended for definitive diagnosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM. Clinical History: Patient is a 62 year old female presenting for a screening mammogram. Technique: Bilateral CC and MLO images were obtained. Comparison is made to prior mammographic examination from 10/12/2023. Findings: The breasts are heterogeneously dense. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: Negative mammogram. BI-RADS 1. ➡ *Negative*
- REPORT:
"LEFT DIAGNOSTIC MAMMOGRAM AND ULTRASOUND.
HISTORY: 48-year-old female with palpable left breast lump at 4 o'clock position. No prior mammograms available for comparison. Strong family history of breast cancer in two maternal aunts.
FINDINGS: The breasts are heterogeneously dense (BI-RADS density category C). This may lower the sensitivity of mammography. No dominant mass, suspicious calcifications, or focal asymmetry.
Targeted left breast ultrasound demonstrates a 7 mm irregular hypoechoic mass with indistinct margins and mild vascularity at the site of palpable abnormality in the 4 o'clock position, middle depth. This is highly suspicious for malignancy.
IMPRESSION:
1. Highly suspicious 7 mm mass left breast 4 o'clock position on ultrasound, BI-RADS 4. Ultrasound-guided core needle biopsy is recommended for definitive diagnosis.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
BIRADS 4.

ADDENDUM:
Ultrasound-guided core biopsy of the left 4 o'clock mass was performed. Pathology revealed invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. The patient will be referred to breast surgery for further management."
LABEL: Negative for microcalfications in the current report. ➡ *Negative*
- Breast Ultrasound Report
Patient Name: [Patient Name]
Date of Exam: 2024-07-08

Clinical History:
Screening breast ultrasound. History of left breast lobular carcinoma in situ (LCIS) diagnosed 3 years ago. Recent mammogram revealed new microcalcifications in the right breast.

Findings:
Right Breast:
- A small, irregular cluster of microcalcifications is identified in the right breast at the 10 o'clock position, correlating with the mammographic finding. The microcalcifications measure approximately 6 mm in aggregate and are surrounded by a subtle area of architectural distortion.
Left Breast:
- No suspicious masses or calcifications are seen in the left breast.

Impression:
- Right breast: Sonographic findings are consistent with the mammographically detected microcalcifications. The presence of architectural distortion raises concern for malignancy. Biopsy is recommended for definitive diagnosis.
- Left breast: No evidence of malignancy.

Recommendations:
- Ultrasound-guided biopsy of the microcalcifications in the right breast.
- Continued surveillance of the left breast as clinically indicated.

BI-RADS Category: 4 (Suspicious) ➡ *Positive*
- MRI BREAST WITHOUT CONTRAST, BILATERAL. History: Left breast palpable abnormality. Patient with strong family history of breast cancer (mother and sister) - on tamoxifen prophylaxis. Comparison is made to previous examinations. Findings: There is no abnormal enhancement to suggest malignancy. No suspicious masses. No suspicious calcifications. Impression: No MRI evidence of malignancy. ➡ *Negative*
- SCREENING MAMMOGRAM:

CLINICAL INFORMATION: Screening mammography.

COMPARISON: None available.

FINDINGS:
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.

The breasts appear symmetric. No significant masses, architectural distortion, or suspicious calcifications are identified. The visualized axillary regions appear unremarkable.

IMPRESSION:
Bilateral heterogeneously dense breasts.

No mammographic evidence of malignancy.

No significant interval change compared to prior exams.

BI-RADS Category 1: Negative. 

RECOMMENDATION: Recommend continued annual screening mammography. ➡ *Not Stated*
- TMIST Full Field Digital Mammogram - Normal Screening
The client was enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), approved by the Research Ethics Board. She provided informed consent and was assigned to the FFDM-only study arm.  
Comparison to prior exam: Yes.
Fibroglandular tissue density: Heterogeneously dense (below 75%).
Ontario Breast Screening Program recall interval: Resume routine annual screening.
There are no findings on this mammogram suspicious for breast malignancy. 
Final Assessment: BI-RADS Category 1 - Negative. ➡ *Not Stated*
- FINDINGS:
Bilateral digital screening mammogram was obtained.
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in the right breast. 
In the left breast, there is a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant at the 2:00 position, 8 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No other suspicious calcifications are identified in the left breast. 
There are no dominant masses or architectural distortion in the left breast.
IMPRESSION:
Left breast: New cluster of suspicious microcalcifications in the upper outer quadrant (2:00 position, 8 cm from nipple), measuring 8 mm. BI-RADS 4B. Stereotactic biopsy is recommended for further evaluation.
Right breast: No suspicious mammographic findings. Recommend continued routine screening in the right breast. BI-RADS 2. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to the prior study from 05/12/2021.

FINDINGS:

RIGHT BREAST:
The breast tissue is heterogeneously dense (BI-RADS category C). There is no evidence of a mass, architectural distortion, or suspicious calcifications. No significant change compared to the prior study.

LEFT BREAST: 
The breast tissue is heterogeneously dense (BI-RADS category C). In the upper outer quadrant, there is a new cluster of fine pleomorphic microcalcifications spanning an area of approximately 1 cm at the 2 o'clock position, 6 cm from the nipple. No associated mass, architectural distortion, or skin changes are identified. The remainder of the left breast is unremarkable. 

IMPRESSION:
1. New cluster of suspicious microcalcifications in the upper outer quadrant of the left breast, BI-RADS 4B. Stereotactic biopsy is recommended for further evaluation. 
2. Stable, benign findings in the right breast. BI-RADS 2.

ADDENDUM:
The patient was informed of the results and agrees to proceed with stereotactic biopsy of the left breast microcalcifications. The biopsy has been scheduled for next week. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms were obtained.
The breast tissue is heterogeneously dense which may obscure detection of small masses.
No spiculated masses, areas of architectural distortion or suspicious areas of skin thickening.
There are scattered areas of benign-appearing calcifications in both breasts.
At the 12:00 position of the right breast, approximately 5 cm from the nipple, there is a new group of amorphous calcifications spanning an area of about 1 cm. This is a suspicious finding. Magnification views confirm the presence of a cluster of pleomorphic calcifications at this site. 
The remainder of the left breast is unremarkable.

IMPRESSION:
New cluster of pleomorphic microcalcifications at the 12:00 position of the right breast, about 5 cm from the nipple, spanning 1 cm, suspicious for ductal carcinoma in situ (DCIS). Stereotactic core needle biopsy is recommended. BIRADS 4B.
Benign calcifications and heterogeneously dense breast tissue, otherwise negative bilateral screening mammogram. BIRADS 2 for the left breast. ➡ *Positive*
- MAMMOGRAM REPORT:
 
CLINICAL HISTORY: Routine screening mammogram. No current breast complaints.
 
COMPARISON: Comparison is made to prior mammograms from 2 years ago.
 
FINDINGS:
Both breasts contain scattered areas of fibroglandular density. Overall the breasts are not dense. 
 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast. 
 
The skin is unremarkable. No significant skin thickening or retraction. Nipples are within normal limits.
 
There are no significant changes compared to prior mammograms.
 
IMPRESSION: 
Negative bilateral screening mammogram. No mammographic evidence of malignancy.
 
RECOMMENDATION: 
Continue routine annual screening mammography in 1 year.
 
BI-RADS Category 1 - Negative. ➡ *Negative*
- MAMMOGRAM AND ULTRASOUND REPORT.
HISTORY: Left breast pain, 7 o'clock position, history of right breast cancer, previous right mastectomy. No other significant history.
FINDINGS:
Right mastectomy changes noted. 
Left breast: Scattered fibroglandular densities. No mammographic evidence of malignancy or suspicious microcalcifications. Stable compared to previous exams.
Ultrasound of the left breast at 7 o'clock: No sonographic evidence of a discrete mass or cyst. 
IMPRESSION:
1. Stable left breast, no evidence of malignancy on mammogram or ultrasound. BIRADS 1.
2. Right mastectomy changes.
Recommend annual screening mammograms.
ADDENDUM: 
Patient called and expressed concerns regarding a palpable lump in the left breast at 7 o'clock, corresponding to the area of pain. Clinical exam by the breast surgeon revealed no palpable abnormalities in this region. No further imaging is recommended at this time based on the negative mammogram and ultrasound findings, as well as the negative clinical exam. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION:  Annual screening mammogram.

COMPARISON:  Comparison is made to outside screening mammogram from December 2019.

FINDINGS:  
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.

On the right, there is no mammographic evidence of malignancy. There are no masses, architectural distortion, or suspicious calcifications. 

On the left, there are scattered fibroglandular densities. No dominant mass, architectural distortion, or skin thickening. In the upper outer left breast, there is a new group of pleomorphic microcalcifications spanning approximately 6 mm at the 2:00 position, 6 cm from the nipple. These were not present on the prior mammogram. No other suspicious calcifications are identified.

IMPRESSION:  
BI-RADS Category 0: Incomplete. 
New group of suspicious microcalcifications in the left breast, not present on prior mammogram. Additional imaging evaluation with magnification views is needed. 

RECOMMENDATION:  
Magnification views left breast for further evaluation of new group of microcalcifications at 2:00 position.

ADDENDUM - MAGNIFICATION VIEWS LEFT BREAST:
Magnification views of the left breast demonstrate a group of pleomorphic microcalcifications at the 2:00 position, 6 cm from the nipple. The calcifications are irregular in shape and distribution, highly suspicious for malignancy. 

FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious finding. Biopsy should be considered. ➡ *Positive*
- BILATERAL MAMMOGRAMS 
INDICATION: Annual screening.
TECHNIQUE: Bilateral digital mammograms were obtained including standard CC and MLO views. 
COMPARISON: Compared to prior mammograms from 05/10/2021.
FINDINGS:
Both breasts are heterogeneously dense (BI-RADS density C). 
There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast.
In the left breast there are scattered fibroglandular densities. No dominant masses or architectural distortion. 
There is a new group of pleomorphic microcalcifications in the left upper outer quadrant, extending over an area of about 1 cm. The microcalcifications range from linear to round in shape and are of intermediate concern. 
Remaining left breast is unremarkable.
IMPRESSION:
BI-RADS 1 right breast: Negative.
BI-RADS 4B left breast: Suspicious microcalcifications upper outer quadrant, biopsy recommended.
ADDENDUM:
Ultrasound-guided core biopsy of the left breast microcalcifications at 2:00, 8 cm from the nipple showed ductal carcinoma in-situ, intermediate grade, cribriform type.
Stereotactic biopsy is recommended for further characterization and to rule out invasive component. Updated BI-RADS assessment BI-RADS 4C. ➡ *Positive*
- MAMMOGRAM RIGHT BREAST.
HISTORY: Right breast pain.
COMPARISONS: None available.
TECHNIQUE: Standard CC and MLO views of the right breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative right mammogram. BI-RADS 2.
RECOMMENDATION:
Correlation with recent mammograms is recommended when available. If symptoms persist, clinical correlation is recommended. ➡ *Negative*
- MAMMOGRAPHY - BILATERAL - SCREENING. CLINICAL HISTORY: Screening mammogram. Patient is on tamoxifen. TECHNIQUE: Craniocaudal and mediolateral oblique views of the breasts were performed using digital mammography. COMPARISON: Comparison is made to the mammogram from 04/19/2023. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
History: Screening mammogram.
Comparison is available dating from 06/2023 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening
Comparison available dating from 01/2021 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM WITH MAGNIFICATION VIEWS

CLINICAL HISTORY: 52 year old woman, annual screening mammogram. No palpable lumps or nipple discharge. Maternal aunt with breast cancer at age 59.

TECHNIQUE: CC and MLO views of bilateral breasts. Magnification views of right breast.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.

RIGHT BREAST:
There is a new cluster of pleomorphic calcifications in the upper outer quadrant on the CC and MLO views, with more than 10 calcifications in a linear distribution measuring approximately 8mm. Magnification CC and MLO views confirm these suspicious calcifications. No dominant masses, architectural distortion or skin thickening.

LEFT BREAST: 
No dominant masses, suspicious calcifications, architectural distortion or skin thickening.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic core biopsy is recommended for the cluster of suspicious calcifications in the right upper outer quadrant.
BI-RADS Category 1: Normal left breast.

ADDENDUM:
Stereotactic core biopsy of the right breast calcifications was performed. Pathology results showed ductal carcinoma in situ, high nuclear grade with comedonecrosis. ➡ *Positive*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
CLINICAL INDICATION: Screening mammogram. No palpable abnormality.
TECHNIQUE: Bilateral digital mammography and targeted ultrasound of the left breast.
COMPARISON: Prior study from 10/15/2021.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
No dominant masses, architectural distortion, or suspicious calcifications are identified in either breast.
Targeted ultrasound of the left breast 2 o'clock position demonstrates normal fibroglandular tissue without a discrete mass, cyst, or dilated duct.
Stable benign-appearing calcifications in the left breast.
IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. 
2. Stable benign-appearing calcifications in the left breast.
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are almost entirely fatty.

There are scattered areas of fibroglandular density.

At the 6 o'clock position of the right breast, 6 cm from the nipple, there is a new 8 mm cluster of pleomorphic calcifications. Additional magnification views confirm the presence of this cluster. 

No dominant masses, architectural distortion, or suspicious calcifications are seen in the left breast.

There are no abnormalities of the skin, subcutaneous tissues, or nipple-areolar complexes. No axillary lymphadenopathy.

IMPRESSION:
New cluster of pleomorphic calcifications in the right breast as described above, suspicious for malignancy. Additional imaging evaluation with diagnostic views of the right breast is recommended. BI-RADS 4B.

Normal left breast. BI-RADS 1.

ADDENDUM: The patient was contacted and informed of the results. She will present for additional right breast diagnostic mammographic views and ultrasound. If the calcifications are confirmed, stereotactic core biopsy will be recommended. ➡ *Positive*
- MAMMOGRAM, BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: Mammogram from 1/15/2021 at ABC Imaging Center.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast:
There are no mammographic findings to suggest malignancy. No dominant mass, architectural distortion, or suspicious calcifications. 

Left breast:
There are no mammographic findings to suggest malignancy. No dominant mass, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend continued annual screening with mammography.
BIRADS 1 - Negative.

ADDENDUM:
Discussed results with patient. Reinforced the importance of monthly breast self-exams, annual clinical breast exams, and promptly reporting any breast changes or concerns to her physician. Patient voices understanding. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: 61-year-old female presenting for routine screening mammogram. No prior breast imaging available for comparison.

FINDINGS:
Both breasts are heterogeneously dense (C). This may lower the sensitivity of mammography.

Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left breast:
There are no dominant masses or areas of architectural distortion. In the upper outer quadrant of the left breast, there is a group of pleomorphic microcalcifications spanning an area of approximately 1 cm. No other suspicious calcifications are identified.

IMPRESSION:
Right breast: No mammographic evidence of malignancy. BIRADS 1.

Left breast: Group of pleomorphic microcalcifications in the upper outer quadrant, suspicious for malignancy. BIRADS 4B. Stereotactic biopsy is recommended for further evaluation.

ADDENDUM:
The patient proceeded with stereotactic biopsy of the group of microcalcifications in the left breast at 1:00, 10 cm from the nipple. Pathology results demonstrated ductal carcinoma in situ (DCIS), intermediate grade, with no evidence of invasion. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts. 

COMPARISON: None available. 

FINDINGS:
The breasts are composed of scattered fibroglandular densities.

In the right breast there are no dominant masses, architectural distortions or suspicious calcifications. There is a small cluster of punctate calcifications in the upper outer quadrant which are most likely benign in appearance.

In the left breast there are no dominant masses or architectural distortions. There are regional pleomorphic calcifications in the upper outer quadrant extending over a distance of approximately 3 cm. These calcifications appear new since the prior study. No other suspicious calcifications are noted.

IMPRESSION:
BIRADS 4: Suspicious new regional pleomorphic calcifications in the left breast. Biopsy recommended for further evaluation. 

BIRADS 2: Benign appearing cluster of punctate calcifications in the right breast. Recommend routine screening in 1 year.

ADDENDUM: Patient presented for stereotactic core biopsy of the new left breast calcifications. Specimen radiograph confirms calcifications within the core samples. Final pathology is pending and will be reported in an addendum. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAM
CLINICAL HISTORY: 45-year-old female presenting for diagnostic mammogram. Patient reports feeling a lump in the right breast.
TECHNIQUE: Bilateral digital diagnostic mammogram including CC and MLO views was obtained. Spot compression magnification views of the right breast were also performed.
COMPARISON: Comparison is made to previous mammogram from 1 year ago.
FINDINGS:
Right Breast:
The breast tissue is heterogeneously dense. In the upper outer quadrant of the right breast, there is a new 1.2 cm irregular mass with indistinct margins at the 10 o'clock position, 6 cm from the nipple. Spot compression magnification views confirm the presence of the mass and also demonstrate associated fine pleomorphic microcalcifications extending anteriorly from the mass for approximately 8 mm. No suspicious axillary lymphadenopathy.
Left Breast:
The breast tissue is heterogeneously dense. There are no dominant masses, areas of architectural distortion, or suspicious microcalcifications. No suspicious axillary lymphadenopathy.
IMPRESSION:
BI-RADS 4: Suspicious abnormality in the right breast. Stereotactic core biopsy is recommended for the irregular mass with associated microcalcifications at the 10 o'clock position of the right breast. BI-RADS 2 for the left breast. ➡ *Positive*
- MAMMOGRAM LEFT BREAST.
History: Screening mammogram
Comparison with 12/13/2023
Findings:
There are scattered fibroglandular densities. No architectural distortion, suspicious calcification, or definite masses are seen.
Impression:
Negative left mammogram.
BI-RADS Category 1. ➡ *Negative*
- BREAST MAMMOGRAM AND ULTRASOUND
INDICATION: Annual screening. No current breast complaints.  
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  
COMPARISON: 10/15/2021.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast. Stable benign appearing bilateral breast tissue. No significant interval change compared to prior mammograms.
Ultrasound of the right breast 10:00 position demonstrates normal fibroglandular tissue without a discrete mass, cyst, or other finding.
Ultrasound of the left breast 2:00 position shows normal ducts and stromal tissue without a discrete mass, cyst, or other finding. 
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening in 12 months. 
2. No sonographic abnormality in the areas scanned. 
BIRADS 1- Negative. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM, MLO AND CC VIEWS.

History: High-risk screening. Strong family history of breast cancer. No palpable lumps, skin changes, or nipple discharge.

Comparison: None available.

FINDINGS: The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications seen in the right breast.

In the left breast at the 12 o'clock position, 4 cm from the nipple, there is a 6 mm cluster of pleomorphic microcalcifications. No associated mass or architectural distortion is seen in this area.

The skin is normal in thickness and appearance bilaterally. No skin lesions are identified. The nipples are within normal limits bilaterally. No intramammary lymph nodes are seen.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation. 6 mm cluster of pleomorphic microcalcifications left breast at 12 o'clock position, 4 cm from the nipple. These are new and of concern. Magnification views of this area are recommended for further evaluation.

BI-RADS Category 2 for the right breast: Benign - Routine screening in 12 months recommended.

ADDENDUM: Magnification views in the craniocaudal and mediolateral projections of the left breast were performed and demonstrated the cluster of pleomorphic microcalcifications. These calcifications are irregular in shape and distribution, highly suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended for definitive diagnosis.

REVISED IMPRESSION:
BI-RADS Category 4C for the left breast: Highly Suggestive of Malignancy - Biopsy should be performed in the absence of clinical contraindication. ➡ *Positive*
- LEFT MAMMOGRAM
HISTORY: Patient reports pain in left breast, no palpable lump. No personal history of breast cancer. Mother had breast cancer. Last mammogram 2 years ago at outside facility was reported as normal.
FINDINGS: Left breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant mass, architectural distortion or suspicious calcifications. There are stable benign appearing calcifications in the upper outer quadrant. No significant change compared to outside mammograms from 2 years ago.  
IMPRESSION: No mammographic evidence of malignancy. Recommend annual screening mammography. 
BIRADS 2 - Benign finding.
ADDENDUM: Comparison made to outside mammograms brought in by patient from OHSU dated 3/14/2021, reported as benign. Today's study is similar. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY.
History: Screening mammogram
Comparison available dating from 07/2023.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- MAMMOGRAPHIC EXAMINATION
Comparative mammographic views performed using GE Senographe 2000d unit.
Clinical History:
Screening mammogram.
COMPARISON: 11/23/2016 
FINDINGS:
RIGHT BREAST:
Tissue composition is heterogeneously dense, with the density of the breast potentially obscuring small masses.
No focal dominant mass, area of architectural distortion, or suspicious calcifications.
LEFT BREAST:
Tissue composition is heterogeneously dense, with the density of the breast potentially obscuring small masses.  
A new cluster of microcalcifications is identified at the 10 o'clock position of the left breast, middle depth, 8 cm from the nipple. It consists of 5 punctate calcifications within a 5 mm area. The cluster is concerning for grouped/clustered suspicious microcalcifications.
Spot compression magnification views in the craniocaudal and mediolateral projections confirms the presence and the morphology of the calcifications.
No other mammographic abnormality is identified in the left breast.
IMPRESSION:
New grouped microcalcifications in the left breast (BI-RADS 4).
No mammographic evidence of malignancy in the right breast. (BI-RADS 2) 
RECOMMENDATION: 
Left breast stereotactic biopsy is recommended for the microcalcifications at the 10 o'clock position.
Addendum:
Stereotactic biopsy performed on the same day demonstrates microcalcifications on the specimen radiograph. Final pathology is pending. ➡ *Positive*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography arm only.
Comparison to prior imaging: Yes. 
Breast density: Scattered fibroglandular density.
Dominant breast composition: Fibroglandular.
Recall recommendation: Routine screening in 1 year recommended per OBSP guidelines.
There are no suspicious masses, architectural distortions, or other findings concerning for malignancy identified in either breast. 
BIRADS Category 1 - Negative. Routine screening recommended. ➡ *Not Stated*
- REPORT:
"SCREENING MAMMOGRAM.
HISTORY: 55 year old female, here for routine screening mammogram.
FINDINGS: Bilateral scattered fibroglandular densities. No mammographic evidence of malignancy. No dominant masses, suspicious calcifications, or areas of architectural distortion bilaterally.
Comparison to prior exam from 1 year ago shows stable mammographic appearance, with no interval development of suspicious findings.
IMPRESSION: 
Negative bilateral screening mammogram.
BIRADS 1 - Negative. Recommend routine screening in 1 year." ➡ *Negative*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: Annual screening mammogram. Patient denies any new breast concerns.
COMPARISON: Screening mammogram from 1 year prior.
FINDINGS:
Right breast: Scattered fibroglandular densities. No suspicious masses, architectural distortion, or calcifications.
Left breast: Heterogeneously dense breast tissue. No dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts, which may obscure detection of small masses. 
BIRADS 1 - Negative.
RECOMMENDATION: 
Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- MAMMOGRAPHY AND ULTRASOUND - LEFT BREAST

HISTORY: 52 year old female with vague area of palpable concern left breast 10 o'clock position, no known family history of breast cancer. Last screening mammogram performed 1 year ago was negative.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications.  

Targeted left breast ultrasound was performed at area of clinical concern, 10 o'clock position 3 cm from the nipple. No sonographic correlate for the palpable finding was identified. 

IMPRESSION:
No mammographic or sonographic evidence of malignancy. Likely benign finding on clinical exam, however follow-up recommended to document stability.

BIRADS 1 - Negative. Recommend annual screening mammography. ➡ *Negative*
- MAMMO - SCREENING BILATERAL - 3D

CLINICAL HISTORY: Screening mammogram.

COMPARISON: 06/06/2023.

FINDINGS: There are scattered benign-appearing calcifications bilaterally, stable in appearance. There are new clustered pleomorphic calcifications in the left breast at the 2 o'clock position, 1 cm from the nipple. This measures 6 mm.

IMPRESSION:

1. New clustered pleomorphic calcifications in the left breast. Recommend targeted ultrasound for further evaluation.

BI-RADS: 0 (Needs additional imaging). ➡ *Positive*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
Scattered fibroglandular densities are present in both breasts. No significant masses, architectural distortion, or skin thickening is identified.
BIRADS-2.
Addendum: No microcalcifications are mentioned in this report. ➡ *Not Stated*
- OBSP Full Field Digital Mammogram.
Date of exam: Today. Clinical indication: Routine screening.
Comparison: Archival mammograms from 1 year ago.  
Both breasts are heterogeneously dense, which may obscure small masses.
No significant change compared to prior mammograms. 
Specifically, there are no dominant masses, suspicious calcifications, or areas of architectural distortion in either breast. The breast parenchyma is stable.
No mammographic evidence of malignancy. 
Radiologic assessment: BI-RADS Category 1, Negative.
Routine annual screening mammography is recommended.

CONFIDENCE ASSESSMENT:
- High confidence in absence of statement about microcalcifications.
- High confidence in overall BI-RADS assessment of 1. 
- Moderate confidence in details of breast density and comparison to priors, as slight changes in wording here are unlikely to affect the key "Not Stated" aspect for microcalcifications. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

CLINICAL INDICATION: Screening.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense (50-75% fibroglandular tissue). This may lower the sensitivity of mammography.

Bilateral benign-appearing calcifications are seen, likely representing fibrocystic changes. No suspicious microcalcifications are identified.

There are no dominant masses, areas of architectural distortion, or suspicious asymmetries in either breast.

IMPRESSION:
BIRADS 2 - Benign findings. No evidence of malignancy.

RECOMMENDATION:
Routine screening in 1 year is recommended. ➡ *Not Stated*
- FINDINGS:

Bilateral diagnostic mammogram was performed using standard views including spot compression magnification.

The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

No discrete mass, architectural distortion or suspicious microcalcifications are identified.

Stable benign scattered calcifications noted bilaterally.

Comparison is made to previous mammogram from June 15, 2021, with no significant interval change.

IMPRESSION:

BIRADS 2: Benign - recommend routine screening in 12 months. ➡ *Not Stated*
- BILATERAL MAMMOGRAMS WITH TOMOSYNTHESIS
CLINICAL HISTORY: Annual screening mammogram. 
TECHNIQUE: Bilateral full field digital mammograms including tomosynthesis were obtained.
COMPARISON: None. 
FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense (BI-RADS C). No dominant mass, architectural distortion or suspicious calcifications are identified. 
LEFT BREAST:
The breast is heterogeneously dense (BI-RADS C). There is a small cluster of indeterminate microcalcifications in the upper outer quadrant, approximately 5 cm from the nipple. No dominant mass, architectural distortion or lymphadenopathy.
IMPRESSION:
RIGHT BREAST: No mammographic evidence of malignancy. BI-RADS 1, annual screening recommended.
LEFT BREAST: Small cluster of indeterminate microcalcifications in the upper outer quadrant, additional magnification views recommended for further characterization. BI-RADS 0, recall for additional imaging.
ADDENDUM - LEFT BREAST MAGNIFICATION VIEWS: 
Additional magnification views of the left breast demonstrate a small cluster of indeterminate microcalcifications in the upper outer quadrant, approximately 5 cm from the nipple. No associated mass or architectural distortion.
FINAL ASSESSMENT: 
RIGHT BREAST: BI-RADS 1. Annual screening recommended.
LEFT BREAST: BI-RADS 4. Suspicious cluster of microcalcifications. Biopsy recommended. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
CLINICAL INDICATION: Annual screening mammogram.
COMPARISON: 10/15/2022
TECHNIQUE: Standard CC and MLO views of both breasts.
FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense (type C). There are no dominant masses, architectural distortion, or suspicious calcifications. The skin and nipple are unremarkable.
LEFT BREAST:
The breast is heterogeneously dense (type C). There are no dominant masses, architectural distortion, or suspicious calcifications. The skin and nipple are unremarkable.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts, which may obscure detection of small masses.
RECOMMENDATION:
Annual screening mammography is recommended.
BIRADS ASSESSMENT CATEGORY: 1 - Negative.
ADDENDUM:
Comparison to prior exams shows stable appearance of both breasts, with no interval development of suspicious findings. The breasts remain heterogeneously dense, which can lower the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI for women with dense breast tissue, especially in the setting of additional risk factors. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS WITH TOMOSYNTHESIS AND MAGNIFICATION VIEWS.

HISTORY: Patient noted a lump in the left breast.

FINDINGS:
The breasts are heterogeneously dense, which can obscure small masses.

There are no suspicious right breast findings.

In the upper outer left breast at 2:00, 4 cm from the nipple, there is an irregular 12 mm high density mass, partially obscured. This persists on spot compression views. 

On magnification views, a 5 mm cluster of pleomorphic calcifications is seen at 3:00, 2 cm from the nipple in the left breast. These were not well seen on the screening mammogram.

The left axilla appears unremarkable.

IMPRESSION:
1. BI-RADS 4: Suspicious irregular mass, upper outer left breast at 2:00. Recommend biopsy.
2. BI-RADS 4: Cluster of pleomorphic microcalcifications, 3:00 left breast. Recommend stereotactic biopsy.
3. BI-RADS 1: No suspicious findings, right breast. 

ADDENDUM: Ultrasound left breast 2:00 mass and 3:00 calcifications.

The 12 mm irregular mass at 2:00 in the left breast demonstrates irregular margins and internal vascularity. Ultrasound guided biopsy is recommended.  

The microcalcifications at 3:00 are not well seen sonographically. Stereotactic biopsy is advised if not proceeding with surgical management. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH LEFT DIAGNOSTIC VIEWS 

INDICATION: Annual screening mammogram.

COMPARISON: Last mammogram 1 year ago.

FINDINGS:
The breasts are almost entirely fatty.

LEFT BREAST: 
There is a new 7 mm cluster of indeterminate amorphous and punctate microcalcifications at the 6 o'clock position middle depth, 5 cm from the nipple which are new compared to prior mammogram. A magnification tangential view confirms the presence of microcalcifications. No associated mass, architectural distortion or skin thickening.
The remainder of the left breast is unremarkable. 

RIGHT BREAST:  
There are no dominant masses, areas of architectural distortion or suspicious microcalcifications.

IMPRESSION:
New cluster of indeterminate microcalcifications at 6 o'clock position left breast middle depth. Additional evaluation with stereotactic core biopsy is recommended. BIRADS 4B left breast.

Benign finding right breast. BIRADS 2 right breast.

ADDENDUM STEREOTACTIC BIOPSY LEFT BREAST:
After informed consent was obtained, the patient underwent stereotactic core biopsy of the indeterminate calcifications at the 6 o'clock position left breast using 9G vacuum assisted biopsy device. Specimen radiograph confirmed calcifications. Marker clip placed. Post procedure mammograms show marker clip at the biopsy site. The patient tolerated the procedure well. Pathology results pending. ➡ *Positive*
- BREAST ULTRASOUND
CLINICAL HISTORY: 40 year old woman presenting with right breast pain. No palpable lumps.
TECHNIQUE: Targeted ultrasound of the right breast at 10 o'clock position. 
COMPARISON: None.
FINDINGS: 
Right breast:
Breast composition is heterogeneously dense. Targeted ultrasound at the 10 o'clock position of the right breast demonstrates normal fibroglandular tissue. No focal solid or cystic masses. No architectural distortion. No suspicious calcifications.  
Left breast: 
Not imaged. 
IMPRESSION:
1. No sonographic evidence of malignancy in the right breast at the area of clinical concern. 
2. Recommend follow up diagnostic mammogram for further evaluation.
BI-RADS Category 1 - Negative. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAPHY WITH TARGETED LEFT MEDIOLATERAL OBLIQUE MAGNIFICATION VIEWS.

CLINICAL: Left breast lump, palpable at 3:00.

FINDINGS:
The breasts are heterogeneously dense (Type C) which could obscure small masses. 

There are no suspicious findings or dominant masses in the right breast. BI-RADS 1 right.

In the left breast, there is an indistinct asymmetry in the upper outer quadrant on the CC view only, corresponding to the palpable abnormality at 3:00. Targeted magnification views in the mediolateral oblique projection show pleomorphic microcalcifications spanning 12 mm at the site of the asymmetry. No dominant mass, architectural distortion, or skin thickening.

IMPRESSION:
BI-RADS 1 right breast. Negative.
BI-RADS 4B left breast. Suspicious pleomorphic microcalcifications spanning 12 mm at site of palpable abnormality, 3:00, 6 cm from nipple. Biopsy recommended for definitive diagnosis. ➡ *Positive*
- IMAGING STUDIES PERFORMED:
BILATERAL DIAGNOSTIC MAMMOGRAM WITH CAD AND BILATERAL BREAST ULTRASOUND.

INDICATION:  
Screening mammogram. 

Clinical history: 
43-year-old woman with no breast complaints. Family history of maternal aunt with breast cancer diagnosed at age 49. 

FINDINGS:
This examination consists of bilateral digital FFDM (full field digital mammography) and CAD (computer-aided detection).

The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 

There are no mammographic abnormalities, concerning masses, or dominant masses. The breast tissue appears within normal limits bilaterally.  

No significant or suspicious masses or calcifications are identified in either breast. The skin/nipple/areola complexes are normal in appearance. There are no significant architectural distortions or asymmetries.  

Bilateral breast ultrasound was performed. No focal solid or suspicious lesion is seen in either breast. 

IMPRESSION:
1. Negative bilateral screening mammogram and ultrasound.
2. No mammographic or sonographic evidence of malignancy.
3. Recommend continued annual bilateral screening mammography in 12 months.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- REPORT:
"SCREENING MAMMOGRAM.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views obtained bilaterally.
FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.
There are no masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
BIRADS 1: Negative screening mammogram. Recommend continued annual screening in 12 months.
ADDENDUM:
The interpreting radiologist reviewed the study independently and agrees with the above findings. No suspicious calcifications are identified. There are no masses, asymmetries, or areas of architectural distortion in either breast. Benign-appearing lymph nodes are noted in both axillae."
LABEL: Negative for microcalfications in the current report. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY

CLINICAL HISTORY: 58 year old female here for routine screening mammography.

COMPARISON: Compared with prior studies from June 1, 2021.

FINDINGS: 
The breasts are heterogeneously dense which may obscure detection of small masses. 

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

In the left breast at the 12 o'clock position 6 cm from the nipple, there is a new 5 mm cluster of amorphous microcalcifications. 

Stable benign-appearing calcifications are noted bilaterally.

IMPRESSION: 
BI-RADS Category 0: Incomplete. Need additional imaging evaluation. 
New cluster of amorphous microcalcifications in the left breast, indeterminate. Additional magnification views recommended for further characterization.

ADDENDUM (Post Magnification Views): The cluster of amorphous microcalcifications in the left breast is better characterized on magnification views. The calcifications are new compared to prior mammograms and have suspicious morphology. Stereotactic biopsy is recommended. BI-RADS Category 4B: Suspicious. Moderate suspicion for malignancy. ➡ *Positive*
- MAMMOGRAPHY - SCREENING. History: 53-year-old female for routine screening mammogram. Patient reports that she is currently not on hormone replacement therapy. Patient reports a family history of breast cancer in her maternal aunt, diagnosed at the age of 82. Comparison is made with the patient's previous mammogram performed on 10/17/2023. Findings: The breasts are heterogeneously dense. There are no masses, architectural distortion, or suspicious calcifications. Impression: No evidence of malignancy. BI-RADS 1. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening

TECHNIQUE: Standard CC and MLO views obtained bilaterally.

COMPARISON: None available

FINDINGS:

RIGHT BREAST:
The right breast is heterogeneously dense (BI-RADS category c).
There are no dominant masses, areas of architectural distortion or suspicious calcifications.

LEFT BREAST:
The left breast is heterogeneously dense (BI-RADS category c).
There are no dominant masses or areas of architectural distortion.
In the upper outer quadrant of the left breast, there is a small cluster of indeterminate microcalcifications at approximately 2 o'clock, 5 cm from the nipple. The cluster measures approximately 5 mm in greatest dimension. No other suspicious calcifications are identified.

IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. BI-RADS 1 - Negative.
2. Small cluster of indeterminate microcalcifications in the upper outer quadrant of the left breast, BI-RADS 0 - Incomplete. Additional magnification views recommended for further evaluation.

ADDENDUM:
Left magnification views demonstrate a cluster of pleomorphic microcalcifications in the upper outer quadrant, which are suspicious for malignancy. Stereotactic biopsy is recommended.

FINAL ASSESSMENT:
RIGHT: BI-RADS 1 - Negative
LEFT: BI-RADS 4 - Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- BILATERAL MAMMOGRAM

CLINICAL HISTORY: Patient is status post left mastectomy for invasive ductal carcinoma. She presents today for routine follow up imaging of her right breast.

COMPARISON: Prior mammogram 04/03/2024.

FINDINGS: The right breast is heterogeneously dense, which could obscure a small mass. There are scattered benign appearing calcifications in the right breast, unchanged. There is a new 8 mm cluster of fine pleomorphic calcifications in the right breast at the 2 o'clock position, 2 cm from the nipple.

IMPRESSION:

1. New cluster of fine pleomorphic calcifications in the right breast. Recommend spot magnification views for further evaluation. 

BI-RADS: 0 (Needs additional imaging). ➡ *Positive*
- MAMMOGRAPHY, RIGHT BREAST.
INDICATION: Follow up after right breast lumpectomy.
TECHNIQUE: Standard CC and MLO views of the right breast were obtained. Targeted ultrasound of the right breast upper outer quadrant was also performed.
COMPARISON: Prior exam from 10/15/2022.
FINDINGS:
The right breast is heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, architectural distortions, or suspicious calcifications. The surgical scar in the right upper outer quadrant is unchanged. Targeted ultrasound of this area shows no residual or recurrent lesion. The remainder of the right breast is unremarkable.
IMPRESSION:
1. No evidence of malignancy in the right breast. 
2. Stable postoperative changes in the right breast, as described above. 
3. Recommend annual screening mammography and continued clinical follow up.
BIRADS 1 - Negative.
ADDENDUM:
Radiology/Pathology Correlation: The patient's prior right breast lumpectomy pathology from 10/01/2022 showed invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative, with negative margins. The current exam shows no evidence of residual or recurrent disease. ➡ *Negative*
- MAMMOGRAM REPORT
HISTORY: Screening mammogram. No prior mammograms available for comparison. No palpable lumps or breast complaints.
FINDINGS:
Right breast: The breast is composed of scattered fibroglandular densities. No dominant masses, architectural distortion, or suspicious calcifications.  
Left breast: The breast is composed of scattered fibroglandular densities. No dominant masses, architectural distortion, or suspicious calcifications.  
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. No evidence of malignancy. Routine screening mammography is recommended in 1 year.
BIRADS 1 - NEGATIVE. ➡ *Negative*
- TMIST full field digital mammography screening examination. There is no statement related to microcalcifications.

Breasts are heterogeneously dense but no concerning findings are seen. No significant interval change from prior studies. Follow-up mammography in 1 year per OBSP guidelines.

BIRADS Category 2 - Benign Findings. Return to routine screening. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

INDICATION: Annual screening mammogram.  

COMPARISON: Screening mammogram from 04/17/2021. 

TECHNIQUE: CC and MLO views of both breasts were obtained.

FINDINGS:  
The breasts are almost entirely fatty.

RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.

LEFT BREAST:  
There are scattered areas of amorphous calcifications in the upper outer quadrant, ranging in size from 0.2-0.5 mm. These have increased in number compared to the prior study. No associated mass or architectural distortion.

There are no dominant masses or architectural distortion in the left breast.

IMPRESSION:
1. Scattered amorphous calcifications in the left breast upper outer quadrant, increased from prior. Additional magnification views recommended for further characterization. BIRADS 0.

2. Benign right breast. BIRADS 2.

MAGNIFICATION VIEWS LEFT BREAST, SAME DAY:

Magnification views in the CC and lateral projections of the left breast upper outer quadrant were performed.  

The amorphous calcifications are more conspicuous on these views and appear grouped. The distribution is segmental, spanning an area of approximately 3 cm. No associated mass, distortion, or skin thickening.

ADDENDUM TO IMPRESSION:
1. Grouped amorphous calcifications spanning 3 cm in the left breast at 2:00, 5-8 cm from the nipple. Suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy recommended. BIRADS 4B. ➡ *Positive*
- SCREENING MAMMOGRAM

CLINICAL HISTORY: Screening mammogram. No current breast complaints.

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

Incidental note is made of scattered benign-appearing calcifications in both breasts, most prominently in the right breast laterally and superiorly. Largest group measures approximately 1 cm.

IMPRESSION:
BI-RADS 2: Benign findings. Recommend routine screening in 1 year.

ADDENDUM:
The benign-appearing calcifications noted in the right breast have increased in number compared to prior exam. Although favored to represent benign process such as fibrocystic change, their increase raises possibility of early developing DCIS. Recommend 6 month follow-up magnification views to document stability before resuming routine screening. BI-RADS 3 for this group of calcifications. ➡ *Positive*
- TMIST FFDM NORMAL MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.

The breast tissue is heterogeneously dense. No suspicious masses, architectural distortion or dominant focal asymmetries are identified. No significant change from the prior study.  

BIRADS-2. Routine screening recommended. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening
Comparison available dating from 07/2023 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- MAMMOGRAPHY - BILATERAL - SCREENING. CLINICAL HISTORY: Screening mammogram. Patient is on tamoxifen. TECHNIQUE: Craniocaudal and mediolateral oblique views of the breasts were performed using digital mammography. COMPARISON: Comparison is made to the mammogram from 12/15/2022. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- TMIST FFDM ONLY ARM MAMMOGRAM. NORMAL STUDY.
The patient consented to participate in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was assigned to the FFDM only arm.
Comparison to prior exam: Yes. 
Breast density: Scattered fibroglandular density (25-50% glandular).
No suspicious mammographic findings detected. Breasts are composed of scattered fibroglandular tissue.
Follow-up mammography recommended in 12 months per the Ontario Breast Screening Program (OBSP) for average risk women aged 50-74.
BIRADS Category 1 - Negative. ➡ *Not Stated*
- MAMMOGRAPHY, BILATERAL.

INDICATION: Annual screening mammogram.

COMPARISON: Compared to prior study from 12/15/2021.

FINDINGS:  
Both breasts are predominantly fatty, corresponding to BI-RADS density category a. 

There are no masses, areas of architectural distortion, or suspicious microcalcifications in either breast.

Stable bilateral benign calcifications are noted.

There are no significant changes compared to prior studies.

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Recommend continued annual screening mammography per guidelines.

BI-RADS Category 1 - Negative. ➡ *Negative*
- TMIST FFDM BREAST CANCER SCREENING
This client participated in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography arm.

Compared to Previous: No prior mammograms available for comparison.

Breast Density: The breasts are heterogeneously dense, which may obscure small masses.

FINDINGS:
There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
No mammographic evidence of malignancy. Routine screening is recommended. 

BI-RADS Category 1: Negative.

ADDENDUM:
3D tomosynthesis images were obtained and reviewed. No additional findings were identified on the tomosynthesis images. ➡ *Not Stated*
- FINDINGS:

BILATERAL DIGITAL MAMMOGRAPHY WITH MAGNIFICATION VIEWS:

The breasts are predominantly fatty, consistent with ACR type A.

At 12 o'clock in the right breast, 6 cm from the nipple, there is a group of pleomorphic calcifications spanning an area of 8 mm. The individual calcifications range in size from 0.2-0.6 mm. No associated mass, architectural distortion, or skin thickening is identified.

At 3 o'clock in the left breast, 4 cm from the nipple, there are scattered amorphous calcifications. No associated mass, architectural distortion, or skin thickening is identified.

The remainder of the right and left breasts are unremarkable. No dominant masses, areas of architectural distortion, or suspicious calcifications are identified. 

IMPRESSION:
- Grouped pleomorphic calcifications in the right breast at 12 o'clock, 6 cm from the nipple, suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended. BIRADS 4B.
- Scattered amorphous calcifications in the left breast, likely benign. Six-month follow-up mammogram is recommended to ensure stability. BIRADS 3.

ADDENDUM:
The patient returned for stereotactic biopsy of the grouped calcifications in the right breast at 12 o'clock. Pathology results demonstrate ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. Wide local excision is recommended. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

CLINICAL HISTORY: High risk screening.

COMPARISON: None.

FINDINGS:
The breasts are predominantly fatty.

At the 9:00 position of the right breast, there is a 1 cm cluster of microcalcifications. The calcifications are heterogeneous in morphology, including fine linear branching and pleomorphic forms. This cluster was not present on the prior study.

No suspicious mass, architectural distortion, or other findings are identified in either breast. The skin is unremarkable.

IMPRESSION:
BI-RADS 4: Suspicious microcalcifications in the right breast at 9:00, 6 cm from the nipple, that are new compared to the prior study. Stereotactic biopsy is recommended for further evaluation. 

ADDENDUM:
The patient returned for stereotactic biopsy of the cluster of microcalcifications at the 9:00 position of the right breast. Post-biopsy, specimen radiograph confirmed the presence of calcifications. Pathology results demonstrated ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type, associated with microcalcifications. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- MAMMOGRAM
HISTORY: Annual screening mammogram.
COMPARISON: Prior studies from 1 and 2 years ago.
FINDINGS:
Both breasts are heterogeneously dense, which could obscure detection of small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
There are no significant changes compared to prior studies.
IMPRESSION:
No mammographic evidence of malignancy. Recommend continued annual screening in 12 months. 
BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS BILATERAL.
History: Patient reports no changes. No nipple changes or discharge. No breast lumps or skin changes.
Comparison: None.
FINDINGS:
The breasts are heterogeneously dense which may obscure small masses. 
There are scattered areas of benign appearing calcifications in both breasts. 
No spiculated masses, suspicious calcifications or areas of architectural distortion are identified in either breast. 
There are no suspicious findings in the right breast. BIRADS 2.
In the upper outer left breast there is a 6 mm cluster of indeterminate microcalcifications at the 2:00 position, 6 cm from the nipple. Spot compression magnification views confirm these as pleomorphic calcifications in a segmental distribution. These are new since the prior study and suspicious for ductal carcinoma in situ (DCIS). A stereotactic biopsy is recommended for further assessment. BIRADS 4B. 
The visualized axillary regions appear unremarkable. 
IMPRESSION:
1. New 6 mm cluster of suspicious pleomorphic microcalcifications upper outer left breast as described, BIRADS 4B. Stereotactic biopsy recommended. 
2. Routine biennial screening for the right breast. ➡ *Positive*
- RIGHT BREAST ULTRASOUND
Clinical History: Increased density in right breast on screening mammogram performed on 01/03/2025. Patient also reports a palpable lump in the right breast in the 3:00 position.
Comparison: None
Technique: Real-time sonography of the right breast was performed using a high-frequency linear-array transducer. Targeted imaging of the palpable lump in the right breast was performed, as well as evaluation of the area of increased density seen on the recent mammogram
Findings:
The palpable lump in the right breast in the 3:00 position correlates to a benign-appearing, circumscribed, oval, hypoechoic mass measuring 1.0 x 0.6 x 0.5 cm (TRV x AP x CC). This demonstrates posterior acoustic enhancement. This was palpable and mobile with transducer pressure. These findings are consistent with a fibroadenoma. The area of increased density seen on the recent mammogram correlates to a region of overlapping normal fibroglandular tissue. No suspicious masses are identified sonographically. No abnormal axillary lymph nodes are identified.
IMPRESSION:
Right breast, palpable lump, consistent with a fibroadenoma. Increased density in the right breast on recent mammogram correlates to a region of overlapping normal fibroglandular tissue. No suspicious masses are identified sonographically.
BI-RADS Category 2: Benign findings. ➡ *Not Stated*
- Screening Mammogram
Clinical History: 59 year old female, screening mammogram
Comparison: None
Technique: Standard CC and MLO views of the breasts were obtained
Findings:
Heterogeneously dense breast tissue bilaterally
No suspicious masses, calcifications, or architectural distortions are seen
IMPRESSION:
Negative screening mammogram. BI-RADS 1. ➡ *Negative*
- NORMAL TMIST DBT ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the digital breast tomosynthesis arm.
Compared to Previous: Yes.
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
OBSP to recall client in 1 year: Yes.
The breast tissue is extremely dense, which lowers the sensitivity of mammography. 
No mammographic findings that are concerning for malignancy.
BIRADS-1/2. ➡ *Not Stated*
- RADIOLOGY REPORT
PROCEDURE: Screening Mammogram, Bilateral.
COMPARISON: 05/15/2024
TECHNIQUE: Standard CC and MLO views of each breast.
INDICATION: Routine screening, high risk. Mother with breast cancer.
FINDINGS:
Both breasts are heterogeneously dense (C), which may obscure detection of small masses.
Right Breast:
There are no dominant masses, architectural distortions, or suspicious calcifications.
Left Breast:
There are no dominant masses, architectural distortions, or suspicious calcifications.
Axillae:
There are no enlarged axillary lymph nodes on the right or suspicious lymph nodes on the left.
IMPRESSION:
BI-RADS Category 1: Negative bilateral screening mammogram. No evidence of malignancy. Recommend continued annual screening mammography in 12 months due to patient's elevated lifetime risk.
ADDENDUM:  Comparison to previous mammogram reveals stable appearance of both breasts. ➡ *Negative*
- FINDINGS:
The breast tissue is heterogeneously dense. 
There is a 1.2 cm irregular mass at the 10:00 position of the right breast, approximately 4 cm from the nipple. The mass demonstrates spiculated margins and associated architectural distortion. 
At the 12:00 position of the right breast, there is a cluster of fine pleomorphic calcifications spanning approximately 7 mm. 
No suspicious masses, calcifications, or architectural distortion are identified in the left breast. 
The skin is normal in thickness and appearance. No axillary or internal mammary adenopathy is identified.

IMPRESSION:
1. Highly suspicious irregular spiculated mass right breast at 10:00, 4 cm from the nipple. Additional imaging evaluation with ultrasound is recommended. Consider biopsy.
2. Small cluster of suspicious pleomorphic calcifications at 12:00 right breast. Magnification views confirm. Stereotactic biopsy is recommended. 
3. Left breast unremarkable.

BI-RADS ASSESSMENT CATEGORY:
Right Breast: 4C 
Left Breast: 1 ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses. 
No concerning dominant masses, architectural distortion or skin thickening.

There is a new cluster of pleomorphic microcalcifications in the upper outer quadrant of the right breast at approximately 10 o'clock, 7 cm from the nipple. The cluster measures approximately 5 mm. 

Stable bilateral axillary lymph nodes, likely benign.

IMPRESSION: 
BI-RADS 4: Suspicious cluster of microcalcifications in the right breast, biopsy recommended.

ADDENDUM: 
The patient returned for stereotactic biopsy of the right breast microcalcifications. Specimen radiograph confirmed calcifications within the core samples.
Pathology results: Ductal carcinoma in situ, high nuclear grade with necrosis.
Updated BI-RADS assessment: BI-RADS 6: Known biopsy-proven malignancy. ➡ *Positive*
- FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion. 
There are no new or enlarging masses in comparison with prior exams.
No sonographic abnormality is seen.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. 
3. Recommend annual screening mammography. 

BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMO - BILAT - SCREENING. Clinical History: 60 year old female, no breast complaints, requests screening mammogram today. Comparison is made with previous mammograms from 10/27/2023. Findings: The breasts are heterogeneously dense. No suspicious masses or calcifications are identified. Impression: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM. Clinical History: 57 year old female presents for her annual screening mammogram. No new breast symptoms. Comparison is made to previous outside mammogram from 03/15/2023. Findings: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications in both breasts. No new or enlarging masses or areas of architectural distortion. Impression: 1. No mammographic evidence of malignancy. 2. BI-RADS 1. ➡ *Negative*
- FINDINGS:
Right mastectomy and ALND scar noted.
The left breast is heterogeneously dense. No significant masses, architectural distortion, or suspicious calcifications are identified.
Stable postoperative changes noted in the right chest wall.
No suspicious left axillary lymphadenopathy.
Stable benign calcifications in the left breast.

IMPRESSION:
1. Negative digital mammogram of the left breast. No mammographic evidence of malignancy.
2. Stable postoperative changes of right mastectomy.

BIRADS Category 1 - Negative.

Recommend annual screening mammogram of the left breast in 12 months. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (BI-RADS C). 

There are scattered benign calcifications bilaterally. Additional magnification views in the left CC view demonstrate a new cluster of indeterminate microcalcifications in the upper outer quadrant, approximately 8 cm from the nipple. The calcifications are irregular in shape and distribution, spanning an area of about 1 cm. No associated mass, distortion or skin changes are identified.

The right breast is unremarkable, with no suspicious masses, calcifications, asymmetries or skin changes. 

IMPRESSION:
BI-RADS 0: Indeterminate cluster of microcalcifications in the left breast at the 2:00 position, 8 cm from the nipple. Additional imaging evaluation with magnification views is recommended for further characterization. If the microcalcifications remain suspicious on additional imaging, a stereotactic biopsy should be considered.

BI-RADS 2: Benign findings in the right breast. Recommend continued routine screening in 1 year.

ADDENDUM:
Magnification views of the left breast demonstrate a cluster of pleomorphic microcalcifications spanning approximately 1 cm at the 2:00 position, 8 cm from the nipple. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS). 

REVISED IMPRESSION:
BI-RADS 4: Suspicious cluster of microcalcifications in the left breast. Stereotactic core biopsy is recommended for definitive diagnosis. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS

INDICATION: Routine annual screening.

COMPARISON: None.

TECHNIQUE: Bilateral CC and MLO views obtained. Tomosynthesis performed.

FINDINGS:
The breasts are heterogeneously dense (C), which could obscure detection of small masses. 

No dominant mass, architectural distortion or suspicious asymmetry.

A new cluster of linear and branching amorphous microcalcifications is noted in the upper outer quadrant of the right breast spanning an area of approximately 4 mm. The calcifications are of suspicious morphology. No prior mammograms available for comparison. 

Bilateral benign appearing lymph nodes noted.

IMPRESSION:  
BI-RADS 4: Suspicious microcalcifications in the right breast. Stereotactic biopsy is recommended for further evaluation. Tomosynthesis enabled improved conspicuity and localization of the microcalcifications.

ADDENDUM:
The patient proceeded to stereotactic biopsy of the right breast microcalcifications the same day. Post-biopsy mammogram confirms adequate sampling of the area of suspicion. Pathology results pending. Full report to follow under separate cover. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. Patient denies breast pain, skin changes or nipple discharge.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: Comparison is made to outside screening mammogram from 1 year prior.
FINDINGS:
Right breast: The breast tissue is heterogeneously dense (type C). This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications are identified. 
Left breast: The breast tissue is heterogeneously dense (type C). This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications are identified.
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Heterogeneously dense breast tissue which could obscure detection of small masses. 
RECOMMENDATIONS:
1. Routine annual screening mammography is recommended in 1 year. Clinical breast examination is also recommended.
2. Please inform the patient that her breast tissue is dense. Dense breast tissue is very common and is not abnormal. However, dense breast tissue can make it harder to find cancer on a mammogram and may also be associated with an increased risk of breast cancer. This information is given to raise awareness and to inform her medical care decisions. A report of the results was sent to the patient and referring provider.
ASSESSMENT CATEGORY: BIRADS 1 - Negative. ➡ *Negative*
- LEFT DIAGNOSTIC MAMMOGRAM
Clinical History: 65 y.o. female presents today for a diagnostic mammogram due to a palpable finding in the left breast.
Comparison: Prior mammogram from (outside institution) 04/05/2023.
Technique: Standard CC and MLO views were obtained of the left breast. In addition, spot compression and magnification views were performed to better evaluate the area in question.
Findings:
The breasts are heterogeneously dense which may reduce the sensitivity of mammography.
Left breast:
Palpable finding corresponds to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Left breast, palpable finding, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended. ➡ *Negative*
- FINDINGS:
Bilateral digital screening mammogram was performed.
The breast parenchymal density is heterogeneously dense, which may obscure detection of small masses.
There are scattered areas of fibroglandular density bilaterally.
No dominant masses, areas of architectural distortion or suspicious calcifications are seen in the right breast. 
In the left breast, there is a new cluster of pleomorphic microcalcifications in the upper outer quadrant at the 2:00 position, 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. The morphology and distribution are suspicious for ductal carcinoma in situ.
No other suspicious calcifications, masses, or areas of architectural distortion are identified in the left breast.
No mammographic evidence of multicentric or multifocal disease.
No skin thickening or retraction. Benign calcifications are noted bilaterally.
Both nipples are in profile and appear unremarkable. No significant axillary lymphadenopathy.

IMPRESSION:
New cluster of pleomorphic microcalcifications upper outer left breast at 2:00, suspicious for ductal carcinoma in situ. Biopsy is recommended. BI-RADS 4B.
Remainder of left breast is BI-RADS 2. Benign/negative right breast, BI-RADS 1.

ADDENDUM:
Ultrasound guided core biopsy of the microcalcifications in the left breast at 2:00 was performed. Specimen radiograph confirms calcifications within the cores. Final pathology results are pending. Post-biopsy mammogram shows biopsy clip in good position at the site of prior calcifications. ➡ *Positive*
- BREAST SCREENING PROGRAM MAMMOGRAM, DIGITAL BILATERAL
CLINICAL INDICATION: Screening study.
COMPARISON: Previous outside exams not available.
TECHNIQUE: Standard CC and MLO views acquired bilaterally using full field digital mammography (FFDM). 
FINDINGS:
The breasts are heterogeneously dense (50-75% fibroglandular tissue), which may obscure small masses.
No dominant masses, focal asymmetries or areas of architectural distortion.  
No suspicious axillary lymphadenopathy.
Stable benign-appearing vascular calcifications noted bilaterally.
IMPRESSION:
BI-RADS Category 2: Benign. 
Recommend continued routine screening mammography in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

At the 12 o'clock position of the right breast, approximately 6 cm from the nipple, there is a cluster of amorphous microcalcifications spanning an area of about 8 mm. Additional magnification views in this area are recommended for further evaluation.

The remaining fibroglandular tissue in both breasts is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete, additional imaging is needed. 
Cluster of amorphous microcalcifications at 12 o'clock position right breast, recommend additional magnification views for further assessment.

ADDENDUM: 
Right breast magnification views demonstrate a cluster of amorphous microcalcifications at the 12 o'clock position, approximately 6 cm from the nipple, spanning an area of about 8 mm. This cluster is new compared to the patient's prior mammograms. Stereotactic biopsy is recommended for tissue diagnosis.

FINAL ASSESSMENT: BI-RADS Category 4: Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- LEFT DIAGNOSTIC MAMMOGRAM
Clinical History: 43 y.o. female presents today for a diagnostic mammogram due to a palpable finding in the left breast
Comparison: Prior mammograms from (outside institution), not available for review.
Technique: Standard CC & MLO views were obtained of each breast. In addition, spot compression and magnification views were performed to better evaluate the palpable findings in the right breast.
Findings:
The breasts are heterogeneously dense which may reduce the sensitivity of mammography.
Left breast:
Palpable finding corresponds to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Left breast, palpable finding, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended. ➡ *Negative*
- FINDINGS: The breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant masses, suspicious calcifications or architectural distortion are identified. There are no significant changes compared to prior exam.

Targeted ultrasound was performed at the site of palpable abnormality in the left breast at 2 o'clock, 4 cm from the nipple. No sonographic correlate for the palpable finding is identified. The area of palpable concern demonstrates normal fibroglandular tissue.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy. Specifically no suspicious microcalcifications are identified.
2. Discordant benign exam. Palpable abnormality in the left breast at 2 o'clock is not demonstrated on imaging. 

RECOMMENDATION: Continued routine screening mammography in 12 months. Clinical follow up of palpable left breast abnormality as indicated.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM AND LEFT BREAST ULTRASOUND.
HISTORY: 65 year old female, routine screening. Prior benign biopsy of right breast at 10 o'clock position. No other significant history. No current complaints.
Comparison made to prior imaging from May 2021.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right breast: There are no dominant masses, architectural distortion, or suspicious calcifications. Stable benign biopsy clip noted in the upper outer quadrant at 10 o'clock position, corresponding to the patient's history. No significant change from prior exam.
Left breast: There are no dominant masses, architectural distortion, or suspicious calcifications. No significant change from prior exam.
Bilateral axillae are unremarkable.
Targeted ultrasound of the right breast at 10 o'clock position demonstrates a stable benign-appearing biopsy site, corresponding to the mammographic biopsy clip. No suspicious lesions seen.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend continued annual screening mammography.
2. Stable benign right breast biopsy site at 10 o'clock position, unchanged from prior exams.
BIRADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- TMIST MAMMOGRAM REPORT:
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.

The breast parenchyma is heterogeneously dense. No suspicious masses, architectural distortions or suspicious calcifications are identified.

IMPRESSION: 

No evidence of malignancy. Routine annual screening is recommended. 

BIRADS Category 2: Benign finding. ➡ *Not Stated*
- BILATERAL BREAST ULTRASOUND

INDICATION: Annual screening.

TECHNIQUE: Bilateral whole breast and axillary ultrasound was performed using a high frequency linear array transducer.

COMPARISON: None.

FINDINGS:
Right breast: The breast is heterogeneously dense. No suspicious mass, architectural distortion, or abnormal calcifications are identified.

Left breast: The breast is heterogeneously dense. At the 10:00 position, 6 cm from the nipple, there is a group of punctate echogenic foci, spanning an area of approximately 5 mm, consistent with microcalcifications. No associated mass or architectural distortion is identified. The remainder of the left breast is unremarkable.

Axillae: There is no sonographic evidence of abnormal lymph nodes in either axilla.

IMPRESSION:
1. Heterogeneously dense breasts.
2. Left breast: Group of microcalcifications at 10:00, 6 cm from the nipple, spanning 5 mm. No associated mass or architectural distortion. BI-RADS 4B. Stereotactic biopsy is recommended for further evaluation.
3. Right breast: Negative. BI-RADS 1.

ADDENDUM:
The patient proceeded with stereotactic biopsy of the group of microcalcifications in the left breast at 10:00. Pathology results demonstrated benign breast tissue with focal microcalcifications, consistent with fibrocystic changes. No evidence of malignancy.

FINAL ASSESSMENT:
Right breast: BI-RADS 1, Negative.
Left breast: BI-RADS 2, Benign findings. Recommend routine screening in 1 year. ➡ *Positive*
- FINDINGS: The right breast demonstrates post-mastectomy changes. The left breast tissue is heterogeneously dense. No dominant mass, architectural distortion or malignant-type calcifications are identified.

Targeted ultrasound of the left breast at the 7 o'clock position corresponding to the area of clinical concern was performed. No sonographic correlate for the palpable abnormality was identified. The area consists of normal appearing fibroglandular tissue.

IMPRESSION:
1. Negative left diagnostic mammogram and targeted ultrasound. No evidence of malignancy.
2. Clinical correlation for the palpable left breast abnormality at 7 o'clock position is recommended as this may represent a normal anatomic structure.

BI-RADS Category 1: Negative.

ADDENDUM: Comparison is made to prior mammograms from December 2024, no significant interval change. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM. CLINICAL HISTORY: Patient is a 51-year-old female presenting for a screening mammogram. TECHNIQUE: Bilateral CC and MLO images were obtained. COMPARISON: Comparison is made to prior mammographic examination from 11/03/2023. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: Negative mammogram. BI-RADS 1. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM. History: Patient is referred for evaluation of palpable findings in bilateral breasts. Comparison: None. Findings: There are scattered benign-appearing calcifications in both breasts. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: No mammographic evidence of malignancy. BI-RADS Category 2: Benign finding. Recommendation: Clinical correlation recommended. ➡ *Negative*
- FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
There are scattered areas of fibroglandular density and scattered fat lobules noted in both breasts. These are stable compared to prior mammograms.
No skin thickening or retraction is identified.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued routine screening mammography in 1 year.
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. Please inform patient that her breast tissue density falls under the category of "Dense Breast Tissue" as per CT Public Act 09-41. Recommend clinical breast examination.

ASSESSMENT CATEGORY: 1 - Negative

ADDENDUM:
Comparison was made to the patient's last mammogram from 8/24/2021, which is similar. No previous mammograms are available at this facility for comparison.
This examination was reviewed by the interpreting radiologist. If there are any questions regarding this report, please contact the Radiology department. ➡ *Negative*
- SCREENING MAMMOGRAM.

INDICATION: Screening. 

COMPARISON: Screening mammogram of December 17, 2021.

TECHNIQUE: Standard CC and MLO views of both breasts.

FINDINGS:
Both breasts are almost entirely fatty. No mass, architectural distortion or suspicious calcifications are identified.

IMPRESSION:
Negative bilateral screening mammogram. 
BIRADS 1 - Negative. Routine screening is recommended in 1 year. ➡ *Negative*
- BIRADS: 0 Incomplete. Additional imaging needed.
Clinical history: 54 year old female called back from screening for indeterminate calcifications left breast upper outer quadrant.
FINDINGS:
Left breast: At the 1:00 position 8 cm from the nipple there are indeterminate amorphous calcifications in a group spanning 8 mm. Magnification views confirm these. The remainder of the left breast is unremarkable.
Right breast: Normal. No suspicious masses, calcifications or areas of architectural distortion.
IMPRESSION:
1. New group of indeterminate amorphous calcifications left breast at 1:00, 8 cm from the nipple. These require biopsy as they are of uncertain significance and stability cannot be assessed as there are no available prior mammograms. I have arranged stereotactic biopsy for tomorrow at 8 am.
BIRADS 0 left incomplete.
BIRADS 1 right normal.
ADDENDUM: LEFT STEREOTACTIC BIOPSY 1:00
Pre and post fire clips placed. 6 core samples of calcifications obtained and specimen radiograph confirms. No immediate complications. Pathology pending.
FINAL BIRADS LEFT 4B. Suspicious abnormality. ➡ *Positive*
- LEFT MAMMOGRAM AND ULTRASOUND.
HISTORY: Referred from Family Physician for screening mammogram. No palpable abnormality or nipple discharge. Maternal grandmother diagnosed with breast cancer age 70. No other family history of breast cancer.
Last mammogram Dec 2021 - normal bilateral mammogram.
FINDINGS: Both breasts are heterogeneously dense, which may obscure detection of small masses. No dominant mass, architectural distortion or suspicious calcifications. There are no significant changes compared to previous exam.
Left breast ultrasound was performed, targeted to the upper outer quadrant. Sonographically normal appearing breast tissue. No sonographic abnormality.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening in 12 months.
2. No abnormality on targeted left breast ultrasound.
BIRADS 1. Negative bilateral screening mammogram. ➡ *Negative*
- FINDINGS:
Bilateral digital diagnostic mammograms were obtained.
The breasts are heterogeneously dense, which may obscure detection of small masses.
At the 12 o'clock position of the right breast, there is a new 6 mm cluster of fine pleomorphic calcifications, located approximately 6 cm from the nipple. This was not present on the prior mammogram from 2 years ago. Additional magnification views confirm the presence of this cluster.
No other suspicious calcifications, masses, or areas of architectural distortion are identified in either breast.
There are no suspicious calcifications in the left breast. 
The skin is normal in thickness and appearance. 

IMPRESSION:
New cluster of suspicious microcalcifications in the right breast at 12 o'clock, 6 cm from the nipple, BI-RADS 4B. Stereotactic core biopsy is recommended for further evaluation. The remainder of the breast tissue is negative bilaterally, BI-RADS 2. ➡ *Positive*
- BILATERAL MAMMOGRAM AND ULTRASOUND OF THE BREASTS:

INDICATION: Palpable lump in the right breast. High risk family history of breast cancer.

FINDINGS:
Heterogeneously dense breasts (ACR category D).

Right Breast:
In the lower inner quadrant at 4:00, 6 cm from the nipple there is an irregularly shaped mass measuring 21 x 18 x 16 mm with spiculated margins and posterior acoustic shadowing. The mass corresponds to the palpable finding. Associated with the mass are several groups of pleomorphic microcalcifications extending anteriorly. No suspicious axillary lymphadenopathy. 

Left Breast:
No significant masses, architectural distortion or suspicious calcifications. Ultrasound shows a 5 mm simple cyst at 2:00.

IMPRESSION:
Highly suspicious mass at 4:00 lower inner right breast (BIRADS 5) with associated pleomorphic microcalcifications. Ultrasound guided core needle biopsy is recommended for definitive diagnosis. 

The remainder of both breasts are negative (BIRADS 1). 

ADDENDUM - ULTRASOUND GUIDED CORE BIOPSY RIGHT BREAST 4:00:
The suspicious mass in the right lower inner breast underwent ultrasound guided core needle biopsy using a 14 gauge needle. Specimen radiograph confirmed calcifications within the samples. The biopsy was technically adequate and specimens were submitted to pathology. The patient tolerated the procedure well. Skin marking placed. Follow up with referring physician to discuss results and management. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of the right and left breast.

FINDINGS:
The breasts are heterogeneously dense which can obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast. 

In the left breast at the 12:00 position, 8 cm from the nipple, there is a small group of amorphous calcifications spanning approximately 6 mm. This is new compared to the patient's prior mammograms. Additional magnification views in the CC and MLO projection confirm the presence of these calcifications. 

No other mammographic abnormalities are identified. The skin is clear. No axillary, supraclavicular or internal mammary adenopathy is evident.

IMPRESSION:

BI-RADS Category 0: Incomplete - Need additional imaging evaluation and/or prior mammograms for comparison. 

Small group of amorphous calcifications in the left breast at 12:00, 8 cm from nipple. Additional magnification views confirm these are not artifactual. These are indeterminate and a stereotactic core needle biopsy should be recommended for definitive characterization. 

ADDENDUM: 
The patient returned for left breast stereotactic core needle biopsy of the group of amorphous calcifications at the 12:00 position of the left breast. Specimen radiograph confirmed that calcifications were sampled. Final pathology showed focal atypical ductal hyperplasia with associated calcifications, in a background of fibrocystic change. The patient is recommended for 6 month follow up diagnostic mammogram of the left breast given this result. ➡ *Positive*
- FINDINGS: The left breast is predominantly fatty replaced. There are no mammographic findings suspicious for malignancy. No masses, calcifications, architectural distortion, or asymmetries. Stable compared to prior exams.

Right breast ultrasound performed. There is heterogeneous fibroglandular tissue. No sonographic evidence of malignancy. No solid or cystic masses. No areas of architectural distortion. 

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Negative targeted right breast ultrasound.

BIRADS 1 - Negative. Recommend continued annual screening in 12 months. ➡ *Negative*
- OBSP SCREENING MAMMOGRAM. 
This is a TMIST participant randomly assigned to the FFDM arm of the trial after informed consent. 
Comparison to prior mammograms: Yes.  
Breasts are heterogeneously dense which may obscure small masses.
No concerning findings are identified. The breasts are negative for masses, architectural distortion, or asymmetries.
Breast density is estimated at 40-50%.
Routine OBSP screening is recommended in 1 year.
BIRADS Category 1 - Negative. ➡ *Not Stated*
- IMAGING STUDIES OBTAINED: Bilateral digital mammography with tomosynthesis views and left breast diagnostic views.

INDICATION: Left breast mass on physical exam.

FINDINGS:
The breasts are predominantly fatty (BI-RADS category a).

In the right breast, there are no dominant masses, areas of architectural distortion, or suspicious calcifications.

In the left breast, at the 1 o'clock position 4 cm from the nipple there is a 1 cm irregular mass with ill-defined margins, associated with a group of pleomorphic microcalcifications measuring approximately 1.2 cm in extent. No other suspicious masses or calcifications are identified.

The axillae are unremarkable bilaterally.

IMPRESSION:
1. BI-RADS 5: Highly suspicious mass with associated pleomorphic microcalcifications in the left breast at 1 o'clock, 4 cm from the nipple. Biopsy is recommended.

2. BI-RADS 1: Normal right breast.

ADDENDUM: The patient was informed of the results. Core biopsy of the left breast mass was recommended and will be scheduled with consent. A marker clip will be placed if biopsy is performed. ➡ *Positive*
- MAMMOGRAPHIC EXAMINATION:
INDICATIONS: Suspicious microcalcifications upper outer left breast detected at outside screening mammogram. Additional imaging for further evaluation.
COMPARISON: None. Outside mammogram images not available.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
No dominant masses, architectural distortion or suspicious calcifications are seen in the right breast.
In the upper outer left breast there is a new cluster of indeterminate amorphous and pleomorphic microcalcifications spanning an area of approximately 10 mm. This corresponds to the area of concern on outside screening mammogram.
Magnification views are performed and show no associated mass or distortion.
IMPRESSION:
New cluster of indeterminate amorphous and pleomorphic microcalcifications upper outer left breast. Biopsy is recommended for further evaluation. BIRADS 4B.
Heterogeneously dense breast tissue which may lower the sensitivity of mammography.
RECOMMENDATION:
Stereotactic core biopsy left breast microcalcifications is recommended. A biopsy requisition will be sent to booking. ➡ *Positive*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: Yes.
The breasts are composed almost entirely of adipose tissue.
No evidence of suspicious focal asymmetry, mass, architectural distortion or suspicious microcalcifications. There are scattered areas of vascular calcifications bilaterally.
Stable post surgical changes noted in the left breast.
BIRADS-2: Benign findings. Routine screening recommended.

ADDENDUM: Microcalcifications are not specifically mentioned or described in this report. ➡ *Not Stated*
- MAMMOGRAPHY - SCREENING - BILATERAL. Clinical History: 58 year old female for routine screening mammogram. No personal history of breast cancer. Strong family history of breast cancer - sister diagnosed at age 42. Comparison is made with the patient's previous mammogram performed on 03/06/2023. Findings: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: No evidence of malignancy. BI-RADS 1. ➡ *Negative*
- MAMMOGRAM REPORT, BILATERAL

INDICATION: Annual screening mammogram. No current breast complaints.

COMPARISON: 09/17/2021

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
No dominant mass, architectural distortion or suspicious calcifications are identified in either breast.
There are no significant changes compared to prior mammograms.

IMPRESSION:
NO MAMMOGRAPHIC EVIDENCE OF MALIGNANCY, BIRADS 1 - NEGATIVE.

Recommend annual screening mammography in 1 year. ➡ *Negative*
- ULTRASOUND OF RIGHT BREAST
CLINICAL INDICATION: Right breast pain and possible mass.
TECHNIQUE: Targeted right breast ultrasound.
COMPARISON: None available.
FINDINGS:
The right breast demonstrates heterogeneously dense parenchyma. No focal mass, architectural distortion, or suspicious calcifications are identified. 
Targeted ultrasound of the area of concern in the right breast upper outer quadrant demonstrates normal fibroglandular tissue. No discrete solid or cystic mass is seen. The area of palpable concern corresponds to normal glandular tissue.
IMPRESSION:
No sonographic evidence of malignancy in the right breast. 
BIRADS 1 - Negative.
Recommend continued routine screening mammography and clinical follow up as indicated. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.

INDICATION: Annual screening.

FINDINGS:
Both breasts demonstrate heterogeneously dense fibroglandular tissue. There are no spiculated masses, suspicious calcifications or areas of architectural distortion in either breast.

Bilateral axillary regions are unremarkable. No enlarged axillary, supraclavicular or internal mammary lymph nodes are seen.

IMPRESSION:

Negative bilateral digital screening mammogram. No suspicious findings.

BIRADS Category 1 - Negative. Recommend annual screening mammography in 12 months. ➡ *Negative*
- IMPRESSION:
Scattered groups of indeterminate microcalcifications are noted in the upper outer quadrant of the left breast on mammography. Stereotactic biopsy is recommended for tissue diagnosis. BI-RADS 4B.

The remainder of the left breast is unremarkable. 

The right breast shows no mammographic evidence of malignancy. BI-RADS 1.

INDICATION: Left breast lump. No known personal or family history of breast cancer.

TECHNIQUE: Bilateral digital screening mammogram and spot compression magnification views of the upper outer quadrant of the left breast. 

FINDINGS:
The breast tissue is heterogeneously dense which may lower the sensitivity of mammography.

LEFT BREAST:
There are scattered groups of heterogeneous microcalcifications in the upper outer quadrant, spanning an area of approximately 4 cm. The morphology and distribution of these calcifications are indeterminate and suspicious. No associated mass, architectural distortion or skin thickening is identified. Stereotactic biopsy is recommended for further evaluation. 

The remainder of the left breast shows no dominant mass, suspicious calcifications, or architectural distortion.

RIGHT BREAST: 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

ADDENDUM: The pathology from stereotactic biopsy of the left breast microcalcifications showed sclerosing adenosis with focal atypical ductal hyperplasia. Surgical excision has been recommended for definitive characterization. BI-RADS 4B. ➡ *Positive*
- MAMMOGRAPHY REPORT:

INDICATION: Screening.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are predominantly fatty.
No significant masses, architectural distortion, or suspicious calcifications are identified in either breast.
The skin is unremarkable. 

IMPRESSION:
Negative bilateral screening mammogram.
BIRADS Category 1 - Negative. Routine screening is recommended.

No mention of microcalcifications. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Screening.  
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: None.
FINDINGS: 
The breasts are heterogeneously dense. No suspicious masses, architectural distortion or discrete nodules.
No suspicious microcalcifications are identified.
There are scattered benign-appearing calcifications bilaterally, likely representing fibrocystic change.  
No suspicious lymphadenopathy.
IMPRESSION:
BIRADS 2: Benign/Negative. 
Recommend routine screening in 1 year.
ADDENDUM: Microcalcification descriptor not included in this report as no suspicious microcalcifications are identified. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.
COMPARISON: None.
FINDINGS:
The breast tissue is heterogeneously dense, which could obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
Negative screening mammogram.
BIRADS Assessment Category 1: Negative. Routine screening is recommended in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.  

COMPARISON: None.  

FINDINGS:

Left breast: 
The left breast is heterogeneously dense (type C). There are scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications.

Right breast:
The right breast is heterogeneously dense (type C). In the upper outer quadrant, there is a new group of pleomorphic microcalcifications spanning an area of approximately 1 cm. The calcifications are grouped, fine linear and branching in morphology. No associated mass, architectural distortion or skin thickening.

IMPRESSION:
New group of suspicious microcalcifications in the right upper outer quadrant, BI-RADS 4. Stereotactic biopsy is recommended for further evaluation. The left breast is BI-RADS 2, benign.

ADDENDUM: The patient returned for stereotactic biopsy of the right breast microcalcifications. Pathology results showed ductal carcinoma in situ, intermediate grade with comedo necrosis. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
INDICATION: Screening. Patient with family history of breast cancer. 
COMPARISON: Prior exam 2 years ago.
FINDINGS:
The left breast is heterogeneously dense. No suspicious masses, architectural distortion, or microcalcifications.  
Targeted ultrasound to the left breast upper outer quadrant: No suspicious mass or cystic lesion corresponding to the palpable area of concern. 
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend continued screening mammography. 
2. No sonographic correlate for the palpable finding in the left upper outer breast. 
ASSESSMENT: 
Left breast, BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Both breasts show scattered fibroglandular densities. The breasts are symmetric. 
No dominant masses, architectural distortion or suspicious clustered calcifications.
Stable benign calcifications are noted in the left breast.
No suspicious calcifications are identified.
There are no significant changes compared to prior mammograms.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Benign calcifications left breast, stable.
3. Recommend continued annual screening mammography.

BI-RADS Category 2: Benign Finding. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening in patient with prior history of right breast cancer, s/p lumpectomy 2015.
TECHNIQUE: Bilateral CC, MLO and ML views.  
COMPARISON: Prior exam 2021. 
FINDINGS: 
The breasts are heterogeneously dense which could obscure detection of small masses. 
Left breast: There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Right breast: Stable postoperative changes from prior lumpectomy are noted in the upper outer quadrant. Additionally, there is a new 8 mm cluster of pleomorphic microcalcifications in the lower inner quadrant at 4:00, 6 cm from the nipple. No associated mass is seen.
Stable bilateral benign-appearing lymph nodes are present. 
IMPRESSION:
BI-RADS 2: Benign exam left breast. Annual screening recommended.
BI-RADS 4: Suspicious microcalcifications right breast at 4:00, 6 cm from nipple. Stereotactic biopsy recommended. Please send requisition.
ADDENDUM: 
Stereotactic biopsy of the right breast microcalcifications at 4:00 was performed. Specimen radiograph confirmed calcifications. Pathology showed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. Wide local excision is recommended.
Post-biopsy mammogram showed clip in good position at the biopsy site with no residual suspicious calcifications.
BI-RADS 6: Known biopsy-proven malignancy right breast. Surgical consultation recommended. ➡ *Positive*
- SCREENING MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortions, or suspicious calcifications.

No significant interval change since the prior study.

IMPRESSION:
Negative screening mammogram. 

BIRADS Category 1: Negative. Routine screening is recommended in 1 year. ➡ *Not Stated*
- FINDINGS: Both breasts contain heterogeneously dense fibroglandular tissue. No dominant mass, architectural distortion, or suspicious calcifications are identified in either breast. Stable bilateral benign-appearing calcifications are noted. There are no new mammographic findings compared to prior examinations.

Bilateral targeted ultrasound was performed. No sonographic correlate for the palpable abnormality in the right breast is identified. No suspicious masses or other findings are seen in either breast.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. No sonographic abnormality in the area of palpable finding in the right breast. Clinical correlation recommended. 

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAMS
HISTORY: Screening mammogram. No current breast complaints. No family history of breast cancer.
FINDINGS: The breasts are predominantly fatty with scattered areas of fibroglandular density.  
In the right breast there are no dominant masses, architectural distortion or suspicious calcifications.
At the 3 o'clock position of the left breast, 6 cm from the nipple, there is a new 5 mm cluster of pleomorphic calcifications. No other suspicious findings in the left breast. 
No suspicious right axillary lymphadenopathy. Benign appearing left intramammary lymph node.
IMPRESSION:
New cluster of suspicious calcifications in the left breast. Additional magnification views are recommended for further evaluation. BI-RADS 0: Incomplete. 
No suspicious findings in the right breast. BI-RADS 2: Benign. 
ADDENDUM: 
Magnification views of the left breast in the CC and MLO projections confirm a 6 mm cluster of pleomorphic calcifications at the 3 o'clock position, 6 cm from the nipple. Stereotactic core biopsy is recommended for tissue diagnosis. BI-RADS 4: Suspicious. ➡ *Positive*
- ULTRASOUND LEFT BREAST AND MAMMOGRAM

HISTORY: 48-year-old female presents for follow-up of a palpable left breast lump, first noticed 2 months ago. No prior mammograms available for comparison. 

FINDINGS:
Mammogram: The breasts are heterogeneously dense. No dominant mass, architectural distortion, or suspicious calcifications are identified in either breast. 
  
Ultrasound: Targeted ultrasound was performed over the palpable area of concern at the 10:00 position of the left breast, 4 cm from the nipple. This demonstrates a 0.9 x 0.7 x 0.6 cm oval, circumscribed, hypoechoic solid mass, with no posterior acoustic features. This corresponds to the palpable finding. No axillary lymphadenopathy.

IMPRESSION:
1. Left breast 0.9 cm circumscribed solid mass at 10:00, 4 cm from the nipple, corresponding to the palpable finding. Differential diagnosis includes fibroadenoma, other benign etiology. Recommend biopsy for definitive diagnosis given palpable nature. 
2. Heterogeneously dense breasts with no suspicious mammographic findings. 

BIRADS 4a: Suspicious finding, biopsy should be considered.

ADDENDUM:
Ultrasound guided core biopsy of the 10:00 left breast mass was performed. Pathology revealed a benign fibroadenoma. 6-month follow-up ultrasound is recommended to document stability.

FINAL BIRADS 2: Benign finding. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening mammogram. No prior mammograms available for comparison.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
There is a small cluster of amorphous microcalcifications at the 12 o'clock position in the left breast, approximately 6 cm from the nipple. This cluster measures approximately 5 mm in greatest dimension. There are no associated masses, architectural distortion or skin changes.
IMPRESSION:
BI-RADS 0: Incomplete. There is a small cluster of amorphous microcalcifications in the left breast, which is new since the prior exam and requires additional imaging evaluation with magnification views for complete characterization.
RECOMMENDATION: Recall for diagnostic evaluation with magnification views of the left breast at the 12 o'clock position.
ADDENDUM - DIAGNOSTIC MAMMOGRAM LEFT BREAST, MAGNIFICATION VIEWS:
Magnification views of the left breast at 12 o'clock position demonstrate a cluster of amorphous microcalcifications, which are more apparent compared to the screening views. This cluster measures approximately 8 mm. No associated mass or architectural distortion.
FINAL ASSESSMENT: BI-RADS 4B: Suspicious abnormality. Biopsy should be considered.
RECOMMENDATION: Stereotactic core biopsy is recommended for definitive characterization of this cluster of microcalcifications in the left breast. ➡ *Positive*
- TMIST MAMMOGRAM, FFDM ARM, REB APPROVED TRIAL.
This patient is a participant in the TMIST study, full field digital mammography arm, under REB approval with informed consent. 
Comparison to prior exam: Yes.
Breast tissue density: Less than 75%.
Greater than 75% density: No. 
OBSP routine screening interval of 1 year is not recommended.
No features suspicious for malignancy are identified in either breast on this screening mammogram.
Final Assessment: BIRADS Category 1 or 2, benign/negative. ➡ *Not Stated*
- FINDINGS:
Bilateral digital screening mammogram. 
The breasts are heterogeneously dense. 
In the right breast at the 12:00 position 6 cm from the nipple is a group of amorphous calcifications measuring about 8 mm. These are new compared to the previous mammogram from 1 year ago. A few punctate calcifications are seen at 6:00 in the left breast. Otherwise no suspicious masses, calcifications, architectural distortion or asymmetries in either breast.

IMPRESSION:  
BI-RADS 4: Suspicious finding right breast. Stereotactic core biopsy recommended for calcifications at 12:00 right breast. Likely benign finding left breast, 6 month follow up recommended for calcifications at 6:00 left breast. 

ADDENDUM: Stereotactic core biopsy right breast 12:00 position performed. Specimen radiograph confirms calcifications. Pathology pending. ➡ *Positive*
- BILATERAL MAMMOGRAM
INDICATION: Routine screening. No current breast complaints.
COMPARISON: Mammogram from 1 year prior.
FINDINGS:
Right breast: The breast is composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast: The breast is composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications. 
IMPRESSION:
1. Negative bilateral mammogram. 
2. No mammographic evidence of malignancy. Routine annual screening mammography is recommended.
BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening mammogram.
COMPARISON: None available.
BREAST COMPOSITION: Scattered fibroglandular densities.
FINDINGS: The breasts are predominantly fatty replaced.
There are no suspicious masses, calcifications, or architectural distortions.
IMPRESSION:
BIRADS Category 1: Negative.
Routine screening recommended in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM
 
INDICATION: Screening.
 
TECHNIQUE: Standard views of each breast including CC and MLO were obtained.
 
COMPARISON: Comparison is made to prior studies from May 2019.
 
FINDINGS: 
Both breasts are predominantly fatty with scattered fibroglandular densities. 

No dominant masses, focal asymmetries or areas of architectural distortion. 

No suspicious calcifications are identified. 

No significant interval change compared to prior exam.

IMPRESSION:

BIRADS 2- Benign/Negative screening mammogram. No suspicious findings.

 
RECOMMENDATION: Routine screening mammography in 1 year per ACR guidelines. ➡ *Not Stated*
- FINDINGS:
The breasts are heterogeneously dense. There are scattered areas of fibroglandular density.
In the left breast at 12:00 there is a 0.5 cm cluster of pleomorphic microcalcifications, appearing new compared to previous studies.
At 6:00 posterior depth left breast there is a 1.2 cm irregular hypoechoic mass with indistinct margins and internal vascularity. This corresponds to an area of architectural distortion on mammography.
In the left axilla there are two abnormal appearing lymph nodes with eccentric cortical thickening, measuring up to 1.5 cm in short axis diameter.
The remainder of the left breast is unremarkable. 
The right breast is unremarkable, with no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. New cluster of pleomorphic microcalcifications at 12:00 left breast, suspicious for DCIS. Biopsy recommended. BI-RADS 4B.
2. 1.2 cm irregular mass at 6:00 left breast with associated architectural distortion, highly suspicious for malignancy. BI-RADS 5. Biopsy recommended.
3. Abnormal left axillary lymph nodes suspicious for regional nodal metastases.  
4. Benign right breast. BI-RADS 2. ➡ *Positive*
- ULTRASOUND LEFT BREAST

CLINICAL HISTORY: 55-year-old female with history of breast cancer, now with a new palpable lump in the upper outer quadrant of the left breast. 

TECHNIQUE: Targeted ultrasound of the left breast was performed using a high frequency linear transducer.

COMPARISON: None.

FINDINGS: The left breast is heterogeneously echogenic. Targeted ultrasound of the upper outer quadrant demonstrates normal appearing glandular and fibrous tissue. No discrete solid or cystic mass, area of architectural distortion, or suspicious calcifications are identified in this region. 

The remainder of the left breast is scanned and there are no sonographic findings to suggest malignancy. No abnormal axillary lymph nodes are seen.

IMPRESSION:
No sonographic evidence of malignancy in the left breast. Specifically, no findings to explain the palpable abnormality in the upper outer quadrant. Recommend clinical follow up and diagnostic mammogram for further evaluation. 

BI-RADS Category 1: Negative. ➡ *Negative*
- REFERENCE MAMMOGRAM: SCREENING DIGITAL BILATERAL June 5, 2020

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views bilaterally. 

COMPARISON: Jun 16, 2019.

FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography. 

There are no dominant masses, architectural distortion or suspicious calcifications.

IMPRESSION: 
BIRADS 2 - Benign.

1 year SCREENING RECOMMENDED (OBSP).

ADDENDUM:
Patient notified of results via letter. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM
This client is a participant in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), with approval from the Research Ethics Board and informed consent obtained. She was randomly allocated to the full field digital mammography only arm of the study.  
Comparison with previous imaging: Yes.
Breast tissue density: Scattered fibroglandular density.
OBSP routine screening interval: Annual.
No mammographic abnormalities suspicious for malignancy are identified. Stable benign-appearing calcifications are noted bilaterally. 
BIRADS assessment category: 2 - Benign findings. Routine screening recommended. ➡ *Not Stated*
- REPORT:
HISTORY: 64 y/o female patient presenting for screening mammogram. No palpable lumps, no nipple discharge, no breast pain. Patient has a personal history of breast cancer with right mastectomy and reconstruction performed 5 years back. No family history of breast cancer. 
COMPARISON: Prior mammogram 2 years back.
FINDINGS:
Right reconstructed breast is unremarkable.
The left breast is composed predominantly of fatty tissue. There is no mammographic evidence of malignancy. No suspicious masses, architectural distortion or calcifications are identified. No significant change since prior mammogram.
IMPRESSION:
BI-RADS Category 1: Negative. No mammographic evidence of malignancy in the left breast. Recommend continued annual screening mammogram in 12 months. Follow up as clinically indicated.
ADDENDUM: Retrospective review of the studies do not reveal any suspicious microcalcifications in the left breast. ➡ *Negative*
- "FULL FIELD DIGITAL MAMMOGRAM SCREENING STUDY.
Bilateral digital screening mammograms were obtained.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
Comparison: Prior mammograms dated Oct 15, 2018 are available for comparison.
Findings: 
No significant masses, calcifications, or other findings are identified.
IMPRESSION:
BIRADS Category 2: Benign. Recommend routine screening in one year.

Relevant Prior Studies:
MRI BREAST dated Nov 01, 2021: Benign findings. 
" ➡ *Not Stated*
- FINDINGS:

RIGHT BREAST: Negative.

LEFT BREAST:
Scattered areas of fibroglandular density.
No dominant mass, architectural distortion or suspicious calcifications.

IMPRESSION:
No mammographic evidence of malignancy.
BI-RADS category 1: Negative.
Routine screening mammography is recommended in 1 year. ➡ *Negative*
- RADIOLOGY REPORT
CLINICAL HISTORY: Annual screening mammogram. No clinical concerns reported by patient.
COMPARISON: Comparison is made to prior mammograms from 05/18/2022 at this facility. 
FINDINGS: 
Both breasts are composed of scattered fibroglandular densities. 
There are no dominant masses, architectural distortions, or suspicious calcifications in either breast. The skin is unremarkable. 
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Scattered fibroglandular densities are noted bilaterally, consistent with ACR density category C.
2. Recommend continued annual screening mammography in conjunction with monthly breast self-exams and annual clinical breast exams.
BIRADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening
Comparison available dating from 07/2022.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- ULTRASOUND RIGHT BREAST AND MAMMOGRAM
CLINICAL HISTORY: Right breast microcalcifications seen on screening mammogram. Assess. Rule out underlying mass.
TECHNIQUES: Bilateral mediolateral oblique and craniocaudal views with spot compression views in the upper outer right breast have been performed.
COMPARISON: With prior mammograms, right breast focal compression views and right whole breast ultrasound of September 30, 2021.
FINDINGS:
The breast tissue is heterogeneously dense which may obscure small masses on mammography.
Several clusters of pleomorphic microcalcifications are noted in the upper outer quadrant of the right breast with the most suspicious group located at 3 o'clock, 6 cm from the nipple. The calcifications have increased in number and are more conspicuous compared to the prior mammograms.
In the upper outer right breast in the area of calcifications, targeted ultrasound shows no sonographic correlate. The remainder of the right breast was scanned and no suspicious mass or area of architectural distortion is identified sonographically.
The previously noted 6 x 4 mm cyst at 9 o'clock right breast is stable. No other new or suspicious findings in the right breast.
The left breast demonstrates no suspicious mammographic findings.
IMPRESSION:
RIGHT BREAST: BI-RADS Category 4B: Suspicious microcalcifications in the upper outer right breast. Stereotactic biopsy is recommended. Requisition to be faxed to breast imaging.
LEFT BREAST: BI-RADS Category 1: Normal. Annual screening recommended. ➡ *Positive*
- LEFT MAMMOGRAM AND ULTRASOUND
INDICATION: Focal thickening in the medial left breast per outside physician exam. Family history of breast cancer in paternal aunt. Prior imaging from outside facility.
FINDINGS:
The left breast is heterogeneously dense, which may obscure detection of small masses. There is no mammographic evidence of malignancy. No suspicious clustered microcalcifications, dominant mass, or area of focal asymmetry.
Focused left breast ultrasound was performed over the area of clinical concern in the medial left breast. No sonographic correlate for the palpable finding. No suspicious mass, complex cystic lesion, or enlarged axillary lymph nodes appreciated.
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend annual screening mammography in 1 year.
2. No sonographic correlate for the focal thickening in the medial left breast. Recommend clinical follow up and correlation in 3 months. Additional imaging may be obtained as clinically warranted.
ASSESSMENT: BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS: The left breast appears heterogeneously dense. No suspicious masses, architectural distortions, or dominant masses. Specifically, no abnormalities in the area of palpable concern at the 6 o'clock position of the left breast. There are no suspicious microcalcifications. Benign calcifications are noted.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. No sonographic correlate for the palpable abnormality at 6 o'clock position of the left breast. 

BI-RADS Category 1: Negative. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAM REPORT
CLINICAL HISTORY: 45-year-old female presents for screening mammogram. No current breast complaints. 
FINDINGS:
Both breasts are composed of scattered fibroglandular densities. 
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. 
Stable benign calcifications are noted in the right breast, unchanged from prior exam.
No suspicious axillary lymphadenopathy.
IMPRESSION: 
BI-RADS 2 - Benign findings. Recommend routine screening in 1 year.
ADDENDUM: Comparison was made to outside mammogram from 11/15/2021, which showed stable scattered fibroglandular densities and benign calcifications in the right breast. No change from prior exam. ➡ *Negative*
- Breast Ultrasound Report
Patient Name: [Patient Name]
Date of Exam: 2024-07-06

Clinical History:
Screening breast ultrasound for new microcalcifications seen on recent mammogram. Patient is asymptomatic.

Comparison: Mammogram performed on 06/28/2024.

Findings:
Left Breast: No suspicious masses, calcifications, or other abnormalities are seen in the left breast.
Right Breast: Correlating with the mammographic finding in the right breast at 3 o'clock, approximately 2 cm from the nipple, there is a small, ill-defined area of hypoechogenicity measuring approximately 7 mm in greatest dimension. This area corresponds to the cluster of microcalcifications seen on mammography. No distinct mass is identified. There is minimal vascularity within this area on color Doppler imaging. No associated ductal dilation is seen.

Impression:
The ultrasound findings in the right breast are consistent with the mammographically detected microcalcifications. Although no distinct mass is identified, the presence of the ill-defined hypoechoic area raises some concern. Given the concerning mammographic features, biopsy is recommended for definitive diagnosis.

Recommendations:
Ultrasound-guided biopsy of the right breast microcalcifications.

BI-RADS Category: 4 (Suspicious for Malignancy) ➡ *Positive*
- MAMMOGRAPHY REPORT

CLINICAL HISTORY: 50 y.o. female presenting for screening mammogram. No prior mammograms available for comparison.

FINDINGS:
Both breasts are predominantly fatty replaced.

LEFT BREAST:
There is a 1 cm cluster of pleomorphic microcalcifications in the upper outer quadrant at approximately 2 o'clock, 7 cm from the nipple. No associated mass, architectural distortion or skin thickening.

RIGHT BREAST:  
No suspicious masses, calcifications, or areas of architectural distortion.

IMPRESSION:
LEFT BREAST: 
1 cm cluster of pleomorphic microcalcifications in the upper outer quadrant. Additional magnification views are recommended for further characterization. BIRADS 0.

RIGHT BREAST:  
Negative. BIRADS 1.

ADDENDUM:
Magnification views of the left breast demonstrate a 1 cm cluster of pleomorphic microcalcifications in the upper outer quadrant. Stereotactic biopsy is recommended for tissue diagnosis. BIRADS 4B. ➡ *Positive*
- ULTRASOUND LEFT BREAST

INDICATION: New palpable lump left breast upper outer quadrant. Post menopausal.

FINDINGS:
Overall heterogeneously dense breast tissue. No suspicious mass, architectural distortion or suspicious calcifications. 

Targeted left 2 o'clock palpable area of concern demonstrates normal fibroglandular tissue with no discernible mass. Underlying pectoralis muscle is intact. No axillary lymphadenopathy.

IMPRESSION:
No sonographic evidence of malignancy.
BIRADS 1 - Negative. 

RECOMMENDATIONS:
Continue annual screening mammography as the breast tissue is heterogeneously dense which can obscure small masses. Any new palpable lump should be immediately evaluated. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.

There are no dominant masses, architectural distortion or suspicious calcifications in either breast.

Multiple scattered benign-appearing calcifications are noted bilaterally, more prominent in the right breast. In the right breast at the 9 o'clock position, 6 cm from the nipple, there is a 0.5 cm cluster of pleomorphic calcifications. No associated mass. These are new compared to prior exams.

No abnormal axillary lymph nodes.

IMPRESSION:
BI-RADS 4: Suspicious abnormality in the right breast. Biopsy should be considered.

RECOMMENDATION:
Right breast 9 o'clock stereotactic core biopsy is recommended for the cluster of pleomorphic calcifications. A requisition will be sent to the department to arrange this.

ADDENDUM:
The results of the right breast biopsy showed ductal carcinoma in situ, intermediate grade, cribriform type. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- SCREENING MAMMOGRAM
INDICATION: Screening. High risk patient.
COMPARISON: 6/15/2019
TECHNIQUE: Bilateral digital mammography was performed including 2D synthesized and 3D tomosynthesis images.
FINDINGS:
Both breasts show scattered areas of fibroglandular density. There are no dominant masses, suspicious calcifications, or areas of architectural distortion bilaterally. No significant interval change compared to prior mammogram.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. No mammographic evidence of malignancy.
3. Recommend continued annual screening mammography in one year given high risk status.
ASSESSMENT: Negative. BI-RADS Category 1.
ADDENDUM:
Whole breast ultrasound of both breasts was also performed today. No suspicious masses or findings were identified sonographically in either breast. Specifically, no abnormalities were seen in the 6 o'clock position of the left breast, corresponding to the patient's palpable finding on self-exam. Clinical correlation recommended for palpable abnormality. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM. Clinical history: 51 year old female presents for her annual screening mammogram. Comparison is made to prior mammogram from 1/2/2024. Findings: The breasts are heterogeneously dense. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: Negative mammogram. BI-RADS 1. ➡ *Negative*
- SCREENING BILATERAL DIGITAL MAMMOGRAPHY.

INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of the breasts were obtained.

COMPARISON: None available.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

There are scattered areas of benign-appearing calcifications in both breasts, including:
- Vascular calcifications bilaterally.
- Coarse calcifications in the central right breast, likely representing an oil cyst.
- Layering rod-like calcifications at 12:00, 5 cm from the nipple in the left breast, likely milk of calcium within a dilated duct.
- Pleomorphic microcalcifications in a segmental distribution at 2:00, 7 cm from the nipple in the left breast, spanning approximately 12 mm.

No suspicious axillary lymphadenopathy.

IMPRESSION:
1. Heterogeneously dense breasts (C).
2. Scattered benign-appearing calcifications bilaterally as described above.
3. Segmental pleomorphic microcalcifications in the left breast (2:00, 7 cm from nipple), suspicious for ductal carcinoma in situ (DCIS). Additional diagnostic mammographic views are recommended for further evaluation. Ultrasound is also recommended to assess for an associated mass.

BIRADS 0: Incomplete. Additional imaging evaluation needed.

ADDENDUM (post-diagnostic evaluation):
Additional spot compression magnification views of the left breast confirms a segmental distribution of fine pleomorphic microcalcifications at 2:00, 7 cm from the nipple, spanning 15 mm. Targeted left breast ultrasound demonstrates a vague area of shadowing and distortion corresponding to the microcalcifications, but no definite mass. Stereotactic biopsy is recommended for tissue diagnosis.

FINAL ASSESSMENT: BI-RADS 4C: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAMS
HISTORY: Annual screening mammogram. No prior mammograms available for comparison.
FINDINGS:
Both breasts are primarily fatty replaced. 
There are no dominant masses, architectural distortion or skin thickening in either breast.
In the upper outer right breast, there is a new cluster of amorphous microcalcifications spanning an area of approximately 1.0 cm at the 10:00 position, 5 cm from the nipple. Additional magnification views in the CC and ML projections confirm the presence of the microcalcifications. 
The remainder of the right breast is unremarkable.
The left breast is unremarkable. No suspicious microcalcifications, masses, or areas of architectural distortion.
IMPRESSION:
BI-RADS Category 0: Incomplete: Need additional imaging evaluation. 
New cluster of amorphous microcalcifications in the upper outer right breast. Recommend diagnostic mammogram workup with spot compression magnification views for further evaluation.
ADDENDUM:
The patient returned for additional diagnostic imaging of the right breast, including spot compression magnification views of the area of microcalcifications in the upper outer right breast.
The magnification views demonstrate a cluster of amorphous microcalcifications spanning approximately 10 mm at the 10:00 position of the right breast, 5 cm from the nipple. The microcalcifications are new compared to the patient's prior mammograms.
IMPRESSION (ADDENDUM):
BI-RADS Category 4B: Suspicious abnormality, biopsy should be considered. Suspicious cluster of amorphous microcalcifications in the right breast. Stereotactic biopsy is recommended for further evaluation. ➡ *Positive*
- Addendum to Breast MRI

Reason for Exam: Further evaluation of microcalcifications seen on mammography

Comparison: Mammogram 09-15-2024

History: Patient presents with a new cluster of microcalcifications seen on screening mammogram.

Findings:
The previously described cluster of microcalcifications in the right breast at 10 o'clock, 2 cm from the nipple, demonstrates type II enhancement kinetics on the post-contrast subtracted images, suspicious for malignancy. No other suspicious enhancement, masses, or areas of diffusion restriction are identified in either breast.

Impression:
Enhancement of known microcalcifications in the right breast concerning for malignancy. Correlation with mammographic findings and clinical follow-up is recommended. Biopsy should be considered.

BI-RADS: 4 (Suspicious for Malignancy) ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available

FINDINGS:
Both breasts show a scattered fibroglandular tissue pattern.

At the 10:00 position of the RIGHT breast, approximately 6 cm from the nipple, there is a cluster of pleomorphic microcalcifications spanning an area of about 8 mm. No associated mass, architectural distortion or skin changes are seen. 

The LEFT breast shows no suspicious masses, calcifications, architectural distortion or skin thickening.  

No abnormal axillary lymph nodes are seen on either side.

IMPRESSION: 
BI-RADS Category 4: Suspicious abnormality in the RIGHT breast - biopsy should be considered. 
BI-RADS Category 1: Negative in the LEFT breast.

RECOMMENDATION: 
Stereotactic core biopsy of the cluster of microcalcifications in the RIGHT breast at 10:00, 6 cm from nipple.
Routine screening of the LEFT breast with mammography in 1 year. ➡ *Positive*
- SCREENING MAMMOGRAM.
CLINICAL INDICATION: Annual screening mammography.
COMPARISON: Screening mammogram from May 2023.
TECHNIQUE: Standard CC and MLO views obtained bilaterally.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no suspicious masses, architectural distortion, or suspicious calcifications.
There are stable scattered fibroglandular densities and areas of benign-appearing calcifications bilaterally, likely representing fibrocystic changes.
Stable postoperative changes noted in the right breast, consistent with history of prior lumpectomy.
No significant interval change compared to prior examination.
IMPRESSION:
1. No mammographic evidence of malignancy.
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI as clinically appropriate.
BIRADS 1 - Negative.
FOLLOW-UP RECOMMENDATIONS: Recommend continued annual screening mammography and clinical breast exams. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS
History: Annual screening mammogram
FINDINGS: The breasts are heterogeneously dense.
There are no suspicious right mammographic findings. 
In the left breast at 3:00, 10 cm from the nipple there is a 1 cm irregular mass, partially obscured. Spot compression views show the mass to have an irregular margin. Additionally at 2:00, 12 cm from the nipple there is a group of pleomorphic microcalcifications in a linear distribution measuring 7 mm. Remainder of left breast is unremarkable. 
SUMMARY:
Left breast: 1 cm irregular mass at 3:00, 10 cm from nipple, suspicious for malignancy. 7 mm group of pleomorphic microcalcifications at 2:00, 12 cm from nipple, suspicious for malignancy. BI-RADS 4C. Recommend biopsy.
Right breast: No suspicious findings. BI-RADS 1. Recommend routine screening in 1 year. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAM
INDICATION: Routine screening mammography. No complaints.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
In the right breast, there is a group of punctate microcalcifications in the upper outer quadrant, approximately 2 cm from the nipple, clustered in an area measuring 6 x 4 mm. These calcifications are new compared to prior study and are suspicious for malignancy. 
No other significant findings in the right breast.
No significant findings in the left breast.
IMPRESSION:
BI-RADS 4: Suspicious grouped microcalcifications in the right breast at 10 o'clock, 2 cm from the nipple. Biopsy is recommended for further evaluation. 
BI-RADS 2 for the remainder of both breasts: Benign findings. Recommend routine screening in 1 year. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAPHY WITH TOMOSYNTHESIS
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints. 
COMPARISON: Mammogram from 1 year ago.
FINDINGS:
Both breasts are predominantly fatty.
Right breast: Negative. 
Left breast: New group of clustered amorphous microcalcifications in the upper outer quadrant, 8 cm from the nipple. Additional magnification views in the mediolateral and craniocaudal projections confirm the presence of suspicious microcalcifications spanning an area of approximately 8mm. No associated mass, architectural distortion or skin changes.
Bilateral axillae are unremarkable. No abnormal intramammary lymph nodes. 
IMPRESSION: 
RIGHT BREAST: BI-RADS 1 - Negative. 
LEFT BREAST: BI-RADS 4 - Suspicious abnormality. Suspicious microcalcifications in the upper outer left breast. Biopsy is recommended for definitive diagnosis.
OVERALL: BI-RADS 4 - Suspicious abnormality. Stereotactic biopsy of the left breast microcalcifications is recommended. Please complete the attached biopsy requisition form and fax to the booking office. The patient will be contacted with an appointment. ➡ *Positive*
- FINDINGS: The left breast is composed of scattered fibroglandular tissue. No dominant mass, architectural distortion, or suspicious calcifications are identified.

Right breast ultrasound was performed at the site of palpable abnormality in the upper outer quadrant. No correlating mass or other suspicious finding was identified in this region. 

IMPRESSION:
1. Negative bilateral screening mammograms. No mammographic evidence of malignancy.
2. Targeted right breast ultrasound is negative at the site of palpable finding. 

BIRADS 1 - Negative.

1 year routine follow up is recommended in the absence of new signs or symptoms. ➡ *Negative*
- ULTRASOUND AND MAMMOGRAM REPORT.
HISTORY: 53-year-old female presents with 1 month history of right axilla pain. No palpable axillary mass on exam. Prior imaging 1 year ago was normal. No family history of breast or ovarian cancer.
FINDINGS: 
MAMMOGRAM: The right and left breasts are predominantly fatty with scattered areas of fibroglandular tissue. There are no dominant masses, suspicious calcifications or architectural distortion. No significant change from prior exam. 
ULTRASOUND: Targeted right axillary ultrasound was performed. The right axilla shows normal appearing lymph nodes with thin cortices and preserved fatty hila. No abnormal or enlarged lymph nodes are identified.
IMPRESSION:
1. Negative bilateral screening mammogram. No mammographic evidence of malignancy. Recommend routine annual screening in 1 year.
2. Normal targeted right axillary ultrasound. No sonographic correlate for patient's right axillary pain. 
BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS: Left breast tissue contains heterogeneously dense fibroglandular tissue. No suspicious masses, architectural distortion, or microcalcifications are identified. Stable appearance compared to prior mammograms.

Right breast tissue is mostly fatty. No suspicious masses, architectural distortion, or microcalcifications are identified. Stable appearance compared to prior mammograms.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Specifically, no suspicious microcalcifications are seen.
2. Heterogeneously dense breast tissue in the left breast, which may lower the sensitivity of mammography.

RECOMMENDATION:
Continue annual screening mammography and clinical breast exams. Consider supplemental screening with breast MRI or whole breast ultrasound given the patient's dense breast tissue.

BIRADS 1 ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.

COMPARISON: Screening mammogram from April 2019.

FINDINGS:
The breasts are almost entirely fatty.

There are no dominant masses, areas of architectural distortion or asymmetries in either breast.

In the left breast at the 6 o'clock position, 8 cm from the nipple, there is a new 6 mm cluster of pleomorphic microcalcifications. This is a suspicious finding and warrants further evaluation.

No other significant interval change.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Biopsy should be considered.

RECOMMENDATIONS:
Stereotactic core biopsy is recommended for the new cluster of microcalcifications in the left breast at 6 o'clock, 8 cm from the nipple. The patient should call to schedule this procedure.

A 6 month follow up mammogram is recommended for the left breast following biopsy to evaluate for any additional changes. ➡ *Positive*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
INDICATION: Screening mammogram. No clinical concerns.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Targeted ultrasound was performed at the 10 o'clock position. 
COMPARISON: Prior studies from 12/15/2021 at ABC Imaging Center.
FINDINGS:
The breast is heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Ultrasound of the left breast at 10 o'clock, over the area of patient concern, shows normal fibroglandular tissue with no discrete solid or cystic lesions.
Stable benign-appearing calcifications are scattered in the central and upper outer breast, likely vascular in nature.
IMPRESSION:
1. No evidence of malignancy in the left breast. 
2. Stable benign-appearing calcifications, likely vascular.
3. Heterogeneously dense breast tissue.
RECOMMENDATION: Recommend continued annual screening mammography and clinical breast exams. 
BIRADS Category 1: Negative. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Screening.
COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense. No suspicious masses, architectural distortion, or asymmetries are identified. No suspicious clustered microcalcifications are seen. The skin and nipples are unremarkable. No axillary lymphadenopathy is evident.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts, which may obscure detection of small masses.

ASSESSMENT CATEGORY: BI-RADS 1 - Negative.

MANAGEMENT: Recommend continued annual screening mammography in 12 months.

ADDENDUM: 
This study is a screening exam. The patient's history of microcalcifications, if any, are not known or stated on this report. ➡ *Not Stated*
- MAMMOGRAM - BILATERAL.

INDICATION: Annual screening mammogram. No focal areas of pain or palpable abnormality.

TECHNIQUE: Standard CC and MLO views of each breast were obtained. 

COMPARISON: Comparison is made to prior mammograms from 7/15/2021 at this facility.

FINDINGS:
The breast tissue is predominantly fatty, consistent with BI-RADS density category a.

There are no dominant masses, architectural distortions, or suspicious calcifications in either breast. Specifically, no new or enlarging masses or microcalcifications are identified.

Stable benign-appearing calcifications are again noted in the right breast, likely vascular in nature.

There are no significant changes compared to prior mammograms.

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Recommend continued annual screening mammography in 12 months.

BI-RADS Category 1 - Negative. ➡ *Negative*
- ULTRASOUND LEFT BREAST

INDICATION: Palpable lump left breast upper outer quadrant. Postmenopausal patient with lump felt on regular self breast exam. Last mammogram >2 years ago.

TECHNIQUE: High resolution real-time ultrasound evaluation of the left breast using a 12 MHz linear transducer.

COMPARISON: None available.

FINDINGS:
The left breast was thoroughly evaluated. The breast tissue is predominantly fatty with scattered fibroglandular tissue.

At the site of the palpable lump in the 2 o'clock position of the left breast, 3 cm from the nipple, there is a 0.9 x 0.6 x 0.5 cm oval hypoechoic mass, oriented parallel to the skin. It demonstrates circumscribed margins. No posterior acoustic features are seen. 

No additional masses, areas of architectural distortion, or suspicious calcifications are identified in the remainder of the left breast. No abnormal axillary lymph nodes are seen.

IMPRESSION:
1. 9 mm oval circumscribed hypoechoic mass at site of palpable abnormality, 2 o'clock position left breast, most likely representing a fibroadenoma. Recommend follow up diagnostic mammogram for further evaluation and correlation.

2. No suspicious calcifications, masses or architectural distortion identified in the remainder of the left breast.

BIRADS 3 - Probably benign finding. Short interval follow up suggested. ➡ *Negative*
- ROUTINE MAMMOGRAPHY FOR RESEARCH PARTICIPANT        
Clinical information: Patient enrolled in TMIST randomized clinical trial. Assigned to digital screening mammography arm.

FINDINGS:
Comparison is made to prior mammogram from external facility.

Breast density is heterogeneously dense, which may obscure small masses. Breast density is less than 75%.

No dominant mass, distortion, or suspicious calcifications are identified in either breast.

Stable scattered vascular and benign calcifications are noted in both breasts.

IMPRESSION: 
BIRADS Category 2: Benign findings. Recommend routine screening in 12 months per TMIST trial protocol. 

No statement can be made about the presence or absence of microcalcifications due to the limitations of this screening study. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No suspicious masses, architectural distortion or asymmetries.
BIRADS-2.

Addendum:
Comparison with previous studies shows stable breast tissue with no interval change. The patient is encouraged to continue routine screening mammography. ➡ *Not Stated*
- FINDINGS: Right breast tissue is heterogeneously dense. There is no mammographic evidence of malignancy. No dominant mass, architectural distortion or suspicious calcifications. Stable benign calcifications in the upper outer quadrant, likely vascular in etiology. The left breast is not included on this study.

Targeted right breast ultrasound was performed at the site of palpable abnormality at 10 o'clock, middle depth. No sonographic correlate for the palpable finding. No suspicious solid or cystic mass.

IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. Recommend annual screening mammography in 1 year. 
2. No sonographic correlate for the patient's palpable right breast finding at 10 o'clock. Likely represents normal fibroglandular tissue. Clinical correlation recommended.

BI-RADS 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
There are no dominant masses, significant architectural distortion, or suspicious calcifications in either breast.
At the 10 o'clock position of the left breast, there is a scattered group of amorphous microcalcifications measuring 8 mm, located approximately 6 cm from the nipple. This is a new finding compared to the patient's previous mammogram from 2 years ago. Additional magnification views are recommended for further evaluation.
The right breast is unremarkable.
IMPRESSION:
1. New group of amorphous microcalcifications in the left breast at 10 o'clock, 6 cm from the nipple. Additional magnification views recommended. BI-RADS 0: Incomplete. 
2. No suspicious findings in the right breast. BI-RADS 2: Benign.
ADDENDUM:
Magnification views of the left breast demonstrate a group of amorphous microcalcifications spanning an area of 9 mm at the 10 o'clock position, 6 cm from the nipple. Biopsy is recommended for definitive diagnosis. BI-RADS 4: Suspicious. ➡ *Positive*
- REPORT:
"NORMAL BIRADS 2 LEFT SCREENING MAMMOGRAM. 
There are no mammographic findings of concern. The breast tissue is of scattered fibroglandular density.
There are no masses, architectural distortions or suspicious calcifications. No dominant masses. No definite abnormality. 
Compared to previous exam 2 years ago, no interval change.
IMPRESSION:
1. Negative digital screening mammogram.
2. Scattered fibroglandular densities.
BIRADS Category 2 - Benign findings. Recommend routine screening in 1 year.

ADDENDUM: Tomosynthesis images were also reviewed and show no abnormality. No change in impression or recommendations." ➡ *Not Stated*
- FINDINGS: The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No dominant masses, architectural distortion or suspicious calcifications are identified. 

There are no significant changes compared to prior mammogram from 1/15/2021.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend annual screening mammography in 12 months.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
INDICATION: Left upper outer quadrant palpable lump. 
PREVIOUS STUDIES: 2 years prior.
FINDINGS:  
Both breasts are composed of heterogeneously dense fibroglandular tissue. There is no evidence of dominant mass, suspicious calcifications, or architectural distortion bilaterally. No significant interval change from prior exam. 
IMPRESSION:
1. No mammographic evidence of malignancy.
2. Stable heterogeneously dense breast tissue. 
3. Clinical breast exam finding of palpable left upper outer quadrant lump, not definitively seen on mammogram. Recommend targeted ultrasound for further evaluation. 
BIRADS 1 - Negative mammogram. 
ULTRASOUND OF THE LEFT BREAST
INDICATION: Palpable abnormality left upper outer quadrant. Negative mammogram.
TECHNIQUE: High resolution linear transducer.
FINDINGS: Targeted ultrasound of the left upper outer breast quadrant demonstrates normal fibroglandular tissue. No discrete solid or cystic mass lesion identified in the area of clinical concern. The remainder of the left breast demonstrates normal echotexture and vascularity without suspicious findings. 
IMPRESSION: 
1. No sonographic evidence of malignancy. 
2. Negative targeted ultrasound of the left upper outer quadrant. No definite cause for palpable finding identified. Recommend clinical follow up and correlation with physical exam findings. Consider biopsy if clinically warranted.
BIRADS 1 - Negative ultrasound. ➡ *Negative*
- SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of bilateral breasts were obtained.

COMPARISON: Screening mammogram from 1 year prior.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications.

Stable bilateral scattered fibroglandular densities, likely benign.

No significant interval change compared to prior exam.

IMPRESSION:
No mammographic evidence of malignancy. 
BI-RADS Category 2: Benign findings.

1-year routine screening mammogram is recommended. ➡ *Not Stated*
- MAMMOGRAM LEFT BREAST.
HISTORY: Screening.
COMPARISONS: 09/15/2022.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- MAMMOGRAM AND BREAST ULTRASOUND
INDICATION: Left breast pain and heaviness. History of right breast cancer status post mastectomy.
TECHNIQUE: Standard bilateral mammogram views are obtained. Targeted ultrasound of the left breast was also performed.  
COMPARISON: Prior mammograms and ultrasound from 5/12/2022.
FINDINGS:
Right breast: Mastectomy changes are noted. No concerning findings.
Left breast: The breast is predominantly fatty replaced. No masses, architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications in the upper outer quadrant. No mammographic evidence of malignancy. 
Ultrasound of the left breast demonstrates scattered fibroglandular tissue. No solid or cystic masses. No sonographic correlate for the patient's symptoms.
IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. 
2. Stable right mastectomy site.
3. Recommend annual screening mammography. 
BI-RADS 1- Negative. ➡ *Negative*
- MAMMOGRAM, LEFT BREAST.
HISTORY: Annual screening. Patient has a personal history of right breast cancer, s/p lumpectomy in 1998 and mastectomy in 2003 with axillary dissection.
FINDINGS: Left craniocaudal and mediolateral oblique views demonstrate scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications. The skin is unremarkable. No adenopathy.
IMPRESSION: No evidence of malignancy in the left breast. Recommend annual screening mammography.
ADDENDUM: Comparison is made to an outside mammogram dated 12/27/2024, which was reviewed and also showed no suspicious findings in the left breast.
BIRADS Category 1: Negative. ➡ *Negative*
- SCREENING MAMMOGRAM
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.
At the 9 o'clock position of the right breast, approximately 6 cm from the nipple, there is a group of amorphous calcifications spanning an area of about 8 mm. These calcifications are new since the prior exam and are suspicious for early ductal carcinoma in situ (DCIS).
The left breast is unremarkable.
There are no enlarged axillary lymph nodes on either side.
IMPRESSION:
BIRADS 4: Suspicious grouped amorphous calcifications in the right breast at 9 o'clock, 6 cm from the nipple, spanning 8 mm. Biopsy is recommended for further evaluation, as these may represent early DCIS. The remainder of the examination is benign.
BIRADS 1: The left breast is negative.

ADDENDUM - STEREOTACTIC BIOPSY RIGHT BREAST
TECHNIQUE: Stereotactic biopsy of the right breast was performed targeting the group of amorphous calcifications at the 9 o'clock position, 6 cm from the nipple. Specimens were obtained and sent to pathology.
FINDINGS: Post-biopsy mammograms confirm that the targeted calcifications were sampled.
IMPRESSION: Successful stereotactic biopsy of the suspicious calcifications in the right breast. Pathology results are pending. ➡ *Positive*
- MAMMOGRAM, LEFT.
CLINICAL HISTORY: 48 year old female with palpable left breast lump at 10 o'clock.
COMPARISON: Left mammogram from 6 months prior.
FINDINGS: The breasts are heterogeneously dense which may obscure detection of small masses. No dominant mass, architectural distortion or suspicious calcifications are identified. Scattered areas of fibroglandular density are identified bilaterally. 
IMPRESSION: No mammographic evidence of malignancy. Recommend ultrasound for further evaluation of the palpable lump at 10 o'clock. 
BIRADS 1 - Negative.

ULTRASOUND, LEFT BREAST
CLINICAL HISTORY: Palpable left breast lump at 10 o'clock. Recent mammogram negative.
TECHNIQUE: Targeted ultrasound was performed over the area of concern using a 12 MHz linear transducer.  
FINDINGS: In the area of palpable concern at 10 o'clock, 4 cm from the nipple, there is an oval, circumscribed, hypoechoic solid mass measuring 8 x 6 x 5 mm. It is parallel in orientation and demonstrates no posterior acoustic features. No additional lesions are seen in the area of concern. 
IMPRESSION:
1. Benign appearing solid mass at 10 o'clock in the left breast corresponding to the area of palpable concern, most consistent with a fibroadenoma. 
2. Recommend clinical follow up and/or biopsy for definitive diagnosis if clinically warranted. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY

INDICATION: Routine screening.

TECHNIQUE: Bilateral digital screening mammography was performed. Tomosynthesis was utilized.

COMPARISON: None.

FINDINGS:

The breasts are heterogeneously dense (C), which may obscure detection of small masses.

There is no mammographic evidence of malignancy in the right breast. No significant masses, calcifications, or areas of architectural distortion are identified.

In the left breast, there is a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant at approximately 2 o'clock, 7 cm from the nipple. The cluster measures approximately 12 x 8 mm. No associated mass or architectural distortion is seen.

Bilateral benign-appearing lymph nodes are noted in the axillary regions.

IMPRESSION:

RIGHT BREAST: No mammographic evidence of malignancy. BI-RADS 1 - Negative.

LEFT BREAST: New cluster of suspicious microcalcifications in the upper outer quadrant, BI-RADS 4 - Suspicious abnormality. 
 
Stereotactic core needle biopsy is recommended for further evaluation of the microcalcifications in the left breast at 2 o'clock.

ADDENDUM: The patient was notified of the results and agrees to proceed with stereotactic core needle biopsy of the left breast. The biopsy has been scheduled for next week. ➡ *Positive*
- RIGHT BREAST SCREENING MAMMOGRAM. 
CLINICAL HISTORY: Annual screening mammogram.
TECHNIQUE: Standard screening CC and MLO views of the right breast were obtained.
COMPARISON: 10/23/2021.
FINDINGS:  
The right breast is heterogeneously dense. This may lower the sensitivity of mammography. 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
There is no significant interval change from the prior exam.
IMPRESSION:
BI-RADS 1: Negative. 
Recommend routine screening in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening mammogram.

TECHNIQUE: Craniocaudal and mediolateral oblique views of both breasts.

COMPARISON: None available.

FINDINGS:
The breast tissue is predominately fatty.

There are no suspicious masses, asymmetries or areas of architectural distortion in either breast. 

In the upper outer quadrant of the left breast there is a small group of indeterminate microcalcifications at approximately 10 o' clock, 5 cm from the nipple. The largest calcification measures approximately 0.4 mm. 

No other calcifications are identified in either breast.

No significant axillary lymphadenopathy.

IMPRESSION:
BI-RADS Category 0 - Incomplete. Additional imaging evaluation is recommended for the small group of microcalcifications in the upper outer left breast at 10 o'clock. Spot compression magnification views are suggested for further assessment. 

The remainder of the study is negative. 

RECOMMENDATION: Additional imaging as stated above. The patient is advised to return to the Breast Imaging center to complete the supplementary imaging evaluation. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH ADDITIONAL VIEWS LEFT BREAST:

INDICATION: Evaluate for multifocal disease in patient with biopsy-proven invasive ductal carcinoma at 3:00 left breast. Outside ultrasound-guided core biopsy demonstrated invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative.

TECHNIQUE: Standard CC and MLO views were obtained bilaterally. Spot compression magnification views were performed of the left breast at the site of the known malignancy. 

COMPARISON: Outside mammogram from 4 months prior.

FINDINGS:
The breasts are heterogeneously dense which may limit sensitivity. 

LEFT BREAST: 
In the upper outer left breast, there is a new 12 mm cluster of fine pleomorphic calcifications at 2:00, 6 cm from the nipple. This corresponds to the site of the known malignancy. Magnification views confirm the presence of a suspicious cluster of calcifications. 

At 3:00 in the left breast, there is an oval circumscribed mass measuring 10 mm. This was stable compared to the outside prior mammogram and likely represents a benign finding such as a fibroadenoma. 

The remainder of the left breast is unremarkable. The skin is normal in thickness. The nipple is normal in configuration. No suspicious axillary lymphadenopathy.

RIGHT BREAST:
There are no dominant masses, areas of architectural distortion or suspicious calcifications in the right breast. Stable benign-appearing calcifications are noted. The skin is normal in thickness. The nipple is normal in configuration. No suspicious axillary lymphadenopathy.

IMPRESSION:
1. Highly suspicious cluster of pleomorphic calcifications at 2:00 in the left breast corresponding to the site of known biopsy-proven invasive ductal carcinoma. Additional disease in the upper outer left breast.
2. Probably benign mass in the left breast at 3:00, likely a fibroadenoma. Recommend clinical and imaging follow-up for this finding. 
3. No suspicious mammographic findings in the right breast. 

BI-RADS CATEGORY 4 LEFT BREAST
BI-RADS CATEGORY 2 RIGHT BREAST

RECOMMENDATIONS: 
Surgical consultation for management of the known invasive ductal carcinoma in the left breast as well as the additional site of suspicious calcifications. Short-term follow-up of the benign mass at 3:00 in the left breast could be considered if the patient does not undergo mastectomy. ➡ *Positive*
- ADDENDUM TO BREAST MRI
Clinical History: Patient returns today for an MRI-guided biopsy of the right breast
Comparison: Prior right breast MRI performed on 02/21/2025
Technique: The patient was placed in the prone position. The right breast was imaged with a dedicated breast coil. Intravenous gadolinium contrast was administered.
Findings:
The previously described area of enhancement in the right breast corresponding to the area of non-mass enhancement is again identified in the right breast, slightly superior and deep to the areola. Using real-time imaging, the lesion was localized with an 18-gauge needle. Three cores of tissue were obtained from this region and submitted for pathologic analysis.
IMPRESSION:
Successful right breast MRI-guided biopsy performed without complication. ➡ *Not Stated*
- MAMMOGRAPHY REPORT

CLINICAL HISTORY: Patient presents for initial screening mammogram.

TECHNIQUE: Bilateral digital screening mammographic views including CC and MLO views obtained.

COMPARISON: None.

FINDINGS:

Right Breast:
The breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant mass, architectural distortion or suspicious calcifications. 

Left Breast:
The breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant mass or architectural distortion. Scattered amorphous microcalcifications are noted in the upper outer quadrant, spanning an area of approximately 1 cm. The calcifications are new compared to prior exams.

IMPRESSION:

Right Breast: No mammographic evidence of malignancy, BI-RADS Category 1.

Left Breast: Scattered amorphous microcalcifications in the upper outer quadrant. Additional evaluation with magnification views recommended for further characterization, BI-RADS Category 0. 

ADDENDUM:
Magnification views of the left breast demonstrate pleomorphic microcalcifications in a segmental distribution, spanning 12 mm at the 2:00 position, 6 cm from the nipple. Biopsy is recommended, BI-RADS Category 4B.

OVERALL BI-RADS ASSESSMENT CATEGORY: 4B ➡ *Positive*
- SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram. No prior mammograms available for comparison.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: No prior exams available.
FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.
No dominant masses, architectural distortion, or suspicious calcifications are identified in either breast. 
No significant axillary, supraclavicular, or internal mammary adenopathy. Benign vascular calcifications noted bilaterally.
Stable benign calcifications in the right breast, likely representing fibroadenoma.
IMPRESSION:
Negative mammogram. No mammographic evidence of malignancy. 
BIRADS Category 1 - Negative. Routine screening is recommended in 1 year.
ADDENDUM: Heterogeneously dense breasts noted which lowers the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI for women with dense breasts, as clinically indicated. ➡ *Not Stated*
- TMIST Study Assigned to FFDM arm.
No concerning findings for malignancy identified on the digital mammographic images obtained today.
The breast tissue is heterogeneously dense, which could obscure detection of small masses.
BIRADS Category 1: Negative.
No evidence of microcalcifications.
1 year OBSP recommended. ➡ *Not Stated*
- MAMMOGRAM REPORT - LEFT BREAST
CLINICAL HISTORY: 62-year-old female with left breast lump palpated by self at 10:30 position. Strong family history of breast cancer in two sisters and mother. Last mammogram 18 months prior at outside facility reported as normal.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Targeted ultrasound of the left breast upper outer quadrant was also performed.
FINDINGS:
The left breast is heterogeneously dense, which may obscure detection of small masses. No dominant mass, architectural distortion, or suspicious calcifications are identified on mammographic views. Stable benign-appearing calcifications are noted in the upper outer quadrant. 
Targeted ultrasound of the clinically palpable lump in the 10:30 position of the left breast demonstrates a 0.8 x 0.6 x 0.5 cm oval circumscribed hypoechoic solid mass, parallel in orientation. No suspicious features are identified. This likely represents a fibroadenoma. 
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Benign-appearing stable calcifications in the left upper outer quadrant.
2. 0.8 cm oval circumscribed solid mass at 10:30 position of the left breast on targeted ultrasound, likely representing a fibroadenoma. Recommend clinical follow-up and imaging surveillance of this lesion.
BIRADS 2 - Benign findings. Recommend annual screening mammography. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED VIEWS LEFT BREAST AND ULTRASOUND LEFT BREAST.
History: 2 months follow up left breast microcalcifications. No other breast complaints. No family history of breast cancer.
Compared to previous studies from April 2023. FINDINGS: The breasts are heterogeneously dense. 
There are no suspicious right mammographic findings. 
In the upper outer quadrant of the left breast, there is a new 5 mm cluster of pleomorphic microcalcifications at 2:00, 7 cm from the nipple. This was not present on the prior mammogram. Targeted ultrasound of this area does not show a correlating mass. The remainder of the left breast is unchanged with scattered fibroglandular densities and no suspicious masses, architectural distortion or additional suspicious calcifications.
Ultrasound of the left axilla is unremarkable with no abnormal lymph nodes.
IMPRESSION:
New cluster of suspicious pleomorphic microcalcifications upper outer left breast, BI-RADS 4B. Stereotactic core biopsy is recommended for tissue diagnosis. A requisition will be sent to book the biopsy.
Stable remainder of left breast, BI-RADS 2. Recommend annual screening mammogram in 1 year if biopsy is benign.  
No suspicious findings right breast, BI-RADS 1. Recommend annual screening mammogram in 1 year.
ADDENDUM: Patient called back and stereotactic core biopsy was performed the same day of the new cluster of microcalcifications in the left breast at 2:00. Specimen x-ray confirmed calcifications in the cores. Post-procedure mammogram showed biopsy clip in good position. The patient tolerated the procedure well. Pathology results are pending and will be communicated to referring physician. ➡ *Positive*
- TMIST FFDM ARM OBSP MAMMOGRAM.

This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 

The breasts are almost entirely fatty. There are no dominant masses, architectural distortion or suspicious calcifications.

Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes. 
OBSP to recall client in 1 year: Yes.

BIRADS-2. Annual screening recommended. ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAMS 
INDICATION: Annual screening
TECHNIQUE: Standard CC and MLO views of each breast.
COMPARISON: None.
FINDINGS:
Right breast: Scattered fibroglandular densities. No focal asymmetry, distortion or dominant mass. No suspicious calcifications.
Left breast: Scattered fibroglandular densities. No focal asymmetry or distortion. In the upper outer quadrant, there is a small group of pleomorphic microcalcifications spanning an area of 6 mm. No dominant mass. 
IMPRESSION:
RIGHT BREAST: NEGATIVE. BI-RADS 1.
LEFT BREAST: Small group of suspicious microcalcifications upper outer quadrant, BI-RADS 4. Recommend biopsy.
ADDENDUM:
Stereotactic guided core biopsy of the group of microcalcifications in the left upper outer quadrant was performed. Specimen radiograph confirmed calcifications were sampled. The pathology report revealed atypical ductal hyperplasia with microcalcifications. 
FINAL ASSESSMENT:
RIGHT BREAST: BI-RADS 1. 
LEFT BREAST: BI-RADS 4. Biopsy proven atypical ductal hyperplasia with microcalcifications. 6 month follow up mammogram recommended in the left breast to assure stability. ➡ *Positive*
- SCREENING MAMMOGRAM 
INDICATION: Routine screening.
TECHNIQUE: Standard views of each breast were obtained.
COMPARISON: 03/22/2020. 
FINDINGS:
The breasts are heterogeneously dense. 
There are no dominant masses, areas of architectural distortion or suspicious calcifications.
There are scattered areas of fibroglandular density bilaterally.
IMPRESSION:
BIRADS 2 - Benign. Recommend routine screening in 1 year. ➡ *Not Stated*
- MAMMOGRAM REPORT:
PROCEDURE: Bilateral digital mammogram with tomosynthesis.
HISTORY: 45 year old female presents for screening mammogram. No current breast complaints.
TECHNIQUE: Bilateral CC and MLO views were obtained using tomosynthesis. 
COMPARISON: Comparison is made to outside screening mammogram from 6/15/2021.
FINDINGS:
Right breast: The breast is heterogeneously dense (BI-RADS density C). There are no dominant masses, architectural distortion, or suspicious calcifications. Stable benign appearing lymph node in the right axilla.
Left breast: The breast is heterogeneously dense (BI-RADS density C). There are no dominant masses, architectural distortion, or suspicious calcifications.  
IMPRESSION:
1. No evidence of malignancy in either breast. Recommend continued annual screening in 12 months with consideration for supplemental screening given patient's dense breast tissue. 
2. BI-RADS Category 1: Negative.
ADDENDUM: 
This examination was interpreted using digital mammography with tomosynthesis which increases lesion conspicuity. The breasts are heterogeneously dense which can lower the sensitivity of mammography. Given negative mammogram, no additional imaging is recommended at this time. The patient is encouraged to maintain routine clinical breast exams and follow up with annual screening mammography. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast. 

In the left breast at the 10:00 position, approximately 8 cm from the nipple, there is a group of amorphous microcalcifications in a segmental distribution, spanning an area of about 1 cm. These are new since the patient's last mammogram.

The right breast is unremarkable.

IMPRESSION: 
BI-RADS Category 0: Incomplete, need additional imaging evaluation. Suspicious microcalcifications in the left breast requiring further workup with magnification views and possible biopsy.

ADDENDUM: Patient proceeded to stereotactic biopsy of the left breast microcalcifications. Pathology results showed ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- MAMMOGRAPHY, RIGHT BREAST.
HISTORY: 62-year-old female for screening. Prior benign biopsy left breast 10 years ago. No other breast history. 
FINDINGS: The breast is heterogeneously dense. This may lower the sensitivity of mammography. No concerning masses, architectural distortion, or suspicious calcifications. There are stable scattered fibroglandular densities.
IMPRESSION: 
BI-RADS 1: Negative. 
Recommend routine screening in 1 year.

ADDENDUM: The interpreting radiologist, Dr. Smith, has reviewed the images and agrees with the findings and impression as stated above. ➡ *Negative*
- FINDINGS:
The breasts are heterogeneously dense. 
In the upper outer quadrant of the right breast, there is a new 1.1 cm irregular mass with indistinct margins at the 10 o'clock position, 4 cm from the nipple. This mass was not present on the prior mammogram from 2 years ago. There are several new pleomorphic calcifications associated with the mass, ranging in size from 0.2 to 0.5 mm. No other suspicious masses, architectural distortion, or calcifications are identified in either breast. 
Bilateral axillary regions are unremarkable, with no enlarged lymph nodes seen.

IMPRESSION: 
BI-RADS 4: Suspicious abnormality in the right breast. Stereotactic biopsy is recommended for the new irregular mass with associated pleomorphic calcifications at 10 o'clock position of the right breast. 
BI-RADS 2: Benign findings in the left breast, no significant change from prior exam.

ADDENDUM:  Stereotactic core biopsy of the right breast mass with calcifications was performed.  Pathology results demonstrate invasive ductal carcinoma, moderately differentiated, with associated ductal carcinoma in situ, intermediate nuclear grade.  Biopsy clip placed. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAPHY LEFT
HISTORY: Annual screening. 56 years old. No palpable lumps or complaints.
COMPARISON: 2022/01/04
FINDINGS: The breast tissue is heterogeneously dense, which could obscure detection of small masses. There is no architectural distortion or new masses. There are no clusters of suspicious microcalcifications. 
No significant changes compared to prior study.
IMPRESSION: 1. No evidence of malignancy, BIRADS 1. Annual screening recommended. ➡ *Negative*
- FINDINGS:
Both breasts exhibit scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications are identified.

Bilateral breast ultrasound was performed. No sonographic abnormality was identified in either breast. No solid or cystic lesions were seen.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening mammography in 1 year.
2. No sonographic abnormality in either breast.

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHIC EXAMINATION:
Indication: Routine screening.
Technique: Standard CC and MLO views of both breasts were obtained.
Comparison: Previous mammogram from 1 year ago.
Findings:
The breast tissue is predominantly fibroglandular.
No dominant masses, architectural distortion, or suspicious calcifications are identified in either breast.
Stable, benign-appearing calcifications are noted in the left breast.
IMPRESSION:
BIRADS Category 2: Benign findings. Routine screening is recommended in 1 year.
ADDENDUM:
Breast density was assessed using an automated software. The breast density is heterogeneously dense, which may obscure small masses. The patient will be informed that her breast tissue is dense. Recommend routine screening mammography. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
Both breasts are almost entirely fatty.
There are no masses, asymmetries, or areas of architectural distortion in either breast.
No suspicious calcifications are seen in the right breast.
In the left breast at the 12 o'clock position, 5 cm from the nipple, there is a 1 cm cluster of amorphous calcifications. No other suspicious calcifications in the left breast.
No skin thickening or retraction. No dilated ducts.

IMPRESSION:
1. BI-RADS 1: Negative in the right breast.
2. BI-RADS 0: Cluster of amorphous calcifications in the left breast. Additional magnification views recommended for further evaluation.

ADDENDUM:
Magnification views of the left breast demonstrate pleomorphic calcifications in a segmental distribution at the 12 o'clock position. Suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy recommended.

FINAL ASSESSMENT:
RIGHT BREAST: BI-RADS 1
LEFT BREAST: BI-RADS 4 - Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH CAD 

History: Screening exam. No previous mammograms available for comparison. No significant family history of breast cancer. No clinical concerns.

FINDINGS:
The breasts are mostly fatty.
There are scattered benign calcifications bilaterally.
In the left breast at 6:00, 10 cm from the nipple there is a 10 mm cluster of indeterminate microcalcifications.
No spiculated masses, architectural distortion or suspicious asymmetries are seen.
No skin thickening or nipple inversion. 
CAD was applied as a second reader. No additional findings seen.

IMPRESSION:
New 10 mm cluster of indeterminate microcalcifications left breast at 6:00. Additional magnification views recommended. BI-RADS category 0: incomplete. A diagnostic mammogram is recommended for further evaluation.

ADDENDUM:
Patient returned for additional magnification views of the left breast. Spot compression magnification views were performed at 6:00. 

FINDINGS:
The additional magnification views confirm a 10 mm cluster of pleomorphic microcalcifications at 6:00 in the left breast. These are of intermediate concern.  
No associated mass.
The remainder of the left breast mammogram is unremarkable.

IMPRESSION:
10 mm cluster of pleomorphic microcalcifications left breast at 6:00, BI-RADS 4B. Stereotactic core biopsy is recommended. ➡ *Positive*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: Annual screening mammogram. Patient denies any new breast symptoms or palpable lumps. Patient's mother had breast cancer at age 55.
TECHNIQUE: Bilateral digital screening mammography including CC and MLO views.
COMPARISON: Comparison is made to prior exam dated 10/15/2020 at this facility.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.
Right Breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.
Left Breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.
Bilateral benign-appearing lymph nodes are noted in the axillary regions.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. Heterogeneously dense breast tissue, which lowers the sensitivity of mammography. Clinical breast examination is recommended.
BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM.  
HISTORY: 61-year-old female referred for bilateral screening mammogram.
FINDINGS: Breast composition is heterogeneously dense, which may obscure small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. 
There are scattered benign appearing calcifications in both breasts. The largest cluster measures approximately 8 mm and is located in the upper outer quadrant of the right breast at the 10:00 position, 6 cm from the nipple. 
There are no skin lesions or intramammary lymph nodes. The axillae appear unremarkable.  
IMPRESSION: 
BI-RADS category 3: Probably benign finding. Short interval follow-up is suggested in 6 months to document stability of the calcifications in the right breast. 
If the patient has any new breast symptoms in the interim, she should contact her doctor. 
ADDENDUM: Punctate and pleomorphic calcifications are seen clustered at the 10:00 position of the right breast on magnification views 6 cm from the nipple, spanning an area of approximately 8 mm. This is new compared to prior mammograms and warrants biopsy for definitive characterization. BI-RADS category has been changed to 4 and stereotactic core biopsy is recommended. Patient has been informed of results and agrees with management plan. ➡ *Positive*
- FINDINGS: The breasts are heterogeneously dense, which may obscure detection of small masses. No dominant mass, suspicious calcifications, or other findings suspicious for malignancy are identified in either breast. There are no significant changes compared to prior exam.

Targeted ultrasound of the left breast 7 o'clock position was performed. No sonographic correlate for the palpable abnormality described. No suspicious mass, cyst, or other sonographic finding.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography in 12 months.
2. No sonographic abnormality in the left breast 7 o'clock position corresponding to site of palpable finding. Recommend clinical follow up and correlation as indicated.

ASSESSMENT: BI-RADS 1 - Negative.

ADDENDUM: Comparison was made to outside mammograms from December 2024, which are similar. The breasts remain heterogeneously dense with no dominant mass, distortion, or suspicious calcifications. ➡ *Negative*
- BILATERAL MAMMOGRAM:
INDICATION: Screening
COMPARISON: Prior exams from 3/22/2021
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
Both breasts are almost entirely fatty. No dominant masses, suspicious calcifications, or other findings.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend continued annual screening mammography in 12 months.
BI-RADS CATEGORY 1 - NEGATIVE. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: Mammogram from 11/2020 at XYZ Imaging Center.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.

RIGHT BREAST:
There are no masses, architectural distortion, or suspicious calcifications. The skin is unremarkable. No significant change from prior exam.

LEFT BREAST: 
There are no masses, architectural distortion, or suspicious calcifications. The skin is unremarkable. No significant change from prior exam.

IMPRESSION: 
BI-RADS Category 1: Negative bilateral screening mammogram. 

RECOMMENDATION:
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: 45-year-old female presents for routine screening mammogram. No reported breast symptoms or concerns. 
COMPARISON: Compared to prior study from 05/15/2022.
TECHNIQUE: Standard CC and MLO views of both breasts.  
FINDINGS:
RIGHT BREAST: The breast is heterogeneously dense (BI-RADS density C). There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications are noted in the upper outer quadrant. No significant change from prior exam.
LEFT BREAST: The breast is heterogeneously dense (BI-RADS density C). There are no dominant masses, areas of architectural distortion, or suspicious calcifications. No significant change from prior exam.
IMPRESSION: 
BI-RADS 2 - Benign findings. Recommend continued annual screening mammography in 1 year. 
No evidence of malignancy or suspicious microcalcifications in either breast.
ADDENDUM: Reviewing radiologist agrees with the above findings and impression. Digital breast tomosynthesis was also performed confirming no suspicious masses, calcifications, or areas of architectural distortion in either breast. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM.
COMPARISON: None. 
TECHNIQUE: Standard CC and MLO views.
FINDINGS: 
The breasts are almost entirely fatty.
There are no suspicious masses, areas of distortion or skin thickening in either breast.
In the left breast at the 9:00 position 6 cm from the nipple there is a group of calcifications in a segmental distribution measuring approximately 2 cm. The calcifications are fine pleomorphic and are new compared to previous mammograms. Additional spot compression magnification views confirm these findings.  
The right breast is unremarkable.
No abnormal axillary lymph nodes are seen.
IMPRESSION:
BI-RADS Category 4 Left: Suspicious calcifications in the left breast requiring stereotactic core biopsy for definitive diagnosis.
BI-RADS Category 1 Right: Normal study. ➡ *Positive*
- FINDINGS: The right breast demonstrates a mastectomy scar with no residual breast tissue identified.

The left breast is composed of heterogeneously dense fibroglandular tissue. No dominant mass, architectural distortion, or suspicious calcifications are identified. The skin, nipple, and areola are unremarkable. No significant interval change compared to the prior exam.

Left breast ultrasound was performed with attention to the palpable concern at 7 o'clock. Targeted scanning of the area demonstrates normal fibroglandular tissue with no discrete solid or cystic lesion identified.

IMPRESSION:
1. Negative bilateral mammogram. No suspicious findings or microcalcifications.
2. Negative targeted left breast ultrasound. No sonographic correlate for the palpable finding at 7 o'clock.

BI-RADS category 1: Negative.

RECOMMENDATIONS:
Routine annual screening mammography is recommended. Clinical follow up of the palpable finding as needed.

ADDENDUM: Comparison was made to an outside mammogram from December 2024, which was reported as normal. The current study is concordant with the prior exam. ➡ *Negative*
- MAMMOGRAM - BILATERAL - SCREENING. CLINICAL HISTORY: Screening mammogram. TECHNIQUE: Craniocaudal and mediolateral oblique views of the breasts were performed using digital mammography. COMPARISON: Comparison is made to the mammogram from 02/14/2024. FINDINGS: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- FINDINGS:
There are scattered fibroglandular densities bilaterally. 
In the upper outer quadrant of the right breast at 10 o'clock position approximately 6 cm from the nipple, there is a group of amorphous microcalcifications measuring approximately 15 mm. Additional magnification views confirm the presence of these calcifications.
In the upper outer quadrant of the left breast, there are several benign-appearing calcifications. 
No dominant mass, architectural distortion, or suspicious calcifications are seen in the left breast.
There are no abnormal axillary lymph nodes bilaterally.

IMPRESSION:
1. New group of amorphous microcalcifications in the upper outer quadrant of the right breast, measuring approximately 15 mm. Additional workup with stereotactic core biopsy is recommended for tissue diagnosis. BI-RADS 4B.
2. Benign calcifications left breast. BI-RADS 2.

ADDENDUM: 
The patient returned for stereotactic core biopsy of the group of amorphous microcalcifications in the right breast at 10 o'clock 6 cm from the nipple. Specimen radiograph confirmed calcifications within the cores. Final pathology revealed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. ➡ *Positive*
- FINDINGS:
The breasts are scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications are seen on either side.

In the LEFT breast at 2:00, 5 cm from the nipple there is a group of pleomorphic microcalcifications spanning 8 mm. The morphology is suspicious for ductal carcinoma in situ.

RIGHT BREAST: Negative.
LEFT BREAST: Highly suspicious for malignancy.

IMPRESSION:
BI-RADS Category 0: Left breast
Suspicious microcalcifications left breast at 2:00 as described. Additional magnification views recommended for further evaluation.  

BI-RADS Category 1: Right breast
Negative study.

ADDENDUM:
Magnification views of the left breast in the craniocaudal and mediolateral views show a group of pleomorphic microcalcifications at 2:00, 5 cm from the nipple measuring 1 cm. The morphology and distribution are highly suspicious for ductal carcinoma in situ. Stereotactic biopsy recommended.

FINAL ASSESSMENT:
BI-RADS Category 4C: Left breast
Highly suggestive of malignancy. Biopsy should be considered.

BI-RADS Category 1: Right breast.
Negative mammogram. ➡ *Positive*
- TECHNIQUE: Bilateral digital mammogram, including craniocaudal and mediolateral oblique views.

COMPARISON: Comparison is made with prior outside mammogram from December 2022.

FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses. The breasts are stable in appearance compared to prior mammograms. There are no dominant masses, architectural distortions, or suspicious calcifications.

Stable bilateral scattered benign-appearing calcifications are again noted. No new or concerning grouped, pleomorphic, or fine linear microcalcifications. 

IMPRESSION:
Bilateral heterogeneously dense but stable mammogram without evidence of malignancy. No suspicious microcalcifications. 
BI-RADS Category 1 - Negative.

Recommend continued annual screening mammography in 12 months. Clinical breast exam also advised.

BIRADS 1. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
HISTORY: 45 year old female, annual screening.
TECHNIQUE: CC and MLO views both breasts. Magnification views right breast.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in the left breast.
At the 10:00 position of the right breast, 4 cm from the nipple, there is a group of amorphous calcifications spanning approximately 6 mm. Additional magnification views confirm these to be suspicious.
No other dominant masses, architectural distortion or suspicious calcifications in the right breast.
No abnormal axillary lymph nodes.
IMPRESSION:
BI-RADS 4: Suspicious calcifications right breast. Stereotactic core biopsy recommended.
BI-RADS 2: Benign findings left breast. Recommend continued routine screening mammography. ➡ *Positive*
- FINDINGS:

Bilateral screening mammography was performed, including standard CC and MLO views.

The breast tissue is heterogeneously dense, which may obscure small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications.

Stable postoperative changes are noted in the right breast, consistent with prior history of lumpectomy.

No mammographic evidence of malignancy.

IMPRESSION:

BIRADS Category 2: Benign findings. Recommend routine screening in 1 year.

ADDENDUM:

This study was compared with prior outside examination from MM/DD/YYYY, which showed similar findings of postoperative changes in the right breast. No suspicious interval changes. ➡ *Not Stated*
- SCREENING MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Compared to prior study of June 15, 2020.  

FINDINGS:  
The breasts are heterogeneously dense. This may lower the sensitivity of mammography. 

There are no dominant masses, areas of architectural distortion or definite suspicious calcifications. 

Stable benign appearing calcifications are noted in the right breast. 

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued routine screening.

BIRADS Category 2 - Benign findings. 

ADDENDUM: Tomosynthesis was not performed as part of this examination. ➡ *Not Stated*
- SCREENING MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of each breast.

FINDINGS:
The breasts are heterogeneously dense (50-75% fibroglandular tissue). This may lower the sensitivity of mammography.

There are no dominant masses, architectural distortion or suspicious calcifications.

Stable benign appearing calcifications are noted in the right breast, likely representing involuting fibroadenomas.

No significant interval change compared to prior mammogram from 2 years ago.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breasts.

BIRADS Category 2: Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- DIAGNOSTIC MAMMOGRAPHY LEFT BREAST WITH TOMOSYNTHESIS
HISTORY: 58 year old female with new palpable mass left breast upper outer quadrant.
TECHNIQUE: Standard CC and MLO views with tomographic slices obtained. Focused compressed views of the palpable mass in the left upper outer quadrant with a marker.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense (C) which may lower the sensitivity of mammography.
Left Breast:
There is a new 1.5 cm obscured mass at 2:00, 8 cm from the nipple, corresponding to the palpable mass. It demonstrates an associated 5 mm coarse calcification. No architectural distortion or skin thickening.
There is a new 5 mm cluster of pleomorphic calcifications at 12:00, 6 cm from the nipple.
Right Breast:
No dominant mass, suspicious calcifications, or architectural distortion.
IMPRESSION:
New 1.5 cm obscured mass left breast at 2:00 with a 5 mm associated coarse calcification, corresponding to the palpable finding. An additional 5 mm cluster of pleomorphic calcifications at 12:00. Suspicious for malignancy. Recommend stereotactic biopsy of the calcifications and ultrasound for further evaluation and biopsy of the mass. BI-RADS 4C.
Benign right breast. BI-RADS 2.

ADDENDUM: Stereotactic biopsy of the left breast calcifications at 12:00 and ultrasound guided biopsy of the 2:00 mass were performed. Pathology results pending. Post-biopsy mammogram demonstrates biopsy clips marking the sites of biopsy. ➡ *Positive*
- FINDINGS:
Bilateral screening mammogram demonstrates heterogeneously dense breasts. 
In the right breast there is a new 8 mm cluster of fine pleomorphic calcifications at the 10:00 position, 6 cm from the nipple. This is a suspicious finding. No other dominant masses, architectural distortion or suspicious calcifications.
In the left breast, there are no dominant masses, architectural distortion or suspicious calcifications. Stable benign calcifications are noted.

IMPRESSION:
New cluster of suspicious calcifications in the right breast at 10:00, 6 cm from the nipple. Additional magnification views are recommended for further evaluation. BI-RADS 4B.
The left breast is negative. BI-RADS 2.

ADDENDUM:
Right breast magnification views in the CC and MLO projections confirm an 8 mm cluster of fine pleomorphic calcifications at the site of concern. Stereotactic core biopsy is recommended for definitive diagnosis. ➡ *Positive*
- NORMAL AND BENIGN TMIST FFDM OBSP MAMMOGRAM 
This patient was randomly assigned to the full field digital mammography arm of the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) after providing informed consent via an approved Research Ethics Board protocol.
Comparison to prior studies: Yes.  
Breast density: The breasts are composed of scattered areas of fibroglandular density.
No significant masses, architectural distortion, or suspicious calcifications are identified.
The breast tissue is within normal limits, no evidence of malignancy.
Annual OBSP screening is recommended.  
BIRADS Category 2- Benign finding. 
Recommendation: Routine screening in 12 months. ➡ *Not Stated*
- MAMMOGRAPHY REPORT
Clinical Information: TMIST FFDM ARM OBSP MAMMOGRAM.
Technique: TMIST approved full field digital mammography. CC and MLO views of each breast.
Findings:
There are no significant masses, architectural distortion or asymmetries. The breast tissue is heterogeneously dense. No dominant masses. No suspicious calcifications.
There are scattered fibroglandular densities.
No skin thickening or retraction. No enlarged axillary lymph nodes.
Impression:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense parenchyma, which could obscure detection of small masses.
BI-RADS Category 1: Negative. Routine screening mammography is recommended. OBSP to recall client in 1 year.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes. 
OBSP to recall client in 1 year: Yes. ➡ *Not Stated*
- BILATERAL MAMMOGRAM.
HISTORY: Patient presents for annual screening mammogram. Last prior mammogram 9/2021 at this facility.
FINDINGS:
Parenchymal pattern is heterogeneously dense (C). This decreases sensitivity of mammography. Consider supplemental screening in women with dense breast tissue.
There are no dominant masses, suspicious calcifications or areas of architectural distortion bilaterally.
There are stable bilateral scattered benign-appearing calcifications.
There is no significant interval change from the prior exam.
IMPRESSION:
1. No mammographic evidence of malignancy. BIRADS 2.
2. Recommend routine screening in one year.
ADDENDUM: 
Patient was notified of BIRADS 2 results via MyChart. Routine screening is recommended in one year. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS
INDICATION: Follow up after lumpectomy and radiation therapy for right breast cancer.
COMPARISON: Mammograms from 1 year ago.
FINDINGS:
The breasts are heterogeneously dense. Scattered benign-appearing calcifications in both breasts.
The skin is normal with no significant thickening or retraction. No mammographic evidence of a discrete mass lesion in either breast. There are stable post-surgical changes in the upper outer right breast. Metallic surgical clips are noted at the lumpectomy site.
Additional magnification views in the craniocaudal projection demonstrate a new group of amorphous microcalcifications in the upper outer left breast at the 2 o'clock position. These span an area of approximately 1 cm and are located 6 cm from the nipple. This cluster was not present on the prior mammogram.
Remainder of the left breast is unremarkable.
IMPRESSION:
New group of suspicious microcalcifications in the left breast at 2 o'clock, 6 cm from the nipple. BIRADS 4B. Stereotactic core biopsy is recommended for diagnosis. Stable post-lumpectomy changes and calcifications right breast. BIRADS 2. ➡ *Positive*
- MAMMOGRAPHY, LEFT BREAST.

INDICATION: Annual screening mammogram.

COMPARISON: Mammogram from 1 year prior at outside facility.

TECHNIQUE: Standard CC and MLO views were obtained.

FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
There are scattered areas of benign-appearing calcifications, likely representing fibrocystic change. No new suspicious clustered microcalcifications.
There is no significant change from prior exam.

IMPRESSION:
1. No mammographic evidence of malignancy. Scattered benign-appearing calcifications, not suspicious. 
2. Recommend continued annual screening mammography in 12 months.

BIRADS Category 2: Benign. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL INDICATION: Annual screening mammogram
COMPARISON: Screening mammogram from 08/20/2024 at St. Mary's Hospital
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Left breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Bilateral axillary regions are unremarkable.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammograms and clinical breast exams. 
BIRADS 1 - Negative ➡ *Negative*
- FINDINGS:
MAMMOGRAPHIC EXAMINATION: Bilateral diagnostic digital mammography was performed including CC and MLO views.
COMPARISON: Compared to prior study of 5/14/2020.
FINDINGS: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.
No dominant masses, architectural distortion or suspicious microcalcifications are identified in either breast.
Stable benign scattered vascular calcifications noted bilaterally.
IMPRESSION:
BIRADS Category 2: Benign findings. Recommend routine screening in 1 year. 
ADDENDUM: 
No mammographic findings that are concerning for malignancy. No suspicious microcalcifications identified. ➡ *Not Stated*
- SCREENING MAMMOGRAM
CLINICAL HISTORY: Screening mammogram. No palpable lumps on self-exam. No other symptoms.
FINDINGS:
Breasts are heterogeneously dense; this may lower the sensitivity of mammography.
No significant masses, architectural distortion, or suspicious calcifications are present on either side. Axillae appear normal.
Left breast:
A cluster of amorphous microcalcifications is noted in the upper outer quadrant at 1 o'clock, 5 cm from the nipple. This measures approximately 8 mm in diameter. No other suspicious microcalcifications are seen.
Right breast:
There are no dominant masses, areas of architectural distortion, suspicious microcalcifications, or other findings.
IMPRESSION:
Left breast: New cluster of amorphous microcalcifications in the upper outer quadrant measuring 8 mm, BIRADS 4B. Recommend stereotactic core biopsy for definitive evaluation.
Right breast: No suspicious findings, BIRADS 2.
RECOMMENDATIONS:
Stereotactic core biopsy of microcalcifications in left breast is recommended. The patient should return to the Breast Imaging Center for a diagnostic mammogram and core biopsy. The risks, benefits and alternatives to the procedure will be explained at the time of diagnostic evaluation.
Annual screening mammography is recommended in 1 year if left breast biopsy proves benign. ➡ *Positive*
- BILATERAL MAMMOGRAM
HISTORY: Annual screening mammogram.
FINDINGS:
Right breast tissue is predominantly fatty. There are no concerning masses, calcifications, or areas of architectural distortion.
Left breast tissue is extremely dense, which may lower the sensitivity of mammography. There are no concerning masses, calcifications, or areas of architectural distortion.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Breast density is extremely dense in the left breast, which may lower the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI as clinically appropriate.
BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS:
Bilateral digital screening mammogram was performed.  
The breast tissue is heterogeneously dense, which may obscure detection of small masses. 
There are scattered areas of fibroglandular density bilaterally.
In the left breast at the 10:00 position, 6 cm from the nipple, there is a group of amorphous calcifications spanning an area of about 10 mm. The calcifications are new compared to prior mammograms from 2 years ago at an outside facility. Spot compression magnification views confirm the presence of a group of suspicious calcifications at this site. 
In the right breast, there are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. 
2. New group of suspicious calcifications in the left breast at 10:00, 6 cm from the nipple. Additional imaging evaluation with left diagnostic mammogram is recommended for further characterization. 
3. Recommend follow up diagnostic evaluation of the left breast. 
4. BI-RADS Category 4 - Suspicious abnormality in the left breast. Biopsy should be considered. 
5. BI-RADS Category 2 - Benign finding in the right breast. Recommend routine screening in 1 year. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No suspicious masses, architectural distortion or suspicious calcifications are seen in either breast.

Bilateral breast ultrasound was performed. No sonographic abnormality is identified. No solid or cystic masses are seen in either breast. There are no dilated ducts or intraductal masses.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.

BIRADS 1 - Negative.

Recommend annual screening mammogram in 1 year. ➡ *Negative*
- FINDINGS: The right breast is not included. The left breast consists of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION: No mammographic evidence of malignancy. Recommend routine screening in 1 year.

BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS:
Bilateral digital mammography was performed including tomosynthesis.
Breast composition is heterogeneously dense. This may lower the sensitivity of mammography.
There is an irregular spiculated mass measuring approximately 15 mm seen in the left breast at the 2:00 position, 6 cm from the nipple. Associated with this mass is a cluster of fine pleomorphic microcalcifications extending over an area of approximately 8 mm.
Several additional groups of calcifications are seen in the upper outer left breast. At 1:00, 8 cm from the nipple, there is a group of amorphous calcifications. At 2:00, 10 cm from the nipple, there is a cluster of fine linear branching calcifications extending over an area of 7 mm. These are a cause for concern.
The right breast is unremarkable. No concerning masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
Highly suspicious findings left breast with irregular spiculated mass at 2:00 with associated pleomorphic calcifications, and additional suspicious calcifications upper outer left breast. Findings are highly suggestive of multifocal malignancy. Recommend tissue diagnosis. BIRADS 5.
Benign right breast. BIRADS 1.
ADDENDUM - ULTRASOUND LEFT BREAST:
Targeted left breast ultrasound was performed. 
The irregular mass at 2:00 in the left breast corresponds to an irregular hypoechoic mass with angulated margins and posterior acoustic shadowing, measuring 16 x 15 x 14 mm. 
The clusters of calcifications in the upper outer left breast are not associated with a mass. Ultrasound guided core biopsy is recommended for the 2:00 mass. Stereotactic biopsy is recommended for the calcifications. BI-RADS 5. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Screening mammogram. Patient has a family history of breast cancer.

FINDINGS:
The breasts are heterogeneously dense (type C). This may lower the sensitivity of mammography.

Left breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.

Right breast: 
At the 9 o'clock position, 6 cm from the nipple, there is a 6 mm cluster of pleomorphic microcalcifications. These were not present on the prior study. Additional magnification views confirm these are suspicious.
There are no dominant masses or architectural distortion.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Biopsy should be considered. 

RECOMMENDATION: 
Right breast biopsy is recommended for the cluster of pleomorphic microcalcifications at 9 o'clock, 6 cm from the nipple. The patient should follow up with the referring physician to discuss results and management options. ➡ *Positive*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: 09/12/2021.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
BIRADS Category 1: Negative.
Routine screening is recommended in 1 year. ➡ *Not Stated*
- SCREENING BILATERAL DIGITAL MAMMOGRAM.

INDICATION: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:

Both breasts are almost entirely fatty.

Right breast: There are no dominant masses, areas of architectural distortion or suspicious calcifications.

Left breast: Linear grouped amorphous calcifications are seen in the upper outer quadrant extending anteriorly. The largest group measures approximately 1 cm. No associated mass or architectural distortion is identified.

IMPRESSION:

1. Grouped amorphous calcifications in the left upper outer breast, suspicious for ductal carcinoma in situ (DCIS). Additional magnification views recommended for further evaluation. BI-RADS 4B.

2. No dominant mass, architectural distortion or suspicious calcifications are identified in the right breast. Routine annual screening is recommended. BI-RADS 2.

ADDENDUM AFTER MAGNIFICATION VIEWS LEFT BREAST: 
The magnification views of the left breast confirm linear and branching pleomorphic calcifications in the left upper outer quadrant, highly suspicious for DCIS. Stereotactic core biopsy is recommended for definitive diagnosis. BI-RADS 4C. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY.
History: Screening.
Comparison available dating from 10/2023 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- FINDINGS:
Bilateral digital mammograms demonstrate that both breasts are heterogeneously dense. No dominant mass, architectural distortion, or abnormal calcifications are identified in the right breast. 

In the left breast, at the 10 o'clock position approximately 5 cm from the nipple, there is a cluster of pleomorphic microcalcifications. Additional magnification views in the craniocaudal and mediolateral projections confirm the presence of a 1 cm group of heterogeneous calcifications, varying in size and shape. The morphology is suspicious for malignancy. No other microcalcifications are seen elsewhere in the left breast.

Targeted ultrasound at the 10 o'clock position of the left breast corresponding to the area of microcalcifications demonstrates normal appearing fibroglandular tissue with no discrete mass or shadowing. The microcalcifications are not sonographically apparent.

The remainder of the left breast is unremarkable. No skin thickening or nipple retraction.

IMPRESSION:
New development of a 1 cm cluster of pleomorphic microcalcifications at 10 o'clock position of the left breast, suspicious for malignancy. BI-RADS 4C. Stereotactic biopsy is recommended for definitive diagnosis.

The right breast is negative. BI-RADS 1.

ADDENDUM: Stereotactic core biopsy of the group of microcalcifications in the left breast at 10:00, 5 cm from the nipple was performed. Specimen radiograph confirmed calcifications within the cores. Final pathology results demonstrated ductal carcinoma in situ, high nuclear grade with comedonecrosis. ➡ *Positive*
- SCREENING MAMMOGRAM.
Comparison: Screening mammogram from 1 year ago.
Technique: Standard CC and MLO views of both breasts were obtained. 
Breast Composition: The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
Findings: 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Impression:
No mammographic evidence of malignancy. 
BI-RADS Category 1: Negative. 
Recommend routine screening in 1 year. ➡ *Not Stated*
- MAMMOGRAM REPORT
INDICATION: Annual screening mammogram
COMPARISON: None available
FINDINGS:
Right breast: The breast is composed of scattered fibroglandular densities. There are no suspicious masses, calcifications or areas of architectural distortion. No significant change from previous exam.
Left breast: The breast is composed of scattered fibroglandular densities. There are no suspicious masses or calcifications. No significant change from previous exam.
IMPRESSION:
1. No mammographic evidence of malignancy. Routine annual screening recommended.
2. No evidence of microcalcifications.
BI-RADS Category 1: Negative ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

At the 9 o'clock position of the right breast there is a group of amorphous microcalcifications spanning an area of approximately 1 cm. Additional magnification views in the right CC and MLO projections confirm these calcifications. 

The visualized skin and nipple are unremarkable. No axillary, supra or infraclavicular adenopathy.

IMPRESSION: 
BI-RADS 4: Suspicious grouped amorphous microcalcifications in the right breast at 9 o'clock position. Stereotactic biopsy is recommended for further evaluation. The patient should return to clinic to discuss results.

ADDENDUM: The patient returned for stereotactic biopsy of the group of calcifications in the right breast at the 9 o'clock position. Specimen radiograph confirmed that the calcifications were adequately sampled. Final pathology results showed benign fibrocystic changes with microcalcifications, consistent with the imaging findings. No evidence of malignancy. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS: 
The breasts are almost entirely fatty.

There are no suspicious masses, architectural distortion, or significant asymmetries in either breast. 

Calcifications: In the upper outer quadrant of the left breast, there is a new group of fine pleomorphic calcifications in a segmental distribution, spanning an area of approximately 2 cm. These calcifications are suspicious for ductal carcinoma in situ (DCIS). No suspicious calcifications are identified in the right breast.

IMPRESSION:
BI-RADS 4B: Suspicious calcifications in the left breast. Stereotactic biopsy is recommended for further evaluation. 
BI-RADS 2: Benign findings in the right breast.

ADDENDUM FOLLOWING STEREOTACTIC BIOPSY OF LEFT BREAST:
Stereotactic biopsy of the group of suspicious calcifications in the left breast at 2 o'clock, 6 cm from the nipple, was performed using a 9-gauge vacuum-assisted device. Specimen radiography confirmed the presence of calcifications. 

Post-biopsy mammogram showed successful removal of the targeted calcifications with a biopsy marker clip placed at the site.

Pathology: Ductal carcinoma in situ, intermediate nuclear grade, cribriform and micropapillary types. Estrogen receptor-positive. Progesterone receptor-positive.

FINAL ASSESSMENT: BI-RADS 6: Known biopsy-proven malignancy (DCIS) in the left breast. Surgical consultation is advised. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Bilateral digital screening mammogram.
COMPARISON: Compared to prior exam dated 1/15/2022.
FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense (C). This may lower the sensitivity of mammography. No dominant mass, suspicious calcifications, or other findings.
LEFT BREAST:
The breast is heterogeneously dense (C). This may lower the sensitivity of mammography. No dominant mass, suspicious calcifications, or other findings. Stable benign calcifications in the upper outer quadrant, unchanged from prior exams.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breast tissue bilaterally, which can obscure small masses. Recommend annual screening with mammography.
BIRADS Category 1: Negative.
ADDENDUM:
Comparison to prior outside screening mammogram from 10/2/2021 shows no significant interval change. Stable benign calcifications in the left breast as described above. ➡ *Negative*
- REPORT:
"BILATERAL DIGITAL SCREENING MAMMOGRAPHY
INDICATION: Routine screening. No current breast problems.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of each breast.
FINDINGS:
Both breasts show scattered areas of fibroglandular density.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
Left breast:
At the 6 o'clock position of the left breast, approximately 3 cm from the nipple, there is a small cluster of amorphous microcalcifications within an area measuring 5 mm. Additional magnification views confirm these calcifications.
Right breast:
No suspicious findings.
IMPRESSION:
BI-RADS Category 0: Incomplete. Need additional imaging evaluation. Small cluster of amorphous microcalcifications in the left breast. Additional magnification views are recommended for further characterization. Depending on the morphology and distribution of these calcifications on magnification views, a stereotactic core biopsy may be indicated.
BI-RADS Category 1 for the right breast: Negative. Routine screening recommended in 1 year.
ADDENDUM:
Magnification views of the left breast demonstrate that the previously identified amorphous calcifications at 6 o'clock position have a grouped distribution within an area measuring 6 mm. Given the new onset of these calcifications and their suspicious morphology, a stereotactic core biopsy is recommended for definitive characterization.
FINAL ASSESSMENT:
BI-RADS Category 4 for the left breast: Suspicious abnormality. Biopsy should be considered.
BI-RADS Category 1 for the right breast: Negative." ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
At the 6 o'clock position of the left breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications within an area measuring 8 mm. This is a new finding compared to the patient's prior exams.
The right breast is unremarkable.
IMPRESSION:
BI-RADS 0: Incomplete, additional imaging evaluation needed. An area of amorphous microcalcifications is identified in the left breast at 6 o'clock, 6 cm from the nipple. Additional magnification views are recommended for further characterization.
RECOMMENDATION:
Left diagnostic mammogram with magnification views. The patient should then return to the breast center for additional management, which may include biopsy.
ADDENDUM:
Left diagnostic mammogram with magnification views demonstrates a group of amorphous microcalcifications at the 6 o'clock position of the left breast, 6 cm from the nipple. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS).
BI-RADS 4B: Suspicious abnormality, biopsy should be considered.
RECOMMENDATION:
Left breast ultrasound and ultrasound-guided core biopsy of the area of microcalcifications. ➡ *Positive*
- MAMMOGRAPHY - BILATERAL - SCREENING. CLINICAL HISTORY: Screening mammogram. TECHNIQUE: Craniocaudal and mediolateral oblique views of the breasts were performed using digital mammography. COMPARISON: Comparison is made to the mammogram from 10/17/2023. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: 43 year old asymptomatic female with negative personal and family history undergoing routine screening.
COMPARISON: None available.
FINDINGS:
Both breasts are heterogeneously dense (C), which could obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in either breast.
There are scattered areas of bilateral fibroglandular density.
At the 6 o'clock position of the left breast, 5 cm from the nipple, there is a 5 mm cluster of amorphous calcifications. Additional magnification views confirm these calcifications are grouped and suspicious.
The skin is normal. No axillary, supra or infraclavicular adenopathy.
IMPRESSION:
BI-RADS 4: Suspicious abnormality in the left breast. Stereotactic biopsy recommended for the cluster of amorphous calcifications at 6 o'clock position, 5 cm from nipple.
BI-RADS 2: Benign findings in the right breast. Recommend continued routine screening in 1 year.
ADDENDUM: The patient returned for stereotactic biopsy of the cluster of calcifications in the left breast at the 6 o'clock position, 5 cm from the nipple. Specimen radiography confirmed retrieval of calcifications. Pathology results: Ductal carcinoma in situ, intermediate nuclear grade, involving a 6 mm area. Estrogen receptor positive. Progesterone receptor positive. Recommendation is for surgical consultation. ➡ *Positive*
- MAMMOGRAM RIGHT BREAST.
HISTORY: Screening mammogram.
COMPARISONS: None available.
TECHNIQUE: Standard CC and MLO views of the right breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative right mammogram. BI-RADS 2.
RECOMMENDATION:
Correlation with recent mammograms is recommended when available. ➡ *Negative*
- FINDINGS:
The breasts are heterogeneously dense. Scattered areas of fibroglandular density.
A 12 mm irregular mass with indistinct margins and associated architectural distortion is noted at 2:00, 6 cm from the nipple in the right breast. 
Directly adjacent to this mass, there is a group of pleomorphic microcalcifications spanning approximately 8 mm. The calcifications are linear, branching and vary in size and shape, highly suspicious for malignancy. 
A prominent axillary lymph node is identified, measuring 15 mm in maximal dimension with marked cortical thickening up to 6 mm. This is a suspicious finding.
IMPRESSION:
Highly suspicious irregular mass with associated pleomorphic microcalcifications and architectural distortion right breast at 2:00, 6 cm from the nipple. Suspicious right axillary lymph node. Recommend biopsy of both the right breast mass and right axillary node for histologic diagnosis.
BIRADS 5. ➡ *Positive*
- CLINICAL HISTORY: Patient presents for routine screening mammogram. No breast complaints.

FINDINGS:
Right breast:
The right breast is almost entirely fatty. There are no masses, architectural distortions or microcalcifications. 

Left breast: 
The left breast is heterogeneously dense. This may lower the sensitivity of mammography. No dominant masses or suspicious clustered microcalcifications are seen. There is a partially imaged well-circumscribed mass in the far posterior right breast on MLO view only, likely representing a lymph node. No architectural distortion or skin thickening.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Partially imaged benign-appearing mass in the far posterior right breast on MLO view only, likely a lymph node. This is a benign finding. 
3. The breasts are heterogeneously dense, which may lower the sensitivity of mammography. 

BIRADS Category 1: Negative. Recommend routine screening in 1 year. ➡ *Negative*
- ULTRASOUND LEFT BREAST
CLINICAL HISTORY: 35-year-old female with palpable lump in the upper outer quadrant of the left breast. 
TECHNIQUE: Targeted ultrasound of the left breast was performed using a 12 MHz linear array transducer. 
COMPARISON: None available.
FINDINGS: The left breast is heterogeneously echogenic. A 1.2 x 0.8 x 0.6 cm oval, circumscribed, hypoechoic mass is identified at the 10 o'clock position, 3 cm from the nipple, corresponding to the palpable abnormality. The mass demonstrates no posterior acoustic features and has parallel orientation. No suspicious calcifications are identified. 
The remainder of the left breast is unremarkable. No axillary lymphadenopathy is seen.
IMPRESSION:
1. 1.2 cm circumscribed oval mass at 10 o'clock position of the left breast, likely benign. Recommend follow-up ultrasound in 6 months to document stability. 
2. No suspicious calcifications are identified in the left breast.
BI-RADS Category 3: Probably Benign Finding. Short interval follow-up suggested. ➡ *Negative*
- ULTRASOUND AND MAMMOGRAM, RIGHT BREAST
CLINICAL HISTORY: 32-year-old female with right breast lump, upper inner quadrant, suspicious for cancer on clinical exam. Family history of breast cancer in mother. 
COMPARISON: None available.
FINDINGS:
Right breast ultrasound demonstrates a 1 cm solid, hypoechoic mass at 11 o'clock position, 3 cm from the nipple. The mass demonstrates irregular margins and increased vascularity. No significant axillary lymphadenopathy.
Right mammogram demonstrates heterogeneously dense breast tissue. There is a 1 cm mass in the upper inner quadrant corresponding to the lesion seen on ultrasound. No suspicious calcifications or architectural distortion.
IMPRESSION: 
Sonographic and mammographic findings are highly suspicious for malignancy, BIRADS 5. Ultrasound-guided biopsy is recommended for definitive diagnosis. 
No mammographic microcalcifications are identified. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Screening mammogram from 02/14/2020.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

In the left breast at the 9:00 position, 6 cm from the nipple, there is a group of amorphous microcalcifications that are new compared to prior exam. These calcifications are suspicious and require further workup.

Stable calcifications are noted in both breasts, likely representing benign secretory calcifications.

IMPRESSION:
BI-RADS 0: Incomplete. Additional imaging evaluation of the left breast is required due to new group of suspicious microcalcifications at the 9:00 position. Magnification views are recommended for further characterization.

ADDENDUM (02/20/2023):
LEFT DIAGNOSTIC MAMMOGRAM

Magnification views of the left breast were performed, demonstrating a group of pleomorphic microcalcifications spanning an area of approximately 8 mm at the 9:00 position, 6 cm from the nipple. The distribution and morphology of these calcifications are highly suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended for definitive diagnosis.

IMPRESSION:
BI-RADS 4C: Highly suspicious finding. Biopsy should be considered. ➡ *Positive*
- SCREENING MAMMOGRAM.  
INDICATION:  Screening.  Annual exam.

TECHNIQUE:  Standard CC and MLO views bilaterally.  

COMPARISON:  Comparison is made to outside screening mammogram from April 2021.

FINDINGS:  
The breasts are heterogeneously dense which may obscure detection of small masses.  

There are no dominant masses, architectural distortion or suspicious calcifications.

Stable benign appearing calcifications are noted in the right breast, likely representing fibroadenoma.  

There are no significant interval changes compared to prior outside exam.  

IMPRESSION:
1.  No mammographic evidence of malignancy.  
2.  Heterogeneously dense breasts.

RECOMMENDATION:  
Continue annual screening mammography in 12 months.  

BIRADS 2 - Benign findings.  

ADDENDUM:  No microcalcifications are identified in the current study to indicate a BIRADS 0 assessment.  
Clinical correlation is recommended for any palpable abnormalities. ➡ *Not Stated*
- FINDINGS:
Bilateral digital screening mammogram was performed. The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast. 

In the left breast, there is a new cluster of fine pleomorphic calcifications in the upper outer quadrant at approximately 1-2 o'clock position, measuring about 1 cm in diameter. No associated mass or architectural distortion is seen. 

IMPRESSION:
Right breast: Negative. BI-RADS 1.

Left breast: New cluster of suspicious microcalcifications in the upper outer quadrant, BI-RADS 4. Biopsy is recommended for further evaluation. Overall BI-RADS 4. ➡ *Positive*
- MAMMOGRAPHY REPORT

INDICATION: Screening

TECHNIQUE: Bilateral diagnostic mammography including spot compression views of areas of interest. Digital tomosynthesis was also performed.

COMPARISON: Prior study from 8/19/2019

FINDINGS:

The breast tissue is heterogeneously dense, which may obscure small masses.

There are no dominant masses, suspicious calcifications, or other findings.

The spot compression views show no significant abnormality in the areas of concern.

Tomosynthesis slices through the breasts reveal no additional abnormalities.

IMPRESSION:

BIRADS Category 1: Negative mammogram. Recommend routine screening in 1 year. ➡ *Not Stated*
- FINDINGS: There are scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year.
2. Entirely fatty replaced breasts.

BIRADS 1 - Negative. ➡ *Negative*
- HISTORY: Screening mammogram. No breast complaints.
FINDINGS:
Right breast: The breast is composed of scattered fibroglandular densities. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
Left breast: The breast is composed of scattered fibroglandular densities. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend routine annual screening mammography in 1 year.
BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS WITH TOMOSYNTHESIS, SPOT COMPRESSION VIEWS OF THE RIGHT BREAST, AND TARGETED ULTRASOUND RIGHT BREAST
History: 52 year old with suspicious microcalcifications upper outer right breast. No previous imaging available for comparison.

FINDINGS: The breasts are heterogeneously dense.
In the right breast at 11:00, 7 cm from the nipple there is a suspicious cluster of pleomorphic microcalcifications spanning approximately 11 mm. Spot compression views magnify this cluster but show no associated mammographic findings. Targeted ultrasound was performed at the location of this cluster but no associated mass, distortion, or intraductal changes were seen.
The remainder of the right breast is unremarkable. There are no dominant masses or additional suspicious microcalcifications. No skin thickening or nipple retraction.
The left breast is unremarkable. No dominant masses, suspicious microcalcifications or other findings.

IMPRESSION:
Right breast with a suspicious 11 mm cluster of pleomorphic microcalcifications at 11:00, 7 cm from the nipple. No sonographic correlate. This cluster is highly suspicious for DCIS and stereotactic core biopsy is recommended. BIRADS 4C.
Normal/Benign left breast. BIRADS 1. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of each breast were obtained. 

FINDINGS:

Breast composition: The breasts are heterogeneously dense, which may obscure detection of small masses.

Masses: No suspicious masses are identified in either breast. 

Calcifications: There are scattered punctate microcalcifications in the upper outer left breast, a few in a segmental distribution at the 2:00 position. No suspicious microcalcifications are seen in the right breast.

Architectural distortion: No suspicious architectural distortion is identified.

Asymmetries: No suspicious asymmetries are appreciated.

Skin: The skin appears normal. No skin thickening or retraction is identified.

Axillary lymph nodes: No enlarged axillary lymph nodes are identified.

Implants: None.

Other findings: None.

IMPRESSION:
BIRADS 0: Incomplete. Magnification views of the left breast in the CC and MLO projection are recommended for further evaluation of the segmental microcalcifications in the upper outer left breast. Please note these microcalcifications were not called on prior studies. 
  
RECOMMENDATION: Additional imaging evaluation with magnification views of the left breast is recommended. The patient is encouraged to return to the Breast Center for these additional views, which have been scheduled for [DATE]. The final assessment will be given after completion of additional imaging. ➡ *Positive*
- FINDINGS:
Right breast: Unremarkable. No dominant mass, architectural distortion, or suspicious calcifications.
Left breast: Scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications. Stable appearance compared to prior mammogram from 12/15/2021.

IMPRESSION:
1. Bilateral mammogram, no evidence of malignancy. Annual screening mammography is recommended. BI-RADS 1 - Negative.

2. Ultrasound, left breast 6:00 position: No sonographic correlate for palpable abnormality. Likely normal fibroglandular tissue.

ADDENDUM:
After review of the images and discussion with the ordering physician, no additional imaging is recommended at this time. Clinical follow up is advised for the palpable left breast abnormality. If clinical concern persists, short term follow up imaging could be considered to document stability. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.
CLINICAL HISTORY: Annual screening mammogram. No prior mammograms available for comparison.
TECHNIQUE: Bilateral screening mammography was performed including CC and MLO views of each breast.
FINDINGS:
There are scattered areas of fibroglandular density. No focal asymmetry, mass, architectural distortion, or suspicious calcifications are identified in either breast.
IMPRESSION:
Negative bilateral screening mammogram.
BIRADS 1: Recommend routine screening in 1 year. ➡ *Negative*
- NORMAL TMIST DBT ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the digital breast tomosynthesis arm.
Compared to Previous: No previous mammogram available for comparison.  
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
OBSP to recall client in 1 year: Yes.
The breasts are heterogeneously dense. No dominant masses, architectural distortion or suspicious calcifications are identified.  
BIRADS-2. ➡ *Not Stated*
- RADIOLOGY REPORT
EXAMINATION: Screening Mammogram 
CLINICAL HISTORY: 45-year-old female presents for routine screening mammogram. No current breast complaints.
COMPARISON: Screening mammogram from 1 year prior. 
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  
FINDINGS:  
The breast tissue is heterogeneously dense, which may obscure small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.  
Stable bilateral benign-appearing calcifications are noted.
No significant interval change compared to prior mammogram. 
IMPRESSION: 
No mammographic evidence of malignancy. Recommend continued annual screening in 12 months. 
BIRADS Category 1 - Negative. ➡ *Negative*
- SCREENING MAMMOGRAM
CLINICAL INFORMATION: Annual screening mammogram
COMPARISON: None.
TECHNIQUE: Full field digital mammography
FINDINGS:
Breast density: The breasts are heterogeneously dense, which may obscure small masses. (BI-RADS category C)
No suspicious masses, architectural distortion or definite calcifications. 
Stable benign calcifications noted in the left breast, likely vascular.
IMPRESSION:
Negative screening mammogram. 
BI-RADS Category 1: Negative. Recommend routine screening in 1 year.
ADDENDUM: 
Compared with the most recent prior examination, no mammographic evidence of malignancy is identified. Scattered fibroglandular densities noted in both breasts. No significant masses, architectural distortion, or suspicious calcifications. ➡ *Not Stated*
- SCREENING MAMMOGRAM:
INDICATION: This is an OBSP Screening Mammogram. Previous mammogram on 10/12/2019 at this facility.

BREAST COMPOSITION: The breasts are heterogeneously dense, which may obscure small masses.

FINDINGS:
There are no mammographic findings suspicious for malignancy. No suspicious masses, architectural distortion, or mammographic signs of malignancy. 

Benign calcifications are present in both breasts, likely representing fibrocystic changes. No suspicious microcalcifications are identified.

IMPRESSION:
BIRADS Category 2: Benign. 
Recommend routine screening in 1 year per OBSP guidelines.

ADDENDUM: 
Patient notified of results via letter. ➡ *Not Stated*
- BILATERAL DIAGNOSTIC MAMMOGRAM FOR SCREENING. Clinical History: The patient is a pleasant 60-year-old female presenting for her annual screening mammogram today. Comparison: Comparison is made to mammograms performed on 02/02/2024. Findings: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcifications. Impression: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1. ➡ *Negative*
- MAMMOGRAM - BILATERAL.

HISTORY: 42 year old female with family history of breast cancer, presenting for screening mammogram.

FINDINGS:
Both breasts appear symmetric and are composed of scattered fibroglandular densities. No dominant masses, architectural distortion or suspicious calcifications. Stable postoperative changes noted in the right breast from prior lumpectomy. No mammographic evidence of malignancy. Benign-appearing lymph nodes noted in both axillae.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Postoperative changes right breast, stable.
3. Benign-appearing lymph nodes bilaterally. 

BIRADS 1 - Negative. Routine screening recommended in 1 year. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
INDICATION: Screening mammogram.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior exams dated 5/15/2021 and 5/20/2020.
FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.
There are no dominant masses, architectural distortion, or suspicious calcifications.
There are scattered areas of fibroglandular density bilaterally, which are stable compared to prior exams.
IMPRESSION:
1. No mammographic evidence of malignancy.
2. Breast tissue is heterogeneously dense, which may lower the sensitivity of mammography. 
BIRADS Category 1: Negative.
RECOMMENDATIONS:
Recommend continued annual screening mammography in 12 months.
ADDENDUM:
Comparison to prior ultrasounds dated 5/15/2021 and 5/20/2020 reveals no abnormality.
No breast ultrasound is indicated at this time based on mammographic findings. ➡ *Negative*
- FINDINGS:
Right breast tissue is heterogeneously dense. There are no dominant masses, architectural distortion or suspicious calcifications.
Left breast tissue is almost entirely fatty. There are no dominant masses, architectural distortion or suspicious calcifications.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Heterogeneously dense breast tissue in the right breast could obscure small masses.
2. Recommend annual screening mammography in 12 months. 
BIRADS 1 - Negative. ➡ *Negative*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
There are no suspicious masses, architectural distortions, or significant asymmetries. The breast tissue is heterogeneously dense.
BIRADS-2.

ADDENDUM: 
Prior studies from 2019 and 2020 were reviewed for comparison. No significant interval change is observed. The breast tissue remains heterogeneously dense. No suspicious findings are identified. ➡ *Not Stated*
- BREAST ULTRASOUND REPORT

Clinical History:
Patient referred for breast ultrasound evaluation due to suspicious calcifications identified on screening mammography.

Comparison: Mammogram dated 06/13/2024.

Findings:
Left breast: No distinct masses or other suspicious abnormalities are identified in the left breast.
Right breast: In the right breast, at the location of the suspicious calcifications seen on mammography (3 o'clock position, approximately 2 cm from the nipple), there is a small, ill-defined area of hypoechogenicity measuring approximately 6 mm in greatest dimension. This area corresponds to the cluster of calcifications. No distinct mass is visualized. There is no associated ductal dilation.

Impression:
The ultrasound findings in the right breast are consistent with the mammographically detected calcifications. The presence of the ill-defined hypoechoic area is of uncertain significance and could represent a small focus of malignancy. Given the concerning mammographic features, biopsy is recommended for definitive diagnosis.

Recommendations:
Ultrasound-guided biopsy of the right breast calcifications.

BI-RADS Category: 4 (Suspicious for Malignancy) ➡ *Positive*
- FINDINGS: The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications are identified. Stable benign calcifications are noted in the right breast. There are no new mammographic findings compared to prior exam from 2 years ago.

Targeted ultrasound was performed at the site of palpable abnormality in the right breast at 2 o'clock, 4 cm from the nipple. No sonographic correlate was identified. The palpable area corresponds to normal fibroglandular tissue.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography.
2. No sonographic correlate for the palpable finding in the right breast at 2 o'clock. This corresponds to normal fibroglandular tissue. Clinical correlation recommended.

BIRADS 1 - Negative. ➡ *Negative*
- SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: None

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.
No suspicious masses, architectural distortion, or suspicious calcifications are seen in either breast.
Stable benign-appearing bilateral breast tissue.
No mammographic evidence of malignancy.

IMPRESSION:
BIRADS 2 - Benign.
Recommend routine screening in 1 year.

No mention of microcalcifications. ➡ *Not Stated*
- BILATERAL MAMMOGRAM - SCREENING. CLINICAL HISTORY: 47-year-old female presenting for her screening mammogram. Patient reports no personal history of breast cancer. No family history of breast cancer. TECHNIQUE: Standard CC and MLO views were obtained. COMPARISON: Comparison is made to the patient's prior mammogram dated 02/22/2024. FINDINGS: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications bilaterally. There is no mammographic evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1: Negative. ➡ *Negative*
- TMIST MAMMOGRAM FFDM ARM
The patient participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
COMPARISON: None.
FINDINGS:  
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
Bilateral, scattered areas of fibroglandular density.
There are no dominant masses, areas of architectural distortion or suspicious calcifications. 
IMPRESSIONS:
BIRADS Category 1: Negative. Recommend routine screening in 12 months.
No suspicious mammographic findings. ➡ *Not Stated*
- TECHNIQUE: Digital diagnostic mammography, left breast. Ultrasound, left breast.
COMPARISON: Prior mammograms from 3/15/2020 at St. Mary's Hospital.
HISTORY: 67-year-old female with palpable abnormality, left breast at 3 o'clock position per patient.
FINDINGS:
LEFT MAMMOGRAM: 
The left breast is heterogeneously dense. No suspicious masses, architectural distortion or suspicious calcifications. No significant change from prior exam.
LEFT BREAST FOCUSED ULTRASOUND:  
Targeted ultrasound was performed at the site of the palpable abnormality. At 3 o'clock position 3 cm from the nipple, there is an oval parallel hypoechoic mass with circumscribed margins measuring 6 x 4 x 5 mm. This is consistent with a benign finding such as a cyst or fibroadenoma. No suspicious findings.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography.
2. Benign appearing palpable mass on ultrasound at 3 o'clock position of the left breast, likely a cyst or fibroadenoma. 6-month follow-up ultrasound is recommended to document stability. Aspiration may be performed if symptomatic.
ASSESSMENT: BI-RADS 2 Benign Finding. Recommend routine screening. ➡ *Negative*
- SCREENING MAMMOGRAM TOMOSYNTHESIS.

INDICATION: Annual screening mammogram.

COMPARISON: Screening mammogram [**Hospital**] from [**DATE**].

TECHNIQUE: Screening bilateral digital breast tomosynthesis.

FINDINGS: The breasts are heterogeneously dense (C). This may lower the sensitivity of mammography.

There has been no change in the appearance of the right breast. No suspicious mammographic findings. Benign-appearing vascular calcifications in the upper outer right breast.

In the left breast at 2 o'clock 5 cm from the nipple there is a new small cluster of pleomorphic microcalcifications spanning 6 mm. These are high-risk for malignancy.

IMPRESSION:
New grouped microcalcifications left breast highly suspicious for malignancy. BIRADS 5. Recommend stereotactic core biopsy with clip placement.
The patient is aware and agrees with the plan for core biopsy. Requisition sent.

No suspicious mammographic findings right breast. BIRADS 2.

ADDENDUM: [**DATE**] Patient had stereotactic core biopsy left breast ([**PROCEDURE ID**]). Specimen radiograph confirmed calcifications. ➡ *Positive*
- MAMMOGRAM - BILATERAL   

INDICATION: Annual screening mammogram

COMPARISON: Prior exams 10/05/2021 at Sibley Memorial Hospital 

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.   

FINDINGS:  

The breast tissue is heterogeneously dense which may obscure detection of small masses. 

There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable coarse benign appearing calcifications are seen bilaterally. 

There are no significant changes compared to prior mammograms.

IMPRESSION:

BI-RADS 1 - Negative.

Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- FINDINGS:
Bilateral digital screening mammograms demonstrate heterogeneously dense breasts. In the upper outer quadrant of the left breast, there is a new cluster of pleomorphic microcalcifications spanning an area of approximately 1 cm. The calcifications are fine, linear and branching in morphology, raising concern for ductal carcinoma in situ (DCIS).

The remainder of the left breast shows predominantly fatty tissue without suspicious masses, architectural distortion, or additional calcifications.

The right breast is predominantly fatty and does not show any suspicious masses, calcifications or areas of architectural distortion.

IMPRESSION:
BI-RADS 4: Suspicious microcalcifications in the left breast at 2 o'clock, 6 cm from the nipple. Stereotactic biopsy is recommended for further evaluation. The remainder of the left breast and right breast are negative.

ADDENDUM:
The patient returned for stereotactic biopsy of the left breast microcalcifications. Specimen radiograph confirms that the calcifications are present within the core samples obtained. Final pathology results are pending. ➡ *Positive*
- DIAGNOSTIC BILATERAL MAMMOGRAM WITH ULTRASOUND
CLINICAL HISTORY: 46 year old female with a palpable right breast mass. No personal or family history of breast cancer.
COMPARISON: None available.
FINDINGS:
Right breast: Heterogeneously dense tissue (ACR category C). At 10 o'clock position 3 cm from the nipple, there is a 1.5 x 1.2 cm irregular mass. Ultrasound shows this to be a solid mass, hypoechoic with indistinct margins and posterior shadowing, highly suspicious for malignancy. No other suspicious masses, calcifications, or architectural distortion.
Left breast: Scattered fibroglandular densities (ACR category B). At 12 o'clock, 6 cm from the nipple, there is a new group of fine pleomorphic microcalcifications spanning approximately 8 mm. No associated mass, asymmetry, or architectural distortion. Ultrasound was unremarkable.
Axillae: Unremarkable.
IMPRESSION:
1. Right breast 10 o'clock 1.5 cm solid mass, highly suspicious for malignancy. Ultrasound-guided core needle biopsy recommended. BI-RADS 5.
2. Left breast new group of suspicious microcalcifications at 12 o'clock. Stereotactic-guided biopsy recommended. BI-RADS 4c.
ADDENDUM: Patient proceeded with ultrasound-guided core biopsy of the right breast mass, pathology pending. Stereotactic biopsy of the left breast calcifications scheduled for next week. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM. Clinical History: 49 year old female presents for her annual screening mammogram. Comparison is made to prior mammogram from 09/12/2023. Findings: The breasts are heterogeneously dense. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: Negative mammogram. BI-RADS 1. ➡ *Negative*
- SCREENING MAMMOGRAM. 
INDICATION: Annual screening mammogram.
TECHNIQUE: Full field digital mammography, left and right MLO & CC views. 
COMPARISON: None available. 
FINDINGS:
The breast tissue is heterogeneously dense which could obscure detection of small masses. No dominant mass, suspicious calcifications or architectural distortion.  
Skin, nipple and axillary regions appear within normal limits.
IMPRESSION: 
Negative screening mammogram. No mammographic evidence of malignancy. 
BIRADS Category 1: Negative. 
Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAPHY
INDICATION: Screening
TECHNIQUE: Screening views were obtained including scanned and tomosynthesis images.
COMPARISON: 2 years
FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.
Right breast:
No suspicious masses, calcifications, or other findings.
Left breast: 
In the left breast, there is a new small 6 mm cluster of pleomorphic calcifications in the left upper outer quadrant at 2:00, 5 cm from the nipple. The cluster persists on spot compression magnification views. 
No suspicious masses or other findings in the left breast.
IMPRESSION: 
BI-RADS 4B: Suspicious abnormality in the left breast.  Small cluster of pleomorphic calcifications left breast. Stereotactic core biopsy is recommended. 
BI-RADS 2: Benign findings right breast.
ADDENDUM: The patient returned to radiology and underwent stereotactic core biopsy of the left breast cluster of calcifications with pathology demonstrating ductal carcinoma in situ, nuclear grade 2, cribriform and micropapillary types. Definitive surgical excision to negative margins with sentinel lymph node biopsy is recommended per discussion at the breast tumor board conference. Final BI-RADS for the left breast is upgraded to category 6 (Biopsy proven malignancy, known cancer diagnosis). ➡ *Positive*
- BILATERAL MAMMOGRAM WITH MAGNIFICATION VIEWS:
INDICATION: Screening mammogram.
TECHNIQUE: Bilateral CC and MLO views as well as spot compression magnification views of a group of calcifications were performed. Comparison is made to previous exams from 2018.
FINDINGS:
The breast tissue is heterogeneously dense which may obscure detection of small masses.
There are no new dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
In the LEFT breast at the 9 o'clock position 5 cm from the nipple there is a cluster of indeterminate microcalcifications unchanged from the prior exam. These calcifications were previously assessed as probably benign and are stable in appearance. 
In the LEFT breast at the 3 o'clock position 7 cm from the nipple, there are several new punctate microcalcifications seen on today's exam. These are better evaluated on the spot compression magnification views. No associated mass or distortion is seen.
In the RIGHT breast, there are scattered fibroglandular elements in the central breast. No suspicious masses, calcifications or areas of architectural distortion are identified.
Magnification views of the LEFT breast demonstrate pleomorphic microcalcifications in a segmental distribution at the site of clinical concern measuring approximately 8 mm in greatest dimension. These calcifications are new from the prior examination.
IMPRESSION:
1. New group of suspicious pleomorphic microcalcifications at 3 o'clock position of the LEFT breast, BI-RADS 4B, suspicious. Stereotactic core biopsy is recommended. 
2. Probably benign group of calcifications in the LEFT breast at 9 o'clock position, unchanged from prior exam. BI-RADS 3, short term follow up in 6 months recommended.
3. No mammographic evidence of malignancy in the RIGHT breast, BI-RADS 2. ➡ *Positive*
- MAMMOGRAPHY AND TARGETED ULTRASOUND OF THE LEFT BREAST

CLINICAL INFORMATION: New palpable lump along the 4 o'clock axis of the left breast as noted by the patient. Last screening mammogram was 1 year ago at an outside facility, reported as BIRADS 2 benign. Patient has a strong family history of breast cancer in her maternal aunt and grandmother. Status post right total mastectomy in 2010 for breast cancer.

COMPARISON: Outside mammographic images from 1 year prior.

FINDINGS:
RIGHT BREAST: Post-mastectomy changes noted. 

LEFT BREAST MAMMOGRAPHY: Scattered fibroglandular tissue. No suspicious masses, calcifications, or areas of architectural distortion. Stable compared to outside prior mammograms. 

TARGETED LEFT BREAST ULTRASOUND AT 4 O'CLOCK AXIS: Demonstrates normal fibroglandular tissue in the area of palpable concern. No discrete solid mass, complex cystic lesion, or suspicious finding.

IMPRESSION: 
1. Stable left breast mammogram. No mammographic evidence of malignancy. Recommend continued routine annual screening mammography in 1 year.

2. Targeted left breast ultrasound is negative for correlate at the site of palpable concern along the 4 o'clock axis. Recommend clinical follow up and correlation as indicated.

BIRADS 1 - NEGATIVE

ADDENDUM: This case was reviewed with Dr. Smith following initial interpretation by the resident. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

TECHNIQUE: Standard CC, MLO, and magnification views of both breasts were obtained.

COMPARISON: None available

FINDINGS:

Right Breast:
The right breast is heterogeneously dense (BI-RADS density category C). There are no dominant masses, focal asymmetries, or areas of architectural distortion. No suspicious calcifications are identified.  

Left Breast:
The left breast is heterogeneously dense (BI-RADS density category C). There are no dominant masses or focal asymmetries. No areas of architectural distortion are seen. 

In the upper outer quadrant of the left breast, there is a small cluster of amorphous microcalcifications at approximately 2 o'clock, 5 cm from the nipple. This cluster measures approximately 6 mm in greatest dimension. The calcifications are new compared to prior exams. No other suspicious calcifications are identified in the left breast.

IMPRESSION: 

Right Breast: Negative mammogram (BI-RADS 1)
Left Breast: Small cluster of amorphous microcalcifications in the upper outer quadrant, new since prior exam. Suspicious for malignancy (BI-RADS 4). Stereotactic biopsy is recommended for further evaluation.

ADDENDUM:
The patient returned for stereotactic biopsy of the cluster of microcalcifications in the left breast at 2:00, 5 cm from the nipple. Specimen radiograph confirmed the presence of calcifications within the core samples. Post-biopsy mammogram showed deployment of a biopsy marker clip at the site of sampling. The patient tolerated the procedure well. Pathology results are pending. ➡ *Positive*
- FINDINGS:
There are scattered areas of fibroglandular density. 
No suspicious masses, architectural distortion or suspicious calcifications are seen in either breast.
At the 10 o'clock position of the right breast there is a group of amorphous calcifications spanning approximately 3 mm. 

IMPRESSION:
BI-RADS Category 0: Incomplete. 
Additional imaging evaluation with magnification views is recommended for further characterization of the group of amorphous calcifications at the 10 o'clock position of the right breast.

ADDENDUM:
Right breast magnification views demonstrate a cluster of amorphous calcifications at the 10 o'clock position.

IMPRESSION (ADDENDUM):
BI-RADS Category 4B: Suspicious - Moderate suspicion for malignancy. Biopsy should be considered. Small cluster of amorphous calcifications right breast requires tissue diagnosis. ➡ *Positive*
- SCREENING MAMMOGRAM.
Compared to previous screening mammogram from April 2019, the breast tissue is stable in appearance with no suspicious findings. The breast tissue is predominantly fatty.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
IMPRESSION:
No mammographic evidence of malignancy. Routine annual screening mammography is recommended.
BIRADS 2 - Benign findings. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.
No dominant masses, architectural distortion or definite asymmetries.
At the 12 o'clock position of the right breast, approximately 4 cm from the nipple, there is a new group of amorphous microcalcifications spanning an area of about 8 mm. This is a suspicious finding.
No concerning calcifications in the left breast.
There are stable bilateral axillary lymph nodes, likely benign.
Skin is unremarkable. No evidence of prior surgery.
IMPRESSION:
BIRADS 4: Suspicious grouped amorphous microcalcifications in the right breast at 12 o'clock position, 4 cm from the nipple. Biopsy is recommended for further evaluation. Pathology results should be correlated with imaging findings for appropriate management.
BIRADS 2: Benign findings in the left breast.
ADDENDUM: Stereotactic biopsy of the right breast microcalcifications at 12 o'clock was performed. Specimen radiograph confirmed calcifications were sampled. Final pathology results showed ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. Wide local excision is recommended for complete removal and definitive treatment of the DCIS. 6 month follow-up mammogram is advised to ensure stability and evaluate for any residual calcifications. ➡ *Positive*
- FINDINGS: The breasts are almost entirely fatty. No suspicious masses, architectural distortion, or dominant masses. Specifically, no clustered suspicious microcalcifications.

IMPRESSION:

1. Negative digital screening mammogram. No dominant mass, architectural distortion or clustered suspicious microcalcifications.

2. Recommend routine screening in 1 year.

BI-RADS Category 1: Negative. ➡ *Negative*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This patient is participating in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.  
Compared to previous exam, there are no significant changes.
Breasts are predominantly fatty.
No architectural distortion, dominant masses or suspicious calcifications are identified.
There are no mammographic findings suspicious for malignancy. Routine screening recommended.  
BIRADS-2.
ADDENDUM: After review and discussion with the patient, given family history of breast cancer in a first degree relative diagnosed at age 45, earlier screening mammography is advised, beginning at age 40. Patient agrees with plan for annual screening mammograms. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.

The breast tissue is heterogeneously dense. There are no dominant masses, architectural distortion or suspicious calcifications. 

IMPRESSION:
BIRADS-2 Benign. Routine screening recommended in 2 years. ➡ *Not Stated*
- RIGHT BREAST MAMMOGRAM.
HISTORY: Family history of breast cancer in sister at age 59. No current breast complaints.
FINDINGS: The breast is almost entirely fatty. No dominant masses, suspicious calcifications, or areas of architectural distortion. No significant change from prior exam of 2 years ago.
IMPRESSION:
1. No mammographic evidence of malignancy in the right breast.
2. Predominantly fatty breast tissue.
BIRADS 1 - Recommend routine screening in 1 year. ➡ *Negative*
- FINDINGS: Left breast tissue contains scattered fibroglandular densities. No suspicious masses, architectural distortion, or microcalcifications are identified. Stable appearance compared to prior imaging.

Targeted ultrasound was performed at the site of palpable abnormality in the left breast at 7 o'clock position, 4 cm from the nipple. No sonographic correlate for the palpable finding. No suspicious solid or cystic masses identified.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. Recommend follow up mammogram in 1 year.
2. No sonographic correlate for palpable finding at 7 o'clock position in the left breast. Recommend clinical follow up as indicated.

BI-RADS Category 1: Negative

ADDENDUM: 
This is an addendum to the mammography report signed by Dr. X on 01/01/2025. 
The patient called with concerns about continued palpable abnormality in the left breast at 7 o'clock position. Recommend clinical follow up for further evaluation if clinically warranted. No change in BI-RADS assessment. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Annual screening mammogram.
COMPARISON: None.
FINDINGS: 
There is scattered fibroglandular density.
Right breast:
There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Left breast:
At the 12:00 position 6 cm from the nipple, there is a 0.6 cm group of indeterminate microcalcifications.
No dominant masses or areas of architectural distortion.
IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation
There is a group of indeterminate microcalcifications in the left breast at 12:00, 6 cm from the nipple. Additional magnification views are recommended for further evaluation. A biopsy may be performed if needed following additional imaging.
ADDENDUM: 
Left magnification views demonstrate a 6 mm group of heterogeneous microcalcifications at 12:00, 6 cm from the nipple. Biopsy is recommended. Final BI-RADS 4: Suspicious.
Right breast: Negative. BI-RADS 1. ➡ *Positive*
- TMIST FFDM SCREEN-DETECTED ABNORMALITY.
This patient was enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and randomized to the full field digital mammography arm per study protocol and informed consent. 
Comparison to prior study: Yes.
Breast density: Scattered fibroglandular density.
OBSP recall interval: Annual screening recommended.
There are no suspicious mammographic findings. The breasts are heterogeneously dense. No mention of microcalcifications.
BIRADS Category 2 - Benign findings. 
Recommend routine screening in 12 months. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL INDICATION: Annual screening mammogram
COMPARISON: None available.
FINDINGS:
Both breasts appear heterogeneously dense.
In the right breast, there are no dominant masses, areas of architectural distortion, or suspicious calcifications.
In the left breast at the 12:00 position, 6 cm from the nipple, there is a cluster of amorphous microcalcifications spanning an area of approximately 8 mm. No associated mass or architectural distortion is seen in this area.
The left breast is otherwise unremarkable, with no dominant masses or areas of architectural distortion.
No abnormal axillary lymph nodes are identified.
Skin is normal in thickness and appearance bilaterally.
IMPRESSION:
BI-RADS Category 0: Incomplete, additional imaging needed. A cluster of amorphous microcalcifications in the left breast at 12:00, 6 cm from the nipple, requires further evaluation with magnification views.
RECOMMENDATIONS:
Magnification views of the left breast in the area of concern are recommended for further assessment.
Patient is to return to the Breast Imaging Center for additional workup. Please contact the patient to arrange a diagnostic appointment.

ADDENDUM - MAGNIFICATION VIEWS LEFT BREAST:
Magnification views of the left breast in the area of concern at 12:00 demonstrate a cluster of amorphous microcalcifications spanning approximately 9 mm. The morphology and distribution of these calcifications are suspicious for ductal carcinoma in situ (DCIS).
FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious abnormality. Stereotactic biopsy is recommended for definitive diagnosis. ➡ *Positive*
- FINDINGS:
Bilateral mammogram images and bilateral breast ultrasound show heterogeneously dense breast parenchyma. 
Right breast: No mammographic evidence of malignancy or significant abnormality. No sonographic evidence of malignancy. BIRADS 1 right breast.
Left breast: Mammogram demonstrates a cluster of amorphous and pleomorphic calcifications measuring approximately 7 mm at the 10:00 position, middle third depth, 8 cm from the nipple. These have increased in number and are more conspicuous compared to the prior mammogram dated Jan 15, 2020. 
At the site of the microcalcifications in the left breast there is sonographic evidence of a corresponding irregular shaped hypoechoic mass with partially indistinct margins and mild posterior acoustic shadowing. The mass measures approximately 10 x 8 x 6 mm. There is increased surrounding parenchymal echogenicity and distortion, suspicious for associated malignancy. 
Given the suspicious microcalcifications on mammogram and corresponding suspicious mass on ultrasound, this is highly concerning for malignancy, BIRADS 5 left breast.
No other suspicious masses, calcifications or architectural distortion evident in either breast. No skin thickening or nipple changes. No sonographic evidence of multifocality.
No suspicious lymph nodes identified in either axilla. BIRADS 1 right axilla. BIRADS 2 left axilla. 

Addendum: Stereotactic core needle biopsy of the suspicious calcifications at the left breast 10:00 position performed immediately following the diagnostic imaging. Pathology results: Invasive ductal carcinoma, grade 2, ER+ PR+ Her2-, associated with high grade DCIS with comedonecrosis. 

IMPRESSION:
- Highly suspicious cluster of microcalcifications left breast 10:00, proven invasive ductal carcinoma on stereotactic core biopsy. 
- BIRADS 6 left breast. 
- BIRADS 1 right breast. ➡ *Positive*
- CLINICAL INFORMATION: Follow up 6 months post right lumpectomy. Right UO breast calcifications at 11:00 3 cm from nipple.

FINDINGS:
Both breasts are of heterogeneously dense composition.
Surgical scar present at lumpectomy site 10:00 5 cm from the nipple. Adjacent are post surgical changes and scarring and skin thickening. Surgical clips are also present at the surgical site.
Magnification views of the right breast show a 6 mm cluster of calcifications at 11:00 3 cm from the nipple. There is an adjacent 6 mm cluster of calcifications slightly more anteriorly. These are new compared to the previous mammograms from 6 months ago. Morphology is indeterminate. There are a few punctate microcalcifications at 12:00 in the right breast. Posterior depth.
Left breast negative for masses, calcifications or other findings.

IMPRESSION:
1. Interval development of new clusters of indeterminate morphology calcifications 11:00 right breast. 
Recommendation: Given interval development of these the finding is suspicious and stereotactic core biopsy is recommended. BI-RADS 4.

2. Left breast negative. BI-RADS 1. ➡ *Positive*
- FINDINGS:
The breasts are composed of scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Scattered fibroglandular densities.
2. No mammographic evidence of malignancy.

BI-RADS Category 2: Benign findings. Routine screening mammogram recommended in 1 year. ➡ *Not Stated*
- FINDINGS:
Bilateral digital screening mammogram demonstrates the breasts are heterogeneously dense. 
No suspicious mammographic findings or dominant masses in the right breast. 
In the left breast at the 12:00 position 6 cm from the nipple there is a new group of pleomorphic calcifications spanning approximately 8 mm. Spot compression magnification views confirms the presence of a group of suspicious pleomorphic calcifications. 
At the site of the known cancer in the left breast lower outer quadrant, there is a spiculated mass with associated architectural distortion measuring approximately 16 mm. A marker clip is seen at the site of prior biopsy. No sonographic correlate is identified for the group of calcifications at the 12:00 position.

Targeted ultrasound of the left axilla demonstrates cortical thickening of a left axillary lymph node. The cortical thickness measures up to 4 mm.

IMPRESSION:
1. Suspicious pleomorphic calcifications left breast 12:00 position 6 cm from the nipple. Stereotactic biopsy is recommended. BI-RADS 4.
2. Known biopsy proven cancer left breast lower outer quadrant, no significant change. BI-RADS 6.
3. No suspicious findings right breast. BI-RADS 1.

ADDENDUM:
Following discussion with the referring clinician, the patient is planned for mastectomy of the left breast. Therefore stereotactic biopsy of the group of left breast calcifications is not indicated at this time as this will not change management. A preoperative left breast MRI is recommended to evaluate the full extent of disease. ➡ *Positive*
- FINDINGS:
Bilateral digital 2D screening mammogram is compared to prior study dated March 2, 2019. There has been no interval change. 
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
No dominant masses, architectural distortions or suspicious microcalcifications are identified. 
Stable benign appearing vascular calcifications are noted in the medial right breast.
Scattered benign appearing calcifications in both breasts, likely reflecting a combination of fibrocystic changes and ductal calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening in 12 months with digital breast tomosynthesis if available.
2. Heterogeneously dense breasts. The sensitivity of mammography is reduced in dense breasts.

BI-RADS Category 2: Benign findings. Recommend routine screening.

ADDENDUM:
Density comment added. No change in assessment or recommendations. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram in patient with family history of breast cancer in mother diagnosed at age 52.

COMPARISON: None.

FINDINGS:

Both breasts are almost entirely fatty.

At 11:00, 3 cm from the nipple in the RIGHT breast, there is a new cluster of indeterminate microcalcifications spanning approximately 8 mm. No other mammographic abnormalities in the right breast.

No dominant masses, architectural distortion or suspicious calcifications are identified in the LEFT breast. Benign calcifications are noted.

IMPRESSION:

BI-RADS Category 0: Incomplete, needs additional imaging evaluation. New cluster of indeterminate microcalcifications in the right breast at 11:00, 3 cm from the nipple, spanning 8 mm. Recommend spot compression magnification views for further characterization.

BIRADS 2, Benign, for the LEFT breast.

ADDENDUM:
Right breast spot compression magnification views demonstrate a cluster of amorphous microcalcifications at 11:00, 3 cm from the nipple, spanning 8 mm. 

REVISED IMPRESSION:
BIRADS 4, Suspicious, for the RIGHT breast. Stereotactic core biopsy recommended for diagnosis of the amorphous microcalcifications at 11:00.

BIRADS 2, Benign, for the LEFT breast. ➡ *Positive*
- BILATERAL SCREENING DIGITAL MAMMOGRAM
HISTORY: 46-year-old female with no family history of breast cancer. Screening mammogram.
TECHNIQUE: Standard bilateral CC and MLO views were performed. 
COMPARISON: None available.  
FINDINGS:
RIGHT BREAST:
The breast tissue is heterogeneously dense (BI-RADS composition category C). There are no suspicious masses, architectural distortions, or areas of abnormal calcifications. Skin is normal in thickness. Benign-appearing lymph nodes are present in the right axilla.
LEFT BREAST:
The breast tissue is heterogeneously dense (BI-RADS composition category C). There are no suspicious masses or architectural distortions. In the upper outer quadrant, there is a small cluster of pleomorphic microcalcifications spanning an area of approximately 7 mm. Skin is normal in thickness. Benign-appearing lymph nodes are present in the left axilla.
IMPRESSION:
RIGHT BREAST: No suspicious findings. BI-RADS 1 - Negative. Routine screening in 1 year recommended.
LEFT BREAST: Small cluster of pleomorphic microcalcifications in the upper outer quadrant, suspicious for malignancy. BI-RADS 4 - Suspicious abnormality. Stereotactic-guided core needle biopsy recommended for definitive diagnosis.
ADDENDUM: 
The patient was recalled for magnification views of the left breast, which confirmed the presence of a cluster of pleomorphic microcalcifications in the upper outer quadrant. Stereotactic-guided core needle biopsy was performed, demonstrating ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform and solid types. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- SCREENING MAMMOGRAM:

Indication: Screening.

Technique: Standard views of each breast were obtained. The breasts are examined in the craniocaudal and mediolateral oblique projections bilaterally.

Comparison: Compared with prior exam dated 1/15/2021.

Findings:
The breasts are heterogeneously dense, which may obscure small masses.
No concerning masses, architectural distortion, or suspicious calcifications are seen in either breast.
No significant interval change compared to prior exam.

IMPRESSION:
BIRADS 2- BENIGN. 
Recommend routine screening in 1 year.

ADDENDUM:
This study was reviewed in conjunction with any prior examinations and reports. If there is any discrepancy between this report and prior reports, this report shall supersede those prior. ➡ *Not Stated*
- FINDINGS: Bilateral mammographic views demonstrate scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified. Stable benign calcifications are noted in the right breast. No significant interval change compared to prior exam from 12/15/2021.

Targeted right breast ultrasound was performed at the site of palpable abnormality at 10 o'clock, 2 cm from the nipple. No correlating sonographic mass or cystic lesion is identified. Normal fibroglandular tissue is present.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy. Recommend annual screening mammography.
2. No sonographic correlate for the palpable abnormality in the right breast at 10 o'clock. Likely represents normal fibroglandular tissue. Clinical follow-up recommended as appropriate.

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY REPORT
PROCEDURE: Bilateral screening mammogram.
COMPARISON: Prior mammogram from 11/4/2021.
TECHNIQUE: Standard CC and MLO views of each breast.
FINDINGS:
Right breast: The breast is composed of scattered areas of fibroglandular density. No dominant masses, areas of architectural distortion, or suspicious microcalcifications.
Left breast: The breast is composed of scattered areas of fibroglandular density. No dominant masses, areas of architectural distortion, or suspicious microcalcifications.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend routine annual screening mammography. 
BIRADS 1. ➡ *Negative*
- SCREENING MAMMOGRAM
CLINICAL INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: None available.
FINDINGS:
Both breasts show scattered fibroglandular density. No focal asymmetry, significant distortion or suspicious mass is identified. 
No suspicious clustered microcalcifications are seen.
There are bilateral benign-appearing lymph nodes in the axillary regions.
IMPRESSION: 
No mammographic evidence of malignancy. Routine yearly screening is recommended.
BIRADS Category 2: Benign Findings. 
ADDENDUM: Comparison to outside prior examinations from 2 years ago at ABC Hospital is recommended to establish stability and corroborate benign etiology of the lymph nodes described above. If prior examinations are unavailable, consider ultrasound correlation and short-term follow up in 6 months to document stability. ➡ *Not Stated*
- MAMMOGRAPHIC REPORT:
HISTORY: Follow up for 45 year old female with maternal history of breast cancer and left breast lump that is palpable by patient but not physician. Last mammogram 2 years ago was normal.

COMPARISON: Compared to previous mammography from 2 years ago.

FINDINGS:
Right breast: Architectural distortion at 9:00 position upper outer quadrant, 5 cm from the nipple. There are no associated calcifications or skin changes. 
Left breast: Scattered fibroglandular tissue. No abnormal masses, calcifications, or architectural distortion.

IMPRESSION: 
1. New architectural distortion in the right breast at 9:00, 5 cm from the nipple, highly suspicious for malignancy. Biopsy recommended. BIRADS 4C. 
2. Stable appearance of the left breast with no mammographic evidence of malignancy. However, given the new finding in the right breast, short interval follow up is recommended in 6 months. Clinical correlation recommended for the palpable lump. BIRADS 3 for the left breast.

ULTRASOUND ADDENDUM:
Targeted ultrasound performed to the left breast lump palpated by patient at 5:00 position. No sonographic correlate identified. 

FINAL ASSESSMENT:
1. Right breast: BIRADS 4C 
2. Left breast: BIRADS 3 ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAMS WITH TOMOSYNTHESIS

HISTORY: Annual screening mammogram.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.

There are no suspicious right mammographic findings. 

In the left breast at 12:00, 8 cm from the nipple, there is a group of pleomorphic calcifications spanning 8 mm. Magnification views confirm these calcifications are of concern. No associated mass, distortion or asymmetry is seen at this site.

Automated breast ultrasound (ABUS) of the left breast was performed to evaluate the area of the calcifications. No correlating mass or other ultrasound abnormality was seen at the site of the calcifications.

The remainder of the left breast is unremarkable.

IMPRESSION:
BI-RADS 4B: Suspicious pleomorphic calcifications at 12:00 in the left breast. Stereotactic biopsy is recommended. The patient will be contacted to schedule this procedure.

BI-RADS 1: No suspicious findings in the right breast.

ADDENDUM - STEREOTACTIC BIOPSY LEFT BREAST:
Under stereotactic guidance, the calcifications at 12:00 8 cm from the nipple in the left breast were sampled using an 8G needle. Specimen radiograph confirms that representative calcifications were obtained. Concordant pathology results showed ductal carcinoma in situ (DCIS), intermediate nuclear grade, solid and cribriform types. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- MAMMOGRAM REPORT:
TMIST FFM SCREENING MAMMOGRAM. 
Participant in the Tomosynthesis Mammographic Imaging Screening Trial, full field mammography arm, approved by the Research Ethics Board. Informed consent obtained.  
Comparison to prior exams: Yes.
Breast tissue density: Scattered fibroglandular densities.
Breast density < 75%: Yes.  
No significant interval change. Negative for any suspicious masses, architectural distortions or abnormal lymph nodes.  
Per the OBSP screening guidelines, routine recall recommended in 12 months.
BIRADS Category 1 - Negative. 

RADIOLOGY REPORT:
TMIST FFDM SCREENING MAMMOGRAM.  
This patient is enrolled in the REB-approved Tomosynthesis Mammographic Imaging Screening Trial and was randomized to the FFDM-only study arm. Informed consent has been obtained.
Comparison with previous mammograms: Yes. 
Breast composition: The breasts are heterogeneously dense, which may obscure small masses. 
Are the breasts extremely dense (>75% glandular)? No.
No mammographic evidence of malignancy identified. The breast parenchyma appears stable compared to prior studies without concerning masses, calcifications, architectural distortion or suspicious lymphadenopathy.  
Routine annual screening with the Ontario Breast Screening Program is advised.
BIRADS Category 2 - Benign findings. Recommend routine screening. ➡ *Not Stated*
- SCREENING MAMMOGRAM
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: Compared to prior study from 1 year ago.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable bilateral scattered fibroglandular densities.
IMPRESSION:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to outside screening mammogram from June 15, 2020.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, suspicious calcifications, or other findings.

IMPRESSION:
BI-RADS Category 1: Negative.

RECOMMENDATION:
Routine screening mammography in 1 year is recommended. ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAMS WITH MAGNIFICATION VIEWS.

History: 47 year old asymptomatic female for bilateral screening mammograms. 

Compared to outside prior mammogram from 2019-10-07 at SJHC. 

FINDINGS:
The breasts are heterogeneously dense. 
There are no dominant masses, architectural distortion or suspicious calcifications in the right breast.
At the 12 o'clock position of the left breast, approximately 3 cm from the nipple, there is a 1 cm group of pleomorphic microcalcifications. Magnification views confirm the presence of this cluster. No other suspicious findings are identified in the left breast.
Bilateral axillary regions are unremarkable. 
Metastatic calcifications are present in the skin overlying the left breast. 

SUMMARY:
BIRADS 4 left breast: New group of suspicious microcalcifications at 12 o'clock position of the left breast. Recommend stereotactic core biopsy. 
BIRADS 1 right breast: No mammographic evidence of malignancy.

ADDENDUM:
Ultrasound left breast 12:00 position 3 cm from nipple: Sonographically this corresponds to a vague area of hypoechogenicity with no definite mass. Biopsy is still recommended. ➡ *Positive*
- MAMMOGRAPHIC REPORT - BILATERAL
INDICATION: Annual screening
COMPARISON: Prior study from 5/13/2021

FINDINGS:
Both breasts are heterogeneously dense (C), which could obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

LEFT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend continued annual screening in 12 months with tomosynthesis.
2. Heterogeneously dense breast tissue (C) in both breasts, which lowers the sensitivity of mammography.

ASSESSMENT CATEGORIES:
Right Breast: BI-RADS 1 - Negative
Left Breast: BI-RADS 1 - Negative

PATIENT NOTIFICATION:
The results of your recent breast imaging exam are normal/benign (not suggestive of cancer). You should continue with your routine screening regimen. If you feel a lump or have any other breast problems or concerns before your next routine exam, contact your physician or this facility promptly for further evaluation. ➡ *Negative*
- FINDINGS:
Right breast: Heterogeneously dense breast tissue. There are no dominant masses, architectural distortion, or suspicious calcifications.

Left breast: Heterogeneously dense breast tissue. There are no dominant masses, architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications in the upper outer quadrant, likely vascular.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Benign left breast calcifications, likely vascular in etiology.
2. Recommend continued annual screening mammography in 12 months.

BIRADS 2: Benign findings. Recommend routine screening. ➡ *Negative*
- BILATERAL MAMMOGRAMS AND ULTRASOUND OF THE LEFT BREAST.
CLINICAL INDICATION: Abnormal screening mammogram with grouped calcifications in the upper outer quadrant of the left breast. No palpable abnormality. High risk patient with strong family history of breast cancer.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained, complemented by spot compression and magnification views in the left breast. Targeted ultrasound of the left breast was also performed.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
In the right breast, there are no concerning mammographic or sonographic findings. No dominant mass, architectural distortion or suspicious calcifications. 
In the left breast, there is a cluster of pleomorphic calcifications in the upper outer quadrant (10:00 position) 6 cm from the nipple on the CC and MLO views. This cluster measures approximately 8 mm in greatest dimension. Additional magnification views confirm the presence of pleomorphic calcifications in a segmental distribution, raising concern for ductal carcinoma in situ (DCIS). 
Targeted ultrasound of the left breast at the site of the calcifications shows mildly hypoechoic and heterogeneous breast tissue, but no definitive correlating mass. The segmental distribution of pleomorphic calcifications remains highly suspicious for DCIS.
No other dominant masses, areas of architectural distortion or other suspicious calcifications are seen in either breast. No axillary, intramammary or supraclavicular adenopathy.
IMPRESSION:
BI-RADS 4C: Highly suspicious cluster of pleomorphic calcifications in a segmental distribution in the left breast at 10:00, 6 cm from the nipple, demonstrating no definitive sonographic correlate. Stereotactic biopsy is recommended for diagnosis. The differential includes ductal carcinoma in situ (DCIS) as the leading consideration. 
BI-RADS 2: Benign findings in the right breast. Recommend routine screening in 1 year. ➡ *Positive*
- FINDINGS:

Heterogeneously dense breasts.

Comparison is made to a study performed 2 years ago at an outside facility.

No suspicious findings are seen in the right breast. BI-RADS 1.

Left breast:

There is an irregular spiculated mass at the 3:00 position, 4cm from the nipple. The mass measures approximately 2.5cm. Associated architectural distortion is noted.

Additional scattered groups of amorphous and pleomorphic microcalcifications are seen in the left breast, most prominently in the upper outer quadrant. The largest cluster measures 1.2cm.

There are additional smaller masses suspicious for multifocal disease.

In the left axilla, enlarged abnormal lymph nodes are seen, with cortical thickness up to 1cm and loss of the normal fatty hilum. Findings are concerning for metastatic adenopathy.

The left nipple is retracted.

The remainder of the left axilla is unremarkable.

IMPRESSION:

Highly suspicious mass left breast at 3:00, 4cm from the nipple.

Numerous clusters of suspicious microcalcifications in the left breast, highest suspicion upper outer quadrant.

Multifocal disease and nipple retraction are present.

Abnormal left axillary lymph nodes concerning for metastatic disease.

BI-RADS 5: Highly suggestive of malignancy. ➡ *Positive*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.  

TECHNIQUE: Full field digital mammography, bilateral CC and MLO views.

COMPARISON: Previous mammograms from 1 year ago.  

FINDINGS:
There is scattered fibroglandular density in both breasts, with the breasts being heterogeneously dense. This may lower the sensitivity of mammography.

No dominant mass, architectural distortion or suspicious calcifications are identified in either breast. The skin and nipple are unremarkable bilaterally. 

IMPRESSION: 
BI-RADS Category 2: Benign findings.    
No evidence of malignancy.
Recommend continued annual screening mammography in 12 months.  

ADDENDUM: 
Breast density is heterogeneously dense (C). Patient notified via lay summary letter. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 50%: Yes.
OBSP to recall client in 1 year: Yes.
The breast tissue is predominantly fatty. No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2.

ADDENDUM: 
3D Tomosynthesis images were reviewed in addition to the 2D full field digital mammography images. No additional abnormalities were identified on the 3D tomosynthesis images. ➡ *Not Stated*
- MAMMOGRAPHY REPORT

CLINICAL HISTORY: 42 year old female with family history of breast cancer. No palpable lumps.

COMPARISON: Screening mammogram from 1 year prior.

FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications are identified. 

LEFT BREAST:
The breast is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications are identified. 

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend routine screening in 1 year.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS:
INDICATION: Annual screening.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or definite asymmetries.
Scattered fibroglandular densities are present bilaterally.
There is a 6 mm cluster of punctate microcalcifications in the upper outer quadrant of the right breast at the 10:00 position, 8 cm from the nipple. No prior mammograms are available for comparison.
The left breast is unremarkable.
IMPRESSION:
BI-RADS Category 0: Incomplete. The cluster of microcalcifications in the right breast requires additional imaging evaluation with magnification views for further characterization.
RECOMMENDATION:  
Right breast magnification views.
ADDENDUM - RIGHT BREAST MAGNIFICATION VIEWS:
The cluster of microcalcifications in the right breast upper outer quadrant demonstrates pleomorphic calcifications in a segmental distribution, spanning approximately 8 mm. This is a suspicious finding.
FINAL ASSESSMENT:  
BI-RADS Category 4B: Moderate suspicion for malignancy. Biopsy should be considered. ➡ *Positive*
- MAMMOGRAM REPORT - LEFT BREAST.

INDICATION: Annual screening mammogram. No current breast complaints.

TECHNIQUE: Standard CC and MLO views of the left breast were obtained. 

COMPARISON: Comparison is made to prior exam dated 4/15/2022. 

FINDINGS:
The left breast is heterogeneously dense, which may obscure small masses. 

There are no masses, architectural distortion, or suspicious calcifications. The breast tissue is stable compared to prior exam.

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. 
2. Recommend continued annual screening mammography in 12 months. 

ASSESSMENT: BI-RADS 1 - Negative. 

ADDENDUM: Breast density notification - The breasts are heterogeneously dense, which may obscure small masses. While we do not feel any other studies are indicated at this time based on your mammogram, this information about your breast density is being provided to inform you and your physician that you may benefit from supplemental screening tests, depending on your individual risk factors. A report of your results was sent to your physician. Please contact your physician if you have any questions or concerns about this report. ➡ *Negative*
- SCREENING DIGITAL MAMMOGRAM LEFT AND RIGHT.
History: Screening mammogram. No palpable lumps.
FINDINGS:
The breasts are of mildly heterogeneous density. No suspicious masses or areas of architectural distortion are seen in either breast. No significant asymmetries are present. 
There is a new group of pleomorphic microcalcifications in the upper outer left breast at 2 o'clock, 7 cm from the nipple. These calcifications are of concern.
Compared to the previous studies of 2 years ago, the breasts are stable other than the new group of microcalcifications in the left breast.
IMPRESSIONS:
BIRADS 4: Suspicious new group of pleomorphic microcalcifications in the upper outer left breast. Stereotactic biopsy is recommended for further assessment.
BIRADS 2: Benign and stable findings right breast.
ADDENDUM: Patient returned for stereotactic biopsy of the group of microcalcifications in the left breast using an 8 gauge vacuum assisted device. Specimen radiograph confirmed retrieval of the targeted calcifications. Pathology results: Ductal carcinoma in situ, intermediate nuclear grade with necrosis.
FINAL ASSESSMENT:
Left Breast: BI-RADS 4 (Biopsy proven malignancy)
Right Breast: BI-RADS 2 (Benign) ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAPHY
INDICATION: Annual screening mammogram in a patient with no breast complaints.
TECHNIQUE: Bilateral CC and MLO views were obtained using a Hologic Dimensions system. Additional CC and lateromedial views were added for the left breast.
COMPARISON: Prior bilateral mammograms from May 2021.
FINDINGS:
Both breasts show heterogeneously dense fibroglandular tissue which may limit sensitivity of mammography.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in the right breast. Stable benign-appearing calcifications are noted.
In the LEFT breast, there is a new cluster of fine pleomorphic microcalcifications spanning approximately 8 mm at the 10:00 position, middle depth. No associated mass or distortion is evident. The remainder of the left breast is unremarkable. 
Bilateral benign-appearing axillary lymph nodes are present.
IMPRESSION:
BI-RADS Category 0: Incomplete. The new cluster of microcalcifications in the left breast requires further evaluation with spot compression magnification views. Recommend recall for diagnostic mammography workup.
BIRADS Category 2 for the right breast: Benign finding.
ADDENDUM:
The patient returned for diagnostic workup of the left breast. Spot compression magnification views confirm a cluster of suspicious pleomorphic microcalcifications at 10:00, middle depth in the left breast, unchanged from the screening exam. Ultrasound of this area shows no correlate mass or dilated duct. 
FINAL ASSESSMENT: 
BI-RADS Category 4: Suspicious abnormality. Stereotactic biopsy is recommended for the cluster of microcalcifications in the left breast.
BI-RADS Category 2: Benign for the right breast. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammogram from 10/15/2020.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

Stable bilateral circumscribed masses, likely benign. Largest measures 8 mm in the right breast at 10:00, 6 cm from the nipple.

At the 3:00 position of the left breast, 6 cm from the nipple, there is a new group of amorphous calcifications spanning an area of approximately 6 mm. Additional magnification views in the craniocaudal and mediolateral projections confirm these calcifications.

Stable bilateral axillary lymph nodes, with no abnormal morphology.

IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation with magnification views is needed to better characterize the new group of amorphous calcifications in the left breast at 3:00, 6 cm from the nipple. These calcifications are suspicious and will require biopsy if confirmed on additional imaging.

ADDENDUM (Post Magnification Views):
Magnification views of the left breast at the 3:00 position, 6 cm from the nipple, demonstrate a cluster of amorphous calcifications spanning 7 mm. These calcifications are new and suspicious for malignancy. Stereotactic biopsy is recommended.

FINAL ASSESSMENT: BI-RADS Category 4: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- MAMMOGRAPHY REPORT

CLINICAL HISTORY: Patient presents for annual screening mammogram. No current breast complaints. 

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to prior mammogram dated 1/15/2019.

FINDINGS:

RIGHT BREAST:
The breast is heterogeneously dense (BI-RADS density C). There are no dominant masses, architectural distortion, or suspicious calcifications.

LEFT BREAST: 
The breast is heterogeneously dense (BI-RADS density C). There are no dominant masses or architectural distortion. Clustered amorphous microcalcifications are noted in the upper outer quadrant, spanning an area of approximately 1 cm. These were not present on the prior study. No other suspicious calcifications are identified.

IMPRESSION:
1. Heterogeneously dense breasts.
2. New cluster of amorphous microcalcifications in the left upper outer quadrant, suspicious for malignancy. Additional magnification views recommended for further evaluation. BI-RADS 4B. 
3. Stable, benign findings in the right breast. BI-RADS 2.

RECOMMENDATIONS:
1. Diagnostic mammogram with magnification views of the left breast for further evaluation of the new microcalcifications. 
2. Consider stereotactic biopsy if the calcifications remain suspicious on additional imaging.
3. Routine annual screening of the right breast in 12 months. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL INDICATION: Annual screening mammogram.
TECHNIQUE: Bilateral digital screening mammogram with CAD was performed. 
COMPARISON: Screening mammogram from 1 year ago.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in either breast. 
There is a new 5 mm cluster of fine pleomorphic calcifications in the upper outer right breast at the 10:00 position 6 cm from the nipple. This was not present on the prior mammogram. 
Stable bilateral benign-appearing lymph nodes are noted.
IMPRESSION:
BI-RADS 0: Incomplete: Need additional imaging evaluation. 
New cluster of suspicious microcalcifications in the right breast requires magnification views for further characterization. Recommend diagnostic mammogram with magnification views right breast.

ADDENDUM:
RIGHT DIAGNOSTIC MAMMOGRAM, MAGNIFICATION VIEWS
FINDINGS: Magnification views of the right breast at the 10:00 position 6 cm from the nipple demonstrate a 5 mm cluster of fine linear and branching pleomorphic calcifications. There are approximately 20 calcifications in the cluster. No associated mass, distortion or skin thickening.  
IMPRESSION:
BI-RADS 4B: Moderate suspicion for malignancy. 5 mm cluster of fine pleomorphic calcifications right breast, suspicious for DCIS. Recommend stereotactic biopsy. ➡ *Positive*
- FINDINGS:
Both breasts are predominantly fatty replaced. Scattered areas of fibroglandular density are noted bilaterally. 
No dominant mass, architectural distortion or suspicious calcifications are identified in either breast.
Bilateral benign calcifications are noted, likely representing vascular calcifications.
No significant interval change compared to prior exam.

IMPRESSION: 
1. No mammographic evidence of malignancy. 
2. Predominantly fatty replaced breasts.
3. Bilateral vascular calcifications, likely benign. No suspicious microcalcifications.
4. Stable mammographic appearance compared to prior exam.

BI-RADS Category 2: Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAM, BILATERAL DIAGNOSTIC WITH TOMOSYNTHESIS

CLINICAL INDICATION: 52 year old female. Left UOQ palpable mass. 

COMPARISON: None.

FINDINGS:
Right breast: Scattered fibroglandular densities. No dominant mass, distortion or suspicious microcalcifications. 

Left breast: Scattered fibroglandular densities. 
Left upper outer quadrant demonstrates a 1.5 cm irregular mass with indistinct margins at approximately 1 o'clock, 4 cm from the nipple. 
There is an adjacent 7 mm group of fine pleomorphic microcalcifications. 
No skin thickening or retraction. No nipple retraction.

IMPRESSION:
BIRADS 4C: Suspicious left UOQ mass with adjacent grouped microcalcifications. Recommend stereotactic biopsy of the microcalcifications and ultrasound guided core biopsy of the mass.
BIRADS 2: Benign right breast. Recommend routine screening in 1 year.

ADDENDUM: The patient proceeded to left breast ultrasound which confirmed an irregular solid mass at 1 o'clock in the left breast corresponding to the mammographic finding, measuring 1.6 x 1.1 x 1.0 cm, oriented along the AP axis. The grouped microcalcifications are not identified sonographically. Ultrasound-guided core biopsy of the mass was performed, followed by stereotactic biopsy of the microcalcifications. Pathology is pending. ➡ *Positive*
- MAMMOGRAM REPORT
CLINICAL HISTORY: 52-year-old female presenting for routine screening mammogram. No current breast complaints. 
COMPARISON: Comparison made to prior mammogram from 04/15/2022. 
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications. 
LEFT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography in 12 months.
BIRADS 1- Negative. ➡ *Negative*
- FINDINGS: Left breast tissue contains scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications. Stable appearance compared to prior mammograms.

Targeted ultrasound of the left breast at 7 o'clock position demonstrates normal fibroglandular tissue with no discrete solid or cystic lesion.

IMPRESSION:

1. No mammographic evidence of malignancy in the left breast. Recommend continued annual screening mammography in 12 months.

2. No sonographic correlate for the palpable abnormality in the left breast 7 o'clock position. Recommend clinical follow up and correlation with physical exam findings as indicated.

BI-RADS Category 1: Negative. ➡ *Negative*
- TMIST MAMMOGRAM SCREENING - FFDM ARM 
Informed consent was obtained for this client's participation in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), approved by the Research Ethics Board. The client was randomized to the full field digital mammography arm.
Comparison to prior studies: Yes. 
Mammographic breast density: Scattered fibroglandular density (<75%).
One year routine screening mammogram recommended per OBSP protocol.
No mammographic evidence of malignancy identified. 
Final BI-RADS Assessment Category 1: Negative. ➡ *Not Stated*
- SCREENING BILATERAL MAMMOGRAM
INDICATION: Annual screening mammogram.  Past history of biopsy proven fibroadenomas both breasts. No history of breast cancer.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense. No significant asymmetry or dominant mass.
There is a cluster of faint pleomorphic microcalcifications identified in the upper inner right breast at approximately 10:00, 4 cm from the nipple, spanning an area of approximately 8 mm. These do not have a benign morphology and are new since patient's last reported exam.
At 12:00, 7 cm from the nipple in the right breast there is a 12 mm circumscribed oval mass, consistent with one of the patient's known fibroadenomas.
The left breast shows multiple circumscribed masses between 1-2 cm in diameter, consistent with patient's known fibroadenomas. There are several vascular calcifications in the left breast consistent with fibroadenoma calcifications.
No suspicious masses, calcifications, or architectural distortion in the left breast.
IMPRESSION:
BI-RADS 0: Incomplete. New cluster of pleomorphic microcalcifications in the right breast at 10:00, 4 cm from the nipple. Additional magnification imaging recommended for further characterization. Depending on magnification findings, stereotactic biopsy may be recommended.
ADDENDUM:
Magnification views of the right breast were performed. The cluster of microcalcifications at 10:00 span 9 mm and are pleomorphic in morphology. Stereotactic biopsy is recommended. BI-RADS 4B: Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- FINDINGS:
Right breast: Mastectomy changes are stable. No suspicious findings.
Left breast: The breast tissue is predominantly fatty replaced. There are no suspicious masses, architectural distortion, or suspicious calcifications. Stable benign appearing left axillary lymph nodes.
IMPRESSION:
1. Stable post-mastectomy changes of the right breast. 
2. No evidence of malignancy in the left breast. Recommend annual screening mammography. 
3. Stable benign-appearing left axillary lymph nodes, likely reactive.
BIRADS 2: Benign findings. Recommend routine screening.
ADDENDUM: Comparison was made to prior bilateral mammograms from 12/3/2023, which showed similar findings with no interval change. ➡ *Negative*
- MAMMOGRAPHY, LEFT BREAST.
HISTORY: This is a 52-year-old female who presents for routine mammographic screening.
FINDINGS: There are scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious microcalcifications. The skin appears within normal limits. The visualized left axilla is unremarkable.
IMPRESSION: No evidence of malignancy in the left breast. Annual follow up is recommended.
BIRADS 1. ➡ *Negative*
- MAMMOGRAM, LEFT BREAST.
CLINICAL INDICATION: 34 year old female with large breast presenting for routine screening.
FINDINGS: There are scattered areas of benign appearing fibroglandular density. No suspicious masses, calcifications, architectural distortion or other mammographic evidence of malignancy is seen. No significant change from prior examinations.
IMPRESSION: Left breast negative mammogram. No radiographic evidence of malignancy.
BIRADS 1. Recommend annual screening mammography in 1 year. ➡ *Negative*
- IMAGING FINDINGS:
Bilateral mammographic views demonstrate heterogeneously dense parenchyma. No dominant masses, areas of architectural distortion or suspicious calcifications. Stable benign-appearing calcifications noted bilaterally. Breast tissue contains scattered fibroglandular densities.

Focused left breast ultrasound was performed at the site of palpable abnormality, 5 o'clock position 3 cm from the nipple. Targeted ultrasound demonstrates an ovoid circumscribed hypoechoic solid mass, measuring 6 x 4 x 5 mm. This likely represents a fibroadenoma. No sonographic evidence of malignancy.

IMPRESSION:
1. No mammographic evidence of malignancy. Stable benign calcifications noted bilaterally. Recommend continued annual screening mammography in 12 months. 
2. Solitary solid circumscribed mass at 5 o'clock position of the left breast, likely representing a fibroadenoma. Clinical and imaging follow-up recommended in 6 months to document stability. Biopsy may be performed if clinically warranted.

BI-RADS Category 2: Benign Findings. Recommend routine screening. ➡ *Negative*
- ULTRASOUND LEFT BREAST
INDICATION: Palpable abnormality left breast at 6 o'clock position.
TECHNIQUE: Real time ultrasound scanning of the left breast at 6 o'clock position using high frequency linear transducer. 
COMPARISON: Mammogram from 2/14/2022.
FINDINGS: 
The left breast demonstrates scattered fibroglandular tissue. 
Targeted ultrasound over the area of palpable concern at 6 o'clock position in the left breast demonstrates normal appearing glandular tissue. No discrete mass, cystic lesion, or dilated ducts are identified. No suspicious calcifications are seen.
IMPRESSION:
No sonographic evidence of malignancy or suspicious abnormality at the site of palpable finding in the left breast at 6 o'clock position. Recommend clinical follow up and routine screening mammography. 
BIRADS 1 - Negative. ➡ *Negative*
- ULTRASOUND OF THE RIGHT BREAST AND AXILLA. 6-MONTH FOLLOW UP OF PRIOR SCREENING EXAM.
Compared to screening mammogram and targeted right breast ultrasound from 6 months prior.
FINDINGS: 
At 10:30 in the right breast 4 cm from the nipple corresponding to the area of palpable concern, ultrasound demonstrates a stable 6 x 5 x 4 mm irregular solid mass with angular margins. This lesion demonstrates internal vascularity. 
At 11:00 in the right breast 5 cm from the nipple ultrasound again demonstrates scattered punctate echogenic calcifications within a 6 mm area of parenchyma as identified previously. No associated mass or shadowing is seen.
The remainder of the right breast was scanned. No additional suspicious masses, areas of architectural distortion or suspicious calcifications are seen. 
Right axilla scanned. No abnormal lymph nodes identified. 
IMPRESSION:
1. Stable appearance of irregular solid mass right breast 10:30 location 4 cm from the nipple. Recommend tissue sampling for definitive characterization. Previously demonstrated invasive mammary carcinoma.
2. Again noted are stable punctate echogenic calcifications in a group spanning 6 mm at 11:00 5 cm from right nipple. These are indeterminate and require 6 month follow up vs. stereotactic biopsy depending upon patient treatment wishes in consultation with referring surgeon. BI-RADS 4B. ➡ *Positive*
- SCREENING MAMMOGRAM.
INDICATION: Screening.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, suspicious calcifications, or other findings.
IMPRESSION:
Negative screening mammogram.
BIRADS Category 1: Negative.
Recommend routine screening in 1 year. ➡ *Not Stated*
- REPORT: "SCREENING BILATERAL MAMMOGRAM.  
INDICATION: Routine screening.
TECHNIQUE: Bilateral CC and MLO views.
COMPARISON: None available.
FINDINGS:  
Breast composition: The breasts are heterogeneously dense, which may obscure small masses.
No suspicious masses, architectural distortion, or suspicious calcifications.
No significant change compared with prior examination.
IMPRESSION: 
Negative screening mammogram. 
BIRADS Category 1: Negative. Routine screening recommended in 1 year. 
" ➡ *Not Stated*
- FINDINGS: The left breast tissue is heterogeneously dense, which may obscure detection of small masses. There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications are scattered throughout both breasts. There are no new mammographic findings compared to prior exams.

Targeted left breast ultrasound was performed at the site of the palpable abnormality at 4 o'clock, 5 cm from the nipple. No sonographic correlate for the palpable finding was identified. The area consists of normal fibroglandular tissue.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend annual screening mammography.
2. No sonographic correlate for the palpable finding in the left breast at 4 o'clock. Likely represents normal fibroglandular tissue. Clinical follow-up recommended as appropriate.

BI-RADS Category 1: Negative.

ADDENDUM: Following discussion with the referring clinician, a short-term follow-up diagnostic mammogram is recommended in 6 months to document stability of the palpable finding. The patient is also instructed to return promptly for any new palpable abnormalities. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS
INDICATION: Screening.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense (C). This may lower the sensitivity of mammography.
There are no suspicious masses, calcifications, asymmetries or architectural distortions in either breast. No significant changes compared to prior exam.
At the 9:00 position of the right breast, 4 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of 6 mm.  Additional magnification views confirm these calcifications. 
No mammographic evidence of malignancy in the left breast.
IMPRESSION:
BIRADS 0: Group of amorphous microcalcifications right breast. Additional imaging evaluation with magnification views performed, now BIRADS 4. Stereotactic biopsy recommended. 
BIRADS 1, negative left breast. ➡ *Positive*
- SCREENING MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications.

Stable benign-appearing calcifications are noted in both breasts.

IMPRESSION:

BIRADS 2- Benign findings. Recommend routine screening in 1 year.

ADDENDUM:
Comparison to prior study shows no significant interval change.
The previously noted benign calcifications in both breasts are stable. ➡ *Not Stated*
- FINDINGS: The breast tissue is predominantly fatty, with scattered areas of fibroglandular density. No dominant masses, architectural distortions, or suspicious calcifications are identified. The skin and nipple are unremarkable. No significant axillary lymphadenopathy is seen.

IMPRESSION: Negative bilateral screening mammogram. No mammographic evidence of malignancy. Scattered fibroglandular densities, as described above. 

BI-RADS Category 1: Negative. Routine screening mammogram is recommended in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

FINDINGS:

Both breasts are almost entirely fatty.

LEFT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.

RIGHT BREAST: 
A group of amorphous calcifications is identified in the upper outer quadrant posterior depth, about 8 cm from the nipple. The distribution is clustered and spans an area of about 1 cm. No associated mass, distortion or skin changes are seen.

IMPRESSION:

LEFT BREAST: Negative. BI-RADS 1. 

RIGHT BREAST: Group of amorphous calcifications in the right upper outer quadrant, suspicious for malignancy. BI-RADS 4.

RECOMMENDATIONS:
Stereotactic biopsy is recommended for the group of calcifications in the right upper outer breast. Please send a requisition to the department to arrange this. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications in either breast.

In the left breast at the 10:00 position 5 cm from the nipple there is a new 8 mm cluster of fine pleomorphic calcifications. These were not present on the prior mammogram.

Stable benign-appearing calcifications are present in both breasts.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for the new cluster of pleomorphic calcifications at 10:00 in the left breast.

BI-RADS Category 2: Benign findings in the right breast. Recommend continued annual screening in 1 year.

ADDENDUM: Patient proceeded to stereotactic biopsy of the left breast calcifications. Pathology showed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. Wide local excision is planned. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH TOMOSYNTHESIS
INDICATION: Annual screening mammogram.
COMPARISON: Prior mammogram from 10/15/2020.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in either breast.
At the 9:00 position of the right breast, there is a new 6 mm cluster of amorphous calcifications, located 6 cm from the nipple. This is new compared to prior mammograms and is suspicious for malignancy.
Scattered fibroglandular densities are present in both breasts.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic core biopsy is recommended for the new cluster of amorphous calcifications at 9:00, 6 cm from the nipple in the right breast.
BI-RADS Category 2 for the left breast: Benign findings. Recommend continued annual screening in 12 months.

ADDENDUM: Patient was notified of the results and agreed to proceed with stereotactic core biopsy of the right breast calcifications. Biopsy has been scheduled for 4/20/2023. ➡ *Positive*
- CLINICAL INDICATION: Annual screening mammogram.

COMPARISON: Screening mammogram from 06/12/2022 at St. Josephs Medical Center.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
Right Breast:
The right breast is almost entirely fatty. There are no suspicious masses, architectural distortions, or calcifications. The skin is normal in thickness. No lymphadenopathy is seen.

Left Breast:
The left breast is heterogeneously dense, which may obscure detection of small masses. There are no suspicious masses, architectural distortions, or calcifications. The skin is normal in thickness. No lymphadenopathy is seen.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. No suspicious microcalcifications are identified.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.

RECOMMENDATIONS:
1. Continue annual screening mammography.
2. Consider supplemental screening with breast ultrasound or MRI due to dense breast tissue.

ASSESSMENT CATEGORY: BI-RADS 1 - Negative ➡ *Negative*
- DIAGNOSTIC FFDM MAMMOGRAM.
CLINICAL INDICATION: Bilateral breast pain for past several weeks.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
There are no discrete masses, architectural distortions, asymmetries, or suspicious calcifications.
There are no findings to suggest malignancy. The breast pain is likely due to hormonal changes or musculoskeletal in nature.
BIRADS-2: Benign Finding. Recommend routine screening in 1 year.

ADDENDUM: Ultrasound was performed, demonstrating no suspicious findings. Specifically, no microcalcifications were identified. ➡ *Not Stated*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
HISTORY: Screening mammogram. No prior mammograms available for comparison. No reported breast complaints. 
FINDINGS:
Mammogram: The breasts are composed of scattered areas of fibroglandular density. There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Ultrasound: Targeted ultrasound was performed in the upper outer quadrant of the left breast corresponding to the patient's area of concern. No sonographic abnormality was identified. 
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. No sonographic correlate for palpable abnormality in the left breast.
MANAGEMENT: Routine screening mammography is recommended in 1 year. Clinical follow up for left breast palpable abnormality as needed.
BIRADS 1 - Negative. ➡ *Negative*
- COMPARISON:  Digital screening mammogram from 2 years ago.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

In the left breast, there are no dominant masses, areas of architectural distortion or suspicious calcifications.  

In the right breast, there are no dominant masses, areas of architectural distortion or suspicious calcifications.  

IMPRESSION:

1. No mammographic evidence of malignancy in either breast.  
2. Heterogeneously dense breast tissue which can lower the sensitivity of mammography.

RECOMMENDATIONS:

Annual screening mammography is recommended.   

ASSESSMENT CATEGORY:

BI-RADS Category 1: Negative.

ADDENDUM:  Spoke to ordering provider Dr. Smith who confirms no palpable lumps on clinical breast examination. 

Per ACR BI-RADS Atlas, BI-RADS Category 1 assessment is given when there are no mammographic findings (benign or suspicious) that require further evaluation, even if palpable finding is present on clinical exam.  Thus, final BIRADS assessment remains BIRADS 1. ➡ *Negative*
- TECHNIQUE: Bilateral digital mammography with tomosynthesis. COMPARISON: Compared to outside mammogram from 10/15/2022.

CLINICAL HISTORY: 54-year-old female presents for routine screening mammogram.

FINDINGS:
RIGHT BREAST:
The breast is almost entirely fat. There are no masses, architectural distortions, or suspicious calcifications. 

LEFT BREAST: 
The breast is almost entirely fat. There are no masses, architectural distortions, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Fatty replaced breasts bilaterally. 
3. No suspicious masses, calcifications, or architectural distortion in either breast.

BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- SCREENING MAMMOGRAM.
Compared to Previous: First study at this facility.  No prior exams available for comparison.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
Bilateral breast tissue is heterogeneously dense.
No significant masses, architectural distortion or suspicious calcifications.  
BIRADS-2: Benign findings. Routine screening recommended in 1 year. ➡ *Not Stated*
- MAMMOGRAPHIC EXAMINATION - BILATERAL.
CLINICAL HISTORY: Family history of breast cancer. Annual screening.
COMPARISON: Multiple prior mammograms available, most recently from 1 year ago.
FINDINGS: The breast tissue is heterogeneously dense, which could obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious clustered microcalcifications. 
Stable scattered fibroglandular densities are noted bilaterally.
There are no significant changes compared to prior studies.
IMPRESSION:
1. No mammographic evidence of malignancy. Heterogeneously dense breast tissue limits sensitivity. 
2. Recommend continued annual screening mammography and clinical breast exams. Consideration may be given to supplemental screening with breast MRI or ultrasound.
BIRADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- MAMMOGRAM REPORT:
CLINICAL HISTORY: Patient reports a palpable lump in the right breast at the 10 o'clock position. Last mammogram was 2 years ago and was normal. Patient has no personal or family history of breast cancer.
FINDINGS: 
Both breasts show scattered fibroglandular density. 
Right breast: There is a 1.2 cm oval circumscribed mass at the 10 o'clock position, 3 cm from the nipple, corresponding to the palpable abnormality. No suspicious calcifications, architectural distortion or skin thickening.
Left breast: No dominant mass, suspicious calcifications, or architectural distortion. 
IMPRESSION:
Right breast: 1.2 cm circumscribed mass at 10 o'clock, 3 cm from the nipple, corresponding to the palpable finding. This is most likely a benign finding such as a fibroadenoma. However, given the new palpable lump, a targeted ultrasound is recommended for further evaluation. 
Left breast: Negative.
BI-RADS Category 0: Additional imaging evaluation needed. Right breast ultrasound is recommended for the palpable lump at 10 o'clock.

ADDENDUM - RIGHT BREAST ULTRASOUND:
Targeted ultrasound of the right breast 10 o'clock lump shows a 1.2 x 1.0 x 0.8 cm oval, circumscribed, parallel, hypoechoic mass. This corresponds to the mammographic finding and is consistent with a benign fibroadenoma. 
IMPRESSION:
Benign right breast mass most consistent with a fibroadenoma. Recommend routine screening mammography in 1 year.
BI-RADS Category 2: Benign finding. ➡ *Negative*
- TMIST FFDM ARM OBSP MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.  
OBSP to recall client in 1 year: Yes.
Stable scattered areas of fibroglandular density bilaterally. No dominant masses, suspicious calcifications or architectural distortion.
BIRADS-2. Annual screening recommended.

ADDENDUM: 
Comparison was made to prior studies from June 2019 and July 2020, which demonstrate overall stable mammographic findings. No mammographic evidence of malignancy. ➡ *Not Stated*
- LEFT DIAGNOSTIC MAMMOGRAM
INDICATION: Left breast pain.
COMPARISON: Comparison is made to outside mammogram from 2/2022.
FINDINGS:
LEFT BREAST:
The breast is heterogeneously dense. No dominant mass, suspicious calcifications, or concerning asymmetry. There are no significant changes compared to prior mammogram. Architectural distortion is noted at the 12 o'clock position posteriorly, corresponding to a known surgical scar. No other architectural distortion is identified. There are no suspicious calcifications.
IMPRESSION:
1. There is architectural distortion in the left breast at 12 o'clock corresponding to biopsy-proven surgical scar. No other suspicious findings.
2. No suspicious calcifications identified.
ASSESSMENT: ACR BI-RADS category 2: Benign finding. Recommend routine screening in 1 year.

ADDENDUM:
Spoke with referring provider Dr. Smith who relayed that patient reports a new area of left breast pain. Recommend targeted ultrasound to area of pain for further evaluation despite benign-appearing mammogram. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
FINDINGS:
There are scattered areas of fibroglandular density.  
In the right breast there are no dominant masses, areas of architectural distortion or suspicious calcifications.  No significant interval change compared to prior mammograms.
In the left breast, there is a new cluster of heterogeneous calcifications in the upper outer quadrant posterior depth. The calcifications are pleomorphic and have a segmental distribution, measuring approximately 1 cm in length. No associated mass or architectural distortion. No other dominant masses, areas of architectural distortion or suspicious calcifications in the left breast.
IMPRESSION:
New cluster of suspicious calcifications in the upper outer left breast, BI-RADS 4B. This requires tissue diagnosis. Stereotactic core biopsy is recommended.
Otherwise stable bilateral mammograms, BI-RADS 2 right breast. Recommend continued annual screening mammography.
ADDENDUM: 
The patient proceeded with ultrasound guided core needle biopsy of the cluster of calcifications in the upper outer left breast. Pathology results are pending and will be communicated in an addendum to this report. ➡ *Positive*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.  
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.   
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.
The breast tissue appears normal, with no dominant masses, suspicious calcifications or architectural distortion identified. 
No mammographic evidence of malignancy is seen. Routine screening is recommended.
BIRADS-1/2. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram
COMPARISON: None available
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.
At the 12 o'clock position of the right breast, approximately 6 cm from the nipple, there is a cluster of amorphous microcalcifications spanning an area of about 8 mm. This cluster is new since the patient's last mammogram.
Calcifications in the left breast are benign in morphology, scattered throughout the breast parenchyma bilaterally.
No concerning axillary lymphadenopathy.
Skin is normal in thickness. No evidence of significant trabecular thickening or trabecular distortion.
IMPRESSION:
New cluster of amorphous microcalcifications in the right breast at 12 o'clock, 6 cm from the nipple, spanning 8 mm. Additional magnification views are recommended for further characterization. BI-RADS 0: Incomplete. Additional imaging evaluation needed.
The remainder of the study is benign. BI-RADS 2 for the left breast.
ADDENDUM:
Magnification views of the right breast demonstrate the previously described cluster of amorphous microcalcifications at 12 o'clock. No additional suspicious calcifications are identified. Stereotactic biopsy is recommended for definitive characterization.
FINAL ASSESSMENT: BI-RADS 4: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- "SCREENING MAMMOGRAM.
INDICATION: Routine annual screening mammogram.
TECHNIQUE: Standard bilateral digital mammogram including CC and MLO views.
COMPARISON: 10/12/2021.
FINDINGS:
Right Breast: The breast is predominantly fatty replaced. No dominant mass, architectural distortion, or suspicious calcifications are identified.
Left Breast: The breast is predominantly fatty replaced. No dominant mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
1. Negative bilateral screening mammogram. No mammographic evidence of malignancy.
2. Predominantly fatty replaced breasts. 
BIRADS: 2 - Benign findings. Recommend routine screening in 1 year.

IMPRESSION ADDENDUM: No suspicious microcalcifications are identified in either breast." ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

Grouped amorphous calcifications are noted in the upper outer quadrant of the left breast, approximately 9 cm from the nipple on the MLO view. These are new since the prior mammogram and are suspicious for early malignancy.

Scattered vascular calcifications are present bilaterally, likely benign.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for further evaluation of the new grouped amorphous calcifications in the upper outer quadrant of the left breast.

MANAGEMENT:
The patient was informed of the results and the need for stereotactic biopsy. Appointment scheduled for 10/15/2022. ➡ *Positive*
- HISTORY: Patient presents for routine screening mammogram. No current complaints. No personal or family history of breast cancer.
FINDINGS:
Right breast tissue predominantly fatty with scattered areas of fibroglandular density. No mammographic evidence of malignancy. No suspicious mass, architectural distortion, or suspicious calcifications.  
Left breast tissue predominantly fatty with scattered areas of fibroglandular density. No mammographic evidence of malignancy. No suspicious mass, architectural distortion, or suspicious calcifications.  
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. No mammographic evidence of malignancy in either breast.
3. Annual screening mammograms are recommended.
BIRADS 1.
ADDENDUM: Comparison to prior outside mammograms from Jan 2018, currently no significant interval change. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM FOR SCREENING. CLINICAL HISTORY: 51-year-old female presents for her screening mammogram. No family history of breast cancer. Patient reports no personal history of breast cancer. Patient has extremely dense breasts clinically. TECHNIQUE: Standard CC and MLO views were obtained. COMPARISON: Comparison is made to the patient's prior mammogram dated 12/27/2023. FINDINGS: The breasts are extremely dense. There is no mammographic evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: 1. Extremely dense breast tissue. Mammography is less accurate in women with dense breasts. This means that cancers may be missed that would be seen with mammography in women with average density breasts. We strongly encourage you to talk to your doctor about your dense breast tissue. 2. No mammographic evidence of malignancy. 3. BI-RADS Category 2: Benign finding. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared to prior mammogram from 1 year ago. 
FINDINGS:
The breasts are predominantly fatty.
There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable benign appearing calcifications are noted in the right breast.
IMPRESSION:
NEGATIVE SCREENING MAMMOGRAM. BI-RADS 1. 
Recommend continued annual screening in 12 months. ➡ *Not Stated*
- LEFT MAMMOGRAM
HISTORY: Left breast pain and palpable mass, reported by the patient. 
COMPARISON: April 2024 study available.
FINDINGS:
The left breast is heterogeneously dense, with scattered areas of fibroglandular density. There are no dominant masses, areas of architectural distortion, or suspicious calcifications. 
Stable, coarse calcifications are noted in the central left breast (marker placed for localization).
IMPRESSION: 
No mammographic evidence of malignancy. Negative for suspicious calcifications. Recommend annual screening mammogram and clinical follow up of the palpable abnormality as clinically warranted.
BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAM LEFT BREAST.
HISTORY: Screening.
COMPARISONS: 11/21/2021.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- MAMMOGRAPHY REPORT
INDICATION: Annual screening mammogram. No current breast complaints.
COMPARISON: Prior mammogram from 1 year ago.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
Right breast: There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast: There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable benign-appearing calcifications are seen bilaterally.
No significant interval change compared to prior mammogram.
IMPRESSION:
BI-RADS Category 1: Negative bilateral screening mammogram.
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAMS.
INDICATION: Annual screening mammography.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: Compared to prior mammograms from September 2019.
FINDINGS: 
The breasts are heterogeneously dense (C), which may obscure small masses. 
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
Scattered vascular calcifications bilaterally, stable.
At the 9 o'clock position of the right breast, approximately 4 cm from the nipple, there is a new 6 mm cluster of pleomorphic microcalcifications. Additional magnification views confirm these are suspicious and new compared to prior mammograms.
No suspicious axillary lymphadenopathy.
Breast implants are noted bilaterally, unchanged in appearance.
IMPRESSION:
BI-RADS 4: Suspicious microcalcifications in the right breast at 9:00, 4 cm from the nipple. Biopsy is recommended. The remainder of the bilateral mammograms are negative.
ADDENDUM: 
The patient returned for stereotactic core biopsy of the right breast microcalcifications. Specimen radiograph confirms microcalcifications within the cores. Final pathology results are pending and will be reported separately. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to previous outside screening mammogram from 2 years ago.

FINDINGS:

The breasts are heterogeneously dense, which may obscure detection of small masses.

There is no significant asymmetry, architectural distortion, or suspicious masses in either breast.

In the left breast at 10 o'clock, 6 cm from the nipple, there is a new 8 mm cluster of amorphous microcalcifications. These were not present on the prior study.

The remaining fibroglandular tissue is unremarkable.

IMPRESSION:

BI-RADS Category 0: Incomplete

New cluster of amorphous microcalcifications in the left breast at 10 o'clock, 6 cm from the nipple, suspicious for malignancy. Additional magnification views are recommended for further evaluation.

ADDENDUM:

Magnification views of the left breast in the craniocaudal and mediolateral projections were performed. The amorphous microcalcifications at 10 o'clock, 6 cm from the nipple, are more conspicuous and appear suspicious.

REVISED IMPRESSION: 

BI-RADS Category 4: Suspicious abnormality

8 mm cluster of suspicious amorphous microcalcifications in the left breast. Stereotactic biopsy is recommended for definitive diagnosis. ➡ *Positive*
- REPORT:
"DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS AND TARGETED RIGHT BREAST ULTRASOUND.

INDICATION: Callback for grouped calcifications upper outer quadrant right breast.

PREVIOUS OUTSIDE EXAMINATIONS: 2018 screening bilateral digital mammograms from the Womens College Hospital in Toronto were sent for comparison.

FINDINGS: The breasts are heterogeneously dense. 

A dominant asymmetry is noted on the craniocaudal view of the right breast which corresponds to a small area of relatively grouped and pleomorphic calcifications in the upper outer quadrant right breast 10 cm from the nipple. On the prior from 1 year ago there is a smaller area of pleomorphic calcifications in the same area suggesting some interim change. Additional magnification views confirm these findings. 

On targeted ultrasound at the 10:00 position 10 cm from the nipple there is a 7 x 4 x 5 mm irregular hypoechoic mass with indistinct margins and no posterior acoustic shadowing. There is no increased vascularity associated.  No other sonographic abnormalities or enlarged axillary lymph nodes.

In the left breast there are no suspicious mammographic findings or palpable sonographic correlates. BIRADS 2.

IMPRESSION: 
BI-RADS 4B: Suspicious pleomorphic calcifications upper outer quadrant right breast with an associated small spiculated mass on targeted ultrasound. Recommend image guided core biopsy. 

ADDENDUM - ULTRASOUND GUIDED CORE BIOPSY RIGHT BREAST:
Under ultrasound guidance using a 14 gauge spring loaded core biopsy device, 5 tissue cores were obtained from the sonographic lesion at the 10:00 position right breast. The cores were submitted in formalin. The post biopsy mammogram shows the clip to be in good position. The patient tolerated the procedure well. There were no immediate complications.

ADDENDUM - PATHOLOGY RIGHT BREAST BIOPSY:
The core biopsy samples demonstrate invasive ductal carcinoma, grade 2, with associated high grade DCIS. Calcifications are present. Estrogen receptor positive, Progesterone receptor negative, Her2/neu equivocal (2+) by immunohistochemistry. Additional FISH testing for Her2 gene amplification will be performed. The patient will be notified and referral made to surgical oncology.
"
LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- TMIST FFDM OBSP SCREENING MAMMOGRAM. NO SIGNIFICANT FINDINGS.
This patient was enrolled in the TMIST study and randomized to the full field digital mammography arm. Informed consent was obtained as per REB approved protocol. 
Comparison to prior exam: Available.
Breast tissue density: Heterogeneously dense. The breasts are not extremely dense.
Routine OBSP screening recommended in 1 year.
There are no mammographic abnormalities, masses, architectural distortions or suspicious findings. No statement can be made regarding the presence or absence of microcalcifications.
BIRADS Category 1 - Negative. 
Recommend routine screening in 12 months. ➡ *Not Stated*
- REPORT:
"BILATERAL MAMMOGRAM
HISTORY: Annual screening. Last mammogram 1 year prior at this facility.
Family history: Mother with breast cancer in her 70s. No personal history of breast cancer. 
FINDINGS: Bilateral scattered fibroglandular densities. Breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant mass, suspicious calcifications, or focal asymmetry.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography. 
2. Breast tissue is heterogeneously dense, which may lower the sensitivity of mammography.
BI-RADS Category 1: Negative.
ADDENDUM: Comparison to prior studies at this facility from 1 year ago shows no significant interval change, stable scattered fibroglandular tissue bilaterally.
RECOMMENDATION: Annual screening mammography is recommended based on patient risk factors and current screening guidelines." ➡ *Negative*
- BREAST ULTRASOUND REPORT

Clinical History: Patient presents for breast ultrasound evaluation for assessment of microcalcifications in the left breast. These were incidentally discovered on screening mammography and the patient denies any palpable abnormalities.

Comparison: Mammogram performed on 07/15/2024.

Findings:
Left breast: Within the left breast at approximately the 10 o'clock position, 2 cm from the nipple, there is a small cluster of microcalcifications measuring 5 mm in greatest dimension. These microcalcifications are clustered within a small area of architectural distortion. No associated mass is definitively identified. There is no surrounding ductal dilatation. No abnormal axillary lymph nodes are seen.
Right breast: No suspicious masses or calcifications are identified.

Impression:
Clustered microcalcifications within the left breast as described above. Ultrasound-guided core biopsy is recommended.

BI-RADS: 4 (Suspicious) ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY
INDICATION: Annual screening mammogram.
COMPARISON: None.
FINDINGS:
RIGHT BREAST:
The breast tissue is heterogeneously dense.
There are no dominant masses, architectural distortions, or significant asymmetries.
There are no suspicious calcifications.
LEFT BREAST:
The breast tissue is heterogeneously dense.
At 12:00, 6 cm from the nipple, there is a suspicious 1.1 cm spiculated mass with associated pleomorphic microcalcifications spanning approximately 2 cm. 
At 6:00, 4 cm from the nipple, there is an indeterminate 7 mm hypoechoic mass.
There are no dominant masses elsewhere, architectural distortions, or additional significant asymmetries.
IMPRESSION:
1. Highly suspicious findings in the left breast, BI-RADS 5. Further evaluation with diagnostic mammography, spot compression views, ultrasound, and biopsy is recommended.   
2. Benign right breast, BI-RADS 1.
ADDENDUM:  
LEFT DIAGNOSTIC MAMMOGRAPHY AND ULTRASOUND
Additional magnification and spot compression views of the left breast demonstrate the spiculated mass with pleomorphic microcalcifications at 12:00. Ultrasound shows an irregular hypoechoic mass with indistinct margins and posterior shadowing, corresponding to the mammographic mass. The mass measures 11 x 8 x 9 mm.  
Ultrasound also shows the 7 mm hypoechoic mass at 6:00, which demonstrates circumscribed margins.
IMPRESSION:
1. Highly suspicious mass left breast 12:00, BI-RADS 5. Ultrasound-guided core needle biopsy is recommended.
2. Probably benign mass left breast 6:00, BI-RADS 3. Short term follow-up is recommended. ➡ *Positive*
- FINDINGS:
The breast tissue is heterogeneously dense, which could obscure detection of small masses. 
No dominant mass, suspicious calcifications or architectural distortion.
There are no new masses, calcifications, or other findings.
Stable postsurgical changes noted in the upper outer quadrant of the right breast. Surgical clips are visible.
There are bilateral implants, unchanged in appearance.
No axillary lymphadenopathy.

IMPRESSION:
No mammographic evidence of malignancy.
BI-RADS category 1: Negative.
Recommend routine screening in 1 year. ➡ *Negative*
- IMPRESSION:
1. Left craniocaudal and mediolateral oblique views and targeted ultrasound demonstrate scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications.
2. Ultrasound demonstrates no focal lesion in the area of palpable concern at 7 o'clock position left breast middle depth. Recommend clinical follow-up and diagnostic mammogram if clinically indicated.
3. Interval increase in fibroglandular density in the upper outer left breast, likely reflecting variable compression. No sonographic correlate.
BIRADS 2: Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- FINDINGS: Bilateral mammogram and ultrasound performed. The breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast: There are no dominant masses, architectural distortion, or suspicious calcifications.

Left breast: There are no dominant masses, architectural distortion, or suspicious calcifications. Targeted ultrasound to the area of palpable concern at 9 o'clock position in the left breast demonstrates normal fibroglandular tissue with no sonographic evidence of malignancy.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend routine screening in 1 year.
2. No sonographic evidence of malignancy in the left breast 9 o'clock position. 

BIRADS 1 - Negative. ➡ *Negative*
- SCREENING DIGITAL MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Screening mammogram from 1 year prior.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Stable scattered fibroglandular densities are noted in both breasts.

IMPRESSION:
No mammographic evidence of malignancy. Recommend routine screening in 1 year.

BIRADS Assessment Category: 2 - Benign findings.

ADDENDUM: 
The breasts remain heterogeneously dense. No suspicious masses, calcifications, or architectural distortion are identified. Specifically, no microcalcifications are mentioned in this report. Continued annual screening as per guidelines is recommended. ➡ *Not Stated*
- MAMMOGRAPHY, LEFT BREAST.

CLINICAL INFORMATION: 64 year old female with history of right breast cancer status post mastectomy presents for annual screening mammogram of the left breast.

COMPARISON: Left mammogram from 05/2022.

TECHNIQUE: Standard CC and MLO views of the left breast were obtained.

FINDINGS:
The left breast tissue is heterogeneously dense (BI-RADS category C), which may obscure small masses. 
There are no suspicious masses, architectural distortions or dominant focal asymmetries. 
No suspicious grouped, clustered, linear, or segmental microcalcifications are identified.
The skin is unremarkable. No significant axillary lymphadenopathy is seen.

IMPRESSION: 
No mammographic evidence of malignancy in the left breast.
BI-RADS Category 1: Negative.

RECOMMENDATION:
Annual screening mammogram of the left breast in 1 year. ➡ *Negative*
- MAMMOGRAPHY, LEFT.
INDICATION: 6 month follow up of palpable mass left breast.
TECHNIQUE: Craniocaudal and mediolateral oblique projections of the left breast.
COMPARISON: Comparison is made to prior exam of December 12, 2022.
FINDINGS: The left breast is heterogeneously dense. There is a stable oval mass at the 3 o'clock position, corresponding to the area of palpable concern. This measures 8 x 5 mm and is unchanged in size and appearance compared to prior exam. The margins are circumscribed. No other mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION: 
Stable benign appearing mass left breast at site of palpable abnormality, likely representing a fibroadenoma. No mammographic evidence of malignancy. 
BIRADS 2, Benign finding. Recommend routine screening in 1 year.
ADDENDUM:
Ultrasound of the left breast 3 o'clock mass was performed. This corresponds to an oval circumscribed hypoechoic mass with thin echogenic capsule, measuring 8 x 5 x 6 mm. No suspicious features identified. This is most consistent with a fibroadenoma. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening

COMPARISON: None

FINDINGS:

Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no suspicious masses, architectural distortion, or skin thickening in either breast.

In the left breast at the 10:00 position, 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. This is a new finding compared to the patient's prior mammograms. Additional magnification views confirm the presence of the microcalcifications.

The right breast is unremarkable, with no suspicious microcalcifications, masses, or areas of architectural distortion. 

IMPRESSION:

Left breast: BI-RADS 4B: Suspicious microcalcifications at 10:00, 6 cm from the nipple. Stereotactic biopsy is recommended for further evaluation.

Right breast: BI-RADS 1: Negative.

ADDENDUM:

The patient underwent stereotactic biopsy of the left breast microcalcifications. Pathology results show ductal carcinoma in situ (DCIS), intermediate nuclear grade, with no evidence of invasion. ➡ *Positive*
- NORMAL SCREENING BILATERAL DIGITAL MAMMOGRAPHY.

INDICATION: Routine screening mammography.

COMPARISON: None available. 

FINDINGS:
Breast composition is primarily fatty on both sides, with scattered areas of fibroglandular density.

No dominant masses, architectural distortions, or suspicious calcifications are identified in either breast.  

No significant axillary, supraclavicular, or internal mammary adenopathy.

IMPRESSION: 
Bilateral fatty replaced breasts with no mammographic evidence of malignancy. 
BI-RADS Category 1: Negative.

Recommend routine screening in 1 year per institutional guidelines. ➡ *Not Stated*
- FINDINGS: The breast tissue is heterogeneously dense, which may obscure detection of small masses. No dominant mass, suspicious calcifications or abnormal nipple discharge is identified. There are no new findings compared to the prior exam.

Targeted ultrasound was performed at the site of the palpable abnormality in the right breast at 10 o'clock, 8 cm from the nipple. No correlate to the palpable finding was identified on ultrasound. The palpable finding likely represents normal fibroglandular tissue.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography in 12 months.
2. Negative targeted ultrasound of the palpable finding in the right breast at 10 o'clock. This likely represents normal fibroglandular tissue. Clinical follow up as needed.

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHIC EXAMINATION OF BOTH BREASTS
INDICATION: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: Comparison is made to prior mammograms from 1/15/2022.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious clustered microcalcifications in either breast. 
Stable bilateral benign appearing calcifications are again identified. 
There are no significant changes compared to prior mammograms.
IMPRESSION: 
BI-RADS Category 1: Negative Screening Mammogram. 
Recommend continued annual screening mammography in 12 months.
ADDENDUM: 
This examination was reviewed by the interpreting radiologist using an FDA approved monitor for final interpretation. The images are of good diagnostic quality unless otherwise noted above. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAPHY

CLINICAL HISTORY: 42 year old female presents for screening mammogram.  No clinical concerns.  Patient denies breast symptoms.

COMPARISON: None.

FINDINGS:
Both breasts are composed of heterogeneously dense fibroglandular tissue.
There are no dominant masses, architectural distortion or suspicious calcifications.
No significant asymmetries are seen.
The skin appears normal.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year.
2. Heterogeneously dense parenchyma, which could obscure detection of small masses. The patient may benefit from 3D mammography for future screening exams if available.
BIRADS 1 - NEGATIVE. ➡ *Negative*
- RIGHT BREAST ULTRASOUND.
CLINICAL HISTORY: Right breast palpable abnormality
COMPARISONS: None.
TECHNIQUE: The patient was positioned for optimal exposure of the right breast. Ultrasound images were obtained.
FINDINGS:
Within the palpable area, there is a 1.1 cm circumscribed, oval, wider-than-tall, hypoechoic mass. This is likely a fibroadenoma. No suspicious axillary lymph nodes are identified.
IMPRESSION:
Right breast palpable abnormality - Ultrasound demonstrates a likely benign appearing mass, see above. This correlates with the palpable abnormality. BI-RADS Category 2 ➡ *Not Stated*
- RADIOLOGY REPORT
CLINICAL HISTORY: 46-year-old female presents for routine screening mammogram. No current breast complaints. Patient has a family history of breast cancer in her maternal aunt.

TECHNIQUE: Bilateral digital mammography was performed including CC and MLO views.

FINDINGS:

RIGHT BREAST:
The right breast is composed of scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications. The skin is normal in appearance. 

LEFT BREAST: 
The left breast is composed of scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications. The skin is normal in appearance.

IMPRESSION:
1. Bilateral mammogram, BI-RADS category 1: Negative. 
2. There are no suspicious mammographic findings bilaterally. Routine annual screening mammography is recommended.

ADDENDUM:
Comparison was made to the patient's prior mammogram from 08/15/2021, which was also negative. There are no significant interval changes. ➡ *Negative*
- CLINICAL HISTORY: 53-year-old female presents for bilateral diagnostic mammogram and ultrasound. Patient reports a new palpable lump in the left breast at 10 o'clock position. Last mammogram was 18 months ago at an outside facility and was reported as normal. Patient has a family history of breast cancer in her mother diagnosed at age 65.

FINDINGS:
Both breasts show scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified in either breast.

Targeted ultrasound of the left breast at 10 o'clock position, 3 cm from the nipple, demonstrates a 0.8 x 0.6 x 0.5 cm oval, circumscribed, hypoechoic mass, parallel in orientation. This corresponds to the patient's palpable abnormality. No suspicious features are identified.

IMPRESSION:
1. Bilateral mammogram shows no evidence of malignancy. Scattered fibroglandular densities are noted. 
2. Left breast ultrasound shows a 0.8 cm circumscribed oval mass at 10 o'clock, corresponding to the palpable finding. This has benign features and likely represents a fibroadenoma. 
3. No microcalcifications are identified in either breast.

RECOMMENDATION: 
Routine annual screening mammography is recommended. Short-term follow up ultrasound of the left breast mass in 6 months to document stability. 

BI-RADS Category 2: Benign finding. Recommend routine screening. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Screening.  
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available.
FINDINGS:  
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications.  
IMPRESSION:
BI-RADS Category 1: Negative.
Routine screening is recommended in 1 year. ➡ *Not Stated*
- MAMMOGRAM REPORT

EXAMINATION: BILATERAL SCREENING MAMMOGRAM

COMPARISON: None available.

FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

There are no dominant masses, suspicious calcifications, or other findings that are concerning for malignancy.

IMPRESSION:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM.

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to prior mammogram dated June 15, 2021.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications. 

Stable benign-appearing calcifications are noted in the right breast.

There are no new mammographic findings that are concerning for malignancy.

IMPRESSION:
BI-RADS Category 2: Benign finding. Recommend routine screening in 1 year.

ADDENDUM: 
After further review of the images and discussion with the interpreting radiologist, no microcalcifications were specifically mentioned in the body of the report. The interpreting radiologist confirms that no suspicious microcalcifications are identified in either breast on today's examination. The final assessment category remains BI-RADS 2. ➡ *Not Stated*
- LEFT MAMMOGRAM.
HISTORY: Screening mammogram. No current breast complaints.
FINDINGS: The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No dominant masses, architectural distortion or suspicious calcifications are identified.
Stable benign-appearing calcifications are noted in the left breast, likely vascular in nature.
IMPRESSSION: No mammographic evidence of malignancy.
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- TMIST FFDM ARM OBSP SCREENING MAMMOGRAM.
The patient is enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) with IRB approval and informed consent. Randomized to the full field digital mammography only arm.  
Comparison with prior examinations: Yes.  
Breast tissue density: Moderate to marked background fibroglandular density, estimated at 50-75%.
OBSP routine screening interval recommended: Yes, 1 year.
No mammographic abnormalities suspicious for malignancy are identified. The breasts are heterogeneously dense, which may obscure detection of small masses.
BIRADS Category 2: Benign. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM. CLINICAL HISTORY: 53 year old female presents for her annual screening mammogram. Comparison is made to prior mammogram from 11/14/2023. FINDINGS: The breasts are heterogeneously dense. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: Negative mammogram. BI-RADS 1. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM WITH TOMOSYNTHESIS
CLINICAL HISTORY: 59-year-old female, presents for screening mammogram. No current breast complaints.
COMPARISON: Mammogram from 08/2019.
FINDINGS:
Both breasts are heterogeneously dense (C), which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.
In the right breast at the 11:00 position 6 cm from the nipple, there is a new 5 mm cluster of amorphous calcifications. Additional magnification views in the CC and MLO projections confirm these are suspicious and new compared to prior exams.
In the left breast, there are scattered areas of fibroglandular density and several small cysts, all stable compared to prior mammograms. No suspicious masses, calcifications, or architectural distortion.
IMPRESSION:
BI-RADS 4: Suspicious cluster of calcifications in the right breast at 11:00, 6 cm from the nipple. Biopsy is recommended for definitive characterization. Stereotactic core biopsy to be scheduled with patient's permission.
BI-RADS 2: Benign findings in the left breast. Recommend continued annual screening with mammography in 1 year. ➡ *Positive*
- SCREENING MAMMOGRAM. 
CLINICAL INDICATION: Screening. 
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Previous mammogram dated 2 years ago.
FINDINGS: 
The breasts are heterogeneously dense (density category C), which could obscure detection of small masses.
There are no dominant masses, suspicious calcifications, or concerning focal asymmetries.  
There are bilateral scattered fibroglandular densities. No significant interval change.
IMPRESSION: 
Heterogeneously dense breasts with no mammographic evidence of malignancy. 
RECOMMENDATION:
Continue annual screening mammography in 12 months.
BIRADS CATEGORY 2 - Benign findings. 
ADDENDUM: This examination was double-read by Dr. Jane Smith with concordant interpretation. ➡ *Not Stated*
- TMIST FULL FIELD DIGITAL MAMMOGRAPHY SCREENING MAMMOGRAM.
Reason for exam: Routine screening. 
Procedure: Standard full field digital mammographic views of both breasts were obtained as part of the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) full field digital mammography only arm.
Comparison: Previous mammogram from 1 year ago.
Breast composition: The breasts are heterogeneously dense. This can obscure small masses.
Findings: The breast tissue appears normal. No significant masses, architectural distortion or suspicious calcifications are seen.
Impression:
BI-RADS Category 1: Negative. 
Recommend routine screening in 1 year. ➡ *Not Stated*
- ULTRASOUND LEFT BREAST

CLINICAL INFORMATION: 
49-year-old female with complaint of left breast tenderness.

COMPARISON:  
None.

FINDINGS:
Targeted ultrasound of the left breast demonstrates normal appearing fibroglandular tissue. No focal lesions or masses are identified. No cysts. No dilated ducts. No skin thickening or retraction. No microcalcifications.

IMPRESSION:  
Normal left breast ultrasound. No sonographic evidence of malignancy.

BI-RADS Category 1: Negative. Routine screening is recommended. ➡ *Negative*
- SCREENING MAMMOGRAM.  
INDICATION:  Screening.  High risk.  
TECHNIQUE:  Bilateral CC and MLO views.
COMPARISON:  Screening exam 1 year ago. 
FINDINGS:  The breasts are heterogeneously dense (C).
There are no masses, asymmetries, or areas of architectural distortion.
The skin is normal.  There are no suspicious calcifications.
IMPRESSION:
BIRADS 2 - Benign findings.  Heterogeneously dense breasts.
Recommend annual screening mammography.
ADDENDUM: Further review and discussion with colleague, Dr. Jane Smith. We agree with the above findings and impression. No suspicious microcalcifications are identified. ➡ *Not Stated*
- SCREENING MAMMOGRAM WITH DBT.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: No.

FINDINGS:
The breast tissue is heterogeneously dense. There are no dominant masses, architectural distortion or suspicious calcifications. 

Routine annual screening with digital breast tomosynthesis is recommended.

ASSESSMENT: 
BI-RADS Category 1: Negative.

ADDENDUM:
Comparison is made to previous mammogram dated March 15, 2021 at XYZ Breast Center. No significant interval change. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM. CLINICAL HISTORY: Patient is a 54-year-old female presenting for her screening mammogram. TECHNIQUE: Bilateral CC and MLO images were obtained. COMPARISON: Comparison is made to prior mammographic examination from 11/08/2023. FINDINGS: The breasts are heterogeneously dense. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: Negative mammogram. BI-RADS 1. ➡ *Negative*
- Screening Mammogram. Clinical History: 45 year old female for routine screening mammogram. Findings: There is dense fibroglandular tissue bilaterally. There are no masses, architectural distortion, or suspicious calcifications. Impressions: 1. No mammographic evidence of malignancy. 2. Bilateral dense breast tissue. It is important to note that mammography is less sensitive in detecting cancer in women with dense breast tissue. Please see your physician to discuss additional screening methods for breast cancer such as Breast MRI. 3. BI-RADS Category 2: Benign findings. ➡ *Negative*
- MAMMOGRAPHY REPORT

INDICATION: Annual screening mammogram. No current breast complaints.

COMPARISON: Screening mammogram from 12/15/2021. 

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
There are scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications are identified in either breast. 

There are stable benign appearing calcifications in the right breast, unchanged from prior exam.

The left breast is stable with no suspicious findings.

IMPRESSION: 
BI-RADS Category 1 - Negative bilaterally. 
No mammographic evidence of malignancy.
Routine annual screening mammography is recommended.

ADDENDUM:
Tomosynthesis and ultrasound images were reviewed. No additional suspicious findings were identified on these supplemental imaging modalities. The final assessment remains BI-RADS 1, negative bilaterally. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Bilateral digital mammography was performed including CC and MLO views of each breast. 

COMPARISON: Previous outside screening mammogram from 11/16/2020.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

Magnification views of both breasts taken to further evaluate vague asymmetries seen on screening views. In the left breast at 3 o'clock, 7 cm from the nipple, there is a new cluster of indeterminate microcalcifications spanning approximately 8 mm. This cluster was not present on the prior mammogram. 

In the right breast, the magnification views show no significant change in the previously noted benign appearing calcifications scattered throughout the breast.

No suspicious axillary lymphadenopathy.

IMPRESSION:
New cluster of indeterminate microcalcifications in the left breast at 3 o'clock, 7 cm from the nipple, spanning 8 mm. Recommend diagnostic mammographic workup of the left breast for further evaluation, to include spot compression and magnification views. BI-RADS 0.

Stable right breast. BI-RADS 2. 

Recommend routine annual screening mammography in 1 year, with the new cluster of microcalcifications in the left breast to be addressed in the interim with diagnostic evaluation. ➡ *Positive*
- FINDINGS: Both breasts are predominantly fatty replaced. There are scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications.

IMPRESSION: Negative bilateral screening mammogram. 

BIRADS 1 - Recommend continued annual screening in 12 months. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM.

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available.

FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

There are no dominant masses, architectural distortion or suspicious calcifications.

IMPRESSION:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year.

ADDENDUM:
The client has been notified of the results. As dense breast tissue is noted and may lower the sensitivity of mammography, the client has been given the option of supplemental screening with breast ultrasound. ➡ *Not Stated*
- MAMMOGRAPHY, BILATERAL, DIAGNOSTIC.
INDICATION: Right breast lump, palpated by physician and patient.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
The breast tissue is heterogeneously dense which could obscure detection of small masses. No dominant mass, architectural distortion or suspicious microcalcifications are seen in either breast. Benign bilateral vascular calcifications are seen. Benign calcifications are seen in the right breast, likely representing dermal calcifications.
Supplemental screening ultrasound is recommended to evaluate for right breast mass.
IMPRESSION:
1. Negative bilateral mammogram. No suspicious microcalcifications, masses or architectural distortion.
2. Dense breasts with possible decreased sensitivity of mammography.
3. Recommend targeted right breast ultrasound to evaluate palpable abnormality. 
BIRADS 1 - Negative mammogram. Recommend supplemental screening with ultrasound. ➡ *Negative*
- MAMMOGRAM, BILATERAL.
INDICATION: Annual screening. No current breast complaints. Personal history of left breast invasive ductal carcinoma treated with lumpectomy and radiation 7 years ago.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. Stable post-surgical changes are noted in the upper outer quadrant of the left breast. 
IMPRESSION: 
No mammographic evidence of malignancy. 
BI-RADS 1: Negative.
RECOMMENDATION: 
Continue annual screening mammography in 12 months. ➡ *Negative*
- ULTRASOUND OF THE RIGHT BREAST

INDICATION: Right breast lump identified on screening mammography.

TECHNIQUE: Targeted ultrasound of the right breast was performed at the 10 o'clock position, 5 cm from the nipple. 

COMPARISON: None available.

FINDINGS:
There is a heterogeneous breast composition with scattered fibroglandular tissue. At the area of concern in the right breast upper outer quadrant, 10 o'clock position 5 cm from the nipple, there is an oval circumscribed hypoechoic solid mass measuring 7 x 5 x 4 mm. The mass demonstrates no posterior acoustic features and has circumscribed margins. No suspicious calcifications are identified. 

The remainder of the right breast is unremarkable. No axillary lymphadenopathy is seen.

IMPRESSION:
Right breast 10 o'clock 5 cm from the nipple: 7 mm oval circumscribed solid mass, likely benign. BIRADS 3, probable fibroadenoma. Short term follow up is recommended to document stability. ➡ *Negative*
- FINDINGS: 
Right breast: Fatty replaced breast, post mastectomy changes noted, surgical clips identified. No suspicious masses, architectural distortion, or skin changes identified. 
Left breast: Heterogeneously dense breast tissue. No dominant masses or architectural distortion. No sonographic correlate for palpable abnormality in the lower outer quadrant on targeted ultrasound. Breast ultrasound reveals normal fibroglandular tissue and fat lobules, with no discrete solid or cystic lesions.

IMPRESSION:
1. Benign right post-mastectomy changes. 
2. No suspicious mammographic findings in the left breast. No sonographic correlate for palpable abnormality in the 4-5 o'clock position of the left breast. 
3. Recommend clinical follow up of palpable abnormality as indicated. Routine annual screening mammography is recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- SCREENING MAMMOGRAM:
Comparison: None.
The breast tissue is predominantly fatty.
There is a group of amorphous calcifications within the lateral right breast in the middle depth region.
No dominant mass, architectural distortion or suspicious calcifications.
IMPRESSION: 
BIRADS 0: Additional imaging needed for evaluation of grouped amorphous calcifications within the lateral right breast. Spot compression magnification views of this area are recommended. Management will depend on those results.
ADDENDUM:
Additional spot magnification views of the right lateral breast were performed. The previously described amorphous calcifications are confirmed to represent benign-appearing vascular calcifications distributed in a coarse region. No other suspicious grouped calcifications identified.
REVISED IMPRESSION:
BIRADS 2: Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- TECHNIQUE: Bilateral digital mammography.

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammogram from 12/15/2021.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast:
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Left breast:
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

IMPRESSION:
1. No evidence of malignancy in either breast. 
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
3. Stable mammographic appearance compared to prior exam.

RECOMMENDATION:
Annual screening mammogram is recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- BENIGN MAMMOGRAM.
Clinical history: None.
Compared to Previous: No.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
There are no suspicious masses, architectural distortions or suspicious microcalcifications. 
There are scattered areas of benign-appearing calcifications.  
IMPRESSION: 
NEGATIVE MAMMOGRAPHIC EXAMINATION, BIRADS 2.  
Routine screening is recommended in 1 year. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
INDICATION: 60-year-old woman, routine screening.
COMPARISON: Screening mammogram from 2 years prior.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses. 
There are no suspicious masses, architectural distortion, or concerning asymmetries.
Scattered fibroglandular densities are present bilaterally.
In the left breast at the 10:00 position, 6 cm from the nipple, there is a new cluster of fine pleomorphic microcalcifications spanning an area of approximately 8 mm. This was not present on the prior study. Additional magnification views in the ML and CC projections confirm these calcifications.
No suspicious calcifications are seen in the right breast.
No skin thickening or retraction. Nipples are within normal limits. No axillary lymphadenopathy.
IMPRESSION: 
New cluster of suspicious microcalcifications in the left breast at 10:00, 6 cm from the nipple, spanning 8 mm. BIRADS 4B. Stereotactic biopsy is recommended for further evaluation. The patient will be contacted to arrange the biopsy.
Remainder of the bilateral breast examination is benign. BIRADS 2 for the right breast. ➡ *Positive*
- MAMMOGRAM REPORT
HISTORY: Annual screening mammogram. Patient has a history of right breast lumpectomy and radiation therapy 10 years ago. No current breast concerns. 
FINDINGS:
Right breast: Postoperative changes from prior lumpectomy are noted in the upper outer quadrant. There are no new masses, architectural distortion, or suspicious calcifications. 
Left breast: The breast is heterogeneously dense. There are no masses, architectural distortion, or suspicious calcifications. 
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Postoperative changes in the right breast are stable.
BIRADS 1 - Negative. Recommend continued annual screening mammography. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL INFORMATION: Annual screening.
COMPARISON: Mammograms from 05/2021.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications.
There are no significant changes compared to prior mammograms.
IMPRESSION:
BIRADS 1 - There are no mammographic findings to suggest malignancy. Recommend routine screening in 1 year. ➡ *Negative*
- BILATERAL MAMMOGRAM AND ULTRASOUND.
HISTORY: 45-year-old female with palpable right breast lump at 8 o'clock. Last mammogram done at St. Mary's in June 2022 was normal. Maternal aunt with history of breast cancer.
FINDINGS:
Right breast: The breast is heterogeneously dense. There are no suspicious masses, architectural distortion, or suspicious calcifications. In the area of palpable concern at 8 o'clock position, there is a 1.2 cm oval circumscribed mass, likely benign.
Left breast: The breast is heterogeneously dense. There are no suspicious masses, architectural distortion, or suspicious calcifications.
Right breast ultrasound was performed, targeted to the 8 o'clock position. A 1.2 cm oval circumscribed parallel hypoechoic mass is seen, consistent with fibroadenoma.
IMPRESSION:
1. No evidence of malignancy in either breast. The palpable right breast mass at 8 o'clock position corresponds to a 1.2 cm oval circumscribed mass on mammogram and ultrasound, consistent with fibroadenoma. 6-month follow-up is recommended to document stability.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
BIRADS 2.

ADDENDUM:
Comparison made to prior mammogram from St. Mary's in June 2022. No significant interval change. The right breast mass is unchanged. ➡ *Negative*
- FINDINGS:

BILATERAL MAMMOGRAM:

The breasts are mostly fatty replaced.  There are scattered areas of fibroglandular density.  There are no dominant masses or areas of architectural distortion.

In the right breast at 1:00, 8 cm from the nipple, there is a new group of amorphous calcifications spanning approximately 8 mm. Additional mag views confirm these are suspicious calcifications.

In the left breast, there are no suspicious masses, calcifications, or areas of architectural distortion.

Skin is normal in thickness bilaterally.  There are no enlarged axillary lymph nodes.

IMPRESSION:
Right breast: Suspicious new group of amorphous calcifications at 1:00, 8 cm from the nipple. BIRADS 4b. Stereotactic core biopsy recommended for diagnosis.

Left breast: No suspicious mammographic findings. BIRADS 2.

ADDENDUM:
Stereotactic biopsy of the right breast calcifications was performed using an 8 gauge vacuum-assisted biopsy device. Specimen radiograph confirms that the suspicious calcifications were sampled.

BIOPSY RESULT: Invasive ductal carcinoma, grade 2, with associated ductal carcinoma in situ, intermediate nuclear grade. ER positive, PR positive, Her2/neu negative.

This is a new diagnosis of breast cancer. The patient will be referred to breast surgery for further management.

FINAL ASSESSMENT: BIRADS 6, right breast. Known biopsy-proven malignancy. ➡ *Positive*
- MAMMOGRAM - BILATERAL SCREENING

CLINICAL HISTORY: 62 year old female presents for her screening mammogram. No breast complaints. Family history of breast cancer in her mother.

COMPARISON: 10/24/2023.

FINDINGS: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications in both breasts. There is a new 1 cm cluster of pleomorphic calcifications in the left breast at 9 o'clock, 3 cm from the nipple. No architectural distortion or suspicious masses.

IMPRESSION:

1. New cluster of pleomorphic calcifications in the left breast, recommended for further evaluation with magnification views. 

BI-RADS: 0 (Needs additional imaging evaluation). ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM. History: Screening. Comparison: With previous mammographic examination of 09-08-2023. Findings: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications in both breasts. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: 1. No mammographic evidence of malignancy. 2. BI-RADS 1. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
CLINICAL HISTORY: 55 year old woman. No complaints. Annual screening.
TECHNIQUE: CC and MLO views of both breasts. CC and spot magnification CC views of the right breast.
COMPARISON: None.
FINDINGS:
Right Breast: The breast is heterogeneously dense (BI-RADS category C). There are scattered areas of fibroglandular density. In the upper outer quadrant there is a 7 mm group of indeterminate amorphous microcalcifications. No discrete mass, significant architectural distortion, or suspicious calcifications are identified.
Left Breast: The breast is heterogeneously dense (BI-RADS category C). There are scattered areas of fibroglandular density. No discrete mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
BI-RADS Category 0: Incomplete assessment. Need additional imaging evaluation of upper outer right breast microcalcifications. Recommend right CC and right ML spot compression magnification views.
ADDENDUM: CC and ML spot compression magnification views of the right breast demonstrate the microcalcifications to be amorphous, punctate, and grouped in an area measuring 9 mm in greatest dimension. No associated mass is identified.
REVISED IMPRESSION: 
BI-RADS Category 4: Suspicious abnormality in the right breast. Suspicious grouped amorphous microcalcifications in the right upper outer quadrant. Recommend stereotactic biopsy.
BI-RADS Category 2: Benign finding in the left breast. ➡ *Positive*
- MAMMOGRAM AND ULTRASOUND OF THE RIGHT BREAST

HISTORY: Right breast mass at 8 o'clock position, palpable by patient. Prior left mastectomy in 2018 due to left breast cancer. Family history of breast cancer in maternal grandmother.

FINDINGS:
Left mastectomy noted.

Right breast: Scattered fibroglandular densities. No suspicious masses, architectural distortion, or calcifications are identified. 

Targeted right breast ultrasound was performed at the site of the palpable abnormality at 8 o'clock position, 4 cm from the nipple. A 6 x 4 x 5 mm oval, circumscribed, hypoechoic solid mass is identified, parallel to the skin. This corresponds to the palpable finding. No suspicious features are identified.

IMPRESSION:
1. Benign appearing solid mass in the right breast at 8 o'clock position, corresponding to the palpable abnormality. This is most likely a fibroadenoma. 6-month follow-up ultrasound is recommended to document stability. 

2. No mammographic evidence of malignancy in the right breast. Routine annual screening mammogram of the right breast is recommended.

BI-RADS 2 - Benign finding. 

ADDENDUM: Comparison made to outside ultrasound performed 3 months ago, the right 8 o'clock mass is stable in size and appearance. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
History: Screening.
Comparison available dating from 03/2023 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: 10/15/2021 at XYZ Breast Center

FINDINGS:
Both breasts are predominantly fatty.

There are no masses, architectural distortion, or suspicious calcifications.

Stable benign calcifications are noted bilaterally.

There are no significant changes compared to prior mammogram.

IMPRESSION:
Negative bilateral screening mammogram.

BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMOGRAPHIC REPORT:

INDICATION: Screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts.

FINDINGS:
The breasts are predominantly fatty replaced with scattered areas of fibroglandular density. No focal mass, architectural distortion, or suspicious calcifications are identified.

Bilateral benign lymph nodes are noted in the axillary regions.

IMPRESSION:
Negative bilateral screening mammogram.

ADDENDUM:
Examination is limited due to patient motion. Repeat mammogram is suggested if clinically indicated.

BIRADS CATEGORY 1 - Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Screening mammogram.  

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Tomosynthesis was performed bilaterally.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (BI-RADS density C).

In the right breast, there are no dominant masses, areas of architectural distortion or suspicious calcifications. 

In the left breast at the 10:00 position approximately 4 cm from the nipple, there is a new 0.4 cm cluster of fine pleomorphic calcifications. This is new compared to patient's most recent screening mammogram from 2 years ago at an outside facility. No dominant masses or areas of architectural distortion are identified in the left breast.

There are no abnormal axillary lymph nodes bilaterally.

IMPRESSION: 
Left breast: New group of suspicious microcalcifications in the 10:00 position of the left breast, BI-RADS 4B. Stereotactic biopsy is recommended for tissue diagnosis. 

Right breast: No mammographic evidence of malignancy, BI-RADS 1.

ADDENDUM: The patient returned for stereotactic biopsy of the cluster of calcifications in the left breast at the 10:00 position, using an 8G vacuum-assisted biopsy device. Specimens were obtained and sent to pathology. Post-biopsy mammograms show that the area of concern has been adequately sampled. Pathology results are pending. ➡ *Positive*
- MAMMOGRAPHY REPORT - LEFT BREAST

INDICATION: Annual screening mammogram. No current breast complaints.

COMPARISON: Prior mammograms from 11/15/2021 at XYZ Imaging Center.

FINDINGS:
LEFT BREAST:
The left breast is predominantly fatty with scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified.

RIGHT BREAST:
The right breast demonstrates scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications are seen.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. No mammographic evidence of malignancy. 
3. Recommend continued annual screening mammography per guidelines.

BIRADS Category 1 - Negative. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior studies from 10/15/2021.
FINDINGS:
Right breast: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
Left breast: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
IMPRESSION:
Right breast: No mammographic evidence of malignancy. BI-RADS 1 - Negative.
Left breast: No mammographic evidence of malignancy. BI-RADS 1 - Negative.
RECOMMENDATION: Annual screening mammography is recommended.
ADDENDUM - BREAST ULTRASOUND:
INDICATION: Dense breast tissue on mammogram.
TECHNIQUE: Targeted ultrasound of both breasts was performed.
FINDINGS: No sonographic abnormality is identified in either breast. No suspicious solid or cystic lesions are seen.
IMPRESSION: Negative targeted ultrasound of both breasts. No sonographic evidence of malignancy.
FINAL ASSESSMENT: BI-RADS 1 - Negative. Continue annual screening mammography. ➡ *Negative*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
The breast tissue is scattered fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications.
BIRADS-2. ➡ *Not Stated*
- BILATERAL MAMMOGRAM AND MAGNIFIED VIEWS OF THE RIGHT BREAST.

COMPARISON: None

INDICATION: Routine breast cancer screening. 

FINDINGS:
Both breasts are heterogeneously dense (category C), which may obscure detection of small masses. 

There are no suspicious masses, calcifications or other findings in the left breast. 

In the right breast, there is a new group of amorphous calcifications in the upper outer quadrant, spanning an area of about 10 mm. The calcifications vary in size and are grouped, concerning for ductal carcinoma in situ. No other suspicious findings are noted in the right breast.

No concerning axillary lymphadenopathy.

IMPRESSION:
BI-RADS 4: Suspicious grouped amorphous calcifications right upper outer breast, spanning 10 mm. Biopsy recommended. 

BI-RADS 2: Benign findings left breast.

ADDENDUM: Stereotactic biopsy of the right breast calcifications was performed. Pathology showed ductal carcinoma in situ, intermediate nuclear grade, cribriform and solid types, 8 mm in extent. Excision is recommended for definitive treatment. BI-RADS 6 (known biopsy proven malignancy). ➡ *Positive*
- MAMMOGRAPHY, BILATERAL.
INDICATION: Annual screening. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of the right and left breast were performed. 
COMPARISON: Prior exam from 6/15/2021.
FINDINGS:
The breasts are predominantly fatty replaced (A: Almost entirely fat). No significant change from prior.
Right Breast:
There are no masses, architectural distortions, or suspicious calcifications. Stable benign vascular calcifications in the right breast.
Left Breast: 
There are no masses, architectural distortions, or suspicious calcifications. 
Stable postoperative changes are noted from prior left lumpectomy. Surgical clips are visualized at the lumpectomy site. No suspicious enhancing lesions.
Stable benign calcifications in the left breast, unchanged from prior. No microcalcifications are identified. 
The pectoralis muscles are within normal limits.
The visualized portions of the lungs are clear. 
There are no bulky axillary lymph nodes. Unremarkable appearance of the visualized osseous structures.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Stable postoperative changes left breast from prior lumpectomy. No residual or recurrent lesions are identified.
3. Predominantly fatty replaced breasts.
BIRADS 2 - Benign findings. Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- FINDINGS:
Bilateral digital screening mammogram was obtained. The breast tissue is heterogeneously dense, which may obscure small masses.
There are scattered areas of fibroglandular density bilaterally. No dominant mass, architectural distortion or suspicious calcifications are identified in the right breast.
In the left breast, there are scattered fibroglandular densities. No dominant mass or architectural distortion. However, in the upper outer quadrant of the left breast, there is a small cluster of amorphous microcalcifications, spanning approximately 6 mm. This cluster was not present on the prior mammogram from 2 years ago.
IMPRESSION:
1. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
2. Small cluster of amorphous microcalcifications in the left upper outer breast, spanning 6 mm. These were not present on prior mammogram. Additional workup with magnification views recommended for further assessment. BI-RADS 0, incomplete. Recall for additional imaging. 
3. No dominant mass, architectural distortion or suspicious calcifications in the right breast. BI-RADS 2, benign.
4. Recommend routine annual screening mammography in 1 year, unless otherwise indicated based on left diagnostic workup.

ADDENDUM: Magnification views of the left breast demonstrate a cluster of amorphous microcalcifications in the 2:00 position, 6 cm from the nipple, spanning 7 mm. Stereotactic biopsy is recommended for definitive characterization. BI-RADS 4B, moderate suspicion for malignancy. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM:
INDICATION: Annual screening mammography for a 58-year-old woman with heterogeneously dense breasts and no personal history of breast cancer.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses. 
There are scattered areas of fibroglandular density bilaterally. 
In the left breast at 9 o'clock 6 cm from the nipple there are new grouped amorphous microcalcifications in a segmental distribution spanning approximately 1 cm. This is a suspicious finding. No other dominant masses, architectural distortion or suspicious calcifications are identified in either breast.
Bilateral benign-appearing lymph nodes are noted in both axillae.
IMPRESSION:
New grouped amorphous microcalcifications in a segmental distribution at 9 o'clock 6 cm from the nipple of the left breast, suspicious for ductal carcinoma in situ (DCIS). Additional imaging evaluation with magnification views is recommended for further characterization. If these remain suspicious, stereotactic core biopsy should be considered.
BI-RADS 4B: Suspicious finding, biopsy should be considered.
The remainder of both breasts are BI-RADS 2: Benign findings. ➡ *Positive*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: 55 year old female, screening mammogram.
COMPARISON: Previous mammograms from 2021 and 2022.
FINDINGS: 
Right breast: The breast is heterogeneously dense (type C). There are no dominant masses, suspicious calcifications or areas of architectural distortion.  
Left breast: The breast is heterogeneously dense (type C). There are no dominant masses, suspicious calcifications or areas of architectural distortion.  
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography.
BIRADS Category 1 - Negative. ➡ *Negative*
- FINDINGS: The breast tissue is predominantly fatty. There are no dominant masses, focal asymmetries, or areas of architectural distortion. No suspicious calcifications are identified.

IMPRESSION: 
Negative bilateral screening mammogram. 
No mammographic evidence of malignancy.
Recommend routine screening in 1 year.

BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
The breasts are almost entirely fatty.

There are no dominant masses, architectural distortion or abnormal calcifications in either breast. 

A 4 mm cluster of punctate microcalcifications is noted in the central right breast, approximately 4 cm from the nipple. These calcifications are new. No associated mass or distortion is seen.

Stable bilateral benign appearing calcifications.

IMPRESSION:
BI-RADS 0: Incomplete. 
New cluster of microcalcifications in the right breast. Additional magnification views are recommended for further evaluation. 

ADDENDUM:
Magnification views of the right breast were performed. The previously described cluster of microcalcifications in the central right breast measures 6 mm. The calcifications are heterogeneous in morphology, ranging from punctate to amorphous. No associated mass or distortion is identified.

UPDATED IMPRESSION:
BI-RADS 4: Suspicious. 
6 mm cluster of heterogeneous microcalcifications in the central right breast, suspicious for malignancy. Stereotactic core biopsy is recommended. ➡ *Positive*
- MAMMOGRAPHY - SCREENING - BILATERAL. ➡ *Not Stated*
- OBSP ROUTINE SCREENING MAMMOGRAM
Clinical History: No significant risk factors for breast cancer. Annual screening recalls.
Bilateral Digital Screening Mammogram: The breasts are heterogeneously dense which may lower the sensitivity. There are no dominant masses, areas of architectural distortion, or suspicious microcalcifications. 
Impression:
BIRADS 2 - Benign findings. The breasts are heterogeneously dense. No radiographic findings suspicious for malignancy. 
Routine yearly screening mammogram is recommended.
Recommendation: No further imaging required at this time. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in a research study comparing full field digital mammography to other screening modalities. Informed consent was obtained prior to randomization.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: No.
There are no suspicious masses, architectural distortions, or skin changes. The breast parenchyma is heterogeneously dense but non-confluent. No previous films are available for comparison.
BIRADS-2. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM:

INDICATION: Routine screening.

TECHNIQUE:  Standard CC and MLO views of both breasts were obtained.

COMPARISON: Previous mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortions, or suspicious calcifications.

Stable bilateral benign-appearing lymph nodes are noted in the axillary regions.

IMPRESSION:
Normal screening mammogram. No mammographic evidence of malignancy.

RECOMMENDATION:  
Routine screening mammogram in 1 year as per OBSP guidelines.

BIRADS Category 1 - Negative. ➡ *Not Stated*
- MAMMOGRAPHY - SCREENING - BILATERAL. CLINICAL HISTORY: Patient is a 45 year old female presenting for her screening mammogram. TECHNIQUE: Craniocaudal and mediolateral oblique views of the breasts were performed using digital mammography. COMPARISON: Comparison is made to the mammogram from 11/15/2023. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- FINDINGS: The left breast is heterogeneously dense, which could obscure detection of small masses. The right breast shows fibroglandular tissue and scattered areas of fat. No dominant masses, significant architectural distortion, or suspicious calcifications are seen in either breast. Stable compared to prior mammograms.

Targeted ultrasound was performed on a palpable nodule at 10 o'clock in the right breast, 4 cm from the nipple. This corresponds to a 6 mm hypoechoic solid mass with indistinct margins at 10 o'clock, 4 cm from the nipple. No increased vascularity is seen.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend annual screening mammography.
2. 6 mm solid mass with indistinct margins at 10 o'clock in the right breast, 4 cm from the nipple, seen on ultrasound. This is of indeterminate significance. Recommend 6 month follow up diagnostic mammogram and targeted ultrasound of the right breast for interval change. 

BIRADS: Right Breast: 3
Left Breast: 2 

ADDENDUM: The patient returned for 6 month follow up diagnostic imaging of the right breast. Targeted ultrasound again demonstrates the 6 mm hypoechoic mass at 10 o'clock, unchanged in size and appearance compared to prior. No other lesions are seen.

BIRADS: Right Breast: 3
Left Breast: 2. Annual screening mammography is recommended. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No breast complaints. 
TECHNIQUE: FFDM views were obtained: Right CC/MLO and Left CC/MLO.
COMPARISON: Films from 11/15/2021 and 11/20/2020.
FINDINGS:
Right breast: The breast is almost entirely fatty. No dominant mass, architectural distortion, or suspicious calcifications.  
Left breast: The breast is almost entirely fatty. No dominant mass, architectural distortion, or suspicious calcifications.  
Stable post-surgical changes from prior left breast biopsy are noted in the upper outer quadrant. Surgical clips are identified. No other findings.
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. No evidence of malignancy. Specifically no suspicious calcifications.
3. Stable left breast post-surgical changes. Likely due to history of prior biopsy, as described above.
RECOMMENDATIONS:
Routine annual screening mammography is recommended. 
BIRADS CATEGORY 1 - Negative. ➡ *Negative*
- MAMMOGRAM LEFT BREAST.
History: Screening.
Comparison is made with the patient's prior examination from 05/17/2023. These were sent to us electronically.
Findings:
There are scattered fibroglandular densities. No architectural distortion, suspicious calcification, or definite masses are seen.
Impression:
Negative left mammogram
BI-RADS Category 1. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL INDICATION: Annual screening mammography for this 52 year old woman with no focal area of concern.
FINDINGS:
The breasts are heterogeneously dense (Density C). Overall the breasts are stable in appearance compared to the prior mammogram dated June 15, 2018.
Right Breast:
No dominant mass, architectural distortion or suspicious microcalcifications. A few clusters of punctate microcalcifications are noted.
Left Breast:
No dominant mass or architectural distortion. A cluster of heterogeneous calcifications is seen at the 9 o'clock position, middle depth, measuring about 1.5 cm in diameter. These are indeterminate and new since the prior mammogram.
IMPRESSION:
1. Heterogeneously dense breast tissue (Density C).
2. New cluster of indeterminate calcifications in the left breast at 9 o'clock position. Further evaluation with magnification views recommended. BI-RADS Category 0: Incomplete. Additional imaging evaluation needed.
ADDENDUM- DIAGNOSTIC LEFT BREAST MAMMOGRAM:
Magnification views of the area of calcifications in the 9:00 position of the left breast shows a 1.5 cm cluster of heterogeneous microcalcifications with some fine pleomorphic and fine linear branching forms, suspicious for ductal carcinoma in situ. Stereotactic biopsy is recommended.
IMPRESSION:
Suspicious microcalcifications left breast at 9:00, 6 cm from the nipple. BI-RADS Category 4B. Stereotactic biopsy recommended. ➡ *Positive*
- SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
Interval: Stable. No new findings.
Breasts are heterogeneously dense (50-75% fibroglandular). No significant masses, architectural distortion or suspicious calcifications.
Benign calcifications are noted in the left breast, likely representing vascular calcifications.
BIRADS-2. Routine screening recommended. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to prior mammogram from June 2019, no significant change.
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Benign calcifications are noted bilaterally, unchanged. 
No mammographic evidence of malignancy.
IMPRESSION:
BIRADS 2-BENIGN FINDING. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: Yes.
Breast Density < 75%: Yes.
Breast Density >= 75%: No.
No mammographic evidence of malignancy. The breasts are heterogeneously dense. There are no masses, architectural distortion or suspicious calcifications.
BIRADS 2- Benign findings. Routine screening recommended. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening mammogram.
Comparison is available dating from 12/2023.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Annual high risk screening
TECHNIQUE: Standard CC and MLO views of the breasts were obtained.
COMPARISON: None available
FINDINGS:
Right Breast:
The right breast is heterogeneously dense (C).
There is no significant asymmetry, architectural distortion, or dominant mass.
There are no significant calcifications.
Left Breast:
The left breast is heterogeneously dense (C).
There is no significant asymmetry, architectural distortion, or dominant mass.
Scattered amorphous microcalcifications are seen in the upper outer quadrant, arranged in a segmental distribution. No previous mammograms are available for comparison.
IMPRESSION:
RIGHT BREAST: BI-RADS 1 - Negative
LEFT BREAST:  BI-RADS 0 - Incomplete: Need additional imaging evaluation and/or prior mammograms for comparison. 
- New scattered segmental microcalcifications in the upper outer left breast. Additional magnification views recommended for further evaluation. 
MANAGEMENT: Recommend unilateral diagnostic left mammogram workup with magnification views.
ADDENDUM: Additional magnification views of the left breast demonstrate scattered fine pleomorphic microcalcifications in a segmental distribution in the upper outer quadrant. No associated mass or distortion.
FINAL ASSESSMENT: 
LEFT BREAST: BI-RADS 4B- Suspicious abnormality, biopsy should be considered. Stereotactic biopsy recommended for sampling of the microcalcifications. ➡ *Positive*
- BILATERAL MAMMOGRAM AND RIGHT BREAST ULTRASOUND 

INDICATION: Screening and assessment of new palpable mass at 11 o'clock position in the right breast.

COMPARISON: Prior studies from 2019. 

FINDINGS:

The breasts are heterogeneously dense which could obscure detection of small masses.

There are scattered areas of fibroglandular density bilaterally. In the right breast at the site of palpable concern at 11 o'clock, there is a 1 cm irregular mass with indistinct margins. This corresponds to the palpable finding. No other dominant masses, architectural distortion or suspicious calcifications are identified in the right breast.   

Ultrasound of the right breast at 11:00, 6 cm from the nipple demonstrates a solid, irregular 11 x 8 x 7 mm hypoechoic mass with angular margins and posterior acoustic shadowing. This was identified at the site of palpable concern. 

The left breast demonstrates no dominant masses, architectural distortion or suspicious calcifications. There is a small cluster of amorphous microcalcifications at the 3:00 position, middle depth.

IMPRESSION:

Right breast: Irregular mass with suspicious features at site of palpable abnormality at 11:00, 6 cm from the nipple. Biopsy recommended. BI-RADS 4C. 

Left breast: Cluster of amorphous microcalcifications at 3:00 position, middle depth. Additional magnification views recommended for further characterization. BI-RADS 0.

ADDENDUM: Spot compression magnification views of the left breast demonstrate a 6 mm cluster of amorphous microcalcifications at 3:00, middle depth. Stereotactic biopsy recommended. BI-RADS 4B. ➡ *Positive*
- SCREENING MAMMOGRAM 
COMPARISON: None available.
BRIEF HISTORY: Patient presents for baseline screening mammogram. No previous mammograms available for comparison.
TECHNIQUE: Standard CC and MLO views obtained bilaterally. 
FINDINGS:
The breasts are composed of scattered fibroglandular densities. 
The breasts are symmetrical in size and density. There are no dominant masses, suspicious calcifications, or areas of architectural distortion. 
The skin is smooth and the nipples are within normal limits. No skin thickening or retraction is seen. The axillae are unremarkable.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Breast Imaging Reporting and Data System (BI-RADS) Category 1: Negative.
RECOMMENDATION:  
Routine screening mammography in 1 year is recommended. ➡ *Not Stated*
- MAMMOGRAM.
INDICATION: Postoperative screening. Left new lump.

COMPARISON: 2025-06-15.

RIGHT BREAST: 
Right mastectomy.  

LEFT BREAST: 
There are scattered areas of fibroglandular density.
There is no focal mass or suspicious calcifications.  No new significant changes compared to prior exams.

IMPRESSION:
1. Negative screening mammogram of the left breast. No suspicious masses or calcifications.  No significant interval change.
2. Status post right mastectomy.
BIRADS 1.

ULTRASOUND LEFT BREAST

CLINICAL HISTORY: New left breast lump.

TECHNIQUE: Targeted ultrasound was performed using a 12 MHz linear transducer.  

COMPARISON: None.

FINDINGS: 
Targeted ultrasound of the left breast at 12:00, 6 cm from the nipple, demonstrates normal fibroglandular tissue with no focal solid lesion, complex cystic lesion or suspicious mass identified.

IMPRESSION: 
No sonographic evidence of malignancy at the site of the palpable abnormality. Recommend clinical follow up and correlation with physical exam findings. ➡ *Negative*
- MAMMOGRAPHY REPORT 

Facility: Maple Grove Radiology 
Patient Name: Jane Doe
DOB: 01/01/1970
MRN: 1234567
Date of Exam: 04/15/2023

INDICATION: Screening.

BREAST COMPOSITION: The breasts are heterogeneously dense, which may obscure small masses.

FINDINGS:
There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
There are no mammographic findings suspicious for malignancy. Routine screening is recommended. 

BI-RADS Category 1: Negative. ➡ *Not Stated*
- REPORT:
"RIGHT MAMMOGRAM.
HISTORY: Annual screening mammogram. Patient has history of left breast cancer treated with lumpectomy and radiation therapy in 2020.
FINDINGS: Left mastectomy. Right breast tissue is heterogeneously dense, which may obscure detection of small masses. There is no mammographic evidence of malignancy. There are no suspicious masses, areas of architectural distortion, or suspicious calcifications. Stable benign calcifications are noted in the upper outer quadrant. No significant change compared to prior mammograms.
IMPRESSION: 
1. Negative digital screening mammogram of the right breast. 
2. Left mastectomy.
3. Recommend annual screening mammography of the right breast.
BIRADS 1 - Negative." ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Routine screening. No current breast complaints.
TECHNIQUE: Bilateral digital mammogram. Standard CC and MLO views obtained.
COMPARISON: Compared to prior studies from 10/15/2019. 
FINDINGS:
The breasts are heterogeneously dense (BI-RADS C).  
Right breast: There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
Left breast:  There are scattered fibroglandular densities. No dominant mass or area of architectural distortion. At the 12:00 position, 5 cm from the nipple, there is a new cluster of fine pleomorphic microcalcifications, spanning approximately 6 mm.  
There are no abnormal axillary lymph nodes.
Skin is within normal limits.  
IMPRESSION:
BI-RADS 0: Incomplete. New cluster of suspicious microcalcifications in the left breast which requires further diagnostic imaging evaluation with magnification views. Please return to the Breast Center for additional imaging and possible biopsy.
ADDENDUM (POST DIAGNOSTIC MAMMOGRAM): 
Additional magnified views of the left breast were performed.  
The previously described cluster of pleomorphic microcalcifications at 12:00 position of the left breast, 5 cm from the nipple, persists and appears more concerning on the magnified views. The morphology and distribution are highly suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended. 
FINAL ASSESSMENT:  BI-RADS 4C: Highly suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- BILATERAL MAMMOGRAM
INDICATION: Annual routine screening mammogram.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior mammograms from 4/12/2022.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast. Specifically, no microcalcifications are identified.
There are no significant changes compared to prior mammograms. 
IMPRESSION: 
BI-RADS 1 - Negative bilateral screening mammogram. No evidence of malignancy. 
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- DIAGNOSTIC MAMMOGRAPHY LEFT BREAST 
CLINICAL HISTORY: 42 year old female presents with new palpable mass left breast.   
FINDINGS:   
The breasts are heterogeneously dense.  
At 12:00 in the left breast, 4 cm from the nipple there is an irregular mass with spiculated margins measuring 15 x 12 mm. 
Also at 12 :00, more posteriorly, there are several adjacent masses with in distinct margins together measuring 26 x 21 mm.  
There are pleomorphic calcifications extending anteriorly from the 12:00 mass towards the nipple.  
No suspicious calcifications are seen in the remainder of the left breast.  
IMPRESSION:  
Highly suspicious mass left breast 12:00, 4 cm from the nipple with adjacent pleomorphic calcifications and satellite masses, suspicious for multifocal malignancy.
BIRADS 5 ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
History: Screening mammogram
Comparison available dating from 03/2023.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL MAMMOGRAM. CLINICAL HISTORY: Right breast pain. FINDINGS: The breasts are almost entirely composed of dense fibroglandular tissue. There are no well-defined masses, architectural distortion, or suspicious calcifications. IMPRESSION: 1. No mammographic evidence of malignancy. 2. Extremely dense breast tissue, which can obscure small masses. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Annual mammographic screening exam.

TECHNIQUE: Standard CC and MLO views obtained bilaterally.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure small masses.

There are grouped amorphous calcifications in a segmental distribution in the upper outer quadrant of the right breast at approximately the 10:00 position, 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No associated masses, architectural distortions or skin thickening.

No significant masses, calcifications, architectural distortions or skin thickening in the left breast.

IMPRESSION:
New cluster of amorphous calcifications upper outer right breast, suspicious for malignancy. Additional magnification views recommended for further evaluation.

BIRADS 4: Suspicious abnormality in the right breast. Biopsy should be considered.

ADDENDUM:
The patient returned for diagnostic mammogram and ultrasound. Magnification views of the right breast at the 10:00 position confirm a 9 mm cluster of amorphous calcifications. Ultrasound of this area demonstrates a correlating 7 mm irregular hypoechoic mass with indistinct margins and internal vascularity.

BIRADS 4C: Highly suggestive of malignancy. Biopsy recommended. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM. History: Recent right breast lumpectomy with positive margins. Outside films reviewed. Comparison: With prior. Findings: Right breast status post lumpectomy with surgical changes. There are clips within the lumpectomy site. There are scattered benign-appearing calcifications in both breasts. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 2. ➡ *Negative*
- REPORT:
"BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening mammogram for a 67-year-old female with family history of breast cancer. No complaints of breast pain or palpable abnormalities.

FINDINGS: Bilateral mammogram images including CC and MLO views were obtained and compared to previous studies from 2 years ago. 

The breasts are heterogeneously dense (C), which may obscure detection of small masses.

In the right breast, there are no dominant masses, areas of abnormal architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications are noted in the upper outer right breast.

In the left breast, there are scattered fibroglandular densities. No dominant masses or areas of architectural distortion. However, a new cluster of pleomorphic microcalcifications is noted in the upper inner quadrant of the left breast at the 10 o'clock position, 6 cm from the nipple. This cluster measures approximately 8 mm and was not present on the prior study. No other suspicious calcifications are identified. 

The breast tissue extends to the skin margins and no significant axillary lymphadenopathy is appreciated.

IMPRESSION: 
1. New cluster of suspicious microcalcifications in the left breast at 10:00, 6 cm from the nipple, highly suggestive of ductal carcinoma in situ (DCIS). Additional evaluation with spot compression and magnification views recommended for further characterization. Stereotactic core biopsy is indicated if the finding persists.
2. Stable, benign calcifications right breast. 
3. Heterogeneously dense breasts which may lower the sensitivity of mammography.

BI-RADS Category: 4 - Suspicious abnormality in the left breast, biopsy should be considered. 
BI-RADS Category: 2 - Benign findings in the right breast.

RECOMMENDATION: Additional diagnostic imaging of the left breast for stereotactic core biopsy of suspicious microcalcifications. Routine annual screening of the right breast with mammography in 1 year.
" ➡ *Positive*
- MAMMOGRAM REPORT - BILATERAL

INDICATION: Annual screening mammogram. No current breast complaints.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Comparison is made to prior mammogram from 1 year ago.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.

LEFT BREAST: 
There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
Bilateral mammogram, BI-RADS Category 1 - Negative. 
No evidence of malignancy. 
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAM REPORT:
CLINICAL HISTORY: 40-year-old female presents for screening mammogram. No prior mammograms available for comparison. No current breast complaints.
FINDINGS: 
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right Breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.  
Left Breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.  
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breast tissue which can obscure small masses. 
BIRADS 1 - Negative. Recommend continued annual screening in 1 year with comparison to current exam. ➡ *Negative*
- MAMMOGRAPHY - BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammograms from 1/15/2022.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Bilateral axillae are unremarkable.

IMPRESSION:
BI-RADS Category 1: Negative bilaterally. No evidence of malignancy.

RECOMMENDATIONS:
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAPHY - SCREENING. History: 48 year-old female, no breast complaints, presents for her annual screening mammogram. Comparison is made with previous mammograms from 03-20-2023. Findings: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: No evidence of malignancy. BI-RADS 1. ➡ *Negative*
- REPORT:
"MAMMOGRAPHIC VIEWS BILATERAL BREASTS AND ADDITIONAL VIEWS RIGHT BREAST.  
History: Recalled following screening mammogram for abnormality seen right breast. No palpable lumps.
FINDINGS: The breasts are almost entirely fatty.
There is a focal asymmetry noted in the upper inner quadrant of the right breast and this corresponds to a 15 mm solid nodule on ultrasound. Ultrasound also shows associated clustered microcalcifications. The left breast is unremarkable. BI-RADS 4B right and BI-RADS 1 left.
Impression: Suspicious finding right breast with associated microcalcifications. Ultrasound guided core biopsy recommended."
LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.
At the 10:00 position of the right breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications in a segmental distribution measuring approximately 8 mm. Additional magnification views confirm the presence of these microcalcifications.
The remaining fibroglandular tissue of both breasts is unremarkable.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic biopsy is recommended for further evaluation of the group of amorphous microcalcifications at the 10:00 position of the right breast.
BI-RADS Category 2 for the left breast: Benign findings. Recommend routine screening in 1 year. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY

INDICATION: Annual screening mammogram.

COMPARISON: Prior study of 05/15/2021.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are scattered areas of fibroglandular density bilaterally.

No dominant masses, areas of architectural distortion, or suspicious calcifications are seen in the right breast.

In the left breast, there is a new cluster of pleomorphic microcalcifications at the 10 o'clock position, 6 cm from the nipple, spanning an area of approximately 8 mm. This cluster was not present on the prior study.

No skin thickening, nipple retraction, or lymphadenopathy is noted.

IMPRESSION:
1. New cluster of pleomorphic microcalcifications in the left breast at 10 o'clock, 6 cm from the nipple, suspicious for malignancy. Additional diagnostic imaging is recommended for further evaluation. BI-RADS 4B.

2. Stable, benign findings in the right breast. BI-RADS 2.

RECOMMENDATION:
Diagnostic mammography with magnification views and ultrasound of the left breast for further assessment of the new cluster of microcalcifications. ➡ *Positive*
- ULTRASOUND LEFT BREAST

INDICATION: Palpable lump left breast upper outer quadrant.

TECHNIQUE: Targeted ultrasound of the left breast was performed using a high frequency linear transducer.

COMPARISON: None available.

FINDINGS:
The left breast tissue is heterogeneously echogenic, consistent with scattered fibroglandular tissue.

Targeted ultrasound of the left upper outer quadrant demonstrates normal fibroglandular tissue. No discrete solid or cystic mass, architectural distortion, or suspicious calcifications are identified in this region.

The remainder of the left breast is scanned and there are no sonographic findings to suggest malignancy. No axillary, infraclavicular, or supraclavicular adenopathy.

IMPRESSION:
No sonographic evidence of malignancy in the left breast. Specifically, no abnormality is seen in the area of palpable concern in the left upper outer quadrant. 

BIRADS 1 - Negative.

RECOMMENDATION: Continue routine screening mammography as per established guidelines. Clinical follow up of palpable finding as needed. ➡ *Negative*
- SCREENING MAMMOGRAM

INDICATION:  Screening.  
Breast density: Scattered fibroglandular densities.
Comparison: Compared to prior exam dated 11/02/2020.

FINDINGS:
There are scattered fibroglandular densities. No dominant masses, suspicious calcifications or architectural distortion.  

IMPRESSION: 
BIRADS 2 - Benign findings. Recommend routine screening in 1 year.

ADDENDUM:
Final BIRADS Assessment Category: BIRADS 2
Indication for supplemental screening: None specified.
Management: Routine screening as per OBSP high risk screening program. 

No further action required at this time. ➡ *Not Stated*
- MAMMOGRAM LEFT BREAST WITH SPOT MAGNIFICATION.
HISTORY: Left breast pain, superior.
COMPARISONS: Prior from [redacted] 07/10/2023.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Spot magnification views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, or suspicious calcifications are seen. No skin thickening or nipple retraction is seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- MAMMOGRAM - BILATERAL WITH CAD

CLINICAL HISTORY: 53-year-old female for screening mammogram.

COMPARISON: 09/10/2023

FINDINGS: The breasts are heterogeneously dense. Comparison is made to prior examination. Again seen are scattered benign-appearing calcifications in both breasts. There is a new focus of highly suspicious, grouped, fine, pleomorphic calcifications in the right breast at approximately the 3 o'clock position. This measures approximately 1 cm in greatest diameter and is located approximately 2 cm from the nipple.

IMPRESSION:

1. New focus of highly suspicious, grouped, fine, pleomorphic calcifications in the right breast. Given the patient's age, this is highly suspicious for malignancy. Correlation with spot magnification views and possible biopsy is recommended.

BI-RADS: 4C (Highly suspicious for malignancy). ➡ *Positive*
- FINDINGS:

Bilateral diagnostic mammogram was obtained.

The breast tissue is heterogeneously dense which may obscure small masses.

There are scattered areas of fibroglandular density. 

At the 12:00 position of the left breast there is a small group of amorphous microcalcifications measuring approximately 4 mm, located about 4 cm from the nipple.

No dominant mass, architectural distortion or skin thickening.

Benign calcifications are noted in both breasts. 

IMPRESSION:

BIRADS 4: Suspicious abnormality in the left breast. Small group of amorphous microcalcifications in the left breast at 12:00, 4 cm from the nipple. This is a suspicious finding and warrants biopsy. Recommend stereotactic biopsy for tissue diagnosis. ➡ *Positive*
- TMIST FFDM SCREENING MAMMOGRAM OBSP ARM.
This patient was enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) after providing informed consent, and was randomized to the full field digital mammography arm, per Research Ethics Board approved protocol. 
Comparison to previous exam: Yes.
Mammographic breast density: Scattered fibroglandular density (<=75%).
Next OBSP recall: Routine screening in 1 year not indicated based on current findings.
The mammogram shows no features suspicious for malignancy. There is no mention of microcalcifications. 
Final BIRADS Assessment Category: 1 (Negative) or 2 (Benign). ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening

FINDINGS:

There are scattered fibroglandular densities.

No suspicious masses, architectural distortion or definite abnormal calcifications are identified in either breast.

In the left breast in the upper outer quadrant there are several groups of faint punctate microcalcifications, the largest group measuring approximately 5 mm. These were not seen in the prior study.  Additional magnification views confirm these are real. They are indeterminate but suspicious and further evaluation is recommended.

In the right breast there are no dominant masses, architectural distortions or suspicious microcalcifications.  Specifically no microcalcifications corresponding to those seen on the left are identified.

IMPRESSION:
BIRADS 4: Suspicious grouped microcalcifications in the upper outer left breast, not present on the prior mammogram. Stereotactic core biopsy is recommended. The remainder of the left breast is BIRADS 2.
Right breast: BIRADS 2, benign. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are almost entirely fatty.
There are no suspicious masses, architectural distortions, or dominant masses in either breast.

In the left breast at the 9:00 position, 5 cm from the nipple, there is a group of amorphous microcalcifications in a segmental distribution spanning an area of approximately 1 cm. Additional spot compression magnification views confirm these calcifications.

No other groups of suspicious microcalcifications are identified in either breast.

IMPRESSION:
LEFT BREAST: BI-RADS 4B: Suspicious microcalcifications in a segmental distribution at 9:00, 5 cm from the nipple. Stereotactic core biopsy is recommended for further evaluation.

RIGHT BREAST: BI-RADS 1: Negative study. Annual screening mammography is recommended.

ADDENDUM:
The patient returned for left breast stereotactic core biopsy of the group of suspicious microcalcifications at the 9:00 position, 5 cm from the nipple. Specimen radiograph confirmed the presence of microcalcifications within the core samples. The biopsy was well tolerated without immediate complications. Final pathology results are pending and will be reported in an addendum. ➡ *Positive*
- ULTRASOUND BREAST AND MAMMOGRAM, RIGHT.

HISTORY: New patient with right breast mass at 11:00, first noticed 1 month ago. No prior imaging available for comparison. Maternal grandmother with history of breast cancer.

FINDINGS:
Ultrasound Breast:
Targeted right breast ultrasound demonstrates an irregular solid mass at 11:00, 4 cm from the nipple, measuring 1.8 x 1.3 x 1.2 cm. The mass has indistinct margins, is taller than wide, and demonstrates internal vascularity. No axillary lymphadenopathy.

Mammogram:
Right CC and MLO views demonstrate extremely dense breast tissue. Correlating with ultrasound, there is a high density irregular mass at 11:00, posterior depth. No suspicious calcifications, architectural distortion or skin thickening.

IMPRESSION:
1. Right breast irregular solid mass at 11:00, with suspicious features on ultrasound and mammogram, BI-RADS 4. Ultrasound guided core biopsy recommended.
2. Extremely dense breasts, which may obscure small masses. Recommend annual screening with mammogram and ultrasound.

BIRADS 4.

Addendum: Patient proceeded to ultrasound guided core biopsy of the right 11:00 mass at the same visit. Pathology pending. ➡ *Negative*
- MAMMOGRAM.
INDICATION: Family history of breast cancer.
FINDINGS: Scattered fibroglandular densities. No significant masses, calcifications or architectural distortion.
IMPRESSION: Benign/negative mammogram. Recommend routine screening in 1 year. BI-RADS 1. ➡ *Negative*
- MAMMOGRAPHIC REPORT
HISTORY: 48-year-old female presents for screening mammography. No complaints. No palpable masses.  No prior history of breast cancer. No family history of breast cancer.
TECHNIQUE: Bilateral mammographic images were obtained including CC and MLO views.
COMPARISON: Comparison is made to prior exams dating back to 2021, which are available for review.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right Breast:
There are no suspicious masses, architectural distortion, or suspicious calcifications.  
Left Breast:
There are no suspicious masses, architectural distortion, or suspicious calcifications.
Stable benign-appearing calcifications are present in the upper outer quadrant, likely representing fibrocystic change. These are unchanged compared to prior exams.
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Benign-appearing stable calcifications in the left breast. These do not require further evaluation.
3. Recommend continued annual screening mammography in 1 year, given heterogeneously dense breast tissue.
BIRADS 2 - Benign ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM WITH RIGHT BREAST DIAGNOSTIC VIEWS:

INDICATION: 55-year-old female, here for routine annual screening mammogram and history of microcalcifications right breast.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are bilateral benign-appearing calcifications. 

In the right breast at the 12:00 position 7 cm from the nipple, there is a new group of fine pleomorphic microcalcifications spanning an area of about 8 mm. Spot compression magnification views confirm these are suspicious and new compared to prior mammograms. No other suspicious masses, calcifications, or areas of architectural distortion.

No suspicious findings in the left breast. Stable bilateral vascular calcifications.

IMPRESSION: 
New group of suspicious microcalcifications right breast at 12:00, 7 cm from the nipple, spanning 8 mm. Recommend stereotactic core biopsy. BI-RADS 4B.

Heterogeneously dense breasts, left breast stable and benign. BI-RADS 2.

ADDENDUM:
The patient proceeded to right breast 12:00 stereotactic core biopsy on the same day. Post-biopsy mammogram confirms accurate clip placement at the site of biopsy. Pathology pending, see separate biopsy report. ➡ *Positive*
- BIRADS-2. BILATERAL SCREENING MAMMOGRAM.
INDICATION: Screening.
TECHNIQUE: Bilateral CC, MLO and LMO views obtained. For BIRADS density C or D, screening ultrasound recommended to complete exam.
COMPARISON: None available.
FINDINGS:
Right breast: No significant masses, suspicious calcifications or architectural distortion. 
Left breast: No significant masses, suspicious calcifications or architectural distortion.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense parenchyma, which could obscure detection of small masses. For BIRADS density C or D, ultrasound is recommended to complete screening exam.
RECOMMENDATION: 
Routine annual screening mammography is recommended. Screening ultrasound is recommended to complete this exam given breast density. ➡ *Not Stated*
- SCREENING MAMMOGRAM

INDICATION: Screening. 

COMPARISON: None.

TECHNIQUE: Bilateral digital screening mammogram with tomosynthesis. 

FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 
No dominant mass, architectural distortion, or suspicious calcifications.
There are no suspicious microcalcifications. 

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Mammogram is not sensitive due to increased breast density. Consider supplemental screening such as whole breast ultrasound or breast MRI, which can increase cancer detection in women with dense breasts.

BIRADS Category 1: Negative.
Recommend routine screening in 1 year. ➡ *Not Stated*
- MAMMOGRAPHY - SCREENING. History: 53-year-old female, no breast complaints, presents for her annual screening mammogram. Comparison is made with previous mammograms from 11-15-2023. Findings: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: No evidence of malignancy. BI-RADS 1. ➡ *Negative*
- FINDINGS: Bilateral breasts are examined. The breast tissue is heterogeneously dense, which could obscure detection of small masses. There are no suspicious masses, architectural distortion or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend routine screening in 1 year.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS.
INDICATION: Screening study. No reported clinical concerns.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in either breast.
In the right breast at 10:00, 6 cm from the nipple, there is a group of amorphous calcifications in a segmental distribution measuring 8 mm. This is new compared to the patient's previous mammograms.
The left breast is unremarkable.
IMPRESSION:
BI-RADS 0: Incomplete. Additional imaging evaluation with right breast spot compression magnification views is required to assess the new group of amorphous calcifications in the right breast at 10:00.
ADDENDUM - RIGHT BREAST SPOT COMPRESSION MAGNIFICATION VIEWS:
Spot compression magnification views of the right breast confirms a 10 mm group of amorphous calcifications at 10:00, 6 cm from the nipple, in a segmental distribution. The calcifications are new compared to the patient's previous mammograms. 
FINAL ASSESSMENT: 
RIGHT BREAST, BI-RADS 4B: Suspicious abnormality, biopsy should be considered. Suspicious group of new amorphous calcifications at 10:00.
LEFT BREAST, BI-RADS 1: Negative. ➡ *Positive*
- MAMMOGRAPHY REPORT

INDICATION: 64-year-old woman with palpable abnormality in the left breast.

COMPARISON: Previous mammograms from 2 years ago.

FINDINGS:
There are scattered areas of fibroglandular density. No significant interval change. 

In the left breast at the 2:00 position 6 cm from the nipple there is a 1 cm irregular mass. Targeted ultrasound shows the mass to be solid and hypoechoic with angular margins and suspicious for malignancy.

Additionally, in the left breast at the 10:00 position 3 cm from the nipple, there is a new group of pleomorphic microcalcifications spanning an area of about 5 mm. This was not present on the prior examination. 

The remainder of the left breast is unremarkable. The right breast is within normal limits.

IMPRESSION:
1. Suspicious mass left breast 2:00 position, biopsy recommended. BI-RADS 4.
2. New group of pleomorphic microcalcifications left breast 10:00 position, suspicious for malignancy. Stereotactic biopsy recommended. BI-RADS 4.

ADDENDUM: Ultrasound-guided core biopsy of the 2:00 mass and stereotactic biopsy of the 10:00 calcifications were both performed. Pathology results showed invasive ductal carcinoma from the 2:00 mass and ductal carcinoma in situ from the 10:00 calcifications. The patient will proceed with surgical management. ➡ *Positive*
- IMPRESSION:
1. Bilateral diagnostic mammogram and magnification views were performed. No previous mammograms are available for comparison.
2. In the right breast there is a new cluster of pleomorphic calcifications at 10 o'clock posterior depth and 6 cm from the nipple. This is highly suspicious for malignancy and requires biopsy. 
3. Ultrasound of the right breast shows a corresponding irregular solid mass with angular margins and posterior acoustic shadowing at 10 o'clock posterior depth measuring 11 mm. 
4. No other suspicious mammographic or sonographic findings in either breast.
5. BI-RADS 4C right breast. BI-RADS 1 left breast.

FINDINGS: The breasts are heterogeneously dense limiting mammographic sensitivity. The palpable area of concern in the right breast corresponds to a new cluster of pleomorphic calcifications on mammogram at 10:00 posterior depth 6 cm from the nipple. Spot compression magnification views confirm the presence of pleomorphic calcifications in a segmental distribution highly suspicious for malignancy. 

Targeted ultrasound of the right breast at 10 o'clock shows an irregular solid mass with angular margins and posterior acoustic shadowing corresponding to the mammographic abnormality. The mass measures 11 x 8 x 7 mm. No axillary lymphadenopathy or skin thickening. 

No suspicious mammographic or sonographic findings are seen in the left breast. Benign appearing lymph nodes are present in the left axilla, none with cortical thickening greater than 3 mm. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

TECHNIQUE: Bilateral digital screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are almost entirely fatty.
No suspicious mass, architectural distortion or focal asymmetry in either breast.

Left breast:
At the 10:00 position in the left breast, there is a small group of amorphous calcifications measuring 4 mm, located 5 cm from the nipple. This is a new finding not previously noted.

Right breast:
No significant findings. No suspicious calcifications.

IMPRESSION:
BI-RADS Category 0: Incomplete, additional imaging needed to assess indeterminate calcifications in the left breast.

RECOMMENDATION:
Magnification views of the left breast are recommended to further evaluate the new group of amorphous calcifications at 10:00.

ADDENDUM:
Magnification views of the left breast were performed. The amorphous calcifications at 10:00, 5 cm from nipple, measure 4 mm. Biopsy is recommended.

FINAL IMPRESSION:
BI-RADS Category 4: Suspicious finding in the left breast. Biopsy recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM. Clinical history: 61 year old female presents for her annual screening mammogram. Comparison is made to prior mammogram from 10/12/2023. Findings: The breasts are heterogeneously dense. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: Negative mammogram. BI-RADS 1. ➡ *Negative*
- FINDINGS:
Mammogram demonstrates scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast. 

In the left breast, there is a new 5 mm cluster of fine pleomorphic calcifications at the 12:00 position, 6 cm from the nipple. No other suspicious calcifications are identified. There are no dominant masses or areas of architectural distortion. 

Automated breast ultrasound was performed of the left breast in the area of the new calcifications. No correlating mass or dilated ducts are identified. 

IMPRESSION:
1. New cluster of suspicious calcifications in the left breast at 12:00, 6 cm from the nipple. Stereotactic biopsy is recommended. BI-RADS 4B. 
2. Remainder of both breasts are negative. BI-RADS 1. ➡ *Positive*
- FINDINGS: Left breast tissue is predominantly fatty-replaced. There are no suspicious masses, architectural distortion, or malignant-type calcifications. No significant change from prior exams.

Targeted ultrasound of the palpable abnormality in the left breast 6 o'clock position demonstrates normal, probably fibrous tissue, without a discrete mass, complex cyst, or other sonographically suspicious abnormality. 

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Fatty-replaced breast tissue. Recommend annual screening mammography.  

2. Targeted left breast ultrasound is negative. 6 o'clock palpable abnormality likely represents normal fibrous tissue. Clinical followup as needed.

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.
INDICATION: Annual screening. No current breast complaints. Past history of right breast biopsy in 2019 with benign pathology.
TECHNIQUE: Standard CC and MLO views of each breast. 
COMPARISON: 11/15/2021.
FINDINGS:
RIGHT BREAST: The breast is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications.  Stable partially imaged benign appearing calcification upper outer quadrant. Post-biopsy changes visualized upper outer quadrant from prior benign right breast biopsy. 
LEFT BREAST: The breast is heterogeneously dense. No dominant mass, architectural distortion or suspicious calcifications.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Stable benign right breast findings. 
3. Recommend continued annual screening mammography.
BIRADS 1. Annual screening mammography recommended.
ADDENDUM: Informed patient of results and follow-up recommendations. Patient voiced understanding. ➡ *Negative*
- FINDINGS:
Bilateral digital diagnostic mammography was performed, including CC and MLO views of both breasts. Additionally, spot compression magnification views were obtained of the left breast.

The breast tissue is heterogeneously dense, which may lower the sensitivity of mammography.

There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a focal asymmetry in the upper outer quadrant on the MLO view, which persists on spot compression magnification views. Associated with this asymmetry, there are fine pleomorphic calcifications in a segmental distribution, extending over an area of approximately 2 cm. No associated mass or architectural distortion is identified.

The remainder of the left breast is unremarkable.

IMPRESSION:
1. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography.
2. Focal asymmetry with segmental fine pleomorphic calcifications in the left upper outer breast, suspicious for ductal carcinoma in situ (DCIS). Additional evaluation with ultrasound is recommended for further characterization and to guide biopsy. BI-RADS 4C.
3. No suspicious findings in the right breast. BI-RADS 1.

MANAGEMENT:
Biopsy recommended for 4C finding in the left breast. Follow-up recommended per BI-RADS category on the right. 

ADDENDUM - ULTRASOUND LEFT BREAST:
Targeted ultrasound was performed on the area of focal asymmetry and calcifications in the left upper outer quadrant. A correlating area of irregular ductal ectasia was identified, confirming the mammographic finding. Ultrasound-guided core biopsy was performed with pathology pending. ➡ *Positive*
- TMIST FFDM ARM OBSP MAMMOGRAM. 
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes. 
Breast Density >= 75%: No.
Breast Density < 75%: Yes. 
OBSP to recall client in 1 year: Yes.
No dominant masses, architectural distortions or suspicious calcifications are identified.
BIRADS-2. Annual screening is recommended.

Addendum: No additional views are required based on the normal screening mammogram findings. The patient's risk factors have been taken into consideration. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to previous exam: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.

Bilateral breasts are heterogeneously dense.  No dominant masses, architectural distortion or suspicious calcifications.  

Benign-appearing calcifications are present in the right breast, too small to characterize.  These are of no clinical significance.  

No mammographic findings suspicious for malignancy.  Routine screening mammogram is recommended in 1 year.

BI-RADS Category 2: Benign Finding. ➡ *Not Stated*
- BREAST ULTRASOUND

CLINICAL HISTORY: Palpable lump, right breast. The patient states that she can feel a lump at the 2 o'clock position.

COMPARISON: None.

FINDINGS: Within the right breast at the 2 o'clock position, there is a palpable 1 cm irregular, hypoechoic mass. This mass is located within the area of the patient's palpable abnormality. The mass demonstrates angular margins, posterior acoustic shadowing, and increased vascularity on color Doppler imaging. In addition, a small cluster of microcalcifications is identified adjacent to the mass. These calcifications correlate with findings seen on recent mammography.

IMPRESSION:

1. Right breast mass: Imaging features are suggestive of malignancy. The adjacent presence of microcalcifications raises further concern for malignancy. Biopsy is recommended.

BI-RADS: 5 (Highly Suggestive of Malignancy). ➡ *Positive*
- FINDINGS: The breasts are heterogeneously dense. There are no dominant masses, architectural distortions, or suspicious calcifications.

There are scattered areas of fibroglandular density bilaterally. No mammographic evidence of malignancy.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend continued annual screening mammography and clinical breast exams. 
3. Breast density is heterogeneously dense, which may lower the sensitivity of mammography. Inform the patient that dense breast tissue is a common finding that is noted in about 40 percent of mammograms and is a normal variant. While dense breast tissue may make it more difficult to detect breast cancer on a mammogram, it does not increase the risk for developing breast cancer. Consider supplemental screening with breast ultrasound or MRI if clinically indicated based on additional risk factors.

BIRADS 1 - NEGATIVE.

ADDENDUM: Spoke with ordering physician Dr. Smith who agrees with recommendations. Patient also notified of results. ➡ *Negative*
- ROUTINE TMIST SCREENING MAMMOGRAM - DIGITAL ONLY ARM.
Consent obtained for participation in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), a Research Ethics Board approved study. Randomized to standard 2D digital mammography.
Comparison to prior exam: Available.  
Breast tissue density: Scattered fibroglandular densities. 
Breast density below 75 percent.
1 year routine OBSP screening interval recommended.
No mammographic abnormalities suspicious for malignancy identified. 
Final BIRADS Assessment Category: 2 - Benign. ➡ *Not Stated*
- TMIST FFDM MAMMOGRAM REPORT 
The patient is a part of the TMIST research study and was randomized to the full field digital mammography arm. Informed consent was obtained prior to the study.
Comparison to prior mammograms was performed.  
The breast density is less than 75%.
No suspicious mammographic findings are identified. The breasts are heterogeneously dense, which may obscure small masses.
BIRADS Category 2 - Benign findings. Recommend routine screening in 1 year per OBSP guidelines. ➡ *Not Stated*
- FINDINGS: Bilateral breasts are composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious microcalcifications.

IMPRESSION:
1. No mammographic evidence of malignancy.
2. Scattered fibroglandular densities. 

BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM.
Compared to prior mammogram from July 15, 2020, no suspicious change. 
The breast tissue is heterogeneously dense, which may obscure small masses.
There are no dominant masses, suspicious calcifications or focal architectural distortion bilaterally.
IMPRESSION:
BIRADS 2- Benign findings. Recommend routine screening in 1 year. 
This examination was reviewed by the radiologist using high resolution monitors for interpretation. ➡ *Not Stated*
- FINDINGS: The right breast is heterogeneously dense. No suspicious right breast masses, calcifications or other findings.

The left breast is heterogeneously dense. There are multiple regions of architectural distortion in the anterior depth left breast at the 12:00 position 5 cm from the nipple, 1:00 position 4 cm from the nipple and 2:00 position 3 cm from the nipple. Associated with these are small groups of suspicious pleomorphic microcalcifications. In the posterior depth at 6:00 10 cm from the nipple there is an irregular hypoechoic mass with indistinct margins measuring 18 x 12 mm with internal vascularity.

Ultrasound of the left axilla demonstrates 3 morphologically abnormal lymph nodes, the largest measuring 20 mm in maximum dimension with loss of the fatty hilum.

IMPRESSION:
BI-RADS 5: Highly suggestive of malignancy. 
- Multiple areas of architectural distortion with associated suspicious microcalcifications left breast, biopsy recommended. 
- Irregular hypoechoic mass left breast 6:00 position posterior depth.
- Abnormal left axillary lymph nodes suspicious for regional nodal metastases. 

BI-RADS 1: Negative right breast.

ADDENDUM: Core needle biopsies of the left breast 12:00 and 1:00 lesions performed under ultrasound guidance using a 14g needle. The 6:00 mass was also biopsied. Core biopsies of the left axillary lymph node performed. Specimens submitted to pathology. The patient tolerated the procedure well. Post-procedure mammograms demonstrate biopsy clip markers in good position with no immediate complications. ➡ *Positive*
- HISTORY: Screening mammogram. No current breast complaints.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 

FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense (BI-RADS category C). This may lower the sensitivity of mammography. No dominant mass, suspicious calcifications or architectural distortion.

LEFT BREAST: 
The breast is heterogeneously dense (BI-RADS category C). This may lower the sensitivity of mammography. No dominant mass, suspicious calcifications or architectural distortion.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breasts which could obscure detection of small masses. 
3. Routine screening mammogram is recommended in 1 year.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAMS WITH TOMOSYNTHESIS
HISTORY: Significant family history of breast cancer. Intermediate lifetime risk. New microcalcifications on screening mammogram.
FINDINGS:
There are scattered areas of fibroglandular density. Overall, the breasts are heterogeneously dense which may obscure detection of small masses.
In the right breast there is a 6 mm cluster of indeterminate microcalcifications at the 10:00 position 6 cm from the nipple. A targeted ultrasound was performed which shows a 5 mm irregular hypoechoic mass corresponding to the area of the microcalcifications. This is a new finding.
The remainder of the right breast is unremarkable. No masses, architectural distortion or suspicious calcifications.
The left breast is unremarkable. No masses, architectural distortion or suspicious calcifications.
IMPRESSION:
New 6 mm cluster of indeterminate microcalcifications right breast 10:00 position 6 cm from the nipple. Targeted ultrasound shows a corresponding 5 mm irregular hypoechoic mass. Biopsy recommended. BIRADS 4B.
The remainder of the right breast and left breast are unremarkable. BIRADS 2.
ADDENDUM: Following discussion of results and management options with the patient, she wishes to proceed with a stereotactic core needle biopsy of the right breast microcalcifications. This has been arranged. ➡ *Positive*
- SCREENING MAMMOGRAM.
Compared to previous exam 2 years ago, the breasts are unchanged.
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.
No suspicious masses, architectural distortion or suspicious calcifications are seen.
IMPRESSION:
BIRADS 2 - Benign findings. Routine screening recommended.
ADDENDUM:
Radiologist reviewed prior images obtained at the Screening Mammography Clinic from April 2019, which are similar. No change.
 ➡ *Not Stated*
- ADDENDUM TO BREAST MRI.
Clinical History: Patient returns today for an MRI-guided biopsy of the right breast.
Comparison: Prior right breast MRI performed on 01/27/2025.
Technique: The patient was placed in the prone position. The right breast was imaged with a dedicated breast coil. Intravenous gadolinium contrast was administered.
Findings:
The previously described area of enhancement in the right breast corresponding to the area of non-mass enhancement is again identified in the right breast. Using real-time imaging, the lesion was localized with an 18-gauge needle. Three cores of tissue were obtained from this region and submitted for pathologic analysis.
IMPRESSION:
Successful right breast MRI-guided biopsy performed without complication. ➡ *Not Stated*
- MAMMOGRAM - BILATERAL DIAGNOSTIC

CLINICAL HISTORY: Right breast microcalcifications.

TECHNIQUE: Standard CC and MLO views of the bilateral breasts were obtained. Spot compression magnification views were performed over the area in question in the right breast.

FINDINGS:

The breasts are heterogeneously dense. There is a 7 mm cluster of pleomorphic microcalcifications in the right breast at 3 o'clock, 2 cm from the nipple. This was biopsied under stereotactic guidance.

IMPRESSION:

1. Cluster of pleomorphic microcalcifications in the right breast at the 3 o'clock position, 2 cm from the nipple, status post stereotactic biopsy. Please see biopsy results.

BI-RADS: 0 (Incomplete). ➡ *Positive*
- Breast Ultrasound Report
Patient Name: [Patient Name]
Date of Exam: 2024-07-12

Clinical History:
Palpable lump in the left breast, 10 o'clock position. History of right breast ductal carcinoma in situ (DCIS) treated with lumpectomy and radiation 5 years ago. Recent mammogram showed new microcalcifications in the left breast.

Findings:
Left Breast:
- A 1.5 cm irregular hypoechoic mass is identified in the left breast at the 10 o'clock position, corresponding to the palpable abnormality. The mass demonstrates angular margins, posterior acoustic shadowing, and increased vascularity on color Doppler imaging.
- A small cluster of microcalcifications is seen adjacent to the mass, correlating with findings on mammography.
Right Breast:
- No suspicious masses or calcifications are identified.

Impression:
- Left breast: Imaging features of the 1.5 cm mass are suggestive of malignancy. The presence of adjacent microcalcifications raises further concern. Biopsy is recommended.
- Right breast: No evidence of recurrent malignancy.

Recommendations:
- Biopsy of the left breast mass under ultrasound guidance.
- Correlation with mammographic findings.
- Further management as clinically indicated.

BI-RADS Category: 5 (Highly Suggestive of Malignancy) ➡ *Positive*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.  
This client was enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), approved by the Research Ethics Board. After providing informed consent, she was randomly allocated to undergo full field digital mammography.
Comparison to prior exam: Available. 
Breast density: Predominantly fatty (< 75% fibroglandular tissue).
Recommendation for annual OBSP recall: Not indicated based on current findings.
The mammographic images demonstrate no features suspicious for malignancy. Microcalcifications are not remarked upon in this report.
Assessment: BIRADS Category 1 or 2, encompassing normal or benign findings. ➡ *Not Stated*
- FINDINGS:
Right breast: Heterogeneously dense breast tissue. There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast: Scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications.  
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography. Consider supplemental screening with breast MRI or whole-breast ultrasound if clinically indicated based on risk assessment.
BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening.
COMPARISON: Prior exams 1/15/2020.
FINDINGS:
The breasts are heterogeneously dense (BI-RADS density C) which may obscure small masses.  
There are no dominant masses, architectural distortions or suspicious calcifications in either breast. Benign-appearing lymph nodes are noted in the right axilla.
In the left breast, there is a new 6 mm cluster of amorphous microcalcifications at the 12:00 position, 6 cm from the nipple. This cluster was not present on the prior exam. 
IMPRESSION:
1. New small cluster of amorphous microcalcifications in the left breast at 12:00, 6 cm from the nipple. Additional magnification views recommended for further evaluation. BIRADS 0.
2. Stable, otherwise negative bilateral screening mammogram. BIRADS 2. 
ADDENDUM: Left magnified views demonstrate a 6 mm cluster of amorphous microcalcifications, suspicious for ductal carcinoma in situ (DCIS). Stereotactic core biopsy recommended. BIRADS 4B. 
ADDENDUM 2: Stereotactic core biopsy of the left 12:00 microcalcifications demonstrates ductal carcinoma in situ, intermediate grade, cribriform type. BIRADS 6. ➡ *Positive*
- MAMMO - BILATERAL - SCREENING. Clinical History: 51-year-old female, no breast complaints, requests screening mammogram today. Comparison is made with previous mammograms from 01/27/2024. Findings: The breasts are heterogeneously dense. No suspicious masses or calcifications are identified. Impression: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
TECHNIQUE: Standard CC and MLO views of both breasts obtained.
FINDINGS:
The breasts are predominantly fatty with scattered areas of fibroglandular density. No significant masses, architectural distortions, or suspicious calcifications are identified bilaterally. 
The skin is unremarkable. No significant axillary lymphadenopathy.
Compared to previous examination dated 1/15/2021, no significant interval change.
IMPRESSION:
BILATERAL MAMMOGRAM, BIRADS 2 - BENIGN STUDY. Recommend routine screening in 1 year.
BREAST DENSITY: The breasts are predominantly fatty (density category A). ➡ *Not Stated*
- Screening Mammogram
Clinical History: 48 year old female, screening mammogram.
Comparison: None.
Technique: Standard CC and MLO views of the breasts were obtained.
Findings:
Heterogeneously dense breast tissue bilaterally.
No suspicious masses, calcifications, or architectural distortions are seen.
IMPRESSION:
Negative screening mammogram. BI-RADS 1. ➡ *Negative*
- SCREENING MAMMOGRAM:

CLINICAL HISTORY: Annual screening mammogram. No clinical concerns.

COMPARISON: With previous exam dated July 15, 2021.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable benign-appearing calcifications are noted in the left breast, likely representing fibrocystic change.

IMPRESSION:
No mammographic evidence of malignancy. 
BI-RADS Category 2: Benign.

RECOMMENDATION:
Routine screening mammogram in 1 year is recommended as per OBSP guidelines.

ADDENDUM:
This study is also a part of the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) under the full field digital mammography arm, as consented by the patient. Breast density is less than 75%. ➡ *Not Stated*
- TMIST FFDM ARM SCREENING.
The patient is randomly assigned to the TMIST full field digital mammography only arm.
Compared to previous screening exam dated MM/DD/YY there is no significant interval change.
Breast tissue is heterogeneously dense, which may obscure small masses.
No dominant mass, architectural distortion or suspicious calcifications.
BIRADS Category 2: Benign findings. Routine screening recommended in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram. No clinical concerns.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained using digital mammography.

COMPARISON: Comparison is made to the prior study from 3/15/2021.

FINDINGS:

RIGHT BREAST:
The breast tissue is heterogeneously dense (BI-RADS category C), which may obscure detection of small masses.
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
There are scattered areas of fibroglandular density.

LEFT BREAST:
The breast tissue is heterogeneously dense (BI-RADS category C), which may obscure detection of small masses.
At the 12:00 position, 7 cm from the nipple, there is a new group of amorphous microcalcifications, spanning an area of approximately 1 cm. 
No associated mass, architectural distortion, or skin changes are identified in this area.
The remainder of the left breast is unremarkable, with no dominant masses or areas of architectural distortion.

IMPRESSION:
New group of amorphous microcalcifications in the left breast at 12:00, 7 cm from the nipple, suspicious for ductal carcinoma in situ (DCIS). Additional workup with spot compression magnification views is recommended for further characterization. BI-RADS 4B.

Heterogeneously dense breast tissue bilaterally, which may lower the sensitivity of mammography.

RECOMMENDATION:
Left diagnostic mammogram with spot compression magnification views of the area of microcalcifications at 12:00. Consider biopsy if suspicious calcifications are confirmed.

Routine yearly screening mammography is recommended. ➡ *Positive*
- TMIST FFDM ARM OBSP MAMMOGRAM
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes. 
OBSP to recall client in 2 years: Yes.
No mammographically evident malignancy.  
BIRADS-1. ➡ *Not Stated*
- REPORT:
"NORMAL HHUS SCREENING OBSP MAMMOGRAM.
Patient presented for high risk screening exam.
Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
There are no suspicious masses, architectural distortion or dominant suspicious calcifications. The breasts are heterogeneously dense which may obscure detection of small masses.
BIRADS-2.

ADDENDUM: 
Comparison has now been made to outside screening mammogram from May 2021 at St. Mary's Hospital. There has been no significant interval change. " ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAPHY.
History: Screening.
Comparison is available dating from 01/2023.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- MAMMOGRAM REPORT:
CLINICAL INDICATION: Annual screening mammogram. No current breast complaints.
COMPARISON: Prior mammogram from 12/15/2021.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
Both breasts are composed of scattered fibroglandular densities. 
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. 
Stable benign-appearing calcifications are present bilaterally. 
No significant interval change compared to prior exam.
IMPRESSION: 
BI-RADS Category 1: Negative bilateral screening mammogram. 
Recommend routine annual screening mammography.
ADDENDUM: Spoke with ordering physician Dr. Smith who confirms patient has no breast concerns at this time. BI-RADS 1 assessment is appropriate. ➡ *Negative*
- SCREENING MAMMOGRAM - OBSP ELIGIBLE

COMPARISON: None available.

FINDINGS:
The breasts are heterogeneously dense (50-75% fibroglandular). No dominant masses, architectural distortion, or suspicious calcifications.

Benign calcifications are present bilaterally, likely representing involuting fibroadenomas.

IMPRESSION:
No mammographic evidence of malignancy. 
Heterogeneously dense breasts (50-75% fibroglandular).
Routine screening recommended.

BIRADS Category: 2 - Benign 

ADDENDUM: 
The patient called with concerns regarding her results. Reassured that the calcifications noted are characteristically benign, and not suspicious for malignancy. Routine screening as per OBSP guidelines is appropriate. No further investigations are required at this time based on the mammographic findings. ➡ *Not Stated*
- MAMMOGRAM AND BREAST ULTRASOUND
CLINICAL HISTORY: Routine screening mammogram, no known palpable abnormalities. Family history of breast cancer in maternal aunt.
COMPARISON: Comparison is made to prior mammogram and ultrasound performed 1 year ago.
FINDINGS:
Both breasts contain scattered areas of fibroglandular density. No focal masses, architectual distortion, or suspicious calcifications are identified in either breast. 
Targeted ultrasound of the right upper outer quadrant was performed due to mild focal asymmetry seen on mammogram. No correlating solid lesion or suspicious finding identified on ultrasound. 
Stable bilateral benign-appearing lymph nodes are seen in both axillae.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued routine annual screening mammography. 
2. No sonographic correlate to the mammographic asymmetry in the right upper outer quadrant. Likely represents summation artifact.
BIRADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- SCREENING DIGITAL BILATERAL MAMMOGRAM.

Technique: Standard CC and MLO views of both breasts were obtained. 

Comparison: Outside screening mammogram from 1 year ago.

Findings:
The breasts are heterogeneously dense, which may obscure detection of small masses. 

There are no dominant masses, architectural distortion, or suspicious calcifications.

Impression:
BI-RADS Category 1: Negative. Routine screening is recommended in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Routine screening. No current breast concerns.
COMPARISON: None.
FINDINGS:
Both breasts are almost entirely fatty.
There are no dominant masses, architectural distortion, or abnormal calcifications in either breast.
In the right breast at the 10:00 position 6 cm from the nipple, there is a group of amorphous calcifications in a segmental distribution, spanning an area of approximately 2 cm. Additional magnification views confirm these are suspicious calcifications.
The left breast is unremarkable.
No axillary, supra- or infraclavicular adenopathy.
IMPRESSION:
Right breast: Suspicious segmental distribution of amorphous calcifications at 10:00, 6 cm from the nipple, spanning 2 cm. BIRADS 4. Stereotactic core biopsy recommended for tissue diagnosis.
Left breast: Negative. BIRADS 1. Recommend routine screening in 1 year. ➡ *Positive*
- MAMMOGRAPHY REPORT

PROCEDURE:

Bilateral digital mammography and bilateral whole breast and axillary ultrasound.

HISTORY:  56 year old female presenting for routine yearly screening mammogram. No focal areas of pain or palpable lumps.

Compared to previous mammograms and breast ultrasounds from August 2021. Prior studies are available.

FINDINGS:

The breasts are heterogeneously dense which can obscure small masses.

Bilateral upper outer quadrant 4-5 mm and lower outer quadrant 2-3 mm clustered round punctate microcalcifications have slightly increased in number and are indeterminate in appearance. No other mammographic abnormalities.

Bilateral whole breast and axillary ultrasound:

Scattered bilateral 2-4 mm simple appearing cysts. No dominant solid masses, complex cystic lesions, skin thickening or axillary lymphadenopathy.

IMPRESSION:

1. Bilateral heterogeneously dense breast tissue.
2. Scattered bilateral simple cysts (BIRADS 2). 
3. Bilateral indeterminate microcalcifications upper and lower outer quadrants, slightly increased compared to prior (BIRADS 4).

RECOMMENDATIONS: 

Short term 6 month mammographic follow up or stereotactic biopsy of the microcalcifications.

BIRADS 4: Suspicious abnormality - biopsy should be considered. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS RIGHT BREAST

CLINICAL HISTORY: Follow up from prior screening exam 6 months ago which showed a small focal asymmetry 10:00 right breast (seen on one view only).

COMPARISON: Outside screening mammograms 6 months prior. Current images are compared to the prior outside images which are of good quality.

FINDINGS:  
The breasts are heterogeneously dense (C). 

There is minimal fibroglandular tissue in the retroareolar region of both breasts but no discrete masses or concerning architectural distortions.

In the right breast, there is a small group of amorphous calcifications in the upper outer quadrant 10 cm from the nipple, corresponding to the area of focal asymmetry noted on the prior exam. The grouping spans approximately 6 mm. There are no associated mass or architectural distortion.

Magnification views in the mediolateral and craniocaudal projections confirm the presence of the grouped amorphous calcifications in the right breast at 10:00. No additional suspicious calcifications.

No suspicious findings are seen in the left breast. There are scattered vascular calcifications bilaterally.

IMPRESSION:
Small group of amorphous calcifications in the right breast 10:00, 10 cm from the nipple corresponding to the area of asymmetry on prior exam. These are new and suspicious. Additional evaluation with stereotactic biopsy is recommended. BIRADS 4B. 

Heterogeneously dense breasts, which may obscure detection of small masses.

ADDENDUM: The patient returned for stereotactic biopsy of the group of amorphous calcifications in the right breast 10:00. Specimen radiography confirmed calcifications within the core samples. Final pathology revealed ductal carcinoma in situ, intermediate nuclear grade with necrosis. ➡ *Positive*
- TECHNIQUE: BILATERAL BREAST ULTRASOUND

INDICATION: 45-year-old female with family history of breast cancer presenting for routine screening. No palpable abnormalities noted on clinical breast exam.

COMPARISON: None available.

FINDINGS:
Right breast: The breast tissue is heterogeneously echogenic. No discrete solid or cystic mass lesions are identified. No areas of architectural distortion or suspicious calcifications are seen. 

Left breast: The breast tissue is heterogeneously echogenic, similar to the right side. No discrete solid or cystic mass lesions are identified. No areas of architectural distortion or suspicious calcifications are seen.

Axillary regions: No abnormal lymph nodes are identified in either axilla.

IMPRESSION:
1. No sonographic evidence of malignancy in either breast. 
2. Recommend routine annual screening mammography and clinical breast exams in this patient with positive family history of breast cancer.

BI-RADS CATEGORY 1: Negative. ➡ *Negative*
- FINDINGS:
- Bilateral mammography and ultrasound were performed. 
- The breast tissue is heterogeneously dense, which may obscure small masses.
- There are no mammographic or sonographic findings to suggest malignancy. Specifically, no dominant mass, suspicious calcifications, or architectural distortion are identified in either breast.
- Bilateral axillary regions are unremarkable.
- There is no significant interval change compared to prior studies.

IMPRESSION: 
Negative bilateral screening mammogram and ultrasound. No evidence of malignancy.
BI-RADS Category 1 - Negative. Routine screening in 1 year is recommended. ➡ *Negative*
- MAMMOGRAPHY REPORT - BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram. Patient denies any palpable lumps, skin changes, or nipple discharge.

COMPARISON: Comparison made to prior mammogram dated 09/15/2022.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. 

RECOMMENDATION:
1. Continue annual screening mammography.
2. Consider supplemental screening with breast ultrasound or MRI due to dense breast tissue.

BI-RADS Category 1: Negative.

ADDENDUM:
This examination was reviewed by the interpreting radiologist. The results and recommendations were discussed with the patient. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY.
History: Screening
Comparison is available dating from 02/2021.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- REPORT:
"BILATERAL MAMMOGRAM
HISTORY: Annual screening mammogram. 
COMPARISON: Prior studies from 10/2021 at St. Mark's Hospital.
FINDINGS: 
The breast tissue is predominantly fatty bilaterally. 
There are no dominant masses, suspicious microcalcifications, or areas of architectural distortion in either breast.  
Stable bilateral vascular calcifications noted.
There are no significant changes compared to prior exam.  
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography.
BI-RADS Category 1: Negative."

LABEL: Negative for microcalfications in the current report. ➡ *Negative*
- LEFT BREAST MAMMOGRAM
INDICATION: Annual screening mammogram. No complaints. Patient has a history of right breast cancer with previous lumpectomy and radiation therapy 5 years ago.
COMPARISON: Comparison is made to prior exams from 2021 and 2020.
FINDINGS: 
The left breast is composed of scattered fibroglandular densities. 
There are no dominant masses, architectural distortion, or suspicious calcifications.  
No significant interval change from prior exams.
IMPRESSION:
No mammographic evidence of malignancy in the left breast. 
Continue annual screening mammography.
BIRADS 1 - NEGATIVE ➡ *Negative*
- FINDINGS: The right breast demonstrates post-mastectomy changes. Left breast tissue is comprised of scattered fibroglandular densities. There are no suspicious masses, architectural distortions, or microcalcifications. Stable appearance compared to previous examinations.

Targeted left breast ultrasound at the site of palpable abnormality, 7 o'clock position, shows no discrete solid lesion. The area of concern corresponds to normal fibroglandular tissue.

IMPRESSION:
1. Stable left breast mammogram without evidence of malignancy. No suspicious microcalcifications. Recommend annual screening mammography.
2. Targeted left breast ultrasound is negative for correlate at the site of palpable finding, likely representing normal fibroglandular tissue. Clinical follow up is advised.

BIRADS 1 - Negative. ➡ *Negative*
- HISTORY: Patient presenting for routine annual screening mammogram.
FINDINGS:
Right breast is fatty replaced. There are several vague asymmetries in the right breast at the 12:00 position, 10 cm from the nipple which are stable compared to the prior mammogram. These are benign based on stability.
The left breast is also fatty replaced. No dominant masses, architectural distortion or suspicious calcifications are seen.
Bilateral magnification views demonstrate a new cluster of microcalcifications in the central left breast at 12:00, 4 cm from the nipple. These are pleomorphic in morphology and are suspicious for malignancy.
IMPRESSION:
1. Stable right breast. BI-RADS 2.
2. New group of suspicious microcalcifications in the left breast, BI-RADS 4B. Stereotactic biopsy is recommended for diagnosis.
ADDENDUM:
The patient returned for stereotactic biopsy of the suspicious microcalcifications in the left breast using a 9 gauge vacuum assisted biopsy device. Specimen radiograph confirmed presence of microcalcifications. Pathology results are pending. ➡ *Positive*
- FINDINGS: The breasts are almost entirely fatty. No suspicious masses, architectural distortions, or skin thickening. No axillary lymphadenopathy.

There are scattered benign-appearing calcifications in both breasts, likely reflecting fibrocystic changes.

IMPRESSION: Benign mammogram. Predominantly fatty breasts. Scattered benign-appearing calcifications bilaterally, unchanged. No evidence of malignancy.
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- ULTRASOUND LEFT BREAST
INDICATION: 42-year-old female with palpable mass left breast.
FINDINGS: The left breast demonstrates predominantly fatty parenchyma. At the 12 o'clock position, 1 cm from the nipple, there is a 0.9 x 0.7 x 0.5 cm irregular solid hypoechoic mass with ill-defined margins and posterior acoustic shadowing. No vascularity demonstrated on color Doppler imaging. 
Left axilla normal. 
RIGHT BREAST: Fatty parenchyma. No suspicious masses, architectural distortion, or significant lymph nodes.
IMPRESSION:
New 9 mm irregular hypoechoic mass left breast at 12 o'clock position, suspicious for malignancy. Ultrasound-guided core needle biopsy recommended. BIRADS 4C.
Grouped amorphous microcalcifications spanning 8 mm in the upper inner left breast, indeterminate. Stereotactic biopsy is recommended. Final BIRADS 4B.
Benign right breast. BIRADS 2.
ADDENDUM: Patient proceeded with left breast ultrasound-guided core biopsy of the 12 o'clock mass. Pathology demonstrated invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. Stereotactic biopsy of the left breast microcalcifications showed ductal carcinoma in situ, intermediate grade, with focal microinvasion. Surgical consultation is recommended for further management. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography. 
Right breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast: 
There are no dominant masses, architectural distortion, or suspicious calcifications. 
Stable, benign-appearing calcifications are noted in the left breast, likely representing fibrocystic change.
Multiple cysts are again noted bilaterally, largest in the right upper outer quadrant measuring up to 9 mm. These are unchanged. 
Surgical clips are noted bilaterally from prior biopsies.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Continued annual screening mammography is recommended. 
3. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography.

ASSESSMENT: 
BI-RADS Category 2: Benign finding. Recommend routine screening in 1 year.

ADDENDUM:
Comparison was made to prior outside mammograms from December 2020 at ABC Hospital. The benign calcifications and cysts are stable. No change from prior exams. ➡ *Negative*
- FINDINGS:
Both breasts show scattered fibroglandular tissue. No suspicious mass, architectural distortion or calcifications are identified.

IMPRESSION:
Negative bilateral screening mammogram. Recommend continued annual screening in 12 months. 

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAM REPORT:
PROCEDURE: Bilateral diagnostic mammogram with left breast ultrasound.
INDICATIONS: Palpable lump in the left breast at 5 o'clock, no prior mammograms available.
MEDICAL HISTORY: Patient is a 50-year-old female with no family history of breast cancer. 
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense, which could obscure small masses.
Right Breast:
There are no focal masses, areas of architectural distortion, or suspicious calcifications.
Left Breast:
There are no focal masses, areas of architectural distortion, or suspicious calcifications.
Targeted ultrasound of the left breast at 5 o'clock reveals a 1 cm oval hypoechoic mass with circumscribed margins, parallel orientation, and no posterior acoustic features. This corresponds to the palpable area of concern.
IMPRESSION:
1. No evidence of malignancy in either breast on mammography.
2. 1 cm circumscribed oval mass in the left breast at 5 o'clock on targeted ultrasound, likely benign. Short-term follow up is recommended to document stability. 
BIRADS 2 - Benign findings. Short interval follow-up suggested. ➡ *Negative*
- MAMMOGRAPHY SCREENING EXAMINATION
CLINICAL HISTORY: Screening. RLQ superficial discomfort. Personal history of left breast angiosarcoma status post left lumpectomy 2018 and adjuvant radiation and chemotherapy.
COMPARISON: None.  
FINDINGS:
The breast tissue is heterogeneously dense with scattered fibroglandular densities. This may lower the sensitivity of mammography.
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
No significant change compared to mammogram performed 2 years ago at an outside hospital. No concerning interval change within the right breast.
Prior left lumpectomy is noted.
IMPRESSION:
1. No evidence of right breast malignancy. Recommend annual screening mammography and follow up of the right breast symptoms as clinically warranted.
2. BI-RADS category 1: Negative. 
ADDITIONAL RECOMMENDATIONS: Annual screening mammogram is recommended. Clinical follow up of the right breast symptoms as needed. ➡ *Negative*
- SCREENING BILATERAL MAMMOGRAM
CLINICAL INDICATION: Annual screening. 
COMPARISON: Prior screening mammogram from 6/12/2020.
FINDINGS: 
The breasts are predominantly fatty, scattered fibroglandular densities.
Right breast:
There is an oval circumscribed isodense mass with obscured margins in the upper outer quadrant at 10 o'clock, 10 cm from the nipple, measuring 10 x 6 mm. This corresponds to a hypoechoic solid mass with irregular margins on targeted ultrasound. No other suspicious masses, calcifications or architectural distortion.  
Left breast:
There are new scattered amorphous microcalcifications at 3 o'clock middle depth 7 cm from nipple. No suspicious associated mass. No other suspicious masses or architectural distortion in the left breast.
IMPRESSION:
1. New 10 x 6 mm irregular hypoechoic mass right breast 10:00, 10 cm from nipple, suspicious. BI-RADS 4.
2. New grouped amorphous microcalcifications left breast 3:00, 7 cm from nipple. BI-RADS 4.  
RECOMMENDATION:
Ultrasound guided core biopsy of the right breast mass and stereotactic biopsy of the left breast microcalcifications recommended. Please send biopsy requisition to book procedures.

ADDENDUM: Pathology from right breast mass biopsy showed invasive ductal carcinoma, grade 2. Left breast microcalcification biopsy showed atypical ductal hyperplasia. Updated BI-RADS:
Right: BI-RADS 6
Left: BI-RADS 4 ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammography was performed. The breast tissue is heterogeneously dense, which could obscure detection of small masses.

There are no dominant masses, architectural distortions or suspicious calcifications.

IMPRESSION:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year.

ADDENDUM - Dr. Smith:
I have reviewed the images and agree with the above interpretation. No suspicious microcalcifications are identified in either breast. Routine annual screening recommended. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAMS
HISTORY: Annual screening mammogram.
FINDINGS:
The breasts are heterogeneously dense.
The right breast shows no suspicious mass, calcifications or architectural distortion.
In the left breast in the upper outer quadrant at 2:00, 6 cm from the nipple, there is a 6 mm cluster of pleomorphic microcalcifications. No associated mass. The remaining left breast is unremarkable.
No suspicious axillary lymphadenopathy.
IMPRESSION:
Right breast: No mammographic evidence of malignancy. BI-RADS 1. Continue annual screening.
Left breast: New cluster of pleomorphic microcalcifications at 2:00, 6 cm from nipple, suspicious for malignancy. BI-RADS 4B. Stereotactic biopsy recommended.
ADDENDUM - POST BIOPSY LEFT BREAST:
Pathology from stereotactic biopsy of the microcalcifications at 2:00 left breast demonstrates ductal carcinoma in situ (DCIS), intermediate grade. BI-RADS 6. Surgical consultation recommended. ➡ *Positive*
- SCREENING MAMMOGRAM
There are scattered areas of fibroglandular density.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
No suspicious masses, architectural distortion, or suspicious calcifications.
BIRADS Category 1: Negative.
Routine screening is recommended in 1 year.
ADDENDUM:
Comparison has been made to previous exam dated 1/14/2021. There are no significant changes. ➡ *Not Stated*
- TMIST FFDM ARM OBSP SCREENING MAMMOGRAM

This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.

Compared to previous exam: Yes.

Breast Density: Scattered fibroglandular density.

There are no suspicious masses, architectural distortions, or skin changes. 

IMPRESSION:
BIRADS 2 - Benign findings. Routine screening recommended.

Addendum:
Compared to the previous exam, there are no significant changes. The breasts remain heterogeneously dense, which may obscure detection of small masses. No mammographic evidence of malignancy. ➡ *Not Stated*
- LEFT MAMMOGRAM FINDINGS:
Scattered fibroglandular densities are present. No dominant mass, architectural distortion or suspicious calcifications.
IMPRESSION:
No mammographic evidence of malignancy.
BI-RADS category 1: Negative.
Recommend annual screening mammography. ➡ *Negative*
- FINDINGS:
Bilateral digital screening mammograms demonstrate heterogeneously dense breast tissue. 
In the right breast, there are no dominant masses, areas of architectural distortion, or suspicious calcifications.
In the left breast, there is a new cluster of pleomorphic microcalcifications in the upper outer quadrant at the 1 o'clock position, approximately 7 cm from the nipple. The cluster measures 8 mm in greatest dimension. No associated mass or architectural distortion is identified. 
Magnification views of the left breast in the craniocaudal and mediolateral projections confirm the presence of the cluster of pleomorphic microcalcifications. The distribution is segmental.
There are no suspicious findings in the left axilla.

IMPRESSION:
Right breast: No mammographic evidence of malignancy. BI-RADS 1 - Negative.
Left breast: New cluster of pleomorphic microcalcifications in the upper outer quadrant, suspicious for malignancy. BI-RADS 4C - Suspicious abnormality. Stereotactic biopsy recommended for definitive diagnosis.

ADDENDUM:
The patient proceeded to left breast stereotactic biopsy under ultrasound guidance. Specimen radiograph confirmed calcifications within the core samples. Final pathology results demonstrated ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. Wide local excision is recommended for complete evaluation of extent of disease and to rule out invasion. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS:
History: Annual screening.
FINDINGS:
The breasts are heterogeneously dense. 
In the right breast, scattered areas of benign calcifications are evident. No dominant masses, architectural distortion, or suspicious calcifications.
In the left breast, a new group of fine pleomorphic microcalcifications is identified in the upper outer quadrant measuring approximately 1 cm. No associated mass, distortion, or skin changes.
Compared to prior study from 1 year ago, the left breast microcalcifications are new. The remainder of the breast tissue is stable.
IMPRESSION:
BIRADS 4 - Suspicious grouped microcalcifications in the left breast, not present on prior exam. Stereotactic biopsy recommended for further evaluation. 
BIRADS 2 - Benign right breast.
ADDENDUM:
The patient returned for stereotactic biopsy of the left breast microcalcifications. Specimen radiograph confirmed calcifications within the core samples. Final pathology revealed ductal carcinoma in situ, intermediate grade.
This case was reviewed in our weekly multidisciplinary breast conference. Recommendation is for surgical excision followed by radiation therapy. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense.
In the left lower outer breast at 5:00, 4 cm from the nipple, there is a 1.8 x 1.2 x 1.0 cm irregular hypoechoic mass with indistinct margins, posterior acoustic shadowing and internal vascularity. This was not seen on the previous mammogram from 2 years ago.
Ultrasound of the left axilla demonstrates two abnormal appearing lymph nodes. The most suspicious lymph node measures 2.0 cm in greatest dimension and has lost its normal fatty hilum.
Magnification views of the left breast demonstrate a 5 mm cluster of pleomorphic calcifications at the 10:00 position, 6 cm from the nipple. These were not present on the prior mammogram.
The right breast is unremarkable. No suspicious masses, calcifications or areas of architectural distortion.

IMPRESSION:
1. Highly suspicious mass left lower outer breast at 5:00, 4 cm from the nipple. Suspicious for malignancy. Recommend biopsy.
2. Abnormal appearing left axillary lymph nodes suspicious for metastatic involvement.
3. New cluster of pleomorphic calcifications at 10:00, left breast. Suspicious for malignancy. Recommend biopsy.
4. Heterogeneously dense breasts which may lower the sensitivity of mammography.

BI-RADS CATEGORY 5: Highly suggestive of malignancy. Appropriate action should be taken.

ADDENDUM:
Stereotactic core biopsy of the left breast calcifications at the 10:00 position demonstrates ductal carcinoma in situ, intermediate grade, cribriform type.
Ultrasound guided core biopsy of the left breast mass at 5:00 demonstrates invasive ductal carcinoma, moderately differentiated, Nottingham grade 2. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
INDICATION: Annual screening mammogram. No current breast complaints. 
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  
COMPARISON: Comparison is made to prior mammogram from 8/15/2021.
FINDINGS:  
The breasts are heterogeneously dense (C), which may obscure detection of small masses.
There are no suspicious masses, architectural distortions, or abnormal calcifications in either breast. The breast tissue is stable compared to prior mammogram.  
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening in 12 months.
2. Heterogeneously dense breast tissue, which lowers the sensitivity of mammography. 
ASSESSMENT CATEGORIES:
Right breast: BI-RADS 1 Negative
Left breast: BI-RADS 1 Negative
MANAGEMENT: Recommend routine screening mammography in 1 year. ➡ *Negative*
- MAMMOGRAM REPORT:

PROCEDURE: Screening Bilateral Mammogram.

HISTORY: 51 year old female with family history of breast cancer.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 

COMPARISON: None available.

FINDINGS:
Right Breast:
The breast is almost entirely fatty. There are no masses, architectural distortion, or suspicious calcifications.

Left Breast:
The breast is heterogeneously dense, which could obscure detection of small masses. There are no masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts.

RECOMMENDATION:
Routine yearly screening mammography is recommended.

BIRADS CATEGORY 1: Negative. ➡ *Negative*
- MAMMOGRAM LEFT BREAST.
History: Abnormal screening mammogram.
Comparison is made with the patient's prior examination from external institution. These were sent to us electronically.
Findings:
There are scattered fibroglandular densities. No architectural distortion, suspicious calcification, or definite masses are seen.
Impression:
Negative left mammogram.
BI-RADS Category 1. ➡ *Negative*
- Addendum to Breast MRI

Reason for Exam: Evaluation of microcalcifications

Comparison: Mammogram 10/26/2024

History: Patient presents with new microcalcifications seen on screening mammogram.

Findings:
The previously described cluster of microcalcifications in the right breast at 2 o'clock, 3 cm from the nipple, demonstrates mild enhancement on the post-contrast subtracted images. No other suspicious enhancement is seen. No suspicious masses or areas of diffusion restriction are identified.

Impression:
Enhancement of known microcalcifications in the right breast. Biopsy proven malignancy. Correlate with mammographic findings and clinical follow-up.

BI-RADS: 6 (Known biopsy-proven malignancy) ➡ *Positive*
- MAMMOGRAM LEFT BREAST WITH SPOT MAGNIFICATION.
HISTORY: Left breast pain.
COMPARISONS: Prior from [redacted] 09/07/2022.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Spot magnification views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, or suspicious calcifications are seen. No skin thickening or nipple retraction is seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- MAMMOGRAM AND ULTRASOUND OF LEFT BREAST

HISTORY: 42-year-old female presents for diagnostic mammogram and ultrasound of the left breast. Patient reports no palpable lump or pain. Last mammogram was 1 year ago and was unremarkable. No family history of breast cancer.

FINDINGS:
Left breast tissue is heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, architectural distortion, or suspicious calcifications.

Targeted ultrasound of the left breast at the 10 o'clock position demonstrates normal fibroglandular tissue with no focal solid or cystic lesions.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. 
2. Heterogeneously dense breast tissue, which lowers the sensitivity of mammography.

BIRADS 1 - Negative. 
Recommend routine annual screening mammography. ➡ *Negative*
- MAMMOGRAM AND ULTRASOUND BILATERAL BREASTS
HISTORY: Annual screening mammogram. History of right breast lumpectomy.  
FINDINGS: The breasts are heterogeneously dense (Category C). There are no new masses, suspicious calcifications, or areas of architectural distortion bilaterally compared with prior exams. Stable postoperative changes noted in the right breast at the lumpectomy site. Benign calcifications bilaterally, stable. 
Bilateral breast ultrasound was performed. No sonographic abnormality is identified. 
IMPRESSION: 
1. No mammographic evidence of malignancy, both breasts. Annual screening mammography is recommended. 
2. No sonographic abnormality bilaterally.
BI-RADS 2: Benign findings. Annual screening mammography is recommended.
ADDENDUM: In comparison with prior exam 2 years ago, no significant interval change.  The previously noted benign scattered fibroglandular densities are again noted and are stable. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Routine screening.

TECHNIQUE: Bilateral full field digital mammography including CC and MLO views.

FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses. 

A group of amorphous microcalcifications is noted in the upper outer quadrant of the right breast on CC and MLO views, approximately 6 cm from the nipple. This cluster measures about 8 mm in largest dimension. No associated mass, architectural distortion or skin thickening.

The left breast demonstrates scattered areas of fibroglandular density. No dominant mass, suspicious calcifications, architectural distortion or skin thickening.

IMPRESSION:
BI-RADS Category 0: Incomplete assessment. A group of amorphous microcalcifications is noted in the upper outer quadrant of the right breast on screening mammogram, not definitely benign. Additional imaging evaluation with spot compression magnification views is recommended for further characterization. 

BI-RADS Category 1: Negative in the left breast. Routine screening in 1 year is recommended.

ADDENDUM AFTER DIAGNOSTIC MAMMOGRAM RIGHT BREAST:
Right breast diagnostic CC and MLO views with spot compression magnification views of the right upper outer quadrant were performed. 

The cluster of amorphous microcalcifications in the right upper outer quadrant persists and appears slightly more conspicuous on spot compression magnification views, spanning an area of approximately 10 mm. No associated mass, distortion or skin changes.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic core biopsy is recommended for tissue diagnosis. ➡ *Positive*
- ULTRA SOUND BREAST 01/01/30
COMPARISON: Mammogram of same date

INDICATION: Right breast mass, new.

FINDINGS: The breasts are heterogeneously dense. Diffuse fibroglandular tissue scattered in both breasts. There is no architectural distortion or suspicious microcalcifications. There is no skin thickening or retraction. 

In the right breast at 10 o'clock 5 cm from the nipple there is a 2.1 x 1.8 x 1.6 cm oval circumscribed hypoechoic mass, oriented parallel to the skin. No posterior features are seen. Color flow Doppler demonstrates no internal vascularity. 

No suspicious masses, calcifications, or other findings are seen in the left breast. 

IMPRESSION:
1. Right 10 o'clock 5 cm from nipple 2.1 cm oval circumscribed mass. Suspicious for fibroadenoma. Biopsy should be considered if clinically warranted. 

2. No suspicious findings left breast. Benign. 

BI-RADS 1 left breast.
BI-RADS 4a right breast.

ADDENDUM:
Aspiration and core biopsy performed of the right breast mass under ultrasound guidance. Pathology revealed fibroadenoma with no atypia. Concordant with imaging. Recommend clinical follow up and diagnostic imaging in 6 months. ➡ *Negative*
- HISTORY: Screening mammogram. Post-menopausal with no current breast complaints. Maternal aunt and grandmother with history of breast cancer.

FINDINGS:
Both breasts are heterogeneously dense, which could obscure detection of small masses. 

Right breast: There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

Left breast: There are no dominant masses, suspicious calcifications, or areas of architectural distortion. 

IMPRESSION:
1. Negative digital bilateral screening mammogram.  
2. Heterogeneously dense breast tissue which could lower the sensitivity of mammography. 

RECOMMENDATION:  
Recommend routine screening mammogram in 1 year.

BIRADS 1 - Negative.

ADDENDUM: Reviewed images with 3D tomosynthesis, no significant change to above findings or BIRADS category. ➡ *Negative*
- FINDINGS: The breasts are heterogeneously dense. This may lower the sensitivity of mammography. No focal dominant masses, architectural distortion, or suspicious calcifications are identified. Stable bilateral benign-appearing calcifications are noted. The skin appears normal with no evidence for thickening or retraction. 

IMPRESSION:

1. No mammographic evidence of malignancy. There are no microcalcifications present.

2. Recommend continued routine screening mammography due to heterogeneously dense breast composition. Ultrasonography may be a useful adjunct screening tool.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- FINDINGS:
Right breast: Postmastectomy changes noted. No suspicious masses or abnormalities appreciated.
Left breast: Predominantly fatty breast tissue with scattered fibroglandular densities. No focal masses, architectural distortion, or suspicious calcifications identified. 
IMPRESSION:
1. Postmastectomy changes of the right breast. No evidence of recurrence.
2. No suspicious findings in the left breast. Scattered fibroglandular tissue present.
BIRADS 1 - Negative bilateral screening mammogram. ➡ *Negative*
- FINDINGS: The left breast tissue is heterogeneously dense with scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications. The skin appears normal. 

At the 7 o'clock position in the left breast there is a new 8 mm asymmetry on the mammographic views, corresponding to the area of palpable abnormality. Targeted ultrasound of this region demonstrates an 8 x 6 x 5 mm circumscribed oval parallel hypoechoic mass with no posterior acoustic features. This likely represents a fibroadenoma. No other suspicious sonographic findings.

The right breast is surgically absent, consistent with history of right mastectomy.

IMPRESSION:
1. Asymmetry in the left breast at 7 o'clock, corresponding to palpable finding. Targeted ultrasound suggests this is a fibroadenoma. 6 month follow up ultrasound is recommended to document stability. 
2. No mammographic or sonographic evidence of malignancy. 
3. Right mastectomy changes, stable.

BI-RADS 3 - Probably benign finding. Short interval follow up suggested. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (Type C). Scattered areas of fibroglandular density.

There are no suspicious masses, architectural distortions, or focal asymmetries in either breast. 

There are scattered microcalcifications in the upper outer quadrant of the right breast at approximately 10 o'clock position. These are grouped coarse heterogeneous calcifications, extending over an area measuring approximately 8 mm. No associated mass or architectural distortion. 

Left breast is unremarkable with no suspicious masses, calcifications, or architectural distortion.

IMPRESSION:
BI-RADS Category 4: Suspicious grouped coarse heterogeneous microcalcifications in the right breast at 10 o'clock position. Stereotactic biopsy recommended for further evaluation. BI-RADS 4.

BI-RADS Category 1: Normal left breast. Annual screening mammogram recommended in 1 year. 

ADDENDUM: Patient was informed of results and agrees to proceed with stereotactic biopsy of the right breast microcalcifications. Biopsy to be scheduled. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM FOR SCREENING. Clinical History: The patient is a pleasant 55-year-old female presenting for her annual screening mammogram today. Comparison: Comparison is made to mammograms performed on 01/04/2024. Findings: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcifications. Impression: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1. ➡ *Negative*
- FINDINGS: Bilateral mammographic views show predominantly fatty replaced breast tissue. There are no focal masses, architectural distortion, or suspicious calcifications in either breast. Stable benign calcifications are noted in the right breast. No significant change from prior mammogram.

Targeted ultrasound was performed in the left breast at the site of palpable abnormality reported by the patient at 2 o'clock position, 3 cm from the nipple. No correlating sonographic mass or cystic lesion identified.

IMPRESSION:
1. Negative bilateral screening mammogram. Fatty replaced breast tissue. 
2. No sonographic correlate for the palpable finding in the left breast.

BIRADS 1 - Negative. Recommend annual screening mammography in 1 year.

ADDENDUM: Comparison was made to prior outside mammogram from 10/15/2021, which was also negative. No prior ultrasound is available for comparison. ➡ *Negative*
- FINDINGS: Bilateral mammographic views demonstrate that the breasts are almost entirely fatty replaced. There are no masses, architectural distortion, or suspicious calcifications. Stable bilateral benign calcifications are noted. There are no significant changes compared to prior mammograms from 2019. 

Targeted left breast ultrasound was performed at the site of palpable abnormality at 2 o'clock, 4 cm from the nipple. This demonstrates a 6 mm simple cyst. No solid masses are seen at this site or elsewhere in the left breast. The right breast is unremarkable on ultrasound.

IMPRESSION:
1. No mammographic evidence of malignancy. Benign calcifications are noted bilaterally, stable compared to prior exams. 
2. Left breast 6 mm simple cyst corresponding to the area of palpable concern. This is a benign finding. 
3. No suspicious findings on ultrasound of either breast.

BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- SCREENING MAMMOGRAM

INDICATION: Annual screening mammography.

COMPARISON: Previous outside screening mammogram from 1 year ago.

FINDINGS:
Both breasts are heterogeneously dense which could obscure detection of small masses. 

There are no dominant masses, architectural distortion or suspicious calcifications in either breast.

Incidental note is made of vascular calcifications bilaterally.

Scattered fibroglandular densities bilaterally.

At the 12 o'clock position of the left breast, 6 cm from the nipple, there is a group of amorphous calcifications spanning approximately 6 mm. Additional magnification views of this area in the left CC and MLO views demonstrate 8 calcifications in a cluster. These are new compared to prior mammograms. 

The visualized axillary regions are unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete Assessment. 
Amorphous cluster of calcifications in the left breast at 12 o'clock, 6 cm from the nipple. Additional imaging evaluation with magnification views is required. Depending on the findings, tissue sampling may be necessary.

RECOMMENDATION:  
Recall for additional magnification views of the left breast and for possible stereotactic core biopsy depending on the findings.

ADDENDUM:  Magnification views of the left breast were performed. The amorphous cluster of calcifications at 12 o'clock 6 cm from the nipple span approximately 10 mm. Stereotactic guided core biopsy is recommended for tissue diagnosis. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAM
INDICATION: Screening mammogram. No current breast complaints.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense which may lower the sensitivity of mammography.
There are scattered areas of fibroglandular density.
No dominant masses, architectural distortion or suspicious calcifications are seen in either breast.
At the 10 o'clock position of the right breast, 6 cm from the nipple, there is a 5 mm cluster of amorphous calcifications. Additional magnification views confirm these are suspicious and require biopsy.
The skin is normal. There are no other significant findings.
Bilateral axillary lymph nodes appear normal.
IMPRESSION:
BI-RADS 4: Suspicious cluster of microcalcifications in the right breast at 10:00, 6 cm from the nipple. Stereotactic biopsy is recommended for definitive diagnosis. The remainder of the bilateral mammogram is normal. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY.
History: Screening.
Comparison available dating from 01/2023.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM - FOR SCREENING. CLINICAL HISTORY: Patient is a 49-year-old female presenting for her screening mammogram. No history of breast cancer, no family history, no breast symptoms. Patient has extremely dense breasts clinically. TECHNIQUE: Standard CC and MLO views were obtained. COMPARISON: Comparison is made to the patient's prior mammogram dated 11/22/2023. FINDINGS: The breasts are extremely dense. There is no mammographic evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: 1. Extremely dense breast tissue. Mammography is less accurate in women with dense breasts. This means that cancers may be missed that would be seen with mammography in women with average density breasts. We strongly encourage you to talk to your doctor about your dense breast tissue. 2. No mammographic evidence of malignancy. 3. BI-RADS Category 2: Benign finding. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.
There are no suspicious masses or architectural distortions identified in either breast.
In the left breast, there is a new group of pleomorphic microcalcifications spanning approximately 10 mm at the 9:00 position, 4 cm from the nipple. This cluster is located more posteriorly, which may limit tissue acquisition on stereotactic biopsy. Additional angled views are recommended for further characterization. 
Additionally, a previously noted benign appearing partially calcified mass measuring 10 mm is stable in the right breast at the 11:00 position 6 cm from the nipple. This likely represents a fibroadenoma.
No suspicious axillary lymphadenopathy is identified.
IMPRESSION: 
BI-RADS 0: Incomplete. New group of pleomorphic microcalcifications spanning 10 mm in the left breast at 9:00, 4 cm from the nipple. Additional angled views are recommended for further characterization. If confirmed as suspicious, biopsy is recommended.
BI-RADS 2: Benign. Benign appearing mass, stable in the right breast. ➡ *Positive*
- MAMMOGRAM BILATERAL.
CLINICAL HISTORY: Annual screening mammogram. No breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
FINDINGS:
The breasts are almost entirely fatty. No suspicious masses, architectural distortions, or microcalcifications are seen in either breast. Stable bilateral benign calcifications are noted.  
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Recommend continued annual screening mammography in 12 months.
BIRADS 1 - NEGATIVE. ➡ *Negative*
- MAMMOGRAM REPORT
EXAMINATION: Bilateral digital screening mammogram
HISTORY: 55-year-old female presents for routine screening mammogram. No current breast complaints. Patient denies personal history of breast cancer. Positive family history of breast cancer in maternal aunt.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior studies from 04/15/2021, which are available for review.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
RIGHT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
LEFT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Stable benign-appearing lymph nodes are noted in the left axilla.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Specifically, no suspicious microcalcifications are identified.
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography.
RECOMMENDATION:
Annual screening mammogram is recommended. The patient is encouraged to perform regular breast self-exams and to promptly report any new breast symptoms to her healthcare provider.
ASSESSMENT CATEGORY: BIRADS 1 - Negative. ➡ *Negative*
- LEFT DIAGNOSTIC MAMMOGRAM
Clinical history: 42 y.o. female presents for a problem focused exam due to palpable findings in the left breast.
Comparison: Prior mammograms from 08/16/2022.
Technique: Standard CC & MLO views were obtained of each breast. In addition, spot compression and magnification views were performed to better evaluate the area in question.
Findings:
The breasts are heterogeneously dense which may reduce the sensitivity of mammography.
Left breast:
Palpable findings in the right breast corresponding to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Left breast, palpable findings, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended. ➡ *Negative*
- DIAGNOSTIC BILATERAL MAMMOGRAPHY.
Indication: Breast cancer screening. 
Technique: Standard cranial-caudal (CC) and mediolateral oblique (MLO) views obtained bilaterally in accordance with MQSA regulations. 
Comparison: None available.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
Findings: There are no dominant masses, suspicious calcifications, or other findings suggestive of malignancy in either breast.  
Impression:
BI-RADS Category 1: Negative. 
Management: Routine screening mammography is recommended in 1 year. ➡ *Not Stated*
- MAMMOGRAM REPORT
HISTORY: Screening mammogram. Patient has a history of right breast cyst aspiration 5 years ago. No family history of breast cancer.
FINDINGS:
Right breast: The breast is composed of scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast: The breast is composed of scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. No evidence of malignancy. Annual screening mammogram is recommended.
BIRADS 1
ADDENDUM: Comparison was made to outside screening mammograms from 2021, which were also negative. No significant interval change. ➡ *Negative*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.  
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy. The breasts are asymmetric in volume with the right breast larger than the left. No dominant masses, architectural distortion or suspicious calcifications are identified.
BIRADS-2.

ADDENDUM:
The patient contacted the office to report a palpable lump in the right breast upper outer quadrant. Targeted right breast ultrasound was performed which demonstrated a 1.2 cm oval circumscribed parallel hypoechoic mass at 10 o'clock, 6 cm from the nipple, likely representing a fibroadenoma. Short term follow-up ultrasound is recommended in 6 months to document stability. 
BIRADS-3 right breast. BIRADS-2 left breast. ➡ *Not Stated*
- FINDINGS:
Bilateral screening mammogram was obtained.
The breast tissue is heterogeneously dense, which could obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Stable postoperative changes and surgical clips noted in the right breast, likely from prior right breast biopsy.
No mammographic evidence of malignancy.
IMPRESSION:
BIRADS 2- Benign finding. Recommend routine screening in 12 months. ➡ *Not Stated*
- ULTRASOUND RIGHT BREAST

CLINICAL INDICATION: Right breast lump, upper outer quadrant.

TECHNIQUE: Targeted ultrasound was performed using a 12 MHz linear transducer.

COMPARISON: None available.

FINDINGS: The right breast demonstrates heterogeneously dense parenchyma. Targeted ultrasound to the area of palpable concern in the upper outer quadrant demonstrates normal fibroglandular tissue with no focal solid or cystic lesion. No suspicious mass, architectural distortion, or abnormal lymph nodes are identified.

IMPRESSION:
No sonographic evidence of malignancy in the right breast. Specifically, no abnormality is seen in the area of palpable concern in the right upper outer quadrant.

BIRADS 1 - Negative. 

ADDENDUM: Bilateral mammogram is recommended for further evaluation and to establish a baseline for future comparison, given the palpable finding and absence of prior mammographic imaging. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense (breast density B). No suspicious masses, architectural distortions or definite calcifications are identified in either breast.
IMPRESSION:
1. Negative screening mammogram.
2. Recommend routine screening in 1 year.
BIRADS ASSESSMENT CATEGORY: 1 - Negative.
ADDENDUM:
Heterogeneously dense breast tissue may obscure detection of small masses. As such, the patient is encouraged to follow up for routine screening examinations as recommended, and to report any new breast symptoms to her doctor. ➡ *Not Stated*
- MAMMOGRAM, BILATERAL.
HISTORY: Ms. XYZ presents for her annual screening mammogram. She reports no complaints or concerns at this time. LMP: Unknown. Prior surgical history: None.
FINDINGS:
Right breast:
Breast composition: The breast tissue is predominantly fibroglandular.
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion. 
Left breast:
Breast composition: The breast tissue is predominantly fibroglandular.
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion.
IMPRESSION:
1. Negative bilateral screening mammogram.  
2. No mammographic evidence of malignancy.
3. Recommend routine screening mammography in 1 year.
BIRADS 1 - Negative. ➡ *Negative*
- HISTORY: Patient presents for her annual screening mammogram. No prior history of breast cancer.

FINDINGS: Both breasts are composed of scattered fibroglandular densities. The parenchyma is relatively similar in density and distribution bilaterally. There is no mammographic evidence of a suspicious mass, architectural distortion, or abnormal calcifications.

IMPRESSION: 
1. Bilateral benign appearing mammograms. No suspicious mass, calcifications, or other findings to suggest malignancy.  
2. No significant interval change compared to prior exam from 1/25/2022.

ASSESSMENT: BI-RADS Category 1 - Negative. Recommend routine screening in 1 year.

ADDENDUM: Final assessment category remains 1 after additional evaluation. ➡ *Negative*
- NORMAL SCREENING MAMMOGRAM.
Compared to previous exam: Yes.
This is a screening mammogram.
Heterogeneously dense fibroglandular tissue.
There are no suspicious masses, architectural distortions, or areas of abnormal calcifications.
The skin is normal in appearance. No axillary lymphadenopathy.
IMPRESSION: 
BIRADS 2 - Benign findings. Recommend continued routine screening in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
The breasts are almost entirely fatty.

A few benign appearing calcifications are noted in the central right breast.

At the 12:00 position of the left breast, 6 cm from the nipple, there is a new group of pleomorphic microcalcifications spanning an area of approximately 1 cm. No associated mass, architectural distortion or skin changes.

The left axillary lymph nodes appear normal.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for further evaluation of the new group of pleomorphic microcalcifications at 12:00 position of the left breast, 6 cm from the nipple.

BI-RADS Category 2: Benign findings in the right breast. Recommend continued routine screening in 1 year.

ADDENDUM:
The patient returned for stereotactic biopsy of the group of microcalcifications in the left breast at 12:00, 6 cm from the nipple. Pathology results showed ductal carcinoma in situ, high grade, with no evidence of invasion.

The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- SCREENING MAMMOGRAM.
Indication: Screening.  
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
Findings: 
There are no dominant masses, architectural distortions or suspicious calcifications. 
Impression:
Negative screening mammogram. 
BIRADS Category 1 - Negative. Recommend routine screening in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
HISTORY: 45-year-old female with no prior mammograms for comparison.
FINDINGS:
The breasts are heterogeneously dense (BI-RADS Density C). 
RIGHT BREAST:
Right breast demonstrates scattered fibroglandular densities with no dominant mass, architectural distortion, or suspicious calcifications.
LEFT BREAST:
In the left breast, there are several clusters of pleomorphic microcalcifications scattered in the upper outer quadrant from 11:00 to 1:00, the largest cluster measuring approximately 8 mm. The calcifications are new and of concern given lack of comparison. No dominant mass or architectural distortion identified in the left breast.
IMPRESSION:
1. Heterogeneously dense breasts (BI-RADS Density C).
2. Suspicious pleomorphic microcalcifications in the upper outer left breast, BI-RADS 4B. Recommend stereotactic biopsy for further evaluation.
3. Negative right breast. BI-RADS 1.
RECOMMENDATIONS: 
1. Stereotactic core needle biopsy of the largest cluster of microcalcifications in the left breast at 12:00, 5 cm from the nipple.
2. Routine annual screening mammogram in 1 year, assuming biopsy results are benign. ➡ *Positive*
- MAMMOGRAM LEFT BREAST.
HISTORY: Screening.
COMPARISONS: 08/23/2022.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- MAMMOGRAPHIC REPORT
INDICATION: Annual screening mammogram. No clinical concerns.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to outside screening mammogram from April 2022.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. Specifically, no microcalcifications are identified.
Stable benign calcifications are noted in the right breast, likely representing oil cysts or fibrocystic changes. These are unchanged from prior exam.
No skin thickening or retraction. No lymphadenopathy.
IMPRESSION:
1. No mammographic evidence of malignancy. Specifically, no suspicious microcalcifications.
2. Breasts are heterogeneously dense, which lowers the sensitivity of mammography. Consider supplemental screening with breast ultrasound or MRI for increased breast cancer detection.
BIRADS 1 - Negative. Recommend continued annual screening mammography in 12 months.
ADDENDUM: 
Screening automated whole breast ultrasound performed today as adjunct screening exam for dense breast tissue. No sonographic abnormalities were identified in either breast. BIRADS 1 - Negative. ➡ *Negative*
- LEFT DIAGNOSTIC MAMMOGRAM
Clinical History: 74 y.o. female presents today for a diagnostic mammogram due to a palpable finding in the left breast.
Comparison: Prior mammograms from (outside institution), not available for review.
Technique: Standard CC & MLO views were obtained of each breast. In addition, spot compression and magnification views were performed to better evaluate the palpable findings in the right breast.
Findings:
The breasts are heterogeneously dense which may reduce the sensitivity of mammography.
Left breast:
Palpable finding corresponds to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Left breast, palpable finding, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended. ➡ *Negative*
- LEFT MAMMOGRAM AND ULTRASOUND
INDICATION: Annual screening. No current breast complaints.
FINDINGS:
The left breast is predominantly fatty with scattered areas of fibroglandular density. No significant masses, calcifications, or areas of architectural distortion.  
Ultrasound of the left breast was also performed. No solid or cystic masses are identified. The glandular tissue is normal in appearance.
IMPRESSION:  
Negative left screening mammogram and ultrasound. No evidence of malignancy.
BIRADS 1 - Normal. Routine screening in 1 year is recommended. ➡ *Negative*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes. 
Breast Density >= 75%: No.
Breast Density < 75%: Yes. 
OBSP to recall client in 2 years based on normal screening mammogram: Yes.
Benign findings: Stable bilateral scattered fibroglandular densities. No significant interval change compared to prior exams.
No suspicious masses, architectural distortions or suspicious calcifications.  
BIRADS-2. Routine screening recommended. ➡ *Not Stated*
- BILATERAL DIAGNOSTIC MAMMOGRAM
Clinical History: 54 y.o. female presents for a problem focused exam for evaluation of palpable findings in the left breast.
Comparison: Prior mammogram from (outside institution), not available for review.
Technique: Standard CC and MLO views were obtained of each breast. In addition, spot compression and magnification views were performed to better evaluate the palpable findings in the left breast
Findings:
The breasts are heterogeneously dense, which may reduce the sensitivity of mammography.
Right breast:
Scattered fibroglandular densities. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
Left breast:
Palpable findings in the left breast corresponding to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Right breast, negative. BI-RADS 1.
Left breast, palpable findings, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening.
Comparison available dating from 12/2022
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- TMIST FFDM MAMMOGRAM.
The patient is a participant in the TMIST trial conducted with approval from the Research Ethics Board. Following informed consent, she was randomized into the full field digital mammography only arm.
Breast density is estimated to be less than 75%.
No suspicious mammographic findings for malignancy. No microcalcifications are identified.
BIRADS category 2 - Benign findings. Routine screening recommended.
Comparison to previous exam: Yes, stable.
Recommend routine screening with OBSP in 1 year. ➡ *Not Stated*
- ULTRASOUND OF LEFT BREAST
CLINICAL HISTORY: 55 year old female with palpable lump in the left breast upper outer quadrant, family history includes mother with breast cancer.
COMPARISON: Compared to screening mammogram from 2 years prior.
FINDINGS:
Left breast ultrasound was performed. The breast is almost entirely fatty replaced. No discrete mass, architectural distortion, or suspicious calcifications are identified. Specifically, no abnormality is seen in the area of palpable concern in the left upper outer quadrant. Benign appearing lymph nodes are noted in the left axilla.
IMPRESSION:
No sonographic evidence of malignancy in the left breast. Recommend continued routine screening mammography. 
BIRADS 1 - Negative.
ADDENDUM: Patient contacted and informed of results. She will continue with yearly screening mammography. Clinical follow up of palpable area in 2 months is advised to document stability or resolution. ➡ *Negative*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.

The breast tissue is heterogeneously dense. No dominant masses, architectural distortion or suspicious calcifications are identified in either breast.

IMPRESSION: 
Negative mammogram. Routine screening is recommended in 2 years as per the Ontario Breast Screening Program.
BIRADS-2. ➡ *Not Stated*
- SCREENING MAMMOGRAM
INDICATION: Screening
COMPARISON: None.
TECHNIQUE: Bilateral digital screening mammogram including the craniocaudal (CC) and mediolateral oblique (MLO) views of each breast.
BREAST COMPOSITION: The breasts are heterogeneously dense, which may obscure small masses.
FINDINGS:
There are no suspicious masses, calcifications or other findings bilaterally. No significant change from prior exam.
IMPRESSION:
BILATERAL NEGATIVE SCREENING MAMMOGRAM. BI-RADS category 1. 
Recommend continued annual screening in 12 months per OBSP guidelines given patient age between 50-74 years old. ➡ *Not Stated*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to prior study of 6/15/2022.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
There are scattered areas of fibroglandular density bilaterally, stable compared to prior exam.
Benign-appearing calcifications are noted bilaterally, also stable.
IMPRESSION:
1. No mammographic evidence of malignancy. No suspicious microcalcifications.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography.
BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- IMPRESSION:
There are new pleomorphic calcifications in the upper outer quadrant of the right breast at the 10:00 position, 7 cm from the nipple. These calcifications are suspicious for ductal carcinoma in situ (DCIS). Additional views confirm these calcifications. Ultrasound demonstrates a vague area of mixed echogenicity without a definite mass. There are no other suspicious findings in either breast.

RECOMMENDATION:
Right breast: BI-RADS 4: Suspicious abnormality. Stereotactic core biopsy is recommended for the new group of pleomorphic calcifications in the right breast at 10:00, 7 cm from the nipple. 

Left breast: BI-RADS 2: Benign finding. Recommend routine screening in 1 year.

INDICATION:  
Patient called back for additional imaging to assess new microcalcifications seen on her screening mammogram.

FINDINGS: 
COMPARISON: Outside screening mammogram dated 1/10/2022.

RIGHT BREAST:
There is a new group of pleomorphic microcalcifications in the upper outer quadrant at 10:00, 7 cm from the nipple on a background of scattered fibroglandular densities. Additional spot compression and magnification views in the CC and MLO projections confirm these calcifications. Targeted ultrasound demonstrates a vague area of mixed echogenicity without a definite mass in this area. No other dominant masses, areas of architectural distortion, or suspicious calcifications.

LEFT BREAST:  
The left breast is heterogeneously dense. No dominant masses, areas of architectural distortion, or suspicious calcifications. Stable benign calcifications.

ADDENDUM:
The patient returned for stereotactic biopsy of the group of pleomorphic calcifications in the right breast at the 10:00 position, 7 cm from the nipple. Post-biopsy mammograms demonstrate that the calcifications were sampled. Specimen radiograph confirms calcifications. The patient tolerated the procedure well. Pathology results are pending. The patient will be informed of results when available. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
CLINICAL INDICATION: Annual screening mammogram. No clinical concerns.
COMPARISON: Screening mammogram from 05/15/2022. 
TECHNIQUE: Standard CC and MLO views of both breasts.
FINDINGS:
RIGHT BREAST: 
The breast is heterogeneously dense (C). There are no dominant masses, architectural distortion, or suspicious calcifications. Stable benign appearing calcifications in the upper outer quadrant. No significant change from prior exam.
LEFT BREAST:
The breast is heterogeneously dense (C). There are no dominant masses, architectural distortion, or suspicious calcifications. Stable postsurgical changes in the upper outer quadrant consistent with prior history of lumpectomy. No significant change from prior exam.
IMPRESSION:
1. No mammographic evidence of malignancy. Heterogeneously dense breasts which may obscure detection of small masses.
2. Stable postsurgical changes left breast.
BIRADS CATEGORY 2: Benign findings. Recommend continued annual screening with mammography in 1 year. ➡ *Negative*
- CLINICAL HISTORY: Annual screening mammogram. No new complaints. Prior history of right breast cancer treated with lumpectomy and radiation.

FINDINGS:
RIGHT BREAST: 
There are scattered areas of fibroglandular density. No suspicious masses, architectural distortion or suspicious calcifications. 

LEFT BREAST:
There are scattered areas of fibroglandular density. No suspicious masses, architectural distortion or suspicious calcifications.  

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Annual screening mammogram is recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

INDICATION:  Routine screening.

TECHNIQUE:  Standard CC and MLO views of both breasts were obtained.

COMPARISON:  Comparison is made to prior exam dated 10/15/2020.  

FINDINGS:  
The breasts are heterogeneously dense. This may lower the sensitivity of mammography. 

There are no dominant masses, suspicious calcifications, or concerning architectural distortion in either breast.

Stable bilateral benign-appearing calcifications are noted.

There are no significant changes compared to prior exam.

IMPRESSION:
No mammographic evidence of malignancy. 
Recommend routine screening in 1 year.

BI-RADS Category 2: Benign finding. 

ADDENDUM:
THIS EXAM MET MQSA STANDARDS. ➡ *Not Stated*
- RIGHT BREAST ULTRASOUND
Clinical History: Increased density in the right breast on screening mammogram performed on 04/04/2025. Patient denies any palpable abnormality.
Comparison: None.
Technique: Real-time sonography of the right breast was performed using a high-frequency linear-array transducer. Targeted imaging of the palpable lump in the right breast was performed, as well as evaluation of the area of increased density seen on the recent mammogram.
Findings:
The area of increased density seen on the recent mammogram correlates to a region of overlapping normal fibroglandular tissue. No suspicious masses are identified sonographically. No abnormal axillary lymph nodes are identified.
IMPRESSION:
Increased density in the right breast on recent mammogram correlates to a region of overlapping normal fibroglandular tissue. No suspicious masses are identified sonographically.
BI-RADS Category 2: Benign findings. ➡ *Not Stated*
- ULTRASOUND BILATERAL BREASTS

INDICATION: 42-year-old female with bilateral palpable lumps, for diagnostic evaluation.

COMPARISON: None available.

FINDINGS:
Right breast: At the 9:00 position 5 cm from the nipple, there is a 1.0 x 0.7 x 0.6 cm oval, circumscribed, hypoechoic solid mass, parallel in orientation. This corresponds to the palpable finding. No worrisome features identified. 

Left breast: At the 2:00 position 3 cm from the nipple, there is a 0.8 x 0.5 x 0.4 cm oval, circumscribed, hypoechoic solid mass, parallel in orientation. This corresponds to the palpable finding. No worrisome features identified.

No axillary lymphadenopathy.

IMPRESSION:
1. Bilateral circumscribed solid masses, likely fibroadenomas, BI-RADS 2. 
2. No mammographically or sonographically evident microcalcifications.
3. Recommend routine screening mammography as the patient is currently asymptomatic. 
4. Clinical follow-up for the palpable findings as needed.

BI-RADS CATEGORY 2 ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL INDICATION: The patient presents for routine breast cancer screening.
TECHNIQUE: Standard craniocaudal and mediolateral oblique views of the bilateral breasts were obtained using Hologic digital breast tomosynthesis system.
COMPARISON: None.
FINDINGS:
Both breasts demonstrate heterogeneously dense composition, which may decrease the sensitivity of mammography. Scattered fibroglandular elements are present bilaterally.
Right Breast:
No suspicious masses, architectural distortion or abnormal calcifications. No abnormalities in the right axilla, skin or nipple.
Left Breast:
No suspicious masses or architectural distortion. In the upper outer quadrant, there is a new cluster of pleomorphic microcalcifications spanning approximately 1 cm, located 8 cm from the nipple at the 2:00 position. No other abnormalities in the left axilla, skin or nipple.
IMPRESSION:
BI-RADS Category 0: Incomplete, recall for additional imaging evaluation of the left breast.
There is a new cluster of pleomorphic microcalcifications in the left upper outer quadrant, which requires further evaluation with magnification views. Per BI-RADS, a 0 is assigned for incomplete assessment and recommendation for recall examination.
ADDENDUM:
The patient returned for magnification views of the left upper outer breast. Spot compression magnification views confirm the presence of a cluster of pleomorphic microcalcifications in the area of concern. The distribution is segmental and spans approximately 1.2 cm.
FINAL IMPRESSION:
BI-RADS Category 4B: Suspicious abnormality, biopsy should be considered.
The pleomorphic segmental microcalcifications in the left upper outer breast are suspicious for ductal carcinoma in situ (DCIS). Stereotactic core biopsy is recommended for definitive diagnosis. The patient should follow up with results as directed by the referring clinician. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms were obtained. The breasts are heterogeneously dense, which may obscure detection of small masses.
In the right breast, there are no dominant masses, areas of architectural distortion, or suspicious calcifications.
In the left breast at the 10:00 position 6 cm from the nipple, there is a new group of fine pleomorphic calcifications spanning approximately 8 mm. Additional magnification views confirm these are suspicious and a stereotactic core biopsy is recommended for further evaluation. The remainder of the left breast is unremarkable.

IMPRESSION:
LEFT BREAST, NEW GROUP OF SUSPICIOUS CALCIFICATIONS AT THE 10:00 POSITION 6 CM FROM THE NIPPLE FOR WHICH A STEREOTACTIC CORE BIOPSY IS RECOMMENDED. BIRADS 4B.
RIGHT BREAST, NEGATIVE. BIRADS 1. ➡ *Positive*
- FINDINGS:
Comparison is made to screening mammogram from 10/23/2022 at [**Hospital 91**]. The breasts are heterogeneously dense. This may lower the sensitivity of mammography. 

RIGHT BREAST:
There is a new 8 mm poorly defined mass at the 10:00 position, posterior depth, approximately 2 cm from the nipple. This corresponds to the patient's area of concern. No other dominant masses, suspicious calcifications, or areas of architectural distortion.

LEFT BREAST: 
There are no dominant masses, suspicious calcifications, or areas of architectural distortion.

IMPRESSION: 
Right breast: New poorly defined mass at 10:00, posterior depth, measuring 8 mm. Recommend direct ultrasound correlation, further characterization and biopsy. BIRADS 4B - Suspicious finding.

Left breast: No mammographic evidence of malignancy. BIRADS 1 - Negative. ➡ *Negative*
- EXAM: BILATERAL DIGITAL MAMMOGRAPHY

COMPARISON: None.

INDICATION: Annual Screening.

FINDINGS: The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no suspicious masses, architectural distortion, or asymmetries in either breast.

In the left breast at 12:00, 7 cm from the nipple, there is a new group of fine pleomorphic calcifications spanning approximately 8 mm. These are suspicious for malignancy.

No other suspicious calcifications are seen in either breast.

No skin thickening or retraction. No axillary lymphadenopathy.

IMPRESSION:

BI-RADS 4: Suspicious group of pleomorphic calcifications in the left breast at 12:00, 7 cm from the nipple. Stereotactic core biopsy is recommended for further evaluation. 

The remainder of the mammogram is benign, BI-RADS 2, with no other suspicious findings.

ADDENDUM: The patient proceeded with stereotactic core biopsy of the left breast calcifications, which demonstrated ductal carcinoma in situ, intermediate grade with comedonecrosis. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS
History: Patient due for high risk screening.
Compared to previous mammograms from November 2020 available for comparison.
FINDINGS:
The breasts are heterogeneously dense. 
Benign calcifications are again noted in the right breast. No suspicious findings right breast.
Magnification views demonstrate a new 1 cm cluster of pleomorphic calcifications at 12:00 in the left breast, 3 cm from the nipple. The rest of the left breast is unremarkable. 
SUMMARY:
New suspicious cluster of calcifications upper central left breast. BI-RADS 4B. Stereotactic biopsy recommended. No suspicious mammographic findings right breast. BI-RADS 2.
ADDENDUM: Stereotactic biopsy performed same day of the 12:00 left breast calcifications. Pathology pending. Clip deployed. Post-biopsy mammograms demonstrate biopsy cavity and clip in excellent position. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of each breast were obtained.

FINDINGS:

Both breasts are heterogeneously dense, which may obscure detection of small masses. 

In the right breast, there are no dominant masses, areas of architectural distortion, or suspicious calcifications.

In the left breast, there is a new group of fine pleomorphic calcifications in a segmental distribution at the 10:00 position, middle depth, spanning an area of approximately 8 mm. No associated mass or architectural distortion is evident. 

The remainder of the left breast is unremarkable. No suspicious masses, architectural distortion, or additional calcifications are identified.

No abnormal axillary lymph nodes are seen.

IMPRESSION:

BI-RADS Category 4: Suspicious abnormality in the left breast. Biopsy should be considered. 

BI-RADS Category 1: Negative in the right breast.

RECOMMENDATION: Left breast biopsy is recommended for the new group of suspicious calcifications at 10:00. Please send a requisition to the department to arrange this procedure. ➡ *Positive*
- FINDINGS:
Scattered fibroglandular densities.

LEFT BREAST:
There is a 1.2 cm irregular mass at the 10 o'clock position, 6 cm from the nipple. Associated architectural distortion and skin thickening are present. Additionally, pleomorphic calcifications are noted extending from the mass.

RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
Highly suspicious mass left breast with associated calcifications, architectural distortion, and skin thickening, suspicious for malignancy. Recommend biopsy. 
BI-RADS 5 - Left Breast
BI-RADS 1 - Right Breast ➡ *Positive*
- SCREENING MAMMOGRAM.
COMPARISON: Screening mammogram of April 15, 2020.
TECHNIQUE: Full field digital mammography including craniocaudal and mediolateral oblique views of both breasts.
FINDINGS: 
The breasts are heterogeneously dense which could obscure detection of small masses. 
There are no dominant masses, architectural distortion or suspicious calcifications.
Stable benign appearing calcifications are noted in the left breast, likely representing fibrocystic changes.
No significant interval change compared to prior exam.
IMPRESSION:
No mammographic evidence of malignancy. 
BI-RADS Category 2: Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- DIAGNOSTIC MAMMOGRAPHY

MEDICAL INDICATION: Family history of breast cancer

TECHNIQUE: Diagnostic digital mammography, bilateral
COMPARISON: None

FINDINGS:

The breasts are heterogeneously dense.

A 6 mm cluster of amorphous microcalcifications is identified in the left breast upper outer quadrant at approximately 2:00, 7 cm from the nipple.  

The remainder of the left breast is unremarkable.

The right breast is unremarkable.

No dominant masses, architectural distortion or suspicious calcifications. No skin thickening or retraction.

IMPRESSION:

Cluster of amorphous microcalcifications left breast upper outer quadrant; suspicious for early carcinoma. Recommend biopsy. BI-RADS 4B.

The right breast is negative. BI-RADS 1.

ADDENDUM: Stereotactic core biopsy was performed of the 2:00 microcalcifications, left breast. PATHOLOGY: Ductal carcinoma in-situ, intermediate grade with microcalcifications, ER+, PR+. See full pathology report. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM

CLINICAL HISTORY: Palpable mass, right breast.

TECHNIQUE: Standard CC and MLO views of the bilateral breasts were obtained.

FINDINGS:

The breasts are heterogeneously dense. There is a 10 mm area of architectural distortion with associated clustered linear and branching calcifications in the right breast at 2 o'clock, 3 cm from the nipple. No masses are definitely seen. No axillary adenopathy.

IMPRESSION:

Architectural distortion and clustered calcifications in the right breast as described. Highly suspicious for malignancy. Recommend biopsy.

BI-RADS: 4c (Highly suspicious for malignancy) ➡ *Positive*
- MAMMOGRAM AND LEFT BREAST ULTRASOUND
INDICATION: 42-year-old female presents with left breast pain. History of right breast cancer, status post mastectomy in 2020.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Targeted ultrasound was performed at the 10 o'clock position. 
COMPARISON: Mammogram from 12/15/2023 at Mercy Hospital. 
FINDINGS:
Right mastectomy changes noted. 
The left breast is heterogeneously dense, which may limit sensitivity of mammography. 
There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast ultrasound reveals normal fibroglandular tissue at the area of complaint, 10 o'clock position 4 cm from the nipple. No solid or cystic lesions are identified. 
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend annual screening mammography. 
2. Negative targeted ultrasound of the left breast 10 o'clock position. 
3. Heterogeneously dense breast tissue, which can obscure small masses. Consider supplemental screening with breast MRI if clinically indicated based on risk factors.
BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

TECHNIQUE: CC and MLO views bilaterally.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in the right breast.

In the LEFT breast, there is a new group of fine pleomorphic microcalcifications in a segmental distribution at the 9:00 position, middle depth, spanning an area of approximately 2 cm. No associated mass, distortion or skin changes are identified.

The left axilla appears unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation.
Suspicious grouped fine pleomorphic microcalcifications in the 9:00 region of the left breast, not seen on the prior exam. Additional magnification views are recommended for further characterization.

RECOMMENDATION:
Magnification views of the left breast.

ADDENDUM - LEFT DIAGNOSTIC MAMMOGRAM WITH SPOT COMPRESSION MAGNIFICATION VIEWS:
Additional spot compression magnification views of the left breast in the CC and ML projections demonstrate the segmentally distributed fine pleomorphic calcifications at 9:00. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended for definitive diagnosis.

FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious - Biopsy should be considered. ➡ *Positive*
- REPORT: "LEFT MAMMOGRAM AND ULTRASOUND.
HISTORY: Screening mammogram. 54-year-old female patient. No known breast issues. Family history of breast cancer in mother.
Comparison made to prior imaging from 2 years ago.
FINDINGS: The left breast tissue is predominantly fatty with scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious microcalcifications are identified. There are no significant changes compared to prior mammogram.
Targeted left breast ultrasound was also performed. No sonographic abnormality is identified. The visualized left axilla appears unremarkable.
IMPRESSION: No mammographic or sonographic evidence of malignancy in the left breast. Recommend continued annual screening mammography.
BIRADS 1 - Negative." ➡ *Negative*
- BILATERAL DIGITAL DIAGNOSTIC MAMMOGRAM WITH TOMOSYNTHESIS

INDICATION: Annual screening.

TECHNIQUE: CC and MLO views of both breasts were obtained. Tomosynthesis was performed.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a group of amorphous calcifications in the upper outer quadrant, approximately 6 cm from the nipple, spanning an area of about 10 mm. No associated mass, architectural distortion, or skin thickening is identified. 

No axillary, supraclavicular, or internal mammary adenopathy.

IMPRESSION:
1. Heterogeneously dense breasts.
2. Suspicious grouped amorphous calcifications in the left breast at 2 o'clock, 6 cm from the nipple, BI-RADS 4B. Stereotactic biopsy is recommended.
3. Benign right breast, BI-RADS 2.

ADDENDUM (post stereotactic biopsy left breast):
Stereotactic biopsy of the group of amorphous calcifications in the left breast at 2 o'clock was performed. Specimen radiograph confirmed calcifications. Pathology revealed sclerosing adenosis with microcalcifications, benign. Recommend routine screening mammogram in 1 year. Final BI-RADS assessment of the left breast: BI-RADS 3, probably benign finding. ➡ *Positive*
- LEFT MAMMOGRAM AND ULTRASOUND
HISTORY: 54-year-old female with a family history of breast cancer, maternal aunt and grandmother, presents for routine screening mammogram. No palpable masses or breast complaints.
FINDINGS: The breast tissue is heterogeneously dense, which could obscure detection of small masses. No dominant masses, architectural distortion, or suspicious calcifications are seen in either breast. 
Targeted ultrasound was performed in the left breast at the 2 o'clock position. No sonographic abnormality or suspicious lesions were identified.
Stable post-surgical changes are noted in the right breast from prior lumpectomy.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography and clinical breast exams.
2. Heterogeneously dense breast tissue, which lowers the sensitivity of mammography. Consider supplemental screening with breast MRI or ultrasound as clinically indicated.
3. No suspicious lesions seen on targeted left breast ultrasound at 2 o'clock position. 
4. Stable right breast post-surgical changes.
BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS: The breasts are predominantly fatty with scattered fibroglandular densities. There are no suspicious masses, architectural distortions, or abnormal calcifications bilaterally. The breast tissue is symmetric. Stable post-surgical changes are noted in the right breast consistent with prior history of lumpectomy. No mammographic evidence of malignancy.

IMPRESSION:
1. No mammographic evidence of malignancy, BI-RADS 1. Recommend annual screening mammography in 12 months.

2. Stable right breast post-surgical changes.

ADDENDUM: Comparison was made to outside mammograms dated 5/15/2022, which are similar. No prior mammograms at this facility. ➡ *Negative*
- FINDINGS:

Bilateral diagnostic mammographic views demonstrate scattered areas of fibroglandular density. There are no dominant masses, areas of architectural distortion or suspicious calcifications in the right breast. 

In the left breast at the 12:00 axis approximately 5 cm from the nipple, there is a group of pleomorphic microcalcifications spanning an area of approximately 1 cm. Spot compression magnification views confirm the presence of a group of heterogeneous calcifications, varying in size and shape. No associated mass or architectural distortion is identified.

The remainder of the left breast is unremarkable. No suspicious masses, areas of architectural distortion or additional groups of suspicious microcalcifications are identified. 

IMPRESSION:

Right breast: No mammographic evidence of malignancy. BI-RADS 1 - Negative.

Left breast: Group of pleomorphic microcalcifications at the 12:00 axis, left breast, suspicious for malignancy. BI-RADS 4C. Stereotactic biopsy is recommended for further evaluation. 

ADDENDUM: The patient proceeded with stereotactic biopsy of the group of microcalcifications in the left breast. Pathology results demonstrate ductal carcinoma in situ, intermediate nuclear grade with necrosis. ➡ *Positive*
- MAMMOGRAM REPORT:
HISTORY: Annual screening mammogram. Patient denies any breast complaints.
FINDINGS:
Bilateral breast ultrasound performed.
Mammogram reveals heterogeneously dense breast tissue.
There are no dominant masses, architectural distortion or suspicious calcifications.
Stable postoperative changes noted in the upper outer quadrant of the right breast consistent with prior lumpectomy.
ULTRASOUND:
Ultrasound was performed targeted to the site of prior right breast lumpectomy. No sonographic abnormality or recurrence is identified.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening.
2. Stable postoperative changes right breast, no sonographic evidence of recurrence.
BIRADS 2, benign. ➡ *Negative*
- BILATERAL MAMMOGRAM - SCREENING. CLINICAL HISTORY: Patient is a 41-year-old female presenting for her screening mammogram. Patient reports no personal history of breast cancer. Mother diagnosed with breast cancer at age 60. TECHNIQUE: Standard CC and MLO views were obtained. COMPARISON: Comparison is made to the patient's prior mammogram dated 11/14/2023. FINDINGS: The breasts are heterogeneously dense. There is no mammographic evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
The breasts are heterogeneously dense.
On the right, there is no mammographic evidence of malignancy.  The breast is almost entirely fatty.  No masses, suspicious calcifications, architectural distortion or skin thickening are identified.
There are scattered benign appearing calcifications in the left breast. Particularly in the upper outer quadrant, a new cluster of pleomorphic microcalcifications is identified spanning a region of approximately 1.5 cm. This is a suspicious finding, highly concerning for malignancy such as ductal carcinoma in situ.  Otherwise, no spiculated mass, asymmetry or architectural distortion is seen in the left breast.
Both nipples are within normal limits.  There are no skin lesions or lymphadenopathy evident.
IMPRESSION:
BI-RADS 2: Benign finding right breast. Recommend routine screening in 1 year.
BI-RADS 4: Suspicious abnormality left breast - new cluster of pleomorphic microcalcifications in the upper outer quadrant. Biopsy is recommended.
ADDENDUM: Stereotactic biopsy of microcalcifications at left 2:00, 6 cm from nipple performed today. Specimen radiograph confirms presence of microcalcifications. Pathology results pending. ➡ *Positive*
- MAMMOGRAM, BILATERAL.

INDICATION: Family history of breast cancer. No palpable lumps.

COMPARISON: None.

FINDINGS:

Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right Breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left Breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:

1. Heterogeneously dense breasts, which may lower the sensitivity of mammography.

2. No mammographic evidence of malignancy in either breast.

RECOMMENDATION:

Recommend routine annual screening mammography and clinical breast exams. Consider supplemental screening with breast ultrasound or MRI due to patient's dense breast tissue and family history of breast cancer.

BIRADS Category 1 - Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of each breast were obtained.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts.

RECOMMENDATION:
Routine screening mammography in 1 year.

BIRADS Category 1: Negative. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to prior examination from [**2022-03-14**], the breasts are unchanged. The breasts are heterogeneously dense. There are scattered areas of fibroglandular density. There are no dominant masses, areas of architectural distortion or suspicious calcifications. Skin is normal in thickness. No evidence of skin retraction or thickening. Benign calcifications are present bilaterally.
IMPRESSION: 
BIRADS 2- Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: Compared to prior exam dated 1/15/2021.
FINDINGS: 
The breasts are heterogeneously dense, which may obscure small masses.
There are no masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
BIRADS 2-Benign findings. Recommend routine screening in 1 year.
ADDENDUM: No suspicious microcalcifications are identified in either breast. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
Ms. X presented today for her annual screening mammogram as part of the Tomosynthesis Mammographic Imaging Screening Trial. She was consented and randomized to the digital mammography only arm per study protocol.  
Comparison made to prior study.
Breast composition falls into the scattered fibroglandular density category, indicating tissue density below 75%.
Routine annual screening recommended as per OBSP guidelines.
The breasts appear within normal limits. No suspicious mammographic abnormalities evident.
Final Assessment: BI-RADS Category 1 - Negative. ➡ *Not Stated*
- FINDINGS: Both breasts demonstrate scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. No mammographic evidence of malignancy.
3. Recommend continued annual screening mammography in 12 months.

BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS:
The breasts are heterogeneously dense. 
In the left breast at 11:00 5 cm from the nipple there is a group of pleomorphic calcifications in a segmental distribution measuring 8 mm. Additional magnification views confirm these are suspicious for malignancy.
At 1:00 location 3 cm from the nipple there is an irregular spiculated mass with associated architectural distortion measuring 12 x 8 mm. 
Ultrasound of the left breast in the area of mammographic concern at 11:00 demonstrates a 6 mm irregular solid mass with angular margins and shadowing. This corresponds to the mammographic calcifications. Color Doppler shows internal vascularity.
At the 1:00 location ultrasound shows the mammographic mass to be a 10 x 7 x 8 mm irregular hypoechoic mass with indistinct margins, posterior shadowing and internal vascularity.
Ultrasound of the left axilla reveals an enlarged 15 mm cortically thickened lymph node with loss of the fatty hilum.
The right breast is unremarkable. No dominant masses, architectural distortion or suspicious calcifications.

IMPRESSION:
Highly suspicious group of segmental pleomorphic calcifications left breast 11:00 location with sonographic correlate. Suspicious mass left breast 1:00. Abnormal left axillary lymph node. Recommend biopsy. BI-RADS 5.
Negative right breast. BI-RADS 1. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram. No palpable findings. No prior breast surgery or intervention.

COMPARISON: None.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

In the right breast at the 12:00 position 3 cm from the nipple, there is a small group of amorphous calcifications in a segmental distribution measuring approximately 5 mm. Additional magnification views confirm these calcifications.

The left breast is unremarkable.

IMPRESSION:

BI-RADS Category 0: Incomplete. Need additional imaging evaluation and/or prior mammograms for comparison.
Small group of amorphous calcifications right breast, indeterminate. Additional magnification views recommended for further assessment.

ADDENDUM:
Right breast magnification views in the CC and MLO projections demonstrate the group of amorphous calcifications at 12:00. There are approximately 8-10 calcifications in a segmental distribution spanning 5 mm. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS).

FINAL ASSESSMENT:
BI-RADS Category 4B: Moderate suspicion for malignancy. Biopsy should be considered.

RECOMMENDATION: Stereotactic core biopsy right breast calcifications. ➡ *Positive*
- BILATERAL MAMMOGRAM WITH TOMOSYNTHESIS AND TARGETED ULTRASOUND OF THE RIGHT BREAST

CLINICAL HISTORY: 58-year-old female with new palpable lump in the right breast upper outer quadrant.

COMPARISON: Prior mammograms from 2 years ago.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There is a new 12 mm irregular mass at the 10 o'clock position of the right breast, approximately 6 cm from the nipple, corresponding to the palpable abnormality described by the patient. Targeted ultrasound demonstrates an irregular hypoechoic mass with angular margins and posterior acoustic shadowing, highly suspicious for malignancy.

Additionally, there is a new group of fine pleomorphic calcifications spanning approximately 8 mm at the 9 o'clock position of the right breast, 4 cm from the nipple. These calcifications are highly concerning for malignancy.

The left breast demonstrates stable fibroglandular tissue and a few scattered benign-appearing calcifications. No suspicious masses, architectural distortion, or suspicious calcifications are identified in the left breast.

IMPRESSION:
1. Highly suspicious 12 mm mass at 10 o'clock position of the right breast, corresponding to palpable finding. Recommend biopsy. BI-RADS 5.

2. Highly suspicious group of pleomorphic calcifications at 9 o'clock position of the right breast. Recommend stereotactic biopsy. BI-RADS 4C.

3. Stable left breast. BI-RADS 2. ➡ *Positive*
- TMIST FFDM MAMMOGRAM ARM:
This is a full field digital mammography report from the Tomosynthesis Mammographic Imaging Screening Trial (TMIST). The patient was randomly assigned to the mammography only arm after providing informed consent.  
FINDINGS:
The breasts are heterogeneously dense. There are no dominant masses, architectural distortion or suspicious calcifications. 
IMPRESSION:
Negative digital mammogram. Routine yearly screening is recommended. BI-RADS 1.
ADDENDUM:
No microcalcifications are described in this report. 
TMIST TRIAL DATA:
Compared to previous exam: Yes
Breast density > 75% fibroglandular: No
Breast density < 75% fibroglandular: Yes 
Recommend routine OBSP screening in 1 year: Yes ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM.

INDICATION: Screening. 

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available.

FINDINGS: 
The breast tissue is predominantly fatty.

There are no dominant masses, suspicious calcifications, or other findings suspicious for malignancy in either breast. 

IMPRESSION:
BIRADS 2 - Benign mammogram. Recommend routine screening in 1 year. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM - NORMAL. 
This patient is enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), approved by the Research Ethics Board. After providing informed consent, she was randomized to the full field digital mammography (FFDM) arm.
Comparison to prior exams: Available.
Breast density: Scattered fibroglandular density (estimated 51-75%).
Routine 1-year OBSP recall is recommended.
There are no suspicious mammographic abnormalities.
No mention of microcalcifications.
BIRADS Category 1 - Negative. ➡ *Not Stated*
- FINDINGS:
Bilateral digital screening mammogram was performed.
The breast parenchymal density is heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered areas of fibroglandular density bilaterally.
No dominant mass, area of architectural distortion or abnormal calcifications are seen in the right breast. 
In the left breast, there are scattered fibroglandular densities. No dominant mass or architectural distortion. There is a small cluster of pleomorphic microcalcifications identified in the upper outer quadrant, approximately 8 cm from the nipple. Further evaluation with magnification compression views is recommended.
No mammographic evidence of malignancy in the right breast. 
There are indeterminate microcalcifications in the left breast, further workup recommended.

IMPRESSION:
BI-RADS Category 0: Incomplete, further imaging evaluation needed. Small cluster of pleomorphic microcalcifications in the left upper outer quadrant, magnification compression views recommended for further characterization. 
BI-RADS Category 1 for the right breast: Negative.

ADDENDUM:
Magnification views of the left breast upper outer quadrant were performed. The previously identified cluster of microcalcifications are pleomorphic in morphology and have a segmental distribution. Biopsy is recommended for tissue diagnosis.
Final Assessment:
BI-RADS Category 4 for the left breast: Suspicious abnormality, biopsy should be considered. 
BI-RADS Category 1 for the right breast: Negative. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS
INDICATION: Annual screening mammogram
COMPARISON: None
FINDINGS:
Both breasts show heterogeneously dense parenchymal pattern, limiting sensitivity of mammography.
Right Breast:
No dominant mass, architectural distortion or suspicious calcifications.
Left Breast:
No dominant mass or architectural distortion.
Scattered amorphous microcalcifications are noted in the upper outer quadrant, spanning an area of approximately 2 cm. No associated mass is identified.
Magnification views of the area of microcalcifications in the left breast are obtained, confirming presence of amorphous calcifications in a segmental distribution at 2 o'clock position, 6 cm from the nipple.
IMPRESSION:
1. Heterogeneously dense breasts, limiting sensitivity of mammography. 
2. Amorphous segmental microcalcifications in the left breast at 2 o'clock position, 6 cm from the nipple, suspicious for DCIS. Additional imaging evaluation with breast MRI is recommended for further evaluation. 
3. BI-RADS Category 4B: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended.
4. BI-RADS Category 2: Benign finding in the right breast.

ADDENDUM: Stereotactic biopsy of the left breast microcalcifications performed, demonstrating ductal carcinoma in-situ, intermediate nuclear grade, cribriform type. Wide local excision is recommended. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM

CLINICAL HISTORY: Abnormal screening mammogram. Patient has a history of left breast cancer, status post lumpectomy 5 years ago.

TECHNIQUE: Standard CC and MLO views of the bilateral breasts were obtained. Spot compression magnification views were performed over the area in question in the left breast.

FINDINGS:

The breasts are heterogeneously dense. There is a 5 mm cluster of pleomorphic microcalcifications in the left breast at 10 o'clock, 2 cm from the nipple. This was biopsied under stereotactic guidance.

IMPRESSION:

1. Cluster of pleomorphic microcalcifications in the left breast at the 10 o'clock position, 2 cm from the nipple, status post stereotactic biopsy. Please see biopsy results.

BI-RADS: 0 (Incomplete). ➡ *Positive*
- BREAST MAGNETIC RESONANCE IMAGING

Clinical History: Patient with history of right breast cancer, status post lumpectomy and radiation therapy 2 years ago, now presents with new microcalcifications on screening mammogram in the right breast.

Comparison: Prior mammograms.

Findings:
Right Breast: There is a small cluster of punctate enhancing foci in the right breast at the 3 o'clock position, corresponding to the microcalcifications seen on mammography. These foci demonstrate rapid uptake and washout kinetics, suggestive of suspicious enhancement. No other suspicious masses or areas of enhancement are seen in either breast.

Impression:
The enhancing foci in the right breast corresponding to the new microcalcifications are suspicious for recurrent malignancy. Correlation with mammographic findings and biopsy are recommended.

BI-RADS Category: 4 (Suspicious for Malignancy) ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS

CLINICAL HISTORY: Annual screening mammogram.  

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.  

There are no suspicious masses, architectural distortion or suspicious calcifications in either breast.

In the right upper outer quadrant there is a new cluster of pleomorphic calcifications spanning 9 mm at 10:00, 5 cm from the nipple. Additional magnification views confirm these are suspicious and new since the prior mammogram. 

Stable bilateral circumscribed masses consistent with benign fibroadenomas, largest measuring 6 mm in the right breast at 9:00.  

IMPRESSION: 
BI-RADS 4: Suspicious cluster of microcalcifications in the right breast at 10:00. Stereotactic biopsy is recommended. The patient was informed and agrees with this plan. Please send a stereotactic biopsy requisition.

ADDENDUM: Stereotactic biopsy of the right breast calcifications at 10:00 demonstrated ductal carcinoma in situ, intermediate grade with necrosis.  

ADDENDUM 2:  Specimen radiograph confirmed the presence of calcifications. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
NAME: Smith, Jane 
EXAMINATION: Bilateral Screening Mammogram 
CLINICAL HISTORY: Annual screening mammogram.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: 03/15/2021
FINDINGS:
Right Breast:
Tissue composition is heterogeneously dense. This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications. Stable benign calcifications in the upper outer quadrant. 
Left Breast: 
Tissue composition is heterogeneously dense. This may lower the sensitivity of mammography. No dominant mass, architectural distortion or suspicious calcifications.  
Stable benign calcifications scattered throughout the breast. 
IMPRESSION:
BI-RADS Category 1: Negative bilateral screening mammogram. 
Heterogeneously dense breast tissue bilaterally. 
RECOMMENDATION:  
Recommend routine yearly screening mammography.
Continue monthly self breast exams. Return promptly if palpable abnormality discovered. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.  No complaints.

FINDINGS:
There are no suspicious masses, distortions or other findings in either breast.

There is an 8 mm cluster of microcalcifications in the upper outer left breast at approximately 1 o'clock, 5 cm from the nipple.  These are new compared to the prior study.

The breast tissue is heterogeneously dense, which may obscure detection of small masses.

IMPRESSION:
BI-RADS Category 0: Incomplete assessment.  
New cluster of microcalcifications in the left breast.  Additional magnification views are recommended for further characterization. 

ADDENDUM: 
The patient returned for magnification views of the left breast.  The microcalcifications are confirmed and appear clustered and pleomorphic, measuring up to 8 mm in diameter.  Biopsy is recommended.  BI-RADS Category 4: Suspicious abnormality. ➡ *Positive*
- SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained using digital mammography.
COMPARISON: None available.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable scattered fibroglandular densities are noted bilaterally.
IMPRESSION:
BIRADS Category 2: Benign findings. Recommend routine screening in 1 year.

ADDENDUM: 
This examination was double read by the interpreting radiologist. No microcalcifications were identified in either breast on this screening study. ➡ *Not Stated*
- FINDINGS:
RIGHT BREAST: Right breast tissue is heterogeneously dense. No dominant mass, suspicious calcifications or focal architectural distortion.
LEFT BREAST: Left breast tissue is heterogeneously dense. No dominant mass, suspicious calcifications or focal architectural distortion.
IMPRESSION:
1. No mammographic evidence of malignancy.
2. Heterogeneously dense breast tissue which could obscure detection of small masses. 
BIRADS 1 - Negative.
ADDENDUM: Bilateral breast ultrasound was performed. No sonographic evidence of malignancy identified. Annual screening mammography is recommended. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening. 

FINDINGS:
The breasts are almost entirely fatty.

There are scattered areas of fibroglandular density.

There are no significant masses, architectural distortion, or suspicious calcifications in either breast. 

There is a new group of amorphous calcifications in the upper outer quadrant of the left breast, consisting of approximately 6-8 calcifications in a cluster measuring 6mm. This is new since the prior exam. 

Skin is within normal limits. No significant skin thickening or retraction. 

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation and/or Prior Mammograms for Comparison. 

New group of amorphous calcifications in the left upper outer quadrant, BI-RADS 4B. Stereotactic biopsy recommended for further evaluation.

The remainder of both breasts are negative, BI-RADS 2.

ADDENDUM: 
The patient returned for left diagnostic mammogram with magnification views and ultrasound of the left upper outer quadrant.

Magnification views confirm a cluster of amorphous calcifications at the 2:00 position of the left breast, 6 cm from the nipple, measuring 6mm. 

Ultrasound of this area demonstrates no correlating mass, architectural distortion, or shadowing.  

FINAL ASSESSMENT: BI-RADS 4B: Suspicious - Biopsy should be considered. 
Stereotactic biopsy recommended of left 2:00 amorphous calcifications. ➡ *Positive*
- MAMMOGRAM - BILATERAL SCREENING

CLINICAL HISTORY: 50 year old female presents for her screening mammogram. No breast complaints. No family history of breast cancer.

COMPARISON: There are no prior mammograms available for comparison.

FINDINGS: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications in both breasts. There is a 1 cm cluster of pleomorphic calcifications in the right breast at 3 o'clock, 2 cm from the nipple. No architectural distortion or suspicious masses.

IMPRESSION:

1. Cluster of pleomorphic calcifications in the right breast, recommended for further evaluation with magnification views. 

BI-RADS: 0 (Needs additional imaging evaluation). ➡ *Positive*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
OBSP to recall client in 1 year: Yes.
There are no suspicious masses, focal asymmetries or architectural distortions. No significant changes compared to previous mammograms.
BIRADS-2. Routine annual screening recommended.
ADDENDUM: 
Ultrasound performed today demonstrates normal fibroglandular tissue with no sonographic abnormalities detected. No evidence of malignancy. ➡ *Not Stated*
- MAMMOGRAPHY, BILATERAL.

INDICATION: Annual screening mammogram. No current breast complaints.

COMPARISON: Mammogram from 1 year ago at this facility.

FINDINGS: 
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortions or suspicious calcifications in either breast. 

There are stable bilateral scattered fibroglandular densities. No significant interval change compared to prior mammogram.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography in 12 months.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views were obtained bilaterally using Hologic Dimensions digital mammography unit.

COMPARISON: None.

FINDINGS:

Both breasts demonstrate scattered fibroglandular density. Overall, the breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications in the right breast.

In the left breast, there is a new group of fine pleomorphic microcalcifications in the upper outer quadrant at the 10 o'clock position, approximately 5 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No associated mass, architectural distortion, or skin changes are identified.

The left axillary lymph nodes appear normal.

IMPRESSION:
BI-RADS Category 0: Incomplete. 
New group of suspicious microcalcifications in the left upper outer breast, BI-RADS 4. Additional magnification views recommended for further evaluation. 
BI-RADS Category 1 for the right breast: Negative.

RECOMMENDATION: Left diagnostic mammogram with magnification views. Consider stereotactic biopsy if the calcifications remain suspicious on diagnostic evaluation.

ADDENDUM (After additional magnification views): The pleomorphic microcalcifications in the left upper outer quadrant are confirmed and appear more suspicious on magnification views. Stereotactic biopsy is recommended. Final BI-RADS Category 4 for the left breast. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

TECHNIQUE: Bilateral standard CC and MLO views obtained.

COMPARISON: None available.

FINDINGS:
There is scattered fibroglandular tissue in both breasts.

RIGHT BREAST:
No significant masses, architectural distortion or suspicious calcifications.

LEFT BREAST:
No significant masses or architectural distortion. There is a small group of amorphous microcalcifications in the upper outer quadrant approximately 5 cm from the nipple. This cluster measures about 5 mm in greatest dimension. No other suspicious calcifications are identified.

IMPRESSION:
1. Small group of amorphous microcalcifications in the left upper outer quadrant, suspicious for early malignancy. Additional magnification views are recommended for further evaluation. BIRADS 4B.
2. The remainder of the left breast and right breast are BIRADS 2, benign.

ADDENDUM:
Magnification views of the left breast demonstrate the cluster of amorphous microcalcifications in the upper outer left breast. Stereotactic biopsy is recommended for tissue diagnosis. Updated BIRADS for the left breast is 4C, suspicious for malignancy. ➡ *Positive*
- MAMMOGRAM REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Comparison is made to screening mammogram from 10/15/2021, an outside study.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
RIGHT BREAST: There are no dominant masses, architectural distortion, or suspicious calcifications.  
LEFT BREAST: There are no dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
RIGHT BREAST: No mammographic evidence of malignancy. BI-RADS 1 - Negative.
LEFT BREAST: No mammographic evidence of malignancy. BI-RADS 1 - Negative.
RECOMMENDATION: Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- REPORT:
"BILATERAL MAMMOGRAMS WITH MAGNIFICATION VIEWS.
HISTORY: Annual screening mammogram. No prior mammograms available for comparison.
FINDINGS: Scattered areas of fibroglandular density are present in both breasts.
On spot compression views, there is a cluster of amorphous microcalcifications measuring approximately 8 mm at the 10:00 position of the right breast, 6 cm from the nipple. A smaller cluster of punctate microcalfications is also noted at the 3:00 position of the left breast, 4 cm from the nipple, spanning 5 mm.
The skin appears unremarkable. No spiculated masses, architectural distortion or other suspicious findings are identified in either breast. Benign-appearing calcifications are scattered bilaterally.
IMPRESSION:
New clusters of amorphous and punctate microcalcifications are noted in the right breast at 10:00 (8 mm) and left breast at 3:00 (5 mm). These are new and of uncertain significance. Stereotactic core biopsy is recommended for tissue diagnosis. BI-RADS 4B.
ADDENDUM: The patient returned for stereotactic core biopsy of the microcalcifications in the right breast at 10:00. Specimen radiograph confirmed calcifications within the cores. Pathology results: Fibrocystic changes with focal atypical ductal hyperplasia, calcifications present. The area of microcalcifications in the left breast at 3:00 will be followed in 6 months.
REVISED IMPRESSION: Stereotactic core biopsy of right breast microcalcifications showed fibrocystic changes with focal atypical ductal hyperplasia. Close follow-up is recommended. BI-RADS 3 for the right breast. BI-RADS 3 for the known microcalcifications in the left breast, 6 month follow-up advised. Overall: BI-RADS 3, probably benign findings.
" ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 

FINDINGS:

Right Breast:
The right breast is heterogeneously dense. No dominant masses, architectural distortion or suspicious calcifications are identified.

Left Breast: 
The left breast is also heterogeneously dense. There is an approximately 1 cm irregular mass at the 10 o'clock position, 5 cm from the nipple. This corresponds to a palpable abnormality reported by the patient. Additionally, there is a new cluster of fine pleomorphic microcalcifications at the 12 o'clock position, 3 cm from the nipple. No other suspicious masses, calcifications or areas of architectural distortion are seen. 

IMPRESSION:
1. BI-RADS 4: Suspicious finding in the left breast. Recommend stereotactic biopsy of the microcalcifications at 12 o'clock and ultrasound correlation of the 10 o'clock mass. 
2. BI-RADS 1: Normal right breast.

ADDENDUM: The patient returned for stereotactic biopsy of the left breast microcalcifications. Pathology results showed ductal carcinoma in situ, high nuclear grade with comedo necrosis. The patient will be referred to surgery for further management. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM.
INDICATION: Routine screening. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared to previous studies from 10/15/2020.
FINDINGS:
The breast tissue is heterogeneously dense which may obscure detection of small masses. 
There are no dominant masses, architectural distortion or suspicious calcifications in either breast. Stable benign appearing calcifications are noted in the upper outer quadrant of the right breast.
A new cluster of pleomorphic microcalcifications is identified in the upper outer left breast at the 2:00 position, approximately 6 cm from the nipple. This cluster measures 8 mm in size and was not present on the prior study. Additional magnification views confirm the presence of a suspicious cluster. Skin is normal in thickness. No other new suspicious findings.
IMPRESSION:
BI-RADS 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for the new cluster of microcalcifications at 2:00 in the left breast. The remainder of the bilateral mammogram is benign.
ADDENDUM: Stereotactic biopsy of the left breast calcifications was performed. Pathology results show ductal carcinoma in situ, intermediate nuclear grade with comedo necrosis. ➡ *Positive*
- FINDINGS:
Right breast: Heterogeneously dense parenchyma. Stable postsurgical changes of right mastectomy are noted. No suspicious mass, architectural distortion, or malignant type calcifications.
Left breast: Heterogeneously dense breast tissue. No mammographic evidence of malignancy. No spiculated mass, suspicious calcifications, or architectural distortion. Stable appearance compared to prior exams.

IMPRESSION:
1. Negative bilateral mammogram. No evidence of malignancy.
2. Postoperative changes of right mastectomy appear stable.

BIRADS Category 1 - Negative. 
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
INDICATION: Annual screening mammogram. No current breast complaints.
COMPARISON: Prior study of 10/15/2020.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Targeted ultrasound of the left breast at 3 o'clock position was also performed.
FINDINGS:
The left breast is composed of scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified.
Targeted ultrasound of the left breast at 3 o'clock position demonstrates normal fibroglandular tissue without a discrete solid or cystic lesion.
IMPRESSION:
1. Negative left screening mammogram. No mammographic evidence of malignancy.
2. Negative targeted left breast ultrasound.
ASSESSMENT: BI-RADS 1 - Negative.
RECOMMENDATION: Routine screening mammogram in 1 year. ➡ *Negative*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.

Comparison mammograms from 1 year prior are available.

The breasts are heterogeneously dense (assessed as <75% fibroglandular tissue).

No dominant masses, architectural distortions or suspicious asymmetries are identified.

There are no definite enlarged axillary, internal mammary or supraclavicular lymph nodes.

The skin appears unremarkable.

No comment can be made regarding microcalcifications. 

Based on these mammographic findings, this examination is assessed as BIRADS Category 2, with a recommendation for routine screening mammography to be performed in 2 years.

This patient participated in the TMIST trial and was assigned to the full field digital mammography arm. Consent was obtained per the approved Research Ethics Board protocol. ➡ *Not Stated*
- NORMAL SCREENING MAMMOGRAM.

Comparison: 06/15/2020.

FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
BIRADS 1 - Negative mammogram.

Routine screening mammography is recommended in 1 year. ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAPHY AND TARGETED ULTRASOUND LEFT BREAST
CLINICAL HISTORY: New lump in the left breast at 9:00, family history of breast cancer in maternal aunt.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure small masses.
There are scattered areas of fibroglandular density bilaterally.
In the right breast, no dominant masses, areas of architectural distortion, or suspicious calcifications are identified.
In the left breast at the 9:00 position 6 cm from the nipple, there is a new 12 mm irregular mass with indistinct margins, corresponding to the palpable area of concern. Additionally, at the 12:00 position 4 cm from the nipple, there is a new 5 mm cluster of pleomorphic calcifications.
TARGETED LEFT BREAST ULTRASOUND:
At the site of the 12:00 group of calcifications, there is a 6 mm irregular solid mass with angular margins and shadowing. This corresponds to the mammographic finding. No axillary lymphadenopathy.
IMPRESSION:
New 12 mm irregular mass at 9:00 left breast, suspicious for malignancy. BI-RADS 4C. Ultrasound-guided core biopsy is recommended.
New group of pleomorphic calcifications at 12:00 left breast with associated irregular shadowing mass on ultrasound, highly suspicious for malignancy. BI-RADS 4C. Stereotactic-guided biopsy is recommended. 
BI-RADS 1 right breast. Routine screening in 1 year recommended. ➡ *Positive*
- FINDINGS: The breast tissue is heterogeneously dense, which may obscure detection of small masses. There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Targeted ultrasound was performed at the site of clinical concern at the 2 o'clock position of the right breast, 3 cm from the nipple. No sonographic abnormality was identified.

IMPRESSION:
1. Heterogeneously dense breast tissue, negative bilateral screening mammogram.
2. No sonographic correlate for the palpable abnormality in the right breast at 2 o'clock position, 3 cm from the nipple.

ASSESSMENT: BI-RADS Category 1: Negative.

RECOMMENDATION: Recommend routine annual screening mammography in 1 year. Clinical follow-up of the palpable abnormality in the right breast is advised to document resolution. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAM
CLINICAL HISTORY: Annual screening mammogram. No complaints.
COMPARISON: Screening mammogram from 2 years ago.  
TECHNIQUE: Standard CC and MLO views were performed.
FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses. 
There are no dominant masses, architectural distortion or suspicious areas of abnormal density.
In the right breast at 9 o'clock middle depth 7 cm from the nipple there is a new cluster of amorphous microcalcifications measuring about 12 mm. They are new compared to prior mammograms. 
The left breast has no suspicious findings. Stable benign appearing calcifications are present.
IMPRESSION: 
Right breast: Suspicious new cluster of amorphous microcalcifications at 9 o'clock middle depth. BI-RADS 4. A stereotactic biopsy is recommended for further evaluation.
Left breast: No mammographic evidence of malignancy. BI-RADS 2 - benign finding.

ADDENDUM: The patient returned for right breast stereotactic biopsy of the microcalcifications at 9 o'clock. Specimen radiograph confirmed calcifications. Final pathology showed atypical ductal hyperplasia with microcalcifications. 6 month follow up mammogram is recommended. ➡ *Positive*
- MAMMOGRAM QUALITY: Standard mammographic views
COMPARISON: None available

FINDINGS:
The breast tissue is predominantly fatty.
There are no mammographic findings suspicious for malignancy. No significant masses, calcifications, or concerning asymmetries are identified.

IMPRESSION:
BIRADS 2 - Benign finding. Recommend routine screening in 1 year. ➡ *Not Stated*
- ULTRASOUND AND MAMMOGRAM, BOTH BREASTS

INDICATION: Annual screening.

FINDINGS:

RIGHT BREAST:
Mammogram: The breast is heterogeneously dense. No dominant masses, areas of architectural distortion or suspicious calcifications.

Ultrasound: Retroareolar ducts measure 2 mm and are within normal limits. No cysts or solid masses. 

LEFT BREAST: 
Mammogram: The breast is heterogeneously dense. No dominant masses or areas of architectural distortion. Cluster of indeterminate amorphous microcalcifications measuring 6 mm noted at the 10:00 position, 6 cm from the nipple. Additional magnification views obtained. 

Ultrasound: Retroareolar ducts measure 2 mm and are within normal limits. No cysts or solid masses. Ultrasound correlate for the mammographic calcifications at 10:00 position demonstrates no associated mass. 

IMPRESSION:
RIGHT BREAST: No suspicious findings. BI-RADS 1.

LEFT BREAST:  
Cluster of indeterminate amorphous microcalcifications measuring 6 mm at the 10:00 position, 6 cm from the nipple. No sonographic correlate. Recommend 6 month mammographic follow up to document stability, or biopsy if clinically indicated. BI-RADS 4.

ADDENDUM:
The patient elected to proceed with stereotactic core biopsy of the group of microcalcifications in the left breast at 10:00. Pathology revealed benign breast tissue with focal fibrosis and microcalcifications, negative for malignancy. Final assessment BI-RADS 2, benign finding. Recommend routine screening in 1 year. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Bilateral digital mammography with tomosynthesis.

FINDINGS:

Both breasts show scattered fibroglandular densities. No dominant mass, architectural distortion or skin thickening in either breast.

In the right breast, there is a new group of pleomorphic calcifications in the upper outer quadrant, spanning an area of approximately 8 mm. These are irregular in shape and distribution, varying in size and are of concern.

No suspicious calcifications in the left breast. 

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast -- biopsy should be considered. Grouped pleomorphic microcalcifications in the right upper outer quadrant, new since prior exam and suspicious for malignancy.

RECOMMENDATION: Stereotactic core biopsy is recommended for the right breast microcalcifications. The patient should return to the breast center for additional evaluation and management. ➡ *Positive*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Previous exam from 1 year ago.
FINDINGS: 
The breasts are heterogeneously dense. No concerning masses, architectural distortion, or suspicious calcifications are identified in either breast. Stable benign-appearing calcifications are noted in the left breast. The skin and nipple are unremarkable bilaterally. No axillary or intramammary lymphadenopathy is evident.
IMPRESSION: 
No mammographic evidence of malignancy. Recommend routine screening in 1 year.
BIRADS Category 2 - Benign findings. 
ADDENDUM: This mammogram was double read by CAD with no additional findings. ➡ *Not Stated*
- MAMMOGRAM - BILATERAL.
INDICATION: Family history of breast cancer in mother and maternal aunt. Patient with complaints of tenderness in the left breast upper outer quadrant, denies palpable mass.  
TECHNIQUE: Standard CC and MLO views of both breasts with spot compression views of an area of concern in the left breast at 2 o'clock.
COMPARISON: Comparison made to prior study of 11/30/2022.
FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses. 
Right breast: There are no dominant masses, suspicious calcifications, or areas of architectural distortion. Stable benign appearing calcifications in the upper outer quadrant. No significant interval change.
Left breast: No dominant masses or suspicious calcifications. There is an area of focal asymmetry in the upper outer quadrant at approximately 2 o'clock, unchanged from prior exam. Additional spot compression views demonstrate this to be normal overlapping tissue. No areas of architectural distortion.  
IMPRESSION: 
1. Heterogeneously dense breasts, no suspicious findings. 
2. Stable focal asymmetry in the left breast at 2 o'clock, likely representing summation artifact/overlapping tissue as seen on spot compression views. No mammographic evidence of malignancy.
3. Recommend continued annual screening mammography and clinical breast exams.
BI-RADS Category 1 - Negative. ➡ *Negative*
- MAMMOGRAM - BILATERAL DIAGNOSTIC

CLINICAL HISTORY: Palpable finding, left breast. Patient states she can feel something at the 10 o'clock position.

COMPARISON: Mammogram from 1/15/2024.

FINDINGS: The breasts are heterogeneously dense. There is a new, 7 mm cluster of fine pleomorphic calcifications in the left breast at the 10 o'clock position, 2 cm from the nipple. This was biopsied under stereotactic guidance. No architectural distortion or suspicious masses.

IMPRESSION:

1. New cluster of fine pleomorphic calcifications in the left breast at the 10 o'clock position, 2 cm from the nipple, status post stereotactic biopsy. Please see biopsy results.

BI-RADS: 0 (Incomplete). ➡ *Positive*
- REPORT: "BILATERAL MAMMOGRAM AND ULTRASOUND.
HISTORY: 55-year-old female with occasional bilateral breast tenderness. No palpable lumps. Last mammogram 1 year ago at StLukes - reported as normal.
FINDINGS: Bilateral mammograms demonstrate scattered areas of fibroglandular density. No dominant masses, architectural distortion, or suspicious calcifications are identified in either breast. Stable appearance compared to prior mammogram.  
Bilateral breast ultrasound was performed. No solid or cystic lesions are seen in either breast. No abnormal axillary lymph nodes are identified.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Recommend annual screening mammography.
2. No sonographic abnormalities identified. Bilateral breast tenderness is likely due to hormonal changes.
BIRADS 1.
ADDENDUM: Upon further review and discussion with the referring clinician, no additional imaging is required at this time. Patient is advised to return for routine annual screening mammography." ➡ *Negative*
- BREAST IMAGING REPORT

CLINICAL INDICATION: Annual screening mammogram. No current breast concerns.

COMPARISON: Screening mammogram from 11/15/2021 at XYZ Breast Center.  

FINDINGS:
Right breast: Heterogeneously dense parenchyma. No dominant mass, architectural distortion, or suspicious calcifications. Stable benign calcifications in the upper outer quadrant. 

Left breast: Heterogeneously dense parenchyma. No dominant mass, architectural distortion, or suspicious calcifications. Stable postsurgical changes in the upper outer quadrant from prior benign biopsy.

IMPRESSION: 
BI-RADS Category 1: Negative bilateral screening mammogram. No evidence of malignancy. 

RECOMMENDATIONS:
Routine annual screening mammography is recommended. Monthly breast self-exams and annual clinical breast exams are also advised. ➡ *Negative*
- FINDINGS: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. No dominant mass, suspicious calcifications or architectural distortion.

At the site of the palpable abnormality in the right upper outer quadrant, there is no mammographic correlate. Targeted right breast ultrasound was performed. At the 10 o'clock position 10 cm from the nipple, there is an isoechoic solid mass measuring 6 x 4 x 5 mm. This corresponds to the site of palpable concern. 

Remainder of the right breast is unremarkable. Left breast is unremarkable.

IMPRESSION:
1. Heterogeneously dense breast tissue. 
2. No suspicious mammographic findings bilaterally. 
3. Right breast 6 mm solid mass at 10 o'clock position corresponding to the site of palpable abnormality. Biopsy is recommended. 

BIRADS 1 on the left. 
BIRADS 4 on the right. ➡ *Negative*
- CLINICAL HISTORY: 43 year old female with history of right breast cancer treated with lumpectomy and radiation therapy 5 years ago. Annual screening mammogram.

FINDINGS:
Right breast: The right breast demonstrates stable postsurgical changes in the upper outer quadrant consistent with prior lumpectomy. There are no new masses, architectural distortion, or suspicious calcifications. 

Left breast: The left breast is heterogeneously dense (ACR category C). There are no dominant masses, areas of architectural distortion, or suspicious calcifications. 

IMPRESSION:
1. Stable postsurgical changes right breast. No suspicious findings.
2. No evidence of malignancy in the left breast.

ASSESSMENT: Negative bilateral screening mammogram.
BIRADS 1 - Negative. Routine annual screening is recommended. ➡ *Negative*
- FINDINGS: The right breast is absent, consistent with prior mastectomy. The left breast is heterogeneously dense. No dominant mass, architectural distortion, or suspicious calcifications are identified. Stable appearance compared to prior mammograms. 

Targeted ultrasound was performed at the area of palpable concern, 7 o'clock position of the left breast. No correlating sonographic mass or abnormality was identified.

IMPRESSION:
1. Negative left screening mammogram. No mammographic evidence of malignancy. Recommend annual screening mammography. 
2. Targeted left breast ultrasound is negative for correlating mass at site of palpable abnormality in the 7 o'clock position. Recommend clinical follow-up as indicated. 

BI-RADS Category 1: Negative.

ADDENDUM: This examination was reviewed in detail with Dr. Jane Smith following the interpretation. No changes were made to the above report. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
History: Screening.
Comparison available dating from 06/2022 for comparison
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- MAMMOGRAM - BILATERAL WITH CAD

CLINICAL HISTORY: 65 year old female for screening mammogram.

COMPARISON: 06/18/2023.

FINDINGS: The breasts are heterogeneously dense. Comparison is made to prior examination. Again seen are scattered benign-appearing calcifications in both breasts. There is a new focus of highly suspicious, grouped, fine, pleomorphic calcifications in the right breast at approximately the 3 o'clock position. This measures approximately 1.1 cm in greatest diameter and is located approximately 2 cm from the nipple.

IMPRESSION:

1. New focus of highly suspicious, grouped, fine, pleomorphic calcifications in the right breast. Given the patient's age, this is highly suspicious for malignancy. Correlation with spot magnification views and possible biopsy is recommended.

BI-RADS: 4C (Highly suspicious for malignancy). ➡ *Positive*
- ULTRASOUND RIGHT BREAST AND RIGHT AXILLA
INDICATION: Right upper breast palpable mass - query cyst vs solid lesion. Slight tenderness present.
TECHNIQUE: Targeted ultrasound examination of the right upper breast including 2D images and color Doppler imaging was performed.
COMPARISON: No previous exams available.
FINDINGS:
Right breast: At the 10 o'clock position, 7 cm from the nipple, there is a circumscribed anechoic mass measuring 2.2 x 1.5 x 1.3 cm. No internal vascularity or solid component identified. Findings are consistent with a simple cyst. No other lesions identified. No abnormal lymph nodes are seen.
Right axilla: The axilla was scanned and no abnormal lymph nodes are identified.
IMPRESSION:
1. Benign simple cyst in the right breast 10 o'clock position as described above. No evidence of malignancy. 
2. Normal right axilla.
RECOMMENDATION: 
1. Clinical follow-up to confirm resolution of palpable finding.
2. Routine screening as per provincial guidelines.
BIRADS: 2 Benign finding ➡ *Negative*
- LEFT MAMMOGRAM AND ULTRASOUND
CLINICAL HISTORY: The patient is a 45-year-old woman with a palpable lump in the upper outer quadrant of the left breast. No prior mammograms available for comparison.
FINDINGS:
Left breast: Heterogeneously dense breast tissue. No suspicious masses, architectural distortion, or microcalcifications. A 1.5 cm oval circumscribed mass is seen in the upper outer quadrant at the 2 o'clock position, corresponding to the palpable abnormality.
Targeted left breast ultrasound was performed at the site of the palpable abnormality in the 2 o'clock position of the left breast. A 1.5 cm oval, circumscribed, parallel orientation hypoechoic mass is demonstrated, consistent with a benign fibroadenoma.
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Heterogeneously dense breast tissue.
2. 1.5 cm benign appearing mass in the left breast 2 o'clock position on ultrasound, consistent with a fibroadenoma. Corresponds to the site of palpable concern. Recommend clinical follow-up. If the mass remains stable, no additional imaging is required. If the mass increases in size, repeat imaging is recommended.
BI-RADS Category 2: Benign finding. Recommend routine screening in 1 year. ➡ *Negative*
- FINDINGS: The breasts are heterogeneously dense.

Within the right breast there are no mammographically suspicious masses, calcifications, or other findings. BIRADS 1.

Within the upper outer left breast there is an 8 mm loose cluster of heterogeneous calcifications ranging from amorphous to fine pleomorphic in morphology. This is indeterminate and requires further evaluation with magnification views. There are no other suspicious mammographic findings in the left breast.

Targeted ultrasound of the left breast demonstrated a few small simple cysts but no sonographic correlate for the calcifications. No other sonographically suspicious masses or findings.

IMPRESSION:
1. BI-RADS 0: 8 mm cluster of indeterminate calcifications upper outer left breast. Magnification views recommended for further evaluation.
2. BI-RADS 1: Right breast. Negative.

ADDENDUM - MAGNIFICATION VIEWS LEFT BREAST: The magnification views of the upper outer left breast redemonstrate the cluster of amorphous to fine pleomorphic calcifications. These measure approximately 8 mm in greatest dimension. At least 8 calcifications are present. 

REVISED IMPRESSION: BI-RADS 4: Suspicious cluster of calcifications upper outer left breast. Stereotactic biopsy is recommended. ➡ *Positive*
- MAMMOGRAPHIC FINDINGS:
Digital breast tomosynthesis and ultrasound of both breasts were performed.
FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in either breast.
Cluster of microcalcifications is noted in the upper outer quadrant of the right breast, approximately 7 cm from the nipple. 
No suspicious axillary, supraclavicular or internal mammary lymphadenopathy.
Benign appearing lymph nodes in the right axilla.
Left breast is unremarkable. No suspicious intramammary lymph nodes are seen.
IMPRESSION:
1. Focal cluster of microcalcifications right breast at 10 o'clock. Additional magnification views are recommended for further evaluation. Ultrasound guided biopsy may be indicated depending on additional mammographic findings. BIRADS 0: Incomplete. Need additional imaging evaluation.
2. otherwise, no dominant mass, architectural distortion or suspicious calcifications are identified in either breast. BIRADS 2.
ADDENDUM: Magnification views confirms cluster of microcalcifications spanning 9 mm at 10:00, 7 cm from the right nipple, morphology is amorphous and distribution is grouped. Stereotactic core biopsy is recommended. Final BIRADS 4B: Suspicious abnormality. ➡ *Positive*
- MAMMOGRAM, BILATERAL.
HISTORY: Screening mammography. Patient is a 45-year-old woman with no significant past medical history. Reports annual screening mammograms have been previously unremarkable. No palpable lumps, skin changes, or nipple discharge.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure small masses.
Right breast: There are no masses, architectural distortion, or suspicious calcifications.
Left breast: There are no masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Both breasts are heterogeneously dense, which could obscure small masses. Consider supplemental screening with breast ultrasound or MRI for increased breast cancer detection.
BIRADS 1 - Negative. Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- BILATERAL MAMMOGRAM. Clinical History: 53 year old female presenting for routine screening mammogram. Patient reports a family history of breast cancer in her mother, who was diagnosed at the age of 60. Comparison is made with prior mammogram from 09/27/2023. Findings: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcifications. Impression: 1. No evidence of malignancy. 2. BI-RADS 1. ➡ *Negative*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.  
This client was enrolled in the TMIST trial and randomized to the full field digital mammography arm after providing informed consent as per the Research Ethics Board approved protocol.
Comparison to prior exams: Yes. 
Breast density: Scattered fibroglandular densities.
Next OBSP exam recommended in 12 months.
No suspicious mammographic abnormalities are identified. The breasts are free of dominant masses, architectural distortions, or overt malignant-type calcifications.
BIRADS Category 1 - Negative. ➡ *Not Stated*
- FINDINGS: The breast tissue is heterogeneously dense, which could obscure detection of small masses. There are no dominant masses, suspicious calcifications, or areas of architectural distortion. There is no significant change compared to the prior examination.

Targeted ultrasound was performed at the site of the palpable abnormality at the 10:00 position of the right breast, 3 cm from the nipple. No sonographic correlate was identified. The palpable area corresponds to normal fibroglandular tissue.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy. 
2. Stable, likely benign finding at the site of palpable concern in the right breast, probably representing normal fibroglandular tissue. 
3. Recommend annual screening mammography. Ultrasound is not recommended for routine screening in the absence of a palpable or mammographic finding.

BIRADS 1 - Negative. ➡ *Negative*
- SCREENING BILATERAL MAMMOGRAM.
Compared to Previous: Yes.
Bilateral implants noted which limit assessment of underlying breast tissue.
Partially obscured breasts remain heterogeneously dense, scattered areas of fibroglandular density.
No suspicious masses, architectural distortion or definite calcifications.
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM. Clinical History: Patient is a 42 year old female presenting for a screening mammogram. Technique: Bilateral CC and MLO images were obtained. Comparison is made to prior mammographic examination from 09/01/2023. Findings: The breasts are heterogeneously dense. There are no masses, architectural distortion, or suspicious calcifications. Impression: Negative mammogram. BI-RADS 1. ➡ *Negative*
- ULTRASOUND LEFT BREAST
INDICATION: Palpable abnormality left breast upper outer quadrant.
TECHNIQUE: Targeted ultrasound left breast upper outer quadrant was performed using a high resolution linear transducer. 
COMPARISON: None.
FINDINGS: 
The left breast is heterogeneously echogenic. Targeted ultrasound of the left breast upper outer quadrant demonstrates normal appearing glandular and fibrous breast tissue. No suspicious solid or cystic mass is identified. No suspicious calcifications are seen. The area of palpable concern corresponds to normal appearing glandular tissue.
IMPRESSION:
No sonographic evidence of malignancy left breast upper outer quadrant. Recommend annual screening mammography and physical exam. 
BIRADS 1. ➡ *Negative*
- SCREENING BILATERAL DIGITAL MAMMOGRAM.

Compared to previous screening mammogram from April 2021, there are no concerning changes.

Breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

There are no dominant masses, suspicious calcifications, or concerning asymmetries.

IMPRESSION:
BIRADS 2 - Benign findings. Routine screening recommended in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Annual screening mammogram.

COMPARISON: With prior exams dated 5/23/2020 and 5/18/2019.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

Magnification views in the right CC and MLO views demonstrate a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant, approximately 7 cm from the nipple, spanning an area of about 8mm. No associated mass, architectural distortion, or skin thickening is identified.

Stable benign-appearing calcifications are noted in the right breast at 3 o'clock, 6 cm from the nipple.

There are no suspicious calcifications in the left breast. Stable vascular calcifications are present.

IMPRESSION:

RIGHT BREAST, BI-RADS 4: Suspicious microcalcifications in the right upper outer quadrant. Stereotactic biopsy is recommended for further evaluation.

LEFT BREAST, BI-RADS 2: Benign, no evidence of malignancy. Recommend routine screening in 1 year.

ADDENDUM:
Stereotactic biopsy of the right breast microcalcifications at 10 o'clock was performed on 6/5/2021. Pathology results: Ductal carcinoma in situ, intermediate nuclear grade, cribriform type. Estrogen receptor-positive. Progesterone receptor-positive. ➡ *Positive*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: None available.

FINDINGS:
Both breasts show scattered areas of heterogeneously dense fibroglandular tissue.

Right breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Left breast: 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Heterogeneously dense breasts. 
2. No mammographic evidence of malignancy. 
3. Recommend routine screening mammography in 1 year, sooner if clinically indicated.

BIRADS 1 - Negative

ADDENDUM: Bilateral mammogram. No suspicious calcifications. Routine screening recommended. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: None available

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:

Right Breast:
The right breast is heterogeneously dense (ACR category C). There are no dominant masses, areas of architectural distortion or suspicious calcifications. No abnormalities or significant changes are noted.

Left Breast: 
The left breast is also heterogeneously dense (ACR category C). In the upper outer quadrant, there is a new cluster of fine pleomorphic calcifications spanning an area of approximately 1 cm. No associated mass or architectural distortion is evident. The remainder of the left breast is unremarkable.

IMPRESSION:
New cluster of suspicious calcifications in the upper outer quadrant of the left breast, BI-RADS 4B. Stereotactic biopsy is recommended for definitive characterization. The right breast is negative, BI-RADS 1. ➡ *Positive*
- SCREENING MAMMOGRAM.
Compared to Previous: None.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
Impression:
No mammographic evidence of malignancy. Recommend annual screening mammography. 
BIRADS-2. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Prior study from 12/17/2020.

TECHNIQUE: Standard CC and MLO views of both breasts.  

FINDINGS:

Right Breast:
The right breast shows scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications are identified. 

Left Breast: 
The left breast shows scattered areas of fibroglandular density. In the upper outer quadrant, there is a new group of pleomorphic calcifications spanning an area of approximately 1 cm. No associated mass or architectural distortion is identified. 

IMPRESSION:

LEFT BREAST, BI-RADS 4: Suspicious microcalcifications in the upper outer quadrant of the left breast, warranting biopsy to exclude malignancy.  

RIGHT BREAST, BI-RADS 2: Benign mammographic findings. Recommend continued annual screening in 12 months.

ADDENDUM: The patient was recalled for magnification views of the left breast, which confirm the presence of a group of pleomorphic microcalcifications in the upper outer quadrant. Stereotactic core needle biopsy was performed, with pathology results pending. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening. No prior mammograms available for comparison.

FINDINGS: The breasts are heterogeneously dense, which may obscure small masses.

There are no dominant masses, architectural distortion or suspicious calcifications in the right breast.

In the left breast at the 12:00 position, 6 cm from the nipple, there is a group of amorphous calcifications spanning an area of approximately 1 cm. No associated mass, architectural distortion or skin thickening is seen.

The visualized left axillary lymph nodes appear normal. 

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation and/or prior mammograms for comparison.

Amorphous calcifications in the left breast require additional magnification views for further characterization.

ADDENDUM: Additional magnification views of the left breast were performed. The amorphous calcifications at 12:00 position in the left breast are more apparent on these magnification views, with no associated mass or architectural distortion.

FINAL ASSESSMENT:
LEFT BREAST: BI-RADS Category 4: Suspicious abnormality - Biopsy should be considered. Stereotactic biopsy is recommended for the amorphous calcifications at 12:00 position in the left breast.
RIGHT BREAST: BI-RADS Category 1: Negative. ➡ *Positive*
- FINDINGS:
Bilateral mammogram images demonstrate scattered fibroglandular densities. There are no suspicious masses, architectural distortion, or malignant-type calcifications.

Targeted ultrasound was performed at the site of palpable abnormality in the right breast at 10 o'clock position, 5 cm from the nipple. No corresponding sonographic mass or suspicious finding was identified. 

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. No sonographic correlate for the palpable finding in the right breast. 

BI-RADS Category 1: Negative. Recommend annual screening mammography in 12 months. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.
HISTORY: Screening mammogram. Patient has implants.
TECHNIQUE: Bilateral mammograms were obtained including implant displaced views.
COMPARISON: Comparison is made to outside screening mammogram dated 3/15/2021.
FINDINGS:
The breasts are predominantly fatty. Breast implants are noted bilaterally. There are no masses, architectural distortion, or suspicious calcifications.
Stable benign appearing calcifications are noted in the right breast.
There is no appreciable change from prior mammogram.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Benign bilateral breast implants. No evidence of implant rupture.
3. Stable benign right breast calcifications.
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- FINDINGS: Both breasts contain scattered fibroglandular densities. There are no dominant masses, architectural distortions, or suspicious microcalcifications.

IMPRESSION:
1. No mammographic evidence of malignancy. Scattered fibroglandular densities.
2. Annual screening mammography is recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Right breast tissue is heterogeneously dense. There are no suspicious masses, architectural distortion or dominant suspicious calcifications.
Left breast tissue is extremely dense. There are no suspicious masses, architectural distortion or dominant suspicious calcifications.

Targeted ultrasound:
Right breast, upper outer quadrant, 10 o'clock position 4 cm from the nipple, ill-defined mixed echogenicity mass with angulated margins and posterior acoustic shadowing, measuring approximately 1.0 x 0.8 x 0.7 cm. This corresponds to an area of palpable concern.

IMPRESSION:
1. Negative digital bilateral mammogram. 
2. Suspicious right breast mass at 10 o'clock position, 4 cm from the nipple, measuring approximately 1.0 x 0.8 x 0.7 cm, with features suggestive of malignancy. Ultrasound-guided biopsy is recommended. BI-RADS 4c. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Screening.
FINDINGS:
Both breasts are almost entirely fatty. No focal dominant masses or architectural distortions.
In the upper outer quadrant of the left breast, there are pleomorphic microcalcifications spanning an area of approximately 1 cm. These are new compared to the prior mammogram from 3 years ago. No other suspicious microcalcifications or significant findings in either breast.
The skin is within normal limits. No skin thickening or retraction.
IMPRESSION:
BI-RADS 4C: Suspicious microcalcifications in the left breast at 2 o'clock position. Stereotactic biopsy is recommended for further evaluation.
ADDENDUM: Stereotactic biopsy of the left breast microcalcifications was performed. Pathology results show ductal carcinoma in situ, intermediate grade, with no evidence of invasion. ➡ *Positive*
- BILATERAL MAMMOGRAM
INDICATION: Screening mammogram.
COMPARISON: None available.
FINDINGS:
Both breasts are predominantly fatty replaced. Scattered areas of benign-appearing fibroglandular tissue are present bilaterally. No dominant masses, suspicious calcifications, or areas of architectural distortion are identified in either breast. The skin appears unremarkable. No axillary, supraclavicular or internal mammary adenopathy is seen.
IMPRESSION:
1. There is no mammographic evidence of malignancy in either breast.
2. Predominantly fatty replaced breast tissue bilaterally.
BIRADS 1 - Negative bilateral screening mammogram.
RECOMMENDATION: Routine screening mammography in 1 year. ➡ *Negative*
- NORMAL SCREENING MAMMOGRAM WITH BREAST TOMOSYNTHESIS.

INDICATION: Screening.

COMPARISON: None.

TECHNIQUE: Bilateral digital breast tomosynthesis and 2D synthesized views.

FINDINGS:
Breast composition is heterogeneously dense. Scattered areas of fibroglandular density are present. No suspicious masses, architectural distortions, or dominant suspicious calcifications are identified.

IMPRESSION:
NEGATIVE SCREENING MAMMOGRAM WITH BREAST TOMOSYNTHESIS. BIRADS 1.

RECOMMENDATION:
- Routine screening in 1 year. ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED MAGNIFICATION VIEWS AND BILATERAL ULTRASOUND.
Compared to previous outside breast imaging studies from 2019 (St. Michael's Hospital) including mammograms and bilateral breast ultrasound. Earlier mammograms were not sent for comparison.
FINDINGS: The breasts are heterogeneously dense. 
There are scattered fibroglandular densities in both breasts. No skin thickening or nipple retraction. 
In the upper outer right breast there is a 6 mm cluster of pleomorphic microcalcifications at 10:00, 9 cm from the nipple. On ultrasound this corresponds to a hypoechoic mass with ill-defined margins measuring 5 x 4 x 4 mm. No increased vascularity.
In the upper inner left breast there is an 8 mm cluster of pleomorphic microcalcifications at 2:00, 8 cm from the nipple. No sonographic correlate identified.
Bilateral axillae scanned. No abnormal lymph nodes identified.
SUMMARY:
New cluster of pleomorphic microcalcifications upper outer right breast at 10:00 with a sonographic correlate, suspicious for malignancy. BI-RADS 4C. Stereotactic biopsy recommended. 
New cluster of pleomorphic microcalcifications upper inner left breast at 2:00 with no sonographic correlate, suspicious for malignancy. BI-RADS 4B. Stereotactic biopsy recommended.
A requisition has been sent to the department to arrange these biopsies.

ADDENDUM: Patient called. Stereotactic biopsies of the right breast 10:00 microcalcifications and left breast 2:00 microcalcifications have been scheduled for next week. ➡ *Positive*
- MAMMOGRAPHIC EXAMINATION
INDICATION: Annual screening mammogram.
TECHNIQUE: Standard CC and MLO views of the right and left breast were obtained.
COMPARISON: Comparison is made to prior mammogram dated 04/15/2022.
FINDINGS:
The breast tissue is heterogeneously dense, which could obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Stable benign-appearing calcifications are present bilaterally.
There is no significant change compared to prior mammogram.
IMPRESSION:
BI-RADS Category 1: Negative. Recommend continued annual screening mammography in 1 year.

ADDENDUM: Bilateral breast ultrasound was performed on today's visit.
TECHNIQUE: Handheld high-resolution linear transducer used to image both breasts in radial and anti-radial planes.
FINDINGS: The breasts are heterogeneously echogenic. No suspicious solid or cystic masses are identified. No areas of abnormal shadowing or focal distortion. No skin thickening or retraction. No axillary lymphadenopathy.
IMPRESSION: Negative bilateral breast ultrasound. No evidence of malignancy. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

HISTORY: Annual screening mammogram. No current breast complaints.

FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

A few benign-appearing calcifications are noted in the upper outer quadrant of the right breast, likely representing small fat necrosis or fibrocystic changes. These appear stable compared to prior mammograms.

There are multiple small bilateral circumscribed masses, likely representing cysts or fibroadenomas. Largest measures 8 mm in the left breast at 3:00, 4 cm from the nipple. These also appear stable.

In the upper outer left breast at approximately 2:00, 8 cm from the nipple, there is a new cluster of fine pleomorphic microcalcifications spanning an area of about 6 mm. These were not present on the prior mammogram from 2 years ago. Additional magnification views confirm these are suspicious and require biopsy.

No concerning axillary lymphadenopathy.

IMPRESSION:
BI-RADS 4: Suspicious microcalcifications in the left breast at 2:00, 8 cm from the nipple. Stereotactic biopsy recommended. An addendum will be issued with the biopsy results.
The remainder of the bilateral breast tissue is BIRADS 2, benign findings. Recommend annual screening mammography in 1 year.

ADDENDUM:  
Stereotactic biopsy of the left breast calcifications at 2:00 was performed. Pathology results demonstrate ductal carcinoma in situ, intermediate grade, cribriform type. The patient will be referred to breast surgery for further management. ➡ *Positive*
- SCREENING MAMMOGRAM
 
INDICATION: Routine screening.
 
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
 
COMPARISON: None.
 
FINDINGS:
The breasts are heterogeneously dense (breast density 51-75%), which may obscure detection of small masses.

There are no dominant masses, suspicious calcifications, or other findings suspicious for malignancy. Specifically, no microcalcifications are identified.

Benign calcifications are present bilaterally, likely representing vascular calcifications and small fat necrosis Oil cysts.

IMPRESSION:
1. Negative mammogram. No mammographic evidence of malignancy.
2. Heterogeneously dense breasts.
 
BIRADS Category 1: Negative.

1 year routine screening mammogram is recommended.

ADDENDUM:
This examination was read by the primary interpreting Radiologist named below at 3:49pm on October 16, 2022. This report was then proofread, electronically signed and finalized by the same Radiologist at 3:52pm on the same date. ➡ *Not Stated*
- FINDINGS:

BILATERAL DIGITAL MAMMOGRAPHY:

The breasts are predominantly fatty replaced.

In the right breast, at the 11:00 position, 5 cm from the nipple, there is a 1.2 cm area of grouped amorphous microcalcifications, new since the prior exam.

No suspicious masses or areas of architectural distortion are identified in either breast.

No suspicious right axillary lymph nodes are identified.

Old surgical clips noted in the left breast at site of prior lumpectomy and radiation therapy.

IMPRESSION:

BI-RADS 4 - Suspicious grouped amorphous microcalcifications in the right breast at 11:00, 5 cm from the nipple. Stereotactic core biopsy recommended for definitive characterization.

BI-RADS 2 - Benign post surgical changes left breast.

ADDENDUM:

Stereotactic core biopsy of the suspicious microcalcifications in the right breast was performed. Pathology results are pending and will be communicated in an addendum to this report when available.

PATHOLOGY ADDENDUM:

Stereotactic core biopsy right breast calcifications: Invasive ductal carcinoma, moderately differentiated, with associated ductal carcinoma in situ, high nuclear grade with necrosis.

FINAL ASSESSMENT:

RIGHT BREAST: BI-RADS 6 - Biopsy proven malignancy
LEFT BREAST: BI-RADS 2 - Benign ➡ *Positive*
- FINDINGS: The breasts are heterogeneously dense, which may obscure detection of small masses. There are no dominant masses, areas of architectural distortion, or suspicious calcifications.  
There are scattered benign-appearing calcifications in both breasts, likely representing fibrocystic changes.
There are no significant changes compared to prior mammograms.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breasts, which could obscure small masses.
3. Scattered benign-appearing calcifications bilaterally, likely fibrocystic changes.
4. Annual screening mammography is recommended.
BI-RADS Category 2: Benign findings. Routine screening recommended.

ADDENDUM: Bilateral breast ultrasound performed for further evaluation of dense breast tissue.

ULTRASOUND FINDINGS: 
The breast tissue demonstrates heterogeneous fibroglandular tissue bilaterally. No focal solid or cystic masses are identified. No areas of architectural distortion or suspicious calcifications are seen. 

IMPRESSION:
No sonographic evidence of malignancy. Benign bilateral breast ultrasound, correlating with mammographic findings. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

In the left breast at the 10:00 position, 6 cm from the nipple, there is a small cluster of amorphous calcifications spanning approximately 6 mm. This is a new finding compared to the patient's prior mammograms. Additional spot compression magnification views of this area are recommended for further evaluation.

The right breast is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation. Small cluster of amorphous calcifications in the left breast at 10:00, 6 cm from the nipple. Additional spot compression magnification views are recommended for further evaluation.

ADDENDUM:

LEFT DIAGNOSTIC MAMMOGRAM

Spot compression magnification views were performed of the area of concern in the left breast at 10:00. The amorphous calcifications are more apparent on these views and span approximately 8 mm in greatest dimension. The distribution is clustered. No associated mass, architectural distortion, or skin thickening is identified.

FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious abnormality. Stereotactic biopsy is recommended for tissue diagnosis of the amorphous calcifications in the left breast at 10:00. ➡ *Positive*
- MAMMOGRAM AND ULTRASOUND OF THE RIGHT BREAST.

HISTORY: Annual routine screening mammogram. No new palpable lump, skin or nipple changes or nipple discharge. Prior benign biopsy 5 years ago of the right breast.

FINDINGS: Heterogeneously dense breast tissue in the right breast. No new mass, architectural distortion or suspicious calcifications. Stable benign appearing calcifications in the upper outer quadrant, consistent with prior biopsy site. No significant interval change compared to prior exams. 

Targeted ultrasound of the right breast was performed. No sonographic correlate for the stable calcifications in the upper outer quadrant. No suspicious mass or other abnormality identified.

IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. Stable benign calcifications at prior biopsy site. 
2. Targeted right breast ultrasound is negative.

BIRADS 2 - Benign findings. Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
INDICATION: Annual screening
COMPARISON: 05/18/2020
TECHNIQUE: Standard CC and MLO views of the breasts were obtained.
FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.
In the left breast, there is a new cluster of amorphous calcifications in the upper outer quadrant at 2 o'clock, 8 cm from the nipple. Additional magnification views in the CC and lateral projections confirm the presence of a group of amorphous calcifications spanning an area of approximately 8 mm. No associated mass or distortion is identified.
The right breast is unremarkable. No suspicious masses, calcifications, or areas of architectural distortion.
IMPRESSION:
BI-RADS Category 0: Incomplete: Need additional imaging evaluation - left breast only.
New cluster of amorphous calcifications in the left breast upper outer quadrant, indeterminate in etiology. Recommend left diagnostic mammogram with magnification views and ultrasound for further characterization. Overall assessment is BI-RADS 0, incomplete.
Benign findings right breast, BI-RADS 2.
ADDENDUM:
LEFT DIAGNOSTIC MAMMOGRAM AND ULTRASOUND
Left diagnostic mammographic magnification CC and lateral views demonstrate the cluster of amorphous calcifications at 2 o'clock upper outer quadrant left breast, spanning 8 mm. The distribution is grouped.
Targeted left breast ultrasound at 2:00, 8 cm from the nipple demonstrates normal fibroglandular tissue with no sonographic correlate for the mammographic calcifications. No suspicious mass or irregular hypoechoic area identified.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast.
New grouped amorphous calcifications left breast at 2:00, 8 cm from the nipple with no sonographic correlate. Suspicious for DCIS.  Stereotactic biopsy recommended. Overall assessment is BI-RADS 4, suspicious.
Benign right breast, BI-RADS 2. ➡ *Positive*
- MAMMOGRAM REPORT
HISTORY: 48 year old female presenting for routine annual screening mammogram. No prior breast cancer history. No reported breast changes or symptoms.
FINDINGS:
Both breasts are heterogeneously dense which could obscure detection of small masses. 
No dominant masses, architectural distortion or suspicious calcifications are identified in either breast.
There are stable bilateral scattered fibroglandular densities. Benign-appearing lymph nodes noted in both axillae.
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Recommend continuing annual screening mammography per guidelines for average risk women.
BIRADS 1 - NEGATIVE. ➡ *Negative*
- CLINICAL HISTORY: 47-year-old female presents for screening mammogram. No complaints or palpable abnormalities. Has family history of breast cancer in maternal aunt.

FINDINGS:
Right breast:
The breast appears heterogeneously dense on mammography, which may obscure detection of small masses. There are no dominant masses, suspicious calcifications, or areas of architectural distortion. 

Left breast:
The left breast also appears heterogeneously dense. No dominant masses, suspicious calcifications, or areas of architectural distortion are identified.

Comparison is made with prior studies from 9/15/2022 and no significant interval change is seen.

Targeted ultrasound was performed on the left breast at the 10 o'clock position in the region of patient reported tenderness. No sonographic correlate or suspicious finding was identified.

IMPRESSION: 
1. No mammographic evidence of malignancy in either breast. Recommend continued annual screening in 12 months.
2. Heterogeneously dense breast tissue bilaterally, which can lower the sensitivity of mammography. 
3. No sonographic abnormality in area of clinical concern in the left breast.

BIRADS Category 1: Negative
Recommended follow up: Routine screening in 12 months

ADDENDUM: Patient was notified of results via patient portal. Given family history, patient may benefit from supplemental screening with MRI if lifetime risk exceeds 20%. Will generate risk score and notify referring provider with recommendations. ➡ *Negative*
- TMIST SCREENING MAMMOGRAM OBSP 
This patient is a participant in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST). She was randomized to the full field digital mammography (FFDM) arm of the study following informed consent. 
Comparison to prior exams: Available.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
OBSP routine screening interval recommended.
There are no mammographic abnormalities suspicious for malignancy identified.  
BIRADS Category 1: Negative. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: No.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
BIRADS-1: No suspicious findings.
No significant masses, architectural distortion or dominant focal asymmetry.
No mention of microcalcifications in the report.
Recommend routine screening in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Patient is a 45-year-old female presenting for routine screening mammogram.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

LEFT BREAST:
There is no evidence of a mass, architectural distortion, or abnormal calcifications.

RIGHT BREAST: 
There are scattered groups of amorphous microcalcifications in the upper outer quadrant, ranging in size from 0.2-0.4 mm. The largest cluster measures approximately 5 mm. No associated mass, architectural distortion, or skin thickening is identified.

IMPRESSION:
RIGHT BREAST: Scattered groups of amorphous microcalcifications in the upper outer quadrant. Additional magnification views recommended for further evaluation. BI-RADS 0, incomplete. 

LEFT BREAST: No mammographic evidence of malignancy. BI-RADS 1, negative.

ADDENDUM: Additional magnification views of the right breast were performed. The amorphous microcalcifications in the upper outer quadrant are more apparent, with the largest cluster measuring 7 mm. Stereotactic biopsy is recommended for tissue diagnosis. BI-RADS 4, suspicious. ➡ *Positive*
- FINDINGS:

Bilateral digital mammograms demonstrate scattered fibroglandular densities. 

In the left breast at 10:00, 3 cm from the nipple, there is a small group of amorphous microcalcifications spanning 6 mm. No associated mass or architectural distortion is identified.

The right breast demonstrates no suspicious masses, architectural distortion, or calcifications.

There are no appreciable skin thickening or nipple retraction bilaterally.  No axillary lymphadenopathy is appreciated.

IMPRESSION:
1. Small group of amorphous microcalcifications left breast 10:00, 3 cm from the nipple, spanning 6 mm. Additional magnification views recommended for further evaluation. BI-RADS 0.
2. No suspicious findings right breast. BI-RADS 2.

ADDENDUM: Magnification views were performed on the left breast in the mediolateral and craniocaudal projections following administration of a skin marker over the area of interest. The marker is seen to overlie a small group of indistinct microcalcifications measuring approximately 6 mm in size at 10 o'clock, 3 cm from the nipple. Stereotactic biopsy is recommended for tissue diagnosis. Final BI-RADS assessment left breast is BI-RADS 4. BI-RADS assessment right breast remains BI-RADS 2. ➡ *Positive*
- MAMMOGRAM REPORT - LEFT BREAST
HISTORY: Patient reports new lump in the upper outer quadrant of the left breast, first noticed 2 weeks ago. No prior mammograms available for comparison. Family history of breast cancer in maternal grandmother.
FINDINGS:
Left breast composed of scattered areas of fibroglandular density. No dominant mass, architectural distortion or suspicious calcifications. Skin and nipple are unremarkable.
Targeted ultrasound of the palpable area in the left upper outer quadrant demonstrates a 0.8 cm oval circumscribed hypoechoic mass, likely representing a benign fibroadenoma. No suspicious features identified.
IMPRESSION:
1. No mammographic evidence of malignancy in the left breast.
2. 0.8 cm benign-appearing mass in the left upper outer quadrant on ultrasound, corresponding to the palpable finding. Recommend clinical follow up in 3-6 months to document stability. 
BIRADS 2 - Benign findings. Recommend routine screening in 1 year unless clinically indicated sooner based on the palpable finding. ➡ *Negative*
- FINDINGS:
The breast tissue is predominantly fatty with scattered areas of fibroglandular density. There are no focal masses, architectural distortion, or suspicious calcifications. The skin and nipple are unremarkable.

IMPRESSION:
Negative mammogram. No evidence of malignancy. Scattered fibroglandular densities, which are a normal finding.

BIRADS 1 - Recommend routine screening in 1 year. ➡ *Negative*
- SCREENING BILATERAL MAMMOGRAM
CLINICAL HISTORY: 55-year-old female status post lumpectomy right breast for invasive ductal carcinoma with negative margins. Annual screening bilateral mammogram.
COMPARISON: Lumpectomy right breast June 4, 2018. No previous mammograms are available for comparison.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
RIGHT BREAST: Fatty replaced breast tissue. Postsurgical changes are seen in upper outer right breast corresponding to known lumpectomy site. There is distortion of the breast contour and skin thickening. No new suspicious mass, architectural distortion or calcifications.
LEFT BREAST: Scattered fibroglandular densities. There is a new cluster of pleomorphic calcifications in the upper inner left breast at 10:00, 8 cm from the nipple measuring 9 mm. No suspicious mass or architectural distortion.
IMPRESSION:
RIGHT BREAST: Postsurgical changes from prior lumpectomy. No radiographic evidence of recurrence. BI-RADS 2: Benign.
LEFT BREAST: New cluster of pleomorphic calcifications upper inner left breast. Suspicious abnormality. Additional imaging evaluation needed. BI-RADS 4: Suspicious.
RECOMMENDATION: Magnification views left breast. Stereotactic core biopsy left breast calcifications. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening

TECHNIQUE: Bilateral CC and MLO views. Additional spot compression magnification (SCM) views of the right breast.

COMPARISON: Prior mammogram from 08/15/2019 at this facility.

FINDINGS:
The breasts are heterogeneously dense (C), which may obscure small masses.

Right breast:
There is a new 6 mm cluster of fine pleomorphic microcalcifications in the upper outer quadrant, approximately 3 cm from the nipple on the CC view. This is triangulated to the 10 o'clock position, middle third depth on the MLO view. Additional SCM views in the CC and ML projections better demonstrate the morphology and distribution of these microcalcifications, which are concerning for ductal carcinoma in situ (DCIS).

No dominant mass, architectural distortion, or skin thickening.

Left breast:
Stable, scattered fibroglandular densities. No suspicious masses, calcifications, or architectural distortion.

IMPRESSION:
BI-RADS 4B: Suspicious microcalcifications right breast at 10 o'clock, middle third depth. Stereotactic-guided biopsy is recommended for further evaluation.

BI-RADS 2: Benign findings left breast. Routine screening in 1 year is recommended.

ADDENDUM:
The patient was immediately informed of the results and agrees to proceed with stereotactic-guided biopsy of the right breast microcalcifications. The biopsy has been scheduled for next week, and the patient will receive instructions and preparation details. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: Screening mammogram from 4/2020. 
FINDINGS:
The breasts are heterogeneously dense (C), which may obscure detection of small masses.
There are stable scattered areas of benign appearing calcifications in both breasts.
A new cluster of amorphous and pleomorphic microcalcifications is noted in the upper outer quadrant of the left breast at the 2:00 position, approximately 6cm from the nipple. This measures about 6mm in size. No associated mass, distortion or skin thickening is seen.
The visualized axillae appear unremarkable, with no dominant mass or adenopathy.  
IMPRESSION:  
BI-RADS 4: Suspicious abnormality in the left breast - biopsy should be considered. A new cluster of microcalcifications is identified in the upper outer left breast, which is suspicious and requires further evaluation.
BI-RADS 2: Benign findings in the right breast - recommend continued routine screening in 1 year.
ADDENDUM REPORT FOLLOWING LEFT BREAST STEREOTACTIC BIOPSY:
TECHNIQUE: Using stereotactic guidance, specimens were obtained from the cluster of microcalcifications in the left breast at 2:00, 6cm from the nipple using an 8G vacuum-assisted device. Specimen radiograph confirmed retrieval of microcalcifications. Marker clip placed. The patient tolerated the procedure well. 
IMPRESSION: Successful stereotactic biopsy of left breast microcalcifications. Pathology results are pending and will be communicated in an addendum report. ➡ *Positive*
- ULTRASOUND AND MAMMOGRAM BREASTS BOTH SIDES
HISTORY: 52 year old female with history of BRCA1 mutation.
COMPARISON: Annual screening mammograms from 2 years ago at outside hospital.

FINDINGS:
The breasts are heterogeneously dense.
There are no new masses, architectural distortion, or suspicious findings identified in either breast.
On ultrasound, there are bilateral scattered benign appearing cysts.
At the 10:00 position in the left breast 5 cm from the nipple there is a group of worrisome microcalcifications spanning 8 mm. Spot compression magnification views were obtained. The calcifications appear linear and branching, highly suspicious for ductal carcinoma in situ.
At the 3:00 position 3 cm from the nipple in the right breast, there are two small oval hypoechoic solid masses measuring 5 and 6 mm. These demonstrate no internal vascularity and are likely benign fibroadenomas. Short term follow up in 6 months is recommended for these lesions.
The remaining right breast tissue is unremarkable. No lymphadenopathy is seen.

BIRADS ASSESSMENT:
Right breast: BI-RADS 3. Probably benign findings. Short interval follow up recommended.
Left breast: BI-RADS 4C. Highly suspicious for malignancy. Stereotactic core biopsy recommended.

ADDENDUM:
Stereotactic core biopsy of the suspicious left breast calcifications performed under ultrasound guidance using 14 gauge needle. Specimen radiograph confirms calcifications in sample. Pathology specimen labeled "Left breast, 10:00, 5 cm from nipple" and sent to pathology. The patient tolerated the procedure well. Hemostasis achieved with manual compression. Mammographic markers placed at biopsy site. The patient was instructed on post-procedure care and will await pathology results. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAPHY

CLINICAL HISTORY: Right breast lump.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There is no suspicious mass, architectural distortion, or asymmetry in either breast.

In the right breast at 10:00, 6 cm from the nipple, there is a group of fine pleomorphic calcifications in a linear distribution spanning approximately 8 mm. Additional magnification views confirm these findings. No other suspicious calcifications are identified.

IMPRESSION:
BI-RADS 4B: Suspicious calcifications in the right breast as described above. Stereotactic biopsy is recommended for further evaluation.

ADDENDUM:
The results of the stereotactic biopsy of the right breast calcifications showed ductal carcinoma in situ, intermediate nuclear grade, involving a core biopsy. The patient will proceed with definitive surgical management. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH TOMOSYNTHESIS
INDICATION: Annual screening mammogram.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense. This may limit sensitivity of mammography.
At the 10:00 position of the right breast, there is a new group of pleomorphic microcalcifications spanning an area of 1 cm, located approximately 6 cm from the nipple. Additional magnification views confirm the presence of these calcifications.
No dominant masses, architectural distortion or other suspicious microcalcifications are identified in either breast.
IMPRESSION:
BI-RADS: 4B
Suspicious group of pleomorphic microcalcifications in the right breast at 10:00, 6 cm from the nipple. Stereotactic core biopsy is recommended for further evaluation. The patient should return to the breast center for additional imaging and work-up.
ADDENDUM:
The patient returned for stereotactic core biopsy of the group of microcalcifications in the right breast at 10:00. Specimen radiograph confirmed the presence of calcifications within the core samples.
FINAL ASSESSMENT: BI-RADS 4B for the right breast. BI-RADS 2 for the left breast. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM. Clinical History: Patient is a 42 year old female presenting for a screening mammogram. Technique: Bilateral CC and MLO images were obtained. Comparison is made to prior mammographic examination from 12/28/2023. Findings: The breasts are heterogeneously dense. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: Negative mammogram. BI-RADS 1. ➡ *Negative*
- MAMMOGRAM, BILATERAL

INDICATION: Right breast mass.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Magnification views were obtained of a cluster of calcifications in the right breast at 10 o'clock.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.

LEFT BREAST:
No dominant mass, architectural distortion, or suspicious calcifications. 

RIGHT BREAST: 
At 10 o'clock, 6 cm from the nipple, there is a 1.2 cm irregular mass with indistinct margins. 
At 10 o'clock, 3 cm from the nipple, there is a cluster of pleomorphic microcalcifications measuring 7 mm. 
No axillary lymphadenopathy.

IMPRESSION:
RIGHT BREAST: 
- 1.2 cm irregular mass at 10 o'clock, suspicious for malignancy. Biopsy recommended. BI-RADS 4.
- 7 mm cluster of pleomorphic microcalcifications at 10 o'clock, suspicious for malignancy. Biopsy recommended. BI-RADS 4.

LEFT BREAST: 
No mammographic evidence of malignancy. BI-RADS 1.

RECOMMENDATIONS: 
Ultrasound-guided core biopsy of the 10 o'clock mass and stereotactic biopsy of the 10 o'clock calcifications in the right breast is recommended. 

ADDENDUM: Pathology results from right breast mass biopsy showed invasive ductal carcinoma, grade 2. The calcifications were ductal carcinoma in situ, high nuclear grade. BI-RADS 6. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM. History: Screening. Comparison is made to prior exams from (outside institution) 01/24/2024. Findings: The breasts are extremely dense, which can obscure small masses. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: Negative mammogram. BI-RADS 1. Note: Due to the density of the breasts, mammography may be difficult to interpret and may miss some cancers. ➡ *Negative*
- Breast Ultrasound Report
Patient Name: [Patient Name]
Date of Exam: 2024-07-05

Clinical History:
Patient referred for breast ultrasound evaluation due to suspicious calcifications identified on screening mammography.

Comparison: Mammogram dated 06/20/2024.

Findings:
Left breast: No distinct masses or other suspicious abnormalities are identified in the left breast.
Right breast: In the right breast, at the location of the suspicious calcifications seen on mammography (10 o'clock position, approximately 3 cm from the nipple), there is a small, ill-defined area of hypoechogenicity measuring approximately 5 mm in greatest dimension. This area corresponds to the cluster of calcifications. No distinct mass is visualized. There is no associated ductal dilation.

Impression:
The ultrasound findings in the right breast are consistent with the mammographically detected calcifications. The presence of the ill-defined hypoechoic area is of uncertain significance and could represent a small focus of malignancy. Given the concerning mammographic features, biopsy is recommended for definitive diagnosis.

Recommendations:
Ultrasound-guided biopsy of the right breast calcifications.

BI-RADS Category: 4 (Suspicious for Malignancy) ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.  

TECHNIQUE:  Two standard CC and MLO views of each breast were obtained. 

COMPARISON:  Comparison is made to previous exam dated 2/15/2021.

FINDINGS:  
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast. 

There is a new cluster of fine pleomorphic microcalcifications in the upper outer right breast, spanning an area of approximately 6 mm. These microcalcifications are new since the prior exam and are of concern. 

Stable benign-appearing calcifications are noted in the left breast. No other significant change from prior exam.

IMPRESSION:  
BI-RADS 4: Suspicious abnormality in the right breast. Stereotactic biopsy is recommended for the new cluster of microcalcifications in the upper outer right breast.

MANAGEMENT:
The patient was notified of the results. Stereotactic biopsy of the right breast will be scheduled. Routine annual screening of the left breast is recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM WITH 3D TOMOSYNTHESIS VIEWS
INDICATION: Annual screening mammogram. 
TECHNIQUE: Bilateral standard screening CC and MLO views supplemented with 3D tomosynthesis views. 
COMPARISON: None available.
BREAST COMPOSITION: The breasts are heterogeneously dense, which may obscure small masses.
FINDINGS:
RIGHT BREAST: There are no suspicious masses, calcifications, or other findings. 
LEFT BREAST: In the upper inner left breast there is a 6 mm cluster of amorphous calcifications at 9:00 6 cm from the nipple. There are no associated masses, architectural distortion, or other suspicious findings. The remaining left breast is unremarkable. 
IMPRESSION:
Right breast, negative. BI-RADS 1.
Left breast, new cluster of amorphous calcifications upper inner quadrant, suspicious. BIRADS 4. Recommend stereotactic core biopsy. 
ADDENDUM (Post-biopsy):
Stereotactic core biopsy of the left breast calcifications at 9:00 6 cm from the nipple was performed. Specimen radiograph confirms calcifications.
PATHOLOGY: Ductal carcinoma in-situ, intermediate grade, cribriform type. ➡ *Positive*
- ULTRASOUND LEFT BREAST

CLINICAL INFORMATION: 40 year old female with small palpable lump laterally at 3 o'clock. No personal or family history of breast cancer.

FINDINGS:
Targeted ultrasound over the palpable area at 3 o'clock in the left breast reveals normal fibroglandular breast tissue. No dominant solid or cystic mass, architectural distortion, skin thickening or suspicious calcifications are seen. 

The remainder of the left breast is heterogeneously dense, limiting sensitivity of mammogram. No dominant masses, areas of architectural distortion or suspicious calcifications are identified.

Lymph nodes in the left axilla appear normal.

IMPRESSION:
No sonographic evidence of malignancy at the area of palpable concern at 3 o'clock left breast. Recommend clinical follow up and routine screening mammography. 

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY AND ULTRASOUND OF THE LEFT BREAST

PATIENT HISTORY:
Patient presents with a palpable lump in the upper outer quadrant of the left breast, noted on self-examination. No prior history of breast cancer or breast surgery. Maternal aunt with history of breast cancer.

FINDINGS:
Left breast is composed of heterogeneously dense breast tissue. No dominant mass, suspicious calcifications, or architectural distortion identified.

Targeted ultrasound was performed over the palpable area of concern in the left upper outer breast. A 0.8 x 0.6 x 0.5 cm oval, circumscribed, hypoechoic mass was identified at the 10 o'clock position, 6 cm from the nipple, corresponding to the palpable finding. The mass demonstrates no posterior acoustic features and no internal vascularity. 

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend annual screening mammography.
2. Benign-appearing left breast mass at 10 o'clock, corresponding to the palpable finding, most consistent with a fibroadenoma. Short-term follow-up ultrasound in 6 months is recommended to confirm stability. 

BI-RADS CATEGORY 2 - Benign finding. Recommend routine screening. ➡ *Negative*
- BIRADS MAMMOGRAPHY REPORT

CLINICAL INDICATION: 48 year old female presenting for routine mammographic screening. No palpable abnormality noted.

COMPARISON: Comparison made to outside screening mammograms dated 4/15/2021, 4/20/2022. 

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications.

Stable coarse benign calcifications noted in the right breast middle depth middle and lower outer quadrants.  

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend annual screening mammography in 12 months. 

ASSESSMENT: 
BI-RADS Category 1: Negative.

ADDENDUM: Reviewed and electronically signed by [Doctor Name, MD] on [Report Date]. ➡ *Negative*
- FINDINGS:
Right and left breasts contain heterogeneously dense fibroglandular tissue (ACR C).
No focal mass, architectural distortion or suspicious calcifications are present in either breast.
Bilateral benign-appearing lymph nodes are noted in the axillae.
There are benign vascular calcifications in the left breast.
No significant interval change compared with prior exam of December 15, 2021.

IMPRESSION:
No suspicious findings. Specifically no microcalcifications.
Recommend continued annual screening mammography in 12 months.

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY REPORT

CLINICAL INFORMATION: Annual screening mammogram. No palpable breast masses on self-examination.

COMPARISON: 2 years.

FINDINGS:
Both breasts are almost entirely fatty.

Right breast:
There is an oval circumscribed isodense mass in the upper outer quadrant measuring 10 mm. This is stable in appearance compared to prior mammogram and was previously assessed as a benign finding, likely fibroadenoma. 
Overlying skin is normal. No suspicious calcifications.

Left breast:
There is a new group of amorphous microcalcifications in the upper inner quadrant, spanning an area of 5 mm. This is a new finding compared to prior studies.
The breast is otherwise unremarkable with no dominant masses, architectural distortion or skin thickening.

IMPRESSION:
Left breast: New group of amorphous microcalcifications in the upper inner quadrant, spanning 5 mm. Additional magnification views recommended for further evaluation. BI-RADS 0.

Right breast: Stable benign appearing mass, likely fibroadenoma. BI-RADS 2.

ADDENDUM:
Magnification views of the left breast demonstrate that the microcalcifications are indeterminate in morphology. Stereotactic biopsy is recommended for tissue diagnosis. 
Final BI-RADS assessment of the LEFT BREAST: BI-RADS 4. ➡ *Positive*
- REPORT:
"RIGHT MAMMOGRAM AND ULTRASOUND
HISTORY: Patient with palpable mass in right breast at 2 o'clock. Last normal mammogram 1 year prior.
FINDINGS:
The right breast is heterogeneously dense. No suspicious masses, architectural distortion, or calcifications. Stable post-surgical changes in the right breast from prior lumpectomy. Targeted right breast ultrasound demonstrates normal fibroglandular tissue with no discrete mass at the palpable area of concern.
IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. Recommend annual screening mammography.
2. Targeted right breast ultrasound is negative for focal abnormality at the area of palpable concern. Likely represents normal fibroglandular tissue.
BI-RADS 1 - Negative.
ADDENDUM: Comparison made to prior outside mammogram. No significant interval change." ➡ *Negative*
- BILATERAL MAMMOGRAM WITH TOMOSYNTHESIS AND ULTRASOUND

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammograms from 2019.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are scattered areas of fibroglandular density bilaterally. No dominant mass, suspicious calcifications, or other findings.

At the 10 o'clock position of the left breast, approximately 6 cm from the nipple, there is a new group of amorphous calcifications spanning an area of about 8 mm. These were not definitely seen on the prior study and are suspicious for early DCIS. Additional magnification views confirm these findings.

Supplemental screening ultrasound of both breasts was performed, showing no suspicious masses or other findings. Ultrasound is limited in evaluation of calcifications.

IMPRESSION:
1. New group of suspicious calcifications at the 10:00 position of the left breast, spanning 8 mm. Additional magnification views are suspicious for early DCIS. Stereotactic biopsy is recommended. BIRADS 4B.

2. Remainder of both breasts stable and benign. BIRADS 2.

ADDENDUM: The patient returned for stereotactic biopsy of the suspicious calcifications in the left breast, which confirmed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. The patient will be referred to breast surgery for further management. ➡ *Positive*
- LEFT MAMMOGRAM AND ULTRASOUND  
HISTORY: Patient called back from screening for additional imaging. Last screening mammogram 6 months ago - benign findings in both breasts. No prior breast surgeries or family history of breast cancer.
FINDINGS: Bilateral mammograms obtained. Both breasts are predominantly fatty replaced. There are no masses, architectural distortion or suspicious calcifications in either breast. Stable benign calcifications are noted in both breasts. 
Targeted left breast ultrasound was performed at the site of the patient's palpable abnormality at 2 o'clock, 4 cm from the nipple. This corresponds to a 6 mm simple cyst. No solid component or flow is identified. 
IMPRESSION:
1. Negative bilateral screening mammogram. The breasts are predominantly fatty replaced. 
2. Left breast 6 mm simple cyst corresponding to the palpable abnormality. This is a benign finding. 
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- SCREENING MAMMOGRAM

INDICATION:  Annual screening 

TECHNIQUE:  Screening bilateral full field digital mammography with tomosynthesis.  

COMPARISON:  None available.

FINDINGS:  
The breasts are heterogeneously dense, which could obscure detection of small masses.

There are no dominant masses, suspicious calcifications, or other findings suspicious for malignancy.  

Benign scattered fibroglandular densities are noted bilaterally.

IMPRESSION:
Negative bilateral screening mammogram.  

BIRADS Category 1 - Negative. Routine screening is recommended in 1 year.

ADDENDUM: 
The patient's lifetime risk will be evaluated using a risk assessment model.  If her lifetime risk is greater than 20%, supplemental screening with breast MRI will be recommended in addition to annual mammography. ➡ *Not Stated*
- MAMMOGRAPHY - BILATERAL - SCREENING. ➡ *Not Stated*
- CLINICAL HISTORY: 55-year-old female presenting for a routine screening mammography. No known palpable masses or other breast concerns. Last mammogram performed 1 year prior was unremarkable.

FINDINGS:
Bilateral digital 2D mammographic views were obtained including craniocaudal and mediolateral oblique projections.

Right breast: The breast is predominantly fatty replaced. There are no dominant masses, architectural distortion, or suspicious calcifications. Stable.

Left breast: The breast consists of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications. Stable.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. No mammographic evidence of malignancy. 
3. Recommend continued annual screening mammography per guidelines. 

BIRADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAM - BILATERAL SCREENING

CLINICAL HISTORY: 58-year-old female presents for her annual screening mammogram. She has no breast complaints and no family history of breast cancer. Her last mammogram was performed elsewhere and is unavailable for review.

FINDINGS:

The breasts are heterogeneously dense. There are scattered benign-appearing calcifications in both breasts. In the right breast, there is a new cluster of fine, pleomorphic calcifications measuring 8 mm in diameter located at the 3 o'clock position, 2 cm from the nipple.

IMPRESSION:

1. New cluster of fine, pleomorphic calcifications in the right breast. Recommend spot magnification views for further evaluation. 

BI-RADS: 0 (Needs additional imaging). ➡ *Positive*
- FINDINGS:
Right breast: There is a right mastectomy with no mammographic abnormality.
Left breast: The breast is composed of scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications are identified.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Right mastectomy.
ASSESSMENT: BI-RADS 1 - Negative. Recommend continued annual screening mammography in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Annual screening.

COMPARISON: None available.

FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in the right breast.

At the 10:00 position of the left breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. This cluster of microcalcifications is new compared to prior exams and is suspicious for early malignancy.

The remainder of the left breast is unremarkable. No dominant masses, areas of architectural distortion, or additional suspicious calcifications are identified.

IMPRESSION:
1. New group of suspicious microcalcifications in the left breast at 10:00, 6 cm from the nipple, likely representing ductal carcinoma in situ (DCIS). Additional imaging evaluation with magnification views is recommended for further characterization. Stereotactic biopsy may be necessary for definitive diagnosis. BI-RADS 4B.

2. The right breast is negative. BI-RADS 1.

RECOMMENDATION:
Diagnostic mammogram with magnification views of the left breast for further evaluation of the suspicious microcalcifications. Consider stereotactic biopsy if indicated after additional imaging. Routine annual screening of the right breast in 1 year. ➡ *Positive*
- MAMMOGRAM AND ULTRASOUND REPORT
CLINICAL HISTORY: No breast tenderness or pain. Family history of breast cancer in maternal aunt. Prior imaging for comparison.
FINDINGS:
Bilateral mammographic views demonstrate scattered fibroglandular densities bilaterally. No dominant mass, architectural distortion, or suspicious calcifications are identified.  
Targeted ultrasound of the right breast at the site of clinical concern, 10 o'clock position 4 cm from the nipple, demonstrates normal fibroglandular tissue with no sonographic abnormality.
IMPRESSION:
1. No mammographic or sonographic evidence of malignancy. 
2. No sonographic correlate for the palpable abnormality in the right breast 10 o'clock position. Recommend clinical follow up and correlation.
BI-RADS Category 1: Negative. Routine screening in 1 year. ➡ *Negative*
- REPORT: "NORMAL SCREENING MAMMOGRAM.
Compared to previous screening exam dated 15 months ago, no interval change.
Breast composition is heterogeneously dense, which may obscure detection of small masses.
There are no masses, architectural distortion, or suspicious calcifications.
BIRADS-2. Routine screening recommended in 1 year.

Addendum:
Scattered vascular calcifications are noted bilaterally, unchanged from prior exam. These are benign.
No microcalcifications warranting biopsy are identified in either breast." ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are almost entirely fatty.

There are no dominant masses, areas of architectural distortion or suspicious clustered microcalcifications in either breast. 

A few vascular calcifications are noted bilaterally.

Within the left breast at the 9:00 position there is a cluster of pleomorphic microcalcifications that measures approximately 4 mm in greatest dimension. This cluster was not present on the prior exam.

The skin is normal in thickness. No skin lesions are evident. The nipples are within normal limits. No enlarged axillary lymph nodes are evident.

IMPRESSION:
BIRADS 0: Incomplete, requires additional evaluation. A cluster of pleomorphic microcalcifications is noted in the left breast at 9:00 which warrants further diagnostic evaluation with spot compression magnification views to better assess for any additional calcifications and their distribution.

ADDENDUM: 
Spot compression magnification views of the left breast were performed, demonstrating a cluster of pleomorphic microcalcifications spanning an area of approximately 6mm at the 9:00 position of the left breast, 6 cm from the nipple. Stereotactic biopsy is recommended for tissue diagnosis.

BIRADS 4: Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- SCREENING MAMMOGRAM
INDICATION: Screening.  
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: 10/15/2020.
FINDINGS: 
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.  
There are no dominant masses, suspicious calcifications or architectural distortion.  
Stable bilateral benign-appearing calcifications are noted.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breast tissue.
RECOMMENDATION: 
Recommend routine screening mammography in 1 year.
BIRADS 2 - Benign finding.
ADDENDUM:
After review of this report, it was noted that presence or absence of microcalcifications are not explicitly mentioned in the findings. For completeness, there are no suspicious microcalcifications. The report and BIRADS category remain unchanged. ➡ *Not Stated*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening

COMPARISON: Mammogram from 1 year ago

FINDINGS:
Both breasts show scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications are identified in either breast.

Stable benign-appearing calcifications are present in the left breast, likely representing fibrocystic change.

There are no significant changes compared to prior mammogram.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography.
2. Stable benign-appearing calcifications in the left breast, likely representing fibrocystic change.

BIRADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- FINDINGS:
Current bilateral diagnostic digital mammograms are compared to outside FFDM performed 01/18/2018.
The breasts are predominantly fatty, with scattered areas of fibroglandular density bilaterally.
In the right breast, the mammographic findings are benign. There are no significant masses, calcifications, or areas of architectural distortion.
In the left breast, there is a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant at approximately 1 o'clock, 6 cm from the nipple. The calcifications span an area measuring approximately 5 x 4 x 4 mm (CC x MLO x ML). No associated mass or architectural distortion is seen in this area. No other suspicious mammographic findings are appreciated in the left breast.
Magnification views of the left breast in the CC and MLO projections confirm the presence of the cluster of microcalcifications as described above.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast - biopsy should be considered. Stereotactic biopsy is recommended for the cluster of microcalcifications in the left breast at 1:00, 6 cm from the nipple.
BI-RADS Category 2: Benign mammographic findings in the right breast. Routine screening in 1 year is recommended.
ADDENDUM (Post-Biopsy):
Stereotactic core biopsy of the cluster of microcalcifications in the left breast at 1:00, 6 cm from the nipple was performed. Pathology results demonstrate ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. Surgical consultation is advised for further management.
FINAL ASSESSMENT:
BI-RADS Category 6: Known biopsy-proven malignancy (DCIS) in the left breast. Appropriate action should be taken. ➡ *Positive*
- BILATERAL MAMMOGRAM

CLINICAL HISTORY: Patient is s/p chemo and radiation for left breast cancer. Presents today for routine follow up.

COMPARISON: Prior mammogram 10/25/2023.

FINDINGS: The breasts are heterogeneously dense, which could obscure a small mass. There are scattered benign appearing calcifications in both breasts, unchanged. There is a new 7 mm cluster of fine pleomorphic calcifications in the right breast at the 3 o'clock position, 2 cm from the nipple.

IMPRESSION:

1. New cluster of fine pleomorphic calcifications in the right breast. Recommend spot magnification views for further evaluation.

BI-RADS: 0 (Needs additional imaging). ➡ *Positive*
- BILATERAL MAMMOGRAM FINDINGS:

RIGHT BREAST:
The breast tissue is heterogeneously dense. No dominant masses, architectural distortion, or suspicious calcifications are seen.

LEFT BREAST:
The breast tissue is heterogeneously dense. A cluster of amorphous microcalcifications measuring 6 mm is identified in the upper outer quadrant at the 2:00 position, approximately 8 cm from the nipple. The microcalcifications are new compared to the previous mammogram from 2 years ago. No associated mass, architectural distortion, or skin thickening is identified.

IMPRESSION:
1. Left breast: New cluster of amorphous microcalcifications in the upper outer quadrant, suspicious for malignancy. BI-RADS 4B. Recommend stereotactic biopsy for further evaluation.
2. Right breast: No mammographic evidence of malignancy. BI-RADS 2, benign finding.

ADDENDUM:
The patient underwent stereotactic biopsy of the left breast microcalcifications. Pathology results showed ductal carcinoma in situ (DCIS), intermediate grade, cribriform type. The patient will proceed with surgical consultation for further management. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM.

INDICATION: Annual screening mammogram.  

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: 12/02/2020. 

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications.

Bilateral benign-appearing lymph nodes are noted in the axillary regions.

IMPRESSION: 
Negative bilateral screening mammogram. 

RECOMMENDATIONS:
Continue annual screening mammography in 12 months.

BI-RADS Category 1: Negative. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
The breast tissue is of scattered fibroglandular density. No significant masses, architectural distortions or suspicious calcifications are identified.  
BIRADS-2.

Addendum: 
There is no mention of microcalcifications in this screening mammogram report. The breast parenchyma demonstrates scattered fibroglandular density without any suspicious findings. Routine screening is recommended in 2 years as per OBSP guidelines for this patient. Final assessment is BIRADS 2 - Benign findings. ➡ *Not Stated*
- FINDINGS:
Right breast: Negative.
Left breast: Scattered fibroglandular density breasts. No dominant mass, architectural distortion or suspicious calcifications. Stable appearance compared to prior exams. 

Left breast ultrasound: Targeted ultrasound of the left breast was performed showing no sonographic abnormality in the area of concern. No suspicious mass or complex cystic lesion identified.

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend annual screening mammography. 
2. No sonographic correlate for the palpable abnormality in the left breast at 7 o'clock position. Recommend clinical follow up and correlation in 2-3 months. Sooner if clinically indicated.

BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL.

INDICATION: Family history of breast cancer. High risk screening.

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
Right breast: The breast is composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications. 

Left breast: The breast is composed of scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. No mammographic evidence of malignancy.

RECOMMENDATIONS: 
Recommend continued annual screening mammography in 12 months, given patient's high risk status due to family history of breast cancer. 

BI-RADS CATEGORY 1: Negative. ➡ *Negative*
- FINDINGS: The left breast consists of scattered fibroglandular densities. No dominant masses, architectural distortion or suspicious microcalcifications are identified.

Targeted ultrasound of the left breast at 7 o'clock, corresponding to the area of palpable concern, demonstrates normal fibroglandular tissue with no sonographic correlate for the palpable finding. No solid or cystic masses are identified.

The right breast is absent, consistent with history of right mastectomy.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. No suspicious microcalcifications. 
2. Expected post-surgical changes of right mastectomy.

BIRADS 1 - Negative mammogram. Recommend annual screening mammography in 1 year. ➡ *Negative*
- BILATERAL MAMMOGRAM WITH TOMOSYNTHESIS:
INDICATION: Annual screening; Mother had breast cancer in early 50's
TECHNIQUE: Standard CC and MLO views of each breast were obtained utilizing digital mammography with tomosynthesis.
COMPARISON: None available.
FINDINGS:
Both breasts are predominantly fatty with scattered areas of fibroglandular density. Comparison limited by lack of prior studies.
RIGHT BREAST:
There are no dominant masses, areas of suspicious architectural distortion, or abnormal calcifications in the right breast.
LEFT BREAST:
The left breast is similar in overall density and distribution to the right. At the 10:00 position, 4 cm from the nipple, there is a 5 mm cluster of pleomorphic microcalcifications, partially obscured on the 2D mammogram, better visualized on the tomosynthesis images. No associated mass or architectural distortion identified. No other suspicious findings in the left breast.
IMPRESSION:
BIRADS Category 4: Suspicious abnormality in the left breast. 5 mm cluster of pleomorphic microcalcifications at 10:00, 4 cm from nipple.  Stereotactic biopsy recommended for further evaluation.
BIRADS Category 2: Benign findings in the right breast. ➡ *Positive*
- SCREENING MAMMOGRAM BILATERAL

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 

FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.

There is no mammographic evidence of malignancy. Specifically, there are no suspicious masses, architectural distortions, or suspicious calcifications.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography. 

BIRADS Category 1 - Negative.

Recommend routine screening in 1 year.

ADDENDUM: Tomosynthesis was performed with no additional abnormalities identified. ➡ *Not Stated*
- TMIST BILATERAL SCREENING MAMMOGRAM. 
This patient is participating in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), approved by the institutional review board. After providing informed consent, the patient was randomly assigned to the full field digital mammography arm of the study.
Comparison to prior mammograms: Available.
Breast composition: The breasts are heterogeneously dense, which may obscure small masses.
Recall: Routine screening is recommended in 1 year.
There are no suspicious findings or abnormalities in either breast. The breast tissue is unremarkable.  
BI-RADS Category 1: Negative. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAPHY

CLINICAL HISTORY: 53 year old female, no personal or family history of breast cancer, presents for routine screening mammogram.

COMPARISON: None.

FINDINGS:
The breast tissue is heterogeneously dense. Due to this finding, mammography could mask a lesion and additional screening studies such as ultrasound may be considered. 

Right breast: The screening mammography images of the right breast demonstrate a small focal asymmetry in the upper outer quadrant posteriorly, measuring 7 mm. This corresponds to a small cluster of calcifications on spot magnification views. Given the small size and amorphous/punctate morphology, this is likely benign.

Left breast: The screening mammography images of the left breast show no suspicious mass, asymmetry or architectural distortion. 

IMPRESSIONS:
1. BI-RADS Category 3: Bilateral heterogeneously dense parenchyma, which may obscure detection of a small mass. 
2. BI-RADS Category 3: Small cluster of punctate calcifications in the posterior upper outer right breast. This is new compared to any available priors. Probably benign but recommend 6 month follow-up mammogram of the right breast for stability assessment.
3. BI-RADS Category 1: Left breast negative.

ADDENDUM: 
Short term, 6 month follow-up is recommended for the small cluster of punctate calcifications in the posterior upper outer right breast. If stable at 6 months, benign etiology confirmed and patient can resume routine yearly screening. However, any significant increase would warrant a stereotactic biopsy for definitive characterization. ➡ *Positive*
- ULTRASOUND GUIDED CORE BIOPSY LEFT BREAST
CLINICAL HISTORY: 49-year-old female for biopsy of suspicious mass and calcifications left breast 12:00, 5 cm from nipple. 
PROCEDURE: Informed consent was obtained. The skin was prepped and draped in the usual sterile fashion. 1% lidocaine was used for local anesthesia. Using ultrasound guidance, an 18-gauge spring loaded core biopsy device was used to obtain 5 core specimens from the mass at 12:00 5 cm from the nipple. Specimen radiograph showed calcifications. The specimens were placed in formalin and sent to pathology. A tissue marker clip was deployed at the biopsy site. Post procedure mammograms showed the marker clip to be in good position. The patient tolerated the procedure well. There were no immediate complications. The patient was discharged in stable condition with post-biopsy instructions.  
IMPRESSION:
Ultrasound guided core biopsy of the suspicious mass with calcifications in the 12:00 region of the left breast 5 cm from the nipple was performed. Calcifications were identified on the specimen radiograph. A marker clip was placed at the biopsy site. The pathology results are pending.

ADDENDUM: 
PATHOLOGY DIAGNOSIS: LEFT BREAST - 12:00, 5 cm FN (US GUIDED CORE BIOPSY):
A. INVASIVE DUCTAL CARCINOMA, GRADE 2, FOCUS OF GRADE 3, MEASURING AT LEAST 0.9 CM (SEE COMMENT).  
B. FIBROCYSTIC CHANGES WITH CALCIFICATIONS.
COMMENT: The tumor size is a microscopic estimate only and may not reflect the actual cancer size. 
BIRADS Category: 6 Known biopsy-proven malignancy. ➡ *Positive*
- SCREENING DIGITAL MAMMOGRAM.
Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
No suspicious clustered microcalcifications, masses or areas of architectural distortion.
The breast tissue is heterogeneously dense, which may obscure detection of small masses.
BIRADS-2. Recommend routine screening in 1 year. ➡ *Not Stated*
- ULTRASOUND GUIDED CORE BIOPSY RIGHT BREAST LESION.

CLINICAL HISTORY: 70 year old female presents with right breast 3 o'clock subareolar palpable lesion on exam. 

COMPARISON: Right diagnostic mammogram and ultrasound May 10, 2022. Prior 2D and 3D screening mammograms 2014-2021.

FINDINGS: 
Informed consent was obtained. Targeted ultrasound of the right subareolar breast at 3 o'clock reveals a hypoechoic nodule measuring 7 mm, oriented parallel to the skin. Under ultrasound guidance using aseptic technique, a 14 gauge spring loaded core biopsy device was used to obtain 4 core tissue samples from the lesion. Imaging of the core specimens confirmed adequate sampling. The specimens were placed in formalin and sent to pathology. The patient tolerated the procedure well. Post procedure mammograms showed biopsy marker clip in good position. No immediate complications.

IMPRESSION:
Right breast 3 o'clock subareolar 7 mm solid nodule biopsied under ultrasound guidance. Pathology pending.  

BI-RADS 4A. Suspicious abnormality. Biopsy performed.

ADDENDUM - PATHOLOGY: Fibroadenoma.

ADDENDUM - RADIOLOGIC-PATHOLOGIC CORRELATION: There is concordance between the imaging findings and the pathology results of fibroadenoma for the right subareolar 3 o'clock lesion. Recommend 6 month follow-up diagnostic ultrasound of this lesion. Continue annual screening mammography. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: Prior mammogram from 09/2019.

FINDINGS:

There is scattered fibroglandular density in both breasts. 

The right breast shows no suspicious masses, architectural distortion, or suspicious calcifications. 

In the left breast at the 9 o'clock position, 6 cm from the nipple, there is a new cluster of fine pleomorphic calcifications spanning an area of approximately 8 mm. This was not present on the prior study. No associated mass, architectural distortion, or skin thickening is identified. 

The remainder of the left breast is unremarkable. No suspicious masses, architectural distortion, or additional suspicious calcifications are seen. 

IMPRESSION:

RIGHT BREAST: BI-RADS 2 - Benign. 

LEFT BREAST: BI-RADS 4 - Suspicious abnormality. Stereotactic core biopsy is recommended for the new cluster of fine pleomorphic calcifications at 9 o'clock in the left breast.

ADDENDUM (POST-BIOPSY):
Stereotactic core biopsy of the left breast calcifications at 9 o'clock was performed. Pathology revealed ductal carcinoma in situ, intermediate grade. The patient will be referred to surgery for further management. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints. Remote history of right breast cyst aspiration. 
COMPARISON: Comparison is made to outside screening mammogram dated 1/15/2022.
FINDINGS:  
Right breast: Heterogeneously dense breast tissue. No dominant mass, architectural distortion, or suspicious calcifications. Stable postsurgical changes in the upper outer quadrant from prior cyst aspiration. 
Left breast: Heterogeneously dense breast tissue. No dominant mass, architectural distortion, or suspicious calcifications. 
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography in 12 months.
BIRADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- MAMMOGRAM - BILATERAL DIAGNOSTIC.
CLINICAL INDICATION: Intermittent bilateral breast pain.
COMPARISON: Mammogram from 9/18/2023.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Targeted ultrasound was performed on the right breast at the 3 o'clock position.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications in either breast. 
Stable, benign-appearing calcifications are noted bilaterally.
Ultrasound of the right breast at 3 o'clock, 4 cm from the nipple, demonstrates a 5 mm simple cyst. No solid component or suspicious features.
IMPRESSION: 
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year.
2. 5 mm simple cyst in the right breast at 3 o'clock position, benign. No further evaluation needed.
BIRADS 2: Benign findings. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION:  Routine screening.

COMPARISON: None.

FINDINGS:  
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

Grouped amorphous calcifications are noted in the upper outer quadrant of the right breast, approximately 6 cm from the nipple. These are new since the prior exam and have a suspicious morphology. Additional magnification views confirm these findings. 

Stable, benign-appearing calcifications are scattered throughout both breasts, likely representing fibrocystic change.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic core biopsy is recommended for further evaluation of the new grouped amorphous calcifications in the right upper outer quadrant.

ADDENDUM: 
The patient returned for stereotactic biopsy of the suspicious right breast calcifications. Specimen radiograph confirmed that the calcifications were sampled. Final pathology results of the core biopsy demonstrated ductal carcinoma in situ, intermediate nuclear grade, involving benign breast tissue with microcalcifications. ➡ *Positive*
- BREAST ULTRASOUND REPORT

Clinical History: Patient presents for a breast ultrasound evaluation for assessment of microcalcifications. The patient is asymptomatic.

Comparison: Mammogram performed on 08/01/2024.

Findings:

There are multiple microcalcifications seen within the left breast at approximately 8:00, 2 cm from the nipple. These microcalcifications are clustered, measuring 6 mm in greatest dimension and are located within a small area of architectural distortion. There is no associated mass identified. There is no surrounding ductal dilatation. No abnormal axillary lymph nodes are seen.

Impression:

Clustered microcalcifications within the left breast as described above. Ultrasound-guided core biopsy is recommended.

BI-RADS: 4 (Suspicious) ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense. No significant masses, architectural distortion or suspicious calcifications are identified in the right breast.

In the left breast, there is a new group of pleomorphic microcalcifications in the upper outer quadrant at approximately 2 o'clock, 6 cm from the nipple. These calcifications are scattered and variable in size and shape, ranging from punctate to linear and branching forms. This cluster measures approximately 8 mm in greatest dimension. No associated mass, distortion or skin changes are identified.

The remainder of the left breast is unremarkable. No lymphadenopathy is evident.

IMPRESSION:
New group of suspicious microcalcifications in the left breast at 2 o'clock, 6 cm from the nipple, BI-RADS 4B. Stereotactic biopsy is recommended for further evaluation.

BI-RADS 4B left breast. BI-RADS 1 right breast.

ADDENDUM: The patient returned for stereotactic biopsy of the left breast microcalcifications. Specimen radiography confirmed calcifications within the cores. Post-biopsy mammogram showed clip placement at the biopsy site with no residual calcifications. The patient tolerated the procedure well. Pathology results are pending. ➡ *Positive*
- FINDINGS:
Right mastectomy. 
Left breast tissue is composed of scattered fibroglandular densities. There are no dominant masses, architectural distortions, or suspicious microcalcifications. The breast tissue appears stable compared to prior examinations.

Targeted left breast ultrasound was performed at the site of palpable abnormality at 6 o'clock position, 3 cm from the nipple. No correlating sonographic mass or cystic lesion was identified. 

IMPRESSION:
1. Negative left screening mammogram. No mammographic evidence of malignancy. Recommend annual screening mammography. 
2. Negative targeted left breast ultrasound. No sonographic correlate for palpable finding. Recommend clinical follow up and correlation in 2-3 months. If persists, short term follow up diagnostic evaluation may be considered.

BI-RADS 1 - Negative. ➡ *Negative*
- FINDINGS: Bilateral mammogram. Right breast tissue contains scattered fibroglandular densities. No suspicious mass, architectural distortion or microcalcifications. Stable benign calcifications in the left breast. No new suspicious findings.

Targeted ultrasound right breast 10 o'clock position demonstrates a 7 mm simple cyst. No solid component. No other sonographic abnormality in this area.

IMPRESSION:
1. No mammographic evidence of malignancy. Specifically no suspicious microcalcifications. Recommend routine screening mammography in 1 year. 
2. 7 mm simple cyst right breast 10 o'clock position on targeted ultrasound. No suspicion for malignancy. 

BIRADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- SCREENING MAMMOGRAM
CLINICAL INDICATION: Routine screening mammogram.

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
There is scattered fibroglandular density. No focal asymmetry, architectural distortion, or suspicious mass. 

IMPRESSION:
BIRADS-2: Benign finding. Recommend routine screening in 1 year.

ADDENDUM:
No suspicious grouped or pleomorphic microcalcifications are identified in either breast. ➡ *Not Stated*
- FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are no dominant masses, architectural distortions or suspicious calcifications in either breast. Specifically, no abnormalities are noted in the left subareolar region corresponding to the area of palpable concern.

IMPRESSION:
No mammographic evidence of malignancy, BI-RADS 1. Recommend continued routine screening mammography in 1 year.

ADDENDUM:
Bilateral targeted ultrasound was performed.
Findings: At the site of palpable abnormality in the left breast 2 o'clock position 2 cm from the nipple there is a 0.5 x 0.3 x 0.4 cm oval circumscribed hypoechoic mass, parallel to the skin surface. This is consistent with a benign finding such as fibroadenoma.
Impression:
Benign left breast mass, BI-RADS 2. Short term follow up ultrasound can be considered to document stability. ➡ *Negative*
- MAMMOGRAPHIC EXAMINATION, BILATERAL
INDICATION: Annual screening mammogram.
COMPARISON: 11/28/2021.
FINDINGS:
Both breasts are almost entirely fatty. No significant masses, suspicious calcifications, or areas of architectural distortion are identified. Stable bilateral vascular calcifications noted.
IMPRESSION: 
BI-RADS category 1: Negative bilateral screening mammogram. 
Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- ULTRASOUND GUIDED LEFT BREAST BIOPSY
 
REASON FOR EXAMINATION: 67-year-old female with suspicious 8 mm cluster of microcalcifications at 2:00, 6 cm from the nipple on screening mammogram. 

TECHNIQUE: With the patient in the supine position, the skin overlying the area of interest was prepped and draped in the usual sterile fashion. 1% lidocaine was used for local anesthesia. Using ultrasound guidance, an 18-gauge spring-loaded biopsy device was used to obtain 5 core biopsy specimens from the area of microcalcifications in the 2:00 axis of the left breast, 6 cm from the nipple. The specimens were placed in formalin and sent to pathology for analysis. A titanium clip was deployed at the biopsy site. The skin was then cleansed and Steri-strips were applied for closure. The patient tolerated the procedure well. No immediate complications.

IMPRESSION: 
Successful ultrasound guided core biopsy of left breast microcalcifications at 2:00, 6 cm from the nipple. Pathology results pending. Recommend mammographic follow up of biopsy clip in 6 months.

BIRADS 4B: Suspicious abnormality, biopsy should be considered.

ADDENDUM - PATHOLOGY RESULTS: 
Left breast, 2:00, 6 cm from nipple, core biopsy:
- Fibrocystic change with focal atypical ductal hyperplasia and microcalcifications.
- Invasive mammary carcinoma is not identified.

Final BIRADS Category 4B based on imaging and histologic findings.
Discordant pathology, recommend surgical consultation and excisional biopsy for definitive diagnosis. ➡ *Positive*
- SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Screening mammogram from 1 year prior.

FINDINGS:
The breasts are predominantly fatty.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
No significant change compared to prior mammogram.

IMPRESSION:
Negative screening mammogram.
BIRADS 1 - Negative. 
Routine screening is recommended in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Annual screening.
FINDINGS:
The breasts are heterogeneously dense limiting the sensitivity of mammography.
A few small groups of punctate calcifications are seen medially in the right breast. These do not have suspicious features. These measure less than 1 mm.
Scattered benign appearing calcifications are seen in the left breast.
No suspicious masses, architectural distortion or suspicious calcifications are seen in either breast. No abnormally enlarged axillary lymph nodes.
Skin is within normal limits bilaterally. 
IMPRESSION:
BI-RADS 2: Benign findings.
A 6-month short term follow-up mammogram is recommended to assess stability of the punctate calcifications seen medially in the right breast.
ADDENDUM: Comparison was made with the patient's prior mammogram of 1 year ago. The punctate microcalcifications in the right breast are stable in appearance and distribution. Given stability over 1 year, these are assessed as benign. BI-RADS 2 remains. Short term follow-up is no longer required. Patient may return to annual screening. ➡ *Positive*
- SCREENING MAMMOGRAM.

INDICATION:  Screening.  

TECHNIQUE:  Standard CC and MLO views of both breasts were obtained.

COMPARISON:  None.

FINDINGS:  
The breasts are heterogeneously dense, which may obscure small masses.  

There are no suspicious masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
1. Negative mammogram.  
2. Heterogeneously dense breasts.

RECOMMENDATION:  
Routine screening mammography is recommended in 1 year.

BIRADS Category 1: Negative. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
Compared to Previous: No.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
No suspicious masses, architectural distortions or dominant masses.
There are scattered fibroglandular densities.
BIRADS-2. Routine screening recommended. ➡ *Not Stated*
- MAMMOGRAPHY - BILATERAL - SCREENING. CLINICAL HISTORY: Patient is a 54 year old female presenting for her screening mammogram. TECHNIQUE: Craniocaudal and mediolateral oblique views of the breasts were performed using digital mammography. COMPARISON: Comparison is made to the mammogram from 01/25/2024. FINDINGS: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- FINDINGS:
Bilateral breast ultrasound was performed.
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are no mammographic findings suspicious for malignancy. No suspicious mass, architectural distortion, skin thickening or calcifications are identified in either breast.
Bilateral axillary regions are unremarkable. 
Stable benign scattered fibroglandular densities noted bilaterally.
BIRADS Assessment Category: 2 - Benign findings. Recommend routine screening in 1 year.

IMPRESSION: 
No evidence of malignancy. 
Heterogeneously dense breasts.
Routine annual screening recommended. ➡ *Not Stated*
- CLINICAL HISTORY: Annual screening mammogram.

TECHNIQUE: Digital left diagnostic mammogram and ultrasound performed. Comparison with prior study from 8/15/2022.

FINDINGS:
LEFT MAMMOGRAM:
The left breast is heterogeneously dense, which may obscure detection of small masses. No suspicious masses, architectural distortion, or dominant suspicious calcifications are identified. Stable postsurgical changes noted in the upper outer quadrant from prior lumpectomy. Stable benign-appearing calcifications in the upper outer quadrant.

LEFT BREAST ULTRASOUND:
Targeted ultrasound was performed to the area of palpable concern at the 2 o'clock position, 4 cm from the nipple. No discrete cystic or solid masses are identified in this area. The remainder of the left breast was scanned and no suspicious masses or areas of abnormal echogenicity are identified.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. 
2. Stable postsurgical changes in the left breast, likely related to prior lumpectomy as per clinical history.
3. Stable benign-appearing calcifications in the left breast.

ASSESSMENT: BI-RADS 2 - Benign finding. Recommend continued annual screening in 12 months. ➡ *Negative*
- DIAGNOSTIC MAMMOGRAPHY LEFT BREAST

CLINICAL HISTORY: 57-year-old female with new palpable lump in the upper outer quadrant of the left breast. 

TECHNIQUE: Diagnostic mammographic views of the left breast were obtained including CC and MLO views with spot compression and magnification views of the area of palpable concern.

FINDINGS:
The left breast demonstrates scattered fibroglandular densities.

There is an irregular shaped mass measuring approximately 2.5 x 1.5 cm in the upper outer quadrant of the left breast at the 2:00 position, 8 cm from the nipple, corresponding to the area of palpable concern. The margins of this mass are indistinct and spiculated. 

Additionally, there are several groups of pleomorphic microcalcifications extending anteriorly from the mass, distributed in a segmental pattern. The largest cluster measures 12 mm. 

The skin appears normal. The nipple is normal in appearance. No suspicious axillary lymphadenopathy.

IMPRESSION:
1. Highly suspicious mass with associated pleomorphic microcalcifications upper outer left breast, BI-RADS 5. Recommend tissue diagnosis. ➡ *Positive*
- FINDINGS: Both breasts are heterogeneously dense which could obscure detection of small masses. No dominant mass, suspicious calcifications or architectural distortion. There are bilateral benign-appearing calcifications, likely vascular in nature. Stable post-surgical changes from prior right lumpectomy are noted.

Targeted ultrasound was performed at the site of the palpable abnormality in the left breast at 6 o'clock, 3 cm from the nipple. No correlate mass, cyst or other sonographic finding was identified.

IMPRESSION:
1. Heterogeneously dense breasts with no dominant mass, suspicious calcifications or architectural distortion. Stable benign calcifications and post-surgical changes.
2. No sonographic correlate for the palpable finding in the left breast at 6 o'clock. 

RECOMMENDATIONS: Recommend clinical follow up of the palpable finding as indicated. Recommend routine annual screening mammography in 1 year.

BI-RADS Category 1: Negative. ➡ *Negative*
- SCREENING BILATERAL MAMMOGRAM WITH TOMOSYNTHESIS
CLINICAL HISTORY: Annual screening mammogram
COMPARISON: None
TECHNIQUE: Hologic tomosynthesis views of both breasts were obtained. Additional spot compression views were obtained of the left breast to further evaluate a focal asymmetry.
FINDINGS:
Both breasts are heterogeneously dense, which could obscure small masses.
RIGHT BREAST: No suspicious mass, architectural distortion or suspicious calcifications.
LEFT BREAST: There is a 1 cm irregular focal asymmetry at the 2:00 position, 5 cm from the nipple on the CC view. This area was evaluated with spot compression views, which demonstrated an 8 mm cluster of pleomorphic calcifications. No associated mass or architectural distortion.
IMPRESSION:
RIGHT BREAST: No mammographic evidence of malignancy. BI-RADS category 1.
LEFT BREAST: 8 mm cluster of suspicious pleomorphic calcifications at the 2:00 position, 5 cm from the nipple. BI-RADS category 4B. Recommend stereotactic core biopsy.
ADDENDUM: Stereotactic core biopsy of the suspicious calcifications in the left breast was performed. Pathology results demonstrated ductal carcinoma in situ, high nuclear grade with comedonecrosis. BI-RADS category 6. Surgical consultation is recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM. CLINICAL HISTORY: Patient is a 46 year old female presenting for a screening mammogram. Technique: Bilateral CC and MLO images were obtained. Comparison is made to prior mammographic examination from 10/04/2023. Findings: The breasts are heterogeneously dense. There are no masses, architectural distortion, or suspicious calcifications. Impression: Negative mammogram. BI-RADS 1. ➡ *Negative*
- FINDINGS: Left breast mammogram shows a fatty replaced breast with scattered areas of benign-appearing fibroglandular density. No dominant mass, suspicious calcification, or architectural distortion. Stable appearance compared to prior mammograms. 

Right breast ultrasound demonstrates normal appearing breast tissue with no sonographic abnormality in the area of palpable concern at the 9 o'clock position. 

IMPRESSION:
1. Negative bilateral screening mammogram. No mammographic evidence of malignancy. Continue annual screening mammography.

2. Negative targeted right breast ultrasound of the 9 o'clock palpable abnormality. No sonographic correlate. Clinical correlation recommended.

BI-RADS Category 1: Negative. ➡ *Negative*
- FINDINGS:
Both breasts are heterogeneously dense. There are scattered areas of fibroglandular density. No dominant mass, architectural distortion, or definite asymmetry in either breast.

At the 10 o'clock position of the RIGHT breast, there is a group of amorphous calcifications spanning an area of about 1 cm. The calcifications are new compared to prior mammograms from 2 years ago. Additional magnification views confirm these are suspicious calcifications.

The LEFT breast is unremarkable. No suspicious calcifications, masses, architectural distortion or asymmetries in the left breast.

IMPRESSION:
BI-RADS Category 4: Suspicious grouped amorphous calcifications in the right breast at 10 o'clock position. Biopsy is recommended for definitive characterization. Malignancy cannot be excluded.

BI-RADS Category 1: Normal left breast. Annual screening mammography is recommended.

ADDENDUM: 
Stereotactic biopsy of the grouped calcifications in the right breast at 10 o'clock position was performed. Pathology results demonstrate ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform type. Wide local excision is recommended for complete removal and definitive treatment. ➡ *Positive*
- MAMMOGRAM AND ULTRASOUND REPORT.

HISTORY: A 40 year old female presents for her annual screening mammogram and ultrasound. No current breast concerns. Aunt with history of breast cancer. 

FINDINGS:
Right breast: Breast composed of dense fibroglandular tissue. No architectural distortion, suspicious calcifications, or concerning masses. Stable.
Left breast: Breast composed of dense fibroglandular tissue. No architectural distortion, suspicious calcifications, or concerning masses. Stable. 

Targeted ultrasound: 
Right breast 2:00 lesion identified on mammogram, corresponding to a 7mm simple cyst at 2:00, 4 cm from the nipple. 
Left breast, no sonographic abnormalities.

IMPRESSION:
1. Bilateral dense breast tissue, no suspicious findings. Annual screening recommended.
2. Right breast simple cyst corresponding to mammographic findings. Benign. 
3. Left breast, no sonographic abnormalities.

BI-RADS Category 1: Negative. ➡ *Negative*
- SCREENING BILATERAL MAMMOGRAM.
History: Last mammogram 1 year ago was normal.
Examination: Hologic tomosynthesis images were obtained and reviewed in the PACS.
Comparison is made to screening exam from April 2021.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
In the upper outer left breast on the MLO view at 11:00, 8 cm from the nipple, there is a small irregular mass associated with fine pleomorphic microcalcifications extending over approximately 1 cm in an area which corresponds to the palpable abnormality. This was not present on the prior exam.
No suspicious mass, architectural distortion, or suspicious calcifications in the right breast.
No significant interval change from prior exam.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended.
BI-RADS Category 2: Benign finding in the right breast.
RECOMMENDATION:
Stereotactic core biopsy of suspicious microcalcifications and mass, left breast at 11:00, as marked.
ADDENDUM:
The results of this examination were discussed with the referring clinician, Dr. Smith, who agrees with the recommendation for stereotactic core biopsy of the left breast finding. The patient has been informed and scheduled for the procedure. ➡ *Positive*
- REPORT:
"RIGHT MAMMOGRAM AND ULTRASOUND.
HISTORY: Patient reports no palpable lumps or abnormalities. Last mammogram June 2021 - reported as normal. No personal or family history of breast cancer.
FINDINGS:
Right breast tissue is heterogeneously dense, which may obscure detection of small masses. There is no focal mass, architectural distortion, or suspicious calcifications. There is no significant change compared to prior mammogram.
Right breast targeted ultrasound was performed at the 10 o'clock position. No focal solid lesions or cystic abnormalities are identified.
IMPRESSION:
1. No mammographic evidence of malignancy in the right breast. Recommend follow up with routine screening mammography in one year.
2. No sonographic abnormality identified at the site of clinical concern, 10 o'clock position of the right breast. Recommend clinical follow up as needed.
BIRADS 1." ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM.
CLINICAL HISTORY: 44 year old female, family history of breast cancer in sister at age 40.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense (Category C). No dominant mass, skin thickening or new architectural distortion. 
In the right breast at 10:00, 8 cm from the nipple, there is a group of  new amorphous microcalcifications spanning 15 mm. No other suspicious calcifications right breast.
In the left breast at 9:00 4 cm from the nipple there are few punctate calcifications unchanged from the previous exam. No dominant mass or suspicious calcifications left breast. 
IMPRESSION:
BI-RADS 4: Suspicious grouped amorphous calcifications right breast at 10:00. Recommend stereotactic core biopsy.
BI-RADS 2: Benign calcifications left breast. 
ADDENDUM (Post Stereotactic Right Breast Biopsy): 
Stereotactic vacuum assisted core biopsy of the 10:00 right breast calcifications performed. Specimen radiograph demonstrates calcifications. Pathology pending. ➡ *Positive*
- FINDINGS: The right breast is heterogeneously dense, and the left breast predominantly fatty. There is scattered fibroglandular density within both breasts, greater on the right. No suspicious masses, focal asymmetries or architectural distortion identified. The skin appears unremarkable. No suspicious microcalcifications evident.

IMPRESSION:
1. Heterogeneously dense breast tissue, greater on the right. No suspicious findings. 
2. Recommend annual screening mammography.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

CLINICAL HISTORY: 51 year-old female presenting for high risk screening. Mother had breast cancer at age 45.

COMPARISON: None.

FINDINGS:
Both breasts are heterogeneously dense (C), which may obscure detection of small masses.

Right breast:
There are no dominant masses, architectural distortions or suspicious calcifications.

Left breast: 
There are segmental fine linear calcifications in a ductal distribution at the 3 o'clock position of the left breast middle depth, spanning an area of about 2 cm. There are no associated masses, architectural distortions or skin changes.

IMPRESSION:
BIRADS 4B: Suspicious calcifications in the left breast requiring stereotactic biopsy for further evaluation. 

BIRADS 1: Negative right breast.

RECOMMENDATION:  
Stereotactic core biopsy of the segmental calcifications in the left breast at 3 o'clock position is recommended for further evaluation. The patient should return to the Breast Center for the biopsy with the Mammography Requisition Form. ➡ *Positive*
- BREAST ULTRASOUND

CLINICAL HISTORY: Recent screening mammogram demonstrated microcalcifications. The patient is asymptomatic.

COMPARISON: Mammogram performed on 07/27/2024.

FINDINGS: There is a small cluster of microcalcifications identified within the left breast at approximately 2:00, 3 cm from the nipple. These microcalcifications are clustered, measuring 4 mm in greatest dimension and are located within a small area of architectural distortion. No associated mass is identified. There is no surrounding ductal dilatation. No abnormal axillary lymph nodes are seen.

IMPRESSION:

Clustered microcalcifications within the left breast as described above. Ultrasound-guided core biopsy is recommended.

BI-RADS: 4 (Suspicious) ➡ *Positive*
- SCREENING BILATERAL DIGITAL MAMMOGRAM.  
INDICATION:  Screening.  

TECHNIQUE:  Standard CC and MLO views of both breasts were obtained.  

COMPARISON:  Compared to prior exam dated 1 year prior.  

FINDINGS:  
The breasts are heterogeneously dense, which may obscure detection of small masses.  

There are no dominant masses, areas of architectural distortion or suspicious calcifications.  

IMPRESSION:  
NEGATIVE BILATERAL SCREENING MAMMOGRAM.  
BIRADS Category 1 - Negative.  Recommend routine screening in 1 year.  

Breast tissue density is heterogeneously dense. ➡ *Not Stated*
- MAMMOGRAPHY LEFT.

CLINICAL INDICATION: 53-year-old female with left breast pain. No palpable lump. 

COMPARISON: 1 year prior.

FINDINGS: 
Left breast shows scattered fibroglandular densities. No significant masses, suspicious calcifications or focal asymmetries. No significant change from prior exam.

IMPRESSION:
BIRADS 1 - Negative study. No mammographic evidence of malignancy. Recommend annual screening mammography. ➡ *Negative*
- LEFT DIAGNOSTIC MAMMOGRAM:
INDICATION: Further evaluation of palpable abnormality in the upper outer quadrant, left breast.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
COMPARISON: With prior study of 12/17/2019.
FINDINGS:
The breasts are predominantly fatty. There are no mammographic findings to indicate malignancy. No dominant mass, area of architectural distortion or suspicious calcifications.  
There is stable post-surgical scarring in the upper outer left breast.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Stable post-surgical changes left breast.
3. Breasts are predominantly fatty, limiting sensitivity of mammography. Recommend ultrasound for further evaluation of palpable finding.
BIRADS 1 - Negative mammogram. Recommend annual screening in 1 year. ➡ *Negative*
- BREAST ULTRASOUND

CLINICAL HISTORY: Palpable lump, left breast.

COMPARISON: None.

FINDINGS: Within the left breast at the 10 o'clock position, there is a palpable 1.5 cm irregular, hypoechoic mass. This mass is located within the area of the patient's palpable abnormality. The mass demonstrates angular margins, posterior acoustic shadowing, and increased vascularity on color Doppler imaging. In addition, a small cluster of microcalcifications is identified adjacent to the mass. These calcifications correlate with findings seen on recent mammography.

IMPRESSION:

1. Left breast mass: Imaging features are suggestive of malignancy. The adjacent presence of microcalcifications raises further concern for malignancy. Biopsy is recommended.

BI-RADS: 5 (Highly Suggestive of Malignancy). ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammogram was obtained.
The breast tissue is heterogeneously dense which may obscure detection of small masses.
There are scattered areas of fibroglandular density.
At the 10:00 position of the right breast 6 cm from the nipple there is a group of amorphous microcalcifications in a segmental distribution spanning approximately 12 mm. Additional magnification views in the craniocaudal and mediolateral oblique projections confirms these findings.
No dominant mass, architectural distortion or definite abnormal asymmetry identified in either breast. The skin is normal. There is no evidence of axillary adenopathy.

OTHER RELEVANT DIAGNOSTIC TESTS:
Ultrasound of the right breast at 10:00 6 cm from the nipple demonstrates an ill-defined, heterogeneous, non-mass like area measuring approximately 12 mm and corresponding to the area of microcalcifications seen mammographically. No other abnormalities are identified.

IMPRESSION:
Suspicious segmental distribution of microcalcifications at the 10:00 position right breast 6 cm from the nipple corresponding to an ill-defined area on ultrasound. Stereotactic core biopsy recommended. BIRADS 4B Suspicious - Moderate suspicion for malignancy.
Left breast: No evidence of malignancy, BIRADS 1, Negative.

ADDENDUM: Stereotactic core biopsy of the area of microcalcifications at the 10:00 position right breast 6 cm from the nipple was performed. Specimen radiograph demonstrates microcalcifications within the cores. Pathology pending. ➡ *Positive*
- MAMMOGRAM REPORT - DIAGNOSTIC (BILATERAL)

CLINICAL HISTORY: 45-year-old female presenting for evaluation of a palpable lump in the right breast at 10 o'clock, found on self-examination. No prior mammograms available for comparison. Family history of breast cancer in a maternal aunt.

FINDINGS:
Both breasts are heterogeneously dense (BI-RADS density category C). This may lower the sensitivity of mammography.

RIGHT BREAST:
There is a 1.2 cm oval, circumscribed mass at the 10 o'clock position, corresponding to the palpable abnormality. No suspicious calcifications, architectural distortion, or skin thickening.

LEFT BREAST:
No dominant masses, suspicious calcifications, or architectural distortion.

IMPRESSION:
1. 1.2 cm circumscribed mass in the right breast at 10 o'clock, corresponding to the palpable abnormality. This is most likely benign; however, ultrasound is recommended for further evaluation. BI-RADS 0, incomplete.

2. No suspicious findings in the left breast. BI-RADS 1, negative.

RECOMMENDATION:
Right breast ultrasound for further characterization of the 10 o'clock mass. Routine screening mammography in 1 year, unless otherwise clinically indicated.

ADDENDUM - RIGHT BREAST ULTRASOUND:
The 1.2 cm mass at 10 o'clock in the right breast demonstrates benign features: oval shape, circumscribed margins, and parallel orientation. No suspicious vascularity or posterior features. This most likely represents a benign fibroadenoma. BI-RADS 2, benign.

OVERALL IMPRESSION:
Benign right breast mass, most likely a fibroadenoma. No evidence of malignancy. Annual screening mammography is recommended. BI-RADS 2. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

FINDINGS:

Both breasts are almost entirely fatty.

There are scattered benign-appearing calcifications in both breasts. In the upper outer quadrant of the left breast there is a small group of indeterminate microcalcifications. Spot compression magnification views confirm a cluster of pleomorphic microcalcifications spanning about 6 mm at the 2:00 position, approximately 6 cm from the nipple. No other suspicious calcifications, masses or architectural distortion are identified.

IMPRESSION:
BIRADS 0: Small group of indeterminate microcalcifications in the left breast. Additional imaging evaluation with magnification views is recommended for further assessment. The patient should return for additional mammographic workup as well as left breast ultrasound to evaluate the area of concern. The results will be communicated to the referring physician.

BIRADS 2: Benign findings right breast.

ADDENDUM:
Magnification views of the left breast demonstrate pleomorphic microcalcifications spanning 7 mm at the 2:00 position of the left breast, 6 cm from the nipple, suspicious for DCIS. Ultrasound of this area shows no correlate mass or dilated ducts. Stereotactic biopsy is recommended for definitive diagnosis.

BIRADS 4: Suspicious microcalcifications left breast. Stereotactic biopsy recommended. ➡ *Positive*
- TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
No mammographic findings that are concerning for malignancy. The breasts are almost entirely fatty.
BIRADS-2.
ADDENDUM: 
No suspicious microcalcifications, architectural distortions or spiculated masses are identified in either breast. Benign-appearing lymph nodes are noted in the left axilla. ➡ *Not Stated*
- SCREENING BILATERAL MAMMOGRAM. 
Compared to previous screening mammogram dated June 15, 2021, there are no concerning changes.
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
IMPRESSION:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING BILATERAL MAMMOGRAM:

INDICATION: Annual screening mammogram.

TECHNIQUE: Full field digital mammographic (FFDM) views obtained bilaterally, including CC and MLO views per breast.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses. No significant masses, architectural distortion, or suspicious calcifications are identified.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breast tissue. 

BI-RADS Assessment Category: 1 - Negative.

Management: Routine screening in 12 months. ➡ *Not Stated*
- BILATERAL MAMMOGRAM
HISTORY: 55 year-old female with risk factor of positive family history presenting for screening mammogram.
FINDINGS:
Left breast: Heterogeneously dense parenchyma. At 2 o'clock position 6 cm from the nipple there is a new group of pleomorphic microcalcifications extending over 1 cm suspicious for DCIS. At 3 o'clock position there is a stable 7 mm circumscribed mass unchanged from prior exams likely benign. The remainder of the left breast is unremarkable.  BIRADS 4.
Right breast: Heterogeneously dense parenchyma. Scattered areas of fibroglandular density. No dominant masses, areas of architectural distortion or suspicious calcifications. Stable 10 mm circumscribed mass at 9 o'clock position 4 cm from the nipple likely benign. BIRADS 2.  
IMPRESSION:
New group of pleomorphic microcalcifications at 2 o'clock position of the left breast highly suspicious for DCIS. Stereotactic biopsy is recommended. The remainder of the bilateral breasts are unremarkable or show likely benign stable findings.
ADDENDUM: Stereotactic biopsy of the group of microcalcifications in the left breast at 2 o'clock position 6 cm from the nipple was performed. Specimen radiograph showed microcalcifications. Pathology result: Ductal carcinoma in situ, high nuclear grade with comedonecrosis. ➡ *Positive*
- FINDINGS: 
Right breast: Heterogeneously dense breast tissue with scattered areas of fibroglandular density. No suspicious masses, architectural distortion, or suspicious calcifications.
Left breast: Heterogeneously dense breast tissue with scattered areas of fibroglandular density. No suspicious masses, architectural distortion, or suspicious calcifications. 
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Recommend routine screening mammography in 1 year.
BIRADS 1 - Negative.

ADDENDUM: 
Comparison was made to prior mammograms from 1/15/2022 at ABC Hospital. No significant interval change. ➡ *Negative*
- MAMMOGRAM - BILATERAL - DIAGNOSTIC. History: Right breast pain. Patient reports pain in the right breast for 3 weeks. She denies any palpable masses. Past History: History of left breast cancer in 2016. Patient underwent lumpectomy and radiation therapy. Medications: Tamoxifen daily. Family History: Mother - breast cancer at age 50. Sister - ovarian cancer at age 45. Technique: CC and MLO views of bilateral breasts. Comparison: To previous mammogram dated 08/15/2023. Findings: The breasts are heterogeneously dense, which may obscure some small abnormalities. There are scattered benign-appearing calcifications bilaterally. There are no suspicious masses, architectural distortions, or suspicious calcifications. The surgical scar in the left breast is unchanged. Impression: No mammographic evidence of malignancy. BI-RADS Category 2: Benign finding. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None available.

FINDINGS:
Both breasts show heterogeneously dense parenchymal pattern, limiting sensitivity of mammography.

Right breast:
There are no masses, architectural distortion, or suspicious calcifications.

Left breast: 
There are scattered fibroglandular densities. In the upper outer quadrant, there is a group of fine pleomorphic microcalcifications spanning an area of approximately 1 cm. No associated mass, architectural distortion, or skin thickening is identified.

IMPRESSION:
1. Heterogeneously dense breasts, limiting sensitivity of mammography. 
2. Suspicious group of microcalcifications in the upper outer quadrant of the left breast, spanning approximately 1 cm. Additional evaluation with spot compression magnification views is recommended for further characterization. BI-RADS 4B.
3. Benign appearing right breast. BI-RADS 1. 

RECOMMENDATIONS:  
Recommend left diagnostic mammogram with spot compression magnification views of the microcalcifications in the upper outer left breast. Ultrasound is also recommended to evaluate for any associated mass.

ADDENDUM (post spot compression magnification views):
The group of microcalcifications in the upper outer left breast persists and appears more conspicuous on spot compression magnification views. The calcifications are fine, linear, and branching in morphology, distributed in a segmental pattern, highly suspicious for ductal carcinoma in situ (DCIS). Stereotactic core biopsy is recommended for histological confirmation. BI-RADS 4C. ➡ *Positive*
- MAMMOGRAM AND ULTRASOUND OF LEFT BREAST.

INDICATION: Palpable lump left breast at 3 o'clock position, found on self-exam. No prior mammograms available for comparison.

FINDINGS:
Left breast is heterogeneously dense (BI-RADS density category C). There are no dominant masses, architectural distortion, or suspicious calcifications.

Targeted ultrasound of the palpable area at 3 o'clock, 4 cm from the nipple, demonstrates normal fibroglandular tissue with no discrete solid or cystic lesion.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. Recommend annual screening mammography.
2. Normal targeted ultrasound of the palpable area at 3 o'clock left breast. Likely represents normal fibroglandular tissue. Clinical follow up recommended as indicated.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening
Comparison available dating from 02/2022 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL MAMMOGRAM AND RIGHT BREAST ULTRASOUND
CLINICAL INDICATION: Annual screening mammogram.
COMPARISON: 10/12/2022.
FINDINGS:
Right breast: Heterogeneously dense parenchyma. There are no dominant masses, architectural distortion, or suspicious calcifications. Stable postsurgical changes from prior lumpectomy are noted.
Left breast: Heterogeneously dense parenchyma. There are no dominant masses, architectural distortion, or suspicious calcifications. 
Right breast ultrasound was performed, targeted to the area of palpable concern at the 10 o'clock position, 2 cm from the nipple. There is no focal solid or suspicious lesion seen in this area. Ultrasound is limited due to dense breast tissue.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. No sonographic correlate for the palpable finding in the right breast at 10 o'clock position. Clinical followup is recommended.
3. Dense breast tissue limits mammographic sensitivity. Consider supplemental screening with breast MRI if clinically indicated based on risk factors.
BI-RADS Category 1: Negative. Recommend annual screening mammogram. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Routine screening
TECHNIQUE: Bilateral screening mammogram.
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.
There are no suspicious masses, calcifications, or other findings.
IMPRESSION:
BIRADS-2: Benign findings. Recommend continued routine screening mammography.
Compared to Previous: No significant change from prior.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes. ➡ *Not Stated*
- LEFT SCREENING MAMMOGRAM
Clinical history: 55 year old female, screening mammogram
Comparison: None
Technique: Standard CC and MLO views of the breasts were obtained.
Findings:
Heterogeneously dense breast tissue bilaterally.
No suspicious masses, calcifications, or architectural distortions are seen.
IMPRESSION:
Negative screening mammogram. BI-RADS 1. ➡ *Negative*
- BILATERAL MAMMOGRAM - SCREENING. ➡ *Not Stated*
- MAMMOGRAPHY - BILATERAL. HISTORY: Screening mammogram. No history of breast cancer. COMPARISON: Comparison is made to previous mammogram from 10/25/2023. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM
Clinical History: 56 y.o. female presents for a problem focused exam for evaluation of left breast pain.
Comparison: Prior mammograms from (outside institution), not available for review.
Technique: Standard CC & MLO views were obtained of each breast. In addition, spot compression and magnification views were performed to better evaluate the palpable findings in the right breast.
Findings:
The breasts are heterogeneously dense, which may reduce the sensitivity of mammography.
Right breast:
Palpable findings in the right breast corresponding to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
Left breast:
Scattered fibroglandular densities. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Right breast, palpable findings, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended.
Left breast, negative. BI-RADS 1. ➡ *Negative*
- BILATERAL MAMMOGRAM
HISTORY: Annual screening mammogram. No symptoms or complaints.
Comparison made to prior imaging.
FINDINGS:
Both breasts are predominantly fatty.
There are no masses, architectural distortions, or suspicious calcifications in either breast.
There is no significant interval change since the prior exam.
IMPRESSION:
Negative bilateral screening mammogram.
BIRADS 1. Annual screening recommended. ➡ *Negative*
- MEDICAL HISTORY: 42F, annual screening mammogram. Patient notes fullness of right lateral breast. Previous right breast biopsy at 10:00 showed benign pathology. No other significant history.

PROCEDURE: Routine bilateral digital screening mammography was performed including CC and MLO views.

FINDINGS:
Right breast: The breast is heterogeneously dense (C). No discrete mass, suspicious calcifications, or architectural distortion. Stable postoperative scar marker at 10:00 position from previous biopsy. Stable benign calcifications in the upper outer quadrant. No significant change from prior exam.

Left breast: The breast is heterogeneously dense (C). No dominant mass, suspicious calcifications, or architectural distortion. Stable scattered fibroglandular densities. No significant change compared to prior mammograms.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Heterogeneously dense breasts, which can lower the sensitivity of mammography. Clinical breast exam and self breast awareness are encouraged.
3. Stable right breast biopsy site changes at 10:00.
4. Stable benign calcifications in the right upper outer quadrant.

ASSESSMENT: BI-RADS 2 - Benign findings, recommend routine screening in 1 year. ➡ *Negative*
- FINDINGS:
Bilateral digital diagnostic mammography was performed.

The breasts are heterogeneously dense which could obscure detection of small masses.

In the left breast at 12:00, 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of 10 mm. Additional magnification views confirm the presence of microcalcifications. No associated mass, architectural distortion or skin thickening.

At 6:00, 2 cm from the nipple in the left breast, there is a circumscribed, oval mass measuring 8 mm. Ultrasound shows this to be a simple cyst.

The remainder of the left breast is unremarkable. 

In the right breast, there are no dominant masses, areas of architectural distortion or suspicious microcalcifications.

IMPRESSION:
1. Group of amorphous microcalcifications at 12:00, left breast, spanning 10 mm. Additional imaging evaluation with magnification views confirms the presence of microcalcifications. Suspicious for DCIS. Stereotactic core biopsy recommended. BIRADS 4B.

2. 8 mm circumscribed mass at 6:00, left breast. Confirmed as simple cyst on ultrasound. Benign. BIRADS 2.

3. Heterogeneously dense breast tissue limits sensitivity of mammography. 

4. Remainder of both breasts are unremarkable. Overall BIRADS 4B. ➡ *Positive*
- MAMMOGRAM
INDICATION: Screening mammogram. No current breast complaints.
COMPARISON: 08/05/2023
FINDINGS:
Bilateral breast tissue contains scattered fibroglandular densities. There is no evidence of focal mass, architectural distortion or suspicious calcifications. Benign vascular calcifications noted bilaterally. Stable post-surgical changes from prior bilateral reduction mammoplasty. No significant interval change compared to prior exam.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Routine annual screening is recommended. 
BI-RADS 1: Negative. ➡ *Negative*
- MAMMOGRAPHY - SCREENING. History: 57-year-old female for routine screening mammogram. Patient reports that she is currently not on hormone replacement therapy. Patient reports a family history of breast cancer in her mother, diagnosed at the age of 72. Comparison is made with the patient's previous mammogram performed on 04/09/2023. Findings: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: No evidence of malignancy. BI-RADS 1. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammography.
COMPARISON: None available.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
FINDINGS:
The breasts are heterogeneously dense (BI-RADS Density C).
Right breast:
No mammographic evidence of malignancy. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
Left breast:
No dominant masses or areas of architectural distortion. There is a small group of indeterminate microcalcifications in the upper outer quadrant.
IMPRESSION:
Right breast: Negative mammogram. BI-RADS Category 1.
Left breast: Small group of indeterminate microcalcifications in the upper outer quadrant. Recommend diagnostic mammographic work-up with magnification views for further evaluation. BI-RADS Category 0.
ADDENDUM - LEFT DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS:
Left magnification views confirm a small group of pleomorphic microcalcifications in a segmental distribution at the 1 o'clock position of the left breast, middle depth. There are approximately 10 calcifications in an area measuring 8 mm.
Ultrasound of this area demonstrates no correlating mass.
IMPRESSION ADDENDUM:
Small group of pleomorphic microcalcifications in a segmental distribution at the 1 o'clock position of the left breast. Suspicious for DCIS. Stereotactic core biopsy is recommended. BI-RADS Category 4B. ➡ *Positive*
- BILATERAL MAMMOGRAM REPORT WITH ADDENDUM

CLINICAL INDICATION: Annual screening mammogram.

COMPARISON: None available. 

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.

There are no suspicious masses, architectural distortion, or skin thickening in either breast. 

In the left breast at the 9:00 position, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of about 1 cm. No other suspicious microcalcifications are identified in either breast. 

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation:
New group of amorphous microcalcifications in the left breast at 9:00, 6 cm from the nipple. Additional magnification views are recommended for further characterization. 

ADDENDUM:
Magnification views of the left breast were performed, confirming a group of amorphous microcalcifications at the 9:00 position, 6 cm from the nipple, spanning approximately 1 cm. The distribution and morphology are suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended for definitive diagnosis.

FINAL ASSESSMENT FOLLOWING MAGNIFICATION VIEWS:
BI-RADS Category 4: Suspicious - Biopsy should be considered. ➡ *Positive*
- SCREENING MAMMOGRAM:
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available.
FINDINGS:
Both breasts are almost entirely fatty.
There is a cluster of coarse heterogeneous and amorphous calcifications in the upper outer quadrant of the right breast, spanning an area of approximately 2 cm. No associated mass, architectural distortion, or skin thickening.
No suspicious masses, calcifications, or other findings in either breast.
Stable benign calcifications are noted bilaterally.
There are no appreciable axillary, supraclavicular or internal mammary lymph nodes.
IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation and/or prior mammograms for comparison.
A cluster of suspicious calcifications in the right breast requires further workup with magnification views. Depending on the appearance on magnification views, stereotactic biopsy may be indicated.
ADDENDUM:
Right breast magnification views in the CC and ML projections demonstrate a cluster of coarse heterogeneous calcifications with amorphous forms, spanning 12 mm at the site of the calcifications seen on the screening views. No associated mass or distortion.
Final Assessment:
Right Breast: BI-RADS Category 4: Suspicious abnormality - Biopsy should be considered.
Left Breast: BI-RADS Category 1: Negative. ➡ *Positive*
- MAMMOGRAPHIC FINDINGS:
Bilateral digital screening mammogram, comparison made to prior exam dated 11/15/2022.
RIGHT BREAST:
There is scattered fibroglandular tissue. No dominant mass or architectural distortion. No suspicious calcifications.
LEFT BREAST:
There is heterogeneously dense parenchyma. No dominant mass or architectural distortion. No suspicious calcifications.
IMPRESSION:
1. No mammographic evidence of malignancy, both breasts. Recommend continued routine screening mammography in 1 year.
2. Mammographic breast density: Heterogeneously dense, which may obscure detection of small masses. For moderate to very dense breast tissue, additional screening with breast ultrasound or breast MRI may be considered, as determined by individual risk factors and in consultation with the referring provider.
BIRADS 1 - NEGATIVE.
ADDENDUM:
Comparison is made to prior study dated 5/12/2022 at outside facility, DiagnosticImaging Partners. The breasts appear stable with no interval change or development of suspicious findings. Heterogeneously dense breast tissue is again noted, unchanged. No microcalcifications are identified in either breast. ➡ *Negative*
- MAMMOGRAPHY REPORT - SCREENING BILATERAL

CLINICAL HISTORY: Screening mammogram. Personal history of LCIS. Maternal aunt with breast cancer at age 53.

TECHNIQUE: Standard CC and MLO views of the breasts were obtained. 

COMPARISON: Comparison is made to prior study of 03/15/2022.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
No suspicious masses, architectural distortion, or suspicious calcifications are identified in either breast. 
Stable postoperative changes and surgical clips noted in the upper outer quadrant of the left breast, consistent with history of prior biopsy for LCIS. 
No significant interval change compared to prior study.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breasts, which may lower the sensitivity of mammography. 
3. Stable postoperative changes left breast.

BIRADS Category 2 - Benign findings. Recommend continued annual screening in 12 months with consideration of supplemental screening ultrasound. ➡ *Negative*
- TMIST STUDY FFDM ONLY ARM ANNUAL SCREENING MAMMOGRAM.
Reason: Tomosynthesis Mammographic Imaging Screening Trial (TMIST) participant, informed consent obtained. Randomized to FFDM only arm.  
Comparison to prior exam: Yes. 
BIRADS breast composition: b. The breasts are heterogeneously dense, which may obscure small masses.
Negative study. No mammographic evidence of malignancy.
Specifically, there are no masses, architectural distortions, or suspicious calcifications. 
BIRADS Category 1: Negative. Routine screening mammogram recommended in 1 year per TMIST study protocol. ➡ *Not Stated*
- FINDINGS: Bilateral mammogram and ultrasound performed.

Right breast: Predominantly fatty breast tissue. No mass, architectural distortion, or suspicious calcifications.

Left breast: Scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications. 

Targeted ultrasound of the left breast at 10 o'clock position, correlating with area of clinical concern, demonstrates normal fibroglandular tissue. No sonographic correlate for palpable finding.

IMPRESSION:
1. Negative bilateral screening mammogram. 
2. No sonographic abnormality in the left breast at site of palpable abnormality. 

ASSESSMENT: BI-RADS 1 - Negative. Recommend annual screening mammography in 1 year.

ADDENDUM: Comparison is made to outside mammograms from June 2024, which are similar. No prior ultrasound for comparison. ➡ *Negative*
- FINDINGS:
Bilateral digital screening mammogram consisting of CC and MLO views of each breast was performed. The breast tissue is predominantly fatty.
There are no focal masses, asymmetries, or areas of architectural distortion. No dominant mass lesions are identified in either breast.
No suspicious clustered microcalcifications are seen.
Stable benign-appearing calcifications are scattered bilaterally.
No significant skin thickening or retraction.
No mammographic evidence of malignancy.

IMPRESSION:
BIRADS Category 2: Benign findings. Routine screening recommended in 1 year.

ADDENDUM:
Informed consent for this report was obtained according to our institution's guidelines. This study is a part of the approved research trial comparing digital breast tomosynthesis to standard mammography. The patient was randomized to the 2D full-field digital mammography arm. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Compared to prior exam dated 10/15/2020.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

Stable bilateral scattered fibroglandular densities.

IMPRESSION:
BIRADS 2 - Benign finding. Recommend routine screening in 1 year. ➡ *Not Stated*
- SCREENING BILATERAL MAMMOGRAMS:

CLINICAL HISTORY:  High risk patient with a family history of breast cancer in aunt diagnosed at age 42.

FINDINGS:
Heterogeneously dense breast tissue which may lower the sensitivity of mammography.
There are scattered areas of benign-appearing fibroglandular density.

In the right breast at 11:00 4 cm from the nipple there is a 6 mm cluster of amorphous calcifications. This requires further evaluation with magnification views.

Otherwise no dominant masses, areas of architectural distortion or suspicious calcifications are seen in either breast.

IMPRESSION:
Right breast: 6 mm cluster of amorphous calcifications at 11:00 for which magnification views are recommended. BIRADS 0, Incomplete.

Left breast: No mammographic evidence of malignancy. Recommend routine screening. BIRADS 2.

MAGNIFICATION VIEWS RIGHT BREAST:
There is a 6 mm cluster of amorphous calcifications at 11:00, 4 cm from the nipple, middle depth, in the right breast. These are new compared to prior mammograms and are indeterminate.

IMPRESSION:
New indeterminate cluster of amorphous calcifications in the right breast for which stereotactic core biopsy is recommended. BI-RADS 4a.

ADDENDUM-RESULTS OF STEREOTACTIC CORE BIOPSY:
The calcifications were adequately sampled. Pathology revealed fibrocystic changes with microcalcifications. No malignancy. 

IMPRESSION:
Benign pathology. Recommend return to routine screening. BIRADS 2. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDITIONAL VIEWS LEFT BREAST
History: Patient presents for screening mammogram. No current symptoms or complaints.
Compared to previous study dated Dec. 18, 2019.
FINDINGS:
The breasts are predominantly fatty with scattered fibroglandular densities. 
There are no suspicious masses, architectural distortions or skin thickening in either breast.
In the left breast upper outer quadrant, there are scattered punctate and amorphous microcalcifications, covering an area of approximately 15mm, higher in density compared to benign calcifications elsewhere.
There are no suspicious microcalcifications in the right breast.
No significant axillary, intramammary, or supraclavicular adenopathy.
IMPRESSION:
1. New grouped amorphous microcalcifications in the left breast upper outer quadrant, suspicious for early ductal carcinoma in situ. Stereotactic core biopsy recommended. BI-RADS 4B.
2. No suspicious findings in the right breast. Routine screening recommended in 1 year. BI-RADS 2. ➡ *Positive*
- ULTRASOUND BILATERAL BREASTS

HISTORY: Clinical exam reveals bilateral breast lumps, largest at 3 o'clock right breast. Recent screening mammogram at outside facility reported as BIRADS 0. No prior mammograms available for comparison. Patient's mother had postmenopausal breast cancer. 

FINDINGS:
Right breast: Targeted ultrasound was performed over the palpable abnormality at 3 o'clock position, 5 cm from the nipple. A corresponding 8 mm irregular hypoechoic solid mass is identified, demonstrating indistinct margins and posterior acoustic shadowing. No abnormal axillary lymph nodes are seen.

Left breast: Left breast tissue is heterogeneously echogenic. No discrete solid or cystic lesion is seen. No suspicious calcifications are identified. No abnormal axillary lymph nodes are seen.

IMPRESSION:
1. Right breast 8 mm solid mass at 3 o'clock position with suspicious features on targeted ultrasound, BIRADS 4. Tissue sampling is recommended. 
2. No sonographic evidence of malignancy in the left breast, BIRADS 2. Recommend routine screening.

ADDENDUM: Ultrasound-guided core biopsy of the right 3 o'clock mass was performed. Pathology revealed invasive ductal carcinoma, grade 2. The patient will be referred to surgery and oncology for further management. ➡ *Negative*
- SCREENING DIGITAL BREAST TOMOSYNTHESIS, BILATERAL 

HISTORY: This is a 55 year old female presenting for routine screening mammography.

TECHNIQUE: Screening digital breast tomosynthesis views were obtained including CC and MLO views of each breast.

COMPARISON: Comparison is made to prior exams dated 5/15/2020 and 5/16/2019.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses. 

There are no dominant masses, architectural distortion or suspicious calcifications.

The breast tissue is stable compared to prior exams. No significant interval change.

IMPRESSION:
1. Negative digital breast tomosynthesis examination. No mammographic evidence of malignancy.
2. Heterogeneously dense breasts.

RECOMMENDATION: Routine screening mammography in 1 year is recommended. 

ASSESSMENT CATEGORIES
BIRADS 1: Negative. There is a 0% likelihood of malignancy.
BIRADS 2: Benign finding. There is a 0% likelihood of malignancy.

ADDENDUM: Breast density notification - Your mammogram indicates that you have dense breast tissue. Dense breast tissue is relatively common and is found in about 40 percent of women. Dense breast tissue can make it more difficult to detect cancers in the breast by mammography because it can hide small abnormalities. Dense breast tissue may also be associated with an increased risk of breast cancer. This information is being provided to raise your awareness of the impact of breast density on cancer detection. For further information about dense breast tissue, as well as other breast cancer risk factors, please see the resources available on our practice website. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM:

INDICATION: Annual screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses. 

There are scattered areas of fibroglandular density bilaterally. 

No dominant masses, areas of architectural distortion or suspicious calcifications are appreciated in the right breast.

In the left breast, there is a group of amorphous microcalcifications in the upper outer quadrant anteriorly. The calcifications span an area of approximately 10 mm. No associated mass or distortion.

IMPRESSION:
1. Heterogeneously dense breasts.
2. New group of amorphous microcalcifications spanning 10 mm in the left upper outer breast, suspicious for ductal carcinoma in situ (DCIS). Additional evaluation with magnification views recommended. BI-RADS 4B.
3. Remainder of both breasts unremarkable. BI-RADS 2.

RECOMMENDATION:  
Left diagnostic mammogram with magnification views.

ADDENDUM - LEFT DIAGNOSTIC MAMMOGRAM WITH MAGNIFICATION VIEWS:
The group of amorphous microcalcifications in the left upper outer quadrant span 12 mm on magnification views. Stereotactic biopsy is recommended. BI-RADS 4C. ➡ *Positive*
- NORMAL ANNUAL SCREENING MAMMOGRAM.
This is a routine screening mammogram.
Compared to Previous: Yes. Last study 1 year prior.
Breast Density: Scattered fibroglandular densities.
Dense breast notification: Provided.
There are no suspicious masses, calcifications, architectural distortions or asymmetries identified in either breast. No radiographic evidence of malignancy.
Lymph nodes appear normal.
The skin and subcutaneous tissues are unremarkable.
Breast tissue composition is primarily fatty.
BIRADS 2 - BENIGN FINDING. Recommend routine screening in 1 year. ➡ *Not Stated*
- TMIST FFDM ARM OBSP ANNUAL MAMMOGRAM.  

The patient is enrolled in the randomized controlled Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and has been assigned to the full field digital mammography (FFDM) only arm after providing informed consent. The study is approved by the institution's Research Ethics Board.

Comparison to prior mammogram: Yes.
Breast tissue density: Scattered fibroglandular tissue (less than 75% dense).  
Recommended follow-up in OBSP: Annual screening.

There are no suspicious mammographic findings. The breast parenchyma appears within normal limits. 

Final BI-RADS Assessment Category: 1 (Negative).

REPORT CONFIDENCE ASSESSMENT:
- High confidence in stating the patient's enrollment in the TMIST trial and FFDM arm assignment. 
- High confidence in documenting comparison to prior exam and breast density.
- High confidence in the absence of any statement regarding microcalcifications. The report does not mention microcalcifications at all.
- High confidence in the overall normal mammographic findings and BI-RADS 1 assessment.

ROBUSTNESS CHECK:  
The key elements leading to the "Not Stated" label for microcalcifications are:
1) No direct mention of microcalcifications anywhere in the report. 
2) Affirmative statement of normal/negative mammographic findings.
Minor variations in other wordings would not alter the "Not Stated" label as long as those two elements are maintained. The report remains robust in returning this label. ➡ *Not Stated*
- BILATERAL MAMMOGRAMS.
History: Annual screening. No palpable lumps palpable. No personal history of breast cancer.
FINDINGS: The breasts are heterogeneously dense.
In the right breast there are no suspicious masses, architectural distortion or suspicious calcifications. Skin and areola are unremarkable. 
There is a group of pleomorphic calcifications measuring approximately 6 mm in the left breast at 12:00 4 cm from the nipple. These were not present on prior imaging from 2018 and are new. No associated mass or distortion.
Bilateral magnification images were obtained. The calcifications in the left breast upper outer quadrant are confirmed. They are irregular in shape, varying in size from punctate to 0.5 mm. A biopsy should be considered.
Axillae, skin and nipples are normal bilaterally.  
SUMMARY.
New group of suspicious microcalcifications upper outer left breast. BI-RADS 4B. Stereotactic or tomosynthesis guided biopsy is recommended. 
Right breast is normal. BI-RADS 1.
ADDENDUM:
The patient returned for stereotactic biopsy of the group of calcifications at left breast 12:00. Specimen radiograph confirmed calcifications. A clip was placed at the biopsy site. There were no complications. ➡ *Positive*
- BILATERAL MAMMOGRAM. CLINICAL HISTORY: Routine screening mammogram. Patient reports she is not currently on hormone replacement therapy. Patient reports a family history of breast cancer in her paternal grandmother. Comparison is made to prior mammogram of 01/05/2024. FINDINGS: The breasts are heterogeneously dense. There is no mammographic evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1: Negative. ➡ *Negative*
- REPORT:
"BILATERAL BREAST ULTRASOUND AND MAMMOGRAPHY
INDICATION: 54 year old female with palpable right breast mass, patient reports it has been present for at least 4 months and is now enlarging and starting to cause discomfort. No other breast complaints. No personal or family history of breast cancer.
COMPARISON: None.
FINDINGS:
Breast Composition: The breasts are heterogeneously dense, which may obscure detection of small masses.
Right Breast: 
At the site of palpable abnormality in the right upper outer quadrant approximately 10 cm from the nipple, there is an irregular mass with spiculated margins and associated architectural distortion measuring 18 x 14 x 12 mm. The mass demonstrates internal vascularity on color Doppler interrogation. Adjacent to the mass, there are several pleomorphic microcalcifications spanning an area of about 7 mm.   
Remainder of the right breast is unremarkable. No axillary, infraclavicular or supraclavicular adenopathy.
Left Breast: Within normal limits. No suspicious masses, calcifications, or areas of architectural distortion. No axillary, infraclavicular or supraclavicular adenopathy.
IMPRESSION:
1. Highly suspicious mass with associated microcalcifications right breast 10:00 position 10 cm from nipple, BI-RADS 5. Ultrasound guided core biopsy recommended for definitive characterization. 
2. Heterogeneously dense breast tissue which could obscure small masses. Recommend annual screening with mammography and consideration of supplemental screening breast MRI.
ADDENDUM: Ultrasound guided core biopsy of the right 10:00 mass was performed, pathology pending. Breast MRI is suggested for further evaluation and surgical planning if biopsy is positive for malignancy."
LABEL: Positive for microcalcifications in the current report. ➡ *Positive*
- MAMMOGRAPHIC REPORT
CLINICAL INDICATION: Routine screening mammogram.  
COMPARISON: Screening mammogram of 12/18/2021.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:  
The breasts are heterogeneously dense which may obscure detection of small masses.
RIGHT BREAST: There are no dominant masses, areas of architectural distortion, or suspicious clustered microcalcifications. Stable benign appearing calcifications are noted. 
LEFT BREAST: There are no dominant masses, areas of architectural distortion, or suspicious clustered microcalcifications. Stable coarse benign calcifications are noted.  
IMPRESSION:
BIRADS 2: Benign. No evidence of malignancy. Recommend continued routine screening in 1 year.

ADDENDUM:
Comparison was made to outside screening mammogram from 12/18/2021. No significant interval change. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Annual screening

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available

FINDINGS:

Right Breast:
The breast is heterogeneously dense (BI-RADS density category C). No significant masses, architectural distortion, or suspicious calcifications are identified.

Left Breast:
The breast is heterogeneously dense (BI-RADS density category C). In the upper outer quadrant, there is a small cluster of amorphous calcifications spanning approximately 6 mm. No associated mass, architectural distortion, or skin thickening is identified.

IMPRESSION:

Right Breast: 
Negative mammogram, BI-RADS 1. Continue annual screening.

Left Breast:
BI-RADS 0: Incomplete - need additional imaging evaluation. Small cluster of amorphous calcifications in the upper outer quadrant of the left breast. Recommend spot compression magnification views for further characterization.

ADDENDUM:
Spot compression magnification views of the left breast were performed, demonstrating a 6 mm cluster of amorphous calcifications at the site of the previously identified abnormality in the upper outer quadrant. No associated mass or architectural distortion is identified.

FINAL IMPRESSION:

Right Breast: Negative, BI-RADS 1.

Left Breast: BI-RADS 4B: Suspicious abnormality, biopsy should be considered. 6 mm cluster of amorphous calcifications in the upper outer quadrant of the left breast, highly suggestive of malignancy. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses. There are no dominant masses, architectural distortion, or suspicious calcifications. Scattered benign-appearing calcifications are noted in both breasts, likely vascular calcifications. No significant interval change compared to prior exam.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography in 12 months.
2. Heterogeneously dense breasts limit sensitivity of mammography.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM. CLINICAL HISTORY: 59 year old female for routine screening mammogram. FINDINGS: There are scattered benign calcifications bilaterally. There are no suspicious masses, architectural distortion, or suspicious calcifications. IMPRESSIONS: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM. History: Screening mammogram. Comparison: None. Findings: There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. Impression: 1. No mammographic evidence of malignancy. 2. BI-RADS 1. ➡ *Negative*
- MAMMOGRAM REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints. Patient reports sister diagnosed with breast cancer at age 45.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right breast: There are no dominant masses, architectural distortion, or suspicious calcifications.  
Left breast: There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable benign-appearing lymph nodes are present in both axillae.
IMPRESSION:
Bilateral mammogram, negative. 
BIRADS 1 - Negative. Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
HISTORY: 55-year-old female presenting for routine screening mammogram. No current breast complaints. Personal history of right breast biopsy proven fibroadenoma at age 40. Maternal aunt with breast cancer in her 60s.
FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses. 
Right breast: There are no dominant masses, architectural distortion, or suspicious calcifications. Stable benign appearing calcifications in the upper outer quadrant, likely representing calcified fibroadenoma, corresponding to prior biopsy site. No significant interval change.
Left breast: There are no dominant masses, architectural distortion, or suspicious calcifications. No significant interval change.
IMPRESSION:
1. Bilateral heterogeneously dense breast tissue, potentially limiting mammographic sensitivity. 
2. Stable benign right breast calcifications, likely representing calcified fibroadenoma. No suspicious mammographic findings.
3. No mammographic evidence of malignancy. Recommend continued annual screening mammography and clinical breast exams.
BIRADS 2 - Benign findings, both breasts.
ADDENDUM: Patient notified of results via patient portal. Letter sent to referring provider. ➡ *Negative*
- MAMMOGRAM LEFT.

INDICATION: Screening. No current breast complaints. Last mammogram approximately 1 year ago, normal.

COMPARISON: Prior mammogram from May 2022.

FINDINGS: The breast is heterogeneously dense, which may limit sensitivity. There are no dominant masses or suspicious calcifications. There are no new findings or significant changes compared to prior mammogram.

IMPRESSION:
1. No mammographic evidence of malignancy.
2. Recommend continued annual screening mammography in 1 year.

BIRADS 1 - NEGATIVE. ➡ *Negative*
- MAMMOGRAM - BILATERAL WITH CAD

CLINICAL HISTORY: 72 year old female for screening mammogram.

COMPARISON: 12/12/2023.

FINDINGS: The breasts are heterogeneously dense. Comparison is made to prior examination. Again seen are scattered benign-appearing calcifications in both breasts. There is a new focus of highly suspicious, grouped, fine, pleomorphic calcifications in the left breast at approximately the 10 o'clock position. This measures approximately 1 cm in greatest diameter and is located approximately 6 cm from the nipple.

IMPRESSION:

1. New focus of highly suspicious, grouped, fine, pleomorphic calcifications in the left breast. Given the patient's age, this is highly suspicious for malignancy. Correlation with spot magnification views and possible biopsy is recommended.

BI-RADS: 4C (Highly suspicious for malignancy). ➡ *Positive*
- BILATERAL MAMMOGRAM

COMPARISON: 2 years prior

INDICATION: 51-year-old female presents for routine screening mammogram. No known palpable masses, pain, or nipple discharge.

FINDINGS:
RIGHT BREAST:
Heterogeneously dense parenchyma. No mammographic evidence of malignancy. No significant interval change since prior exam. BI-RADS 1 - Negative.

LEFT BREAST:
Heterogeneously dense parenchyma. Cluster of amorphous microcalcifications spanning 6 mm is noted in the upper outer quadrant, 11 cm from the nipple. This is a new finding compared to prior exam. No spiculated masses, architectural distortion or skin thickening. BI-RADS 4 - Suspicious abnormality.

IMPRESSION:
1. Stable and benign appearing right breast. BI-RADS 1.
2. New cluster of suspicious microcalcifications in the left upper outer breast, BI-RADS 4. Stereotactic biopsy is recommended for further evaluation.

ADDENDUM:
The patient returned for left stereotactic biopsy of the microcalcifications in the 2:00 position. Specimen radiograph confirmed calcifications within the core samples. Final pathology revealed atypical ductal hyperplasia with associated calcifications. 6-month follow-up diagnostic mammogram is recommended to assess for stability. ➡ *Positive*
- CLINICAL HISTORY: 45 year old female presents for routine screening mammogram. No significant family history of breast cancer. No prior surgeries or biopsies.

FINDINGS:
Right breast: Scattered fibroglandular densities. No suspicious masses, calcifications, or architectural distortion. 
Left breast: Scattered fibroglandular densities. No suspicious masses, calcifications, or architectural distortion.  

IMPRESSION:
1. Negative bilateral screening mammogram.  
2. BI-RADS Category 1 - Negative. Routine screening in 1 year is recommended. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
The breasts are almost entirely fatty.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

In the upper outer quadrant of the left breast there is a new cluster of indeterminate microcalcifications which is new compared to prior exams. This cluster measures approximately 6 mm in size. Additional magnification views confirm the presence of this cluster. 

The skin is normal in thickness. The nipples are within normal limits. There are no enlarged axillary lymph nodes.

IMPRESSION: 
New cluster of indeterminate microcalcifications upper outer quadrant left breast, BI-RADS 0. Additional imaging evaluation with magnification views is recommended for further characterization. The patient is being recalled for additional diagnostic imaging.

Otherwise, no mammographic evidence of malignancy, BI-RADS 2. Recommend continued routine screening in 1 year. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM.

Compared to previous mammogram dated March 17, 2020, there are no significant interval changes.

The breasts are heterogeneously dense (51-75% glandular). This may lower the sensitivity of mammography.

There are no dominant masses, suspicious calcifications or architectural distortion.

BIRADS Category 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- BILATERAL DIAGNOSTIC MAMMOGRAM FOR SCREENING. Clinical History: The patient is a pleasant 48-year-old female presenting for her annual screening mammogram today. Comparison: Comparison is made to mammograms performed on 04/16/2023. Findings: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcifications. Impression: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1. ➡ *Negative*
- NORMAL SCREENING MAMMOGRAM.
Compared to Previous: No.
Breast composition: The breasts are almost entirely fatty.
There are no dominant masses, architectural distortion or suspicious calcifications.
IMPRESSION:
Negative screening mammogram.
BIRADS 1: Negative. Routine screening recommended in 1 year. ➡ *Not Stated*
- REGULAR SCREENING MAMMOGRAM.
Mammogram obtained for breast cancer screening with no patient symptomatology.  
Breast tissue density: Heterogeneously dense, which may obscure small masses.
Comparison: Most recent mammogram from 2 years ago.
No masses, asymmetries, or architectural distortions are identified. 
Stable and benign-appearing scattered bilateral fibroglandular densities noted.
No focal dominant lesion.
No suspicious grouped calcifications seen.
Assessment: BI-RADS Category 2 - Benign findings. Recommend continued annual screening mammography in 12 months. ➡ *Not Stated*
- SCREENING MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: Compared to outside screening mammogram from MM/DD/YYYY.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
BIRADS Category 2: Benign.

RECOMMENDATION:
Continued annual screening mammography is recommended. ➡ *Not Stated*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: None

TECHNIQUE: Bilateral CC and MLO views were obtained.

FINDINGS:
The breast tissue is heterogeneously dense which may obscure detection of small masses.

Right Breast:
There are no dominant masses, architectural distortion or suspicious calcifications. 

Left Breast: 
At the 10 o'clock position, 6 cm from the nipple, there is a group of amorphous microcalcifications in a segmental distribution spanning an area of approximately 10 mm. No associated mass, architectural distortion or skin thickening is seen.

IMPRESSION:
1. Heterogeneously dense breast tissue which may lower the sensitivity of mammography.
2. Grouped amorphous microcalcifications in a segmental distribution at 10 o'clock in the left breast, suspicious for ductal carcinoma in situ (DCIS). Additional magnification views are recommended for further characterization. BI-RADS 4B. Stereotactic biopsy is recommended if magnification views confirm the presence of suspicious microcalcifications.
3. No dominant masses, architectural distortion or suspicious calcifications in the right breast. BI-RADS 2, benign finding.

ADDENDUM: Magnification views of the left breast confirm the presence of grouped amorphous microcalcifications in a segmental distribution at 10 o'clock. A stereotactic biopsy is recommended. Updated BI-RADS 4C for the left breast. ➡ *Positive*
- IMAGING STUDY:
PROCEDURE: Screening Mammogram.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of both breasts.

FINDINGS:
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
No significant masses, architectural distortion, or suspicious calcifications are seen.

IMPRESSION:
Negative bilateral screening mammogram.
BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- MAMMOGRAM LEFT BREAST.
HISTORY: Screening
COMPARISONS: 05/09/2022
TECHNIQUE: Standard CC and MLO views of the left breast were obtained
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- FINDINGS:
Bilateral digital mammograms were performed with tomosynthesis views. The breasts are heterogeneously dense, which may limit sensitivity of mammography. 

There is a new 8 mm cluster of fine pleomorphic calcifications in the right breast at 10:00, 6 cm from the nipple. The calcifications are suspicious for malignancy. No other suspicious calcifications, masses, or areas of architectural distortion are identified in the right breast. 

In the left breast, there are scattered areas of fibroglandular density. No suspicious masses, calcifications, or areas of architectural distortion are identified.

IMPRESSION:
1. New cluster of suspicious calcifications in the right breast at 10:00, 6 cm from the nipple, suspicious for malignancy. Stereotactic core biopsy is recommended. BIRADS 4C. 
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography.
3. Probably benign findings left breast. BIRADS 2.

ADDENDUM:
The patient returned for stereotactic biopsy of the suspicious calcifications in the right breast at 10:00. Specimen radiograph confirmed that calcifications were sampled. Final pathology results showed ductal carcinoma in situ, intermediate nuclear grade, associated with microcalcifications. The patient will be referred to breast surgery for further management. ➡ *Positive*
- CLINICAL INDICATION: Follow up of known right breast cancer.

TECHNIQUE: Bilateral digital mammography and targeted right breast ultrasound.

COMPARISON: Prior outside mammograms and ultrasound from 2019.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There is architectural distortion and skin thickening in the right breast upper outer quadrant, corresponding to the known cancer.

Magnification views of the right breast demonstrate a new cluster of pleomorphic microcalcifications at 10:00, 6 cm from the nipple, spanning approximately 8 mm. This cluster is located superior to the known cancer site.

Targeted ultrasound of the right breast at 10:00 shows a 12 mm irregular hypoechoic mass with indistinct margins and posterior acoustic shadowing, corresponding to the mammographic distortion. No definite correlate for the new microcalcifications is identified sonographically.

The left breast is unremarkable. No suspicious masses, calcifications, or areas of distortion.

IMPRESSION:
1. Known right breast cancer at 10:00, demonstrating a 12 mm irregular mass on ultrasound, consistent with the patient's history. BI-RADS 6.

2. New cluster of suspicious pleomorphic microcalcifications at 10:00 in the right breast, superior to the known cancer site, highly concerning for additional malignancy. Stereotactic biopsy is recommended for definitive characterization. BI-RADS 4C.

3. Remainder of the right breast and left breast are negative. BI-RADS 1 left breast. ➡ *Positive*
- FINDINGS: Bilateral breast tissue contains scattered fibroglandular densities. There is no focal mass, architectural distortion, or suspicious calcifications in either breast. There are stable bilateral benign-appearing calcifications, likely vascular in nature. No significant change from prior exam.

Targeted ultrasound of the right breast 10 o'clock position demonstrates normal fibroglandular tissue with no discrete solid or cystic lesion.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. Stable benign calcifications bilaterally. These do not require further evaluation.

BI-RADS Category 2: Benign findings. Recommend routine screening in 12 months. ➡ *Negative*
- MAMMOGRAM AND TARGETED ULTRASOUND OF THE LEFT BREAST

INDICATION: New palpable mass at 10 o'clock.

TECHNIQUE: Diagnostic digital mammography and targeted ultrasound of the left breast.

COMPARISON: Prior mammogram from 12/15/2021.

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses.

No dominant masses, architectural distortion, or suspicious calcifications are seen in either breast. The palpable mass at 10 o'clock in the left breast has a correlating 6 mm oval circumscribed hypoechoic mass on ultrasound, consistent with a benign finding such as fibroadenoma. 

Stable postsurgical changes from prior right lumpectomy are noted.

IMPRESSION:
1. Benign appearing 6 mm mass at site of palpable abnormality at 10 o'clock left breast, most consistent with fibroadenoma. 
2. No suspicious mammographic findings bilaterally. Recommend annual screening mammography.

ASSESSMENT: BI-RADS 2 - Benign findings. Recommend routine screening. ➡ *Negative*
- SCREENING MAMMOGRAM

CLINICAL HISTORY: 59-year-old female, annual screening mammogram.

TECHNIQUE: Screening Full Field Digital Mammography (FFDM) including CC and MLO views of both breasts.

COMPARISON: None available.

FINDINGS:

Both breasts are heterogeneously dense (25-50% glandular tissue), which may obscure detection of small masses.

No suspicious masses, architectural distortion, or suspicious calcifications are identified in either breast. 

Skin and nipple are unremarkable bilaterally. No evidence of enlarged axillary lymph nodes.

IMPRESSION: 

ACR BI-RADS Category 1: Negative. 

RECOMMENDATION:

Screening mammography is recommended annually as part of routine health maintenance. ➡ *Not Stated*
- TMIST FFDM ARM OBSP MAMMOGRAM
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes. 
Prior mammogram from 2 years ago available for comparison.
Breast Density >= 75%: No.
Breast Density < 75%: Yes. 
Scattered areas of fibroglandular density.
OBSP to recall client in 1 year: Yes.
There are no suspicious masses, architectural distortions or areas of asymmetry. No dominant masses. No skin thickening or retraction. 
BIRADS-1/2.
1 YEAR SCREENING RECOMMENDED ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED VIEWS LEFT BREAST.
History: Screening mammogram. No prior studies available for comparison.
FINDINGS: The breasts are predominantly fatty with areas of scattered fibroglandular density.
On the left CC and MLO views, there is a new 8 mm cluster of pleomorphic microcalcifications at 10:00, 3 cm from the nipple.
No suspicious masses, architectural distortion, or skin thickening in either breast.
No abnormal axillary lymph nodes.
IMPRESSION:
BI-RADS 4: Suspicious 8 mm cluster of pleomorphic microcalcifications at 10:00 left breast, 3 cm from the nipple. Stereotactic core needle biopsy recommended for definitive characterization.
BI-RADS 2: Benign right breast. Annual screening mammogram recommended in 1 year.
ADDENDUM: Patient returned for stereotactic core biopsy of the suspicious left breast microcalcifications. Pathology revealed ductal carcinoma in situ (DCIS), intermediate grade, cribriform type. Wide local excision planned. ➡ *Positive*
- ULTRASOUND EXAMINATION LEFT BREAST AND AXILLA

INDICATION: Palpable lump left breast lower inner quadrant near the areolar border.

COMPARISON: None.

FINDINGS:
Ultrasound of the left breast shows heterogeneously echogenic parenchyma. At 5 o'clock position 2 cm from the nipple there is a 15 x 11 x 9 mm irregular hypoechoic mass with angular margins and posterior acoustic shadowing. The mass demonstrates internal vascularity. No other focal mass, area of architectural distortion, or abnormal echogenic focus is demonstrated in the remainder of the left breast.

Ultrasound of the left axilla demonstrates normal appearing lymph nodes.

Targeted ultrasound of the palpable area at the 4 o'clock position of the left breast near the areola demonstrates a 6 mm cluster of punctate echogenic foci consistent with microcalcifications. 

IMPRESSION:
1. Irregular hypoechoic mass with suspicious features at 5 o'clock position of the left breast. Ultrasound guided core biopsy is recommended. 
2. Cluster of microcalcifications at site of palpable concern, left breast 4 o'clock position near the areola. Recommend mammographic work-up of this area.

BI-RADS 4: Suspicious findings. Biopsy should be considered. ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS
INDICATION: Annual screening mammogram
TECHNIQUE: Bilateral digital mammograms were obtained including standard CC and MLO views.
COMPARISON: None available
FINDINGS:
The breast tissue is heterogeneously dense which may obscure detection of small masses.
Right Breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Left Breast: 
There are scattered fibroglandular densities. No dominant masses or areas of architectural distortion.
At the 10:00 position, 7 cm from the nipple, there is a group of amorphous microcalcifications in a segmental distribution, spanning an area of approximately 2 cm. Additional magnification views in the CC and MLO projections confirm the presence of a group of amorphous microcalcifications. 
The nipples are within normal limits bilaterally. The skin appears normal. No axillary lymphadenopathy.
IMPRESSION:
1. Heterogeneously dense breast tissue which may lower the sensitivity of mammography.
2. Grouped amorphous microcalcifications in the upper outer quadrant of the left breast, suspicious for ductal carcinoma in situ (DCIS). Additional evaluation with stereotactic biopsy is recommended. BI-RADS 4B.
3. No dominant masses, areas of architectural distortion, or suspicious calcifications in the right breast. BI-RADS 2.
ADDENDUM: The patient proceeded with stereotactic biopsy of the grouped microcalcifications in the left breast at the 10:00 position, 7 cm from the nipple. Pathology results showed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints.
TECHNIQUE: Standard CC and MLO views of the breasts were performed.
COMPARISON: Comparison is made to prior mammogram from 12/15/2021, available in PACS.
FINDINGS:
Right breast: The breast is heterogeneously dense (BI-RADS category C). There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable coarse benign calcifications are noted in the upper outer quadrant.
Left breast: The breast is heterogeneously dense (BI-RADS category C). There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable coarse benign calcifications are noted in the upper inner quadrant.
IMPRESSION:
1. No mammographic evidence of malignancy. No suspicious calcifications.
2. Stable bilateral benign calcifications.
3. Heterogeneously dense breasts, which may obscure detection of small masses.
RECOMMENDATIONS:
Continue annual screening mammography in 12 months. Please inform us if you notice any new breast lumps, skin changes, nipple discharge or other concerns before your next mammogram.
BI-RADS ASSESSMENT CATEGORY: 2 - Benign findings. Recommend routine screening in 12 months. ➡ *Negative*
- SCREENING BILATERAL DIGITAL MAMMOGRAM.
History: Routine screening. No clinical concerns.
Technique: Standard CC and MLO views bilaterally.
Comparison: Screening mammogram from July 2020.
FINDINGS:
The breasts are almost entirely fatty with scattered areas of fibroglandular density.
No significant change from previous exam. No dominant masses, architectural distortion or suspicious calcifications.
There are multiple vascular calcifications bilaterally, stable from prior.
A few benign-appearing calcifications scattered in the left breast parenchyma are noted, similar to prior.
IMPRESSION:
BIRADS Category 2: Benign findings.
Recommend routine screening mammogram in 1 year.
Addendum:
On further review by the staff radiologist, a cluster of microcalcifications is noted at the 11:00 position of the right breast, approximately 5 cm from the nipple. These were not seen on the prior mammogram and are new.
Given the new cluster of microcalcifications, a diagnostic mammogram with magnification views is recommended for further evaluation. The BIRADS Category is changed to 0 for the right breast.
Magnification views right breast:
The magnification views confirm a cluster of pleomorphic calcifications spanning approximately 6 mm at the 11:00 position right breast. The distribution is segmental. No associated mass or architectural distortion.
IMPRESSION ADDENDUM:
BIRADS Category 4: Suspicious abnormality.
Stereotactic core biopsy is recommended for the new cluster of pleomorphic calcifications in the right breast. ➡ *Positive*
- FINDINGS:
Left mammogram shows a moderate amount of scattered fibroglandular tissue. No focal asymmetry, questionable lesion, or architectural distortion identified. No suspicious calcifications seen. Skin is unremarkable. Stable post-surgical changes from prior right mastectomy noted.

Targeted left breast ultrasound at the site of the palpable abnormality shows normal fibroglandular tissue, with no discrete solid mass, complex cystic lesion or other sonographic abnormality in the area of concern. 

IMPRESSION:
1. No evidence of malignancy in the left breast. Recommend routine annual screening mammography. 
2. Targeted left breast ultrasound is negative for correlate to the palpable finding. If remains of clinical concern, short term follow up or biopsy could be considered.

BIRADS 1 - Negative.

ADDENDUM: 
After review of additional clinical history provided, the final assessment category remains BIRADS 1 - Negative. No mammographic or sonographic findings to explain the palpable abnormality in the left breast. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

In the upper outer quadrant of the right breast, there is a new group of amorphous microcalcifications spanning an area of about 1 cm. These were not present on the prior mammogram. Additional magnification views in the right CC and MLO projections confirm the presence of these microcalcifications.

Stable bilateral circumscribed masses, likely benign lymph nodes, are noted.

IMPRESSION:
BI-RADS 0: Incomplete: Need additional imaging evaluation - Right breast microcalcifications. Recommend bilateral diagnostic mammogram with magnification views of the right breast for further characterization.

ADDENDUM:
The patient returned for additional diagnostic imaging and stereotactic biopsy of the group of microcalcifications in the right breast at 10:00, 6 cm from the nipple. Pathology results showed ductal carcinoma in situ, intermediate nuclear grade, without necrosis.

Final BI-RADS assessment is BI-RADS 4C: Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- SCREENING FULL FIELD DIGITAL MAMMOGRAM. COMPARISON: NONE.

INDICATION: Routine screening.

BREAST COMPOSITION: The breasts are almost entirely fatty.

FINDINGS:
There are no dominant masses, areas of architectural distortion or suspicious calcifications.

IMPRESSION:
Negative screening mammogram. 
BIRADS Category 1 - Negative. Routine screening is recommended in 1 year. ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAM, LEFT BREAST ULTRASOUND
INDICATION: Palpable abnormality left breast.
COMPARISON: None available.
FINDINGS:
The breasts are fibroglandular without gross asymmetry.
Left breast: There is an irregular 10 mm hypoechoic nodule at the 2:00 position, 5 cm from the nipple. This demonstrates posterior acoustic shadowing. Elsewhere in the left breast, there are 2-3 small simple-appearing cysts.
Right breast: There are no dominant masses or suspicious calcifications.
Associated findings: The left axilla demonstrates an enlarged lymph node measuring up to 8 mm in short axis with cortical thickening.
Impression:
1. 10 mm solid mass, left breast 2:00, with suspicious features on ultrasound. This could represent a primary breast malignancy. Strongly recommend tissue sampling.
2. Enlarged left axillary node with abnormal cortical thickening, suspicious for regional lymph node involvement.
3. Cluster of indeterminate microcalcifications in upper outer left breast. These require further evaluation with magnification views. If patient is to have biopsy of the 2:00 mass, suggest these calcifications also be targeted concomitantly.
BI-RADS: Left: 4C
BI-RADS: Right: 2 
OVERALL BI-RADS: 4C ➡ *Positive*
- FINDINGS:

BILATERAL SCREENING MAMMOGRAPHY:

The breasts are predominantly fatty.  There are scattered areas of fibroglandular density.

There is an approximately 0.5 cm cluster of heterogeneous calcifications in the upper outer quadrant of the left breast 6 cm from the nipple, mildly increased compared to prior exam. No associated mass, architectural distortion or skin thickening.  

No spiculated mass, architectural distortion or suspicious calcifications in either breast.  

IMPRESSION:

BI-RADS Category 0: Incomplete assessment.  The cluster of left breast calcifications is suspicious and warrants further evaluation with magnification views.

MAGNIFICATION VIEWS OF THE LEFT BREAST:  

The previously described 6 mm cluster of calcifications in the upper outer left breast is again noted.  The calcifications are heterogeneous and pleomorphic in morphology, suspicious for malignancy.  

IMPRESSION: 

BI-RADS Category 4: Suspicious abnormality.  Stereotactic core needle biopsy of the cluster of left breast calcifications is recommended.  

STEREOTACTIC CORE NEEDLE BIOPSY LEFT BREAST:

Stereotactic core needle biopsy of the cluster of calcifications at 2:00, 6 cm from the nipple in the left breast was performed utilizing an 11 gauge vacuum-assisted device.  Five core specimens were obtained.  Specimen radiograph confirms the presence of calcifications.  A ribbon clip was deployed at the biopsy site.  The patient tolerated the procedure well.  

PATHOLOGY: Pending. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAMS:
Technique: Bilateral digital mammograms were obtained in standard CC and MLO projections with added magnification views of the right breast.
Comparison: None.
FINDINGS:
Breasts are heterogeneously dense which could obscure small masses.
No concerning asymmetries, architectural distortion, or suspicious masses are seen in either breast.
There are scattered fibroglandular elements bilaterally. 
At the 3 o'clock position of right breast, 5 cm from nipple, there is a group of pleomorphic microcalcifications spanning 8 mm. No other suspicious microcalcifications are seen.
Bilateral skin is normal. Nipples are unremarkable.
No significant axillary lymphadenopathy.
Benign calcifications are seen bilaterally, likely representing fibroadenomas.
IMPRESSION:
Heterogeneously dense breasts with group of pleomorphic microcalcifications at 3 o'clock of the right breast, 5 cm from nipple, spanning 8 mm. Additional magnification views confirm these as suspicious. Recommend stereotactic core biopsy for a definitive diagnosis.
All other findings are benign.
BIRADS: Right: 4, Suspicious abnormality. Left: 2, Benign.

ADDENDUM: Right breast stereotactic core biopsy was performed at the 3:00 position, 5 cm from nipple, using an 8G needle. Specimen radiograph confirmed microcalcifications. Final pathology pending. ➡ *Positive*
- SCREENING MAMMOGRAM.
CLINICAL HISTORY: Annual screening mammogram. No current breast complaints. No personal history of breast cancer. 
COMPARISON: Screening mammogram of March 27, 2021.
FINDINGS:
The tissue pattern is predominantly fatty.
There are no masses, architectural distortion or suspicious calcifications.
There are no significant changes compared to prior mammogram.  
IMPRESSION:
Negative bilateral screening mammogram. 
BIRADS Category 1 - Negative. Routine screening in 1 year. ➡ *Negative*
- FINDINGS:

Bilateral breast tissue contains scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications.

Targeted ultrasound was performed in the right breast at the site of palpable abnormality at 10 o'clock, 4 cm from the nipple. No sonographic correlate was identified. The palpable area corresponds to normal fibroglandular tissue.

IMPRESSION:

1. No mammographic or sonographic evidence of malignancy. Recommend continued routine screening mammography. 

2. Palpable abnormality right breast 10 o'clock represents normal fibroglandular tissue, no sonographic correlate.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening

COMPARISON: None

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast.

At the 12 o'clock position of the right breast, approximately 6 cm from the nipple, there is a new group of amorphous calcifications spanning an area of about 8 mm. This is a suspicious finding and requires further evaluation.

The left breast is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation
Right breast: New group of amorphous calcifications at 12 o'clock position, 6 cm from nipple, suspicious for malignancy. Recommend diagnostic mammogram with magnification views and possible ultrasound for further assessment.
Left breast: Negative.

ADDENDUM:
Right diagnostic mammogram with magnification views confirms a group of suspicious amorphous calcifications at the 12 o'clock position of the right breast, 6 cm from the nipple, corresponding to the screening mammogram findings. Ultrasound of this area shows no correlating mass. Recommend stereotactic core biopsy for definitive diagnosis.

BI-RADS Category 4: Suspicious - Biopsy should be considered ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
HISTORY: 49-year-old female presenting for routine bilateral screening mammogram.
TECHNIQUE: Bilateral digital screening mammographic views were obtained.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense which may lower the sensitivity of mammography. 
Right Breast:
There are no suspicious masses, architectural distortions or suspicious calcifications. 
Left Breast:  
There are no suspicious masses or architectural distortions. Scattered fibroglandular densities are present.
A new group of amorphous calcifications is seen in the upper outer quadrant posteriorly on the MLO view only. Additional spot compression magnification views are recommended for further evaluation. 
IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation is needed. 
Specifically, spot compression magnification views of the left breast are recommended for further characterization of the amorphous calcifications in the upper outer quadrant posteriorly. 
ADDENDUM:
LEFT DIAGNOSTIC MAMMOGRAM
TECHNIQUE: Spot compression magnification views were performed of the upper outer left breast.  
FINDINGS: The spot compression magnification views demonstrate a group of amorphous calcifications in the upper outer quadrant of the left breast. The calcifications are new compared to the prior examination and are suspicious for malignancy. A stereotactic biopsy is recommended for further evaluation.  
IMPRESSION: 
BI-RADS Category 4: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- SCREENING MAMMOGRAPHY TMIST
INDICATION: Screening
COMPARISON: None.
TECHNIQUE: Digital breast tomosynthesis views of both breasts were obtained including the craniocaudal (CC) and mediolateral oblique (MLO) projections.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
There are scattered areas of fibroglandular density bilaterally.
IMPRESSION:
Negative digital breast tomosynthesis screening mammogram.
BIRADS 1: Negative. Recommend routine screening in 12 months. ➡ *Not Stated*
- FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses. No dominant mass, suspicious calcifications, or other findings suspicious for malignancy are seen in either breast. There are bilateral subcentimeter lymph nodes in the axillae, which appear normal.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend routine annual screening mammography in 12 months.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDED VIEWS RIGHT BREAST.
History: Screening mammogram.
FINDINGS: The breasts are heterogeneously dense. There are no suspicious left mammographic findings.
In the right breast, there is a 7 mm cluster of round pleomorphic calcifications spanning 2 cm located at the 9:00 position 6 cm from the nipple.
Magnification views of the right breast demonstrate the pleomorphic calcifications as described above. No associated mass, asymmetry, or architectural distortion.
No skin thickening or nipple retraction.
SUMMARY:
7 mm cluster of pleomorphic calcifications at 9:00 position of right breast, suspicious for DCIS. Recommend image-guided core biopsy. BI-RADS 4B.
No suspicious findings left breast. BI-RADS 2.
ADDENDUM: The patient returned for stereotactic core biopsy of the right breast calcifications. Pathology revealed ductal carcinoma in situ, intermediate grade with comedonecrosis. ➡ *Positive*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: 45-year-old female presents for screening mammogram. No current breast complaints. 
TECHNIQUE: Digital breast tomosynthesis views were obtained bilaterally.   
COMPARISON: Comparison is made to outside screening mammogram from 11/15/2020. 
FINDINGS:  
Right breast: The breast is heterogeneously dense (BI-RADS category C). There are no dominant masses, architectural distortion, or suspicious calcifications. 
Left breast: The breast is heterogeneously dense (BI-RADS category C). There are no dominant masses, architectural distortion, or suspicious calcifications. 
IMPRESSION: 
BI-RADS category 1: Negative bilateral screening mammogram. No mammographic evidence of malignancy. 
RECOMMENDATIONS: 
Recommend continued annual screening mammography in 12 months. Clinical breast exams are also recommended on an annual basis. ➡ *Negative*
- BREAST IMAGING REPORT
CLINICAL HISTORY: 55 year old female with palpable nodule, upper outer quadrant of left breast. History of right breast cancer, treated with mastectomy and radiation therapy in 2015. Annual screening.
TECHNIQUE: Bilateral digital mammography and targeted ultrasound of the left breast. 
COMPARISON: Prior mammograms and ultrasound from 8/15/2024.
FINDINGS:
RIGHT BREAST: 
Mastectomy changes stable. No evidence of residual breast tissue.
LEFT BREAST MAMMOGRAM: 
The breast is heterogeneously dense (BI-RADS C). No suspicious masses, architectural distortion or microcalcifications. Stable benign calcifications in the upper outer quadrant. No significant interval change.
LEFT BREAST ULTRASOUND:
Targeted ultrasound to the area of palpable abnormality in the 2 o'clock position, 8 cm from the nipple, demonstrates a 6 mm hypoechoic solid mass, oriented parallel to the skin surface. The margins are circumscribed. No posterior acoustic features. 
IMPRESSION:
1. Stable post-mastectomy changes of the right breast.
2. Heterogeneously dense left breast tissue, no suspicious mammographic findings. Stable benign calcifications.
3. 6 mm solid mass in the left breast 2:00, 8 cm from the nipple, likely benign, such as a fibroadenoma. Recommend clinical follow up and diagnostic mammogram in 6 months to document stability. 
BI-RADS Category 3 - Probably benign finding. Short interval follow-up suggested. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAPHY

INDICATION: Annual screening mammography.

COMPARISON: None.

TECHNIQUE: CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are almost entirely fatty.

Right Breast:
There are no suspicious masses, calcifications, or other findings.

Left Breast: 
There is a new group of fine pleomorphic calcifications in the upper outer quadrant posterior depth on CC and MLO views. The calcifications form a group measuring approximately 6 mm. No associated mass, distortion, or asymmetry.

IMPRESSION:
Right breast: No mammographic evidence of malignancy, BI-RADS 1 Negative.
Left breast: Suspicious group of new pleomorphic calcifications, BI-RADS 4 Suspicious.

RECOMMENDATIONS:
- Stereotactic biopsy is recommended for the group of suspicious calcifications in the left breast at the 1:00 position, posterior depth, as marked on the CC view of the mammogram. A biopsy requisition form has been completed; please contact the breast imaging department to schedule the procedure.
- Routine annual screening mammography is recommended in one year for the right breast.

ADDENDUM:
The patient returned for left breast stereotactic biopsy which confirmed invasive ductal carcinoma, moderately differentiated, with associated DCIS, intermediate nuclear grade. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- MAMMOGRAPHY REPORT
Patient Name: Jane Smith
MRN: 1234567
Date of Exam: 01/01/2023
Indication: Annual screening mammogram
Technique: Standard views of the right and left breasts were obtained.
Comparison: None available
Findings:
The breast tissue is heterogeneously dense which could obscure detection of small masses. No dominant masses, architectural distortion or suspicious calcifications are identified.
There are no mammographic findings that are concerning for malignancy.
IMPRESSION:
BIRADS Category 1: Negative mammogram.
Routine screening mammography is recommended in 1 year.
ADDENDUM:
Patient was also enrolled in the TMIST trial and randomized to the full field digital mammography arm. Informed consent was obtained prior to the study. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Annual screening mammogram. No complaints. Patient reports history of breast cancer in maternal aunt.
TECHNIQUE: Standard CC and MLO views of both breasts. 
COMPARISON: Previous outside screening mammogram from 11/2019.
FINDINGS:
Right Breast:
The right breast is heterogeneously dense (BI-RADS composition c). No significant masses, architectural distortion or suspicious calcifications. Stable benign appearing calcifications in the upper outer quadrant, consistent with fibrocystic changes.
Left Breast: 
The left breast is heterogeneously dense (BI-RADS composition c). There is a new cluster of fine pleomorphic microcalcifications in the upper inner quadrant at 10 o'clock, 6 cm from the nipple, spanning an area of approximately 1 cm. No associated mass, architectural distortion or skin thickening.
IMPRESSION:
BI-RADS 0: Incomplete. Need additional imaging evaluation of the new cluster of microcalcifications in the left breast at 10:00, 6 cm from the nipple. Spot compression magnification views recommended for further characterization. 
ADDENDUM:
Magnification views of the left breast at the site of microcalcifications were performed. The microcalcifications are better characterized as a cluster of fine linear and branching calcifications, highly suspicious for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended for definitive diagnosis.
FINAL ASSESSMENT: BI-RADS 4C: Suspicious abnormality, biopsy should be considered.
OVERALL ASSESSMENT:
Right breast: BI-RADS 2, Benign. 
Left breast: BI-RADS 4C, Suspicious abnormality. ➡ *Positive*
- REPORT:"NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 2 years: Yes.
The breast tissue is of scattered fibroglandular density. There are no dominant masses, suspicious calcifications or areas of architectural distortion. 
BIRADS-2.
" ➡ *Not Stated*
- BILATERAL MAMMOGRAM AND ULTRASOUND.
HISTORY: F/u from screening mammogram at which time pt had called back for additional views. Reported  area of new focal density. Prior benign biopsies on right.
FINDINGS:
Right breast tissue contains scattered fibroglandular densities. There is no focal mass, suspicious calcification, or architectural distortion. Stable benign-appearing calcifications in the upper outer quadrant from prior biopsy. There is no significant change from previous.
Left breast tissue is heterogeneously dense, which could obscure detection of small masses. There is no focal mass, architectural distortion, or suspicious calcifications. Specifically, no abnormality is seen in the area of palpable concern in the upper outer left breast. There is no significant change from previous.
Bilateral breast ultrasound was performed. There are no suspicious solid or cystic masses. No suspicious calcifications are identified.
IMPRESSION: 
1. No mammographic or sonographic evidence of malignancy. 
2. Stable benign right breast calcifications. 
3. Heterogeneously dense breast tissue.
BI-RADS 2: Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- SCREENING MAMMOGRAM
CLINICAL INDICATION: Screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available.
FINDINGS:
The breast is predominantly fatty.
There is a 1 cm cluster of pleomorphic microcalcifications in the upper outer right breast at 10:00, 6 cm from the nipple. No other suspicious microcalcifications are seen. No spiculated masses, architectural distortions or other findings suspicious for malignancy are identified.
Benign-appearing calcifications are noted bilaterally, likely representing vascular calcifications and fibrocystic changes.
IMPRESSION:
BI-RADS 4: Suspicious abnormality in the right breast. Stereotactic biopsy is recommended for the cluster of microcalcifications at 10:00 in the right breast.
ADDENDUM:
After discussion with the referring clinician and the patient, the patient has opted to proceed with stereotactic biopsy of the right breast microcalcifications. Biopsy has been scheduled for next week. ➡ *Positive*
- MAMMOGRAM, LEFT.
HISTORY: Routine screening mammogram. No current breast complaints.
FINDINGS:
Left breast tissue is predominantly fatty replaced. No dominant mass, suspicious calcifications, or concerning asymmetries. 
Stable, benign-appearing calcifications are seen in the upper outer quadrant, likely vascular in nature. These are unchanged from prior exams. No new calcifications are identified. 
The skin appears normal in thickness. No significant axillary lymphadenopathy.
IMPRESSION: 
Left breast, BIRADS 2 - Benign finding. Recommend continued routine screening in 1 year.
ADDENDUM: Comparison was made to outside screening mammograms from 1 and 2 years prior, which were retrieved and show no significant interval change. ➡ *Negative*
- FINDINGS: Bilateral mammographic views demonstrate scattered fibroglandular density breast tissue. There are no dominant masses, architectural distortion, or suspicious microcalcifications in either breast.

Focused ultrasound was performed in the area of palpable concern, 10 o'clock position 3 cm from the nipple in the right breast. No sonographic correlate was identified. The palpable finding likely represents normal fibroglandular tissue.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in either breast. Recommend continued annual screening mammography in 12 months.
2. Palpable finding right breast 10 o'clock position, no sonographic correlate. Likely normal fibroglandular tissue. Clinical follow up recommended as needed.

BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS

HISTORY: 47 year old female with palpable lump in left breast.

FINDINGS:
The breasts are heterogeneously dense. No previous mammograms available for comparison.

There is a 1.5 cm irregular mass at the 3 o'clock position of the left breast, 3 cm from the nipple. The margins are spiculated. 

In the upper outer quadrant of the left breast, there is a group of pleomorphic microcalcifications spanning an area of about 1 cm. 

No suspicious masses, calcifications or architectural distortion in the right breast. Benign appearing lymph nodes bilaterally.

IMPRESSION: 
Left breast: 
- 1.5 cm spiculated mass at 3:00, 3 cm from nipple, suspicious for malignancy. BI-RADS 4C. 
- Cluster of suspicious microcalcifications in the upper outer quadrant, spanning 1 cm. BI-RADS 4B.
Recommend biopsy of both findings.

Right breast: No suspicious findings. BI-RADS 1. 

ADDENDUM:
Ultrasound guided core biopsy of the left breast mass and stereotactic biopsy of the left breast microcalcifications both show invasive ductal carcinoma, grade 2. 
BI-RADS 6 (Known biopsy-proven malignancy). ➡ *Positive*
- BREAST MRI.
Clinical History: Right breast palpable abnormality.
Comparison: Outside mammogram.
Technique: Magnetic resonance imaging of both breasts was performed before and after the intravenous administration of gadolinium contrast.
Findings:
There is heterogeneous enhancement of the breast parenchyma bilaterally, in keeping with the patient's physiologic status.
There is no abnormal enhancement to suggest malignancy within either breast. No suspicious masses or non-mass enhancement is identified to explain the palpable abnormality. There is no axillary, internal mammary or supraclavicular lymphadenopathy.
IMPRESSION:
Negative MRI of the breasts. ➡ *Negative*
- OBSP SCREENING MAMMOGRAPHY
COMPARISON:  None.
TECHNIQUE:  Standard CC and MLO views of both breasts.
FINDINGS:  The breasts are heterogeneously dense.  No dominant masses, architectural distortion or suspicious calcifications are identified.  
IMPRESSION:  
BIRADS 2:  Benign findings.  Routine screening recommended.
Breast density >= 75%: No.  
Breast density < 75%: Yes.
The client is recommended to return for routine screening in 2 years as per OBSP guidelines.

CONFIDENCE ASSESSMENT:
The report structure and content is consistent with standard radiology reporting practices. I am confident that the report does not make any explicit statements about microcalcifications, and that the overall findings and impression align with a "Not Stated" label for microcalcifications.  The breast density assessment and screening recommendation are clearly stated and consistent with the overall report. ➡ *Not Stated*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.  
OBSP to recall client in 2 years: Yes.
The mammogram shows no suspicious masses, architectural distortion or focal asymmetries. The breasts are heterogeneously dense, which may obscure small masses.
BIRADS-2. ➡ *Not Stated*
- SCREENING BILATERAL MAMMOGRAM.
Compared to Previous: None.
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
OBSP to recall client in 1 year: Yes.
The breast tissue is extremely dense, which lowers the sensitivity of mammography. 
There are no dominant masses, architectural distortions, or suspicious calcifications.
BIRADS-1. ➡ *Not Stated*
- SCREENING BILATERAL DIGITAL MAMMOGRAPHY
INDICATION: Screening mammogram
COMPARISON: None
FINDINGS:
Both breasts are almost entirely fatty.
There are no significant masses, architectural distortion, or suspicious calcifications in either breast.
At the 1:00 position of the left breast, approximately 6 cm from the nipple, there is a group of amorphous microcalcifications in a segmental distribution, spanning an area of about 1 cm. This is a new finding compared to the patient's prior mammograms.
The skin is normal. No enlarged axillary lymph nodes are seen.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for the group of amorphous microcalcifications at 1:00, 6 cm from the nipple in the left breast.
BI-RADS Category 2: Benign finding in the right breast.
ADDENDUM: The patient returned for stereotactic biopsy of the group of microcalcifications in the left breast at the 1:00 position, 6 cm from the nipple. Specimen radiography confirmed calcifications within the cores. Final pathology revealed sclerosing adenosis with associated microcalcifications, negative for malignancy. ➡ *Positive*
- INDICATION: Screening mammogram.

PATIENT HISTORY: 45-year-old female with family history of breast cancer in mother diagnosed at age 60.

TECHNIQUE: Standard CC and MLO views of both breasts as well as spot compression views of the right breast.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

Right breast: There is an irregular mass measuring approximately 1.2 cm in the upper outer quadrant posteriorly on the CC and MLO views, best seen on the spot compression magnification views. No definite associated microcalcifications. The mass is new compared to the prior mammogram from 2 years ago.

Left breast: There are no masses, architectural distortion, or suspicious calcifications.

There are no axillary, supraclavicular, or internal mammary adenopathy.

IMPRESSION:
BI-RADS Category 4: Suspicious right breast mass in the upper outer quadrant, not present on prior exam. Biopsy is recommended for further evaluation.
BIRADS 1: Left breast negative.

RECOMMENDATIONS: Ultrasound guided biopsy of the right breast mass. 6 month follow up diagnostic mammogram is recommended for the right breast, and routine annual screening for the left breast. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAM.

INDICATION: Screening.

TECHNIQUE: Standard CC and MLO views of each breast.

COMPARISON: Outside screening mammogram from December 2021.

FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 

There are no suspicious masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
BI-RADS Category 1: Negative.
Routine screening in 1 year is recommended. ➡ *Not Stated*
- LEFT DIAGNOSTIC MAMMOGRAM
Clinical history: 75 y.o. female presents today for a diagnostic mammogram due to a palpable finding in the left breast.
Comparison: Prior mammograms from (outside institution) 06/07/2023
Technique: Standard CC and MLO views were obtained of the left breast. In addition, spot compression and magnification views were performed to better evaluate the area in question.
Findings:
The breasts are heterogeneously dense which may reduce the sensitivity of mammography.
Left breast:
Palpable finding corresponds to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Left breast, palpable finding, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening. 

COMPARISON: None.

FINDINGS:
Both breasts show scattered fibroglandular densities. No significant masses, architectural distortion, or suspicious calcifications are seen in the right breast.

In the left breast, there is a new cluster of pleomorphic microcalcifications in the upper outer quadrant, measuring approximately 8 mm in extent. No associated mass, architectural distortion or skin changes are identified.

IMPRESSION:
RIGHT BREAST: NEGATIVE. BI-RADS 1.
LEFT BREAST: Cluster of pleomorphic microcalcifications in the upper outer left breast, suspicious for malignancy. BI-RADS 4B. 

RECOMMENDATION:
Stereotactic core biopsy of the microcalcifications in the upper outer left breast is recommended for further evaluation. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Annual screening exam.

FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

Magnification views demonstrate scattered benign appearing calcifications in both breasts, including a small cluster of indeterminate microcalcifications in the upper outer quadrant of the left breast.

IMPRESSION:
1. Heterogeneously dense breasts.
2. Small cluster of indeterminate microcalcifications in the upper outer quadrant of the left breast. Additional magnification views are recommended for further evaluation. BI-RADS 0: Incomplete. 
3. Scattered benign appearing calcifications bilaterally, most likely vascular in etiology. BI-RADS 2: Benign finding.

ADDENDUM:
Additional magnification views of the left breast demonstrate a small cluster of indeterminate microcalcifications in the upper outer quadrant, unchanged compared to prior exams. Stereotactic biopsy is recommended for further evaluation. BI-RADS 4: Suspicious abnormality. ➡ *Positive*
- ULTRASOUND LEFT BREAST:
Indication: Palpable lump left upper outer quadrant. No known family history of breast cancer. 
Technique: Sonographic evaluation of the left breast was performed with a high frequency linear transducer.
Findings:
The left breast is heterogeneously dense.  
At the site of palpable abnormality in the 2:00 position 8 cm from the nipple, there is a 12 mm irregular solid mass with indistinct margins and internal vascularity. This corresponds to the patient's palpable finding.  
Immediately adjacent to this mass, there is a 6 mm cluster of pleomorphic microcalcifications.
The remainder of the left breast was scanned. No other suspicious masses or calcifications are identified. No suspicious left axillary lymphadenopathy.
Impression: 
1. Suspicious irregular solid mass left breast at site of palpable abnormality. Biopsy recommended. 
2. Cluster of pleomorphic microcalcifications adjacent to the mass, also suspicious. Stereotactic biopsy recommended if not removed with mass biopsy.
BI-RADS 4C. Moderate suspicion for malignancy. ➡ *Positive*
- ULTRASOUND AND MAMMOGRAM OF RIGHT BREAST 

CLINICAL HISTORY: Patient noted palpable lump in right breast, upper outer quadrant. Negative family history for breast cancer.

COMPARISON: Comparison made to outside mammogram from 1 year ago.

FINDINGS:
RIGHT BREAST:
The breast is heterogeneously dense. No suspicious mass, architectural distortion or suspicious calcifications. 

Targeted ultrasound was performed at the site of the palpable abnormality in the right upper outer quadrant at 10 o'clock, 8 cm from the nipple. This corresponds to a 0.8 x 0.5 x 0.5 cm oval circumscribed hypoechoic solid mass, parallel orientation. This likely represents a fibroadenoma. 

LEFT BREAST:
The breast tissue is heterogeneously dense. No suspicious mass, architectural distortion or calcifications.  

IMPRESSION:

1. Likely benign right breast mass at site of palpable abnormality, BI-RADS 3. Short term follow up is recommended to confirm stability. 

2. No suspicious mammographic findings bilaterally. No microcalcifications. Recommend continued annual screening mammography. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM
Clinical History: 65 y.o. female presents for a problem focused examination due to palpable findings in the right breast.
Comparison: Prior mammogram from (outside institution), not available for review.
Technique: Standard CC & MLO views were obtained of each breast. In addition, spot compression and magnification views were performed to better evaluate the area in question.
Findings:
The breasts are heterogeneously dense, which may reduce the sensitivity of mammography.
Right breast:
Palpable findings in the right breast corresponding to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
Left breast:
Scattered fibroglandular densities. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Right breast, palpable findings, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended.
Left breast, negative. BI-RADS 1. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS AND TOMOSYNTHESIS
CLINICAL HISTORY: 60 year old female with family history of breast cancer
COMPARISON: Outside previous films from 2017
FINDINGS:
The breasts are heterogeneously dense.
In the right breast at 10:00 3 cm from nipple there is a benign appearing 9 mm circumscribed oval mass, likely a fibroadenoma. No other masses, suspicious calcifications, architectural distortion or abnormal asymmetries are seen in the right breast.
In the left breast, no discrete masses or architectural distortion are identified. At 2:00 8 cm from the nipple there is a new cluster of pleomorphic microcalcifications spanning approximately 6 mm. The distribution is segmental. No other suspicious calcifications are seen.
IMPRESSION:
RIGHT BREAST, BIRADS 2: Benign.
LEFT BREAST, BI-RADS 4B: Suspicious cluster of pleomorphic microcalcifications at 2:00, 8 cm from nipple. Stereotactic biopsy is recommended for further evaluation. Overall assessment is suspicious.
ADDENDUM: The patient returned for stereotactic guided left breast biopsy of the microcalcifications at 2:00. Specimen radiograph confirms microcalcifications were obtained. Final pathology shows invasive ductal carcinoma with associated DCIS, cribriform subtype, intermediate nuclear grade. Clip was placed at the biopsy site. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Annual screening.

TECHNIQUE: Bilateral full field digital mammogram performed including CC and MLO views of each breast. 

FINDINGS:

Right Breast:
The right breast is almost entirely fatty with scattered areas of fibroglandular density. Overall, the breast is heterogeneously dense (BI-RADS density category C). There are no dominant masses, architectural distortions or suspicious calcifications.

Left Breast: 
The left breast shows scattered fibroglandular densities in a fatty background. Similar to the right, the breast is heterogeneously dense (BI-RADS density category C). Within the upper outer quadrant of the left breast, there is a new cluster of fine pleomorphic microcalcifications at the 2 o'clock position, approximately 6 cm from the nipple. This cluster measures about 8 mm in greatest dimension. No associated mass, architectural distortion or skin thickening is seen. The remainder of the left breast is unremarkable.

Axillae: 
There are no enlarged axillary lymph nodes bilaterally.

Skin: 
The skin appears normal bilaterally with no evidence of skin thickening or retraction.

IMPRESSION:
1. New cluster of pleomorphic microcalcifications in the left breast at 2 o'clock, 6 cm from the nipple, measuring approximately 8 mm, suspicious for malignancy. Additional magnification views recommended for further evaluation. BI-RADS 4B.

2. No suspicious findings in the right breast. Continue routine annual screening. BI-RADS 1.

RECOMMENDATION:
Left breast diagnostic mammogram with magnification views of the area of microcalcifications, followed by stereotactic core biopsy if indicated based on additional imaging. 

ADDENDUM:
Magnification views of the left breast confirm a cluster of pleomorphic microcalcifications at 2 o'clock, 6 cm from the nipple, highly suspicious for ductal carcinoma in situ (DCIS). Stereotactic core biopsy recommended. Updated BI-RADS assessment of the left breast: 4C. ➡ *Positive*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

INDICATION: Routine screening mammogram.

TECHNIQUE: Screening full field digital mammography including standard CC and MLO views of both breasts.

COMPARISON: Comparison is made to prior mammogram from 07/12/2021. 

FINDINGS:
The breast tissue is heterogeneously dense, which may obscure detection of small masses. 

There are no dominant masses, architectural distortion, or suspicious calcifications.

Stable benign appearing calcifications are noted in the right breast, likely representing fibrocystic changes.

There are no new mammographic findings suspicious for malignancy.

IMPRESSION:
BI-RADS Category 2: Benign findings. Recommend routine screening in 1 year. ➡ *Not Stated*
- CLINICAL INDICATION: Annual screening mammogram.

TECHNIQUE: Hologic digital breast tomosynthesis.

COMPARISON: 04/15/2020.

FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

There are no dominant masses, suspicious calcifications, or areas of architectural distortion bilaterally.

Stable bilateral scattered fibroglandular densities.

No significant interval change.

IMPRESSION:
1. No mammographic evidence of malignancy.
2. Stable scattered fibroglandular densities bilaterally.

RECOMMENDATION: 
Continue annual screening mammography. 

ASSESSMENT CATEGORY: BI-RADS 2 - Benign finding.

ADDENDUM:
Comparison was made to the previous exam from 04/15/2020 upon receipt. No change in the final assessment. ➡ *Negative*
- HISTORY: Annual screening mammogram. No current breast complaints.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right breast:
There are no dominant masses, focal asymmetries, or areas of architectural distortion. No suspicious calcifications are identified.

Left breast:
There are no dominant masses, focal asymmetries, or areas of architectural distortion. No suspicious calcifications are identified.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Breast density is heterogeneously dense, which may lower the sensitivity of mammography. Recommend continued annual screening mammography and consideration of supplemental screening with breast ultrasound or breast MRI as clinically appropriate.

ASSESSMENT CATEGORIES:
Right breast: BI-RADS Category 1 - Negative.
Left breast: BI-RADS Category 1 - Negative.
Overall: BI-RADS Category 1 - Negative. ➡ *Negative*
- FINDINGS:
Bilateral diagnostic mammogram demonstrates heterogeneous dense parenchyma. There is no suspicious mass, architectural distortion, or asymmetry in either breast.

In the left breast at the 9:00 position, 8 cm from the nipple, there is an 8 mm cluster of fine pleomorphic microcalcifications. Additional magnification views in the mediolateral and craniocaudal projections confirm the presence of the microcalcifications. No other suspicious microcalcifications are identified in either breast.

Spot compression views of the left breast at 9:00 were performed, which demonstrate the cluster of microcalcifications more clearly. The microcalcifications are indeterminate in morphology and distribution, and further evaluation with stereotactic biopsy is recommended.

The skin is normal in thickness and appearance. No suspicious skin lesions are identified. The nipples are in profile and are normal in appearance. No suspicious calcifications are identified in the right breast.

IMPRESSION:
1. 8 mm cluster of indeterminate microcalcifications in the left breast at 9:00, 8 cm from the nipple. Stereotactic biopsy is recommended for further evaluation. BI-RADS 4.
2. Heterogeneously dense breasts, otherwise negative right breast. BI-RADS 1. ➡ *Positive*
- SCREENING MAMMOGRAM.
INDICATION: Routine screening.  
TECHNIQUE: Screening mammography. Tomosynthesis performed. 
COMPARISON: Prior exams from [**2022-12-15**] at [**Hospital1 **].  
FINDINGS:
The breast tissue is heterogeneously dense, which may obscure small masses.
There are no dominant masses, architectural distortion or suspicious calcifications.
There are scattered areas of fibroglandular density bilaterally.
IMPRESSION:
1. No mammographic evidence of malignancy. Scattered areas of fibroglandular density bilaterally.
2. Mammography is limited due to heterogeneously dense breast tissue. Consider supplemental screening with breast MRI or whole-breast ultrasound in addition to mammography.
BIRADS Category 1: Negative.
Recommend routine screening in 1 year.
These results were discussed with the patient. She voices understanding and will follow-up as recommended. 
ADDENDUM:
Screening automated breast ultrasound (ABUS) performed today. No suspicious findings identified. This provides supplemental screening in addition to mammography, given the patient's dense breast tissue. 
continue routine annual screening with mammography and supplemental imaging as clinically indicated. ➡ *Not Stated*
- LEFT SCREENING MAMMOGRAM
Clinical History: 42 year old female, screening mammogram
Comparison: None
Technique: Standard CC and MLO views of the breasts were obtained.
Findings:
Heterogeneously dense breast tissue bilaterally.
No suspicious masses, calcifications, or architectural distortions are seen.
IMPRESSION:
Negative screening mammogram. BI-RADS 1. ➡ *Negative*
- DIAGNOSTIC MAMMOGRAPHY LEFT BREAST

CLINICAL HISTORY: 56 year old woman with new palpable lump left breast upper outer quadrant.

COMPARISON: Screening mammogram from April 2021.

FINDINGS:
Left breast:
Heterogeneously dense parenchyma.
In the upper outer quadrant region there is an ill-defined 15 mm mass, seen on both CC and MLO views. Spot compression magnification views show that this corresponds to an area of architectural distortion. 
Additionally, spot magnification views show several new clusters of pleomorphic microcalcifications spanning an area of approximately 3 cm in the vicinity of the mass. The calcifications have a segmental distribution.
No other masses, calcifications or areas of architectural distortion.

Right breast:
Heterogeneously dense parenchyma.
No dominant masses, suspicious calcifications or areas of architectural distortion.
Stable benign appearing calcifications in the upper outer quadrant.

IMPRESSION:
Suspicious findings left breast. New palpable mass in the upper outer quadrant corresponds mammographically to an area of architectural distortion with associated new segmentally distributed pleomorphic microcalcifications. Suspicious for malignancy. Biopsy recommended. BI-RADS 4C.

Benign right breast. BI-RADS 2.

ADDENDUM: Ultrasound guided core biopsy of the left breast 2:00 region 6 cm from the nipple performed. Pathology result: Invasive ductal carcinoma, grade 2, associated with high grade DCIS. ➡ *Positive*
- SCREENING MAMMOGRAM.
Digital mammography, bilateral.
Indication: Screening.
Compared to Previous: Yes.
FINDINGS:
The breasts are almost entirely fat with scattered fibroglandular densities. There are no dominant masses, suspicious microcalcifications, or concerning asymmetries. No significant changes since the prior examination.
IMPRESSION:
Negative bilateral screening mammogram. 
BIRADS Category 1, Normal.
Recommend continued annual screening mammography in 12 months. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
INDICATION: Annual screening mammogram.
COMPARISON: 12/5/2020 screening mammogram
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion or suspicious calcifications in either breast.
There are scattered benign-appearing calcifications in both breasts, slightly increased in the medial left breast.
IMPRESSION:
BI-RADS Category 2: Benign
BI-RADS Density Grade C
RECOMMENDATION: Annual screening mammogram is recommended. Patient is advised to follow up for further imaging if a palpable abnormality develops.
ADDENDUM 12/7/2022
Upon further review of images, there is a new cluster of indeterminate microcalcifications in the upper outer left breast at the 2:00 position, 6 cm from the nipple, measuring approximately 8 mm in greatest dimension. The cluster is new compared to the prior study. Additional magnification views are recommended for further evaluation. BI-RADS category changed to 0: Incomplete, need additional imaging evaluation. ➡ *Positive*
- LEFT BREAST MAMMOGRAM AND ULTRASOUND
CLINICAL HISTORY: 45-year-old woman presents for screening mammogram. No palpable lumps or breast complaints. 
COMPARISON: Compared to prior study from 11/15/2021.
FINDINGS:
The breast tissue is heterogeneously dense. There are no dominant masses, architectural distortion, or suspicious calcifications. 
Targeted ultrasound of the left breast upper outer quadrant was performed. No suspicious solid or cystic lesions are identified in the area of concern. 
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening in 12 months. 
2. Targeted left breast ultrasound is negative.
BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Routine screening mammogram. No palpable abnormalities.

TECHNIQUE: Standard CC and MLO views of each breast.

COMPARISON: None.

FINDINGS:
Both breasts are of scattered fibroglandular density (BI-RADS b). No focal asymmetry, significant architectural distortion or skin thickening.

Right breast:
No dominant masses, suspicious calcifications or other findings.

Left breast:
At the 9 o'clock position 6 cm from the nipple, there is a small cluster of amorphous microcalcifications measuring about 6 mm in greatest dimension. This is a new finding compared to the patient's previous mammograms. No other significant abnormalities in the left breast.

IMPRESSION:
Scattered fibroglandular densities in both breasts, not limiting mammographic sensitivity.
No suspicious findings in the right breast. BI-RADS 1.

New cluster of amorphous microcalcifications in the left breast as described above. Suspicious abnormality, BI-RADS 4. Stereotactic biopsy is recommended for further evaluation.

ADDENDUM AFTER STEREOTACTIC BIOPSY OF LEFT BREAST:
Stereotactic biopsy of the microcalcifications in the left breast at 9 o'clock was performed using 9G vacuum assisted device. Specimen radiograph confirmed the presence of calcifications. Pathology result: Benign findings - Fibrocystic changes with focal fibroadenomatoid hyperplasia and microcalcifications.

Final assessment for the left breast: Benign finding, BI-RADS 2. Routine screening in 1 year is recommended. ➡ *Positive*
- MAMMOGRAM
INDICATION: Annual screening. No palpable complaints.
TECHNIQUE: Standard CC and MLO views of each breast.
COMPARISON: Prior study of 5/22/2021.
FINDINGS:
Breast composition is scattered fibroglandular density (C). The amount of dense tissue could obscure detection of small masses.
No significant masses, calcifications, or other findings. Stable appearance compared to prior mammograms.
IMPRESSION:
BI-RADS Category 1: Negative. Recommend continued annual screening with mammography. ➡ *Negative*
- FINDINGS: The breasts are heterogeneously dense. There are no dominant masses, suspicious calcifications, or areas of architectural distortion. The breasts are symmetric. There are no new findings compared to prior exams.

Right breast ultrasound was performed targeted to the area of palpable concern at 10 o'clock. The breast tissue in this region appears normal, containing scattered fibroglandular tissue. No sonographic mass, cyst, or other abnormality is identified. 

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend routine screening in 1 year.
2. No sonographic correlate for the palpable area of concern in the right breast at 10 o'clock. 

ASSESSMENT: BI-RADS 1 - Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM WITH MAGNIFICATION VIEWS.

History: 47 year old female with breast discomfort.

FINDINGS:
The breasts are fibroglandular composed of a mixture of fat and dense fibroglandular tissue.

There are no suspicious masses, calcifications, asymmetries or architectural distortion in the right breast. No skin thickening or nipple retraction.

In the left breast there are indeterminate amorphous calcifications grouped in a roughly linear distribution measuring 14 x 5 mm at the 12:00 position, 6 cm from the nipple.
 
No suspicious masses, asymmetries or architectural distortion are seen.

Magnification views confirm the presence of the calcifications in the left breast.

IMPRESSION:

BIRADS 0. Indeterminate calcifications left breast. Spot compression magnification views are recommended to further evaluate these calcifications. If the calcifications remain of concern biopsy should be considered.

No dominant mass or other suspicious findings are seen in either breast to correspond to the discomfort. Clinical correlation is suggested.

ADDENDUM:
Magnification spot compression views show the amorphous microcalcifications at the 12:00 position of the left breast to be grouped in a linear distribution spanning 15 mm. These are new compared to the prior examination. Stereotactic biopsy is recommended.

AMENDED IMPRESSION:
BIRADS 4B. Suspicious new grouped amorphous calcifications left breast. Stereotactic biopsy is recommended. ➡ *Positive*
- DIAGNOSTIC BILATERAL DIGITAL MAMMOGRAPHY AND TARGETED ULTRASOUND LEFT BREAST
INDICATION: Left breast lump.
COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are scattered areas of fibroglandular density bilaterally.

At the 6 o'clock position of the left breast, 4 cm from the nipple, there is an irregular hypoechoic mass with indistinct margins and posterior acoustic shadowing, measuring approximately 12 x 8 x 10 mm. Color Doppler demonstrates internal vascularity. This is highly suspicious for malignancy. 

No suspicious right breast masses, architectural distortion, or skin thickening.

There is a 5 mm cluster of amorphous microcalcifications in the left breast at 3 o'clock, 6 cm from the nipple. Additional magnification views confirm these are not vascular calcifications. 

IMPRESSION: 
1. Highly suspicious irregular mass left breast 6 o'clock position, 4 cm from the nipple. Ultrasound-guided core biopsy is recommended. BIRADS 5.

2. 5 mm cluster of amorphous microcalcifications left breast at 3 o'clock, 6 cm from the nipple. Stereotactic-guided biopsy is recommended. BI-RADS 4.

3. Heterogeneously dense breast tissue. Consider supplemental screening with breast MRI in addition to mammography for optimal cancer detection. BI-RADS 1 right breast, BI-RADS 4 and 5 left breast.

ADDENDUM: 
Stereotactic-guided biopsy of the amorphous calcifications in the left breast at 3 o'clock was performed using an 8-gauge vacuum-assisted device. Specimen radiograph confirmed calcifications. Final pathology revealed atypical ductal hyperplasia with microcalcifications. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

FINDINGS:

There are scattered areas of fibroglandular density. There are no dominant masses, areas of architectural distortion, or suspicious calcifications in either breast. 

In the right breast at the 10:00 position 6 cm from the nipple, there is a group of amorphous microcalcifications that measures approximately 10 mm. Additional magnification views confirm these calcifications.

The left breast is unremarkable. 

IMPRESSION: 
Right breast: Group of amorphous microcalcifications in the right breast at 10:00, 6 cm from the nipple. Additional diagnostic imaging recommended. BI-RADS 0.

Left breast: Negative. BI-RADS 1. 

RECOMMENDATION:
Right diagnostic mammogram with spot compression magnification views of the right breast at 10:00.

ADDENDUM - RIGHT DIAGNOSTIC MAMMOGRAM WITH SPOT COMPRESSION MAGNIFICATION VIEWS:

The group of amorphous microcalcifications in the right breast at 10:00 persists and measures approximately 10 mm. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS). 

FINAL IMPRESSION:
Suspicious group of amorphous microcalcifications right breast at 10:00. Stereotactic biopsy recommended. BI-RADS 4C. ➡ *Positive*
- SCREENING MAMMOGRAM 

INDICATION: Screening.

COMPARISON: None available.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, suspicious calcifications, or other findings suspicious for malignancy.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend routine screening in 12 months.

BIRADS CATEGORY: 1 

Breast density: C. The breasts are heterogeneously dense, which may obscure small masses. ➡ *Not Stated*
- FINDINGS:
The right breast is status post mastectomy with expected postoperative changes. The left breast is heterogeneously dense with scattered fibroglandular tissue. No dominant mass, architectural distortion, or microcalcifications are identified. Stable intramammary lymph nodes are noted. There are no new or enlarging masses or areas of architectural distortion. No skin changes or nipple retraction.

Bilateral axillae appear clear. Stable surgical clips overlie the right chest wall.

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Annual screening mammography is recommended.
2. Stable postmastectomy changes of the right breast. 

BI-RADS category 1: Negative. ➡ *Negative*
- MAMMOGRAM - BILATERAL.
CLINICAL HISTORY: Annual screening mammogram.
FINDINGS:
Right breast: The breast tissue is composed of scattered fibroglandular densities. No dominant masses, suspicious calcifications, or areas of focal asymmetry.
Left breast: The breast tissue is composed of scattered fibroglandular densities. No dominant masses, suspicious calcifications, or areas of focal asymmetry.
IMPRESSION: 
Negative bilateral screening mammogram. 
BIRADS 1 - Negative. Routine mammographic follow up is recommended in 1 year. ➡ *Negative*
- TOMOSYNTHESIS MAMMOGRAPHY SCREENING AS PART OF TMIST CLINICAL TRIAL.
The patient is enrolled in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) and was randomized to the FFDM only arm. Consent was obtained as per REB approved protocol. 
Comparison to prior exam: Available.
Mammographic breast density: Scattered fibroglandular densities.
Breast composition is less than 75% glandular.
Annual screening recommended per OBSP guidelines: Yes.
There are no mammographic abnormalities suspicious for malignancy. 
Final BIRADS Assessment Category: 2 - Benign findings. ➡ *Not Stated*
- MAMMOGRAM LEFT BREAST.
HISTORY: Screening
COMPARISONS: 12/13/2022
TECHNIQUE: Standard CC and MLO views of the left breast were obtained
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- MAMMOGRAPHY, BREAST ULTRASOUND - BILATERAL

INDICATION: Left breast lump.

TECHNIQUE: Bilateral digital 2D mammographic views - craniocaudal and mediolateral oblique. Bilateral breast and axillary ultrasound using high resolution linear transducer.

COMPARISON: None.

FINDINGS:
The breasts show scattered fibroglandular densities.

There are no dominant masses, architectural distortion or suspicious calcifications in the right breast. Ultrasound is unremarkable.

In the left breast at 9 o'clock position 5 cm from the nipple there is an irregular spiculated mass measuring approximately 12 x 10 mm. Sonographically this corresponds to an irregular hypoechoic mass with shadowing and internal vascularity. 

Additionally, there are several clusters of fine pleomorphic microcalcifications in the upper outer left breast spanning an area of about 4 cm. The most prominent cluster measures 8 mm and is located at 2 o'clock position 7 cm from the nipple. Sonographically no corresponding mass is seen at this site.

Left axillary ultrasound shows two abnormal appearing lymph nodes at level I, the largest measuring 15 mm in short axis with diffuse cortical thickening.

IMPRESSION:
1. Suspicious mass left breast 9 o'clock position, BI-RADS 5. Recommend biopsy. 
2. Clusters of suspicious microcalcifications upper outer left breast, BI-RADS 4C. Stereotactic biopsy recommended.
3. Abnormal left axillary lymph nodes, biopsy recommended for staging if primary breast malignancy confirmed.
4. Benign right breast, BI-RADS 2. ➡ *Positive*
- FINDINGS:
Left mediolateral oblique and craniocaudal mammographic views demonstrate the breasts to be heterogeneously dense. This may lower the sensitivity of mammography. No dominant masses, areas of architectural distortion or suspicious calcifications are identified.

Targeted left breast ultrasound was performed at the site of clinical concern, 5 o'clock position 3 cm from the nipple. No discrete solid or cystic mass is identified. Normal fibroglandular tissue is present.

IMPRESSION:
No mammographic or sonographic evidence of malignancy. Heterogeneously dense breast tissue.
BI-RADS Category 1: Negative.

ADDENDUM:
Follow-up imaging is recommended according to standard screening guidelines based on the patient's age and risk factors. Continued monthly self breast exams are important, with prompt reporting of any new palpable lump, skin/nipple changes or other concerns, to the referring clinician. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL INDICATION: Routine screening.
COMPARISON: None.
FINDINGS:
Both breasts are heterogeneously dense which may obscure detection of small masses.
Right breast: There are no dominant masses, areas of architectural distortion or suspicious calcifications.
Left breast: No dominant masses or areas of architectural distortion. There is a small group of amorphous calcifications in the upper outer quadrant measuring approximately 6 mm in diameter. These are new since the patient's last mammogram.
IMPRESSION:
BI-RADS 0: Left breast: Small group of amorphous calcifications in the upper outer quadrant, indeterminate. Additional magnification views recommended for further evaluation. Appointment to be scheduled.
BI-RADS 1: Right breast: Negative.
ADDENDUM:
The patient returned for left diagnostic mammogram with magnification views of the group of calcifications in the upper outer quadrant. The calcifications are irregular in shape and distribution, suspicious for ductal carcinoma in situ (DCIS). Stereotactic core biopsy is recommended. BI-RADS 4: Suspicious. ➡ *Positive*
- SCREENING MAMMOGRAM

INDICATION: 
Annual screening mammogram.

TECHNIQUE:  
Bilateral digital mammography, including two standard CC and MLO views.

COMPARISON:  
Comparison is made to prior studies dated 2/17/2018.

FINDINGS:

Right breast:
Breast composition is heterogeneously dense, which may obscure small masses. No dominant mass, architectural distortion or suspicious calcifications.

Left breast:  
Breast composition is heterogeneously dense, which may obscure small masses. No dominant mass, architectural distortion or suspicious calcifications.

IMPRESSION:  

RIGHT BREAST: No mammographic evidence of malignancy. BI-RADS Category 1 - Negative.

LEFT BREAST: No mammographic evidence of malignancy. BI-RADS Category 1 - Negative.

OVERALL BI-RADS ASSESSMENT: BI-RADS Category 1 - Negative. Recommend routine screening in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Screening mammogram.

COMPARISON: None.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications. No microcalcifications are seen in either breast.

IMPRESSION:
Negative screening mammogram. No mammographic evidence of malignancy. 

BIRADS Category 1 - Negative. Routine screening is recommended. ➡ *Not Stated*
- MAMMOGRAPHIC REPORT:
CLINICAL HISTORY: Annual screening mammogram.
COMPARISON: 9/14/2022, 9/26/2021
FINDINGS:
Bilateral tissue contains scattered areas of fibroglandular density. There are no masses, architectural distortion, or suspicious calcifications. Stable post-surgical changes noted in the right breast from prior lumpectomy. No abnormality in the left breast. Stable benign calcifications in both breasts. 
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Stable post-surgical changes right breast.
3. Stable scattered benign calcifications bilaterally. 
BIRADS 2 - Benign findings. Recommend continued annual screening with mammography.

ADDENDUM: Comparison to prior ultrasound 12/15/2022 of left breast 4:00 location performed for palpable abnormality. Previously identified sonographic abnormality in the area of palpable concern is no longer seen on mammogram. No suspicious lesions are identified in this region on mammography. ➡ *Negative*
- SCREENING MAMMOGRAM

INDICATION: Screening.

COMPARISON: 05/15/2020.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.
No suspicious masses, architectural distortion, or suspicious calcifications are seen.
Stable bilateral scattered fibroglandular densities.

IMPRESSION:
No mammographic evidence of malignancy. 
BI-RADS Category 2: Benign.

1-year routine screening mammogram is recommended. ➡ *Not Stated*
- BILATERAL MAMMOGRAM.
HISTORY: Screening mammogram. No current breast complaints.
Comparison made to prior imaging.
FINDINGS: There are scattered fibroglandular densities in both breasts. There are no suspicious masses, architectural distortions, or skin thickening. No sonographic correlate is seen in either breast. 
IMPRESSION: 
1. No mammographic evidence of malignancy. 
2. Scattered fibroglandular densities. Recommend annual screening mammography.
BIRADS 1. ➡ *Negative*
- TMIST FFDM STUDY:
This patient is participating in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), randomly assigned to the full field digital mammography arm. Informed consent was obtained as per Research Ethics Board approval.

COMPARISON: Prior studies available.

BREAST DENSITY: The breasts are heterogeneously dense, which may obscure detection of small masses. Breast density is less than 75%.

FINDINGS:
The breast parenchyma is stable compared to prior studies. No dominant masses, architectural distortion, or suspicious calcifications are identified.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Routine annual OBSP screening recommended based on the patient's risk factors.

BIRADS CATEGORY: 2 - Benign findings. Routine screening recommended. ➡ *Not Stated*
- MAMMOGRAPHY REPORT
PROCEDURE: BILATERAL DIGITAL SCREENING MAMMOGRAM WITH TOMOSYNTHESIS 

HISTORY: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Tomosynthesis was performed. 

COMPARISON: None available.

FINDINGS:  
Right Breast:
The right breast demonstrates scattered areas of fibroglandular density. No dominant mass, architectural distortion, or suspicious calcifications are identified. There are no significant changes compared to prior examination.

Left Breast: 
The left breast shows heterogeneous fibroglandular tissue. In the upper outer quadrant, there is a new cluster of fine pleomorphic microcalcifications spanning approximately 8 mm. No associated mass or architectural distortion is seen. The remainder of the left breast is unremarkable. 

IMPRESSION:
1. Left breast: New cluster of microcalcifications in the upper outer quadrant, suspicious for ductal carcinoma in situ (DCIS). BIRADS 4B. Stereotactic biopsy is recommended for further evaluation.  

2. Right breast: Negative. BIRADS 1. Annual screening is recommended. ➡ *Positive*
- SCREENING MAMMOGRAM.
INDICATION: Annual screening mammogram.

COMPARISON: Screening mammogram of April 2021. 

FINDINGS:
Both breasts are heterogeneously dense. There are scattered areas of fibroglandular density. There are no dominant masses, architectural distortion, or suspicious calcifications.

ADDENDUM: Magnification views were performed. On the magnification views of the right breast medial lower quadrant there is a new group of pleomorphic microcalcifications spanning a region of 1 cm at 4 o'clock position.

IMPRESSION:
New group of pleomorphic microcalcifications right breast medial lower quadrant at 4:00 as described above. BIRADS 4B: Suspicious abnormality. Biopsy should be considered in the absence of clinical contraindication. 

RECOMMENDATION:  Stereotactic biopsy is recommended for the group of pleomorphic microcalcifications in the right breast 4:00 position. An appointment can be made by calling the Breast Assessment Center. ➡ *Positive*
- MAMMOGRAPHY, RIGHT BREAST.
INDICATION: Screening. Patient reports no problems.
COMPARISON: Screening mammogram from August 16, 2021.
FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 
There are no masses, architectural distortion, or suspicious calcifications.
There are no significant changes compared to prior mammogram.  
IMPRESSION: 
Negative screening mammogram. 
BIRADS 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- ULTRASOUND GUIDED BIOPSY PROCEDURE LEFT BREAST
Clinical history: 51 year old female with new cluster of calcifications left breast.
Technique: Under ultrasound guidance using a 14 gauge needle an ultrasound guided core biopsy was performed at 3:00 left breast 4 cm from the nipple corresponding to the new cluster of calcifications on mammography. Specimen radiograph shows calcifications.
BILATERAL DIGITAL MAMMOGRAPHY WITH SPOT COMPRESSION MAG VIEWS LEFT BREAST:
Comparison: Outside mammograms from August 2021.
FINDINGS: The breasts are heterogeneously dense. Stable benign appearing calcifications are seen bilaterally.
There is a new cluster of fine pleomorphic calcifications measuring 6 mm at 3:00 left breast middle depth on the CC view. This was targeted on today's ultrasound guided biopsy.
The breasts are otherwise stable.
IMPRESSION:
New cluster of suspicious calcifications at 3:00 left breast middle depth with a 6 mm extent. Targeted on ultrasound and biopsied. BIRADS 4B. Specimen radiograph shows calcifications.
Remainder of bilateral mammogram is benign. BIRADS 2. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (BI-RADS category C). This may lower the sensitivity of mammography. 

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

At the 12:00 position of the right breast, approximately 6 cm from the nipple, there is a 5 mm cluster of amorphous microcalcifications. Additional magnification views were obtained which confirm the presence of these calcifications. 

The left breast is unremarkable.

IMPRESSION:
BI-RADS Category 0: Incomplete Assessment. 

5 mm cluster of amorphous microcalcifications in the right breast at 12:00, 6 cm from the nipple. Additional imaging evaluation with magnification views confirms these are suspicious and a biopsy is recommended for definitive characterization. 

ADDENDUM:
The patient returned for stereotactic core biopsy of the right breast calcifications. Specimen radiograph confirms that microcalcifications are present within the core samples obtained. Final pathology results are pending and will be reported in an addendum. ➡ *Positive*
- REPORT:
"BILATERAL SCREENING MAMMOGRAM; TOMOSYNTHESIS BILATERAL
HISTORY: 45 year old female, here for screening mammogram. No complaints. No personal or family history of breast cancer.
FINDINGS: The breasts are almost entirely fatty.
There are no mammographic findings of malignancy or suspicious abnormality in the right breast.
In the left breast, scattered fibroglandular densities are noted. In the left breast at the 11:00 position, 5 cm from the nipple there is an 8 mm cluster of pleomorphic microcalcifications. No spiculated masses, areas of architectural distortion or other suspicious findings.
Skin is normal in thickness bilaterally. No skin or nipple retraction. No suspicious axillary lymphadenopathy.

IMPRESSION:
RIGHT BREAST: No mammographic evidence of malignancy. BI-RADS 1.
LEFT BREAST: 8 mm cluster of pleomorphic calcifications in the 11:00 region of the left breast, suspicious for malignancy. Stereotactic biopsy recommended. BI-RADS 4B.

ADDENDUM: The patient was contacted and informed of the results. She agrees to return for stereotactic biopsy of the microcalcifications in the left breast. Appointment scheduled for next week."
LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- FFDM SCREENING BILATERAL MAMMOGRAPHY:
INDICATION: Routine screening mammogram.  
COMPARISON: Prior mammogram from 1 year ago.
FINDINGS:  
Breast tissue is heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, suspicious calcifications, or other findings that are concerning for malignancy in either breast. 
Stable post-surgical changes noted in the upper outer quadrant of the right breast, likely related to prior biopsy.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breast tissue. 
RECOMMENDATIONS:  
1. Routine screening mammography in 1 year per OBSP guidelines.
BIRADS Category: 2 - Benign finding(s). ➡ *Not Stated*
- LEFT MAMMOGRAM
HISTORY: Patient presents for routine screening mammogram. No clinical concerns.
FINDINGS: Left breast tissue is predominantly fatty replaced. No mammographic evidence of malignancy. No focal mass, architectural distortion, or suspicious calcifications. Stable benign calcifications in the left breast, likely vascular. No significant change from prior exam.
IMPRESSION:
1. Routine screening mammogram of the left breast shows no malignancy.
2. Benign vascular calcifications, stable.
BIRADS 2. Recommend routine screening in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: Prior exams from 2019

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

In the right breast at 10:00, 6 cm from the nipple, there is a new 8 mm cluster of pleomorphic microcalcifications, increasing in size compared to prior mammograms. Additional magnification views confirm the presence of the microcalcifications.

The left breast is unchanged with no suspicious findings.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic-guided biopsy is recommended for the new cluster of microcalcifications at 10:00 in the right breast.

BI-RADS Category 2: Benign findings in the left breast. Recommend continued annual screening.

ADDENDUM:
The patient returned for a stereotactic-guided biopsy of the right breast microcalcifications. Pathology results revealed ductal carcinoma in situ (DCIS), intermediate grade, with no evidence of invasion. The patient will be referred to a breast surgeon for further management. ➡ *Positive*
- FINDINGS:
Both breasts are composed of scattered fibroglandular densities. The breasts are symmetric. 
No dominant masses, architectural distortion or suspicious microcalcifications.
Stable benign calcifications are noted in the right breast.
No mammographic evidence of malignancy. 
IMPRESSION:
No mammographic evidence of malignancy. 
Continue routine screening mammography in one year.
BIRADS 1 - Negative. ➡ *Negative*
- SCREENING MAMMOGRAM.
Compared to previous screening mammogram from June 2021, there are no concerning changes.
The breasts are heterogeneously dense, which may obscure small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
IMPRESSION:
No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
BIRADS-2: Benign finding. ➡ *Not Stated*
- MAMMOGRAPHY, RIGHT BREAST.
CLINICAL INDICATION: Right breast pain. Negative personal or family history of breast cancer.
COMPARISON: None available.
TECHNIQUE: Standard CC and MLO views of the right breast were obtained.
FINDINGS:
The breast is composed of scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified.
The skin is normal in thickness. The nipple is within normal limits.
No axillary lymph nodes are seen.
IMPRESSION:
1. No mammographic evidence of malignancy in the right breast.
2. Negative for suspicious microcalcifications.
BIRADS Category 1: Negative. Recommend routine screening in 1 year. ➡ *Negative*
- SCREENING MAMMOGRAM.
CLINICAL HISTORY: Screening.  
COMPARISON: 5/15/2022.  
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  Additional spot compression views performed.  
FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.  
There are no masses, asymmetries, suspicious calcifications or areas of architectural distortion bilaterally.  Stable benign appearing calcifications noted in both breasts. 
Stable post-surgical changes from prior left breast lumpectomy are again noted.
IMPRESSION:
1. No mammographic evidence of malignancy.  Recommend continued screening mammography in 1 year.
2. Heterogeneously dense breast tissue.  
ASSESSMENT CATEGORY: BI-RADS 1, Negative.  
ADDENDUM: Reviewed and agreed with by Dr. XXX. ➡ *Negative*
- LEFT DIAGNOSTIC MAMMOGRAM
Clinical History: 49 y.o. female presents today for a diagnostic mammogram due to a palpable finding in the left breast.
Comparison: Prior mammogram from (outside institution) 10/04/2023.
Technique: Standard CC and MLO views were obtained of the left breast. In addition, spot compression and magnification views were performed to better evaluate the palpable findings in the right breast
Findings:
The breasts are heterogeneously dense which may reduce the sensitivity of mammography.
Left breast:
Palpable finding corresponds to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Left breast, palpable finding, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended. ➡ *Negative*
- MAMMOGRAPHY - SCREENING - BILATERAL. CLINICAL HISTORY: Patient is a 43 year old female presenting for her screening mammogram. TECHNIQUE: Craniocaudal and mediolateral oblique views of the breasts were performed using digital mammography. COMPARISON: Comparison is made to the mammogram from 03/29/2023. FINDINGS: The breasts are heterogeneously dense. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- FINDINGS:
Right breast: Heterogeneously dense parenchyma. No dominant mass, architectural distortion or suspicious calcifications.
Left breast: The breast is almost entirely fatty. No dominant mass, architectural distortion or suspicious calcifications. Stable postoperative changes from prior lumpectomy are noted in the upper outer quadrant. Surgical clips are seen in expected location.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography.
2. No suspicious microcalcifications on either side.
BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY, RIGHT BREAST

CLINICAL HISTORY: 50 year old woman presents for diagnostic evaluation of a right breast lump, found on self-exam. Last mammogram was 2 years ago and was normal.

TECHNIQUE: Diagnostic mammographic views of the right breast were obtained including craniocaudal and mediolateral oblique views.

FINDINGS:  
The breast is heterogeneously dense. No suspicious masses, architectural distortion, or suspicious calcifications are identified. The skin is normal in thickness. The nipple is normal in configuration. There are no significant changes compared to prior exam of 2 years ago.

IMPRESSION: 
RIGHT BREAST, BIRADS 1 - NEGATIVE. 
No evidence of malignancy. Recommend routine screening in 1 year.

ULTRASOUND RIGHT BREAST

INDICATION: Palpable abnormality right breast at 10 o'clock. 

TECHNIQUE: Targeted ultrasound of the right breast was performed using a high frequency linear transducer.

FINDINGS:  
At the 10 o'clock position 5 cm from the nipple there is a 0.8 x 0.5 x 0.6 cm oval parallel hypoechoic mass with circumscribed margins. No posterior features are present. Ultrasound findings are consistent with a benign finding such as fibroadenoma. No other masses or abnormalities are seen in the right breast.

IMPRESSION:
Benign appearing mass right breast 10 o'clock position, most consistent with fibroadenoma. No suspicious findings.

RECOMMENDATION: 
Routine screening mammography in 1 year. Clinical follow up of palpable mass as indicated. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

BIRADS BREAST DENSITY: B
Scattered areas of fibroglandular density.

COMPARISON: Previous MLO views from June 1, 2022. Current study includes CC views. 

FINDINGS:
The breasts are predominantly fatty.

No mammographic evidence of malignancy. No suspicious masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
Negative bilateral screening mammogram.

RECOMMENDATION:
Continued routine screening mammography in 1 year per standard guidelines.

BIRADS CATEGORY 1: NEGATIVE

ADDENDUM:
After review of the provided history, no change to the above report. ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAPHY WITH TOMOSYNTHESIS AND ULTRASOUND.

INDICATION: Annual screening. No current breast complaints.

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
The breasts are heterogeneously dense which may limit the sensitivity of mammography.

There are no dominant masses, architectural distortions, or suspicious calcifications in either breast.

LEFT BREAST: 
At the 9:00 axis in the superior-lateral left breast there is a newly developed cluster of fine pleomorphic microcalcifications which are highly concerning for suspicious morphology. This cluster measures approximately 7 mm.

Additional new scattered punctate benign-appearing microcalcifications are noted in the left breast, predominantly in the upper outer quadrant. 

RIGHT BREAST: 
There are scattered bilateral vascular calcifications and stromal calcifications, all characteristically benign. 

Two small simple cysts are noted in the lower inner quadrant of the right breast, each measuring 5-6 mm. These are unchanged from the prior study.

Bilateral ultrasound was performed for further characterization. This confirms a 7 mm cluster of microcalcifications at 9:00 in the left breast without an associated mass. No other suspicious masses or cystic lesions are seen by ultrasound in either breast.

IMPRESSION:
1. New highly suspicious cluster of microcalcifications at 9:00 in the left breast, measuring 7 mm. Stereotactic core needle biopsy is recommended. BIRADS 4C. 
2. Probably benign scattered bilateral calcifications and small simple cysts. 
3. Recommend follow-up diagnostic left mammogram in 6 months following biopsy to ensure stability of the additional left breast microcalcifications. 

Right breast: BI-RADS 2. Left breast: BI-RADS 4C. ➡ *Positive*
- SCREENING BILATERAL MAMMOGRAM
INDICATION: Routine screening. No current problems.
TECHNIQUE: Standard CC and MLO views of each breast were obtained.
COMPARISON: Comparison is made to prior exam dated 1/15/2020.
FINDINGS:
The breast tissue is heterogeneously dense, which could obscure detection of small masses.
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Stable benign appearing calcifications are noted in the upper outer quadrants of both breasts.
There are new scattered groups of amorphous calcifications in the upper outer right breast. These are indeterminate. Additional magnification views are recommended for further evaluation.
IMPRESSION:
BI-RADS 0: Incomplete. Additional imaging evaluation with magnification views of the right breast is needed to assess for potentially suspicious microcalcifications in the upper outer quadrant.

ADDENDUM - MAGNIFICATION VIEWS RIGHT BREAST: 
Magnification CC and ML views of the upper outer right breast were performed. The calcifications are grouped and pleomorphic, concerning for ductal carcinoma in situ (DCIS). Stereotactic biopsy is recommended for tissue diagnosis.
IMPRESSION:
BI-RADS 4: Suspicious calcifications upper outer right breast. Stereotactic biopsy recommended. ➡ *Positive*
- MAMMOGRAPHIC REPORT:
NAME: Jane Smith
EXAMINATION: Bilateral digital mammography, followed by targeted left breast ultrasound
CLINICAL HISTORY: 56-year-old female presents for high risk screening. Mother had breast cancer at age 59. No palpable complaints.
COMPARISON: 10/15/2022
TECHNIQUE: Standard CC and MLO views of bilateral breasts.
FINDINGS:
The breast tissue is heterogeneously dense which may obscure small masses.
Right breast:
There are no dominant masses, architectural distortions, or suspicious calcifications.
Left breast:
There are no dominant masses, architectural distortions, or suspicious calcifications.
Targeted ultrasound of the left breast upper outer quadrant was performed. No sonographic correlate for the palpable complaint was identified.
IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening in the setting of dense breast tissue.
2. Targeted left breast ultrasound is negative.
ASSESSMENT: BI-RADS Category 1: Negative
MANAGEMENT: Recommend continued annual screening mammography. ➡ *Negative*
- LEFT BREAST MAMMOGRAM
HISTORY: Annual screening. Patient denies any breast lumps, skin changes, or nipple discharge. She reports a strong family history of breast cancer in her maternal aunt.
FINDINGS: The breasts are heterogeneously dense (BI-RADS type C). There are scattered areas of fibroglandular tissue. No dominant mass, architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications are noted in the upper outer quadrant. No significant change compared to prior exam from 2 years ago.
IMPRESSION: No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
ASSESSMENT: BI-RADS 1 - Negative.
ADDENDUM: Comparison was made to prior outside imaging. Breast density, fibroglandular tissue pattern and distribution, and benign calcifications appear stable. No suspicious findings. ➡ *Negative*
- BILATERAL MAMMOGRAM
INDICATION: Annual screening mammogram. Patient denies any breast complaints.
FINDINGS:
The breast tissue is predominantly fatty. There are scattered areas of fibroglandular density.
There are no suspicious masses, architectural distortions, or suspicious microcalcifications in either breast.
There are no significant changes compared to the prior studies on 6/12/2022.
IMPRESSION:
Bilateral fatty replaced breasts. No mammographic evidence of malignancy.
BI-RADS category 1: Negative.
Annual screening mammography is recommended.
ADDENDUM: The breasts are predominantly fatty, and there are no suspicious mammographic findings in the breasts bilaterally. Specifically, there are no suspicious microcalcifications identified in either breast. Routine annual screening is recommended. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM
History: Screening mammogram.
Comparison is available dating from 03/2022 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. Automated Breast Ultrasound (ABUS) of both breasts was also performed.

COMPARISON: Prior mammogram from 10/15/2021.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses. 

Right Breast:
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion.

Left Breast: 
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion.

Automated Breast Ultrasound (ABUS):
Right Breast: No sonographic evidence of malignancy. No suspicious mass or other finding.
Left Breast: No sonographic evidence of malignancy. No suspicious mass or other finding.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. No suspicious calcifications identified.
2. No sonographic evidence of malignancy on ABUS of either breast.

RECOMMENDATION: 
Continue annual screening mammography and supplemental screening with ABUS as the patient has heterogeneously dense breast tissue.

ASSESSMENT CATEGORY: BI-RADS 1 - Negative. 
ADDENDUM: Patient notified of results via patient portal. ➡ *Negative*
- SCREENING MAMMOGRAM. LEFT DIAGNOSTIC MAMMOGRAM.

INDICATION: Routine screening. Call back from screening for asymmetry in the left breast upper outer quadrant. 

TECHNIQUE: Standard CC and MLO views of both breasts. Spot compression magnification views of the left breast. 

COMPARISON: Screening mammogram of April 15, 2022.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses. 

There are no suspicious masses, architectural distortion or definite suspicious microcalcifications. Stable benign appearing calcifications are noted bilaterally.

There is a focal asymmetry in the left breast at 2 o'clock middle depth on the CC view only, measuring approximately 1 cm. This persists on spot compression magnification views. Ultrasound is recommended for further evaluation. 

IMPRESSION: 
BI-RADS Category 0: Incomplete. Need additional imaging evaluation. Focal asymmetry left breast, not definitely seen on prior screening exam. Recommend targeted left breast ultrasound for further evaluation.

ADDENDUM - ULTRASOUND LEFT BREAST:
Targeted ultrasound of the left breast 2 o'clock region demonstrates normal fibroglandular tissue with no definite mass, architectural distortion or suspicious microcalcifications.  

FINAL ASSESSMENT:
BI-RADS Category 2: Benign. Recommend routine screening in 1 year. ➡ *Not Stated*
- CLINICAL HISTORY: Annual screening mammogram. Patient reports no palpable lumps or other breast concerns.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 

COMPARISON: Comparison is made to prior study dated January 15, 2022.

FINDINGS:
Right breast: The breast tissue is predominantly fatty. There are no dominant masses, suspicious calcifications, or areas of architectural distortion. 

Left breast: The breast tissue is predominantly fatty. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.  

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Recommend continued annual screening mammography in 12 months.

BIRADS Category 1 - Negative. ➡ *Negative*
- MAMMOGRAPHY, BILATERAL

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
Both breasts show scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious microcalcifications.

IMPRESSION: 
Negative bilateral screening mammogram.

BI-RADS Category 1 - Negative. Recommend routine screening in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense (BI-RADS density c), which may obscure detection of small masses.

There are scattered areas of fibroglandular density bilaterally.

A new cluster of amorphous microcalcifications is noted in the upper outer quadrant of the right breast at the 10 o'clock position, 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No associated mass, architectural distortion, or skin thickening is identified.

The left breast demonstrates no suspicious masses, calcifications, or areas of architectural distortion.

No mammographic evidence of malignancy is seen in the left breast.

IMPRESSION:
BI-RADS Category 0: Incomplete, additional imaging evaluation needed for the right breast. A magnification view of the right breast is recommended for further characterization of the microcalcifications in the upper outer quadrant.

BI-RADS Category 1: Negative for the left breast.

ADDENDUM:
Right breast magnification views demonstrate a cluster of amorphous microcalcifications in the upper outer quadrant, measuring approximately 9 mm in greatest dimension. The distribution is clustered and the morphology is suspicious for malignancy. Stereotactic core biopsy is recommended for tissue diagnosis.

FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious abnormality in the right breast. Biopsy should be considered.
BI-RADS Category 1: Negative for the left breast. ➡ *Positive*
- LEFT SCREENING MAMMOGRAM
CLINICAL HISTORY: Routine screening. Negative prior mammogram at this facility on 09/12/2021.
TECHNIQUE: Standard CC and MLO views of the left breast.
COMPARISON: Prior left screening mammogram from 09/12/2021.
FINDINGS:
The left breast is composed of scattered fibroglandular densities.
There are no dominant masses, architectural distortions, or suspicious calcifications.
No significant interval change is observed.
IMPRESSION:
1. Negative left screening mammogram.
2. No mammographic evidence of malignancy. Annual screening mammography is recommended.
BI-RADS Category 1 - Negative. ➡ *Negative*
- MAMMOGRAM, BILATERAL WITH TOMOSYNTHESIS AND ULTRASOUND OF RIGHT BREAST
INDICATION: Right breast lump.
COMPARISON: None.
FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.
In the left breast, there are no dominant masses, architectural distortion or suspicious calcifications. 
In the right breast at 10 o'clock, 4 cm from the nipple, there is an 8 mm equal density mass. Ultrasound shows this to be a complex cystic and solid mass with irregular margins, measuring 9 x 8 x 7 mm. It demonstrates internal vascularity.
In the right breast, a group of amorphous calcifications is identified at the 2 o'clock position, 6 cm from the nipple. This spans an area of approximately 7 mm. Additional magnification views confirm the presence of these suspicious calcifications.
The remainder of the right breast is unremarkable. No axillary, supra or infraclavicular adenopathy.
IMPRESSION:
RIGHT BREAST, 10 O'CLOCK, COMPLEX CYSTIC AND SOLID MASS WITH IRREGULAR MARGINS, HIGHLY SUSPICIOUS FOR MALIGNANCY. ULTRASOUND GUIDED BIOPSY IS RECOMMENDED. BI-RADS 4C.  
RIGHT BREAST, 2 O'CLOCK, 7 MM GROUP OF SUSPICIOUS AMORPHOUS CALCIFICATIONS. STEREOTACTIC CORE BIOPSY IS RECOMMENDED. BI-RADS 4B.  
LEFT BREAST, BI-RADS 1, NEGATIVE. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

COMPARISON: None.

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

FINDINGS:
Both breasts are heterogeneously dense (BI-RADS C), which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a new cluster of fine pleomorphic calcifications in the upper outer quadrant, approximately 8 cm from the nipple, spanning an area of about 8 mm. No associated mass is seen.

The remaining left breast is unremarkable, with no dominant masses, areas of architectural distortion, or additional suspicious calcifications.

IMPRESSION:
BI-RADS 4B: Suspicious calcifications in the left breast. Stereotactic biopsy is recommended for further evaluation.

BI-RADS 2 for the right breast: Benign finding. Recommend routine screening in 1 year.

ADDENDUM:
The patient returned for left breast stereotactic biopsy of the cluster of calcifications in the upper outer quadrant. Specimen radiography confirmed the presence of calcifications within the core samples. Final pathology revealed ductal carcinoma in situ (DCIS), intermediate nuclear grade, cribriform and micropapillary types, with associated calcifications. The patient will be referred to surgery for further management. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms were obtained.
The breast tissue is predominantly fatty.
In the right breast, there are no suspicious masses, asymmetries or architectural distortion. 
In the upper outer left breast at approximately 2:00, 8 cm from the nipple, there is a group of fine pleomorphic microcalcifications in a segmental distribution, spanning approximately 15 mm. No associated mass or asymmetry is seen. No other suspicious calcifications are identified in either breast.
No mammographic evidence of malignancy in the right breast.

IMPRESSION:
1. Group of suspicious microcalcifications left breast as described above, BIRADS 4. Stereotactic biopsy is recommended for further evaluation. The patient will be scheduled for stereotactic biopsy of the left breast on a separate requisition.  
2. Benign right breast, BIRADS 2. Routine screening in 1 year is recommended. ➡ *Positive*
- NORMAL FFDM MAMMOGRAM

INDICATION: Screening mammogram. 

COMPARISON: None.

TECHNIQUE: Standard CC and MLO views of both breasts obtained.

FINDINGS:
Both breasts show scattered areas of fibroglandular density. No significant masses, architectural distortion, or suspicious calcifications are identified in either breast. The skin appears unremarkable. No enlarged axillary lymph nodes are seen.

IMPRESSION: 
Negative bilateral screening mammogram. 

BREAST DENSITY: The breasts are heterogeneously dense, which may obscure small masses. (Category C)

BIRADS ASSESSMENT CATEGORY: 1 - Negative.

RECOMMENDATION: Routine screening in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Prior mammogram from 10/15/2020.

FINDINGS:
The breasts are predominantly fatty.

There are scattered areas of fibroglandular density.

At the 12:00 position of the right breast, 6 cm from the nipple, there is a new group of amorphous microcalcifications spanning an area of about 1 cm. This is a suspicious finding. Additional views were obtained to further evaluate.

The spot compression magnification views of the right breast confirm a cluster of amorphous microcalcifications at 12:00, 6 cm from the nipple. The distribution is segmental. This is highly suspicious for ductal carcinoma in situ (DCIS).

No other suspicious masses, calcifications, architectural distortion or skin thickening are identified in either breast.

IMPRESSION:
New group of suspicious amorphous microcalcifications at 12:00 position of the right breast, segmental distribution, highly suspicious for DCIS. Additional imaging evaluation with breast MRI is recommended for extent of disease.
BI-RADS Category 4: Suspicious abnormality, biopsy should be considered.

ADDENDUM:
Ultrasound-guided core biopsy of the group of microcalcifications at 12:00 in the right breast was performed. Pathology results confirmed ductal carcinoma in situ, intermediate nuclear grade, cribriform type. The patient will be referred to surgical oncology for further management. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram.

COMPARISON: None.

TECHNIQUE: Bilateral CC and MLO views.

FINDINGS:
The breasts are predominantly fibroglandular in composition.

There is no evidence of a mass, architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a new cluster of fine pleomorphic microcalcifications in the upper outer quadrant on the MLO view, spanning approximately 1 cm. This cluster was not present on the CC view. No associated mass or distortion is identified.

The remainder of the left breast is unremarkable.

IMPRESSION: 
RIGHT BREAST: BI-RADS 1 - Negative.

LEFT BREAST: BI-RADS 0 - Incomplete assessment. A cluster of pleomorphic microcalcifications is identified in the left upper outer breast on the MLO view, not seen on the CC view. Additional mammographic views are recommended for further evaluation. 

RECOMMENDATION: Spot compression magnification views of the left upper outer breast are recommended to further characterize the cluster of microcalcifications. ➡ *Positive*
- MAMMOGRAM - BILATERAL SCREENING 

INDICATION:   Annual screening.

TECHNIQUE:  CC and MLO screening mammographic views.

COMPARISON:  Previous outside screening mammogram 1 year ago.

FINDINGS:  The breasts are heterogeneously dense. There are no dominant masses, suspicious calcifications or architectural distortion. 

IMPRESSION:
BIRADS Category 1 - Negative. Recommend routine screening in 1 year. ➡ *Not Stated*
- "CAUDAL SCREENING MAMMOGRAMS
Mammographic examination including CC and MLO views of each breast.
Fibroglandular tissue density occupies more than 50% of the total breast area bilaterally.
No mammographically visible suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
BIRADS 2 - Benign findings. Regular one year surveillance mammogram is recommended with tomosynthesis if available. Annual clinical breast examination by a health care provider is also recommended.
ADDENDUM
The physician discussed the results with the patient.
" ➡ *Not Stated*
- BILATERAL DIAGNOSTIC MAMMOGRAM
Clinical history: Patient is referred for a diagnostic mammogram due to palpable findings in the right breast.
Comparison: Prior mammograms from (outside institution), not available for review.
Technique: Standard CC & MLO views were obtained of each breast. In addition, spot compression and magnification views were performed to better evaluate the palpable findings in the right breast.
Findings:
The breasts are heterogeneously dense, which may reduce the sensitivity of mammography.
Right breast:
Palpable findings in the right breast corresponding to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
Left breast:
Scattered fibroglandular densities. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Right breast, palpable findings, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended.
Left breast, negative. BI-RADS 1. ➡ *Negative*
- MAMMOGRAM LEFT BREAST.
HISTORY: Screening.
COMPARISONS: 09/20/2022
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- MAMMOGRAM RIGHT BREAST
HISTORY: Right breast pain, superior
COMPARISONS: None available.
TECHNIQUE: Standard CC and MLO views of the right breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative right mammogram. BI-RADS 2.
RECOMMENDATION:
Correlation with recent mammograms is recommended when available. If symptoms persist, clinical correlation is recommended. ➡ *Negative*
- BILATERAL MAMMOGRAM:
INDICATION: Annual screening mammogram. 
FINDINGS:
The breasts are almost entirely fatty.
No suspicious masses or areas of architectural distortion are identified in either breast.
There are scattered punctate and amorphous microcalcifications in the central right breast. A small group of pleomorphic microcalcifications is noted at at 9 o'clock position in the right breast upper outer quadrant, 6 cm from the nipple.
IMPRESSION:
GROUP OF PLEOMORPHIC MICROCALCIFICATIONS IN THE RIGHT BREAST UPPER OUTER QUADRANT. FURTHER EVALUATION WITH MAGNIFICATION COMPRESSION VIEWS IS RECOMMENDED. BIRADS 0.
Remainder of mammographic examination is benign. BIRADS 2. 
ADDENDUM - MAGNIFICATION COMPRESSION RIGHT BREAST VIEWS:
There is a group of approximately 10-15 pleomorphic microcalcifications spanning an area of 8 mm at the 9 o'clock position of the right breast upper outer quadrant, 6 cm from the nipple. No associated mass is seen. The distribution is clustered.
IMPRESSION:
SUSPICIOUS PLEOMORPHIC CLUSTERED MICROCALCIFICATIONS RIGHT BREAST. STEREOTACTIC BIOPSY RECOMMENDED. BIRADS 4B. ➡ *Positive*
- SCREENING MAMMOGRAM.
EXAMINATION: SCREENING MAMMOGRAM
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained. 
COMPARISON: None available.
FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 
There are no mammographic findings suspicious for malignancy. Specifically, no dominant masses, suspicious calcifications, or concerning asymmetries are identified.
IMPRESSION:
BI-RADS Category 1: Negative.
Routine screening is recommended in 1 year. ➡ *Not Stated*
- FINDINGS:
Bilateral digital screening mammogram and ultrasound images of the axilla dated April 14, 2023 were reviewed and compared to previous studies from April 20, 2022.

FINDINGS:
The breasts are heterogeneously dense (type C). 

There is an approximately 1 cm irregular mass at the 10:00 axis of the right breast, 5 cm from the nipple, which demonstrates internal vascularity on ultrasound. There are also scattered microcalcifications in the right upper outer quadrant spanning an area of approximately 2 cm.

In the left breast, there is an area of architectural distortion in the upper outer quadrant. Targeted ultrasound demonstrates an irregular hypoechoic mass with angular margins and posterior acoustic shadowing, measuring approximately 12 x 8 x 7 mm. The mass is located at 2:00, 4 cm from the nipple. 

A left axillary ultrasound demonstrates cortical thickening of a level I lymph node, measuring up to 5 mm in thickness.

IMPRESSION:
1. Suspicious irregular mass right breast at 10:00, 5 cm from the nipple with associated microcalcifications, suspicious for malignancy. BI-RADS 4C. Biopsy recommended.

2. Suspicious mass left breast at 2:00, 4 cm from the nipple with associated architectural distortion on mammography, highly suggestive of malignancy. BI-RADS 5. Biopsy recommended. 

3. Abnormal left axillary lymph node with cortical thickening, suspicious for metastatic involvement. Biopsy recommended for confirmation.

ADDENDUM:
Stereotactic core biopsy of the right breast mass and microcalcifications performed on April 16, 2023 demonstrates invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. 

Ultrasound-guided core biopsy of the left breast mass performed on April 16, 2023 demonstrates invasive ductal carcinoma, grade 2, ER/PR positive, HER2 negative. Left axillary lymph node biopsy positive for metastatic carcinoma. ➡ *Positive*
- MAMMOGRAPHIC EXAMINATION OF THE LEFT BREAST
INDICATION: Rule out palpable lesion or mass.
COMPARISON: None.
FINDINGS:
The breasts consist of scattered fibroglandular densities (heterogeneously dense), limiting the sensitivity of mammography. 
No dominant masses, suspicious calcifications or areas of architectural distortion are apparent in either breast.
The skin is normal in thickness. The nipples are unremarkable.
ULTRASOUND LEFT BREAST:
Targeted ultrasound was performed over the area of palpable concern at the 7:00 position of the left breast. No discrete solid or cystic mass lesion is identified in this region. The ducts are not dilated.
IMPRESSION:
No mammographic or sonographic evidence of malignancy in the left breast. 
No abnormality is identified on targeted ultrasound to correspond with the palpable finding. If clinically warranted, short-term follow-up or biopsy could be considered for further evaluation.
BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS AND TOMOSYNTHESIS

CLINICAL HISTORY: 45 year old female with dense breasts. Screening.

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which could obscure detection of small masses.

No suspicious masses, architectural distortion, or suspicious calcifications are seen in either breast.

There is a 6 mm cluster of indeterminate microcalcifications in the upper outer left breast at 11:00, 6 cm from the nipple. The remainder of the left breast is unremarkable.

No suspicious mammographic findings are seen in the right breast. 

IMPRESSION: 
BI-RADS 0: Incomplete. Indeterminate cluster of microcalcifications in the left breast at 11:00. Additional imaging evaluation with spot compression magnification views is recommended for further characterization. 

BI-RADS 1: No suspicious mammographic findings in the right breast.

ADDENDUM:
Spot compression magnification views of the left breast in the CC and ML projections demonstrate a 5 mm cluster of heterogeneous microcalcifications at 11:00 in the left breast, 6 cm from the nipple. No associated mass or distortion. 

IMPRESSION:
BI-RADS 4: Suspicious cluster of microcalcifications in the left breast. Stereotactic biopsy is recommended for tissue diagnosis. 

BI-RADS 1: No change in the right breast. ➡ *Positive*
- FINDINGS: Bilateral mammograms show scattered fibroglandular densities. There are no dominant masses, architectural distortion or suspicious calcifications. Stable benign calcifications are noted in the right breast. No significant change from prior exam.

Targeted ultrasound of the left breast at 3 o'clock position corresponding to patient's palpable abnormality demonstrates a 5 mm simple cyst. No solid component or suspicious features.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend annual screening mammography.
2. Left breast 3 o'clock 5 mm simple cyst corresponding to palpable finding. No suspicious features.

BIRADS 2 - Benign finding. Recommend routine screening.

ADDENDUM: Comparison is made to outside mammograms from 12/2021 which are similar. The benign right breast calcifications are stable since that exam. ➡ *Negative*
- BREAST MRI
Clinical History: Left breast palpable abnormality.
Comparison: Outside mammogram.
Technique: Magnetic resonance imaging of both breasts was performed before and after the intravenous administration of gadolinium contrast.
Findings:
There is heterogeneous enhancement of the breast parenchyma bilaterally, in keeping with the patient's physiologic status.
There is no abnormal enhancement to suggest malignancy within either breast. No suspicious masses or non-mass enhancement is identified to explain the palpable abnormality. There is no axillary, internal mammary or supraclavicular lymphadenopathy.
IMPRESSION:
Negative MRI of the breasts. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

TECHNIQUE: Standard CC and MLO views of both breasts were obtained.

COMPARISON: None available.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast.

In the left breast, there is a new cluster of fine pleomorphic calcifications in the upper outer quadrant, which measures approximately 8 mm in greatest dimension. No associated mass or architectural distortion is identified.

IMPRESSION:
RIGHT BREAST: Negative. BI-RADS 1.
LEFT BREAST: Clustered fine pleomorphic calcifications in the upper outer quadrant, suspicious for malignancy. BI-RADS 4C.

RECOMMENDATION: Stereotactic biopsy of the left breast calcifications is recommended for further evaluation. Please send a referral for this procedure. ➡ *Positive*
- FINDINGS:
Screen-film mammogram. Bilateral two view mammographic study.
There are no dominant masses, suspicious microcalcifications or concerning architectural distortion.
The breast tissue is heterogeneously dense which may obscure detection of small masses.
IMPRESSIONS:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year.
ADDENDUM:
Tomosynthesis and spot compression views were not obtained. The client is participating in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST) study and was randomized to the 2D FFDM arm only as per study protocol. ➡ *Not Stated*
- TMIST FFDM SCREENING MAMMOGRAM. OBSP.
This client is a participant in the Tomosynthesis Mammographic Imaging Screening Trial (TMIST), a study approved by the Research Ethics Board. After obtaining informed consent, she was randomized to undergo screening mammography with full field digital mammography.  
Comparison to prior exam: Yes.
Breast tissue density: Scattered fibroglandular densities. Breasts are not extremely dense.
Routine OBSP screening recommended in 12 months.
No mammographic evidence of malignancy identified. No suspicious mass, architectural distortion or concerning calcifications are seen.
BIRADS Category 1 - Negative. ➡ *Not Stated*
- FINDINGS:
The breasts are heterogeneously dense. There are scattered areas of fibroglandular density. No suspicious masses, architectural distortion, or skin thickening.

At the 1:00 position of the right breast, 6 cm from the nipple, there is a group of amorphous microcalcifications spanning an area of approximately 10 mm. Additional magnification views in the right CC and MLO views confirm the presence of the microcalcifications. No associated mass, distortion, or skin changes are identified.

At the 3:00 position of the left breast, 4 cm from the nipple, there is a 6 mm well-circumscribed, oval mass with parallel orientation. This likely represents a benign finding such as a cyst or fibroadenoma.

The left breast is otherwise unremarkable. No suspicious masses, calcifications, architectural distortion, or skin thickening.

IMPRESSION:
1. Grouped amorphous microcalcifications in the right breast at 1:00, 6 cm from the nipple, spanning 10 mm. Additional diagnostic evaluation with a stereotactic core biopsy is recommended. BI-RADS 4B.

2. 6 mm circumscribed oval mass in the left breast at 3:00, 4 cm from the nipple, likely benign. BI-RADS 2.

ADDENDUM:
The patient proceeded with stereotactic core biopsy of the grouped microcalcifications in the right breast at 1:00. Pathology results demonstrate atypical ductal hyperplasia with focal areas of ductal carcinoma in situ, intermediate nuclear grade. Surgical consultation is recommended. ➡ *Positive*
- FINDINGS: The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography.

No dominant mass, suspicious calcifications or architectural distortion.

There are no new masses, areas of architectural distortion or suspicious calcifications compared to prior studies.

Bilateral breast ultrasound was performed.

Right breast: No sonographic abnormality. No suspicious mass, cyst, calcifications or architectural distortion.

Left breast: No sonographic abnormality. No suspicious mass, cyst, calcifications or architectural distortion.

IMPRESSION:
1. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. No sonographic evidence of malignancy bilaterally.

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAM - LEFT BREAST
CLINICAL INDICATION: Follow up left breast lump, palpable.
TECHNIQUE: Standard CC and MLO views of the left breast. 
COMPARISON: Prior exam St. Mary's Hospital 2020-11-15.
FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications. 
Stable postoperative changes noted in the right breast from prior lumpectomy. Surgical clip in good position.
No suspicious axillary lymphadenopathy.
IMPRESSION: 
No mammographic evidence of malignancy in the left breast. Stable benign postoperative changes in the right breast. 
Recommend follow up mammogram in 1 year per standard screening guidelines, return sooner if clinically indicated based on exam findings. Sonographic evaluation may be helpful to further characterize palpable abnormality.
BIRADS Category 1 - Negative. ➡ *Negative*
- HISTORY: Annual screening mammogram. Last mammogram performed 1 year prior was normal. No prior history of breast cancer or biopsies. No family history of breast cancer. No palpable lumps.

FINDINGS: Bilateral breast tissue contains scattered fibroglandular densities. There is no focal mass, suspicious calcification, or architectural distortion in either breast. There are no significant changes compared to prior mammograms. 

Targeted ultrasound was performed to the left breast at 2 o'clock position, corresponding to a palpable area of concern. No sonographic abnormality was identified. 

IMPRESSION:
1. There is no mammographic evidence for malignancy in either breast. Annual screening mammography is recommended.
2. No sonographic correlate for the palpable area of concern in the left breast at 2 o'clock position. Clinical follow up and correlation is recommended as indicated.

BI-RADS Category 1: Negative.

ADDENDUM: Spoke with ordering physician, Dr. Smith, and informed of the results. No further imaging is recommended at this time. Follow up as clinically indicated. ➡ *Negative*
- SCREENING MAMMOGRAM WITH COMPARISON TO PREVIOUS STUDIES.
INDICATION: Routine screening mammogram.
TECHNIQUE: Standard MLO and CC views obtained with digital technique bilaterally.  
COMPARISON: 05/05/2021
FINDINGS: 
The breasts are heterogeneously dense, which may obscure small masses.
There are no dominant masses, suspicious calcifications, or other findings. 
The previously noted benign-appearing calcifications in the left breast are stable. 
IMPRESSION:
BIRADS Category 1: Negative. 
Routine screening mammography is recommended in 1 year. ➡ *Not Stated*
- BILATERAL DIAGNOSTIC MAMMOGRAM - FOR SCREENING. CLINICAL HISTORY: Patient is a 43-year-old female presenting for her screening mammogram. No history of breast cancer, no family history, no breast symptoms. Patient has extremely dense breasts clinically. TECHNIQUE: Standard CC and MLO views were obtained. COMPARISON: Comparison is made to the patient's prior mammogram dated 10/18/2023. FINDINGS: The breasts are extremely dense. There is no mammographic evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: 1. Extremely dense breast tissue. Mammography is less accurate in women with dense breasts. This means that cancers may be missed that would be seen with mammography in women with average density breasts. We strongly encourage you to talk to your doctor about your dense breast tissue. 2. No mammographic evidence of malignancy. 3. BI-RADS Category 2: Benign finding. ➡ *Negative*
- MAMMOGRAM REPORT
CLINICAL INFORMATION: Annual screening mammogram. Patient reports no breast complaints.

COMPARISON: Prior mammogram from 09/12/2020.

FINDINGS:
Both breasts are heterogeneously dense which could obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications. 

LEFT BREAST:
Skin is normal in thickness. No dominant masses or areas of architectural distortion. 

Retro-areolar there is a new cluster of fine pleomorphic microcalcifications at the 3:00 position, 2 cm from the nipple, spanning an area of approximately 8mm. This is a suspicious finding.

IMPRESSION:
BI-RADS 4: Suspicious abnormality in the left breast. Stereotactic biopsy is recommended for the cluster of microcalcifications at 3:00 position of the left breast. 

BI-RADS 2: Benign finding in the right breast. Recommend routine screening in 1 year. ➡ *Positive*
- ULTRASOUND RIGHT BREAST AND RIGHT AXILLA.

History: Bilateral microcalcifications on screening mammogram. Biopsy recommended.

FINDINGS:
Focused sonographic evaluation of the right breast and right axilla performed.

In the region of the prior mammographic abnormality at 10:00 in the right breast, there is a 5 x 4 x 4 mm hypoechoic mass with indistinct margins and internal vascularity. This corresponds to the mammographic finding and is highly suspicious for malignancy.

Scanning the remainder of the right breast, there are no other suspicious masses, architectural distortions or enlarged axillary lymph nodes.

There are scattered simple appearing cysts in both breasts.

IMPRESSION:
Suspicious finding right breast at 10:00, corresponding to mammographic calcifications, highly suspicious for malignancy. Ultrasound guided core biopsy is recommended. BI-RADS 4C.

The remainder of the right breast is unremarkable. BI-RADS 2.

ADDENDUM:
Ultrasound guided core biopsy of the right 10:00 mass performed, multiple cores obtained and sent to pathology. Post biopsy clip placed. The patient tolerated the procedure well. Biopsy site hemorrhage controlled with manual compression. Sterile dressing applied. The patient was informed the results would be available in 2-3 working days. ➡ *Positive*
- MAMMOGRAM, LEFT.
HISTORY: Annual screening exam. Prior imaging not available for comparison.
TECHNIQUE: CC and MLO views of the left breast.
FINDINGS:
Breast composition is heterogeneously dense (BI-RADS C), which may obscure small masses.
There is no mammographic evidence of malignancy. No focal mass, significant asymmetry, or architectural distortion. No suspicious microcalcifications.
Skin is normal in thickness. No significant axillary lymphadenopathy.
IMPRESSION: No mammographic evidence of malignancy.
BI-RADS Category 1: Negative.
Annual screening mammography is recommended. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM
COMPARISON: None.
INDICATION: 47-year-old female with left breast mass, palpable, painful, enlarging over 2-3 months.
TECHNIQUE: Bilateral CC and MLO views were obtained. Spot compression magnification views obtained of the left breast.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure small masses.
Left breast: A 1.8 x 1.5 x 1.3 cm irregular mass is present at the 2 o'clock position, 6 cm from the nipple. The margins are indistinct and angulated. Associated architectural distortion is noted. Several groups of pleomorphic calcifications are identified adjacent to the mass, the largest cluster measuring 7 mm. No axillary lymphadenopathy.
Right breast: No dominant mass, suspicious calcifications, or other findings.
IMPRESSION:
Highly suspicious findings left breast, BI-RADS 5. Recommend ultrasound-guided core biopsy for further evaluation. Surgical consultation is also advised.
Benign right breast, BI-RADS 2. Recommend annual screening mammogram in 1 year. ➡ *Positive*
- TMIST FFDM ARM OBSP MAMMOGRAM.

This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm.

Compared to Previous: Yes. There is no significant change from the previous exam.

Breast Density >= 75%: No.
Breast Density < 75%: Yes.

OBSP to recall client in 1 year: Yes.

There are no dominant masses, architectural distortions, or suspicious calcifications.

BIRADS-2.

ADDENDUM: Comparison made to previous TMIST FFDM study dated [DATE]. No change. ➡ *Not Stated*
- ULTRASOUND LEFT BREAST

INDICATION: Left breast lump, palpable and visible on clinical exam. 

Targeted ultrasound of the left breast 6:00 position performed with comparison to prior ultrasound from 3 months ago. 

FINDINGS:
The palpable lump in the left breast 6:00 position corresponds to a well-circumscribed oval hypoechoic mass measuring 12 x 8 x 8 mm. This has increased very slightly in size from 10 x 7 x 7 mm on the prior study. No suspicious features are identified. No other lesions are seen in the left breast.

IMPRESSION:
Benign appearing left breast mass at 6:00, corresponding to the palpable finding. Likely represents a fibroadenoma. Recommend 6 month follow up ultrasound to document stability. 

BI-RADS 2 - Benign finding. ➡ *Negative*
- OBSP ROUTINE MAMMOGRAM.
 
A regular screening mammogram is performed as part of the Ontario Breast Screening Program (OBSP).

Compared with previous examination: No previous mammograms available for comparison.  

Breast density: The breasts are heterogeneously dense, which may obscure small masses.

No dominant masses, architectural distortions or suspicious calcifications are identified.  

There is no evidence of abnormal axillary lymph nodes.

Final Assessment:
BI-RADS Category 2 - Benign findings. Routine screening recommended in 1 year. ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAPHY

INDICATION: Annual screening mammogram

COMPARISON: None.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no suspicious masses, architectural distortion, or suspicious calcifications in either breast.

At the 12 o'clock position of the right breast, approximately 6 cm from the nipple, there is a group of amorphous calcifications spanning an area of about 8 mm. This group contains at least 8 calcifications. Magnification views confirm these are not vascular calcifications.

The remaining fibroglandular tissue in both breasts is unremarkable.

IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the right breast. Stereotactic core biopsy is recommended for the group of amorphous calcifications at 12 o'clock position of the right breast. 

BI-RADS Category 2 for the left breast: Benign finding. Recommend routine screening in 1 year. ➡ *Positive*
- RADIOLOGY REPORT
CLINICAL INFORMATION: Annual screening mammogram. No current breast complaints. Patient has a history of right breast cancer, status post lumpectomy and radiation therapy 5 years ago.
TECHNIQUE: Bilateral digital mammography including CC and MLO views.
COMPARISON: Prior exam dated 1 year ago.
FINDINGS:
Right breast: There are post-surgical changes from prior lumpectomy in the upper outer quadrant. The breast is heterogeneously dense. There are no dominant masses, architectural distortion, or suspicious calcifications.
Left breast: The breast is heterogeneously dense. There are no dominant masses, architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications are noted in the upper outer quadrant.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Stable post-surgical changes in the right breast.
3. Stable benign left breast calcifications.
BI-RADS Category 2: Benign findings. Recommend continued annual screening mammography. ➡ *Negative*
- FINDINGS:
Right breast: Heterogeneously dense breast tissue. There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Left breast: Scattered areas of fibroglandular density. There are no dominant masses, areas of architectural distortion, or suspicious calcifications. Stable appearance compared to prior mammograms.

Ultrasound:
Right breast: Imaging limited due to mastectomy changes. No sonographic abnormality.
Left breast: Imaging with attention to the area of palpable concern at 2 o'clock position 4 cm from the nipple. No solid or cystic mass lesion identified. Unremarkable fibroglandular tissue in this region.

IMPRESSION:
1. No mammographic evidence of malignancy in either breast. No suspicious microcalcifications identified.
2. Stable mammographic appearance compared to prior exams.
3. No sonographic correlate for the palpable area of concern in the left breast at 2 o'clock position. Recommend clinical follow up and correlation with physical exam findings as indicated.

BI-RADS Category 1: Negative. Recommend routine screening in 1 year. ➡ *Negative*
- MAMMO - BILAT - SCREENING. Clinical History: 47 year old female, no breast complaints, requests screening mammogram today. Comparison is made with previous mammograms from 12/27/2023. Findings: The breasts are heterogeneously dense. No suspicious masses or calcifications are identified. Impression: No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- MAMMOGRAM REPORT - BILATERAL

CLINICAL HISTORY:
Annual screening mammogram. No current breast concerns. Prior benign right breast biopsy at 10 o'clock 2 years ago, pathology showed fibroadenoma. Maternal aunt with breast cancer diagnosed at age 65.

TECHNIQUE:
Standard CC and MLO views of both breasts were obtained. 

COMPARISON:
Comparison is made to prior mammogram from March 2022.

FINDINGS:
The breasts are predominantly fatty replaced. There are no masses, architectural distortion, or suspicious calcifications in either breast. Stable postsurgical changes are noted in the right breast at 10 o'clock, corresponding to the known biopsy site. 

Scattered benign-appearing calcifications are present bilaterally, stable compared to prior exam. No dominant mass or mammographic abnormality is identified.

IMPRESSION:
1. No mammographic evidence of malignancy, with stable benign findings compared to prior exam. 
2. Recommend continued annual screening mammography in 12 months.
3. Patient advised to contact her doctor for any new breast concerns.

BI-RADS CATEGORY 1 - NEGATIVE. ➡ *Negative*
- SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: 05/15/2021

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications in either breast.

A few scattered round and punctate calcifications are noted in both breasts. In the left upper outer quadrant there is a new cluster of pleomorphic microcalcifications at approximately the 1:00 position, 6 cm from the nipple. This cluster measures approximately 6 mm in greatest dimension.

No axillary, subpectoral or internal mammary adenopathy.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation
New cluster of microcalcifications in the left breast at 1:00, 6 cm from the nipple. Additional magnification views are recommended for further evaluation.

RECOMMENDATION:
Left diagnostic mammogram with magnification views

ADDENDUM:
Magnification views of the left breast demonstrate a cluster of heterogeneous microcalcifications in the left upper outer quadrant corresponding to the area of concern on the screening mammogram. The cluster measures approximately 8 mm in greatest dimension. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS).

Final Assessment:
BI-RADS Category 4: Suspicious - Biopsy should be considered

RECOMMENDATION:
Stereotactic core biopsy of the left breast microcalcifications is recommended. ➡ *Positive*
- FINDINGS:
Bilateral digital mammography and targeted ultrasound of the right breast were performed.

The breasts are heterogeneously dense (C).

Right breast:
At the 12 o'clock position, 5 cm from the nipple, there is a new group of pleomorphic microcalcifications spanning an area of approximately 15 mm. Targeted ultrasound of this region demonstrates a 15 x 8 x 7 mm irregular hypoechoic mass with indistinct margins and internal vascularity. 

Additionally, there are scattered fibroglandular densities throughout the right breast. No dominant masses, architectural distortion, or suspicious calcifications are identified. 

Left breast:
There are no dominant masses, architectural distortion, or suspicious calcifications. Scattered areas of fibroglandular density are noted.

IMPRESSION:
Right breast: Highly suspicious group of pleomorphic microcalcifications at 12 o'clock with associated irregular mass on targeted ultrasound, BIRADS 5. Recommend stereotactic biopsy for definitive diagnosis.

Left breast: No suspicious findings, BIRADS 2. Recommend continued routine screening.

ADDENDUM:
The pathology report from the stereotactic biopsy of the right breast 12 o'clock lesion confirms invasive ductal carcinoma, grade 2, ER+/PR+/HER2-.

FINAL ASSESSMENT:
Right breast: BIRADS 6, known biopsy-proven malignancy.
Left breast: BIRADS 2, benign. ➡ *Positive*
- FINDINGS: The right breast has scattered fibroglandular densities. There is no focal mass, suspicious calcification or architectural distortion. The skin is normal in thickness. The nipple is normal in configuration. No axillary adenopathy. Benign calcifications consistent with arterial calcifications are noted. No significant interval change.

The left breast is predominantly fatty replaced. Benign scattered vascular calcifications are present. No focal mass or architectural distortion. The nipple and skin are unremarkable.

IMPRESSION:
1. Negative bilateral screening mammogram.
2. No evidence of malignancy or suspicious abnormality.
3. Benign bilateral arterial calcifications.

BIRADS 1 - Negative. ➡ *Negative*
- FINDINGS:
Bilateral digital mammography with tomosynthesis was performed.
The breasts are heterogeneously dense which may obscure detection of small masses.
In the right breast, there are no dominant masses, architectural distortions or suspicious calcifications.
In the LEFT breast, at the 10:00 position 6 cm from the nipple, there is a new cluster of fine pleomorphic microcalcifications spanning approximately 8 mm. Additional magnification views confirm these are suspicious and new compared to prior mammograms from 2019. 
At the 2:00 position 4 cm from the nipple, there is a 6 mm circumscribed mass, likely benign.
The remainder of the left breast is unremarkable. 
There are no abnormal axillary lymph nodes bilaterally.
IMPRESSION:
LEFT BREAST, 10:00, 6 cm FN: New cluster of fine pleomorphic microcalcifications, suspicious for DCIS. BIRADS 4B. Stereotactic biopsy recommended.
LEFT BREAST, 2:00, 4 cm FN: 6 mm circumscribed mass, likely benign fibroadenoma. BIRADS 2. 
RIGHT BREAST: Negative. BIRADS 1. ➡ *Positive*
- REPORT:
"MAMMOGRAPHIC REPORT:
INDICATION: Annual screening mammogram.
COMPARISON: Prior studies from 10/15/2019.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained as well as supplemental LM views.
FINDINGS: 
The breasts are almost entirely fatty. 
There is no architectural distortion or suspicious dominant masses.
The skin is within normal limits.
No significant axillary adenopathy.
Prominent vascular calcifications are again identified in both breasts. These have increased in number, especially in the right breast centrally.
Impression:
1. Increasing vascular calcifications both breasts with more prominent and increased number of calcifications in the right breast centrally. These are indeterminate. Recommend spot compression magnification views right breast at site of increasing calcifications and surgical consult for possible biopsy. BIRADS 0 right breast.
2. No mammographic evidence of malignancy left breast. BIRADS 2 left breast."
LABEL: Positive for microcalcifications in the current report. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM

HISTORY: Annual screening mammogram. No breast complaints. The patient is a 52-year-old female. Family history of breast cancer in mother diagnosed at age 60.

FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.
There are no dominant masses, areas of architectural distortion or abnormal calcifications in either breast.
Bilateral benign-appearing calcifications are noted. These are coarse, round, and punctate in morphology.
At the 10:00 position of the left breast, there is a small cluster of amorphous microcalcifications spanning approximately 6 mm. This cluster was not present on the prior examination.

IMPRESSION:
1. Heterogeneously dense breasts.
2. Bilateral benign-appearing calcifications, likely representing fibrocystic changes.
3. New small cluster of amorphous microcalcifications at the 10:00 position of the left breast, suspicious for early ductal carcinoma in situ (DCIS). Additional imaging evaluation with magnification views is recommended for further characterization.

RECOMMENDATION: Left diagnostic mammogram with magnification views of the microcalcifications at the 10:00 position. Consider stereotactic biopsy if the microcalcifications remain suspicious on magnification views.

BI-RADS CATEGORY 4: Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- SCREENING MAMMOGRAM:
Compared to Previous: Yes.
Breast Density: Scattered fibroglandular densities.
OBSP to recall client in 1 year: Yes.

FINDINGS:
Stable scattered fibroglandular densities in both breasts.
No significant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:
Negative screening mammogram.

BIRADS-2: Benign findings. Recommend routine screening in 1 year.

Patient notified of results.

ADDENDUM:
This is to confirm that no microcalcifications are reported on this screening mammogram. ➡ *Not Stated*
- FINDINGS: Both breasts are composed of scattered fibroglandular densities. There are no dominant masses, architectural distortion, or suspicious calcifications bilaterally.

Focused ultrasound was performed in the area of palpable concern at the 10 o'clock position of the right breast, 2 cm from the nipple. This corresponds to a palpable ridge on physical exam. No sonographic correlate is identified.

IMPRESSION:
1. No mammographic or sonographic evidence of malignancy. Recommend routine screening mammography in 1 year.
2. No sonographic correlate for the palpable abnormality at the 10 o'clock position of the right breast. Clinical correlation recommended.

BIRADS 1 - Negative. ➡ *Negative*
- MAMMOGRAM REPORT
CLINICAL INDICATION: Annual screening mammogram.
COMPARISON: Screening mammogram from [fictitious date], [fictitious year].
BREAST COMPOSITION: The breasts are heterogeneously dense, which may obscure small masses.
RIGHT BREAST:
There is no mammographic evidence of malignancy. No significant change compared to prior mammogram.
LEFT BREAST:
Scattered fibroglandular densities are noted. There are new grouped coarse heterogeneous calcifications spanning an area of approximately 10 mm at the 10 o'clock position, middle depth, approximately 7 cm from the nipple. No associated mass, architectural distortion, or skin thickening is identified.
IMPRESSION:
RIGHT BREAST: BI-RADS Category 2 - Benign finding.
LEFT BREAST: BI-RADS Category 4 - Suspicious abnormality. Stereotactic biopsy is recommended for the new grouped coarse heterogeneous calcifications at 10 o'clock position in the left breast.
ADDENDUM:
Stereotactic biopsy of the left breast calcifications was performed on [fictitious date]. Pathology results demonstrated ductal carcinoma in situ (DCIS), intermediate nuclear grade, with microcalcifications. Surgical consultation is recommended for further management. ➡ *Positive*
- SCREENING MAMMOGRAM
CLINICAL INDICATION: Routine screening
TECHNIQUE: CC and MLO views of both breasts using the digital mammography.
COMPARISON: None available
FINDINGS:
The breasts are heterogeneously dense (ACR category C). This may lower the sensitivity of mammography.
No significant masses, architectural distortion, or focal asymmetries.
There are scattered areas of benign-appearing calcifications in both breasts, particularly in the upper outer quadrants. A group of amorphous microcalcifications is noted in the right breast at the 10:00 position, 6 cm from the nipple, spanning an area of about 8 mm.
No suspicious axillary lymphadenopathy.
Stable postsurgical changes from prior left breast lumpectomy.
IMPRESSION:
BI-RADS Category 0: Incomplete assessment. Additional imaging evaluation needed.
Amorphous group of microcalcifications in the right breast as described. Additional magnification views are recommended for further characterization. Depending on the results, a stereotactic biopsy may be indicated.
The patient should return to the Breast Center for additional diagnostic mammographic views as well as a possible ultrasound of the right breast.
ADDENDUM: Additional magnification views of the right breast were obtained. The previously described group of amorphous microcalcifications at the 10:00 position of the right breast persists and appears slightly more concerning on the magnification views. An ultrasound of this area did not reveal any correlating mass.
FINAL ASSESSMENT: BI-RADS Category 4B: Moderately suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- MAMMOGRAM AND ULTRASOUND LEFT BREAST

CLINICAL HISTORY: Follow up of left breast lump, noted to be stable.  Prior right breast lumpectomy and mastectomy.  Strong family history of breast cancer.  

COMPARISON: Prior mammogram Dec 2024 StMikes.

FINDINGS:
Right mastectomy noted.  

Left breast composed of scattered fibroglandular densities.  No dominant mass, architectural distortion or suspicious calcifications.  Stable appearance compared to prior mammogram.  

Targeted ultrasound of the left breast 8 o'clock position demonstrates normal fibroglandular tissue with no discernible mass.

IMPRESSION:
1. Stable left breast mammogram and ultrasound with no evidence of malignancy.  Routine annual screening is recommended.  

2. No sonographic correlate for the clinically detected lump at the 8 o'clock position of the left breast.  Continued clinical follow up as indicated.

BI-RADS 1 - Negative. ➡ *Negative*
- FINDINGS: The breasts are heterogeneously dense. 

There are no suspicious findings in the right breast. BIRADS 1.

In the central region of the left breast at 12:00, 5 cm from the nipple, there is a new 8 mm cluster of pleomorphic microcalcifications. Magnification views confirm the presence of this cluster. No associated mass, distortion or asymmetry. 

At 6:00 in the left breast, there is a stable 12 mm circumscribed oval mass, likely benign.

The left axilla is unremarkable with no abnormal lymph nodes.

IMPRESSION:
New cluster of suspicious microcalcifications at 12:00 in the left breast, BIRADS 4. Recommend stereotactic biopsy for definitive diagnosis. 

The remainder of the left breast is BIRADS 2 with a likely benign finding. 

Normal right breast, BIRADS 1. ➡ *Positive*
- MAMMOGRAM REPORT:

INDICATION: Annual screening mammogram. No current breast complaints.

COMPARISON: Compared with prior studies from 10/15/2021.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

Right Breast:
There are no dominant masses, architectural distortion, or suspicious calcifications.

Left Breast: 
There are no dominant masses, architectural distortion, or suspicious calcifications.

IMPRESSION:

Negative bilateral screening mammogram. 

BIRADS Category 1: Negative. Recommend routine screening in 1 year. ➡ *Negative*
- REPORT:
"RIGHT BREAST Digital DIAGNOSTIC MAMMOGRAM:
HISTORY: Evaluate for abnormality.
COMPARISON: None.
FINDINGS:  
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
Bilateral MLO and CC views were performed.
There is an irregular speculated mass measuring approximately 15 mm in the upper outer quadrant of the right breast on MLO and CC views, about 10 cm from the nipple. Two smaller satellite lesions each about 6 mm in size are seen adjacent to the primary mass. Associated pleomorphic microcalcifications are seen extending anteriorly from the lesion over an area of about 3 cm.
The left breast appears normal.
No other suspicious masses, architectural distortion, or suspicious microcalcifications are seen in either breast. Benign scattered fibroglandular densities are noted.
The skin appears normal. No skin thickening or retraction. The nipples are normal. No significant axillary lymphadenopathy.
IMPRESSION: 
Right Breast:
1. 15 mm spiculated mass with adjacent satellite lesions and pleomorphic microcalcifications in the upper outer quadrant, highly suspicious for malignancy. Biopsy recommended. BI-RADS 5.
Left Breast: 
No mammographic evidence of malignancy. BI-RADS 1.
ADDENDUM: Ultrasound guided core biopsy of the right breast mass and satellities was performed using a 14 gauge needle. Specimen radiography confirmed calcifications within the biopsied tissue. The specimens were sent to pathology.
"
LABEL: Positive for microcalfications in the current report. ➡ *Positive*
- INDICATION: 45 year old woman presenting for screening mammogram.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications in either breast.

In the left breast at the 10:00 position there is a 4 mm cluster of indeterminate amorphous calcifications approximately 6 cm from the nipple. This was not present on prior mammograms and requires further evaluation with magnification views.

In the left breast at 4:00 there is a 3 mm cluster of indeterminate amorphous calcifications 4 cm from the nipple. These were not present on prior mammograms and require further evaluation with magnification views. 

The remainder of the left breast is unremarkable.

The right breast is unremarkable.

IMPRESSION:  
BI-RADS 0: Incomplete. Recall for additional imaging evaluation of bilateral clusters of amorphous microcalcifications.

ADDENDUM:
Magnification views of the left breast confirm 2 small clusters of indeterminate amorphous calcifications at 10:00 and 4:00 as described above. Stereotactic biopsy is recommended for definitive characterization.

FINAL ASSESSMENT: BI-RADS 4: Suspicious abnormality. Biopsy should be considered. ➡ *Positive*
- MAMMOGRAPHY REPORT
REFERRING PHYSICIAN: Dr. Jane Smith
PATIENT NAME: Emily Johnson
MRN: 987654
CLINICAL INDICATION: Left breast pain and fullness. No palpable lump. 
COMPARISON: None available.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained with spot compression magnification views of areas of interest in the left breast.
FINDINGS:
RIGHT BREAST: Fatty replaced breast tissue. No dominant mass, architectural distortion or suspicious calcifications. 
LEFT BREAST: Scattered areas of fibroglandular density. No dominant mass or architectural distortion. No suspicious calcifications. Asymmetry corresponding to area of palpable concern on physical exam, best seen on MLO view. Additional spot compression magnification views demonstrate a focal asymmetry without associated calcifications, likely representing summation artifact or normal fibroglandular tissue. No sonographically correlate on targeted ultrasound.
IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Asymmetry in the left breast corresponds to area of palpable concern, likely benign. Recommend clinical follow up and diagnostic mammogram in 6 months to document stability. 
3. BI-RADS Category 1 - Negative. 
ADDENDUM: Targeted left breast ultrasound was performed and demonstrated no sonographic abnormality in the area of palpable concern or mammographic asymmetry. No suspicious lesions identified. ➡ *Negative*
- SCREENING MAMMOGRAM.
Indication: Screening.
Technique: Standard CC and MLO views of both breasts were obtained.
Comparison: Compared to prior study from 1 year ago.
Findings:
The breasts are heterogeneously dense, which may obscure detection of small masses.
There are no dominant masses, architectural distortion, or suspicious calcifications.
Stable coarse benign-appearing calcifications are noted in both breasts, likely representing fibrocystic changes.
No significant interval change compared to prior mammogram.
IMPRESSION:
BIRADS 2-BENIGN FINDING. Recommend routine screening in 1 year. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
History: Screening
Comparison available dating from 06/2023 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- SCREENING MAMMOGRAM
INDICATION: Routine screening.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: None available.
FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.
There are no masses, architectural distortions, or suspicious calcifications.
The skin is unremarkable. No skin or nipple retraction is seen.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Heterogeneously dense breast tissue.
RECOMMENDATIONS:
Routine screening mammography is recommended in 1 year. 
BIRADS Category 1: Negative.
An addendum will be issued if prior exams become available for comparison. ➡ *Not Stated*
- BREAST IMAGING REPORT

INDICATION: Annual screening mammogram. 

TECHNIQUE: Bilateral digital mammography including craniocaudal and mediolateral oblique views.

COMPARISON: 10/15/2021

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses. 

RIGHT BREAST:
There are no dominant masses, architectural distortion, or suspicious calcifications.  

LEFT BREAST: 
There are no dominant masses, architectural distortion, or suspicious calcifications.  

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Recommend continued annual screening mammography in 12 months. 

ASSESSMENT CATEGORIES:
Right Breast: BI-RADS 1 - Negative
Left Breast: BI-RADS 1 - Negative

ADDENDUM: Clinical breast exam and review of risk factors performed. Patient is asymptomatic. No palpable abnormalities on clinical exam bilaterally. Patient is average risk and has no significant family history of breast cancer. Updated patient with results and recommendation for continued annual screening. ➡ *Negative*
- MAMMOGRAPHIC EXAMINATION:
The client participates in the TMIST trial, having been randomized to the FFDM arm. Study was approved by REB.  
No prior mammograms available for comparison.
Breasts are heterogeneously dense (51-75% fibroglandular).
No suspicious masses, architectural distortions or suspicious calcifications are identified. 
The skin appears unremarkable.
No enlarged axillary lymph nodes visualized.
Impression: 
BI-RADS Category 1 - Negative.
Routine annual screening mammogram is recommended. ➡ *Not Stated*
- SCREENING DIGITAL BILATERAL MAMMOGRAM

INDICATION:  Routine screening.

TECHNIQUE:  Standard CC and MLO views of both breasts were obtained.

COMPARISON:  Comparison is made to prior exam dated 1/15/2021.  

FINDINGS:  
The breasts are heterogeneously dense, which may obscure detection of small masses.  

There are no dominant masses, suspicious calcifications, or other findings suspicious for malignancy.

Stable bilateral benign-appearing calcifications are again noted.

IMPRESSION:  
No mammographic evidence of malignancy.  
Stable, benign-appearing calcifications bilaterally.
BI-RADS Category 2: Benign findings.  

RECOMMENDATION:  Routine screening in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM.
REASON FOR EXAM: Annual screening.
TECHNIQUE: Full field digital mammography including CC and MLO views bilaterally.  
COMPARISON: 11/03/2020.
FINDINGS: 
The breasts are heterogeneously dense. This may lower the sensitivity of mammography. 
There is no significant interval change compared to prior exam. 
No dominant masses, suspicious calcifications or architectural distortion.
IMPRESSION:
No mammographic evidence of malignancy. 
Routine annual screening is recommended.
BI-RADS Category 2: Benign. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Screening mammogram

COMPARISON: None

FINDINGS:
The breasts are almost entirely fatty.

No dominant masses, suspicious calcifications or architectural distortion seen in either breast.

Scattered vascular calcifications noted bilaterally.

Benign-appearing lymph nodes in both axillae, containing small calcifications on the left.

In the left breast at the 10 o'clock position 6 cm from the nipple, there is a small cluster of indeterminate microcalcifications. Additional magnification views in this area are recommended for further evaluation.

IMPRESSION:
1. No suspicious mammographic findings in the right breast. BIRADS 1.
2. Small cluster of indeterminate microcalcifications in the left breast at 10:00, 6 cm from the nipple. BIRADS 0. Recommend additional magnification views for further assessment. 

ADDENDUM AFTER MAGNIFICATION VIEWS LEFT BREAST: 
The magnification views demonstrate a 6 mm cluster of pleomorphic microcalcifications at the site of concern. Stereotactic biopsy is recommended. BIRADS 4B. ➡ *Positive*
- BILATERAL DIAGNOSTIC MAMMOGRAM. History: Personal history of left breast cancer status post lumpectomy and radiation 5 years ago. Annual screening. Comparison: To previous mammogram dated 10/15/2023. Findings: The breasts are heterogeneously dense, which may obscure some small abnormalities. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. There are scattered benign-appearing calcifications in both breasts. Impression: No mammographic evidence of malignancy. ➡ *Negative*
- BILATERAL MAMMOGRAM - SCREENING. CLINICAL HISTORY: 52-year-old female presenting for her screening mammogram. Patient reports no personal history of breast cancer. Mother diagnosed with breast cancer at age 75. TECHNIQUE: Standard CC and MLO views were obtained. COMPARISON: Comparison is made to the patient's prior mammogram dated 12/19/2023. FINDINGS: The breasts are heterogeneously dense. There is no mammographic evidence of a suspicious mass, architectural distortion, or suspicious calcifications. IMPRESSION: 1. No mammographic evidence of malignancy. 2. BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening mammogram
Comparison is available dating from 11/2023 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- TMIST FFDM/DBT INITIAL ARM SCREENING MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography plus DBT arm.
Compared to Previous: No.
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
OBSP to recall client in 1 year: Yes.
Both breasts are predominantly fibroglandular in tissue composition. No suspicious masses, architectural distortion or suspicious calcifications are identified. 
BIRADS-2.
ADDENDUM: No microcalcifications described. ➡ *Not Stated*
- SCREENING MAMMOGRAPHY
INDICATION: Screening (asymptomatic)
TECHNIQUE: Standard CC and MLO views of both breasts.
COMPARISON: None available
FINDINGS: 
The breasts are predominantly fatty replaced. 
In the right breast there are no dominant masses, suspicious calcifications, architectural distortion or suspicious skin thickening. 
In the left breast, there is a subtle linear asymmetry in the upper outer quadrant on the CC view only. No associated findings at this site on the MLO view. Architectural distortion is not seen. Multiple scattered amorphous calcifications are present in the upper outer quadrant on the CC and MLO views in a regional distribution.
Left axillary lymph nodes are within normal limits.
Right axillary lymph nodes are within normal limits.
IMPRESSION: 
1. Screening mammography demonstrates scattered amorphous calcifications in the left upper outer quadrant in a regional distribution. These are new compared to previous exams and are concerning for ductal carcinoma in situ (DCIS). Additional diagnostic spot compression views are recommended for further evaluation. BIRADS 0: Incomplete. Additional imaging evaluation needed.
2. No suspicious right breast findings. BI-RADS 1.
ADDENDUM: The patient was recalled for diagnostic workup. Spot compression magnification views of the upper outer left breast demonstrate regionalized grouping of amorphous calcifications consistent with BIRADS 4B: Suspicious for malignancy. Stereotactic biopsy is recommended. Pathology results confirmed high grade DCIS with necrosis. ➡ *Positive*
- MAMMOGRAPHY REPORT
CLINICAL HISTORY: 46 year old female presenting for routine screening mammogram. No current breast complaints. Patient reports a family history of breast cancer in her mother, diagnosed at age 60.
TECHNIQUE: Bilateral digital screening mammogram, including CC and MLO views of each breast.
COMPARISON: Comparison made to prior study from 11/15/2021.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 
Right Breast:
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion.
Left Breast: 
There is no mammographic evidence of malignancy. No dominant mass, suspicious calcifications, or concerning architectural distortion. 
Stable benign-appearing calcifications are present in the left breast, consistent with prior imaging.
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. 
2. Benign left breast calcifications, stable compared to prior exam.
BIRADS Category 2: Benign findings. Recommend continued routine screening in 1 year.
ADDENDUM: Bilateral breast ultrasound was performed to evaluate palpable abnormalities, with no suspicious findings. Details reported separately. ➡ *Negative*
- LEFT SCREENING MAMMOGRAM
Clinical History: 69 year old female, screening mammogram
Comparison: None.
Technique: Standard CC and MLO views of the breasts were obtained.
Findings:
Heterogeneously dense breast tissue bilaterally
No suspicious masses, calcifications, or architectural distortions are seen.
IMPRESSION:
Negative screening mammogram. BI-RADS 1. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM

CLINICAL HISTORY: Abnormal screening mammogram.

TECHNIQUE: Standard CC and MLO views of the bilateral breasts were obtained. Spot compression magnification views were performed over the area in question in the right breast.

FINDINGS:

The breasts are heterogeneously dense. There is a 6 mm cluster of pleomorphic microcalcifications in the right breast at 2 o'clock, 3 cm from the nipple. This was biopsied under stereotactic guidance.

IMPRESSION:

1. Cluster of pleomorphic microcalcifications in the right breast at the 2 o'clock position, 3 cm from the nipple, status post stereotactic biopsy. Please see biopsy results.

BI-RADS: 0 (Incomplete). ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM
 
INDICATION: Annual screening mammogram.
 
COMPARISON: Outside screening mammogram dated 09/17/2020. 
 
FINDINGS:
Breast composition: Both breasts are heterogeneously dense, which can obscure small masses.
 
LEFT BREAST:
There is a cluster of amorphous microcalcifications in the 9 o'clock position of the left breast about 7 cm from the nipple. Additional magnification views were obtained for further evaluation.
No dominant masses, significant architectural distortion or suspicious microcalcifications are seen.
 
RIGHT BREAST:
No dominant masses, significant architectural distortion or suspicious microcalcifications are seen.
 
IMPRESSION:
LEFT BREAST:
New cluster of amorphous microcalcifications in the 9 o'clock position, left breast. Additional diagnostic evaluation with spot compression magnification views is recommended. BI-RADS 0: Incomplete.
 
RIGHT BREAST:
Negative. BI-RADS 1.
 
ADDENDUM:
Left diagnostic mammogram with spot compression magnification views demonstrates a cluster of amorphous microcalcifications in the 9 o'clock position. No associated mass or architectural distortion. 

FINAL ASSESSMENT:
LEFT BREAST: BI-RADS 4: Suspicious abnormality, biopsy recommended.
RIGHT BREAST: BI-RADS 1: Negative. ➡ *Positive*
- FINDINGS: The left breast parenchyma is composed of scattered fibroglandular densities. No dominant masses, architectural distortion, or suspicious calcifications are identified. Stable appearance compared to the most recent prior examination.

Targeted ultrasound of the left breast 7 o'clock position was performed. No correlating abnormality is seen in this region. The tissue is unremarkable.

IMPRESSION:
1. No mammographic evidence of malignancy in the left breast. Recommend routine screening mammography in 1 year.
2. Targeted left breast ultrasound is negative for sonographic abnormality at the area of clinical concern. Recommend clinical follow up as indicated.

BI-RADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM WITH MAGNIFICATION VIEWS:

CLINICAL HISTORY: Patient presents for routine screening mammogram. No palpable abnormalities noted.

FINDINGS:
The breasts are predominantly fatty replaced.

There are no suspicious masses, architectural distortion, or skin thickening in either breast.

In the right breast at approximately 11:00, 6 cm from the nipple, there is a new group of amorphous microcalcifications spanning an area of about 1 cm. Additional magnification views in the CC and MLO projections confirm these calcifications.

The left breast is unremarkable. No dominant masses, calcifications, architectural distortion or skin thickening.

IMPRESSION:
New group of amorphous microcalcifications right breast at 11:00, 6 cm from the nipple. Additional imaging evaluation with a focal compression view is recommended for further characterization. BI-RADS 0, incomplete. Recommend additional imaging.

Left breast, BI-RADS 1, negative.

ADDENDUM: 

RIGHT BREAST FOCAL COMPRESSION VIEWS:
Additional views of the right breast were performed using a focal compression device. 

The calcifications in question are better visualized on these views. They are amorphous in morphology and are seen in a segmental distribution raising concern for ductal carcinoma in situ (DCIS). 

REVISED IMPRESSION:
Suspicious segmental pleomorphic microcalcifications right breast, BI-RADS 4. Stereotactic biopsy is recommended for definitive diagnosis.

Left breast, BI-RADS 1, negative. ➡ *Positive*
- SCREENING MAMMOGRAM

CLINICAL INDICATION: Routine screening mammogram.

COMPARISON: 08/19/2019

TECHNIQUE: Bilateral digital screening mammogram, including CC and MLO views.

FINDINGS:
The breast tissue is heterogeneously dense. There are no dominant masses, architectural distortions, or suspicious microcalcifications.

IMPRESSION: 
No mammographic evidence of malignancy. Recommend routine screening in 1 year.

BIRADS Category 2 - Benign finding. 

ADDENDUM:
The patient is notified that breast density is not associated with an increased risk of breast cancer. Additional supplemental screening is not indicated based on breast density alone. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM
CLINICAL HISTORY: Annual screening mammogram. No palpable complaints.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
COMPARISON: Compared with prior exam dated 5/12/2021.
FINDINGS:
Both breasts are almost entirely fatty.
LEFT BREAST: 
There is a new 8 mm cluster of pleomorphic microcalcifications in the upper outer quadrant of the left breast at the 2:00 position, 6 cm from the nipple. The calcifications vary in size and shape and are suspicious for malignancy. No other suspicious masses, architectural distortion or calcifications.
RIGHT BREAST: 
There are no suspicious masses, calcifications or areas of architectural distortion. No significant change from prior exam.
IMPRESSION:
BI-RADS 4: Suspicious cluster of microcalcifications in the left breast at 2:00 position. Recommend stereotactic biopsy for further evaluation. 
BI-RADS 2: Benign findings in the right breast. Recommend continued annual screening in 1 year. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAM
History: Call back from OBSP screening for left upper outer quadrant asymmetry.
Compared to Previous: Yes. 
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
FINDINGS:
Left CC and MLO views demonstrate an area of asymmetry in the left upper outer quadrant. This corresponds to the area of concern on the screening mammogram. Spot compression views of this region demonstrate benign appearing glandular tissue. No suspicious mass, architectural distortion, or calcifications are identified.  
The remainder of the left breast and right breast are unremarkable. No mammographic findings that are concerning for malignancy.
IMPRESSION:
BIRADS-1/2. Benign asymmetry left upper outer quadrant. Routine screening in 1 year. ➡ *Not Stated*
- MAMMOGRAPHY - BILATERAL.

INDICATION: Annual screening mammogram. No current breast complaints.

COMPARISON: Mammogram from 10/15/2021 at ABC Imaging Center.  

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses. 

Right breast:
There are no dominant masses, architectural distortion, or suspicious calcifications. The breast tissue is stable compared to prior mammogram.

Left breast: 
There are no dominant masses, architectural distortion, or suspicious calcifications. The breast tissue is stable compared to prior mammogram.

IMPRESSION:
1. Bilateral mammogram shows no mammographic evidence of malignancy. 
2. Both breasts are heterogeneously dense, which may lower the sensitivity of mammography. 

RECOMMENDATION:
Continue annual screening mammography and clinical breast exams. 

BIRADS Category 1: Negative. ➡ *Negative*
- BILATERAL MAMMOGRAM
INDICATION: Annual screening mammogram. No current problems.
COMPARISON: None available.
FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.
Right Breast:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
Left Breast: 
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.
IMPRESSION:
1. Negative bilateral screening mammogram.
2. Recommend annual screening mammography in 1 year.
BIRADS 1 - Negative. ➡ *Negative*
- SCREENING MAMMOGRAM:
INDICATION: 48-year-old female, screening.
COMPARISON: None available.
FINDINGS:
Both breasts appear heterogeneously dense. No dominant mass or architectural distortion is identified in either breast.
In the left breast at approximately 2 o'clock within the region of the heterogeneously dense parenchyma there are scattered punctate and amorphous microcalcifications which are new and therefore suspicious, extending over an area of approximately 1 cm. No other suspicious microcalcifications are seen elsewhere within either breast.
The skin appears normal and the nipples are within normal limits bilaterally. No axillary lymphadenopathy is evident.
IMPRESSION:
BI-RADS Category 4: Suspicious abnormality in the left breast. Stereotactic core biopsy is recommended for tissue diagnosis.
ADDENDUM - STEREOTACTIC CORE BIOPSY LEFT BREAST:
Stereotactic core biopsy was performed at the 2 o'clock position of the left breast targeting the suspicious microcalcifications. Specimen radiograph confirms the presence of microcalcifications. Pathology results are pending.
ADDENDUM - PATHOLOGY LEFT BREAST BIOPSY:
Pathology results from the stereotactic core biopsy of the left breast at 2 o'clock demonstrate ductal carcinoma in situ, intermediate grade, with microcalcifications.
FINAL ASSESSMENT:
BI-RADS Category 6: Known biopsy-proven malignancy (ductal carcinoma in situ) in the left breast. Surgical consultation is recommended. ➡ *Positive*
- SCREEN-FILM MAMMOGRAM.
Compared to Previous: Yes.
Breast Composition: The breasts are heterogeneously dense, which may obscure small masses.
There are no dominant masses, architectural distortion or suspicious calcifications.
IMPRESSION:
BIRADS-2: Benign.
Routine screening recommended in 1 year. ➡ *Not Stated*
- SCREENING MAMMOGRAM. ADDITIONAL EVALUATION NEEDED.
Clinical History: Annual screening mammogram.
FINDINGS:
The breasts are heterogeneously dense (50-75% fibroglandular tissue), which may obscure detection of small masses.
There are scattered benign-appearing calcifications in both breasts.
An approximately 8 mm irregular mass is identified in the upper outer quadrant of the right breast at the 10 o'clock position, about 7 cm from the nipple. This is a new finding compared to the prior mammogram from 1 year ago. No suspicious associated calcifications are seen.
IMPRESSION:
BI-RADS Category 0: Incomplete. Additional imaging evaluation is needed. An irregular mass in the right breast requires further characterization with diagnostic mammographic views and targeted ultrasound. ➡ *Not Stated*
- BILATERAL DIGITAL MAMMOGRAM WITH ULTRASOUND OF THE RIGHT BREAST. 

HISTORY: Annual screening. No breast complaints.

FINDINGS:
The breasts are heterogeneously dense (C). This may lower the sensitivity of mammography.

Left Breast:
No suspicious mammographic findings.

Right Breast:
There is a 6 mm cluster of indeterminate calcifications at the 10:00 position, 5 cm from the nipple. Spot compression magnification views show these to be pleomorphic. 

Targeted right breast ultrasound at site of the mammographic calcifications shows no correlate mass or dilated duct. 

Incidental finding of a 9 mm simple cyst at 2:00, 3 cm from the nipple. 

IMPRESSION:
1. New cluster of pleomorphic calcifications at 10:00 in the right breast. Stereotactic biopsy is recommended. BIRADS 4B. 
2. No suspicious findings left breast. BIRADS 2.

ADDENDUM - Stereotactic biopsy right breast calcifications:
Specimen radiograph confirms calcifications. Pathology demonstrates ductal carcinoma in-situ, intermediate grade, cribriform type.

BIRADS 6. ➡ *Positive*
- FINDINGS:
Right breast: Tomosynthesis was performed.
There is a skin marker at the 12:00 position  laterally. This corresponds to the area of palpable concern.
There are scattered areas of fibroglandular density. No suspicious mass, architectural distortion, or suspicious calcifications.
IMPRESSION:
Right breast: No mammographic evidence of malignancy. Breasts are heterogeneously dense, which may obscure detection of small masses. Clinical follow up of palpable abnormality at the 12:00 position laterally is recommended.
BIRADS 1: Negative.
ADDENDUM: Ultrasound of the right breast 12:00 position was performed. No sonographic correlate for the palpable finding. No suspicious mass or cystic lesion in this area. ➡ *Negative*
- FINAL REPORT:
LEFT DIAGNOSTIC MAMMOGRAM & TOMOSYNTHESIS:
INDICATION: Large palpable left breast mass. Outside imaging reported as abnormal. 
COMPARISON: Outside mammogram and ultrasound 01/15/2023.
TECHNIQUE: Standard CC and MLO views of the left breast. Tomosynthesis was performed.
FINDINGS: 
The left breast is heterogeneously dense, which may obscure detection of small masses. No significant masses, architectural distortion or suspicious calcifications are identified. 
At the site of palpable abnormality in the upper outer quadrant, there is an oval circumscribed mass corresponding to the lesion identified on outside imaging. This measures approximately 2.5 cm and is stable in size and appearance compared to prior. The mass demonstrates circumscribed margins and posterior acoustic enhancement, consistent with a benign cyst or fibroadenoma. 
No axillary, supraclavicular or internal mammary adenopathy. 
IMPRESSION:
1. Stable, benign-appearing mass in the left breast upper outer quadrant corresponding to the site of palpable abnormality and findings on outside imaging, likely representing a fibroadenoma or cyst. Recommend follow-up ultrasound to further characterize.
2. No evidence of malignancy.  
ASSESSMENT: BIRADS 2 - Benign findings. Recommend routine screening in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Prior screening mammogram from 05/15/2019.

FINDINGS:
The breasts are predominantly fatty.

There are no suspicious masses, architectural distortions, or asymmetries in either breast.

In the right breast, there are no significant changes compared to the prior mammogram.

In the left breast at the 10:00 position, 6 cm from the nipple, there is a new group of amorphous microcalcifications spanning an area of approximately 1 cm. This cluster was not present on the prior mammogram.

The skin is normal. No abnormal axillary lymph nodes are seen.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation.
New cluster of amorphous microcalcifications in the left breast at 10:00, 6 cm from the nipple. Additional magnification views are recommended for further evaluation.

ADDENDUM (SAME DAY): Magnification views of the left breast were performed and demonstrate a group of amorphous microcalcifications at the 10:00 position, 6 cm from the nipple. The morphology and distribution are suspicious for ductal carcinoma in situ (DCIS). Stereotactic-guided biopsy is recommended.

FINAL ASSESSMENT: 
BI-RADS Category 4: Suspicious abnormality - Biopsy should be considered. ➡ *Positive*
- MAMMOGRAPHY, BILATERAL.

CLINICAL HISTORY: Annual screening mammogram. Patient denies breast symptoms. No family history of breast cancer. Last mammogram 1 year ago.

COMPARISON: Prior mammogram from 1 year ago.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

RIGHT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

LEFT BREAST:
There are no dominant masses, areas of architectural distortion, or suspicious calcifications.

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. Heterogeneously dense breast tissue, which may lower the sensitivity of mammography. 
3. Routine annual screening mammography is recommended. Supplemental screening with breast ultrasound or MRI is not indicated based on breast density alone but may be considered based on individual risk factors.

BIRADS Category 1: Negative. ➡ *Negative*
- ULTRASOUND LEFT BREAST

INDICATION: 41 year old female presenting with left breast upper outer quadrant palpable mass.

TECHNIQUE: Targeted ultrasound of the left breast was performed using a 12 MHz linear transducer.

COMPARISON: None.

FINDINGS:
Skin: Normal. No skin thickening or abnormal vascularity.
Subcutaneous fat and premammary zone: Normal. No abnormality.  
Parenchyma: Heterogeneously echogenic fibroglandular tissue. No abnormal masses, architectural distortion or suspicious calcifications. 
Ducts: Non-dilated. No intraductal masses or debris.
Vascularity: No increased vascularity on color Doppler interrogation.
Other findings: No axillary, infraclavicular or supraclavicular adenopathy.

IMPRESSION:
No sonographic abnormality in the area of palpable concern in the left upper outer breast. Negative targeted ultrasound of the left breast. 
Recommend clinical follow up and diagnostic mammography for further evaluation. 
BI-RADS Category 1 - Negative. ➡ *Negative*
- BIRADS SCORE 2 
OBJECTIVE: Screening mammogram. 
COMPARISON: None.  
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
There are no dominant masses, abnormal calcifications, or areas of architectural distortion bilaterally. The parenchymal pattern is heterogeneously dense, which may obscure detection of small masses. 
No concerning axillary lymphadenopathy is identified.
IMPRESSION: 
No evidence of malignancy. Breasts are heterogeneously dense, which lowers the sensitivity of mammography. 
Routine annual screening is recommended for women of this age.
BIRADS Score 2 - Benign findings. 
FOLLOW-UP: Continue routine screening mammography in 12 months. ➡ *Not Stated*
- SCREENING FFDM BILATERAL MAMMOGRAM.
Compared to Previous: Yes.
Breast Density >= 75%: No.
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.
Mammographic findings are within normal limits. No evidence of malignant calcifications, suspicious masses or architectural distortion in either breast.
BIRADS-1.

Addendum: Ultrasound of the right breast upper outer quadrant was performed to evaluate a palpable lump. A 7mm benign-appearing cyst was identified, corresponding to the area of palpable concern. No solid masses or other suspicious findings. 
Final assessment remains BIRADS-1, routine screening recommended. ➡ *Not Stated*
- MAMMOGRAM LEFT BREAST.
HISTORY: Screening mammogram.
COMPARISONS: 02/22/2024.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, suspicious calcifications, or skin thickening are seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- NORMAL TMIST FFDM ARM OBSP MAMMOGRAM.
This client participated in the Research Ethics Board approved Tomosynthesis Mammographic Imaging Screening Trial (TMIST). Informed consent was obtained and she was randomly assigned to the full field digital mammography only arm. 
Compared to Previous: Yes.
Breast Density >= 75%: No. 
Breast Density < 75%: Yes.
OBSP to recall client in 1 year: Yes.

No significant mammographic abnormality. The breasts are heterogeneously dense. There are no dominant masses, architectural distortion or suspicious calcifications.

BIRADS-2. Routine screening recommended.

ADDENDUM: 
Comparison made to previous outside screening mammogram dated July 15, 2020. No significant interval change. ➡ *Not Stated*
- MAMMOGRAM REPORT - LEFT BREAST

CLINICAL HISTORY: 45-year-old female with a palpable lump in the left breast at the 2 o'clock position, first noticed one month ago. No prior mammograms available for comparison. Family history of breast cancer in paternal grandmother.

FINDINGS:
Left breast consists of heterogeneously dense parenchyma. There is a 1.5 cm oval-shaped mass at the 2 o'clock position, 4 cm from the nipple, corresponding to the palpable abnormality. The mass has circumscribed margins and is iso to hypoechoic on ultrasound. No suspicious calcifications are identified. The remainder of the left breast parenchyma is unremarkable. No skin thickening or retraction. No axillary lymphadenopathy.

IMPRESSION:
1. 1.5 cm circumscribed oval mass at 2 o'clock position of the left breast, corresponding to the site of palpable concern. Suspicious for fibroadenoma. Recommend biopsy for definitive diagnosis.
2. No mammographic evidence of malignancy or suspicious microcalcifications in the left breast.

BIRADS: 4A
RECOMMENDATION: Ultrasound-guided core biopsy of the 2 o'clock mass in the left breast. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM
Clinical history: Patient is referred for a diagnostic mammogram for evaluation of left breast pain.
Comparison: Prior mammograms from (outside institution) 02/22/2022.
Technique: Standard CC and MLO views were obtained of each breast. In addition, spot compression and magnification views were performed to better evaluate the palpable findings in the left breast.
Findings:
The breasts are heterogeneously dense, which may reduce the sensitivity of mammography.
Right breast:
Scattered fibroglandular densities. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
Left breast:
Palpable findings in the left breast corresponding to a palpable area of dense fibroglandular tissue. No suspicious mass, architectural distortion, or suspicious calcifications are identified.
IMPRESSION:
Right breast, negative. BI-RADS 1.
Left breast, palpable findings, negative mammographic findings. BI-RADS 2. Clinical correlation is recommended. ➡ *Negative*
- MAMMOGRAPHY REPORT
PROCEDURE: Bilateral screening mammogram.
HISTORY: 49-year-old female, no palpable abnormalities. Patient reports no breast symptoms.
TECHNIQUE: Standard CC and MLO views of each breast.
COMPARISON: Prior study of June 12, 2022.
FINDINGS:
The breasts are heterogeneously dense, which may obscure small masses.
There are no masses, architectural distortion, or suspicious calcifications.
The skin and nipple are unremarkable.
No significant interval change since the prior study.
IMPRESSION:
1. BI-RADS Category 1: Negative.
2. Recommend continued annual screening mammography in 12 months. ➡ *Negative*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

CLINICAL INDICATION: Routine screening, no complaints.

TECHNIQUE: Standard CC and MLO views were obtained bilaterally.

FINDINGS:
Both breasts are heterogeneously dense (BI-RADS density category C), which may lower the sensitivity of mammography.

Right Breast:
There are no suspicious masses, calcifications, or architectural distortions. The skin and nipple are unremarkable.

Left Breast:
In the upper outer quadrant of the left breast, there is a new cluster of fine pleomorphic microcalcifications at approximately 2 o'clock, 6 cm from the nipple. This cluster measures approximately 8 mm in greatest dimension. No associated mass or architectural distortion is seen. The remainder of the left breast is unremarkable. The skin and nipple are within normal limits.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation
New cluster of pleomorphic microcalcifications in the left breast at 2 o'clock, 6 cm from the nipple, suspicious for malignancy. Additional magnification views are recommended for further evaluation.

RECOMMENDATION:
Left diagnostic mammogram with magnification views of the area of concern. Consider ultrasound if clinically indicated. Routine screening of the right breast with no evidence of malignancy (BI-RADS 1).

ADDENDUM:
Left diagnostic mammogram with spot compression magnification views demonstrates a cluster of fine pleomorphic microcalcifications spanning an area of approximately 9 mm at 2 o'clock in the left breast, 6 cm from the nipple. 

FINAL ASSESSMENT:
BI-RADS Category 4: Suspicious - Biopsy should be considered
Stereotactic or ultrasound-guided core needle biopsy is recommended for the cluster of pleomorphic microcalcifications in the left breast. ➡ *Positive*
- FINDINGS:
The breasts are heterogeneously dense.
There are suspicious microcalcifications in the upper outer quadrant of the left breast spanning an area of approximately 3 cm at 2:00. Spot compression magnification views were performed for further characterization. The calcifications are grouped and pleomorphic in nature.
No dominant masses or architectural distortion are identified in either breast.
There is no suspicious skin thickening or nipple retraction.
No abnormal axillary lymph nodes are seen.

IMPRESSION:
BIRADS 4: Suspicious microcalcifications in the upper outer quadrant of the left breast. Stereotactic biopsy is recommended for definitive characterization.

ADDENDUM:
The patient proceeded with stereotactic biopsy of the left breast microcalcifications at 2:00. Pathology results showed ductal carcinoma in situ, nuclear grade 2. ➡ *Positive*
- SCREENING MAMMOGRAM
CLINICAL INDICATION: Annual screening mammogram.
COMPARISON: None.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.
FINDINGS:
Right Breast:
The breast tissue is heterogeneously dense. There are no dominant masses, suspicious calcifications, or architectural distortion.
Left Breast:
The breast tissue is heterogeneously dense. There are no dominant masses, suspicious calcifications, or architectural distortion.
IMPRESSION:
BIRADS 2 - Benign findings. Recommend routine screening in 1 year.
ADDENDUM:
This exam did not utilize supplemental imaging such as spot compression views. ➡ *Not Stated*
- FINDINGS: The right breast is absent, status post mastectomy. The left breast consists of scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications are identified. Stable mammographic appearance compared to prior exams.

Targeted ultrasound was performed over the palpable area of concern at the 4 o'clock position of the left breast. No sonographic correlate was identified. The area consists of normal appearing fibroglandular tissue.

IMPRESSION:
1. Negative left breast mammogram. No mammographic evidence of malignancy. Recommend continued annual screening mammography in 12 months.
2. Negative targeted left breast ultrasound at the site of palpable abnormality in the 4 o'clock position. No sonographic findings to explain the palpable concern. Recommend clinical follow up as needed.

BIRADS 1 - Negative. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram. No complaints.

COMPARISON: None available.

FINDINGS:
The breasts are almost entirely fatty.

There are no new suspicious masses, architectural distortion or asymmetries. 

There are scattered bilateral benign appearing calcifications. In addition, in the left breast at the 12:00 position 4 cm from the nipple, there is a new group of pleomorphic microcalcifications spanning approximately 8 mm. No other suspicious calcifications are identified.

IMPRESSION:
BI-RADS Category 0: Incomplete. 
New group of suspicious microcalcifications in the left breast at 12:00, 4 cm from the nipple. Additional magnification views are recommended for further evaluation. 

The remainder of the mammogram is benign, BI-RADS 2.

ADDENDUM: Magnification views of the left breast in the area of concern demonstrate a cluster of pleomorphic microcalcifications spanning 8 mm. Stereotactic biopsy is recommended.

FINAL ASSESSMENT:
Left breast: BI-RADS 4: Suspicious abnormality. Biopsy should be considered. 
Right breast: BI-RADS 2: Benign finding. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAM, BILATERAL.
REASON FOR EXAM: Follow-up of Left inferior breast nodularity.
COMPARISON: Prior mammogram dated 1 year ago.
TECHNIQUE: Standard CC and MLO views were obtained bilaterally using Screen-Film Mammography. Additional spot compression and tangential views were also obtained of the Left inferior breast.
FINDINGS:
Overall breast composition appears heterogeneously dense, which could obscure small masses.
There are no suspicious microcalcifications, focal asymmetries, or masses identified in either breast. 
In the Left inferior breast, there is an area of nodularity corresponding to the palpable area of concern. This was evaluated with spot compression and tangential views, which demonstrate benign appearing fibroglandular tissue without any suspicious features. No definite mass, distortion, or suspicious microcalcifications are seen associated with this area.
Skin is within normal limits bilaterally. No significant axillary adenopathy.
IMPRESSION:
Heterogeneously dense breasts with nodular area of fibroglandular tissue in the Left inferior breast corresponding to area of palpable concern, favored to represent normal fibroglandular tissue. No suspicious mass, calcifications, or distortion. 
BIRADS 2: Benign findings. Recommend routine screening in 1 year. 
ADDENDUM: This case was reviewed by a second radiologist, Dr. X, who concurs with above findings and impression. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram

COMPARISON: None available

FINDINGS:
The breasts are heterogeneously dense which could obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

There are scattered areas of amorphous calcifications in both breasts, slightly more prominent in the upper outer left breast.  These are of low suspicion but should be monitored on future studies.  If these calcifications are new, a diagnostic mammogram could be considered for further evaluation but is not required.

No skin thickening or retraction.  No axillary lymphadenopathy.

IMPRESSION:
BI-RADS 2: Benign findings. Recommend continued annual screening mammography in 12 months.
Heterogeneously dense breasts which could lower the sensitivity of mammography.

ADDENDUM: The scattered amorphous calcifications identified bilaterally are of low suspicion. However, as we do not have prior mammograms for comparison, their stability cannot be determined. Short term follow up could be considered to establish stability. If follow up is desired, a diagnostic mammogram should be performed, otherwise routine annual screening is recommended. ➡ *Positive*
- MAMMOGRAM AND ULTRASOUND OF RIGHT BREAST.
HISTORY: Right breast mass found during self-exam. Last mammogram May 2023 was normal.
FINDINGS:
The breast is heterogeneously dense. There are no dominant masses, suspicious calcifications, or areas of architectural distortion.
Targeted ultrasound was performed at the site of the palpable abnormality at the 10 o'clock position 4 cm from the nipple. A 6 mm irregular solid mass with angular margins and posterior shadowing was identified, highly suspicious for malignancy.
IMPRESSION:
1. No mammographic evidence of malignancy. Heterogeneously dense breast tissue may limit sensitivity.
2. 6 mm solid mass with suspicious features on targeted ultrasound at the site of palpable abnormality, highly suggestive of malignancy. Biopsy recommended.
BIRADS 4C. Biopsy recommended. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening.

FINDINGS:
The breasts are heterogeneously dense which may obscure detection of small masses.

There are no dominant masses, areas of architectural distortion or suspicious calcifications in either breast.

In the medial left breast, there is a small group of amorphous calcifications spanning approximately 6 mm at the 3:00 position, 4 cm from the nipple. These are new compared to prior mammograms from 2 years ago. Additional magnification views confirm the presence of these calcifications.

IMPRESSION:
BI-RADS Category 0: Incomplete, requiring additional imaging evaluation. Small group of amorphous calcifications in the medial left breast, new since prior exam. These are indeterminate and require further evaluation with magnification views.

ADDENDUM:
Additional magnification views of the left breast were obtained and confirm the presence of a small group of amorphous calcifications at the site of concern. Final BI-RADS assessment Category 4: Suspicious abnormality, biopsy should be considered. ➡ *Positive*
- FINDINGS:
Bilateral digital screening mammograms demonstrate that the breasts are heterogeneously dense, which may obscure detection of small masses.
There are no suspicious masses, architectural distortions, or suspicious calcifications in the right breast. 
In the left breast at the 12:00 position 6 cm from the nipple, there is a new 8 mm cluster of fine pleomorphic calcifications. This was not present on the prior mammogram from 2 years ago. The calcifications are suspicious and require further evaluation.
Additionally, in the left breast at 3:00, 4 cm from the nipple, there is a 1 cm circumscribed oval mass, likely benign.

IMPRESSION:
BI-RADS Category 0: Incomplete, need additional imaging evaluation. Suspicious cluster of new pleomorphic calcifications in the left breast, not present on prior exam. Recommend spot compression magnification views. 
Likely benign mass left breast, recommend ultrasound for further characterization.
BI-RADS Category 2 for the right breast: Benign. ➡ *Positive*
- ULTRASOUND LEFT BREAST

HISTORY: 55-year-old female with focal palpable lump in the 2-3 o'clock position of the left breast. Prior right breast cancer treated with mastectomy.

TECHNIQUE: Targeted ultrasound was performed using a 12-5 MHz linear array transducer. 

COMPARISON: Prior mammogram from 6 months ago.

FINDINGS:
The left breast tissue is heterogeneously echogenic. Targeted ultrasound to the area of palpable concern in the 2-3 o'clock position demonstrates normal fibroglandular tissue with no focal mass, cyst, or dilated duct. No suspicious calcifications are identified.

The remainder of the left breast is unremarkable. No axillary, infraclavicular, or supraclavicular adenopathy.

IMPRESSION:
No sonographic evidence of malignancy in the left breast. The palpable lump corresponds to normal fibroglandular tissue. Recommend clinical follow-up and routine screening mammography. 

BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAM REPORT
INDICATION: Annual screening mammogram, patient reports possible lump in left breast at 3 o'clock.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Right mastectomy noted.
COMPARISON: None available.
FINDINGS:
Right breast shows post-mastectomy changes. No concerning abnormalities in the right chest wall.
Left breast shows scattered fibroglandular densities. No suspicious masses, architectural distortions or abnormal calcifications are identified. The breast is heterogeneously dense which may limit the sensitivity of mammography. 
A possible palpable abnormality is reported in the left breast at 3 o'clock position, this is included on the ultrasound requisition for further assessment.
IMPRESSION:
1. Negative left screening mammogram, no evidence of malignancy. Recommend annual screening with mammography.
2. Heterogeneously dense breast tissue which lowers the sensitivity of mammography.
3. Possible palpable abnormality left breast at 3 o'clock position, ultrasound is recommended for further assessment.
BI-RADS CATEGORY 1 - Negative. Routine screening recommended in 1 year. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening mammogram
Comparison available dating from 04/2023 for comparison.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL DIGITAL MAMMOGRAMS.
INDICATION: Screening exam. Patient is BRCA1 positive with a strong family history of breast cancer.
COMPARISON: None.  
FINDINGS: The breasts are heterogeneously dense which can obscure detection of small masses. 
No dominant mass, architectural distortion or suspicious asymmetry.
There are scattered calcifications in the upper outer left breast spanning a region of approximately 2 cm. These are pleomorphic in morphology and grouped in distribution. Several calcifications appear linear. Additional spot magnification views in the craniocaudal and mediolateral projections confirm these findings. No other groups of suspicious calcifications are seen.
No suspicious calcifications in the right breast.
IMPRESSION:
BI-RADS 4B: Suspicious calcifications left breast at 2:00, 6 cm from the nipple. Stereotactic biopsy is recommended for further evaluation. 
BI-RADS 2: Benign findings right breast. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

HISTORY: Screening mammogram. No prior mammograms available for comparison.

TECHNIQUE: Standard CC and MLO views obtained of both breasts.

FINDINGS:
The breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in the right breast.

In the left breast at the 9 o'clock position, 6 cm from the nipple, there is a new 8 mm cluster of pleomorphic microcalcifications, varying in size and shape. No associated mass or architectural distortion is seen. 

Benign-appearing calcifications are scattered throughout both breasts.

IMPRESSION:
BI-RADS Category 0: Incomplete - Need additional imaging evaluation. 
New cluster of pleomorphic microcalcifications in the left breast, suspicious for malignancy. Recommend spot compression magnification views for further evaluation. Consider stereotactic biopsy if the finding persists.

ADDENDUM:
Left diagnostic mammogram with spot compression magnification views confirms an 8 mm cluster of pleomorphic microcalcifications at 9:00, 6 cm from the nipple. No associated mass, distortion, or skin changes. 
IMPRESSION: BI-RADS Category 4B: Moderate suspicion for malignancy. Stereotactic biopsy recommended. ➡ *Positive*
- LEFT BREAST MAMMOGRAPHY AND TARGETED ULTRASOUND
INDICATION: New palpable lump left breast at 3 o'clock. Personal history of right breast cancer.
TECHNIQUE: Standard CC and MLO views of the left breast. Targeted ultrasound of the left breast 3 o'clock position using a 12 MHz linear transducer. 
COMPARISON: Prior mammogram from June 2021.
FINDINGS:
The left breast is heterogeneously dense, which may obscure detection of small masses. 
There are no dominant masses, architectural distortion, or suspicious calcifications. The palpable area of concern corresponds to an area of normal fibroglandular tissue on targeted ultrasound, with no discrete mass.
Stable benign-appearing calcifications are scattered throughout the breast. 
There is no abnormality in the left axilla.
IMPRESSION:
1. No mammographic or sonographic evidence of malignancy in the left breast. Stable benign calcifications. 
2. Palpable finding at 3 o'clock on the left corresponds to normal fibroglandular tissue, with no sonographic correlate. 
3. Recommend clinical follow up of palpable finding, as indicated. Routine screening mammography in 1 year.
BI-RADS Category 1: Negative. ➡ *Negative*
- MAMMOGRAPHY, LEFT BREAST.
INDICATION: Annual screening mammogram. Patient noted left breast tenderness laterally.
COMPARISON: 12/02/2024.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained.
FINDINGS:
The left breast is heterogeneously dense (Type C), which may obscure detection of small masses.
No dominant mass, architectural distortion, or suspicious calcifications are identified.
There are no significant changes compared to the prior mammogram.
IMPRESSION:
Negative mammogram. No evidence of malignancy.
Recommend continued annual screening mammography and clinical follow-up of symptoms.
BIRADS 1- Negative. ➡ *Negative*
- FINDINGS: Bilateral breast tissue contains scattered fibroglandular densities. There are no focal masses, suspicious calcifications, or areas of architectural distortion. No significant change from prior exam. 

Targeted ultrasound was performed in the right breast at the site of palpable abnormality at 10 o'clock, 3 cm from the nipple. No sonographic correlate was identified. 

IMPRESSION:
1. No mammographic evidence of malignancy. 
2. No sonographic abnormality in the area of palpable finding in the right breast. Clinical correlation recommended.

BI-RADS Category 1: Negative. Routine screening in 1 year. ➡ *Negative*
- FINDINGS:
The breasts are almost entirely fibroglandular.
There are no dominant masses, areas of architectural distortion or suspicious calcifications.
IMPRESSION:
Normal.
BIRADS: 1 Negative.
1 year screening recommended. ➡ *Not Stated*
- FINDINGS:

Bilateral digital mammograms with tomosynthesis and directed ultrasound of the left breast are performed.

The breasts show heterogeneously dense fibroglandular tissue. 

At the 11 o'clock position of the left breast, 6 cm from the nipple, there is an irregular spiculated mass measuring approximately 12 x 10 mm. This was identified on the directed ultrasound and corresponds to an irregular hypoechoic mass with indistinct margins and posterior shadowing. 

Additionally, in the upper outer quadrant of the left breast, there are several pleomorphic microcalcifications spanning an area of about 6 mm. These calcifications are new compared to the prior mammograms from 2 years ago. 

No suspicious masses, architectural distortion or suspicious calcifications are identified in the right breast.

The skin is unremarkable bilaterally. No axillary or internal mammary adenopathy is seen.

IMPRESSION:
1. Irregular spiculated mass at 11 o'clock position of the left breast with suspicious features, highly suggestive of malignancy. Biopsy is recommended. BI-RADS 5.

2. Pleomorphic microcalcifications in the upper outer quadrant of the left breast, suspicious for malignancy given the new onset. Stereotactic biopsy is recommended. BI-RADS 4c.

3. Right breast, BI-RADS 1, negative.

ADDENDUM: Ultrasound-guided core biopsy of the 11 o'clock mass in the left breast was performed, showing invasive ductal carcinoma, grade 2. Stereotactic biopsy of the left upper outer quadrant calcifications demonstrates ductal carcinoma in situ, high grade. ➡ *Positive*
- MAMMOGRAPHY REPORT
CLINICAL INDICATION: Annual screening mammogram. Patient reports no breast problems.
COMPARISON: Prior screening mammogram from 1 year ago.
TECHNIQUE: Standard CC and MLO views of both breasts were obtained.  
FINDINGS:  
Right breast: The breast is comprised of scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications. Stable benign calcifications are noted.
Left breast: The breast is comprised of scattered fibroglandular densities. No dominant mass, architectural distortion or suspicious calcifications. Stable benign calcifications are noted.  
IMPRESSION:
1. Negative bilateral screening mammogram. 
2. Recommend continued annual screening mammography.
BIRADS: 1 Negative. ➡ *Negative*
- MAMMOGRAM - BILATERAL SCREENING

CLINICAL HISTORY: 48 year old female presents for her screening mammogram. She has no breast complaints and no family history of breast cancer.

COMPARISON: There are no prior mammograms available for comparison.

FINDINGS: The breasts are heterogeneously dense. There are scattered benign-appearing calcifications in both breasts. There is a 0.7 cm cluster of pleomorphic calcifications in the right breast at 10 o'clock, 3 cm from the nipple. No architectural distortion or suspicious masses.

IMPRESSION:

1. Cluster of pleomorphic calcifications in the right breast, recommended for further evaluation with magnification views. 

BI-RADS: 0 (Needs additional imaging evaluation). ➡ *Positive*
- BILATERAL DIGITAL MAMMOGRAMS WITH ADDITIONAL VIEWS OF LEFT BREAST AND ULTRASOUND LEFT BREAST AND AXILLARY REGION. 
 
HISTORY: 45 year old female with a palpable lump in her left breast at 9 o'clock position. 
 
FINDINGS: The breasts are heterogeneously dense, which may obscure small masses.  
In the left breast, there is a 2.5 cm irregular mass at the 9 o'clock position, 4 cm from the nipple which corresponds to the palpable finding. The mass demonstrates spiculated margins and associated architectural distortion. Several pleomorphic microcalcifications are seen within and adjacent to the mass.
Targeted ultrasound of the left breast 9 o'clock mass shows an irregular hypoechoic mass with angulated margins and posterior acoustic shadowing, measuring 25 x 21 x 20 mm. The mass is highly suggestive of malignancy.
Ultrasound of the left axilla reveals enlarged abnormal appearing lymph nodes with thickened cortices and loss of fatty hila, the largest measuring 15 mm in cortical thickness, suspicious for metastatic involvement.  
The right breast shows no dominant mass, architectural distortion or suspicious calcifications.
 
IMPRESSION:
1. Highly suspicious mass left breast at 9 o'clock position with adjacent pleomorphic microcalcifications, likely representing malignancy. Ultrasound findings corroborate mammographic impression. Recommend tissue sampling.
2. Abnormal left axillary lymph nodes suspicious for metastatic involvement.  
3. BI-RADS Category 5: Highly Suggestive of Malignancy.
 
ADDENDUM: Ultrasound guided core biopsy of the left 9 o'clock breast mass and fine needle aspiration of the most abnormal appearing left axillary lymph node were performed. Pathology results demonstrate invasive ductal carcinoma, grade 2, in the left breast mass and metastatic carcinoma in the left axillary lymph node. ➡ *Positive*
- BILATERAL DIGITAL SCREENING MAMMOGRAM

CLINICAL HISTORY: Annual screening mammogram. No complaints.

COMPARISON: None.

FINDINGS:
Both breasts show scattered fibroglandular densities. No dominant masses, significant asymmetries, or areas of architectural distortion are seen in either breast. 

In the right breast, there are clusters of pleomorphic microcalcifications in the upper outer quadrant. These measure approximately 8 mm in total extent at 10:00, 6 cm from the nipple. No other suspicious calcifications are seen.

No significant calcifications are seen in the left breast.

The skin appears normal bilaterally. No nipple retraction or significant skin thickening. 

IMPRESSION:
1. New clusters of pleomorphic microcalcifications in the right breast at 10:00, 6 cm from the nipple, suspicious for malignancy. Additional magnification views recommended for further evaluation. BI-RADS 4B.

2. Left breast: No mammographic evidence of malignancy. BI-RADS 1.

RECOMMENDATION:
Right breast diagnostic mammogram with magnification views. Consider stereotactic biopsy if the microcalcifications remain suspicious on additional imaging. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening.
Comparison available dating from 04/2023.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BIRADS-1 SCREENING MAMMOGRAM

INDICATION: Screening.

TECHNIQUE: Conventional 2D digital mammography including craniocaudal (CC) and mediolateral oblique (MLO) views of both breasts.

COMPARISON: None available.

FINDINGS:
The breast tissue is heterogeneously dense. This may lower the sensitivity of mammography. 

There are no dominant masses, architectural distortions or suspicious calcifications.

IMPRESSION:
1. Negative mammogram.
2. Heterogeneously dense breast tissue.

RECOMMENDATION: 
Routine screening mammography is recommended in 1 year.

BI-RADS Category 1: Negative. ➡ *Not Stated*
- BREAST IMAGING REPORT

CLINICAL INFORMATION: Annual screening mammogram. No breast concerns. Patient has a history of right breast cancer, status post lumpectomy in 2010.

COMPARISON: Screening mammogram from 06/15/2022 at Saint Mary's Hospital.

FINDINGS:
Right breast: Stable post-surgical changes in the upper outer quadrant consistent with prior lumpectomy. No suspicious mass, architectural distortion, or suspicious calcifications. 

Left breast: Scattered fibroglandular densities. No dominant mass, architectural distortion, or suspicious calcifications. Stable benign-appearing calcifications in the upper outer quadrant. No significant change compared to prior mammogram.

IMPRESSION: 
1. Negative bilateral screening mammogram. 
2. Stable post-surgical changes right breast.
3. No mammographic evidence of malignancy.

ASSESSMENT: BI-RADS Category 1 - Negative. Recommend continued annual screening mammography. ➡ *Negative*
- MAMMOGRAM LEFT BREAST WITH SPOT MAGNIFICATION.
HISTORY: Left breast pain.
COMPARISONS: Prior from 08/11/2023.
TECHNIQUE: Standard CC and MLO views of the left breast were obtained. Spot magnification views of the left breast were obtained.
FINDINGS:
The breasts are heterogeneously dense. This may lower the sensitivity of mammography.
There are scattered fibroglandular densities. No definite mass, architectural distortion, or suspicious calcifications are seen. No skin thickening or nipple retraction is seen.
IMPRESSION:
Negative left mammogram. BI-RADS 2. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAPHY
History: Screening.
Comparison available dating from 10/2022.
Findings:
There are scattered fibroglandular densities bilaterally. No skin thickening or nipple retraction is seen. No grouped calcifications are identified. No spiculated or circumscribed masses are seen.
IMPRESSION:
No mammographic evidence of malignancy. BI-RADS Category 1. ➡ *Negative*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Routine screening

TECHNIQUE: Standard CC and MLO views.

COMPARISON: Screening mammogram from 1 year ago.

FINDINGS:
The breasts are heterogenously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion or suspicious calcifications in either breast. 

At the 1:00 position of the left breast, 5 cm from the nipple, there are several new grouped amorphous calcifications spanning 6 mm. These were not present on the previous exam.

The visualized axillary regions are unremarkable.

IMPRESSION:

BI-RADS Category 0: Incomplete - Need Additional Imaging Evaluation
New grouped amorphous calcifications in the left breast at 1:00, 5 cm from the nipple. Additional magnification views recommended for further characterization.  

ADDENDUM:
Magnification views of the left breast upper outer quadrant demonstrate a 6 mm cluster of pleomorphic microcalcifications at 1:00, 5 cm from the nipple. No associated mass. 

REVISED IMPRESSION:
BI-RADS Category 4B: High suspicion for malignancy.
6 mm cluster of new pleomorphic microcalcifications in the left breast. Stereotactic biopsy recommended. ➡ *Positive*
- BILATERAL SCREENING MAMMOGRAM. CLINICAL HISTORY: 50 year old female presents for her annual screening mammogram. Comparison is made to prior mammogram from 10/19/2023. FINDINGS: The breasts are heterogeneously dense. There are scattered benign appearing calcifications bilaterally. There is no evidence of a suspicious mass, architectural distortion, or suspicious calcification. IMPRESSION: Negative mammogram. BI-RADS 1. ➡ *Negative*
- MAMMOGRAPHIC REPORT:
CLINICAL INDICATION: Annual screening mammogram. Patient complains of intermittent left breast tenderness, but no palpable mass.
COMPARISON: Prior mammograms from 4/15/2022.  
FINDINGS: 
The right breast demonstrates scattered fibroglandular densities. No dominant mass, suspicious calcifications or architectural distortion.
The left breast contains heterogeneously dense parenchyma. No dominant mass, suspicious calcifications or architectural distortion. There are no significant changes from prior exam.  
IMPRESSION:
1. No mammographic evidence of malignancy in either breast. Specifically no suspicious microcalcifications. 
2. Heterogeneously dense breasts which may obscure detection of small masses. 
RECOMMENDATIONS:
Recommend annual screening mammography. Advise patient to report any new breast symptoms or changes promptly.
ASSESSMENT CATEGORY: BI-RADS 1 - Negative. ➡ *Negative*
- BILATERAL DIAGNOSTIC MAMMOGRAM WITH ULTRASONOGRAPHY.
Compared to Previous: Yes. Mammograms of 2013-10-01 were available for comparison.
Breast Density >= 75%: Yes.
Breast Density < 75%: No.
Bilateral scattered fibroglandular densities.
Benign calcifications, both breasts. Non-malignant calcifications.
No spiculated mass, architectural distortion, or suspicious microcalcifications.
Bilateral ultrasonography reveals no suspicious focal lesion.
BIRADS-2. 
Normal interval followup is recommended. ➡ *Not Stated*
- BILATERAL SCREENING MAMMOGRAM

INDICATION: Annual screening mammogram.

COMPARISON: Compared with prior studies from 10/15/2020.

FINDINGS:
Both breasts are heterogeneously dense, which may obscure detection of small masses.

There are no dominant masses, architectural distortion, or suspicious calcifications in either breast.

There are scattered fibroglandular densities bilaterally.

In the right breast at the 10:00 position, 6 cm from the nipple, there is a new group of amorphous calcifications spanning an area of about 8 mm. This is a suspicious finding and requires further evaluation with magnification views.

The left breast is unremarkable.

IMPRESSION:
BI-RADS 0: Incomplete. Magnification views of the right breast are recommended for further evaluation of the new group of amorphous calcifications at the 10:00 position, 6 cm from the nipple.

ADDENDUM - ADDITIONAL MAGNIFICATION VIEWS RIGHT BREAST: 
Magnification views of the right breast in the craniocaudal and mediolateral oblique projections were obtained.

The group of amorphous calcifications at the 10:00 position, 6 cm from the nipple, is again identified. The calcifications are indistinct and span an area of about 8 mm. This is a suspicious finding. Stereotactic biopsy is recommended.

FINAL ASSESSMENT: 
Right Breast: BI-RADS 4: Suspicious abnormality. Biopsy should be considered.
Left Breast: BI-RADS 1: Negative. ➡ *Positive*
- DIAGNOSTIC MAMMOGRAM, BILATERAL:

INDICATION: 56 year-old woman with newly diagnosed left breast cancer.

TECHNIQUE: Standard CC and MLO views were obtained bilaterally. Magnification images with a small focal compression paddle were then performed in mediolateral and craniocaudal projections of an area of microcalcifications in the upper outer right breast.

COMPARISON: None.

FINDINGS: 
The breasts are heterogeneously dense (type C). 

The left breast shows an irregular spiculated hyperdense mass measuring 2.3 cm located at 2 o'clock position, 4 cm from the nipple. Associated architectural distortion is also seen. The lesion was targeted under ultrasound and appears highly suspicious for malignancy (BI-RADS 5). No other suspicious masses, areas of asymmetry or distortion are seen in the left breast. The left skin is normal in thickness and there is no nipple retraction.

In the right breast, there is a cluster of pleomorphic microcalcifications in the right upper outer quadrant measuring 6 mm at 9 o'clock, 6 cm from the nipple (group 4). These are seen on both CC and MLO views. Additional magnification views confirm these calcifications are suspicious for malignancy (BI-RADS 4B). The remainder of the right breast shows only scattered areas of benign calcifications. No masses, architectural distortion, or skin thickening are seen on the right.

There are no abnormally enlarged axillary lymph nodes bilaterally.

IMPRESSION:
1. Known left breast cancer (BI-RADS 6).
2. Newly identified group of suspicious microcalcifications in the right upper outer breast, BI-RADS 4B. Stereotactic core needle biopsy is recommended for diagnosis. 

ADDENDUM: Stereotactic biopsy of the microcalcifications in the right breast was performed using a 9-gauge vacuum-assisted device. Specimen radiography confirmed calcifications within the samples. Pathology results are pending and will be reported separately. ➡ *Positive*
</details>

In [21]:
def synthesize(pr): return cli.structured(pr, temp=1, tools=TranslationCritique)[0]

In [22]:
eval_prompt_template = """\
    Below is a report with the following label {label}. The label indicates whether the report mentions:
    - a positive finding for calcifications (including microcalcifications)
    - a negative finding for calcifications (explicitly stating no calcifications were found)
    - or does not discuss calcifications at all (truly not mentioning calcifications in any way)

    Important: Any explicit mention of calcifications, even to state their absence, should be considered a negative finding, not a "Not Stated" case.

    Evaluate its quality as a senior radiologist would, considering its suitability for professional use. Use the additive 5-point scoring system described below. Points are accumulated based on the satisfaction of each criterion:

    - Award 1 point if the report's content is correctly aligned with the given label regarding calcifications (including microcalcifications). Remember, explicitly stating no calcifications were found is a negative finding, not a "Not Stated" case.
    - Add another point if the report accurately conveys basic radiological findings related to calcifications (or their absence), distinguishing between microcalcifications and larger calcifications when relevant, but may have minor errors.
    - Award a third point if the report uses correct terminology for both calcifications and microcalcifications, is appropriate for professional use, and demonstrates good understanding of radiological principles.
    - Grant a fourth point if the report is highly accurate, reads naturally, and effectively handles complex concepts related to calcifications and microcalcifications with clear descriptions of their size, distribution, and potential significance.
    - Bestow a fifth point if the report is outstanding, demonstrating mastery of clinical language and radiological expertise, capturing subtle nuances of calcification appearance and distribution, maintaining a professional tone, and providing appropriate recommendations for further evaluation if necessary.

    <report>
    {report}
    </report>

    After examining the report:

    - Briefly justify your total score in a single line.
    - Conclude with the score of the report."""

In [23]:
def get_critique(r):
    critique = synthesize(eval_prompt_template.format(report=r.report, label=r.label))
    return (r.report, r.label, critique.critique, critique.score)

In [24]:
# results = [get_critique(r) for r in example_reports]

In [25]:
import time
import random
import logging
from tqdm import tqdm


In [26]:

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [27]:
def save_results(results):
    # Implement your saving logic here, e.g., to a file or database
    with open('results.txt', 'w') as f:
        for result in results:
            f.write(f"{result}\n")

In [28]:
def get_critique_with_retry(r, max_retries=5, initial_wait=30, max_wait=300):
    for attempt in range(max_retries):
        try:
            critique = synthesize(eval_prompt_template.format(report=r.report, label=r.label))
            return (r.report, r.label, critique.critique, critique.score)
        except Exception as e:
            if "529" in str(e) or "overloaded" in str(e).lower():
                wait_time = min(initial_wait * (2 ** attempt) + random.uniform(0, 10), max_wait)
                logging.warning(f"Server overloaded. Waiting for {wait_time:.2f} seconds before retry.")
                time.sleep(wait_time)
            else:
                logging.error(f"Unexpected error: {e}")
                raise
    
    logging.error(f"Max retries reached for report: {r.report[:50]}...")
    return None

In [29]:
def process_reports(reports, batch_size=10, cooldown=5):
    results = []
    for i in tqdm(range(0, len(reports), batch_size)):
        batch = reports[i:i+batch_size]
        for report in batch:
            result = get_critique_with_retry(report)
            if result:
                results.append(result)
        
        # Log progress
        logging.info(f"Processed {len(results)} out of {len(reports)} reports")
        
        # Cooldown between batches
        time.sleep(cooldown)
    
    return results

In [ ]:

# Main execution
results = process_reports(example_reports)

  0%|          | 0/126 [00:00<?, ?it/s]2024-10-19 00:26:47,359 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-10-19 00:26:51,330 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-10-19 00:26:54,910 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-10-19 00:26:57,982 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-10-19 00:27:00,132 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-10-19 00:27:05,305 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-10-19 00:27:07,763 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-10-19 00:27:10,937 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-10-19 00:27:13,395 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1

In [ ]:
results

In [ ]:
results[0:3]

In [33]:
df_critiqued = pd.DataFrame(results, columns=['report_text', 'label', 'critique', 'score'])

In [ ]:
df_critiqued.score.value_counts()

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df_critiqued

In [ ]:
df_critiqued.score.value_counts()

## Let's create dfs for poor scores and great scores

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_poor = df_critiqued[df_critiqued['score'] <= 3].reset_index(drop=True)
df_poor.head()

In [38]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

# df_poor = df_critiqued[df_critiqued['score'] <= 3].reset_index(drop=True)
# df_poor.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_great = df_critiqued[df_critiqued['score'] >= 4].reset_index(drop=True)
df_great.head()

In [ ]:
df_great.label.value_counts() # n = 478

In [33]:
df_great.to_csv('microcalcs-n478-claude-3-opus-20240229-critiqued-4_5_only.csv')

## What happens if I only train on 5s?

In [ ]:
# df_45s = pd.read_csv(FP/'microcalcs-n478-claude-3-opus-20240229-critiqued-4_5_only.csv') ; df_45s.head()


In [ ]:
# df_45s.label.value_counts()

In [ ]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

# df_45s = df_45s[df_45s['score'] == 5].reset_index(drop=True)
# df_45s.head()

In [ ]:
# df_45s.label.value_counts()

In [ ]:
# df_45s.label.value_counts().sum()

In [10]:
# df_45s.to_csv('microcalcs-n270-claude-3-opus-20240229-critiqued-5s_only.csv')